<a href="https://colab.research.google.com/github/thanhnhan311201/CS114.L11.KHCL/blob/master/Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/thanhnhan311201/CS114.L11.KHCL.git

fatal: destination path 'CS114.L11.KHCL' already exists and is not an empty directory.


In [3]:
!pip install stop-words

In [2]:
from nltk.corpus import stopwords
import nltk
import json
import numpy as np
import pandas as pd
from collections import deque 


In [8]:
#Load dataset
file_ = pd.read_json("/content/CS114.L11.KHCL/LogisticsRegression/Sarcasm_Project/Sarcasm_Headlines_Dataset.json",lines=True)
file_.head()
del file_['article_link']
# Training data
data = file_.to_numpy()[:20000,:]
# Testing data
data2 = file_.to_numpy()[20000:,:]


In [6]:
# Lọc lại văn bản, chỉ giữ lại chữ và số
text = ''
for i in range(len(data)):
  text += data[i,0]
text = deque(text)
while i < len(text):
  if not((text[i] >='a' and text[i] <='z') or (text[i] >='0' and text[i] <='9') or (text[i] == ' ')):
    text.remove(text[i])
    i-=1
  i+=1
text = ''.join(text)
print(text)

former versace store clerk sues over secret black code for minority shoppersthe roseanne revival catches up to our thorny political mood for better and worsemom starting to fear sons web series closest thing she will have to grandchildboehner just wants wife to listen not come up with alternative debtreduction ideasjk rowling wishes snape happy birthday in the most magical wayadvancing the worlds womenthe fascinating case for eating labgrown meatthis ceo will send your kids to school if you work for his companytop snake handler leaves sinking huckabee campaignfridays morning email inside trumps presser for the agesairline passengers tackle man who rushes cockpit in bomb threatfacebook reportedly working on healthcare features and appsnorth korea praises trump and urges us voters to reject dull hillaryactually cnns jeffrey lord has been indefensible for a whilebarcelona holds huge protest in support of refugeesnuclear bomb detonates during rehearsal for spiderman musicalcosby lawyer ask

In [7]:
# thống kê số lượng từ xuất hiện trong toàn bộ dataset
text = deque(text.split())
dic = {}
for i in range(len(text)):
  dic[text[i]] = text.count(text[i])
print(dic)

{'former': 43, 'versace': 1, 'store': 23, 'clerk': 7, 'sues': 12, 'over': 343, 'secret': 51, 'black': 155, 'code': 9, 'for': 2439, 'minority': 8, 'shoppersthe': 1, 'roseanne': 2, 'revival': 7, 'catches': 9, 'up': 517, 'to': 6139, 'our': 124, 'thorny': 1, 'political': 60, 'mood': 7, 'better': 83, 'and': 1419, 'worsemom': 1, 'starting': 29, 'fear': 26, 'sons': 27, 'web': 11, 'series': 31, 'closest': 4, 'thing': 86, 'she': 151, 'will': 396, 'have': 368, 'grandchildboehner': 1, 'just': 419, 'wants': 105, 'wife': 45, 'listen': 12, 'not': 525, 'come': 77, 'with': 1302, 'alternative': 9, 'debtreduction': 1, 'ideasjk': 1, 'rowling': 4, 'wishes': 26, 'snape': 2, 'happy': 29, 'birthday': 35, 'in': 3051, 'the': 3269, 'most': 183, 'magical': 18, 'wayadvancing': 1, 'worlds': 53, 'womenthe': 3, 'fascinating': 3, 'case': 47, 'eating': 41, 'labgrown': 1, 'meatthis': 1, 'ceo': 39, 'send': 26, 'your': 448, 'kids': 97, 'school': 127, 'if': 189, 'you': 632, 'work': 100, 'his': 449, 'companytop': 1, 'snake

In [5]:
# download bộ những từ không có ý nghĩa nhiều trong tiếng anh
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
dic = {'former': 43, 'versace': 1, 'store': 23, 'clerk': 7, 'sues': 12, 'over': 343, 'secret': 51, 'black': 155, 'code': 9, 'for': 2439, 'minority': 8, 'shoppersthe': 1, 'roseanne': 2, 'revival': 7, 'catches': 9, 'up': 517, 'to': 6139, 'our': 124, 'thorny': 1, 'political': 60, 'mood': 7, 'better': 83, 'and': 1419, 'worsemom': 1, 'starting': 29, 'fear': 26, 'sons': 27, 'web': 11, 'series': 31, 'closest': 4, 'thing': 86, 'she': 151, 'will': 396, 'have': 368, 'grandchildboehner': 1, 'just': 419, 'wants': 105, 'wife': 45, 'listen': 12, 'not': 525, 'come': 77, 'with': 1302, 'alternative': 9, 'debtreduction': 1, 'ideasjk': 1, 'rowling': 4, 'wishes': 26, 'snape': 2, 'happy': 29, 'birthday': 35, 'in': 3051, 'the': 3269, 'most': 183, 'magical': 18, 'wayadvancing': 1, 'worlds': 53, 'womenthe': 3, 'fascinating': 3, 'case': 47, 'eating': 41, 'labgrown': 1, 'meatthis': 1, 'ceo': 39, 'send': 26, 'your': 448, 'kids': 97, 'school': 127, 'if': 189, 'you': 632, 'work': 100, 'his': 449, 'companytop': 1, 'snake': 5, 'handler': 3, 'leaves': 56, 'sinking': 2, 'huckabee': 9, 'campaignfridays': 1, 'morning': 67, 'email': 74, 'inside': 49, 'trumps': 219, 'presser': 1, 'agesairline': 1, 'passengers': 10, 'tackle': 6, 'man': 515, 'who': 403, 'rushes': 4, 'cockpit': 2, 'bomb': 10, 'threatfacebook': 1, 'reportedly': 36, 'working': 49, 'on': 1727, 'healthcare': 14, 'features': 16, 'appsnorth': 1, 'korea': 45, 'praises': 10, 'trump': 618, 'urges': 31, 'us': 269, 'voters': 56, 'reject': 6, 'dull': 3, 'hillaryactually': 1, 'cnns': 1, 'jeffrey': 2, 'lord': 7, 'has': 415, 'been': 94, 'indefensible': 2, 'a': 2077, 'whilebarcelona': 1, 'holds': 21, 'huge': 33, 'protest': 34, 'support': 63, 'of': 4229, 'refugeesnuclear': 1, 'detonates': 2, 'during': 161, 'rehearsal': 2, 'spiderman': 2, 'musicalcosby': 1, 'lawyer': 20, 'asks': 71, 'why': 148, 'accusers': 8, 'didnt': 72, 'forward': 21, 'be': 610, 'smeared': 1, 'by': 656, 'legal': 25, 'team': 55, 'years': 157, 'agostock': 1, 'analysts': 3, 'confused': 7, 'frightened': 2, 'boar': 4, 'marketbloombergs': 1, 'program': 22, 'build': 17, 'cities': 19, 'got': 108, 'biggercraig': 1, 'hicks': 6, 'indictedcourtroom': 1, 'sketch': 5, 'artist': 27, 'clear': 16, 'manga': 2, 'influencestrump': 1, 'assures': 24, 'nation': 65, 'that': 569, 'decision': 25, 'syrian': 31, 'airstrikes': 5, 'came': 23, 'after': 607, 'carefully': 8, 'considering': 12, 'all': 364, 'passing': 18, 'whimsqatar': 1, 'deporting': 2, 'dutch': 2, 'woman': 185, 'reported': 9, 'was': 231, 'drugged': 1, 'rapedthis': 1, 'is': 1190, 'shouldnt': 26, 'go': 107, 'circusted': 1, 'cruz': 42, 'hits': 34, 'panic': 7, 'button': 4, 'we': 206, 'could': 209, 'lose': 32, 'both': 14, 'houses': 9, 'congresswhy': 1, 'writers': 8, 'must': 58, 'plan': 95, 'surprisedobama': 1, 'visits': 16, 'arlington': 1, 'national': 80, 'cemetery': 2, 'honor': 21, 'veteransexcon': 1, 'back': 199, 'behind': 61, 'barafter': 1, 'careful': 3, 'consideration': 3, 'bush': 54, 'recommends': 25, 'oil': 30, 'drillingremembrance': 1, 'beginning': 17, 'taskallies': 1, 'islamist': 4, 'motive': 2, 'killing': 34, 'nemtsov': 1, 'nonsensegillian': 1, 'jacobs': 1, 'what': 315, 'its': 276, 'like': 272, 'kiss': 5, 'adam': 9, 'brodyuber': 1, 'vows': 38, 'repay': 3, 'nyc': 13, 'drivers': 10, 'tens': 2, 'millions': 29, 'tax': 73, 'snafuapple': 1, 'may': 152, 'poached': 1, 'electric': 11, 'motorcycle': 7, 'company': 49, 'deathdrugresistant': 1, 'bacteria': 6, 'often': 5, 'lurk': 1, 'childrens': 18, 'dogs': 15, 'sandboxesif': 1, 'see': 98, 'muslim': 37, 'at': 854, 'airportgiant': 1, 'altoid': 1, 'heading': 6, 'toward': 21, 'earthmoana': 1, 'sails': 1, 'straight': 38, 'top': 59, 'box': 38, 'office': 69, 'massive': 33, '811': 1, 'million': 92, 'openingselig': 1, 'counted': 1, 'money': 59, 'while': 116, 'baseball': 12, 'lost': 45, 'next': 102, 'generation': 10, 'fansrobin': 1, 'williams': 26, 'inflicted': 1, 'holiday': 40, 'moviegoers': 1, 'eighth': 5, 'yeardevin': 1, 'nunes': 3, 'never': 121, 'reveal': 29, 'source': 9, 'surveillance': 6, 'claimsscott': 1, 'used': 75, 'stop': 101, 'breathing': 6, 'nearly': 26, '40': 18, 'times': 48, 'an': 235, 'hour': 17, 'this': 405, 'device': 7, 'changed': 15, 'liferescuers': 1, 'heroically': 4, 'help': 84, 'beached': 2, 'garbage': 6, 'into': 374, 'oceanmedics': 1, 'drop': 15, 'soccer': 16, 'player': 21, 'from': 878, 'stretcher': 1, 'hes': 121, 'tickedgive': 1, 'gift': 35, 'play': 38, 'seasonchristian': 1, 'bale': 3, 'sikh': 6, 'temple': 2, 'victimsspicer': 1, 'denies': 13, 'ending': 10, 'maternity': 5, 'care': 84, 'guarantee': 1, 'would': 166, 'mean': 24, 'women': 164, 'pay': 46, 'more': 351, 'health': 124, 'careright': 1, 'live': 84, 'life': 159, 'complete': 18, 'stunned': 6, 'horror': 14, 'added': 9, 'constitutionnasa': 1, 'now': 203, 'almost': 28, 'positive': 14, 'mars': 12, 'rockymonster': 1, 'undeterred': 1, 'nightlightdiy': 1, 'sports': 27, 'equipment': 2, 'closet1': 1, 'dead': 116, '3': 75, 'injured': 17, 'shooting': 59, 'ti': 1, 'concert': 9, 'nyclongtime': 1, 'teacher': 42, 'retires': 7, 'without': 91, 'changing': 15, 'single': 59, 'students': 80, 'lifedonald': 2, 'wins': 45, 'northeast': 1, 'polls': 18, 'saydonald': 1, 'wouldnt': 15, 'had': 87, 'ready': 61, 'cash': 13, 'selffinance': 1, 'entire': 76, 'campaign': 102, 'analysisnew': 1, 'star': 98, 'wars': 33, 'film': 58, 'once': 47, 'again': 38, 'disappoints': 2, 'diehard': 1, 'nien': 1, 'nunb': 1, 'fansbats': 1, 'shooed': 1, 'out': 545, 'nations': 54, 'waterslide': 1, 'tunnels': 2, 'preparation': 3, 'summermobile': 1, 'news': 98, 'crew': 15, 'reports': 45, 'own': 119, 'van': 8, 'breaking': 29, 'downun': 1, 'rights': 57, 'chief': 40, 'calls': 127, 'humanitarian': 4, 'situation': 7, 'syria': 21, 'outragehow': 1, 'track': 18, 'santa': 15, 'claus': 3, 'flight': 17, 'around': 103, 'world': 159, 'christmas': 49, 'eveobama': 1, 'colorado': 8, 'appraisedtrouble': 1, 'tvs': 4, 'africabrita': 1, 'unveils': 61, 'new': 810, 'inthroat': 1, 'water': 48, 'filtersreport': 1, 'john': 61, 'grisham': 2, 'slowly': 21, 'but': 176, 'surely': 2, 'climbing': 2, 'list': 37, 'greatest': 24, 'living': 49, 'american': 127, 'authorsghost': 1, 'cant': 164, 'make': 230, 'simple': 27, 'cup': 16, 'coffee': 25, 'everyone': 55, 'freaking': 10, 'outtupperware': 1, 'truly': 12, 'recover': 4, 'red': 43, 'curry': 9, 'leftoversone': 1, 'planets': 3, 'powerful': 36, 'forces': 25, 'change': 90, 'adolescent': 2, 'girlhow': 1, 'does': 59, 'draymond': 1, 'green': 26, 'take': 142, 'game': 74, 'level': 11, 'tuning': 3, 'wnbaarea': 1, 'said': 27, 'sorry': 14, '118': 1, 'yesterdayryan': 1, 'lochte': 4, 'apologizes': 30, 'behavior': 12, 'rionorth': 1, 'dakota': 10, 'heard': 18, '48': 1, 'hours4': 1, 'lessons': 32, 'prison': 20, 'taught': 13, 'me': 74, 'about': 754, 'power': 65, 'controlsick': 1, 'fucks': 4, 'line': 57, 'gape': 2, 'bodywhat': 1, 'americas': 38, 'first': 268, 'fraternity': 2, 'really': 166, 'likethe': 1, 'vicious': 2, 'knot': 2, 'untangling': 1, 'process': 10, 'contains': 10, 'solutions': 5, 'timethis': 1, 'congressman': 38, 'thinks': 75, 'can': 303, 'fix': 22, 'economy': 14, 'drinking': 20, 'beerbreast': 1, 'implants': 3, 'found': 78, 'cause': 21, 'problems': 19, 'laboratory': 1, 'micetrump': 1, 'orders': 28, 'strikes': 17, 'retaliation': 3, 'chemical': 10, 'attackceos': 1, 'funeral': 8, 'networking': 3, 'dreamyearround': 1, 'schooling': 1, 'how': 294, 'it': 446, 'students90': 1, 'zen': 3, 'teachers': 25, 'pledge': 6, 'culture': 17, 'fosters': 1, 'abusenypd': 1, 'weighs': 9, 'allowing': 8, 'chokeholds': 1, 'following': 41, 'eric': 12, 'garner': 4, 'deathsonny': 1, 'bono': 3, 'foundation': 14, 'prevents': 4, 'atrisk': 3, 'youths': 5, 'skiing': 2, 'treespope': 1, 'francis': 39, 'reminds': 30, 'caring': 6, 'earth': 24, 'everyones': 3, 'responsibility6yearold': 1, 'cries': 3, 'when': 184, 'told': 44, 'mtm': 1, 'productions': 3, 'kitten': 4, 'nowexboyfriend': 1, 'thought': 37, 'hed': 8, 'check': 22, 'throw': 20, 'day': 193, 'offdoctors': 1, 'restore': 7, 'ken': 6, 'burns': 10, 'fullcolor': 1, 'vision': 12, 'removing': 5, 'tumor': 2, 'filmmakers': 1, 'visual': 6, 'cortexgravity': 1, '12': 29, 'slave': 5, 'tie': 5, '2014': 21, 'pga': 1, 'awardsfor': 1, 'coming': 62, '150': 6, 'lgbtq': 22, 'people': 227, 'last': 136, 'yearjohn': 1, 'oliver': 10, 'lays': 6, 'disturbing': 7, 'ways': 95, 'which': 28, 'impacts': 1, 'americahouse': 1, 'gop': 117, 'crackdown': 7, 'continuesi': 1, 'sure': 64, 'skills': 13, 'please': 13, 'my': 190, 'grandson': 6, 'says': 282, 'queen': 16, 'elizabeth': 13, 'disrobing': 1, 'before': 180, 'meghan': 6, 'markleman': 1, 'visiting': 13, 'hometown': 7, 'amazed': 2, 'find': 74, 'childhood': 15, 'insecurities': 2, 'still': 233, 'therelook': 1, 'attacked': 5, 'giant': 20, 'bugclinton': 1, 'becomes': 26, 'president': 117, '18': 16, 'feet': 13, 'pole': 4, 'vaulthunter': 1, 's': 2, 'thompson': 6, 'shoots': 17, 'mouth': 12, 'off': 231, 'one': 337, 'timeinhofes': 1, 'grand': 9, 'climate': 76, 'conspiracy': 13, 'theory': 18, 'barbra': 1, 'streisand5': 1, 'questions': 42, 'i': 194, 'wish': 16, 'younger': 10, 'asking': 34, 'mealbuquerque': 1, 'shooter': 11, 'loose': 8, 'gunman': 8, '1': 53, 'injuredwhat': 1, 'everything': 36, 'flooddemi': 1, 'lovato': 5, 'drops': 16, 'emotional': 31, 'nightingale': 1, 'music': 49, 'vidmiley': 1, 'cyrus': 12, 'liam': 2, 'hemsworth': 8, 'smooch': 2, 'nye': 3, 'noticesjealous': 1, 'gps': 16, 'clearly': 28, 'wifeparks': 1, 'rec': 2, 'natalie': 2, 'morales': 1, 'comes': 59, 'as': 548, 'queerthe': 1, 'italy': 4, 'tours': 1, '2015kaia': 1, 'gerber': 1, 'cindy': 3, 'crawfords': 2, 'daughter': 39, 'lands': 11, 'major': 37, 'fashion': 38, 'campaignla': 1, 'grants': 7, 'clippers': 2, 'netsjames': 1, 'corden': 15, 'roasts': 3, 'david': 22, 'beckham': 3, 'matching': 2, 'outfits': 5, 'poshcaf': 1, 'adds': 37, 'heartbreaking': 5, 'little': 127, 'lunch': 14, 'menumcdonalds': 1, 'packaging': 1, '100': 23, 'percent': 38, '2025kris': 1, 'jenner': 25, 'turned': 14, 'way': 171, 'drunken': 4, 'valentines': 13, 'karaokewhale': 1, 'regrets': 6, '290000': 1, 'plastic': 14, 'poker': 3, 'chips': 4, 'fell': 6, 'container': 4, 'shiptexas': 1, 'ebola': 23, 'patient': 11, 'fighting': 31, 'lifeis': 1, 'too': 125, 'late': 32, 'audition': 2, 'perfect': 60, 'actor': 22, 'role': 23, 'poking': 1, 'head': 39, 'room': 41, 'producers': 9, 'were': 84, 'giving': 43, 'hoperobert': 1, 'de': 11, 'niro': 2, 'turn': 28, '58': 3, 'movie': 53, 'rolescott': 1, 'pruitt': 9, 'sort': 14, 'answers': 5, 'whether': 34, 'believes': 14, 'changeaccused': 1, 'father': 30, 'son': 57, 'urinal': 3, 'thieves': 2, 'flushed': 1, 'copsjust': 1, 'slow': 7, 'youll': 25, 'fine': 11, 'drunk': 25, 'driver': 22, 'self': 56, 'speeding': 2, 'away': 68, 'stolen': 9, 'police': 112, 'car3': 1, 'libertarians': 1, 'fuel': 6, '7': 40, 'super': 59, 'pac': 6, 'philadelphias': 2, 'mayoral': 2, 'democratic': 34, 'primaryair': 1, 'force': 35, 'no': 250, 'longer': 32, 'require': 7, 'so': 165, 'god': 31, 'enlistment': 1, 'oathspaul': 1, 'newman': 3, 'dies': 43, 'consuming': 5, '51': 1, 'hardboiled': 1, 'eggsyak': 1, 'chews': 3, 'thoughtfullyman': 1, 'worried': 36, 'drug': 45, 'dealer': 3, 'whos': 15, 'picking': 4, 'phonedad': 1, 'hotel': 23, '10': 75, 'miles': 13, 'city': 68, 'youre': 54, 'visitingexplosion': 1, 'fells': 1, 'building': 22, 'outside': 37, 'paris': 23, 'least': 40, '2the': 1, 'donald': 220, 'military': 41, 'expertise': 3, 'finally': 97, 'revealedunion': 1, 'claims': 60, 'sanders': 70, 'staffers': 11, 'posed': 2, 'members': 30, 'influence': 6, 'workersseattle': 1, 'mayors': 3, 'accuser': 3, 'sexabuse': 1, 'lawsuit': 14, 'forwardsouthern': 1, 'comfort': 7, 'comforts': 5, 'southernerswedish': 1, 'featured': 6, 'shown': 4, 'fox': 20, 'say': 113, 'they': 197, 'selectively': 1, 'editedchromat': 1, '2': 84, 'plussize': 4, 'models': 6, 'runwaycouncilman': 1, 'baltimore': 5, 'rappers': 1, 'inspire': 7, 'students10': 1, 'qualities': 2, 'inner': 9, 'spiritchubby': 1, 'jewish': 15, 'boy': 43, 'dreams': 27, 'being': 160, 'apatow': 3, 'musethe': 1, 'only': 163, 'parenting': 18, 'advice': 24, 'id': 11, 'dare': 2, 'givefamily': 1, 'chooses': 6, 'different': 42, 'dog': 64, 'than': 234, 'reincarnated': 1, 'grandfathercelebrities': 1, 'celebrate': 30, 'fourth': 13, 'july': 10, 'some': 124, 'fun': 39, 'suninternational': 1, 'womens': 45, 'western': 8, 'save': 58, 'worldmichelle': 1, 'phan': 1, 'youtubes': 1, 'beauty': 19, 'bestie': 1, 'empowers': 2, 'incaitlyn': 1, 'responds': 34, 'ricky': 1, 'gervais': 1, 'golden': 25, 'globes': 5, 'jokesjoy': 1, 'behar': 2, 'michael': 35, 'flynn': 6, 'guilty': 25, 'plea': 8, 'pure': 8, 'joy6': 1, 'things': 146, 'tells': 55, 'their': 203, 'weight': 27, 'loss': 19, 'journeyfan': 1, 'disappointed': 14, 'learn': 22, 'l': 2, 'ron': 4, 'hubbard': 1, 'scientologistmarriage': 1, 'breaks': 35, 'procreative': 1, 'differencesneyo': 1, 'helps': 20, 'raise': 20, '23': 10, 'california': 35, 'engineering': 5, 'schoolelizabeth': 1, 'warren': 15, 'racist': 24, 'bullysix': 1, 'hurt': 19, 'commuter': 1, 'bus': 28, 'crashfox': 1, 'problem': 32, 'solvers': 2, 'headskim': 1, 'gordon': 2, 'clarifies': 6, 'her': 262, 'comments': 16, 'lana': 2, 'del': 6, 'rey': 2, 'feminismbashar': 1, 'alassad': 3, 'tries': 25, 'tiny': 14, 'bit': 13, 'sarin': 1, 'gas': 21, 'likemy': 1, 'niece': 2, 'cancer': 39, 'im': 48, 'ticked': 1, 'itsupermodel': 1, 'stephanie': 4, 'seymour': 2, 'arrested': 29, 'charged': 21, 'drivingscoliosis': 1, 'need': 141, 'knowcontroversy': 1, 'erupts': 6, 'uk': 12, 'retailer': 1, 'removes': 13, 'gender': 17, 'labels': 2, 'clothesteacher': 1, 'bitches': 1, 'paycheck': 4, 'sixthgrade': 2, 'classarea': 1, 'insurance': 16, 'salesman': 7, 'celebrates': 48, '14th': 2, 'year': 81, 'quoting': 1, 'fletchfriends': 1, 'always': 63, 'best': 172, 'neil': 7, 'labuteman': 1, 'watch': 66, 'another': 87, 'scifi': 5, 'physicist': 1, 'friendbiologists': 1, 'announce': 26, 'theyre': 36, 'done': 24, 'rodentsmegyn': 1, 'kelly': 23, 'story': 61, 'methe': 1, 'project': 16, 'going': 154, 'viralcreating': 1, 'leverage': 1, 'where': 114, 'none': 4, 'seems': 16, 'existnew': 1, 'taps': 6, 'love': 136, 'rapping': 5, 'kangaroosnewspaper': 1, 'worry': 12, 'spending': 25, 'much': 86, 'time': 209, 'facebook': 45, 'healthy': 28, 'itthe': 6, 'against': 153, 'lives': 45, 'matter': 21, 'central': 6, 'meaning': 7, 'amendmentvatican': 1, 'putting': 24, 'feelers': 1, 'public': 61, 'react': 4, 'crusadethe': 1, 'wire': 3, 'cast': 17, 'reunited': 6, 'uplift': 1, 'communityobamas': 1, 'supreme': 61, 'court': 103, 'nominee': 14, 'bragged': 3, 'sending': 14, 'freedeath': 1, 'dc': 23, 'security': 60, 'guard': 15, 'custody': 5, 'ruled': 4, 'homicideboehner': 1, 'leave': 32, 'successor': 1, 'clean': 26, 'slatetrump': 1, 'makes': 129, 'lastminute': 8, 'push': 30, 'appeal': 5, 'whitesfacing': 1, 'rising': 8, 'seas': 1, 'remote': 8, 'alaskan': 1, 'village': 7, 'votes': 24, 'move': 34, 'againall': 2, 'childs': 18, 'fondest': 1, 'memories': 7, 'dad': 64, 'trying': 88, 'thingsthe': 1, 'worst': 33, 'place': 46, 'childmom': 1, 'sentenced': 12, 'encouraging': 4, 'boyfriends': 5, 'sex': 77, 'assault': 41, 'babydonald': 1, 'valuesthe': 1, 'march': 39, 'inspired': 12, 'them': 59, 'run': 60, 'unseating': 1, 'menwas': 1, 'worth': 26, 'americagop': 2, 'senators': 26, 'arent': 27, 'accept': 9, 'championpricegouging': 1, 'pharma': 1, 'takes': 127, 'companyman': 3, 'failing': 15, 'heed': 2, 'harsh': 5, 'past': 43, 'sonic': 2, 'bacon': 9, 'cheeseburger': 3, 'toasterken': 1, 'completes': 12, 'documentary': 24, 'fucking': 57, 'liars': 6, 'claimed': 2, 'watched': 10, 'jazz': 9, 'seriesmagicmarkered': 1, 'initials': 2, 'fail': 10, 'deter': 3, 'breakroom': 3, 'ricecake': 2, 'thiefstraight': 1, 'outta': 3, 'compton': 2, 'stunning': 21, 'surprisecarly': 1, 'rae': 5, 'jepsen': 2, 'released': 14, 'thatgod': 1, 'humbled': 2, 'answer': 16, 'jeopardy': 2, 'cluefemale': 1, 'barista': 1, 'getting': 128, 'lot': 41, 'avoiding': 3, 'touching': 9, 'male': 25, 'patrons': 9, 'hands': 25, 'paygunman': 1, 'kills': 38, 'wounds': 1, 'four': 18, 'german': 8, 'nightclubcouple': 1, 'point': 25, 'comfortable': 10, 'using': 41, 'toilet': 8, 'same': 49, 'timemountain': 1, 'lion': 9, 'tracked': 2, 'scientists': 26, 'near': 30, 'malibu': 1, 'roadnoah': 1, 'latenight': 6, 'debut': 4, 'belting': 1, 'cryscalia': 1, 'goes': 68, 'abortion': 32, 'bender': 2, 'passed': 7, 'justicecampbells': 1, 'big': 122, 'cansized': 1, 'noodleis': 1, 'area': 92, 'finish': 16, 'those': 47, 'friestaylor': 1, 'swift': 27, 'gremlin': 1, 'voice': 24, 'writing': 21, '1989restaurants': 1, 'challenge': 18, 'rewards': 4, 'any': 90, 'patron': 7, 'consume': 3, 'reasonably': 1, 'portioned': 1, 'mealfun': 1, 'sticker': 2, 'placed': 8, 'ventilatorthe': 1, 'eat': 23, 'avocados': 2, 'avocado': 2, 'pastathat': 1, 'mcdonalds': 5, 'plate': 8, '70s': 2, 'holy': 6, 'shit': 25, 'there': 59, 'isan': 3, 'open': 55, 'letter': 42, 'progressives': 2, 'tpp': 3, 'yet': 27, 'progressive': 10, 'trade': 21, 'agreement': 2, 'historymodernday': 1, 'lancelot': 1, 'offers': 44, 'abortionunited': 1, 'airlines': 24, 'offering': 19, 'immigrants': 19, 'special': 38, 'flights': 9, 'circle': 6, 'awaiting': 2, 'gaps': 4, 'travel': 37, 'bansun': 1, 'thinking': 36, 'collapsing': 2, 'withjustice': 1, 'thomas': 12, 'retirement': 21, 'report': 58, 'bogustrump': 1, 'retweets': 1, 'video': 86, 'antimuslim': 7, 'hate': 28, 'grouplongsilent': 1, 'friend': 52, 'woodwork': 2, 'post': 23, 'insulating': 1, 'windowsstruggling': 1, 'airline': 4, 'helped': 21, 'friendly': 6, 'giant7': 1, 'breakup': 2, 'bosssure': 1, 'frankie': 1, 'muniz': 1, 'malcolm': 3, 'middle': 55, 'rebootusing': 1, 'united': 25, 'fiasco': 3, 'flourish': 1, 'futureunemployed': 1, 'sibling': 2, 'group': 49, 'mothers': 34, 'gift3822': 1, 'voted': 10, 'favorite': 43, 'pin': 2, 'numberthank': 1, 'thursday': 1, 'dominic': 1, 'casulli': 1, 'encouragement': 1, 'part': 80, '1a': 1, 'toe': 1, 'arctic': 5, 'ocean': 13, 'canadas': 3, 'northwest': 1, 'territories': 1, 'looney': 4, 'front': 36, '2it': 1, 'happen': 17, 'memark': 1, 'zuckerberg': 8, 'cited': 1, 'contempt': 4, 'congress': 54, 'refusing': 7, 'shut': 24, 'fuck': 30, 'he': 435, 'started': 9, 'dorm': 8, 'roomrubio': 1, 'supporters': 26, 'get': 289, 'scuffle': 1, 'rubiobotthe': 1, 'strangers': 10, 'review': 19, 'dr': 12, 'lloyd': 5, 'sedererdsm5': 1, 'updated': 8, 'accommodate': 5, 'legitimately': 1, 'ordered': 9, 'kill': 50, 'moonpurina': 1, 'debuts': 23, 'slovenly': 1, 'feast': 4, 'nastyass': 1, 'shelter': 8, 'catsobama': 1, 'chides': 1, 'darrell': 1, 'issa': 4, 'touting': 1, 'alliance': 3, 'him': 113, 'reelection': 6, 'fightpope': 1, 'iraq': 22, 'peace': 27, 'message': 39, 'meets': 22, 'reality': 26, 'warantman': 1, 'wasp': 1, 'trailer': 34, 'brings': 40, 'avengers': 3, 'infinity': 2, 'warfrontier': 1, 'customers': 20, 'deal': 56, 'itmoses': 1, 'seaeveryone': 1, 'forgets': 5, 'bring': 37, 'swimsuits': 1, 'coworkers': 14, 'partygeorge': 1, 'takei': 4, 'accused': 38, 'groping': 5, 'model': 15, '1981new': 1, 'york': 66, 'color': 17, 'target': 11, 'under70': 1, 'demographicthe': 1, 'hollywood': 34, 'boys': 33, 'club': 18, 'supports': 8, 'casey': 2, 'affleck': 11, 'total': 18, 'disgrace3': 1, 'myths': 5, 'low': 14, 'libidogirl': 1, 'scouts': 8, 'rocked': 2, 'cookies': 2, 'fundraising': 10, 'scandalchristian': 1, 'juggler': 2, 'wasted': 5, 'secular': 1, 'jugglerhuffpollster': 1, 'americans': 179, 'progress': 6, 'improvement': 3, 'voting': 22, 'rightsworld': 1, 'miss': 22, 'goal': 7, 'universal': 2, 'education': 36, '50': 27, 'unjennifer': 1, 'lopezs': 2, 'name': 45, 'jennifer': 13, 'lopez': 5, 'againhow': 2, 'rebuild': 5, 'credit': 14, 'bankruptcy': 1, 'fastdiscover': 1, 'rapidly': 3, 'transforming': 3, 'google': 19, 'timelapseman': 1, 'minute': 7, 'wonders': 11, 'do': 161, 'itvoting': 1, 'lines': 11, 'are': 469, 'shorter': 5, 'mostly': 11, 'whitesconrad': 1, 'bain': 1, 'steps': 25, 'down': 195, 'kitschreference': 1, 'laureateamanda': 1, 'slavin': 1, 'statisticlaurie': 1, 'hernandez': 3, 'val': 1, 'chmerkovskiy': 1, 'already': 67, 'dancing': 11, 'stars': 37, 'couplenew': 1, 'photos': 54, 'show': 139, 'hurricane': 24, 'matthews': 1, 'path': 8, 'destruction': 6, 'haitiflorida': 1, 'lawmakers': 13, 'vote': 46, 'ban': 42, 'marriage': 47, 'under': 98, 'age': 43, '17nasa': 1, 'inadvertently': 3, 'launches': 24, 'unmanned': 2, 'space': 26, 'shuttlenick': 1, 'viall': 1, 'bachelorwhy': 1, 'america': 71, 'demonizes': 1, 'teachersnew': 1, 'instant': 5, 'lottery': 6, 'three': 39, 'win': 48, '19839947': 1, 'losepodcast': 1, 'cry': 5, 'helpfashion': 1, 'designer': 12, 'peter': 8, 'soms': 1, 'legendary': 7, 'minestrone': 1, 'soup': 2, 'recipephotography': 1, 'spotlighting': 1, 'iconic': 9, '70': 7, 'proves': 17, 'comeclairvoyant': 1, 'vince': 2, 'vaughn': 1, 'accepts': 10, 'offeredreport': 1, 'kenan': 1, 'leaving': 30, 'snl': 12, 'deemed': 5, 'inaccuratewoman': 1, 'hid': 1, 'heroin': 10, 'oxy': 1, 'fake': 24, 'butt': 6, 'cops': 21, 'saydont': 1, 'meisis': 1, 'g41': 1, 'worldmelania': 1, 'hangs': 6, 'decayed': 1, 'badger': 1, 'carcass': 4, 'white': 177, 'house': 178, 'mantel': 1, 'traditional': 9, 'slovenian': 1, 'decorwoman': 1, 'admits': 55, 'having': 63, 'jodie': 1, 'foster': 4, 'embarrassed': 10, 'herselfsarah': 1, 'defends': 18, 'sexist': 14, 'attack': 56, 'kirsten': 2, 'gillibrandsaying': 1, 'goodbye': 11, 'hollywoods': 4, 'hottest': 7, 'seediest': 1, 'addressbush': 1, 'increasingly': 15, 'focused': 5, 'revisionist': 1, 'history': 67, 'him11': 1, 'later': 12, 'human': 64, 'genome': 1, 'paves': 3, 'genomic': 1, 'technonlogyman': 1, 'claiming': 4, 'boko': 2, 'haram': 2, 'leader': 29, 'appears': 12, 'videohow': 1, 'knowing': 9, 'proof': 11, 'heavenon': 1, 'pilgrimage': 3, 'indiawill': 1, 'religious': 18, 'freedom': 19, 'advocates': 9, 'oppose': 8, 'roy': 17, 'mooreuaw': 1, 'fca': 1, 'negotiating': 5, 'hourbyhour': 1, 'contract': 8, 'extensionif': 1, 'bernie': 29, 'wanted': 19, 'solve': 12, 'trumpthere': 1, 'right': 120, 'writerest': 1, 'evening': 7, 'spent': 21, 'declaring': 1, 'asshole': 10, 'ruin': 11, 'eveningnorth': 1, 'carolina': 26, 'doesnt': 119, 'seem': 7, 'want': 101, 'camera': 12, 'footagethe': 1, 'department': 45, 'officially': 15, 'wont': 93, 'transgender': 31, 'experiencing': 7, 'bathroom': 25, 'discriminationwhy': 1, 'participating': 2, 'fridayhooded': 1, 'drown': 1, 'child': 84, 'potomac': 1, 'prevent': 26, 'affair': 12, 'outmerkel': 1, 'isis': 20, 'poses': 9, 'risk': 22, 'europealcoholic': 1, 'kindergarten': 3, 'stretches': 2, 'naptime': 1, 'hoursreport': 2, 'good': 139, 'kinddevastating': 1, 'floods': 7, 'west': 28, 'virginiaobamas': 1, 'oratory': 1, 'performancegrandmother': 1, 'palms': 2, 'fixing': 2, 'boxing': 2, 'matchbumper': 1, 'nilla': 1, 'crop': 3, 'spells': 3, 'profit': 4, 'wafer': 1, 'growers6': 1, 'summer': 41, 'salads': 3, 'actually': 96, 'cravethe': 1, 'keys': 7, 'album': 22, 'blue': 25, 'available': 11, 'stream': 8, 'fullisis': 1, 'vs': 21, 'isil': 1, 'whats': 20, 'namenational': 1, 'trust': 11, 'historic': 21, 'preservation': 1, 'andy': 5, 'rooneys': 1, 'upkeeplist': 1, 'names': 34, 'gchat': 2, 'sidebar': 1, 'portal': 3, 'mans': 82, 'livestomi': 1, 'lahren': 3, 'suing': 5, 'glenn': 5, 'beck': 1, 'theblazegreg': 1, 'hardy': 3, 'unapologetically': 2, 'domestic': 19, 'abuse': 26, 'allegationstourists': 1, 'describe': 5, 'scenes': 6, 'tunisian': 1, 'beach': 18, 'massacretrump': 1, 'blinkphotos': 1, 'ahs': 1, 'set': 47, 'explain': 18, 'creepy': 13, 'blond': 3, 'childrenstudy': 1, 'couples': 24, 'delaying': 3, 'divorce': 13, 'until': 60, 'old': 81, 'enough': 69, 'remember': 29, 'every': 134, 'painful': 9, 'detailneverbeforeheard': 1, 'buzzword': 1, 'flying': 14, 'goodman': 1, 'certain': 3, 'arepsychologists': 1, 'smartphone': 8, 'warning': 17, 'labelsclinton': 1, 'receives': 27, '400000': 4, 'honorary': 1, 'degrees': 5, 'college': 82, 'commencement': 10, 'speechscavengerhunt': 1, 'party': 86, 'twinethe': 1, 'republican': 57, 'obamacare': 39, 'dilemma': 1, '6minute': 2, 'videomass': 1, 'graves': 3, 'suggest': 5, 'systematic': 1, 'rohingya': 8, 'myanmarfinal': 1, 'uboat': 1, 'surrenders': 2, 'allied': 1, 'powersprivate': 1, 'companies': 13, 'lock': 3, 'despite': 20, 'doj': 6, 'decisionwomans': 1, 'feared': 6, 'forever': 12, 'scarffedup': 1, 'brookstone': 1, 'bodymassage': 1, 'chair': 21, 'entertaining': 3, 'serious': 14, 'buyerstrump': 1, 'senate': 57, 'chamber': 6, 'speak': 20, 'comey': 22, 'alonegrateful': 1, 'lyricist': 1, 'internet': 29, 'pioneer': 1, 'perry': 15, 'barlow': 1, '70new': 1, 'finds': 151, 'interested': 3, 'science': 14, 'moon': 11, 'looks': 56, 'usualnew': 1, 'moves': 22, 'content': 10, 'give': 92, 'unless': 13, '200k': 1, 'convenient': 2, 'sectionphilippines': 1, 'civilians': 6, 'addictswoman': 1, 'perfectly': 21, 'shes': 42, 'nursechicago': 1, 'monitor': 6, 'interactions': 1, 'bullet': 4, 'camsamid': 1, 'chattering': 1, 'global': 32, 'elite': 4, 'silent': 8, 'interludeat': 1, 'ethical': 5, 'issues': 32, 'dogging': 1, 'epa': 15, 'administrator': 3, 'scott': 26, 'pruittwont': 1, 'ask': 44, 'dont': 102, 'tellweird': 1, 'dot': 1, 'bowlvegas': 1, 'baby': 82, 'well': 24, 'minus': 1, 'babiespeople': 1, 'kanyes': 2, 'song': 25, 'real': 104, 'friends': 64, 'because': 67, 'good20': 1, 'suede': 1, 'pieces': 6, 'wear': 19, 'springfcc': 1, 'sentences': 2, 'artie': 1, 'lange': 1, 'deathbehold': 1, 'title': 11, 'season': 62, '7footballloving': 1, 'harassed': 3, 'wearing': 35, 'turbans': 1, 'nfl': 29, 'gameto': 1, 'healthnew': 1, 'job': 65, 'posting': 6, 'craigslist': 3, 'secretary': 36, 'interiorseth': 1, 'meyers': 13, 'loses': 35, 'thanksgivings': 1, 'proximity': 1, 'christmas15': 1, 'knicks': 5, 'carmelo': 1, 'anthonyparents': 1, 'fight': 62, 'two': 73, 'daughters': 15, 'tragic': 5, 'diagnosisthe': 1, 'quiet': 12, 'practice': 8, 'voicefcc': 1, 'verizon': 3, 'websites': 2, 'affected': 10, 'net': 10, 'neutrality': 11, 'repealeveryone': 1, 'second': 46, 'harder': 15, 'wrongreport': 1, 'nowhere': 2, 'okay': 11, 'act': 24, 'trumpman': 1, 'unleashes': 4, 'trumpinspired': 3, 'antilgbtq': 7, 'rant': 6, 'churchwhite': 1, 'sprinter': 1, 'finishes': 6, 'fifthhow': 1, 'finding': 20, 'wendy': 1, 'davisobama': 1, 'begins': 24, 'inauguration': 15, 'festivities': 1, 'ceremonial': 3, 'drone': 11, 'flyoverlindsay': 1, 'wagner': 1, 'anything': 31, 'offered': 6, 'herdemocratic': 1, 'senator': 25, 'strides': 1, 'corridors': 1, 'powerlessnessheres': 1, 'rage': 3, 'opened': 5, 'door': 23, 'james': 50, 'firinglooking': 1, 'happiness': 6, 'wrong': 36, 'placesswarm': 1, 'drones': 3, 'dance': 25, 'beethovens': 1, 'symphony': 3, '5': 75, 'night': 49, 'skyfatal': 1, 'crash': 26, 'cements': 1, 'bff': 1, 'statusnomakeup': 1, 'look': 99, 'easier': 11, 'achieve': 4, 'elle': 1, 'claimsthe': 1, 'outfit': 4, 'kicked': 5, 'schools': 26, 'gymbluetooth': 1, 'headset': 1, 'worn': 5, 'throughout': 17, 'datebreakroom': 1, 'tension': 4, 'alltime': 6, 'high': 79, 'mug': 6, 'disputedepression': 1, 'symptom': 2, 'checklist': 2, 'speaking': 14, 'poem': 4, 'ever': 64, 'couldparents': 1, 'formally': 4, 'transfer': 3, 'expectations': 2, 'childsaturday': 1, 'halloween': 23, 'spooktacular': 1, 'montagesteven': 1, 'spielberg': 6, 'career': 28, 'salvagedlocal': 1, 'laundromat': 3, 'employs': 2, 'social': 54, 'media': 66, 'coordinatorpartygoer': 1, 'kegright': 1, 'handheld': 2, 'deadly': 28, 'metal': 5, 'pellets': 1, 'speed': 9, 'thisoscar': 1, 'countdown': 1, '2002': 1, 'beginsdisaster': 1, 'movies': 18, 'tame': 2, 'compared': 7, 'happened': 28, '33': 5, 'billion': 37, 'agowashington': 1, 'officer': 27, 'carrying': 13, 'knifein': 1, 'face': 63, 'hidden': 7, 'surgery': 6, 'mattera': 1, 'fans': 55, 'santawolf': 1, 'blitzer': 5, 'decks': 1, 'boston': 6, 'hasnt': 28, 'healed': 2, 'sox': 1, 'baseballdonald': 1, 'blood': 21, 'sport': 1, 'choicetime': 1, 'end': 78, 'subsidies': 1, 'destroying': 4, 'forestsnetflix': 1, 'planned': 28, 'louis': 21, 'ck': 6, 'comedy': 12, 'specialtrump': 1, 'undercuts': 1, 'easy': 25, 'dig': 2, 'bill': 96, 'clintons': 27, 'infidelitiesnarcissist': 1, 'mentally': 13, 'undresses': 1, 'selfwe': 1, 'exclusive': 9, 'hillarys': 3, 'dnc': 20, 'speech': 44, 'noteswatch': 1, 'kasich': 9, 'lead': 35, 'awkward': 13, 'bowie': 6, 'singalongpro': 1, 'governing': 2, 'fakedhow': 1, 'detox': 1, 'ed': 5, 'become': 42, 'productiveoutdoor': 1, 'guest': 20, 'excited': 45, 'barely': 15, 'audible': 2, 'future': 51, 'sitting': 20, 'tree': 19, 'rootnorth': 1, 'planning': 23, 'fire': 64, 'missiles': 3, 'guamgorillagram': 1, 'employee': 44, 'shot': 47, 'securityhow': 1, '4monthold': 1, 'fall': 27, 'reading15': 1, 'weirdest': 4, 'left': 48, 'uberarkansas': 1, 'executes': 4, 'inmate': 11, 'yearsmeet': 1, 'other': 99, 'baldwin': 5, 'brother': 18, 'jamesa': 1, 'sadder': 3, 'pride': 13, 'washington': 34, 'inactionbrad': 1, 'pitt': 9, 'stumbles': 5, 'across': 30, 'cardboard': 3, 'gwyneth': 3, 'paltrows': 2, 'atticshonda': 1, 'rhimes': 2, 'motivation': 2, 'journeyobama': 1, 'plans': 55, 'inequalityteaching': 1, 'learning': 16, 'ratings': 3, 'frameworkfashionforward': 1, 'pigeon': 2, 'bread': 7, 'necklace': 2, 'bold': 6, 'style': 16, 'choicevideo': 1, 'relaunches': 2, 'investigation': 24, 'death': 103, 'held': 18, 'chicago': 16, 'policewhoopi': 1, 'goldberg': 1, 'oscars': 21, 'won': 19, 'oncehousefly': 1, 'tracks': 9, 'cucumber': 1, 'slicehillary': 1, 'clinton': 158, 'opens': 36, 'presidential': 65, 'library': 4, 'charting': 2, 'course': 19, 'purely': 1, 'theoretical': 2, 'tenure': 1, 'commander': 4, 'chiefbounty': 1, 'brawny': 3, 'ceos': 4, 'patience': 4, 'mutual': 2, 'friendsherman': 1, 'alexie': 1, 'artists': 18, 'noisecanceling': 1, 'headphonesvisibly': 1, 'flustricken': 1, 'choir': 2, 'kid': 34, 'dragging': 4, 'whole': 33, 'pageantnbc': 1, 'correspondent': 1, 'ayman': 1, 'mohyeldin': 1, 'returning': 8, 'gazatim': 1, 'robbins': 2, 'tired': 23, 'typecast': 2, 'relatively': 2, 'tall': 7, 'characterssen': 1, 'tom': 23, 'cotton': 4, 'tough': 12, 'guy': 78, 'resume': 3, 'waterboardingleatherclad': 1, 'ted': 20, 'greeting': 3, 'renoarea': 1, 'fetish': 1, 'clubmans': 1, 'family': 126, 'rises': 10, 'recordhigh': 1, 'prioritymike': 1, 'ditka': 1, 'paying': 12, 'attention': 8, 'historywhat': 3, 'mom': 73, 'evolving': 3, 'planningsimple': 1, 'task': 4, 'feels': 38, '10000': 13, 'bouldersreport': 1, 'crazy': 15, 'probably': 57, 'classmate': 3, 'raised': 9, 'grandmotherivanka': 1, 'incorrectly': 3, 'judaism': 2, 'largest': 15, 'religionsrescued': 1, 'obsessed': 7, 'blankets': 2, 'since': 43, 'babycounty': 1, 'fair': 13, 'judges': 3, 'blown': 5, 'heiferthe': 1, 'toys': 7, 'shop': 24, 'prime': 18, 'agesthe': 2, 'phenomenonclinton': 1, 'staff': 18, 'readies': 3, 'emp': 1, 'launch': 24, 'disable': 1, 'electronic': 2, 'deviceskhloe': 1, 'kardashian': 35, 'joins': 14, 'waist': 4, 'gangnext': 1, 'weeks': 22, 'debate': 59, 'utah': 3, 'canceleda': 1, 'trove': 1, 'mystery': 9, 'beautiful': 45, 'lifebear': 1, 'pool': 12, 'cutest': 5, 'backyard': 6, 'invasion': 3, 'everdustin': 1, 'hoffman': 4, 'alleged': 12, 'joint': 6, 'nbc': 7, 'interviewanother': 1, 'betsy': 7, 'devos': 16, 'teacher6': 1, 'tips': 43, 'boost': 10, 'careervideo': 1, 'shows': 88, 'transphobic': 4, 'preaching': 1, 'customerxmasgiftsfromtrump': 1, 'very': 74, 'unmerry': 1, 'christmasronda': 1, 'rousey': 2, 'eerily': 2, 'predicted': 2, 'losepolice': 1, 'release': 22, 'haircutprogressed': 1, 'photo': 46, 'missing': 43, 'womana': 1, 'brief': 9, 'pun': 1, 'intended': 9, 'lawyers': 13, 'moviesgifs': 1, 'memphis': 3, 'grizzlies': 2, 'weathered': 1, 'epic': 13, 'oklahoma': 7, 'thunder': 2, 'stormstomach': 1, 'sets': 23, 'aside': 11, 'synthetic': 1, 'additives': 1, 'few': 43, 'minutes': 46, 'figure': 18, 'digest': 1, 'themrussia': 1, 'compromising': 1, 'information': 11, 'trumpwhat': 3, 'comics': 5, 'offer': 15, 'bible': 12, 'readersbreaking': 1, 'uniformcongress': 1, 'approves': 16, '250': 5, 'billion7': 1, 'forgotten': 12, 'facts': 12, 'war': 90, 'booktaylor': 1, 'dating': 33, 'watertown': 1, 'boatrapture': 1, 'wreaks': 2, 'havoc': 5, 'local': 60, 'book': 55, 'clubthe': 1, 'key': 32, 'setting': 8, 'achievable': 2, 'goalsthe': 2, 'shopping': 12, 'guide': 51, 'cyber': 8, 'monday': 11, 'needdonald': 1, 'refugee': 23, 'crisis': 27, 'threats': 15, 'identity': 10, 'drove': 4, 'brexitscience': 1, 'fiction': 4, 'writer': 15, 'unstoppable': 1, 'machine': 13, 'based': 6, 'mothergunman': 1, 'advisers': 5, 'kabul': 3, 'shootingkris': 1, 'kobach': 3, 'private': 19, 'government': 65, 'business10yearold': 1, 'girlliking': 1, 'allegationsstray': 1, 'lumber': 1, 'section': 12, 'home': 102, 'depotoffice': 1, 'manager': 19, 'rulecharles': 1, 'darwin': 3, 'sunmineanother': 1, 'motherssheets': 1, 'breakupwant': 1, 'meetings': 4, 'productive': 6, 'start': 49, 'walkingman': 1, 'overjoyed': 1, 'purchase': 11, 'days': 48, 'egg': 9, 'mcmuffins': 1, 'morningwatch': 1, 'stone': 12, 'brewing': 2, 'evacuates': 2, 'wildfire': 7, 'approachesmilitary': 1, 'prosecutor': 7, 'cia': 15, 'interrogation': 3, 'accuratenational': 1, 'park': 26, 'service': 42, 'studies': 4, 'sites': 7, 'possible': 29, 'recognitionsafeguarding': 1, 'system': 30, 'sabotagejonathan': 1, 'safran': 1, 'foer': 1, 'guesses': 3, 'silly': 4, 'dream': 43, 'becoming': 19, 'writerskydiver': 1, 'luke': 5, 'aikins': 1, 'jump': 5, 'parachutestudy': 1, 'universe': 9, 'shrunk': 1, '19': 7, 'inches': 6, 'yearpowercrazed': 1, 'orkin': 1, 'groundcouples': 1, 'fuckedup': 2, 'presex': 1, 'ritual': 3, 'involves': 11, 'tucking': 2, 'bedwill': 1, 'republicans': 39, 'kowtow': 1, 'far': 24, 'rightviral': 1, 'captures': 11, 'incredible': 18, 'moment': 37, 'between': 66, 'homeless': 25, 'mangoogle': 1, 'employees': 33, '15th': 2, 'anniversary': 24, 'solarpowered': 2, 'lego': 2, 'drag': 17, 'race': 36, 'reffed': 1, 'poguestudy': 1, 'average': 20, 'each': 68, 'yearrnc': 1, 'builds': 8, 'levee': 1, 'poor': 28, 'protect': 24, 'convention': 14, 'siteromney': 1, 'delivers': 16, 'stern': 3, 'china': 27, 'directly': 13, 'fluent': 1, 'mandarinthe': 1, 'sent': 17, 'stock': 14, 'market': 18, 'tumblingvoters': 1, 'shocked': 15, 'christie': 18, 'botched': 4, 'such': 12, 'coverupdesperate': 1, 'dole': 4, 'promises': 24, 'prom': 6, 'evera': 1, 'kind': 39, 'momuber': 1, 'halts': 4, 'selfdriving': 5, 'car': 53, 'tests': 8, 'test': 24, 'anywayfrom': 1, 'tiger': 10, 'fresh': 11, 'boat': 15, 'eddie': 3, 'huangs': 1, 'asianamerican': 1, 'momdonald': 1, 'roger': 9, 'goodell': 4, 'weak': 8, 'stupid': 12, 'dopetrump': 1, 'libel': 1, 'laws': 13, 'thats': 25, 'statesshirtless': 1, 'goofball': 1, 'flag': 11, 'underwear': 4, 'invades': 2, 'field': 13, 'serieslocal': 1, 'moviegoer': 1, 'enjoying': 13, 'farwhy': 1, 'chinese': 16, 'parents': 95, 'youamericans': 1, 'creating': 10, 'solvestheater': 1, 'long': 56, 'borne': 3, 'shakespeare': 2, 'blunt': 3, 'upbraidings': 1, 'bitter': 4, 'scoffsa': 1, 'lancet': 1, 'breakthrough': 5, 'publishing': 4, 'faith': 13, 'healtharea': 1, 'grasshopper': 1, 'thorax': 1, 'himselffreemasons': 1, 'return': 30, 'jupiterbanjowielding': 1, 'matt': 16, 'damon': 6, 'bid': 13, 'original': 11, 'songafter': 1, 'criticism': 9, 'cleveland': 7, 'officials': 38, 'outline': 2, 'plansnew': 1, 'teen': 48, 'trend': 11, 'walking': 32, 'wet': 6, 'nude': 14, 'couldnt': 15, 'caught': 22, 'worse': 23, 'timejustin': 1, 'bieber': 10, 'interrupts': 3, 'performance': 18, 'scold': 1, 'spanish': 8, 'audiencemudbound': 1, 'oscar': 22, 'nominations': 4, 'netflix': 15, 'leaguesarea': 1, 'uses': 30, 'wtc': 1, 'excuse': 9, 'call': 70, 'exgirlfriendmccain': 1, 'gets': 142, 'hammered': 2, 'vfw12yearolds': 1, 'demonstrates': 3, 'awareness': 8, 'familys': 11, 'financial': 26, 'predicamentkendrick': 1, 'lamar': 9, 'taylor': 19, 'weeknd': 2, '2016': 51, 'grammy': 7, 'nominationsslight': 1, 'inconvenience': 1, 'avoidedfrustrated': 1, 'fcc': 4, 'unable': 26, 'use': 68, 'word': 38, 'frigginnbc': 1, 'dramatize': 1, 'menendez': 1, 'brothers': 15, 'murders': 2, 'law': 61, 'order': 34, 'true': 22, 'crime': 19, 'spinoffpolice': 1, 'assume': 6, 'truck': 16, 'deliberately': 2, 'driven': 3, 'berlin': 2, 'marketarea': 1, 'cleans': 7, 'apartment': 19, 'relationshipafter': 1, 'orlando': 11, 'chefs': 2, 'provide': 9, 'thousands': 38, 'free': 59, 'mealsgop': 1, 'sen': 5, 'jeff': 14, 'flake': 3, 'might': 65, 'deserve': 8, 'leaddentist': 1, 'buy': 43, 'candymta': 1, 'assure': 9, 'yorkers': 7, 'todays': 12, 'subway': 14, 'fucked': 6, 'normalroutine': 1, 'drunkdriving': 1, 'trip': 32, 'turns': 33, 'five': 27, 'teensgrizzly': 1, 'bear': 20, 'sprained': 1, 'paw': 1, 'mauling': 1, 'hunter': 5, 'rangertrump': 1, 'selects': 3, 'longtime': 6, 'personal': 45, 'plane': 25, 'faanew': 1, 'antidepressant': 1, 'worserestaurant': 1, 'teaches': 14, 'inmates': 12, 'cook': 15, 'feet5': 1, 'entrepreneurial': 3, 'rules': 31, 'bylaser': 1, 'pointer': 1, 'aimed': 2, '1997': 2, 'annoying': 9, 'planet': 17, '13': 14, 'lightyears': 1, 'awayman': 1, 'skipped': 1, 'airports': 3, 'moving': 28, 'walkway': 1, 'immediately': 22, 'realizes': 30, 'arrogant': 3, 'fool': 5, 'been7': 1, 'know': 155, 'christmaspaul': 1, 'beatty': 1, 'booker': 4, 'prize': 14, 'fictionyour': 1, 'female': 45, 'heroes': 6, 'seriesnation': 1, 'checks': 10, 'cnncom': 1, 'pals': 4, 'tsarnaevs': 1, 'castros': 1, 'tosmall': 1, 'town': 43, 'wonder': 7, 'taking': 52, 'epidemic': 10, 'therenew': 1, 'superpacs': 1, 'record': 48, 'hauldomestic': 1, 'survivor': 12, 'gives': 59, 'young': 41, 'victims': 27, 'haddogs': 1, 'smell': 7, 'grandmas': 4, 'scent': 2, 'quest': 6, 'hersmbs': 1, 'businesspresidents': 1, 'manufacturing': 2, 'council': 14, 'shoe': 3, 'carnivalworlds': 1, 'physicists': 2, 'study': 68, 'physicshow': 1, 'difficult': 12, 'conversation': 31, '62heroic': 1, 'turtle': 5, 'dials': 1, '911what': 1, 'miscarriagebusinesses': 1, 'antilgbt': 7, 'bills': 6, 'cost': 19, 'texas': 31, 'billionsaccused': 1, 'killer': 16, 'army': 14, 'whod': 1, 'askedavid': 1, 'fisherman': 2, 'ruins': 6, 'fishing': 4, 'sonbritish': 1, 'hunt': 6, 'witches': 1, 'marks': 10, 'halloweennation': 2, 'relieved': 14, 'insufferable': 2, 'thrones': 24, 'seasonthe': 3, 'volunteer': 7, 'programs': 4, 'thisnasa': 1, 'completely': 29, 'forgot': 6, 'probe': 16, 'today3m': 1, 'introduces': 80, 'protective': 1, 'foam': 1, 'eye': 16, 'plugsmichael': 1, 'b': 12, 'jordan': 10, 'fahrenheit': 2, '451': 1, 'trailerastronomers': 1, 'predict': 3, 'asteroid': 1, 'hit': 36, 'theaters': 4, 'summerhero': 1, 'common': 20, 'talks': 45, 'plumber': 1, 'housesocial': 1, 'unity': 5, 'important': 32, 'pm': 10, 'modi': 1, 'indias': 4, '70th': 2, 'independence': 9, 'dayroe': 1, 'made': 94, 'abortions': 4, 'keep': 69, 'providers': 4, 'safehussein': 1, 'judge': 44, 'hoping': 33, 'speedy': 2, 'assassinationsavvy': 1, 'registers': 2, 'sleepy': 4, 'romney': 25, 'twitter': 37, 'account': 12, 'candidate': 42, 'tireddreamers': 1, 'nightmare': 6, 'deportation': 9, 'forcethird': 1, 'disease': 13, 'eliminated': 2, 'hotair': 1, 'hand': 21, 'dryerswhoops': 1, 'selfie': 7, 'snapper': 1, 'smashes': 2, 'sculpture': 3, 'exhibit': 6, 'opensaspiring': 1, 'publicly': 4, 'voicing': 1, 'regret': 9, 'woody': 6, 'allenswiss': 1, 'maintain': 4, 'delicious': 12, 'pastries9': 1, 'admitthousands': 1, 'orleans': 6, 'households': 1, 'powerwatch': 1, 'underreported': 2, 'reuniting': 2, 'families': 19, 'remains': 16, 'migrantsyahoos': 1, 'newfront': 2, 'pulses': 1, 'steve': 22, 'aokis': 1, 'edm': 1, 'beatlook': 1, 'goddamn': 9, 'beautifulhow': 1, 'islamic': 8, 'state': 117, 'obama': 130, 'administration': 42, 'considers': 7, 'deploying': 1, 'troops': 11, 'focusing': 1, 'assad': 4, 'syriaerics': 1, 'bogosians': 1, 'operation': 4, 'nemesis': 2, 'genocide': 5, 'avengedantarctic': 1, 'observational': 2, 'comic': 6, 'running': 53, 'ideasthousands': 1, 'moscow': 3, 'housing': 7, 'plannew': 1, 'pixar': 5, 'required': 4, 'adorable': 34, 'sexual': 90, 'harassment': 22, 'videocosmopolitan': 1, 'releases': 63, '40year': 1, 'compendium': 1, '812683': 1, 'manrow': 1, 'asterisks': 1, 'spices': 3, 'otherwise': 2, 'ordinary': 7, 'emailthis': 1, 'tattoo': 8, 'safe': 17, 'accepting': 10, 'queer': 29, 'bodiessears': 1, 'gold': 21, 'card': 24, 'holder': 5, 'pushing': 8, 'searsauthor': 1, 'lindsay': 3, 'serial': 14, 'dexter': 1, 'final': 33, 'novelafterwork': 1, 'drinks': 12, 'enter': 10, 'third': 34, 'excruciating': 2, 'minutekoch': 1, 'network': 12, '400': 13, '2015those': 1, 'werent': 3, 'nooses': 1, 'university': 23, 'delawareman': 1, 'sentence': 12, 'lined': 1, 'conversationarea': 1, 'waitress': 3, 'hell': 38, 'ass': 7, 'tell': 82, 'now30': 1, 'unbeleafably': 1, 'gifts': 13, 'plant': 9, 'loverscuomo': 1, 'surprise': 29, 'afghanistan': 11, 'tripdrummers': 1, 'girlfriend': 20, 'should': 163, 'singnobody': 1, 'matthew': 6, 'mcconaugheys': 1, 'youtube': 8, 'channel': 16, 'nowpope': 1, 'oldwhat': 1, 'parent': 9, 'mevideo': 1, 'icantbreathe': 1, 'floorman': 1, 'somehow': 26, 'overcomes': 2, 'alcoholism': 3, 'jesusdonald': 1, 'w': 5, 'bushmom': 1, 'full': 46, 'refer': 5, 'bisquick': 1, 'impossibly': 1, 'piecongressman': 1, 'idiot': 9, 'egypt': 3, 'mosque': 6, 'promote': 8, 'border': 19, 'wallluke': 1, 'bryan': 3, 'confederate': 9, 'symbol': 4, 'racismrich': 1, 'firstgrader': 2, 'buys': 13, 'sheet': 3, 'starswe': 1, 'aint': 6, 'germanskentucky': 1, 'governor': 27, 'echoes': 2, 'sides': 3, 'blame': 17, 'charlottesville': 6, 'violencehow': 1, 'caitlyn': 5, 'helping': 25, 'menew': 1, 'apple': 20, 'consumers': 8, 'iphone': 17, 'handwoman': 1, 'dozing': 1, 'dave': 7, 'eggers': 2, 'againmanager': 1, 'combination': 2, 'taco': 8, 'bellkfc': 1, 'secretly': 12, 'bellcould': 1, 'hillary': 79, 'defeat': 11, 'democrats': 41, '2008wild': 1, 'tales': 5, 'outstanding': 1, 'cannesdonald': 1, 'prefers': 4, 'violent': 13, 'football': 21, 'players': 16, 'espn': 4, 'analystblack': 1, 'ribbon': 2, 'balsamthe': 1, 'buttermilk': 1, 'biscuit': 2, 'recipes': 20, 'needai': 1, 'weiwei': 1, 'commemorates': 1, 'drowned': 2, 'refugees': 22, 'festivalthe': 1, 'restrictions': 6, 'journalists': 12, 'agreed': 5, 'attend': 18, 'koch': 4, 'conference7yearold': 1, 'puts': 36, 'uno': 1, 'facelittle': 1, 'stands': 12, 'talljake': 1, 'tapper': 5, 'oneword': 2, 'response': 42, 'kellyanne': 5, 'conway': 7, 'sketchpoll': 1, 'birth': 41, 'control': 45, 'mandate': 1, 'eve': 8, 'rulingdeath': 1, 'row': 13, 'kosher': 1, 'foodjada': 1, 'pinkett': 1, 'smith': 13, 'gabrielle': 1, 'union': 31, 'feud': 5, '17': 18, 'yearshuffpost': 1, 'rise': 40, 'february': 8, '10dear': 1, 'mr': 16, 'dispatch': 1, 'bowling': 2, 'greenshinto': 1, 'priestess': 1, 'killed': 60, 'sword': 2, 'tokyo': 1, 'shrinejournalists': 1, 'correspondents': 5, 'associations': 1, 'michelle': 16, 'wolfnina': 1, 'dobrev': 2, 'addresses': 6, 'rumored': 5, 'vampire': 2, 'diariestheresa': 1, 'edging': 1, 'towards': 5, 'scolds': 3, 'kerry': 18, 'israelchild': 1, 'assured': 6, 'diesnew': 1, 'prank': 3, '4yearold': 6, 'glued': 1, 'toiletprosecutors': 1, 'tamir': 3, 'rice': 9, 'bizarrely': 2, 'pointed': 3, 'toy': 9, 'gun': 85, 'witness': 2, 'allegewhats': 1, 'picture': 26, 'everythingafi': 1, 'docs': 1, 'policy': 30, 'artschool': 1, 'bully': 5, 'molestedthe': 1, 'daydad': 1, 'teaching': 7, 'safety3': 1, 'unexpected': 14, 'mindful': 3, 'screen': 16, 'timedee': 1, 'bogettis': 1, 'momentpaul': 1, 'ryan': 43, 'confident': 7, 'warm': 7, 'realize': 18, 'cruel': 6, 'meaningless': 1, 'farcewartorn': 1, 'bloodsoaked': 1, 'kosovo': 1, 'bombing': 11, 'helpstarving': 1, 'fantasybill': 1, 'waiting': 33, 'primaries': 2, 'endorse': 6, 'candidatewhat': 1, 'volkswagen': 1, 'recalledto': 1, 'read': 28, 'or': 149, '2hillary': 2, 'iowa': 26, 'pollseveral': 1, 'accuse': 6, 'executive': 30, 'don': 7, 'hazen': 1, 'harassmentsuccess': 1, 'placelast': 1, 'wakeup': 7, 'manpollsters': 1, 'admit': 10, 'underestimated': 1, 'adrenal': 1, 'glandslinmanuel': 1, 'miranda': 8, 'freestyles': 1, 'lifes': 3, 'minor': 10, 'inconveniences': 1, 'ellenjohn': 1, 'lewis': 6, 'civil': 24, 'museum': 18, 'opening': 23, 'goingis': 1, 'trouble': 10, '2015': 17, 'npt': 1, 'conferencearea': 1, 'ahead': 23, 'consider': 10, 'date4': 1, 'mauled': 3, 'congressional': 21, 'showhuffpost': 1, 'hill': 16, 'agents': 12, 'glad': 12, 'dark': 30, 'sunglasses': 6, 'hide': 14, 'bloodshot': 1, 'eyeskite': 1, 'flyer': 3, 'zonebacon': 1, 'sprints': 2, 'downloadable': 2, 'ring': 13, 'scentsi': 1, 'photograph': 4, 'rememberus': 1, 'bobsled': 2, 'pays': 20, 'tribute': 16, 'medalist': 1, 'steven': 5, 'holcombamong': 1, 'fes': 1, 'many': 56, 'virtues': 1, 'art': 40, 'hospitality': 1, 'vintage': 3, 'clothingpower': 1, 'foes': 3, 'backers': 1, 'venus': 3, 'actuallyjournalists': 1, 'refuse': 5, 'nonanswer': 1, 'answernukes': 1, 'schismweird': 1, 'relative': 2, 'reunion': 8, 'knows': 43, 'related': 2, 'othernegro': 1, 'week': 46, '19391940': 1, 'fairtrump': 1, 'threatens': 24, 'veto': 4, 'wall': 36, 'funding': 20, 'then': 16, 'signs': 31, 'it6': 1, 'diy': 4, 'stress': 14, 'hacks': 6, 'closetthe': 1, 'concern': 7, 'phone': 39, 'taiwana': 1, 'weird': 22, 'wonderful': 5, 'cabaret': 1, 'chronicle': 1, 'karen': 2, 'mason': 2, 'revisits': 1, 'roots': 3, 'mamawhy': 1, 'risks': 4, 'changethe': 1, 'louvre': 1, 'gardens': 2, 'teeming': 1, 'ratstrump': 1, 'macarenaboy': 1, 'george': 20, 'politics': 22, 'junkie': 5, 'morearea': 1, 'decides': 19, 'status': 7, 'tipped': 1, 'debatesprint': 1, 'tmobile': 2, 'merge': 2, 'grotesque': 7, 'hybridpatrick': 1, 'stewart': 16, 'reads': 4, 'hilariously': 11, 'bad': 79, 'reviews': 5, 'tourist': 8, 'attractionssafeguarding': 1, 'wellbeing': 2, 'childrenthe': 1, 'threat': 20, 'angertarget': 1, 'essentials': 4, 'aisle': 3, 'browsed': 1, 'exclusively': 3, '30yearold': 1, 'men': 66, 'studio': 14, 'apartmentssuzanne': 1, 'somers': 1, 'named': 24, 'thighmaster': 1, 'generaljust': 1, 'wildly': 10, 'hot': 40, 'reminder': 13, 'isaac': 1, 'playing': 26, 'hamleta': 1, 'hbo': 5, 'terror': 21, 'attackwatch': 1, 'stories': 16, 'believe': 52, 'dirtiest': 1, 'jobsgolf': 1, 'sensation': 2, 'spieth': 1, 'masters': 2, 'horrible': 10, 'meltdown': 1, 'danny': 4, 'willett': 1, 'winsfirst': 1, 'gay': 85, 'couple': 69, 'license': 3, 'jailed': 7, 'kentucky': 3, 'clerks': 2, 'officeworlds': 1, 'jews': 4, 'remurdering': 1, 'christjohn': 1, 'mccain': 13, 'czar': 3, 'hater': 2, 'czarfather': 1, 'tower': 6, 'climber': 1, 'also': 27, 'youthat': 1, 'rehabsanders': 1, 'welfare': 8, 'reform': 19, 'tradewhite': 1, 'nationalist': 3, 'denouncement': 1, 'groups': 19, 'kumbaya': 1, 'nonsenseold': 1, 'milwaukee': 4, 'records': 9, 'confirms': 21, 'punches': 2, 'shouldernew': 1, 'details': 13, 'paint': 6, 'unsettling': 4, 'pictures': 9, 'london': 18, 'attackersdont': 1, 'breast': 12, 'cancerbeing': 1, 'wallflower': 1, 'isnt': 43, 'site': 12, 'sayswoody': 1, 'harrelson': 2, 'spends': 40, 'hours': 31, 'drawing': 9, 'marijuana': 20, 'leaf': 1, 'binderburger': 1, 'king': 42, 'franchise': 4, 'owner': 41, 'sad': 22, 'touches': 2, 'restaurantverb': 1, 'follow': 7, 'noun': 1, 'prepositional': 1, 'phrase': 8, 'followa': 1, 'valentine': 4, 'youthe': 4, 'importance': 12, 'collaborativerussian': 1, 'scrambling': 5, 'delegitimize': 1, 'democracy': 12, 'faster': 5, 'intendedobama': 1, 'freaked': 5, 'receiving': 3, 'dayadele': 1, 'tweets': 41, 'apology': 14, 'stage': 25, 'rigging': 1, 'glasgow': 1, 'concertgoerpastor': 1, 'talking': 46, 'nonchristian': 1, 'bloodreligious': 1, 'clauses': 1, 'contention': 1, 'australia': 6, 'crafts': 2, 'equality': 12, 'lawsvoters': 1, 'november': 3, 'loving': 8, 'likely': 29, 'nomineeseveryone': 1, 'compliments': 5, 'grandmother': 9, 'small': 40, 'feeble': 2, 'gotteneating': 1, 'donuts': 5, 'originally': 2, 'evenings': 1, 'planwhite': 1, 'pretty': 68, 'uzbekistan': 1, 'diplomat': 3, 'stole': 7, 'bunch': 15, 'soapjared': 1, 'kushner': 7, 'quietly': 28, 'transfers': 2, 'east': 23, 'todo': 3, 'list15': 1, 'environment': 10, 'constant': 5, 'fails': 31, 'rehabilitate': 1, 'prisonerinternet': 1, 'personality': 5, 'buckley': 1, 'broadwaylegends': 1, 'yorks': 5, 'latex': 2, 'ball': 14, 'voguingman': 1, 'unfortunately': 5, 'sleeps': 4, 'babyone': 1, 'ruben': 1, 'studdard': 1, 'reference': 3, 'wafts': 1, 'gently': 8, 'cool': 21, 'airhubble': 1, 'telescope': 4, 'discovers': 9, 'amelia': 2, 'earhart': 1, 'statue': 11, 'distant': 6, 'planettrump': 1, 'telling': 27, 'jokes': 10, 'nobodys': 3, 'laughinghundreds': 1, 'miniature': 1, 'sean': 14, 'hannitys': 2, 'burst': 3, 'ailes': 3, 'corpsekellie': 1, 'pickler': 1, 'misses': 15, 'buzzer': 1, 'celebrity': 16, 'failmilitary': 1, 'institutes': 3, 'let': 55, 'guess': 5, 'policyjeb': 1, 'insists': 12, 'outsiderwashington': 1, 'journalist': 9, 'iran': 31, 'meal': 16, 'familycoloradans': 1, 'deregistered': 1, 'request': 5, 'voter': 27, 'data': 18, 'signing': 5, 'againwatch': 3, 'vin': 3, 'diesel': 7, 'am': 30, 'groot': 2, 'languagesartists': 1, 'image': 12, 'honors': 15, 'defied': 1, 'neonazi': 4, 'marchvoters': 1, 'hope': 31, 'politicians': 9, 'election': 70, 'cycle': 7, 'twonew': 2, 'resolution': 5, 'colleges': 9, 'normal': 15, 'cubaparis': 1, 'arrest': 12, 'notre': 1, 'dame': 2, 'cylindersmy': 1, '8': 30, 'productsfinancial': 1, 'burden': 5, 'harm': 4, 'quality': 11, 'lifehigh': 1, 'schooler': 3, 'impregnated': 2, 'midnightcnn': 1, 'anchor': 7, 'interviews': 7, 'al': 20, 'jazeera': 4, 'interviewed': 3, 'libyan': 4, 'rebelsthe': 1, 'quadruple': 1, 'bottom': 6, 'comethis': 1, 'holding': 28, 'repairing': 1, 'broken': 11, 'relationshipin': 1, 'obamas': 35, 'mad': 12, 'dash': 2, 'landsthe': 1, 'landscapeburger': 1, 'hellish': 1, 'workthis': 2, 'road': 21, 'sign': 25, 'less': 39, 'helpful': 2, 'looksday': 1, 'chalked': 1, '1015': 1, 'amaustralian': 1, 'lawmaker': 10, 'opponents': 7, 'ad': 46, 'draws': 5, 'ire': 1, 'orlandokinda': 1, 'pan': 4, 'pranks': 2, 'disney': 12, 'worldbrownribboncampaign': 1, 'diversity': 18, 'whitehigh': 1, 'elects': 3, '45yearold': 1, 'homecoming': 1, 'historygeorge': 1, 'thorogood': 1, 'fan': 35, 'disgusted': 5, 'musician': 3, 'licensed': 1, 'bone': 6, 'commercial': 14, 'purposesfor': 1, 'needs': 45, 'diversifydarrelle': 1, 'revis': 1, 'unconsciousdont': 1, 'thismove': 1, 'houseboat': 1, 'regretted': 2, 'daybuick': 1, 'regal': 1, 'vehicle': 6, 'class': 27, 'idling': 2, 'offtrack': 1, 'betting': 1, 'parlorarizona': 1, 'iced': 1, 'tea': 9, '4foottall': 1, 'cansblms': 1, 'alicia': 3, 'garza': 1, 'census': 6, 'mobilize': 2, 'powerpolice': 1, 'audio': 5, 'leaked': 4, 'unarmed': 6, 'christian': 16, 'taylorfor': 1, 'firsttime': 2, 'marathoner': 2, 'theres': 24, 'strength': 9, 'numbershate': 1, 'preachers': 3, 'qatar': 4, 'campus': 5, 'undeserved': 1, 'incitement10thgrade': 1, 'prodigy': 2, 'studying': 1, 'mathematics': 1, '10thgrade': 1, 'level3': 1, 'reminders': 3, 'resilient': 3, 'kidsnation': 1, 'horrified': 12, 'reading': 19, 'petraeus': 4, 'scandaltrump': 2, 'relaxes': 3, 'slipping': 1, 'nice': 23, 'realityarea': 1, 'liberal': 7, 'recognizes': 4, 'fanciful': 1, 'inaccurate': 2, 'mental': 30, 'country': 35, 'inupdate': 1, 'onion': 14, 'suspending': 1, 'production': 10, 'basketball': 11, 'infographic': 1, 'directed': 2, 'brett': 3, 'ratnergeorge': 1, 'foreman': 2, 'grill': 7, 'grillstudy': 1, 'leading': 29, 'indicator': 2, 'crashing': 6, 'downshooting': 1, 'copenhagen': 1, 'synagogue': 3, 'officers': 15, 'woundedparis': 1, 'hilton': 4, 'impersonates': 1, 'kim': 33, 'kanye': 11, 'lineinternational': 1, 'station': 17, 'tented': 1, 'spray': 7, 'xenomorphsexboyfriend': 1, 'hopes': 20, 'terrible': 23, 'incompatible': 1, 'friendsteen': 1, 'accepted': 4, '113': 1, 'ride': 14, 'hbcutransgender': 1, 'danica': 1, 'roem': 1, 'barrier': 7, 'electedshit': 1, 'something': 48, 'obnoxious': 1, 'busweekend': 1, 'encounter': 3, 'coworker': 18, 'acknowledgedstudy': 1, '74': 3, 'contractors': 2, 'accidentally': 22, 'walling': 1, 'themselves': 9, 'constructiongoth': 1, 'scaryass': 1, 'birdhouseoutbreak': 1, 'vavavooms': 1, 'traced': 6, 'miniskirtwearing': 1, 'blondegrandmother': 1, 'tattoos': 2, 'price': 17, 'rightthe': 1, '7th': 2, 'heaven': 7, 'reunites': 4, 'yearsare': 1, 'meeting': 29, 'richthere': 1, 'nothing': 39, 'libertarian': 2, 'conservativesyorkshire': 1, 'terrier': 2, 'monogrammedpope': 1, 'angel': 8, 'flew': 2, 'sistine': 2, 'chapel': 3, 'windowremainder': 1, 'ross': 7, 'ice': 33, 'shelf': 4, 'smithsonian': 2, 'freezerpanicked': 1, 'agriculture': 7, 'momentarily': 2, 'corn': 5, 'introverts': 2, 'throwing': 10, 'solid': 3, 'partyarea': 2, 'client': 3, 'fontfifth': 1, 'harmonys': 1, 'lauren': 3, 'jauregui': 1, 'blasts': 20, 'toxic': 13, 'homophobia': 2, 'poignant': 4, 'exchangeusda': 1, 'salmonsix': 1, 'teenagers': 5, 'britain': 4, 'suspected': 4, 'polish': 3, 'crimecubans': 1, 'dictator': 2, 'doing': 59, 'wrongobama': 1, 'fills': 6, 'lukewarm': 1, 'glassdoor': 1, 'exiting': 2, 'presidencybill': 1, 'henderson': 3, 'vocalist': 1, '90very': 1, 'specific': 2, 'food': 64, 'pyramid': 3, 'shrimp': 9, 'scampis': 1, 'per': 19, 'yearalaskan': 1, 'gray': 5, 'wolf': 12, 'snow': 11, 'nosehow': 1, 'emotionally': 5, 'electionman': 1, 'friends6': 1, 'trashfree': 1, 'lunchcandidate': 1, 'delighted': 4, 'factorywoman': 1, 'exactly': 16, 'knife': 8, 'shed': 11, 'grab': 5, 'cutlery': 1, 'drawer': 5, 'event': 16, 'invasionbored': 1, 'predator': 4, 'pumps': 4, 'rounds': 3, 'mountain': 7, 'goat30yearold': 1, 'nes': 1, 'wasting': 6, 'gamesschools': 1, 'enact': 1, 'drama': 7, 'therapyworld': 1, 'map': 9, 'rearranged': 1, 'geography': 3, 'americansnations': 1, 'alphabeticallywoman': 1, 'competentprofessor': 1, 'threatened': 17, 'firing': 8, 'wheaton': 1, 'rulesjfk': 1, 'jr': 29, '10000th': 2, 'couplingman': 1, 'codependent': 1, 'relationshipinternet': 1, 'popup': 2, 'quiz': 6, 'insultingsuspension': 1, 'disbelief': 1, 'unrewardedthe': 1, 'annotationsancient': 1, 'beast': 4, 'avatar': 2, 'creaturetennessee': 1, 'passes': 26, 'erect': 2, 'memorial': 14, 'abortionreport': 1, 'castle': 6, 'friends45yearold': 1, 'understand': 9, 'youth': 16, 'voteword': 1, 'millennials': 8, 'forced': 58, 'headline': 5, 'pageviewsdozens': 1, 'thankful': 11, 'toetotoe': 1, 'muhammad': 10, 'alifox': 1, 'struggling': 29, 'attract': 3, '6075': 1, 'demographicafter': 1, 'string': 9, 'accidents': 3, 'uhaul': 3, 'announces': 87, 'closure': 1, 'aircraft': 2, 'divisionwhy': 1, 'stakes': 2, 'bother': 7, 'tearsgrowing': 1, 'holocaust': 5, 'writerthe': 1, 'surprising': 10, 'horses': 4, 'ease': 5, 'alzheimers': 7, 'symptomsbeaver': 1, 'wait': 35, 'damhuffpollster': 1, 'org': 1, 'chartmorbidly': 1, 'obese': 6, 'enjoys': 14, 'disabled': 8, 'privileges': 2, 'motorized': 2, 'cartno': 1, 'jeb': 11, 'softly': 1, 'whispers': 9, 'sleeping': 8, 'grandsonmadcap': 1, 'romp': 2, 'escalates': 2, 'zany': 1, 'hijinks5': 1, 'longevity': 2, 'secrets': 7, 'healthiest': 3, 'cultures8': 1, 'perks': 1, 'divorced': 6, 'holidaysnation': 1, 'awesome': 15, 'highlightwhy': 1, 'moms': 38, 'demand': 28, 'action': 20, 'participate': 4, 'womanhillary': 1, 'asian': 12, 'outreach': 2, 'director': 36, 'campaigninterview': 1, 'louise': 1, 'munson': 1, 'playwright': 1, 'luigisuspect': 1, 'explosives': 2, 'areawriter': 1, 'divest': 2, 'lena': 6, 'dunham': 9, 'controversyarea': 1, 'hdthe': 1, 'overeatingdon': 1, 'lemon': 5, 'spicer': 13, 'dumber': 2, 'listened': 1, 'thatkansas': 1, 'refused': 5, 'investigate': 7, 'assaults': 4, 'offcampus': 1, 'saysnfl': 1, 'avery': 3, 'williamson': 1, 'wears': 11, '911': 27, 'cleats': 1, 'finetrump': 1, 'confidant': 2, 'floats': 3, 'rbgformerrickgarland': 1, 'scotus': 3, 'swapus': 1, 'continues': 15, 'proud': 22, 'tradition': 4, 'linessouth': 1, 'korean': 17, 'north': 66, 'koreas': 5, 'jong': 12, 'un': 26, 'talk': 56, 'summitnations': 1, 'advisors': 3, 'recommend': 15, 'capturing': 2, 'creature': 3, 'wishesmichelle': 1, 'carson': 14, 'wild': 18, 'rupauls': 3, 'racecontent': 1, 'awkwardly': 6, 'website': 17, 'workspossible': 1, 'viking': 2, 'rewrite': 3, 'historynew': 2, 'emerge': 4, 'murder': 28, 'snared': 1, 'attorney': 13, 'highway': 10, 'patrolmenwatching': 1, 'faces': 25, 'highlight': 4, 'english': 10, 'yearno': 1, 'jared': 6, 'fogle': 2, 'mealsean': 1, 'champion': 7, 'amendmentuncovered': 1, 'community': 27, 'servicesdebra': 1, 'messing': 3, 'freak': 13, 'grace': 9, 'updatenation': 1, 'famous': 15, 'togetherarea': 2, 'winded': 1, 'particularly': 3, 'lengthy': 3, 'wendys': 3, 'orderthe': 1, '6': 43, 'reasons': 41, 'marriedbiden': 1, 'busted': 5, 'parking': 19, 'selling': 13, 'bootleg': 1, 'tshirtswhite': 1, 'person': 38, 'waved': 1, 'beeping': 1, 'walgreens': 3, 'barrierkhloe': 1, 'cocaine': 4, 'kylie': 10, 'jenners': 7, 'graduation': 8, 'partygenuine': 1, 'seen': 27, 'showschewbacca': 1, 'himself': 27, 'chewbacca': 4, 'maskyet': 1, 'pick': 25, 'habit': 4, 'spreading': 1, 'dangerous': 21, 'theoriesdog': 1, 'absolute': 4, 'hardestmississippi': 1, 'bans': 9, 'soft': 5, 'smaller': 8, '20': 49, 'ouncesexplosion': 1, 'fedex': 3, 'facility': 3, 'san': 20, 'antonio': 1, 'linked': 11, 'austin': 8, 'bombings': 4, 'fbi': 27, 'sayscupid': 1, 'cop': 31, 'gave': 27, 'roses': 2, 'cards': 10, 'instead': 14, 'ticketsfox': 1, 'host': 32, 'disavows': 1, 'internment': 1, 'camps': 3, 'panelists': 1, 'rounding': 1, 'muslimsreince': 1, 'priebus': 7, 'trumpminnesota': 1, 'caf': 2, 'charges': 12, '35': 8, 'cent': 5, 'fee': 3, 'minimum': 10, 'wage': 8, 'hikeasshole': 1, 'greeted': 2, 'baggage': 2, 'claim': 18, 'familyan': 1, 'dreamer': 1, 'trumphillary': 1, 'flags': 6, 'removal': 3, 'mlk': 3, 'ceremonydavid': 1, 'brooks': 6, 'manhood': 3, 'eastwhy': 1, 'terrorists': 10, 'useveryone': 1, 'grandma': 12, 'staying': 13, 'awakethe': 1, 'nypd': 3, 'spying': 3, 'cell': 6, 'phones': 4, '2008republicans': 1, 'urge': 15, 'crack': 10, 'sanctuary': 7, 'citiesfloor': 1, '90': 15, 'defibrillator': 3, 'locationsvoters': 1, '14': 15, 'states': 41, 'navigating': 1, 'ballotsarne': 1, 'duncan': 3, 'stressed': 5, 'preparing': 7, 'standardized': 2, 'examtom': 1, 'brokaw': 1, 'touched': 7, 'defend': 18, 'filthy': 4, 'pervert': 1, 'himselfstop': 1, 'complaining': 3, 'evolution': 9, 'text': 13, 'language': 9, 'periodhalfdressed': 1, 'frantically': 13, 'scrambles': 2, 'hearing': 18, 'toyotathon': 1, 'deals': 7, 'longtech': 1, 'dadtrump': 1, 'fulfills': 4, 'promise': 11, 'immigration': 25, 'someone': 38, 'else': 15, 'tvcommunication': 1, 'matters': 10, 'outson': 1, 'surprised': 19, 'johnny': 7, 'songman': 1, 'suddenly': 24, 'regretting': 4, 'taken': 23, 'seriously': 11, 'peerselevenyearold': 1, 'miniskirt': 1, 'vague': 2, 'notion': 3, 'iscinemax': 1, 'award': 20, 'skinematographydoctor': 1, 'excitement': 2, 'ultrarare': 1, 'diseasedozens': 1, 'trains': 4, 'collide': 2, 'pennsylvaniaed': 1, 'sheeran': 3, 'sang': 3, 'chasing': 4, 'cars': 13, 'wedding': 47, 'swooninggame': 1, 'moved': 9, 'beyond': 8, 'written': 11, 'script': 5, 'halfway': 8, 'through': 114, 'current': 16, 'seasongirls': 1, 'theyll': 11, 'gross': 9, 'inun': 1, 'rebukes': 1, 'jerusalem': 6, 'overwhelming': 3, 'votefathers': 1, 'manwhite': 1, 'geologic': 1, 'era': 10, 'controlprofessors': 1, 'try': 27, 'bae': 1, 'fleek': 2, 'meanhouse': 1, 'seeks': 12, 'block': 18, 'carbon': 6, 'rulesfred': 1, 'durst': 2, 'paints': 5, 'limp': 2, 'bizkit': 1, 'bridgedemocrats': 1, 'poll': 21, 'showing': 9, 'aheadbored': 1, 'jongun': 8, 'stacks': 1, 'populace': 8, 'timehow': 2, 'humans': 23, 'laying': 5, 'welcome': 14, 'mat': 3, 'mosquitoes': 2, 'diseases': 5, 'carryjennifer': 1, 'lawrence': 4, 'honored': 12, 'robert': 19, 'glaad': 1, 'awards': 19, 'how11': 1, 'grapefruitstudy': 1, 'meowing': 1, 'owners': 16, 'cats': 7, 'meow': 2, 'back14': 1, 'bond': 5, 'daughtersstar': 1, 'jedi': 8, 'pluralstudy': 1, 'headaches': 1, 'bodys': 1, 'communicating': 1, 'pillshow': 1, 'jimmy': 14, 'carter': 10, 'learned': 37, 'rosalynn': 1, 'partnergop': 1, 'complains': 5, 'repealreligious': 1, 'leaders': 34, 'appalled': 4, 'orderslice': 1, 'blast': 8, 'wigs': 2, 'costume': 12, 'shop10': 1, 'places': 20, 'gone': 10, 'agochicagos': 1, 'laquan': 4, 'mcdonaldbig': 1, 'eyes': 20, 'feminist': 11, 'youve': 12, 'metbloomberg': 1, 'gadfly': 1, 'shakeup': 1, 'commentary': 1, 'spaceharry': 1, 'reid': 4, 'acting': 15, 'puppets': 1, 'nralittle': 1, 'tobacco': 4, 'hundred': 1, 'lawsuitwomen': 1, 'business': 57, 'qa': 8, 'rebecca': 1, 'randstad': 1, 'professional': 8, 'solutionsmike': 1, 'pence': 33, '200000': 4, 'husbands': 10, 'fathers': 25, 'permitted': 1, 'marchjeff': 1, 'searing': 1, 'critical': 8, 'applause': 2, 'unionmichelle': 1, 'explains': 30, 'uncertain': 4, 'terms': 6, 'officerepublican': 1, 'stamp': 5, 'cut': 27, 'benefits': 10, 'size': 7, 'governmentrussians': 1, 'mint': 3, 'coin': 1, 'inaugurationkristen': 1, 'wiigs': 1, 'kimmel': 12, 'moviepatients': 1, 'limited': 2, 'erdear': 1, 'boomers': 7, 'step': 18, 'asidedont': 1, 'spend': 32, 'bitcoin': 4, 'olivers': 1, 'cryptocurrency': 1, 'warningcelebrities': 1, 'mourn': 5, 'anne': 7, 'meara': 1, 'deathheres': 1, 'enrollment': 2, 'yearrelationship': 1, 'experts': 10, 'bathing': 5, 'suitor': 2, 'glimpses': 1, 'treesmarried': 1, 'longs': 3, 'resented': 2, 'anothergroup': 1, 'chanting': 6, 'shots': 8, 'compelling': 2, 'pointblind': 1, 'date': 25, 'pronounces': 2, 'syllable': 1, 'comfortableto': 1, 'againthe': 6, 'sciencebacked': 2, 'reason': 29, 'therapist': 5, 'morningturkish': 1, 'engage': 3, 'controlclaire': 1, 'danes': 3, 'expecting': 7, 'number': 25, 'hugh': 3, 'dancyvlogger': 1, 'shamed': 3, 'walmart': 3, 'fitting': 1, 'stretch': 2, 'clothesat': 1, 'reverberate': 1, 'presenttrump': 1, 'hints': 8, 'replacement': 5, 'mindbadass': 1, 'engagement': 11, 'temperaturedirector': 1, 'paul': 45, 'feig': 3, 'weinstein': 15, 'allegationsadvertising': 1, 'firm': 7, 'muteresistant': 1, 'commercialsrookie': 1, 'justice': 29, 'gorsuch': 6, 'assigned': 3, 'overnight': 3, 'shiftmike': 1, 'oath': 7, 'countrys': 7, 'vice': 15, 'presidentrex': 1, 'tillerson': 9, 'blindsided': 3, 'worked': 12, 'departmentfight': 1, 'treasury': 4, 'underscores': 2, 'battles': 3, 'within': 18, 'partyrommel': 1, 'hummel': 1, 'dominate': 2, 'lista': 1, 'myth': 3, 'spot': 23, 'narcissistdollhousing': 1, 'worsen': 2, 'older': 22, 'saysslushing': 1, 'ski': 4, 'seasonhere': 1, 'candidates': 30, 'think': 71, 'novembermissing': 1, 'maryland': 2, 'toddlers': 2, 'body': 49, 'ohio': 15, 'creekterrified': 1, 'fade': 1, 'visible': 6, 'spectrumadele': 1, 'titanicthemed': 1, '30th': 3, 'birthdaychanging': 1, 'weather': 13, 'inspires': 15, 'conversationalistfourth': 1, 'tool': 3, 'discoveredtexas': 2, 'districts': 1, 'trash': 14, 'leftover': 2, 'foodhentai': 1, 'board': 26, 'surprisingly': 6, 'closeknit': 2, 'supportive': 1, 'communityhootie': 1, 'blowfish': 1, 'racial': 10, 'barriers': 1, 'pussiesstudy': 1, 'snapping': 2, 'recall': 7, 'actorsdonald': 1, 'pardon': 6, 'russia': 36, 'persistssyria': 1, 'ceasefire': 7, 'backed': 4, 'turkey': 11, 'initial': 3, 'clashesfrench': 1, 'parliament': 5, 'debates': 6, 'deep': 23, 'sleep': 32, 'lifereality': 1, 'reward': 3, 'tv': 58, 'ripe': 2, 'authenticity': 1, 'autonomytranscript': 1, 'emails': 6, 'tabloid': 1, 'reporters': 5, 'harvey': 21, 'dirt': 2, 'womentrumps': 1, 'americaa': 2, 'shining': 2, 'hillphotographer': 1, 'assumes': 1, 'endless': 7, 'identities': 2, 'clothesstripper': 1, 'talked': 2, 'homicide': 5, 'detectives': 2, 'todayjustin': 1, 'jealous': 2, 'frankelevate': 1, 'leadership': 9, '2016womans': 1, 'facedown': 1, 'dummy': 2, 'repeated': 6, 'calls259': 1, 'objects': 7, 'gummi': 1, 'formkremlin': 1, 'agent': 16, 'even': 98, 'compromise': 4, 'staffer': 5, 'resign': 6, 'monthsthe': 1, 'blurryheres': 1, 'pgrated': 1, 'evangelical': 3, 'christiansdonald': 1, 'unqualified': 2, 'majority': 46, 'saybody': 1, 'given': 34, 'false': 10, 'piece': 15, 'fruit': 10, '9': 19, 'daysmissouri': 1, 'filibuster': 4, '39': 3, 'antigay': 8, 'bill9yearold': 1, 'reporter': 23, 'cute': 13, 'landed': 4, 'dealthis': 1, 'missed': 12, 'connection': 10, 'delightfully': 2, 'feministafrican': 1, 'wartornthree': 1, 'tourists': 6, 'six': 16, 'borneo': 1, 'shipwreckgod': 1, 'successful': 12, 'category': 3, 'hurricanelone': 1, 'geek': 3, 'sits': 4, 'silmarillion': 1, 'recessthis': 1, 'happens': 20, 'arya': 2, 'stark': 3, 'write': 15, 'yearbook': 2, 'quotealex': 1, 'jones': 29, 'pleads': 8, 'sandy': 6, 'hook': 7, 'imagine': 5, 'pain': 8, 'expensive': 8, 'himrescue': 1, 'animals': 16, 'thanks': 20, 'online': 32, 'donationsgoldfish': 1, 'stand': 31, 'bowlmatejohn': 1, 'legend': 16, 'speaks': 20, 'caused': 9, 'mass': 31, 'incarcerationpolice': 1, 'shoving': 2, 'wheelchair': 7, 'streetnasa': 1, 'located': 3, 'capabilitiesjudge': 1, 'asked': 25, 'knees': 2, 'together': 34, 'resignswhite': 1, 'prepares': 20, '15': 31, 'cuts': 15, 'packagegop': 1, 'heads': 15, 'south': 34, 'known': 11, 'dirty': 12, 'tricks': 6, 'brawlskindergartener': 1, 'allegedly': 24, 'barred': 4, 'momsgeorge': 1, 'trashes': 2, 'oreilly': 8, 'wise': 6, 'notohio': 1, 'decide': 10, 'legalizing': 1, 'marijuanathe': 1, 'ifttt': 1, 'vacationlast': 1, 'raisin': 2, 'prostate': 1, 'cancermicrosoft': 1, 'surface': 7, 'laptop': 4, 'plus': 4, 'gizmosfda': 1, 'psilocybin': 1, 'reduces': 6, 'mindlessly': 1, 'societys': 1, 'lemmingfourth': 1, 'legionnaires': 2, 'outbreaknational': 1, 'hurricanes': 5, 'nameswilliam': 1, 'h': 2, 'macy': 1, 'ultimate': 13, 'promwarm': 1, 'temperatures': 3, 'winter': 19, 'gearsarah': 1, 'flatly': 2, 'rejects': 8, 'jim': 14, 'acostas': 1, 'assertion': 1, 'acostareno': 1, 'corruptionbillions': 1, 'signals': 3, 'neurons': 1, 'allow': 19, 'brain': 12, 'imperioli': 1, 'likeunapologetic': 1, 'selfportraits': 1, 'shatter': 2, 'perceptions': 2, 'disabilityscientists': 1, 'discover': 38, 'serves': 5, 'function': 2, 'easing': 1, 'anxietylibraries': 1, 'burning': 14, 'sarajevo': 1, 'mosulwhat': 1, 'realized': 6, 'searching': 13, 'soulmateastoria': 1, 'characters': 13, 'charity': 9, 'stagerdolby': 1, 'theatre': 2, 'usher': 5, 'throws': 31, 'attempting': 11, 'camcorder600pound': 1, 'butter': 6, 'cow': 3, 'caucusbill': 1, 'tearfully': 8, 'packs': 5, 'framed': 2, 'upskirt': 1, 'desktrumps': 1, 'victim': 14, 'gestapo': 1, 'wing': 3, 'activists': 17, 'claimcandidates': 1, 'conditions': 10, 'highaltitude': 1, 'drillsthe': 1, 'desks': 2, 'mms': 2, 'moremagazine': 1, 'editor': 7, 'undergoes': 3, 'sleek': 1, 'redesigntown': 1, 'nervously': 8, 'welcomes': 9, 'veteran': 15, 'homelocal': 1, 'student': 55, 'poetits': 1, 'racy': 2, 'theatersariana': 1, 'grande': 8, 'performs': 4, 'break': 35, 'snlunconditional': 1, '15yearold': 2, 'called': 32, 'bitch': 2, 'hollisternewborn': 1, 'prince': 22, 'cambridge': 6, 'consolidating': 1, 'imprisoned': 2, 'londoncongress': 1, 'abandons': 2, 'wikiconstitution2016': 1, 'electorateselfdefense': 1, 'instructor': 7, 'keeps': 34, 'himselfadorable': 1, 'puppy': 9, 'nets': 2, 'handjob10': 1, 'whine': 1, 'aboutman': 1, 'bitten': 1, 'radioactive': 1, 'sloth': 1, 'lyingaroundallday': 1, 'menquake': 1, '500': 6, 'hoursteen': 1, 'recruit': 4, 'statement': 6, 'training': 16, 'camp10': 1, 'illinois': 5, 'safest': 1, 'citiesunconsciousness': 1, 'faked': 3, 'anesthesiologist': 1, 'feel': 42, 'bettertrump': 1, 'hails': 3, 'fierce': 3, 'protector': 1, 'amendment': 8, 'temporarily': 5, 'suspend': 3, 'assemblehistorical': 1, 'archives': 25, 'nearby': 5, 'childbirthprison': 1, 'warden': 1, 'el': 3, 'chapos': 1, 'tunnel': 3, 'capturedmothers': 1, 'precious': 3, 'misunderstood': 1, 'metnew': 1, 'tapes': 1, 'prepare': 8, 'infants': 2, 'entertainment': 7, 'law5': 1, 'shy': 3, 'sociablewho': 1, '911california': 1, 'businesses': 9, 'insurerwhat': 1, 'infected': 2, 'measles': 7, 'chartlabor': 1, 'letting': 12, 'unemployed': 3, 'feetrepublicans': 1, 'doubts': 2, 'proposalpope': 1, 'beatifies': 1, 'sainthoodpresence': 1, 'round': 10, 'triggers': 2, 'juggling': 2, 'reflex': 1, 'manthe': 2, 'lgbt': 22, 'question': 23, 'ben': 15, 'speechlessyoull': 1, 'inked': 1, 'seeing': 27, 'masterful': 1, 'workmany': 1, 'developing': 10, 'tools': 5, 'governingmike': 1, 'charlottesvillenew': 1, 'sharks': 3, 'angry': 14, 'dolphinshigh': 1, 'nonbinding': 1, 'resolutiontwitter': 1, 'dismissal': 2, 'alleging': 1, 'federal': 36, 'judgeteacher': 1, 'shares': 33, 'boob': 2, 'studentsreport': 1, 'types': 6, 'crawl': 4, 'sleepyoung': 1, 'prefer': 8, 'findsnation': 1, 'canadian': 6, 'politicsalmost': 1, 'enroll': 3, 'sabotage': 4, 'attemptswould': 1, 'goldman': 6, 'sachs': 8, 'speechthe': 1, 'chance': 24, 'moore': 28, 'lie': 12, 'low14': 1, 'snapshots': 1, 'summed': 2, 'parenthood': 16, '2014ryan': 1, 'seacrest': 3, 'sells': 9, 'squad': 7, 'goals': 4, 'cbspope': 1, 'weptgop': 1, 'bob': 16, 'corker': 3, 'demonstrated': 1, 'stability': 2, 'competencebiden': 1, 'butterfly': 4, 'speechnew': 2, 'provides': 9, 'terrifying': 11, 'glimpse': 11, 'regime': 3, 'placing': 1, 'collection': 6, 'teas': 1, 'draweryou': 1, 'yourself': 16, 'firstquick': 1, 'painless': 1, 'tops': 10, 'veal': 2, 'calftrump': 1, 'dish': 3, 'steakhouse': 3, 'nyt': 2, 'listens': 2, 'innow': 1, 'shitty': 9, 'charts4': 1, 'scrolling': 1, 'bed': 14, 'referred': 7, 'winding': 3, 'downthis': 1, 'proposal': 7, 'boyfriend': 13, 'musical': 13, 'believereport': 1, 'selves': 6, 'samesex': 17, 'couplesupreme': 1, 'understudy': 2, 'scalia83rd': 1, 'definition': 5, 'partybecause': 1, 'legislators': 3, 'healthfighting': 1, 'monsterssurprise': 1, 'christopher': 7, 'nolan': 1, 'netflixlinmanuel': 1, 'raffling': 1, 'hamilton': 8, 'immigrantsseeing': 1, 'blows': 11, 'steam': 4, 'parkamanda': 1, 'seyfried': 1, 'childcollege': 1, 'freshman': 9, 'loves': 25, 'itrefugee': 1, 'blueslanthanum': 1, 'quits': 6, 'periodic': 3, 'table': 16, 'elements7': 1, 'podcast': 4, 'seasons': 7, 'bingelistening': 1, 'pleasurewatch': 1, 'chef': 8, 'voltaggio': 1, 'onestar': 1, 'yelp': 5, 'reviewsfrustrated': 1, 'restaurant': 30, 'burgera': 1, 'triple': 6, 'amputees': 1, 'togetherrands': 1, 'twoferthese': 1, 'arab': 6, 'cartoonists': 3, 'expression': 3, 'daysome': 1, 'sense': 17, 'knocked': 5, 'girlfriends': 10, 'sonbiologists': 1, 'evolutionary': 3, 'ugly': 8, 'peoplepolicewoman': 1, 'flashed': 1, 'badge': 1, 'conference': 10, 'reporthot': 1, 'craze': 1, 'sweeps': 6, 'chicagoruth': 1, 'bader': 3, 'ginsburg': 3, 'cooler': 7, 'prevail': 2, 'vacancyelementary': 1, 'swearbps': 1, 'fines': 5, 'gulf': 4, 'thoughttwo': 1, 'hipsters': 1, 'angrily': 2, 'hipsterman': 1, 'fatally': 7, 'st': 28, 'county': 8, 'policelara': 1, 'boyles': 1, 'publicist': 3, 'warns': 47, 'interviewer': 3, 'upfrontriverboat': 1, 'horseracing': 1, 'utterlyreagan': 1, '5000ahead': 1, 'funeralshe': 1, 'survived': 4, 'relive': 1, 'lifeus': 1, 'leads': 27, 'mexicanfood': 1, 'availabilityreport': 1, 'peaceful': 2, 'pressing': 4, 'issue': 18, 'electionus': 1, 'upset': 13, 'aliens': 5, 'land': 15, 'italy27': 1, 'vegan': 2, 'feastapple': 1, 'ipad': 1, 'extra': 19, 'storage': 4, 'drawerthese': 1, '360degree': 3, 'vacationteacher': 1, 'frustrated': 3, 'beginner': 2, 'yoga': 12, 'focus': 20, 'chakras': 1, 'energy': 21, 'blastparents': 1, 'godmother': 2, 'uncomfortable': 8, 'arrangementarea': 1, 'slams': 30, 'waits': 5, 'ringbritney': 1, 'spears': 11, 'receive': 13, 'prestigious': 1, 'supportlawabiding': 1, 'citizen': 6, 'herself': 9, 'weekly': 3, 'cheat': 3, 'dayf': 1, 'fitzgerald': 3, 'estate': 9, 'wondering': 19, 'knocking': 5, 'yetconservatives': 1, 'users': 34, 'filter': 3, 'rainbow': 8, 'picsaviva': 1, 'sees': 22, 'lightruby': 1, 'tuesday': 7, 'waiter': 4, 'jill': 5, 'stein': 1, 'disrupting': 1, 'dinersstressedout': 1, 'hannity': 9, 'cabins': 1, 'maine': 5, 'allnew': 2, 'link': 18, 'jeanette': 1, 'beautifulnations': 1, 'cutting': 5, 'strobe': 1, 'light': 27, 'bedtimeisrael': 1, 'terrorizes': 1, 'palestinians': 5, 'gazathis': 1, 'retirementlunar': 1, 'olympic': 23, 'continue': 18, 'search': 24, 'vaulterbernie': 1, 'mike': 24, 'lee': 14, 'saudis': 1, 'themboss': 1, 'handle': 9, 'yearsof': 1, 'busy': 14, 'bartender': 2, 'mind': 21, 'friendsdo': 1, 'income': 8, 'taxes': 9, 'benefitsthe': 1, 'hilarious': 23, 'hipster': 3, 'classifieds': 1, 'onlinesecondgrade': 1, 'nuts': 5, 'cowbelltemporary': 1, 'worker': 18, 'permanently': 3, 'scarredantarctic': 1, 'ideasarea': 1, 'metallica': 4, 'tour': 25, 'museumcongressman': 1, 'raw': 4, 'dealactivists': 1, 'militants': 5, 'regimeheld': 1, 'areathe': 1, 'fate': 9, 'brutality': 4, 'kenyagod': 1, 'tip': 7, 'finger': 7, 'hole': 6, 'accidentgary': 1, 'richrath': 1, 'guitarist': 2, 'songwriter': 2, 'reo': 1, 'speedwagon': 1, '6515': 1, 'songs': 10, 'defined': 1, 'boomer': 4, 'generationconvict': 1, 'generating': 2, '80000': 1, '100000': 7, 'profits': 6, 'prisonbookworms': 1, '7yearold': 9, 'wrote': 8, 'anthem': 9, 'youmetallica': 1, 'directors': 8, 'riff': 2, 'negative': 9, 'impact': 12, 'shareholder': 1, 'valuebig': 1, 'bird': 12, 'beastie': 1, 'mashup': 2, 'streetthis': 1, 'unorthodox': 1, 'guided': 3, 'meditation': 12, 'habitthis': 1, 'unsafe': 2, '4th': 4, 'yearsroy': 1, 'withdraw': 2, 'alabama': 13, '13yearoldtop': 1, 'grew': 2, 'facebookastronomers': 1, '300th': 1, 'discovering': 5, 'skykatie': 1, 'ledecky': 1, '18yearold': 3, 'swimming': 7, 'recordtwitter': 1, 'tire': 3, 'conor': 2, 'mcgregors': 1, 'staminahistorical': 1, 'immoral': 3, 'woodcut': 2, 'discovered': 18, 'hay': 2, 'loftevil': 1, 'genius': 11, 'cat': 32, 'subpoenaedplan': 1, 'voicedwatch': 1, 'heroic': 2, 'rescue': 12, 'scared': 6, 'dashing': 1, 'freewayofficials': 1, 'investigating': 8, 'hefners': 1, 'suspect': 25, 'foreplaycompletely': 1, 'unrealistic': 1, 'character': 18, 'complex': 6, 'multifaceted': 1, 'personalitynew': 1, 'bin': 13, 'laden': 12, 'tape': 8, 'previously': 16, 'unreleased': 2, 'monologueschinas': 1, 'sexiest': 6, 'panda': 8, 'obliterates': 1, 'latest': 44, 'rompwomen': 1, 'laura': 2, 'tenison': 1, 'founder': 6, 'managing': 4, 'jojo': 2, 'maman': 1, 'bbsir': 1, 'mixalot': 1, 'wasnt': 11, 'backchurchgoing': 1, 'widows': 1, 'hotnaked': 1, 'whiskey': 3, 'nsfwpatton': 1, 'oswalt': 3, 'fking': 5, 'hillaryball': 1, 'franks': 1, 'footwide': 1, 'hotdogsariana': 1, 'donut': 2, 'donutlickingcuba': 1, 'cararmy': 1, 'influenced': 3, 'bergdahl': 2, 'casethe': 3, 'chew': 1, 'cohost': 1, 'mothersalqaeda': 1, 'marching': 3, 'band': 25, 'join': 10, 'macys': 3, 'parade': 9, 'auditioncongress': 1, 'medicrueltytaco': 1, 'bell': 11, 'fast': 22, 'foodpier': 1, 'formal': 4, 'rattan': 1, 'marchindian': 1, 'sweatshop': 2, 'tooloser': 1, 'pregnantwoman': 1, 'ninth': 2, 'easythings': 1, 'apart': 7, 'easily': 5, 'asghar': 1, 'farhadis': 1, 'ellyowner': 1, 'fuzzy': 3, 'objectauthorities': 1, 'radicalized': 1, 'serving': 5, 'congressgross': 1, 'product': 13, 'surpassed': 1, 'byproductsmaintaining': 1, 'abramson': 2, 'videophilip': 1, 'morris': 7, 'marlboro': 1, 'sinus': 2, 'cigarettesman': 1, '6figure': 1, 'tumblr': 1, 'nerve': 3, 'appear': 13, 'television2015': 1, 'mix': 3, 'experimental': 2, 'festival': 18, 'nycexecutioner': 1, 'enters': 24, 'lethal': 6, 'injection': 7, 'bag': 21, 'depotreport': 1, 'quickly': 13, 'violence': 42, 'spill': 7, 'to8': 1, 'snowvacationing': 1, 'controlemails': 1, 'richard': 11, 'spencer': 3, 'bounced': 4, '10565': 1, 'florida': 25, 'event10': 1, 'sustainable': 7, 'etsy': 1, 'stores': 5, 'supportboarding': 1, 'williamprince': 1, 'charles': 7, 'hear': 25, 'motherfamily': 1, 'lowincome': 4, 'housingplastic': 1, 'treeobama': 1, 'biden': 29, 'tammy': 1, 'duckworth': 3, 'senatea': 1, 'fears': 9, 'dawn': 3, 'era10': 1, 'explodeheres': 1, 'selfies': 2, 'pythonsjim': 1, 'morrison': 4, 'stares': 13, 'creepily': 1, 'windowhillary': 1, 'sends': 21, 'pizza': 18, 'camping': 2, 'signing7': 1, 'micethe': 1, 'coparenting': 1, 'holidaysmc': 1, 'serch': 1, 'updates': 12, 'gasface': 1, 'recipientsdear': 1, 'edwards': 5, 'scholarship': 5, 'funddepartment': 1, 'defense': 21, '83': 4, 'shootspicassos': 1, 'portrait': 9, 'pubescent': 2, 'girl': 53, '115': 2, 'backdrop': 1, 'toojimmy': 1, 'soften': 1, 'daca': 7, 'babyoneyearold': 1, 'wordspoll': 1, '98': 7, 'rundown': 2, 'strip': 13, 'mall': 11, 'saidgw': 1, 'governments': 5, 'choice': 11, 'worship': 4, 'notdemoted': 1, 'unsure': 12, 'desk': 12, 'considered': 7, 'punishmentrescued': 1, 'welcomefda': 1, 'femalelibidoenhancing': 1, 'manliability': 1, 'waiver': 1, 'lowered': 4, 'mine': 4, 'shaftspanx': 1, 'shapewear': 1, 'hood': 4, 'smooth': 3, 'unsightly': 2, 'headsa': 1, 'familythe': 3, 'barrel': 3, 'outmitch': 1, 'mcconnell': 18, 'zika': 8, 'legislation': 11, 'cleansatan': 1, 'revise': 2, 'bar': 27, 'systemmartin': 1, 'omalley': 6, 'ohios': 3, 'primary': 18, 'ballotbernie': 1, 'took': 18, 'walk': 21, 'freddie': 3, 'grays': 2, 'neighborhood': 9, 'heres': 39, 'residents': 25, 'ittea': 1, 'movement': 18, 'hopelessly': 2, 'divided': 5, 'enraged': 3, 'apoplectic': 1, 'factionsnbc': 1, 'relocates': 2, 'nightly': 5, 'nightus': 1, 'expectancy': 1, 'falls': 21, 'die': 23, 'illnessesa': 1, 'nobel': 8, 'fashionbin': 1, 'returns': 32, 'seareport': 1, '5000': 7, 'calories': 3, 'hourcat': 1, 'grumpy': 1, 'today': 16, 'dying': 23, 'esophageal': 1, 'cancermelania': 1, 'mocked': 6, 'teach': 16, 'responsible': 9, 'digital': 13, 'citizens': 19, 'tweetlea': 1, 'delaria': 2, 'butch': 1, 'hard': 40, 'workmckinney': 1, 'involved': 6, 'incident': 4, 'resignssuspected': 1, 'bomber': 9, 'confrontation': 3, 'policeally': 1, 'sheedy': 1, '80s': 6, 'basket': 2, 'case6': 1, 'jets': 2, 'debaclelearned': 1, 'sage': 1, 'points': 19, 'powerball': 4, 'taxesgod': 1, 'decisionmaking': 1, 'coinallison': 1, 'schmitt': 1, 'depression': 19, 'hold': 22, 'backbankrupt': 1, 'r': 5, 'euthanize': 2, 'hatchimalsconcept': 1, 'designers': 6, 'doors': 6, 'open7': 1, 'intentionsfor': 1, 'jet': 8, 'planebrian': 1, 'boitano': 1, 'sobs': 1, 'darkjune': 1, 'weddings': 2, 'dayhow': 4, 'grocery': 5, 'lasting': 8, 'beautyparenting': 1, 'viralconnecticut': 1, 'newtown': 2, 'earned': 3, 'supporteditors': 1, '401': 1, 'soups': 1, 'cookbook': 1, 'fightingwith': 1, 'presidency': 10, 'hanging': 6, 'balance': 5, 'latino': 12, 'turnoutobama': 1, 'studentsrepublicans': 1, 'controlshe': 1, 'dropped': 12, 'waitingwedding': 1, 'invitation': 6, 'includes': 4, 'depressing': 8, 'churchpolice': 1, 'seek': 19, 'random': 5, 'hostingsfinding': 1, 'threadjennifer': 1, 'billboard': 7, 'gown': 2, 'seethrough': 1, 'obviserena': 1, 'olympics': 13, 'thirdround': 1, 'lossmichael': 1, 'bloombergs': 1, 'ambitions': 4, 'shake': 11, 'companyclimate': 1, 'newest': 9, 'prescription': 9, 'children': 59, 'outdoorsselena': 1, 'gomez': 8, 'cordens': 4, 'rollercoaster': 1, 'karaoke': 4, 'quite': 13, 'ridewrinklefree': 1, 'pants': 11, 'theyd': 2, 'tested': 7, 'muchcat': 1, 'halfassed': 4, 'observation': 2, 'possibly': 6, 'ingesting': 1, 'thingmueller': 1, 'oval': 8, 'spills': 7, 'a1': 1, 'sauce': 4, 'bust': 5, 'winston': 1, 'churchillpolio': 1, 'stopped': 7, 'worldwide': 9, 'gates': 10, 'foundationwoman': 1, 'bangs': 2, 'yearsclassic': 1, 'boringbaby': 1, 'crib': 1, 'recalled3': 1, 'trends': 5, 'anyone': 25, 'pull': 18, 'offlibertarian': 1, 'reluctantly': 5, 'departmentcoroners': 1, 'denver': 2, 'sunshine': 4, 'shoulders': 2, 'crashman': 2, 'idea': 47, 'moodbee': 1, 'hospitalchristopher': 1, 'cross': 9, 'reaches': 16, 'mexican': 13, 'borderwhite': 1, 'privilege': 2, 'squandered': 2, 'buyannoying': 1, 'precedes': 2, 'nouns': 1, 'thearea': 1, 'committeenew': 1, 'liberty': 8, 'blacklivesmatter': 2, 'shirts': 3, 'gamelimits': 1, 'limitlessdoing': 1, 'wraplove': 1, 'jesus': 21, 'honk40': 1, 'symptoms': 2, 'womanalan': 1, 'turing': 1, 'sigma': 2, 'progresssteven': 1, 'tyler': 2, 'joe': 18, 'scareman': 1, 'jumped': 1, 'fence': 2, 'able': 18, 'housetrumps': 1, 'remark': 5, 'drives': 12, '24': 8, 'clintonjpmorgan': 1, 'chase': 6, 'acquires': 11, 'stearns': 1, 'tedioustoread': 1, 'articlenations': 1, 'agree': 16, 'saysif': 1, 'graduating': 2, 'thishistoric': 1, 'byrd': 1, 'imploded': 1, 'controlled': 4, 'demolitionrep': 1, 'elijah': 1, 'cummings': 2, 'policecommunity': 1, 'relations': 4, 'generationmeg': 1, 'whitman': 1, 'compares': 10, 'hitler': 3, 'mussolini10': 1, 'notable': 1, 'books': 23, 'historyobituary': 1, 'cites': 7, 'teens': 24, 'carsnewly': 1, 'tenured': 1, 'professor': 9, 'everfollowup': 1, 'confirm': 15, 'personalities': 3, 'healthyrace': 1, 'forgetting': 3, 'ferguson': 26, 'remembering': 4, '1967': 1, 'contemplating': 3, 'futuregrandfather': 1, 'coollooking': 1, 'possiblehow': 1, 'girls': 54, 'pollfinetuning': 1, 'program8': 1, 'latinas': 1, 'thankama': 1, 'making': 81, 'betterso': 1, 'fuckin': 2, 'outpouring': 1, 'sympathetic': 2, 'texts': 3, 'scaramuccis': 1, 'familyslightly': 1, 'overweight': 8, 'middleaged': 4, 'rest': 26, 'church': 18, 'choircongressional': 1, 'distances': 2, 'atheist': 2, 'labelscientists': 1, 'firstever': 9, 'elusive': 5, 'mineralarmy': 1, 'soldiers': 7, 'lover': 7, 'stabbed': 3, 'fbiexhausted': 1, 'olympian': 4, 'rent': 3, 'pyeongchang': 2, 'nightobama': 1, 'hiroshima': 2, 'visit': 31, 'apologizenew': 1, 'mets': 1, 'gm': 2, 'alderson': 1, 'collapses': 5, 'conferenceunlikely': 1, 'allies': 9, 'unite': 5, 'airport': 16, 'gate': 3, 'agentthe': 1, 'stormigrassroots': 1, 'addiction': 6, 'crisisceline': 1, 'dion': 2, 'served': 4, 'luxurious': 3, 'crystal': 5, 'gobletspring': 1, 'sprung': 1, 'early': 27, 'itobamas': 1, 'fifth': 12, 'coast': 6, 'lotno': 1, 'adult': 15, 'conversations': 8, 'prayers': 10, 'moments': 12, 'silencewhy': 1, 'did': 43, 'wikileaks': 5, 'x': 7, 'greenwald': 1, 'wouldntbumble': 1, 'bee': 16, 'tuna': 2, 'supermarket': 3, 'circular': 3, 'coverpudgy': 1, 'doughboy': 2, 'rosy': 1, 'cheeks': 1, 'presses': 2, 'nose': 4, 'window': 9, 'chocolate': 9, 'shopapple': 1, 'fritter': 1, 'here': 26, 'needthis': 1, 'nervous': 8, 'seventh': 4, 'yearpresident': 1, 'barack': 7, 'backs': 11, 'expanding': 6, 'securitywoman': 1, 'caress': 1, 'dry': 5, 'cracked': 4, 'bleeding': 6, 'handsentirety': 1, 'protected': 4, 'wingariel': 1, 'castro': 3, 'failed': 20, 'systementertainmenthistory': 1, 'buffs': 1, 'reenact': 4, 'battle': 18, 'starsseth': 1, 'ridicules': 1, 'trumppfizer': 1, 'abandoning': 3, 'controversial': 16, 'planwatch': 1, 'harry': 19, 'rihanna': 4, 'hiv': 11, 'togethermila': 1, 'kunis': 2, 'kate': 9, 'mckinnon': 2, 'trailerpapa': 1, 'johns': 2, '3day': 3, 'deliverytrumps': 1, 'urged': 8, 'dreamersnational': 1, 'france': 11, 'annihilate': 1, 'radicalsmississippi': 1, 'support4': 1, 'hallmark': 2, 'saved': 18, 'spirithpps': 1, '2mark': 1, 'twains': 1, 'walt': 1, 'whitmanpalin': 1, 'brushing': 1, 'foreign': 27, 'epcotnew': 1, 'caesars': 2, 'marketing': 21, 'strategy': 11, 'hoods': 1, 'carsa': 1, 'plethora': 1, 'patio': 2, 'plantskim': 1, 'reveals': 81, 'robbers': 3, 'attackstephen': 1, 'hawkings': 2, 'disability': 3, 'overcomeisis': 1, 'weapons': 14, 'monitornations': 1, 'quaint': 2, 'huffington': 6, 'seemed': 4, 'industrygovernment': 1, 'hiding': 11, 'embarrassingly': 1, 'lame': 5, '1973': 2, 'extraterrestrial': 2, 'encountera': 1, 'spooky': 2, 'booblack': 1, 'friday': 4, 'webhistorical': 1, 'popular': 20, 'hymns': 1, 'sung': 1, 'latepyer': 1, 'moss': 1, 'tackling': 4, 'depressionwhite': 1, 'sanction': 2, 'downgop': 1, 'operatives': 1, 'winfriday': 1, 'meet': 39, 'brian': 5, 'schweitzerpersonal': 1, 'trainer': 3, 'deskdustin': 1, 'lance': 3, 'great': 58, 'reply': 2, 'kidsman': 4, 'rosie': 1, 'odonnells': 1, 'chelseabank': 1, 'underdraft': 1, 'feenew': 1, 'negativity': 1, 'promotes': 5, 'ugliness': 2, 'shapes': 2, 'sizessunday': 1, 'roundupthe': 2, 'chip': 4, 'joanna': 1, 'gaines': 1, 'secretpolice': 1, 'settlement': 8, 'funds': 6, 'kidshow': 1, 'trustflorida': 1, 'clarify': 1, 'rulingdog': 1, 'whatever': 18, 'todayprince': 1, 'air': 38, 'mattress': 4, 'corner': 8, 'windsor': 1, 'castleamerican': 1, 'split': 12, 'memo': 8, 'authorizing': 1, 'themlook': 1, 'responses': 5, 'beautifulantidepressant': 1, 'expected': 14, 'mess': 6, 'ownspy': 1, 'ridiculous': 7, 'opportunities': 2, 'doesman': 1, 'pierce': 2, 'orange': 7, 'peel': 2, 'fingernail': 1, 'impression': 16, 'to7': 1, 'summerhow': 1, 'apps': 3, 'fewer': 6, 'life18': 1, 'whisk': 1, 'summers': 3, 'youthvatican': 1, 'damage': 10, 'popes': 5, 'tolerant': 1, 'remarksevidence': 1, 'linking': 1, 'supremacist': 7, 'thincrumpledup': 1, 'potato': 4, 'spotted': 10, 'canstormy': 1, 'daniels': 12, 'flouting': 1, 'nda': 1, '60': 16, 'minutestearful': 1, 'anthropologists': 2, 'ancestor': 1, 'latethis': 1, 'poet': 4, 'racebill': 1, 'cosby': 13, 'feeling': 32, 'nowmagical': 1, 'dronemy': 1, 'mississippi': 9, 'represents': 2, 'hatewhy': 1, 'hampshiresome': 1, 'bizarre': 16, 'arguments': 5, 'courtplayground': 1, 'treated': 8, 'pugonpug': 1, 'actionaffordable': 1, 'amazon': 20, 'rainforest': 3, 'locationfederal': 1, 'tosses': 5, 'clock': 12, 'ahmed': 3, 'mohameds': 1, 'discrimination': 6, 'lawsuitpolice': 1, 'gunmananother': 1, 'continuing': 4, 'debilitating': 4, 'workplace': 7, 'stressreport': 1, 'guests': 9, 'podcasts': 1, 'weekisrael': 1, 'african': 12, 'migrants': 10, 'asylumseekers': 1, 'jaileveryone': 1, 'solely': 2, 'tolerate': 2, 'otherjfk': 1, 'cougars': 1, 'winyou': 1, 'iron': 7, 'voiced': 1, 'despondent': 2, 'packing': 3, 'officesiblings': 1, 'patiently': 4, 'close': 28, 'otherthe': 1, 'scandal': 20, 'mengod': 1, 'japanman': 1, 'basks': 1, 'triumphant': 2, 'glory': 2, 'purchases': 7, 'exact': 4, 'value': 14, 'cardcashier': 1, 'valuable': 8, 'illegal': 9, 'skillsretiree': 1, 'gearing': 4, 'errands': 1, 'lady': 16, 'friendmans': 1, 'bloodstream': 1, 'hourlong': 2, 'intermission': 2, 'alcohol': 3, 'blitzeschris': 1, 'emits': 2, 'loud': 11, 'sob': 1, 'crowd': 21, '4': 43, 'agofrancebound': 1, 'airliner': 3, 'grounded': 2, 'amsterdam': 2, 'threatening': 11, 'tweetus': 1, 'pushes': 10, 'sanctionsben': 1, 'surveys': 3, 'findmorbidly': 1, 'hobbitedison': 1, 'antisolar': 1, 'pr': 5, 'revealednation': 1, 'porndo': 1, 'solsticeisrael': 1, 'soldier': 6, 'captured': 5, 'imminentsheryl': 1, 'crows': 1, 'freshness': 1, 'expireswhy': 1, '83yearold': 2, 'walked': 3, 'looking': 61, 'hugparents': 1, '6yearold': 4, 'sorely': 2, 'knockknockjoke': 1, 'bookhow': 1, 'dress': 24, 'superfan': 1, 'goodnew': 1, 'evidence': 23, 'suggests': 30, 'impress': 2, 'friendsgrocerystore': 1, 'freezers': 2, 'wreckthe': 1, 'cuba': 8, 'strategic': 2, 'policypope': 1, 'miter': 1, 'faceshield': 1, 'comply': 1, 'vatican': 8, 'safety': 11, 'measurescancer': 1, 'topples': 3, 'chavez': 1, 'bloodless': 1, 'coupexhibitionist': 1, 'zoo': 10, 'elephants': 2, 'gather': 12, 'screwingsuspect': 1, 'stockholm': 3, 'confesses': 2, 'terrorist': 6, 'sayswhy': 1, 'marchmost': 1, 'sell': 7, 'presidentwebsites': 1, 'builtin': 1, 'engine': 4, 'pathetictrump': 1, 'associates': 2, 'growing': 20, 'frustration': 2, 'crisiscandidate': 1, 'opponent': 4, 'racism': 11, 'happensa': 1, 'massachusetts': 5, 'coalition': 2, 'interestsarea': 1, 'mofo': 1, 'chillrussell': 1, 'simmons': 5, 'rally': 18, 'yorkthis': 1, 'weekend': 25, 'moviesi': 1, 'myself': 2, 'felt': 8, 'ategunman': 1, 'mcdonaldshuffpollster': 1, 'nationally': 1, 'struggles': 9, 'battleground': 2, 'stateschallenging': 1, 'warsaw': 1, 'pact': 3, 'within45yearold': 1, 'selfconscious': 3, 'changes': 18, 'throughbungling': 1, 'bicycleriding': 1, 'robbery': 3, 'foiled': 4, 'weatherask': 1, 'chick': 5, 'trans': 30, 'attentionbathroom': 1, 'rep': 10, 'bisexualwoman': 1, 'switching': 1, 'brandsvacationing': 1, 'detective': 5, 'pretend': 8, 'woodswomen': 1, 'teuwen': 2, 'cofounder': 6, 'communicationstrio': 1, 'cutups': 1, 'attempts': 18, 'horse': 10, 'landlordmilitary': 1, 'recruiter': 3, 'onetaylor': 1, 'hatala': 1, 'larsen': 1, 'routinejon': 1, 'battling': 5, 'walkers': 5, 'ikea': 1, 'rugusbacked': 1, 'militias': 3, 'raqqa': 2, 'isisgermany': 1, 'arrests': 7, 'connected': 6, 'attackspope': 1, 'facebooklastminute': 1, 'bowl': 29, 'dcorwhen': 1, 'count': 7, 'reopening': 1, 'hikersman': 1, 'faced': 3, 'possession': 3, 'freed': 3, 'battleentertainment': 1, 'critic': 3, 'lets': 41, 'redo': 1, 'sorority': 4, 'gradereport': 1, 'computer': 13, 'presidentgrandma': 1, 'swallowing': 3, 'grandpa': 4, 'reportsmother': 1, 'doctordefiant': 1, 'londoners': 2, 'sit': 14, 'street': 35, 'ramadan': 1, 'highrise': 1, 'firegrumblethor': 1, 'mischievous': 1, 'pleased': 5, 'mayhem': 2, 'antics': 1, 'wrought': 1, 'upon': 7, 'housefbi': 2, 'relationsgay': 1, 'scant': 1, 'mention': 8, 'among': 28, 'values': 4, 'summited': 1, 'chest': 4, 'updatenuns': 1, 'pipeline': 7, 'erecting': 1, 'patharea': 1, 'thinkingnation': 1, 'channeling': 1, 'outrage': 2, 'addressed5yearold': 1, 'belle': 1, 'birthdayinsatiable': 1, 'droplet': 1, 'barrels': 2, 'windowpane': 1, 'pathdangerous': 1, 'delusionalcrazy': 1, 'cheap': 13, 'fly': 6, 'countries': 12, '30': 32, '160talktome': 1, 'discuss': 15, 'biggest': 31, 'challenges': 10, 'happiest': 7, 'momentsdepartment': 1, 'hires': 9, 'spread': 7, 'evenly': 1, 'schoolshillary': 1, 'barking': 2, 'totally': 36, 'pawsometerrifying': 1, 'tornado': 4, 'forgetevangelical': 1, 'haggard': 2, 'molested': 1, 'congressmangrimacing': 1, 'drafts': 1, 'charleyhorse': 1, 'researchmitt': 1, 'lifecareerdriven': 1, 'tied': 10, 'jobpope': 1, 'immortal': 1, 'beings': 7, 'dayholy': 1, 'owes': 2, 'dollars': 4, 'filling': 3, 'taxesparents': 1, 'newborns': 5, 'perspectivegrindr': 1, 'regular': 12, 'teststrembling': 1, 'pallid': 1, 'rnc': 14, 'attendees': 11, 'undergo': 4, 'firearm': 3, 'withdrawalconan': 1, 'obrien': 4, 'rough': 10, 'reception': 1, 'haiti': 4, 'trumprecording': 1, 'academy': 6, 'aging': 9, 'musicians': 6, 'december': 7, 'included': 3, '2017': 17, 'tributesman': 1, 'rush': 11, 'lennon': 2, 'beat': 21, 'wifetrump': 1, 'germans': 2, 'vowed': 2, 'sales': 22, 'uslate': 1, 'hosts': 9, 'gags': 1, '71st': 1, 'birthdaynews': 1, 'wartime': 1, 'atrocity': 1, 'brevityscott': 1, 'weekmaking': 1, 'postbreakup': 1, 'masterpiece': 1, 'location': 4, 'permit': 1, 'timecheney': 1, 'motorcade': 2, 'halfopen': 1, 'drawbridgeben': 1, 'earthfierce': 1, 'frozenthemed': 1, 'tball': 2, 'viralevery': 1, 'exwifereport': 1, 'industry': 23, '2009trump': 1, 'ransack': 1, 'muellers': 3, 'steal': 9, 'dadjessica': 1, 'biel': 2, 'silas': 2, 'definitely': 5, 'justin': 15, 'timberlakesupreme': 1, 'sector': 3, 'unionsmans': 1, 'ideas': 23, 'convince': 6, 'fullinvestigation': 1, 'exposes': 7, 'ebay': 2, 'user': 15, 'pulitzer': 4, 'medalshow': 1, 'paradisesamsung': 1, 'smart': 16, 'documentsgrown': 1, 'refers': 2, 'novel': 9, 'ruined': 10, 'arnoth': 1, 'locatedtrump': 1, 'deliver': 12, 'unionworld': 1, 'prematurity': 1, 'born': 10, 'soonnew': 2, 'mop': 1, 'nasty': 7, 'hellcareer': 1, 'spider': 4, '3000': 3, 'pointnotre': 1, 'suspects': 12, 'train': 17, 'sayscity': 1, 'meaningful': 2, 'municipal': 1, 'bondsdad': 1, 'kingdom': 1, 'princessstolen': 1, 'downtown': 1, 'philadelphiapoliticians': 1, 'bash': 3, 'pocahontas': 3, 'slur': 3, 'navajo': 1, 'eventdrink': 1, 'pistachiosnation': 1, 'prevails': 1, 'lesson': 9, '911donald': 1, 'gonna': 4, 'explode': 3, 'quicklyhomesick': 1, 'stays': 10, 'automated': 1, 'maralago': 5, 'reservations': 1, 'linenaturist': 1, 'retreat': 3, 'ends': 24, 'bonerthis': 2, 'enchanting': 1, 'fairy': 5, 'tale': 13, 'magictina': 1, 'frost': 2, 'las': 8, 'vegas': 17, 'wakes': 9, 'comastar': 1, 'trek': 4, 'alien': 8, 'forehead': 2, 'wrinklestilda': 1, 'swinton': 1, 'clip': 8, 'bigger': 10, 'splash': 1, 'anywayhow': 1, 'bikini': 10, 'buying': 18, 'planstudy': 1, 'marine': 11, 'species': 19, 'cleaning': 14, 'spillsstudy': 1, 'relationship': 18, 'timewhere': 1, '750': 1, 'homicidesfamily': 1, '38piece': 1, 'astrazeneca': 1, 'assorted': 1, 'pill': 5, 'samplertesting': 1, 'teacheranatomy': 1, 'uncouplingfox': 1, 'meteorologist': 3, 'legs': 1, 'fatbloodsoaked': 1, 'mayor': 20, 'bloomberg': 5, 'homelessness': 2, 'citywestminster': 1, 'finalists': 1, 'form': 20, 'iditarod': 1, 'teamdeceased': 1, 'souls': 6, 'river': 7, 'styx': 1, 'ferry': 1, 'crossing': 5, 'underworld': 1, 'transit': 2, 'strikeevery': 1, 'tomboys': 1, 'modern': 14, 'ladyhanes': 1, 'wneck': 1, 'tshirtnicolas': 1, 'sarkozy': 2, 'nationwide': 8, 'burkinis': 1, 'electedcomic': 1, 'liz': 2, 'mieles': 1, 'animated': 2, 'damaged': 3, 'season3': 1, 'persistencethursdays': 1, 'aim': 2, 'rightsthe': 1, 'mens': 5, 'summeris': 1, 'degree': 2, 'investmentseasonal': 1, 'chronic': 5, 'months6': 1, 'celebs': 5, 'globesthank': 1, 'americaman': 1, 'upside': 5, 'recovering': 7, 'deathrand': 1, 'eats': 11, 'hoax': 2, 'met': 20, 'isisafter': 1, 'banana': 5, 'republicsnl': 1, 'version': 10, 'angela': 3, 'merkel': 4, 'yeargrab': 1, 'wine': 12, 'boxes': 3, 'backwoman': 1, 'willing': 8, 'settings': 1, 'boot': 2, 'camproommate': 1, 'cycling': 1, 'nowfacebook': 1, 'sourcemodern': 1, 'gym': 12, 'manners': 1, 'etiquette': 2, 'workoutthe': 1, 'g': 1, 'maps': 8, 'hundreds': 14, 'storiesstephen': 1, 'colbert': 23, 'citizenship': 6, 'newswhy': 1, 'advicemom': 1, 'figures': 14, 'watchers': 1, 'workheavy': 1, 'presence': 6, 'ensure': 5, 'adequately': 2, 'provokedmorning': 1, 'reimpregnates': 1, 'guiltridden': 1, 'womenarea': 3, 'worsehmm': 1, 'vaccines': 5, 'panderingcreating': 1, 'drugexit': 1, 'delayed': 3, 'seconds': 14, 'avoid': 25, 'pleasantries': 1, 'neighbortrump': 1, 'minister': 15, 'hates': 13, 'refugeesnude': 1, 'posing': 4, 'civics': 1, 'classvote': 1, 'wastedyoure': 1, 'bedroom': 5, 'nightmcdonalds': 1, 'caved': 1, 'ton': 4, 'pissed': 10, 'rick': 12, 'morty': 2, 'fanslet': 1, 'teacherwhat': 1, 'understood': 1, 'fatherdaughter': 1, 'awayplan': 1, 'drumming': 2, 'lifehousehow': 1, 'beisraeli': 1, 'palestinian': 13, 'wielding': 2, 'checkpointdelicate': 1, 'kept': 12, 'awake': 3, 'oclockthe': 1, 'decluttering': 2, 'weve': 10, 'heardjetblue': 1, '49': 2, '2day': 3, 'flash': 5, 'salesurgeon': 1, 'general': 28, 'twisting': 2, 'popalex': 1, 'delarge': 1, 'droogs': 1, 'amidst': 1, 'allegations': 18, 'misconductdying': 1, 'baboon': 1, 'hearttransplant': 1, 'listtornado': 1, 'creeped': 5, 'filming': 4, 'itthumbtack': 1, 'carpet': 16, 'largeshocking': 1, 'nashville': 4, 'cliffhanger': 1, 'connie': 1, 'brittons': 1, 'swan': 2, 'songwhy': 1, 'russell': 3, 'nominationbreaking': 1, 'cousin': 7, 'mark': 28, 'allmany': 1, 'harmed': 2, 'catering': 2, 'filmmonday': 1, 'grow': 16, 'psa': 2, 'forgivenessjimmy': 1, 'theater': 7, 'plannerd': 1, 'obscure': 2, 'crush': 3, 'everrunning': 1, 'novelwe': 1, 'these': 51, 'designsiowa': 1, 'telemedicine': 2, 'banimportant': 1, 'paper': 18, 'tragically': 6, 'smudged': 1, 'breadstick': 1, 'greasestudy': 1, 'moron': 6, 'attractive': 9, 'potential': 14, 'mateif': 1, 'menstruated': 1, 'periods': 3, 'tabooblearyeyed': 1, 'youpopular': 1, 'younbc': 1, 'obtains': 1, 'antiisis': 1, 'raid': 2, 'operativehometown': 1, 'wistfully': 3, 'toured': 1, 'via': 3, 'viewmarvel': 1, 'fullsized': 1, 'teaser': 3, 'antman': 1, 'trailerisis': 1, 'laid': 6, 'boobytraps': 1, 'mosul': 1, 'injure': 1, 'civilianseducation': 1, 'fundinglandlord': 1, 'convinced': 8, 'heat': 11, 'workingtrumps': 1, 'critics': 17, 'alikepatton': 1, 'icky': 2, 'sauna': 1, 'analogy': 3, 'trumpfamily': 2, 'grandfather': 4, 'enjoy': 12, 'watching': 37, 'league': 11, 'imaginabletexas': 1, 'execute': 3, 'murdering': 2, 'bloodhes': 1, 'baaaaackmc': 1, 'hammer': 5, 'afraid': 10, 'hammers4': 1, 'couscous': 1, 'dayrecurring': 1, 'zhang': 1, 'ziyi': 1, 'fantasy': 8, 'facekiller': 1, 'swears': 8, 'hernbc': 1, 'add': 16, 'dateline': 1, 'flursdayepa': 1, 'annual': 19, 'tap': 6, 'drink': 12, 'tastes': 4, 'kinda': 1, 'offnew': 1, 'forone': 1, 'document': 6, 'highways': 3, 'tore': 1, 'apartdick': 1, 'clark': 4, 'theretexas': 1, 'sandra': 4, 'bland': 4, 'systemnewlyweds': 1, 'saving': 15, 'marriagefamily': 1, 'evilsong': 1, 'leprosy': 1, 'educationresearchers': 1, 'spanking': 3, 'incredibly': 6, 'funhumane': 1, 'society': 13, 'quit': 11, 'failure': 10, 'oust': 1, 'harassmentoverweight': 1, 'spam': 2, 'featuring': 2, 'picturetexas': 1, 'medicaid': 6, 'parenthoodpoliticians': 1, 'schneiderman': 1, 'physical': 7, 'allegationsbanksy': 1, 'exdrug': 1, 'addict': 4, 'artarea': 1, 'womans': 22, 'baseless': 1, 'hatred': 4, 'hathaway': 5, 'reciprocatedthis': 1, 'wearydog': 1, 'passenger': 9, 'put': 56, 'carrier': 4, 'overhead': 4, 'binthe': 1, 'trumpbunch': 1, 'apparently': 25, 'saw': 11, 'brendan': 1, 'fraser': 2, 'mummy': 1, 'moviereport': 1, 'jobs': 20, 'lack': 15, 'cutpfizer': 1, 'mercifully': 2, 'batch': 1, 'trial': 15, 'patientsstaples': 1, 'backtoschool': 4, 'seasonbill': 1, 'kristol': 2, 'rand': 6, 'overrated': 1, '2016militaryindustrial': 1, 'recalls': 24, 'aftermath': 6, '911when': 1, 'upreport': 1, 'dip': 4, 'posts': 9, 'slimy': 1, 'gary': 4, 'webbcocksucker': 1, 'beats': 8, 'motherfuckertrump': 1, 'raises': 18, 'urban': 12, 'communities': 6, 'zero': 8, 'times5': 1, 'interesting': 6, 'restaurants': 6, 'worldsanders': 1, 'impresses': 2, 'jumping': 4, 'balcony': 5, 'poolfirefighters': 1, 'gaining': 6, 'ground': 19, 'californias': 6, 'deadliest': 4, 'blazesman': 1, 'position': 7, 'actzenefits': 1, 'stairwellsdonald': 1, 'idwhy': 1, 'insomniateen': 1, 'clever': 6, 'classsuccessful': 1, 'airstrike': 1, 'iraqis': 1, 'terroriststhe': 1, 'enigmatic': 1, 'josef': 1, 'koudelkatrump': 1, 'himbiden': 1, '20minute': 3, 'postdebate': 2, 'janna': 1, 'ryanwatching': 1, 'dvd': 7, 'reportsthis': 1, 'supermodel': 1, 'outed': 1, 'became': 15, 'pioneer70yearold': 1, 'dressing': 4, 'entirely': 9, 'purplenew': 1, 'usrussia': 2, 'safetyhey': 1, 'toored': 1, 'organizers': 2, 'renting': 3, 'porta': 1, 'pottyeagles': 1, 'interview': 18, 'attackcongressman': 1, 'boehners': 2, 'alert': 12, 'skin': 18, 'orangecost': 1, 'highnorth': 1, 'paintings': 4, 'ilnew': 1, 'chrome': 2, 'extension': 3, 'blocks': 9, 'shootersheres': 1, 'reserve': 4, 'dealpuerto': 1, 'ricans': 4, 'month': 17, 'mediathe': 2, 'rock': 44, 'announced': 1, '2020': 7, 'ofprincess': 1, 'charlotte': 5, 'bonding': 3, 'brotherthe': 1, 'nonprofit': 6, 'revolutiondhs': 1, 'individual': 7, 'alqaeda': 2, 'operative': 1, 'familydecembers': 1, 'peoplereport': 1, 'supposed': 6, 'cashiers': 1, 'bitchholiday': 1, 'blues': 4, 'spirituality': 2, 'rescuecadburys': 1, 'imported': 3, 'depressednegligent': 1, 'oaf': 1, 'sloppily': 1, 'playershobby': 1, 'lobby': 12, 'gops': 11, 'problemopposition': 1, 'protesters': 19, 'venezuela': 3, 'maduroalcoholthemed': 1, 'opensthe': 1, 'warped': 2, 'environmentalism': 2, 'industrial': 2, 'meat': 13, 'producersnoop': 1, 'dogg': 3, 'perform': 12, 'courtesy': 2, 'pharmamousy': 1, 'brunette': 2, 'glasses': 10, 'sizzling': 3, 'sexpotsears': 1, 'extremists': 4, 'willis': 1, 'towerclinton': 1, 'savageswhat': 1, 'injuryjared': 1, 'leto': 1, 'fk': 3, 'swiftthe': 1, 'higher': 16, 'moneyis': 1, 'happyatheists': 1, 'sue': 12, 'pennsylvania': 7, 'invocationsthis': 1, 'abused': 6, 'courtpassion': 1, 'demanding': 2, 'balloon': 6, 'inspiringeric': 1, 'swalwell': 1, 'bidwatch': 1, 'murrays': 2, 'murray': 7, 'updatehow': 1, 'closet': 7, 'ditch': 8, 'keepdads': 1, 'corps': 2, 'evident': 1, 'christmaspresent': 1, 'openingis': 1, 'solomon': 1, 'mediating': 2, 'judgingheinz': 1, 'industrialsized': 1, 'ketchup': 3, 'packetstephen': 1, 'warriors': 2, 'pantherseagles': 1, 'singer': 16, 'groveling': 2, '180': 2, 'pathetic': 5, 'parkland': 7, 'survivorspart': 1, 'engaged': 6, 'companys': 11, 'competitive': 1, 'advantagewed': 1, 'stephen': 15, 'colbears10yearold': 1, 'responders': 3, 'bike': 9, 'scene': 20, 'tylers': 1, 'legboeing': 1, 'wings': 4, 'planehumble': 1, 'ascetic': 1, 'declines': 4, 'inflight': 1, 'beverage': 1, 'servicedavid': 1, 'axelrods': 1, 'view': 15, 'economic': 19, 'storm': 13, 'inherited': 1, '2009goldman': 1, 'blowing': 5, 'nursing': 10, 'itman': 2, 'perot': 1, 'granted': 4, 'wishesjon': 1, 'halls': 2, 'congressnation': 1, 'presidentturkish': 1, 'cneyt': 1, 'arkinthis': 1, 'diet': 14, 'cokewhoisdsharp': 1, 'injects': 2, 'viral': 13, 'vine': 2, 'violin': 1, 'huffpost': 6, '6x60liberal': 1, 'immigration5': 1, 'married': 15, 'momtrump': 1, 'briefly': 8, 'believing': 2, 'rhetoricscientists': 1, 'effective': 7, 'lossgod': 1, 'millennia': 3, 'stormshow': 1, 'guns': 17, 'gays': 6, 'infiltrated': 1, 'corporate': 17, 'americachrissy': 1, 'teigen': 14, 'haters': 6, 'instagramwhat': 1, 'scandalous': 1, 'catholic': 13, 'churchentertainment': 1, 'tonight': 9, 'paramount': 2, 'releaserip': 1, 'ms': 4, 'paintgeologists': 1, 'continents': 1, 'drifted': 1, 'fallingoutlamar': 1, 'odom': 2, 'recovery': 4, 'serieslets': 1, 'disneys': 3, 'mulanoatmeal': 1, 'breakfast': 16, 'bars': 7, 'packed': 8, 'proteinbad': 1, 'filmbreaking': 1, 'ps4': 1, 'christmasshotgun': 1, 'abdomen': 1, 'pisses': 1, 'wilford': 1, 'brimley': 1, 'morewine': 1, 'dentaloffice': 1, 'receptionists': 1, 'head7': 1, 'pregnancy': 14, 'losshuffpost': 1, 'pollster': 2, 'chartshillary': 1, 'northern': 8, 'mariana': 1, 'islands': 4, 'caucuswhite': 1, 'coat': 5, 'hanger': 1, 'teammateamazing': 1, 'hated': 3, 'clich': 1, 'monthsnation': 1, 'happily': 5, 'reassured': 1, 'exxonmobil': 2, '449': 1, '2012candy': 1, 'raisinets': 1, 'bloated': 1, 'coffersrescue': 1, 'saves': 17, 'dangling': 4, 'lift': 6, 'dramatic': 5, 'videonew': 1, 'businessmissing': 1, 'elected': 6, 'aruban': 1, 'parliamentwhat': 1, 'lovecnn': 1, 'cnn': 13, 'shuttle': 4, 'channelbefore': 1, 'sunrise': 1, 'vienna': 1, 'austriaa': 1, 'joyous': 1, 'eid': 1, 'somalia': 3, 'ugaaso': 1, 'abukar': 1, 'boocows': 1, 'instagram': 11, 'capture': 11, 'celebrations': 2, 'mogadishuregistered': 1, 'offenders': 3, 'soon': 12, 'convictions': 1, 'noted': 2, 'passportssenate': 1, 'grapples': 1, 'deficitobama': 1, 'difference': 7, 'copsman': 1, 'diesnl': 1, 'praised': 4, 'draw': 4, 'skitelizabeth': 1, 'inaugurationare': 1, 'lifea': 2, 'hitech': 1, 'veggie': 1, 'burger': 10, 'itll': 5, 'convert': 4, 'eaterswhat': 1, 'interviewexhausted': 1, 'doctor': 17, 'wake': 16, 'morningarea': 1, 'concept': 6, 'suggested': 2, 'donationpower': 1, 'costslocal': 1, 'wellattended': 1, 'familyfirefighters': 1, 'nightclub': 5, 'blazespeculation': 1, 'royal': 15, 'endslowes': 1, 'plunger': 1, 'collapsible': 1, 'handlelocal': 1, 'othersclinton': 1, 'badtzmaru': 1, 'sanrio': 1, 'character7': 1, 'essential': 7, 'longlasting': 3, 'marriagerex': 1, 'pompeo': 5, 'quick': 18, 'explaining': 7, 'countriestall': 1, 'clogs': 1, 'busts': 4, 'stereotypes': 4, 'height': 4, 'morethe': 1, 'oceans': 4, 'gentle': 2, 'giantno': 1, 'peacethe': 2, 'predictable': 2, 'blowback': 1, 'supporting': 11, 'sectarian': 1, 'authoritarianism': 1, 'bahrainwe': 1, 'increase': 19, 'aid': 17, 'famineexcercise': 1, 'therekerry': 1, 'voicea': 1, 'syriastate': 1, 'emailsman': 1, 'lighter': 1, 'starts': 13, 'blazesweden': 1, 'experiment': 6, 'sixhour': 2, 'workdayjoan': 1, 'moran': 2, 'timelooselipped': 1, 'rudy': 6, 'giuliani': 10, 'represent': 3, 'departmentpapa': 1, 'treatment': 16, 'bulbous': 1, 'deformed': 1, 'creatures': 1, 'lactate': 1, 'saucemembers': 1, 'u2': 2, 'stare': 3, 'directions20': 1, 'teenage': 10, 'thingsus': 1, 'hating': 2, 'suffer': 5, 'consequencesfrustrated': 1, 'wishing': 3, 'autocratic': 2, 'cultcan': 1, 'graphicbill': 1, 'moyers': 1, 'departure': 3, 'holehow': 1, 'buyindustrial': 1, 'magic': 11, 'creates': 13, 'believable': 2, 'storylinestarting': 1, 'unicorn': 4, 'fireeyethe': 1, 'definitive': 4, 'foodsepa': 1, 'delegation': 3, 'pollution': 4, 'chinacomedian': 1, 'bashes': 3, 'casting': 6, 'openly': 10, 'yearsivanka': 1, 'labor': 20, 'benefit': 9, 'womenman': 2, 'boxersresearchers': 1, 'frogs': 1, 'mate': 4, 'cloacagod': 1, 'pledges': 12, 'researchnational': 1, 'pork': 4, 'deficiencydeputy': 1, 'flipped': 1, 'spring': 26, 'valley': 9, 'acted': 4, 'reprehensibly': 1, 'saythoughts': 1, '54': 3, 'below': 3, 'cabaretmohawked': 1, 'rex': 5, 'plutocratic': 1, 'fascist': 1, 'pigs': 3, 'mansmitten': 1, 'foot': 10, 'fetishist': 2, 'twothis': 1, 'slomo': 1, 'watermelon': 4, 'mortar': 3, 'pornhow': 1, 'cable': 4, 'hostuniversity': 1, 'professors': 5, 'classroomsnation': 1, 'datingliterally': 1, 'nowarea': 3, '200': 15, 'products': 9, 'calm': 9, 'herice': 1, 'terrorize': 1, 'hispanicswhats': 1, 'storygop': 1, 'billnorth': 1, 'attacknew': 1, 'extended': 4, 'paternity': 3, 'dads': 22, 'colleagues': 4, 'respectcharlton': 1, 'heston': 1, 'seriousman': 2, 'trapped': 11, 'boulder': 2, 'braces': 7, 'interviewscoobydoo': 1, 'cursebiden': 1, 'arrives': 8, 'fog': 1, 'machinefuneral': 1, 'pistoriushappy': 1, 'weirdogunfire': 1, 'protester': 4, 'struck': 7, 'carking': 1, 'latifah': 1, 'wifethis': 1, 'disorder': 6, 'corpsewendy': 1, 'faints': 1, 'dressed': 12, 'libertyboyfriend': 1, 'express': 7, 'secondhand': 2, 'outrageprison': 1, 'escapee': 1, 'courtikea': 1, 'daythe': 7, 'innocence': 2, 'bystanding': 1, 'bullyingtrybeatingmelightly': 1, 'pakistani': 4, 'wifebeating': 1, 'billpolitical': 1, 'drawings': 4, 'uncle': 5, 'samput': 1, 'spin': 7, 'salsaprotester': 1, 'heckles': 1, 'cancerpalestinians': 1, 'suspicious': 6, 'alaqsa': 1, 'promoted': 2, 'kerrybody': 1, 'ballots': 1, 'belated': 2, 'apologies': 2, 'montananeurosurgeon': 1, 'heckled': 1, 'deckbreaking': 1, 'thriller': 2, 'jeffery': 1, 'deaver': 2, 'gamewoman': 1, 'period': 6, 'oversierra': 1, 'leone': 3, 'downzales': 1, 'casual': 6, 'diamond': 3, 'ringsasianamerican': 1, 'discrimination12': 1, 'remind': 12, 'flawed': 2, 'amazing': 19, 'beingrubio': 1, 'refutes': 3, 'undocumented': 9, 'cold': 20, 'onto': 24, 'stagekitchenaid': 1, 'springloaded': 1, 'toaster': 3, 'allows': 22, 'rad': 2, 'schoolers': 3, 'midair': 3, 'housegood': 1, 'avidstampcollector': 1, 'routine': 7, 'workingit': 1, 'sweet': 23, 'jobwith': 1, 'cameras': 4, 'transparency': 4, 'caveatsbarbara': 1, 'boxer': 5, 'reckless': 3, 'irresponsible': 1, 'billcant': 1, 'reportsour': 1, 'postgay': 1, 'moviesfour': 1, 'emergency': 12, 'workers': 25, 'duty': 6, 'chokehold': 2, 'deatharabamerican': 1, 'thirdgrader': 1, 'recess': 1, 'crying': 13, 'saying': 22, 'anyonegirlfriends': 1, 'hothollywood': 1, 'weinsteindonald': 1, 'catching': 7, 'hackers': 9, 'cybersecurity': 2, 'pros': 2, 'gigglespreventing': 1, 'madmen': 1, 'nuclear': 43, 'materialadmit': 1, 'dupedjust': 1, 'failurethese': 1, 'backseat': 1, 'taxi': 1, 'capsulethe': 1, 'daily': 19, 'livescompletely': 1, 'unfair': 2, 'ended': 4, 'offender': 3, 'registry': 9, 'urination': 1, 'childmark': 1, 'testimony': 9, 'poring': 1, 'datajournalists': 1, 'flock': 1, 'homeobama': 2, 'policies': 5, 'lifetexan': 1, 'empty': 14, 'chili': 2, 'cookoffpoke': 1, 'stick': 6, 'raccoons': 1, 'deathmike': 1, 'hearttoheart': 1, 'breathdoris': 1, 'duke': 3, 'norton': 1, 'simon': 4, 'kindred': 1, 'spirits': 5, 'apartmark': 1, 'hamill': 5, 'disneyland': 3, 'outin': 1, 'theresa': 2, 'unitedjustice': 1, 'atts': 1, 'merger': 3, 'warnerdwade': 1, 'udonis': 1, 'haslem': 1, 'lebron': 5, 'opt': 2, 'outnew': 1, 'cover': 32, 'magazine': 23, 'predictor': 2, 'revealing': 5, 'allwatch': 1, 'timberlake': 8, 'literally': 12, 'robot': 7, 'videosymphony': 1, 'orchestra': 2, 'simply': 6, 'cannot': 7, 'collaboration': 5, 'mellencampputting': 1, 'cream': 12, 'consideredstorm': 1, 'financially': 4, 'strained': 3, 'houston': 6, 'hospitalspeople': 1, 'disabilities': 4, 'itus': 3, '600000': 2, 'acres': 7, 'pristine': 1, 'generations': 8, 'pollutefda': 1, 'declares': 19, 'munchos': 1, 'disodium': 1, 'guanylatedad': 1, 'mommadonna': 1, 'milliondollar': 2, 'schemeon': 1, 'margins': 2, 'intellectual': 2, 'disabilitiesmarco': 1, 'rubio': 25, 'doingarea': 1, 'afternoon': 3, 'personwoman': 1, 'stood': 3, 'nothingpodcast': 1, 'fishflea': 1, 'vendor': 2, 'unidentifiable': 1, 'lump': 1, '15how': 1, 'explained': 3, 'sentencetaylor': 1, 'mourns': 8, 'dornerhillary': 1, 'gloss': 1, 'context': 3, 'disagree': 2, 'debaterosemary': 1, 'farina': 1, 'signature': 6, 'successknee': 1, 'osteoarthritis': 1, 'trial84': 1, 'rescued': 11, 'hampshire': 12, 'squalorsenator': 1, 'simpler': 2, 'abominable': 1, 'peacenew': 1, 'tandem': 2, 'mobility': 3, 'scooter': 1, 'releasedpastor': 1, 'courts': 7, 'cake': 9, 'unhinged': 4, 'rantwoman': 1, 'went': 27, 'attackwe': 1, 'differences': 5, 'opinion': 13, 'respect': 13, 'betrayer': 1, 'causestudio': 1, 'israelipalestinian': 4, 'conflict': 5, 'zohan': 1, 'handbill': 1, 'wellfed': 2, 'slaves': 2, 'blames': 14, 'farleft': 1, 'attacksanother': 1, 'rages': 2, 'factoryhome': 1, 'oncedolphin': 1, 'vacation': 15, 'stockbrokerman': 1, 'wantthe': 2, 'innocent': 7, 'europes': 3, 'crisisstressedout': 1, 'cvs': 2, 'cigarettes': 4, 'monthsbutt': 1, 'bandit': 1, 'duo': 3, 'gunpointexxon': 1, 'mobil': 1, 'decades': 6, 'documents': 10, 'blowheres': 1, 'hocus': 1, 'pocus': 1, 'remake': 8, 'knew': 15, 'wantedjustin': 1, 'trudeau': 3, 'sock': 1, 'moreplanet': 1, 'explodesnews': 1, 'roundup': 13, 'august': 3, '29': 1, '2017fighting': 1, 'yemen': 6, 'dialogue': 5, 'distantthese': 1, 'landmarks': 2, 'decadesjustin': 1, 'invites': 4, 'controversy': 7, 'cornrow': 1, 'picmales': 1, 'circumcised': 1, 'reduce': 12, 'mozambique': 1, 'shift': 9, 'norms': 2, 'surrounding': 3, 'sexmassachusetts': 1, 'doctors': 15, 'patients': 9, 'gunscruz': 1, 'ballot': 2, 'conventionmiddle': 1, 'troublemaking': 1, 'geniethe': 1, 'artisanal': 1, 'supplies': 3, 'listrobert': 1, 'mueller': 13, 'ascends': 1, 'sky': 4, 'umbrella': 1, 'honesty5': 1, 'retireemile': 1, 'hirsch': 1, 'jail': 11, 'pleading': 2, 'assaultdo': 1, 'obsessive': 1, 'disordercamp': 1, 'counselor': 6, 'assigning': 1, 'matchmaker': 1, 'presiding': 1, 'marriagenew': 1, 'yorker': 5, 'wonscottish': 1, 'demands': 27, 'referendum': 2, 'independencewinston': 1, 'churchills': 1, 'nickname': 3, 'trumptotally': 1, 'mexico': 12, 'chastises': 1, 'partymichael': 1, 'jacksons': 3, 'reputation': 5, 'punctuality': 1, 'ruinscynthia': 1, 'erivo': 1, 'harriet': 3, 'tubman': 3, 'biopic': 3, 'harrietthis': 1, 'sued': 9, 'competitionthe': 1, 'revolution': 5, 'adding': 8, 'mixanticipating': 1, 'clashes': 3, 'retainerhillary': 1, 'endorsement': 6, 'unionpresident': 1, 'restraint': 1, 'ballad': 1, 'singershouseguest': 1, 'blanket': 3, 'washed': 3, 'usean': 1, 'sophisticated': 1, 'bots': 1, 'influencing': 2, 'educationboehner': 1, 'dealafter': 1, 'datingrugged': 1, 'sportutility': 1, 'lotheres': 1, 'stuck': 23, 'pastbeautiful': 1, 'marred': 3, 'hideous': 4, 'afterbirthhow': 1, 'addictiontwitter': 1, 'verify': 2, 'harasswhy': 1, 'skipping': 3, 'choicetrump': 1, 'phonemaker': 1, 'project4year': 1, 'malaysia': 1, 'weekthe': 4, 'suck': 3, 'fat': 8, 'dick': 9, 'veiled': 2, 'trumpthe': 5, 'presidentrumsfeld': 1, 'shirt': 15, 'daybiden': 1, 'tossed': 3, 'housekelly': 1, 'clarkson': 3, 'covered': 11, 'bestjebbushcom': 1, 'websitehotel': 1, 'hopping': 1, 'bartendertippers': 1, 'thumb': 3, 'delivered': 4, 'gore': 8, 'headquarterstwo': 1, 'thumbs': 4, 'nonsensefor': 1, 'parks': 10, 'greatlets': 1, 'naked': 14, 'pose': 5, 'frozen': 11, 'chickenswoman': 1, 'choked': 2, 'alone': 16, 'meetingsdreamers': 1, 'footballsdepression': 1, 'finances': 3, 'combine': 1, 'produce': 5, 'culinary': 3, 'abominationmissing': 1, 'alaska': 6, 'died': 21, 'murdersuicide': 2, 'policewhale': 1, 'beachedplows': 1, 'roads': 4, 'signsbreaking': 1, 'attendant': 6, 'currently': 18, 'pass': 23, 'cranberry': 1, 'juice': 4, 'laptop8': 1, 'stats': 2, 'prove': 11, 'anxiety': 11, 'seriouslybeautiful': 1, 'cinnamon': 2, 'roll': 16, 'pureethiopia': 1, 'prisoners': 7, 'reconciliationhow': 1, 'maybe': 7, 'muchthe': 1, 'briefing': 6, 'mohameddominos': 1, 'thanksgiving': 14, 'pizzagraffiti': 1, 'heart': 38, 'itpope': 1, 'richest': 2, 'peoplepostal': 1, 'stamps': 5, 'honoring': 7, 'postal': 3, 'servicehow': 1, 'fame': 11, 'fortune': 4, 'shaped': 4, 'artistdry': 1, 'flavorless': 2, 'cupcake': 1, 'disappointing': 4, 'biteun': 1, 'syrians': 4, 'besieged': 2, 'daraya': 1, 'yearskim': 1, 'reopens': 4, 'longclosed': 1, 'hotline': 4, 'koreacouple': 1, 'dinner': 25, 'reportsape': 1, 'appointed': 2, 'czarkit': 1, 'harington': 2, 'rose': 10, 'leslie': 5, 'sound': 24, 'nsfwromney': 1, 'volunteers': 3, 'doortodoor': 1, 'presidents': 16, 'deaddog': 1, 'skype': 2, 'windowon': 1, 'hungernew': 1, 'bears': 8, 'resemblance': 2, 'girlfriendlindsey': 1, 'graham': 13, 'tollfree': 1, 'telephone': 3, 'operatorpresident': 1, 'judicial': 3, 'nominees': 4, 'drive': 14, 'samantha': 3, 'drinkromney': 1, 'replace': 18, 'americaobamas': 1, 'housegett': 1, 'viviane': 1, 'amsalem': 1, 'craziness': 1, 'israeli': 7, 'societyfrustrated': 1, 'dumbass': 2, 'cartoon': 4, 'hugging': 3, 'immigrantstrump': 1, 'jefferson': 7, 'davis': 13, 'guestnetanyahu': 1, 'debatewhere': 1, 'smokemassive': 1, 'magma': 1, 'yellowstonei': 1, 'clicked': 2, 'internetparents': 1, 'resources': 3, 'visitamish': 1, 'quilt': 1, 'sale': 5, 'couplerobert': 1, 'e': 6, 'honorable': 2, 'traitorjimmy': 1, 'fallon': 8, 'silliest': 1, 'bets': 3, 'viewers': 12, 'madethe': 1, 'decadesold': 3, 'hymn': 1, 'sitindisney': 1, 'virgin': 6, 'princessbiden': 1, 'lucky': 11, 'suitselena': 1, 'fuels': 2, 'zedd': 1, 'rumors': 4, 'photoif': 1, 'ipadman': 1, 'reince': 2, 'controlling': 4, 'comfortingonly': 1, 'remembrance': 2, 'dayunemployed': 1, 'barker': 1, 'rightfixins': 1, 'pyramidozzy': 1, 'osbourne': 1, 'bites': 7, 'fivepound': 1, 'rabbita': 1, 'coward': 3, 'charleston': 4, 'continuesrookie': 1, 'nascar': 1, 'loststudy': 1, '72': 4, 'highfives': 1, 'unwarrantedjohn': 1, 'arkansas': 3, 'executions': 3, 'spectacular': 4, 'wrecktrump': 1, 'ally': 10, 'immediatelyrest': 1, 'leelah': 2, 'alcornclimate': 1, 'lies': 14, 'napkins': 2, 'takeout': 4, 'orderextrump': 1, 'adviser': 13, 'informant': 2, 'spied': 1, 'embarrassingnurturing': 1, 'mama': 5, 'familysighing': 1, 'weekindian': 1, 'casino': 6, 'saddest': 2, 'earthclassmatescom': 1, 'facebookclintons': 1, 'sight': 9, 'restoredzz': 1, 'classic': 17, 'legslaw': 1, 'enforcement': 5, 'alton': 4, 'sterling': 7, 'place6': 1, 'jersey': 9, 'newspapers': 4, 'resigndamning': 1, 'participated': 2, 'disappearance': 3, '43': 7, 'studentsbeginning': 1, 'collegetexas': 1, 'lt': 3, 'gov': 10, 'costing': 4, 'moneywhats': 1, 'twitterdemocrats': 1, 'fold': 2, 'panel': 8, 'shootingarea': 1, 'mandatory': 5, 'exposure': 4, 'penises': 1, 'peerslindsey': 1, 'buckingham': 3, 'fleetwood': 2, 'macstage': 1, 'hells': 2, 'bellesspringer': 1, 'audience': 25, 'killnations': 1, 'wildlife': 8, 'fleeing': 2, 'canadaeach': 1, 'class12': 1, 'items': 15, 'remain': 13, 'singleoctopussy': 1, 'villain': 4, 'jourdan': 1, '93shower': 1, 'snarls': 1, 'jungle': 5, 'turning': 25, 'onrepublicans': 1, 'disclosing': 1, 'findings': 2, 'conflicts': 3, 'tiesmoney': 1, 'sakehilary': 1, 'duff': 3, 'reconcile': 1, 'comriea': 1, 'obituary': 4, 'soninlawryan': 1, 'reynolds': 4, 'wished': 2, 'howgold': 1, 'medal': 3, 'burgers': 5, 'daygrandma': 1, 'relationshipme': 1, 'earl': 3, 'interviewprecious': 1, 'candidatelast': 1, 'mentions': 8, 'possibility': 5, '25year': 4, 'quagmirecouple': 1, '35000': 2, 'saydoddfrank': 1, 'fourwhat': 2, 'means': 16, 'lovevideo': 1, 'kicking': 3, 'mock': 4, 'electionleading': 1, 'expert': 8, 'endorses': 9, 'legalization': 4, 'oregonswiss': 1, 'threaten': 6, 'ricola': 1, 'embargocuriosity': 1, 'panorama': 1, 'martian': 3, 'duneinternational': 1, 'daygruff': 1, 'nononsense': 1, 'exploiting': 2, 'powerscientists': 1, 'identify': 5, 'collapsedespite': 1, 'hopeful': 7, 'americaneardeath': 1, 'experience': 19, 'followed': 2, 'rightonthemoney': 1, 'experiencetrevor': 1, 'noah': 22, 'smoking': 9, 'weed': 10, 'tvsen': 1, 'fully': 12, 'cooperating': 1, 'investigationfacebook': 1, 'betatesting': 1, 'downvote': 1, 'buttonevery': 1, 'deserves': 7, 'chancegoogle': 1, 'techjawa': 1, 'transportationjames': 1, 'diverse': 5, 'brilliant': 7, 'attackrepublicans': 1, 'shutdown': 14, 'boehner': 15, 'exitslongtime': 1, 'grateful': 9, 'tanzania27': 1, 'pompom': 1, 'hats': 2, 'hitsnew': 1, 'infertile': 1, 'innercity': 1, 'teensrepublicans': 1, 'reagan': 5, 'decree': 1, 'protects': 4, 'lois': 1, 'lernerhow': 1, 'influencehats': 1, 'apples': 3, 'umbrellas': 1, 'pipe': 5, 'magritte': 1, 'magician': 1, 'artgoogle': 1, 'goalsthis': 1, 'thinkcvs': 1, 'cheaper': 5, 'cvsbrand': 1, 'magazinefermilab': 1, 'generous': 4, 'anonymous': 7, 'particle': 3, 'donationuser': 1, 'hygiene': 1, 'differentiatorbird': 1, 'ledge': 1, 'thingthe': 2, 'struggle': 16, 'fit': 20, 'inneglect': 1, 'results': 26, 'promotionbartender': 1, 'remarkable': 4, 'tolerance': 2, 'alcoholicshuffpollster': 1, 'gallup': 4, 'bows': 2, 'pollingbush': 1, 'trapsnetwork': 1, 'dumbing': 1, 'envelopeover': 1, '450': 3, 'tehran': 1, 'antigovernment': 1, 'demonstrationsthe': 1, 'clothes': 5, 'illegally': 1, 'refugeesparkingramp': 1, 'slightlynavigating': 1, 'poverty': 8, 'tricky': 2, 'businessread': 1, 'calling': 18, 'boss': 15, 'motherfkercashstrapped': 1, 'conduct': 4, 'hammering': 2, 'squadman': 1, 'app': 13, 'somewherekanye': 1, 'flowers': 3, 'swiftindian': 1, 'sitar13': 1, 'babiesmlb': 1, 'yordano': 1, 'ventura': 1, 'marte': 1, 'separate': 9, 'crashesultrasnuggly': 1, 'hero': 14, 'pillow': 3, 'hugs': 3, 'sleepbush': 1, 'press': 30, 'conferencei': 1, 'denied': 2, 'entry': 6, 'courthouse': 2, 'womansole': 1, 'remaining': 12, 'lung': 5, 'filled': 13, 'rich': 20, 'satisfying': 5, 'flavorpatton': 1, 'wifes': 3, 'newly': 7, 'published': 3, 'graveafter': 1, 'applies': 3, 'donated': 6, 'buildingbelgian': 1, 'princess': 11, 'damages': 2, 'ministers': 1, 'starter': 2, 'incidentnew': 2, 'omnigrain': 1, 'cheerios': 1, 'existing': 3, 'grain': 1, 'earthmondays': 1, 'agendaheres': 1, 'lying': 9, 'fraudpeople': 1, 'beyonc': 5, 'jayz': 2, 'slideclive': 1, 'cussler': 1, 'thrilling': 3, 'printerwhy': 1, 'speechwoman': 1, 'riding': 7, 'busthe': 1, 'keeping': 11, 'personalzapp': 1, 'institute': 2, 'adjusts': 1, 'bounceounce': 1, 'ratioramadan': 1, 'fervor': 1, 'capitalistshow': 1, 'marriageunesco': 1, 'baptizedkids': 1, 'protecting': 11, 'changedriver': 1, 'rattled': 2, 'brush': 3, 'secondshow': 1, 'laramies': 1, 'awakens': 4, 'usicy': 1, 'snowball': 2, '9yearold': 2, 'crybeloved': 1, 'showbiz': 1, 'treasure': 4, 'douglas': 2, 'thingsnew': 1, 'roommate': 4, 'robust': 2, 'puttering': 1, 'apartmentkids': 1, 'observations': 1, 'crucial': 6, 'survivaljohn': 1, 'heartfelt': 3, 'orlandogrowth': 1, 'potentialthis': 1, 'stepmom': 1, 'biomoms': 1, 'salt': 4, 'pepper': 4, 'pairingnaked': 1, 'truth': 22, 'worrying': 5, 'bodysupreme': 1, 'bushera': 1, 'post911': 1, 'violationswhy': 1, 'gqs': 1, 'amy': 4, 'schumer': 17, 'disappointingpreschooler': 1, 'borrow': 1, 'classmates': 2, 'notes': 5, 'shapesdisillusioned': 1, 'admissions': 4, 'membership': 3, 'pitch': 9, 'anymorea': 1, 'ashtangascrabble': 1, 'comeon': 1, 'pointsenglish': 1, 'armed': 9, 'weapon': 6, 'shakespearehotel': 1, 'volcanowaterfall': 1, 'sweetest': 2, 'digs': 2, 'findhome': 1, 'depot': 5, 'bluetooth': 1, 'cordless': 2, 'hosestudy': 1, 'words': 25, 'frontrunnersdirtydenier': 1, 'klineflorida': 1, 'active': 4, 'virus': 7, 'transmission': 1, 'zonesobamas': 1, 'approval': 11, 'pollhome': 1, 'moreeggs': 1, 'weekgop': 1, 'trumpmore': 1, 'demonstrators': 1, 'rallies': 4, 'pariswatch': 1, 'jeremy': 2, 'piven': 2, 'dishes': 7, 'pbs': 3, 'selfridgeenglish': 1, 'ages': 2, 'modifier': 1, 'slidedriverless': 1, 'hype': 2, 'hubris': 3, 'distractionsarea': 1, 'windowportlandia': 1, 'mra': 1, 'dudescbs': 1, 'laugh': 13, 'walkoutfinding': 1, 'dory': 1, 'slowing': 1, 'officekeshas': 1, 'revenge': 8, 'praying': 3, 'healingcelebrities': 1, 'innaacp': 1, 'apologize': 4, 'lewisdwight': 1, 'howard': 5, 'finished': 7, 'masquerading': 1, 'superstaredge': 1, 'victorious': 3, 'toddlerhero': 1, 'heros': 1, 'lap': 1, 'dancemarco': 1, 'clue': 4, 'meansis': 1, 'fourday': 1, 'ideaarea': 1, 'xenical': 1, 'propecia': 1, 'claritin': 2, 'paxil': 1, 'drixoral': 1, 'lipitor': 1, 'tavistdlegendary': 1, 'broadcaster': 2, '82mom': 1, 'birthwhat': 1, 'knowthis': 1, 'biomom': 1, 'stepmoms': 1, 'friendship': 8, 'short': 23, 'inspiringnation': 1, 'them24': 1, 'odd': 3, 'absolutely': 20, 'runningtrevor': 1, 'superheroes': 2, 'capesthe': 1, 'closetmarco': 1, 'climbs': 3, 'garden': 8, 'forbidden': 6, 'midnight': 6, 'pachillary': 1, 'herselfearth': 1, 'ranked': 3, 'planetnight': 1, 'concerned': 23, 'insurance5': 1, 'carbs': 1, 'reallyall': 1, 'yall': 2, 'yo': 2, 'selvescourt': 1, 'extends': 8, 'registration': 2, 'deadline': 3, 'refusesobama': 1, 'clinches': 2, 'cabernet': 1, 'sauvignon': 1, 'votedisney': 1, 'lil': 4, 'dumpling': 1, 'sounds': 11, 'darlingrick': 1, 'mistakenly': 5, 'puerto': 22, 'rico': 15, 'countryfrom': 1, 'hunted': 1, 'attraction': 5, 'strategies': 4, 'prospectsmusicals': 1, 'yes': 16, 'musicals': 2, 'podcastingthe': 1, 'noninterventioniststhe': 1, '2016scientists': 1, 'sunwe': 1, 'porn': 10, '130000a': 1, 'century': 10, 'thrillwhy': 1, 'bucked': 1, 'wore': 10, 'ringbrexit': 1, 'trumpism': 1, 'fascismhurricane': 1, 'nicole': 1, 'bermudamorocco': 1, 'cracks': 7, 'journalistswinner': 1, 'pieeating': 1, 'contestlocal': 1, 'bull': 7, 'traveling': 10, 'spain': 2, 'bullstreasure': 1, 'hunters': 2, 'wreck': 3, '18thcentury': 1, 'carnival': 4, 'cruise': 13, 'shipmediocre': 1, 'painters': 2, 'talent': 4, 'painterobama': 1, 'costs': 16, 'populaceman': 1, 'pisssounds': 1, 'hockey': 5, 'chambersarea': 1, 'sister': 10, 'cuterthe': 1, 'peterson': 1, 'farm': 8, 'bros': 3, 'beef': 8, 'chipotle': 4, 'behaivoronly': 1, 'dogdavid': 1, 'analyzes': 2, 'sopranos': 3, 'shotbyshotgrandpa': 1, 'capmike': 1, 'vip': 2, 'receptionreddit': 1, 'page': 6, 'hosting': 3, 'nudeswoefully': 1, 'misguided': 1, 'stocking': 1, 'gallons': 4, 'milk': 11, 'armageddonhussein': 1, 'ironclad': 3, 'alibihalloween': 1, 'decorations': 1, 'blending': 1, 'nicely': 1, 'lightsreport': 1, 'youunions': 1, 'plot': 10, 'landmark': 4, 'rulingdunbar': 1, 'discontinue': 2, 'print': 4, 'edition': 2, 'newsletterhere': 1, 'feelings': 7, 'trumpcarevince': 1, 'gilligans': 1, 'spoils': 3, 'gilliganit': 1, 'appearance': 12, 'awkwardactivity': 1, 'athletic': 3, 'shoesi': 1, 'trumpsupporting': 1, 'biracial': 1, 'happenednypd': 1, 'ramarley': 1, 'judgmentsaudi': 1, 'crown': 3, 'atrocities': 1, 'yementasting': 1, 'wizarding': 1, 'potter': 7, 'diagon': 1, 'alleyim': 1, 'bi': 2, '21': 7, 'itnew': 3, 'caters': 1, 'travelerseast': 1, 'rated': 4, 'numberone': 4, 'magazinethe': 1, 'corruption': 7, 'beneath': 7, 'cuomos': 3, 'pushsee': 1, 'secondgrandson': 1, 'hairthe': 4, 'winners': 5, 'everwill': 1, 'robotics': 1, 'breed': 3, 'professionalshardees': 1, 'shame': 12, 'curtains': 2, 'behinda': 1, 'tribe': 1, 'quests': 1, 'phife': 1, 'dawg': 1, 'himnorth': 1, 'leastentertained': 1, 'earthman': 1, 'approaches': 2, 'powdered': 1, 'doughnuts': 3, 'unguarded': 1, 'clutch': 2, 'eggsdante': 1, 'virgil': 1, 'lathe': 1, 'conservative': 12, 'movements': 4, 'raging': 2, 'contradictionthe': 1, 'oldest': 7, 'findsdream': 1, 'drinkthings': 1, 'didpbs': 1, 'moderators': 3, 'adviceteacher': 1, 'dedicates': 2, 'bitchman': 1, 'vandalizes': 1, 'matterrobots': 1, 'asimovs': 1, 'roboticsglorious': 1, 'tomorrow': 5, 'postponed': 2, 'indefinitelynew': 1, 'favorably': 1, 'congressboeing': 1, '40000foot': 1, 'slide23': 1, 'sexy': 11, 'parentslightest': 1, 'amount': 12, 'contact': 12, 'apologized': 1, 'foris': 2, 'statistical': 1, 'analysis': 4, 'booktrump': 1, 'acknowledge': 4, 'dojs': 1, 'wellparis': 1, 'mysterious': 11, 'plimpton': 1, 'essay': 10, 'ghostdavid': 1, 'allan': 3, 'coe': 1, 'kick': 15, 'assdobby': 1, 'houseelf': 1, 'generosity': 1, 'beyondwoodstock': 1, '99': 3, 'revenue': 3, 'projections': 1, 'displayed': 2, 'multicolored': 1, 'laminated': 1, 'boards': 4, 'somewhere': 4, 'lawednesdays': 1, 'missile': 18, 'continental': 1, 'rangecustomer': 1, 'declined': 1, 'assistance': 1, 'floor': 13, 'crawling': 7, 'backbudget': 1, 'dreadlockedeverything': 1, 'subwayphilip': 1, 'cigarettesunderstanding': 1, 'collegethis': 1, 'bouncer': 2, 'vomitthe': 1, 'races': 3, 'feijoadamustsee': 1, 'falltop': 1, 'confounded': 1, 'trump9': 1, 'careerreport': 1, '57': 2, 'activism': 3, 'petitions': 1, 'discontinued': 1, 'itemslionel': 1, 'richie': 2, 'decided': 5, 'adopt': 2, 'nicoleicymi': 1, 'sexism': 5, 'lab': 5, 'ratsman': 1, 'fort': 2, 'wayne': 3, 'experiencethe': 1, '2018': 14, 'numbersmitt': 1, 'orrin': 1, 'hatch': 3, 'reportlethal': 1, 'recasting': 1, 'colead': 1, 'due': 16, 'abuseeveryone': 1, 'trainee': 1, 'gonerwhy': 1, 'prenuptial': 1, 'agreementchrissy': 1, 'gasps': 1, 'cardi': 2, 'threesome': 1, 'rihannahigh': 1, '85': 5, 'concert7': 1, 'ofman': 1, 'endfamily': 1, 'autistic': 3, 'amtraks': 1, 'webpagesyrian': 1, 'kurds': 4, 'bashar': 3, 'thwarting': 1, 'regionsteve': 1, 'kerr': 1, 'impassioned': 3, 'hearmother': 1, 'ferries': 1, 'options': 6, 'gap': 10, 'roomus': 2, 'soars': 3, 'bernankes': 1, 'reassuring': 1, 'pacific': 5, 'rimlocal': 1, 'actions': 3, 'efforts': 6, 'address': 15, 'change87': 1, 'shamebasedeva': 1, 'longoria': 2, 'tans': 1, 'spectrumcuba': 1, 'ambitious': 3, 'touristsheres': 1, 'score': 7, 'pasta': 3, 'olive': 3, 'gardenguy': 1, 'pringles': 1, 'convicted': 3, 'orderrussia': 1, 'isisbadass': 1, 'churchgoer': 1, 'hymnalturkish': 1, 'youno': 1, 'futurebusinessowned': 1, 'outnumber': 2, 'womenowned': 1, 'businessesleatherclad': 1, 'nomads': 1, 'seize': 7, 'australianation': 1, 'skim': 1, 'headlines': 2, 'samsung': 1, 'lawsuitpeople': 1, 'hiddlestons': 2, 'ts': 1, 'tophistorical': 1, 'jamesons': 1, 'miracle': 6, 'concoctioncameraman': 1, 'sole': 4, 'audienceobama': 1, 'resigns': 13, 'seattlemathematical': 1, 'skill': 1, 'downplayed': 2, 'splitting': 5, 'checkhuffpost': 1, 'greece': 4, 'preelection': 1, 'poll21': 1, 'pug': 2, 'desperately': 8, 'homelebron': 1, 'guarantees': 2, 'numerous': 4, 'gamesthe': 1, 'inclusion': 4, 'undo': 3, 'exclusionary': 1, 'policiesfederal': 1, 'mcveighs': 1, 'numberoutofcontrol': 1, 'gesture': 4, 'tumbling': 1, 'stagemans': 1, 'versatile': 1, 'projected': 3, 'situationreflections': 1, 'spouse': 6, 'angerobama': 1, 'equally': 5, 'powerfulafter': 1, 'backlash': 7, 'fema': 1, 'main': 3, 'websitearea': 1, 'dildo': 2, 'horny': 1, 'againworlds': 1, 'innovative': 3, 'companiesdonald': 1, 'truther': 2, 'winning': 15, 'floridamichelle': 1, 'panther': 7, 'inspiring': 10, 'backgroundscake': 1, 'theresunday': 1, 'roundupcity': 1, 'planner': 5, 'designing': 1, 'realizing': 15, 'philadelphia': 7, 'horseseach': 1, 'quadruplets': 1, 'harvard': 3, 'yaleva': 1, 'loan': 6, 'veterans': 10, 'downseagull': 1, 'diarrhea': 1, 'crowded': 3, 'timeparamedic': 1, 'bride': 6, 'weddingan': 1, 'environmentcolombia': 1, 'plunged': 1, 'uncertainty': 2, 'narrowly': 6, 'farc': 1, 'rebelsarea': 1, 'mylantaprofessor': 1, 'pressured': 4, 'evaluationdonna': 1, 'brazile': 1, 'rodham': 1, 'palace': 3, 'solar': 12, 'cultdeep': 1, 'trading': 5, 'spaces': 4, 'marathonthe': 2, 'awful': 9, 'allergies': 3, 'itoutofstate': 1, 'seendying': 1, '37yearold': 1, 'upwar': 1, 'criminal': 15, 'grandpathe': 1, 'omar': 1, 'khadr': 1, 'reaffirms': 3, 'valueshealthylooking': 1, 'drose': 1, 'halfcourt': 2, 'trapstarz': 1, 'outlander': 2, 'premiere': 9, 'tartan': 1, 'galawalter': 1, 'trials5': 1, 'resistance': 9, 'resolutionsraising': 1, 'childjulia': 1, 'louisdreyfus': 1, 'diagnosishawaii': 1, 'conversion': 6, 'therapy': 9, 'minorschris': 1, 'chappelle': 4, 'standup': 5, 'weekendalternativemedicine': 1, 'practitioner': 2, 'refuses': 26, 'method': 5, 'paymentalmost': 1, 'half': 18, '14fda': 1, 'aspirin': 1, 'attackman': 3, 'sickroyal': 1, '3621': 1, '2010': 3, 'visitwhite': 1, 'mainstream': 7, 'outletsreport': 1, 'adjectives': 1, 'tony': 10, 'snarky': 3, 'mediadishwasher': 1, 'brokejohn': 1, 'bunkers': 1, 'selfdestruct': 1, 'sequence': 2, 'russian': 29, 'oligarch': 1, 'taunts': 5, 'bank': 17, 'monitorspoll': 1, 'unlikable': 1, 'thirdparty': 1, 'candidatesay': 1, 'eggs': 7, 'trollsfighting': 1, 'rabies': 2, 'philippinesnew': 1, 'incredibles': 1, 'homemuscular': 1, 'guys': 22, 'catchhoward': 1, 'dean': 2, 'proposesenormous': 1, 'indoorsdonald': 1, 'istrump': 2, 'rescind': 1, 'directives': 1, 'everybodybigoted': 1, 'barbecueroot': 1, 'beer': 19, 'float': 1, 'creamcourtney': 1, 'screams': 3, 'manicuristrepressed': 1, 'molestation': 3, 'memory': 11, 'built': 6, 'beafter': 1, 'joey': 1, 'arias': 2, 'icon': 6, 'nightlife': 5, 'legendarea': 1, 'mother': 36, 'displays': 4, 'extensive': 2, 'goya': 1, 'collectionman': 1, 'rides': 4, 'outworlds': 1, 'arteriesranking': 1, 'usa': 4, 'debaclefollowing': 1, 'shifts': 8, 'revelations': 2, 'leaksbible': 1, 'homewoman': 1, 'dragged': 3, 'virginia': 14, 'listing': 5, 'donorsnew': 1, 'skip': 7, 'michelles': 2, 'partyawkward': 1, 'masturbated': 3, 'aboutdonald': 1, 'directorbush': 1, 'caribbean': 2, 'summit': 6, 'aboard': 3, 'catamaran': 1, 'onemothers': 1, 'breastfeed': 1, 'lower': 12, 'multiple': 8, 'sclerosis': 1, 'riskchristian': 1, 'uninspiredthe': 1, 'extraordinary': 3, 'elisabeth': 1, 'kblerrosswhat': 1, 'libya': 4, 'prioritizes': 1, 'suffering': 7, 'anotherstudy': 1, 'marriages': 2, 'matched': 4, 'yearsawful': 1, 'repeat': 6, 'againsexpositive': 1, 'marilyn': 2, 'minter': 1, 'glam': 3, 'glitter': 2, 'gunkbobby': 1, 'jindal': 1, 'nominationone': 2, 'lily': 1, 'cole': 5, 'economyall': 1, 'accomplishments': 3, 'overshadowed': 2, 'hefty': 1, 'weightfreddie': 1, 'prinze': 2, 'recoversnew': 1, 'holmes': 5, 'strict': 5, 'background': 3, 'purchasing': 5, 'firearmstrump': 1, 'agency': 6, 'paid': 15, 'muslims': 16, 'protection': 7, 'mecca': 3, 'pilgrimagefrance': 1, 'effort': 17, 'peacegoldman': 1, 'mortgage': 5, 'securitieshow': 1, 'berlinbaby': 1, 'stancestudy': 1, 'strongly': 5, 'opposed': 1, 'otherjohn': 1, 'trumppeter': 1, 'otoole': 1, 'montageas': 1, 'iranians': 2, 'saudi': 15, 'arabia': 5, 'fightthe': 2, '2014fbi': 1, 'withhold': 2, 'unlocking': 1, 'applefantasy': 1, 'leap': 1, 'teamearliest': 1, 'footprints': 1, 'canadafamily': 1, 'rewrites': 1, 'da': 4, 'seasonsiblings': 1, 'lowheres': 1, 'slang': 2, 'terminology': 1, 'brighten': 3, 'sundaywebsters': 1, 'melty': 1, 'lexicon300': 1, 'refund': 1, 'justify': 4, '700': 4, 'miscellaneous': 1, 'purchasessleepover': 1, 'bedsenators': 1, 'restrictionsthis': 1, 'gaga': 8, 'parody': 9, 'suckedun': 1, 'independent': 5, 'inquiry': 5, 'abuses': 3, 'yemenhillary': 1, 'intimidating': 1, 'fragrance': 2, 'linecop': 1, 'crashed': 3, 'cruiser': 1, 'owl': 1, 'headthe': 1, 'lingering': 2, 'ex': 9, 'breakupsman': 1, 'slide': 6, 'remembers': 15, '911baby': 1, 'beating': 9, 'drivinghere': 1, 'picked': 8, 'fartroubling': 1, 'flaunting': 1, 'itwooden': 1, 'dayarea': 1, 'humiliating': 7, 'pagemr': 1, 'yousons': 1, 'checked': 4, 'online12': 1, 'publicists': 1, 'transform': 4, 'dwayne': 3, 'johnsonreport': 1, '15000': 4, 'vanish': 1, 'fest': 4, 'hayride': 2, 'wagons': 1, 'yearfda': 1, 'nasalspray': 1, 'overdose': 3, 'naloxonetrevor': 1, 'zuckerbergs': 5, 'parentsfrustrated': 1, 'purringexperts': 1, 'warn': 9, 'retirees': 2, 'overwhelm': 1, 'scenic': 2, 'railway': 2, '203096yearold': 1, 'iris': 2, 'apfel': 1, 'barbieone': 1, 'arizona': 8, 'eight': 6, 'memberswhen': 1, 'proofhow': 1, 'synagogues': 1, 'nonjewish': 1, 'congregants': 1, 'yom': 2, 'kippurdonald': 1, 'wynn': 3, 'hastily': 5, 'formed': 3, 'rivals': 2, 'deeply': 11, 'questionable': 2, 'motivesbush': 1, 'rolls': 17, 'toll': 11, 'zeroprince': 1, 'markle': 6, 'debating': 2, 'hawaiian': 3, 'luau': 1, 'xfilesthemed': 1, 'weddingprotagonist': 1, 'cunning': 1, 'viewer': 6, 'recognizing': 2, 'films': 12, 'subtextbush': 1, 'peacefully': 2, 'sleepi': 1, 'equal': 15, 'menstruateswollen': 1, 'rushing': 5, 'leaks': 3, 'spouting': 1, 'keystone': 3, 'pipelineretired': 1, 'research': 13, 'chimps': 1, 'homereallife': 1, 'twist': 6, 'endings': 1, 'onenew': 1, 'paidiran': 1, 'exchange': 10, 'detailed': 5, 'diagram': 2, 'atomic': 2, 'bombregular': 1, 'forum': 3, 'recent': 16, 'decline': 5, 'discussionan': 1, 'allen': 8, 'iverson': 1, 'realest': 1, 'hall': 27, 'famerclinton': 1, 'clinch': 2, 'nomination': 6, 'californianew': 1, 'titanic': 3, 'icebergs': 1, 'viewstudy': 1, 'expressing': 5, 'anger': 14, 'unhealthy': 4, 'satisfyingmom': 1, 'feed': 6, 'gorgeous': 8, 'phototwitter': 1, 'confessed': 1, 'mayo': 3, 'coffee5': 1, 'salon': 3, 'haircut': 4, 'wrongfeeding': 1, 'soul': 9, 'stomachassessing': 1, 'deathhalf': 1, 'screened': 1, 'placebo': 3, 'filmstrangers': 1, 'stayed': 3, 'blizzardirish': 1, 'patricks': 6, 'praise': 8, 'trumpeveryone': 1, 'simultaneously': 2, 'happybush': 1, 'threepound': 1, 'kidney': 3, 'stoneamerica': 1, 'ferreras': 1, 'tone': 3, 'deaf': 5, 'cheerbjrk': 1, 'firefly': 1, 'bubblebill': 1, 'dates': 5, 'tour280': 1, 'meryl': 6, 'streeps': 1, 'honoredlaura': 1, 'noisily': 1, 'devours': 1, 'infantmoral': 1, 'tacked': 2, 'lifezoo': 1, 'hourly': 1, 'aphid': 1, 'birthcorporation': 1, 'branded': 1, 'itplanned': 1, 'antiabortion': 4, 'undercover': 3, 'videosworlds': 1, 'wealthiest': 2, 'billionaires': 1, 'trillion': 5, 'richer': 3, 'bloombergqueen': 1, 'iis': 1, 'shines': 4, 'darknesskittens': 1, 'netbaffled': 1, 'defame': 1, 'partydisappointed': 1, '8month': 2, 'waitlist': 1, 'pentagonthats': 1, 'raven': 1, 'secretswriter': 1, 'unwilling': 2, 'screenplay': 1, 'longfiorina': 1, 'sharia': 1, '47': 2, 'peoplefourth': 1, 'icecream': 2, 'cakewife': 1, 'husband': 14, 'marital': 3, 'problemsjob': 1, 'applicant': 3, 'intimate': 7, 'knowledge': 10, 'pagedefiant': 1, 'recalled': 3, 'cribwhich': 1, 'celebrities': 11, 'share': 20, 'astrological': 1, 'signlocal': 1, 'assholeted': 1, 'insiders': 2, 'outsiders': 1, 'gamegore': 1, 'tvellie': 1, 'goulding': 1, 'remixjulie': 1, 'andrews': 2, 'ive': 9, 'allynyc': 1, 'killingsstudy': 1, 'logical': 2, 'planes': 4, 'flyeminem': 1, 'elton': 1, 'grammys': 4, 'ripping': 5, 'plierscan': 1, 'honestly': 2, 'weddingsmiracle': 1, 'scrutiny': 2, 'nhl': 2, 'concussionsnation': 1, 'creaking': 1, 'infrastructurehillary': 1, 'basic': 9, 'bargain': 1, 'economybreaking': 1, 'jobblack': 1, 'embodiment': 2, 'glorythe': 1, 'leaderfleet': 1, 'stemcell': 1, 'trucks': 2, 'electedfriendship': 1, 'blossoms': 1, 'unrequited': 1, 'lovehouse': 1, 'immigrationwaitress': 1, 'overtipperthe': 1, 'owns': 9, 'household': 1, 'wealthcow': 1, 'usda': 4, 'ratinga': 1, 'daughterzamboni': 1, 'indicted': 7, 'iceshaving': 2, 'scandalsenile': 1, 'allowed': 8, 'solved': 2, 'crisisis': 1, 'racisttrump': 1, 'shootingnew': 2, 'longterm': 6, 'benefitsmicrosoft': 1, 'squiggly': 1, 'advanced': 4, 'audiencereport': 1, 'breakfastscientists': 1, 'sun': 5, 'hotwounded': 1, 'friendlyfire': 1, 'bullets': 2, 'morebiden': 1, 'invokes': 2, 'workrenowned': 1, 'ornithologist': 1, 'birdotherwise': 1, 'reasonable': 3, 'sincerely': 2, 'moonspace': 1, 'boardroom': 2, 'massaging': 1, 'various': 2, 'genitalscode': 1, 'uppitythis': 1, 'suspended': 11, 'mosthellmanns': 1, 'meatonthebottom': 1, 'cupsgiant': 1, 'aboutwhy': 1, 'selfdefense': 2, 'narrative': 5, 'ownersthree': 1, 'homejews': 1, 'unfiltered': 1, 'podcasttop': 1, 'official': 30, 'scathing': 5, 'letterwhere': 1, 'fireworks': 4, 'illinoisbush': 1, 'lovely': 4, 'chilean': 3, 'ambassadori': 2, 'youjlo': 1, 'arod': 1, 'donate': 6, '25000': 4, 'victimsremembering': 1, 'lynn': 2, 'walker': 9, 'huntleyin': 1, 'hatefilled': 1, 'greatobama': 1, 'postit': 1, 'counter': 2, 'note': 16, 'extralegal': 1, 'apparatuslast': 1, 'voluntarily': 2, 'diescuba': 1, 'viewneighbors': 1, 'wifi': 2, 'password': 3, 'goodchrist': 1, 'unlicensed': 2, 'imagearea': 1, 'bored': 5, 'ownsharvey': 1, 'crimesheres': 1, 'identified': 2, 'lgbtrant': 1, 'litterdelirious': 1, 'rover': 4, 'hallucinates': 1, 'marsreport': 1, 'bucks': 4, 'jukeboxthe': 1, 'underemployed': 1, '20000': 7, 'tocynthia': 1, 'nixons': 2, 'ads': 11, 'shred': 4, 'mta': 1, 'commuters': 2, 'rejoicemccain': 1, 'speechwriter': 1, 'smilescience': 1, 'increases': 10, 'suavity': 1, 'trenchcoatgod': 1, 'hurting': 6, '20piece': 3, 'spicy': 4, 'wingssenate': 1, 'subcommittee': 2, 'development': 6, 'familyreal': 1, 'fill': 9, 'eatingdisorder': 1, 'slotcelebrity': 1, 'saddened': 1, 'celebrityearliest': 1, 'tshirt': 6, 'foundevery': 1, 'seecall': 1, 'babyant': 1, 'notice': 6, 'pretzel': 3, 'crumb': 1, 'herkellyonmymind': 1, 'reflecting': 1, 'gissendanerbarbara': 1, 'surrounded': 4, 'loved': 25, 'ones': 10, 'jebcoast': 1, 'stranded': 3, 'yacht': 3, 'sweat': 7, 'morequeer': 1, 'bobby': 4, 'berk': 1, 'makeover': 2, 'incredibleovercoming': 1, 'adversity': 2, 'timephotographer': 1, 'grandmothers': 3, 'illness': 8, 'morefbi': 1, 'aides': 6, 'including': 3, 'huma': 1, 'abedin': 1, 'probehow': 1, 'fueling': 3, 'womenjefferson': 1, 'starship': 1, 'mallanalyst': 1, 'danger': 4, '2018curing': 1, 'blindness': 1, 'smartphoneuncool': 1, 'zookeeper': 2, 'gorillaswatch': 1, 'comedian': 6, 'mow': 1, 'argument': 8, 'heardno': 1, 'invited': 8, 'daves': 1, 'partythe': 3, 'shrink': 4, 'pink': 7, 'advertisers': 4, 'womenproof': 1, 'desperate': 18, 'sidefamily': 1, 'workfeminists': 1, 'babiesants': 1, '239hour': 1, 'workday25': 1, 'bandsblackberry': 1, 'exists': 2, 'alrightdaddy': 1, 'yankee': 2, 'singing': 9, 'despacito': 1, 'translationwendy': 1, 'vindicates': 2, 'filibusterpreteen': 1, 'gigglingateverything': 1, 'phase': 5, 'neversmiling': 1, 'phaseone': 1, 'listeria': 2, 'packaged': 2, 'saladsix': 1, 'warnings': 4, 'fifty': 3, 'shades': 6, 'greyus': 1, 'lauded': 2, 'genderneutral': 1, 'fieldtaking': 1, 'arms': 7, 'birds': 7, 'buffet': 1, 'salmonnew': 1, 'healthier': 6, 'menu': 8, 'homenew': 2, 'mcmahon': 2, 'autobiography': 2, 'slept': 8, 'womenextremely': 1, 'vibrant': 1, 'sustain': 2, 'buffalo': 5, 'wingsyo': 1, 'soy': 2, 'blancoobama': 1, '2014hopeless': 1, 'resignation': 5, 'bumpwould': 1, '1433pound': 1, 'meteorite': 1, '11': 9, 'millionchuck': 1, 'forboys': 1, 'whalesong': 1, 'imitation': 1, 'anythingmyanmar': 1, 'earthquake': 9, 'ancient': 12, 'pagodas8': 1, 'parenthoodsatellite': 1, 'bounce': 1, 'signal': 3, 'swearing': 2, 'phoneman': 1, 'escape': 13, 'themwhen': 1, 'homecare': 1, 'workthe': 1, 'brexit': 5, 'repudiate': 2, 'hatredstarved': 1, 'thrown': 12, 'againipod': 1, 'childreninstagram': 1, 'influencers': 1, 'whyrat': 1, 'fancy': 4, 'catch': 8, 'oncraig': 1, 'kilborn': 2, 'showpabst': 1, 'drinker': 1, 'pabst': 1, 'pabsttrump': 1, 'stateowned': 1, 'dishonest': 1, 'mediacelebrity': 1, '747': 1, 'collisionice': 1, 'detains': 2, 'tim': 15, 'kaine': 9, 'rallyheres': 1, 'affects': 4, 'lesbian': 9, 'coupleshow': 1, 'seasonally': 2, 'februaryincreasingly': 1, 'settle': 5, 'attainable': 1, '35to44yearold': 1, 'demographicnikki': 1, 'haley': 3, 'swipe': 2, 'marco': 13, 'amnestyiran': 1, 'protests': 13, 'revolutionhow': 1, 'create': 23, 'hotelworthy': 1, 'bathroomscientific': 1, 'victory': 10, 'how17': 1, 'geeky': 1, 'therefriend': 1, 'freechris': 1, 'controlyalie': 1, 'lad': 1, 'sharply': 4, 'neckwhite': 1, 'obamathe': 1, 'antizika': 1, 'gmo': 1, 'mosquitos': 1, 'rests': 1, 'floridaplease': 1, 'everywheremagnificent': 1, 'sunset': 1, 'judy': 2, 'houraarp': 1, 'blasted': 6, 'touch': 10, 'primeas': 1, 'chevy': 1, 'awardwinning': 2, 'sustainability': 1, 'screwed': 3, 'everperformers': 1, 'incorporate': 1, 'spewing': 2, 'sewage': 3, 'rio': 3, 'ceremonyhow': 1, 'criticize': 4, 'ruining': 6, 'selfesteemnew': 1, 'liver': 2, 'difficulty': 2, 'lou': 1, 'reedno': 1, 'regularly': 7, 'happenshow': 1, 'terrorism': 6, 'diseaseconsumer': 1, 'entering': 7, 'demographicssan': 1, 'francisco': 9, 'vandals': 2, 'signsman': 2, 'runs': 19, '4yearoldthe': 1, 'cartgate': 1, 'honduraswhite': 1, 'addressed': 1, 'gerald': 1, 'ford': 9, 'presidentarea': 1, 'strongugandas': 1, 'anglican': 1, 'doubles': 5, 'lawman': 1, 'baklava': 1, 'profiled': 1, 'himreport': 3, 'christianity': 1, 'startsillinois': 1, 'elves': 2, 'politiciansknife': 1, 'condemned': 2, 'saranwrapped': 1, 'brownie': 1, 'panan': 1, 'speakershundreds': 1, 'bowtwo': 1, 'anchors': 4, 'yorknew': 2, 'sandwich': 14, 'wifirattlesnakes': 1, 'observedcontroversial': 1, 'photoediting': 1, 'makeup': 8, 'featureguide': 1, '2stephen': 1, 'enemies': 2, 'funniest': 22, 'alter': 3, 'egosdrunken': 1, 'episode': 20, 'repeatrude': 1, 'funnynewsguild': 1, 'reuters': 1, 'employeested': 1, 'infuriatingibtihaj': 1, 'fencing': 1, 'bronzefbi': 1, 'shuts': 12, 'prominent': 3, 'recruitment': 2, 'websitebush': 1, 'incumbency': 1, 'campaigncongress': 2, 'natural': 13, 'disaster': 17, 'digitalenhancement': 1, 'fundingfda': 1, 'shapeepa': 1, 'squirts': 1, 'supplybarack': 1, 'robocall': 2, 'doug': 6, 'racelionel': 1, 'messi': 2, 'kobe': 3, 'bryant': 2, 'basketballwine': 1, 'burntdown': 1, 'candles': 2, 'strewn': 1, 'petals': 1, 'dolphins': 5, 'courting': 1, 'pete': 3, 'carrollneil': 1, 'bridge': 6, 'reasonssamsonite': 1, 'roller': 7, 'walletemployer': 1, 'botches': 3, 'interviewgod': 1, 'huffing': 1, 'gaseous': 1, 'planetthis': 1, '249': 1, 'razor': 2, 'sapphire': 1, 'pricefda': 1, 'cancels': 9, 'population': 13, 'ate': 6, 'allsurfrock': 1, 'dale': 2, 'plays': 13, '78enthusiasm': 1, 'unbearablewatch': 1, 'zayn': 3, 'wanna': 4, 'videoromney': 1, 'pennsylvanian': 1, 'ann': 6, 'hourarea': 1, '3yearold': 3, 'dingdongreal': 1, 'flies': 5, 'bodywhy': 1, 'brag': 2, 'solangeopinions': 1, 'versus': 3, 'opinionatedhealth': 1, 'insurers': 4, 'volley': 1, 'optionwhy': 1, 'defending': 5, 'horrific': 9, 'cruelty': 2, 'animalspaul': 1, 'krugman': 2, 'unprecedented': 4, 'trumppurritos': 1, 'burritos': 2, 'thingsreport': 1, '93': 4, 'fineits': 1, 'christina': 2, 'aguilerahersheys': 1, 'candyclinton': 1, 'wyoming': 1, 'heats': 1, 'upthis': 2, 'satisfactiontwo': 1, 'perspectives': 2, 'assisted': 4, 'livingits': 1, 'horrifying': 7, 'partyingobama': 1, 'adopts': 4, '44yearold': 1, 'portuguese': 2, 'manman': 2, 'duckson': 1, 'sweating': 1, 'stuffwill': 1, 'charla': 1, 'nashs': 1, 'messageheres': 1, 'miserable': 4, 'yearprocessed': 1, 'carcinogenic': 1, 'whosaudi': 1, 'arabian': 3, 'itpink': 1, 'humbling': 1, 'parentingnfl': 1, 'concussion': 1, 'freeantiabortion': 1, 'marchers': 2, 'washingtontrump': 1, 'threatprairie': 1, 'rezoned': 1, 'usesnyder': 1, 'endorsing': 3, 'presidenttrump': 3, 'wasteful': 2, 'nea': 1, 'produced': 2, 'claes': 1, 'oldenburgs': 1, 'threeway': 1, 'plugon': 1, 'losing': 19, 'friendauthor': 1, 'metaphorreality': 1, 'duggar': 1, 'dillardlook': 1, 'jawdropping': 2, 'hideaways': 1, 'mapshepard': 1, 'faireys': 1, 'blatantly': 1, 'condemns': 10, 'demagogue': 1, 'trumpanother': 1, 'induction': 1, 'punching': 2, 'otherautomakers': 1, 'above': 8, 'windowsdear': 1, 'watchingtrump': 1, 'picks': 14, 'strange': 7, 'meals': 12, 'wheelscongress': 1, 'budget': 18, 'allocated': 2, 'laser': 8, 'stufftop': 1, 'campusobamacare': 1, 'repeal': 26, 'knock': 2, 'insuranceside': 1, 'salad': 4, 'hamburger': 1, 'toppingsa': 1, 'nonsensewedding': 1, 'photographer': 7, 'brides': 2, 'dadour': 1, 'rails': 2, 'enoughfriends': 1, 'birthdaythese': 1, 'gifd': 1, 'pop': 13, 'culturebeethovens': 1, 'sortrecognizing': 1, 'mindfulness': 3, 'practiceamericans': 1, 'showsthey': 1, 'giants': 6, 'lacks': 5, 'drugsno': 1, 'trumprussia': 1, 'legacy': 17, 'poisoned': 5, 'electionpeoples': 1, 'lima': 1, 'unites': 2, 'voices': 7, 'americaswestboro': 1, 'baptist': 3, 'picketing': 1, 'arbus': 1, 'funeralparents': 1, 'kidnapped': 2, 'pleahow': 1, 'isisskywriter': 1, 'suicide': 24, 'notetop': 1, 'seniorsfugitive': 1, 'heroine': 1, 'hair': 24, 'perfectlymeet': 1, 'mogul': 5, 'zombieat': 1, '16': 10, 'ivory': 7, 'resort': 7, 'towncrowd': 1, 'nakedphotos': 1, 'dreyfuss': 1, 'backstagethis': 1, 'nba': 13, 'careerlawyers': 1, 'opposing': 6, 'cite': 1, 'kidswithpreexistingconditionscangofuckthemselves': 1, 'clausemedia': 1, 'ignores': 5, 'unicyclistrepublican': 1, 'cub': 4, 'sayslocal': 2, 'roadsheryl': 1, 'sandberg': 1, 'speechman': 1, 'longlost': 2, 'meet84yearold': 1, 'graduates': 8, 'summa': 1, 'cum': 2, 'laude': 1, 'distinctionrare': 1, 'quarter': 3, '26': 5, 'centspunxsutawney': 1, 'phil': 3, 'beheaded': 3, 'prediction': 1, 'groundhog': 2, 'slaughtering': 2, 'daybored': 1, 'postmaster': 1, 'beard': 4, 'meetingprime': 1, 'norway': 1, 'laidinternet': 1, 'charmed': 1, 'datesenate': 1, 'celebrating': 7, 'founding': 3, 'beginningchris': 1, 'browns': 4, 'movethis': 2, 'september': 5, '1218': 1, '191415': 1, 'accent': 4, 'chairs': 4, 'missingthe': 1, 'artweekla': 1, '2014what': 1, 'vidcon': 1, 'upsilence': 1, 'thoughts': 13, 'techniqueairline': 1, 'alliancelebron': 1, 'finals': 1, 'thisstarting': 1, 'routinehow': 1, 'locked': 6, 'suitcase': 2, 'knowpentagon': 1, 'ethnic': 3, 'cleansingbritney': 1, 'rocks': 7, 'shorts': 2, 'heels': 2, 'disneylandmarketing': 1, 'keebler': 1, 'beloved': 9, 'culturerestrictive': 1, 'itadele': 1, 'bummed': 2, 'brangelina': 1, 'elsehow': 1, 'storythe': 2, 'mayorsean': 1, 'berserk': 1, 'awardwhy': 1, 'whiteno': 1, 'justicehow': 1, 'thank': 14, 'hiredthese': 1, 'indiaus': 1, 'hong': 4, 'kong': 5, 'onhistorical': 1, 'sold': 9, 'carved': 1, 'wooden': 5, 'headslazy': 1, 'rescuer': 1, 'oily': 1, 'pelicans': 1, 'pile': 9, 'sink': 5, 'daysfordham': 1, 'recordsnetanyahus': 1, 'fractured': 1, 'israel': 17, 'americanetflixs': 1, 'glow': 2, 'wrestlingfilled': 1, 'dreamfrozen': 1, 'tundra': 1, 'emptiness': 1, 'stretching': 3, 'weighed': 3, 'mike4763offduty': 1, 'intruder': 1, 'posted': 5, 'saymichelle': 1, 'labored': 2, 'breaths': 1, 'closes': 3, 'eyesif': 1, 'gasolinelgbt': 1, 'momentwrong': 1, 'side': 24, 'mallsadly': 1, 'ship': 5, 'sinksafghan': 1, 'turnbritney': 1, 'recovered': 1, 'stroke': 2, 'toxici': 1, 'penguin': 3, 'squeak': 1, 'seaworlds': 1, 'antarcticacrank': 1, 'caller': 4, 'jerking': 1, 'aroundwhile': 1, 'waswhat': 1, 'orlandoall': 1, 'hollow': 3, 'intel': 4, 'handlegmail': 1, 'pities': 2, 'hotmail': 1, 'userinvestors': 1, 'stake': 2, 'greenspans': 2, 'rate': 7, 'increaselondon': 1, 'attackare': 1, 'artistor': 1, 'masterpieceyoung': 1, 'brains': 1, 'seasonrecent': 1, 'mlb': 4, 'incidents': 4, 'sportsgod': 1, 'earths': 6, 'core4': 1, 'lowrisk': 1, 'networkreport': 1, 'masculinity': 2, 'pulled': 12, 'muscles': 2, 'yearlongshot': 1, 'merrick': 3, 'garland': 6, 'courtmike': 1, 'ash': 2, 'wednesday': 6, 'services': 8, 'dozens': 14, 'timestoo': 1, 'rescueunemployed': 1, 'moviechuck': 1, 'perez': 3, 'yakdonald': 1, 'giddy': 1, 'rachel': 8, 'maddow': 4, 'airing': 4, 'returnfarmers': 1, 'almanac': 1, 'predicting': 2, 'primetime': 1, 'dramaslisa': 1, 'daysone': 1, 'indivisible': 1, 'dew': 1, '999': 2, 'stomach': 5, 'bacteriathe': 1, 'numbers': 14, 'queenconcerned': 1, 'encourages': 9, 'daylivestock': 1, 'faircan': 1, 'littleknown': 1, 'spice': 5, 'lesssuburbanite': 1, 'poisoning': 2, 'brita': 2, 'filterheres': 1, 'afghanistanpigeon': 1, 'invoke': 1, 'flightalabama': 1, 'bo': 5, 'bice': 1, 'calendaruber': 1, '13ted': 1, 'cave': 4, 'alongcrazed': 1, 'frothmouthed': 1, 'grandchildren': 2, 'nowattractive': 1, 'wealthy': 5, 'worksnooping': 1, 'youpuerto': 1, 'default': 3, 'paymentpaul': 1, 'logs': 2, '4613th': 1, 'opportunity': 10, 'thisquantum': 1, 'lipconnecticut': 1, 'soda': 7, 'taxokie': 1, 'hears': 4, 'sams': 1, 'mexico4': 1, 'dentists': 1, 'advocate': 4, 'penaltywild': 1, 'rutting': 1, 'pour': 4, 'flooroutofcontrol': 1, 'safely': 4, 'selfkitchen': 1, 'warned': 3, 'regional': 4, 'managerexperts': 1, 'hazards': 1, 'backlogfrances': 1, 'le': 4, 'pen': 6, 'denounces': 2, 'clintonus': 1, 'detained': 9, 'patrol': 4, 'spanishneill': 1, 'blomkamp': 1, 'instagramcarol': 1, 'chapmans': 1, 'sleepberserk': 1, 'hairdresser': 1, 'permissiontrance': 1, 'unreal': 3, 'othervatican': 1, 'rosary': 1, 'windowsanother': 1, 'russias': 4, 'meddlingcubs': 1, 'loop': 1, 'pastmoms': 1, 'exhausted4': 1, 'resolutions': 7, 'runner': 3, 'maketrump': 1, 'kennedy': 7, 'center': 26, 'event9': 1, 'senior': 14, 'collapse': 5, 'overcrowded': 2, 'truman': 3, 'balconyblack': 1, 'realness': 1, 'everybody': 7, 'watchingblack': 1, 'colin': 9, 'kaepernick': 7, 'muchreport': 1, 'exciting': 10, 'except': 8, 'youchelsea': 1, 'africas': 1, 'elephantsthe': 1, 'forcewoman': 1, '2036military': 1, 'limiting': 3, 'severe': 2, 'ptsd': 1, 'combat': 7, 'toursformer': 1, 'ethics': 9, 'arpaio': 3, 'harbinger': 2, 'comenew': 1, 'peoplethree': 1, 'loungethis': 1, 'trueyou': 1, 'soulscivil': 1, 'missouri': 1, 'automatically': 3, 'update': 9, 'recordskushner': 1, 'excia': 1, 'suggestsbush': 1, 'olympians': 3, 'lots': 7, 'goldtrump': 1, 'accord': 2, 'musicmunchstrosity': 1, 'created': 19, 'fritolayboratorygiant': 1, 'clot': 1, 'dislodges': 1, 'femoral': 1, 'arterysport': 1, 'nelson': 7, 'mandelas': 1, 'timemost': 1, 'disgusting': 14, 'towel': 5, 'relegated': 1, 'bath': 4, 'matarea': 1, 'jumpingandtouchingtopsofdoorways': 1, 'phasehow': 1, 'outof': 1, 'statebakerys': 1, 'closing': 4, 'clairshealthy': 1, 'frosted': 2, 'paleo': 1, 'carrot': 2, 'cookiesfreedom': 1, 'maturityscientists': 1, 'vocal': 3, 'cords': 3, 'developed': 3, 'speculating': 1, 'acquaintances': 2, 'orientationgeorgia': 1, '11yearold': 3, 'boyyou': 1, 'mindhow': 1, 'cutout': 5, 'survive': 14, 'presidencyis': 1, 'outdated': 3, 'astraynew': 1, 'requires': 7, 'gere': 1, 'personally': 3, 'inform': 6, 'neighborhoodsixth': 1, 'groped': 4, '1991longtime': 1, 'pewter': 1, 'objectviolinist': 1, 'sick': 30, 'mozart': 1, 'coversin': 1, 'sea': 16, 'gypsies': 1, 'photosgarth': 1, 'pie': 3, 'nowamazon': 1, 'warehouses': 1, 'stocked': 2, 'launch5yearold': 1, 'bunker': 1, 'grandpahollywood': 1, 'bigbudget': 1, 'mrs': 5, 'smithman': 1, 'cleopatra': 1, '2525': 1, 'readso': 1, 'allman': 1, 'receptionist': 2, 'hitting': 14, 'himguard': 1, 'execution': 7, 'himnew': 1, 'digitalonly': 1, 'subscribersjosh': 1, 'greedypoll': 1, 'blockading': 1, 'courtoakland': 1, 'gentrification': 3, 'tech': 13, 'boom': 4, 'citynras': 1, 'lobbyist': 4, 'implies': 1, 'sidehomosexual': 1, 'dolphin': 3, 'highly': 11, 'gaynarcould': 1, 'quincy': 1, 'campaignunstable': 1, 'organizing': 5, 'nightkinkos': 1, 'pulls': 15, 'copykey': 1, 'switcheroomade': 1, 'theology': 2, 'caroline': 4, 'mackenziepolice': 1, 'incarceration': 2, 'rate11yearold': 1, 'periodhow': 1, 'folk': 1, 'artlabor': 1, 'unions': 3, 'blamed': 4, 'derailing': 2, 'effortsthe': 1, 'itchewing': 1, 'choking': 2, 'nutritional': 2, 'equivalencebo': 1, 'sunny': 1, 'selection': 4, 'portraitssharon': 1, 'backstage': 2, 'dramacomplete': 1, 'brittany': 1, 'murphy': 4, 'kwatinetzdrought': 1, 'californians': 1, 'electricity': 3, 'expenseslearning': 1, 'communications': 4, 'homethe': 1, 'calculated': 1, 'outlaw': 2, 'usloretta': 1, 'lynch': 4, 'terrorhalliburton': 1, 'pry': 2, 'fillings': 1, 'corpses': 1, 'teethhow': 1, 'netflixs': 5, 'girlboss': 1, 'perpetuates': 1, 'stereotypesthe': 1, 'punchespolice': 1, 'hat': 5, 'updatei': 1, 'tried': 17, 'kit': 4, 'to5': 1, 'nutritionistapproved': 1, 'tipseating': 1, 'enthusiast': 2, 'eclairweeping': 1, 'screaming': 9, 'tombstoneobama': 1, 'transfat': 1, 'reportheres': 1, 'blake': 7, 'farenthold': 3, 'resigned': 3, 'abruptlycomedy': 1, 'millionth': 2, 'cheech': 1, 'chong': 1, 'smokinstranger': 1, 'eleven': 2, 'ten16yearold': 1, 'sydney': 1, 'mclaughlin': 1, 'meter': 1, 'hurdlesbook': 1, 'readceiling': 1, 'transforms': 7, 'conditioning': 2, 'frosty': 2, 'wonderlandairline': 1, 'comedianhow': 1, 'advance': 4, 'statesnation': 1, 'outcome': 2, 'heardukraine': 1, 'transportation': 12, 'crimeaexxonmobil': 1, 'lenient': 1, 'voluntarilymeteorologist': 1, 'comeback': 5, 'questioned': 2, 'predictionman': 1, 'n': 10, 'syncreport': 1, 'arsenal': 1, '2000heres': 1, 'broadway': 6, 'mientustrump': 1, 'ball10': 1, 'divorcecollege': 1, 'roommates': 8, 'kingsize': 1, 'bedmaryland': 1, 'larry': 9, 'hogan': 5, 'partisan': 3, 'gridlock': 1, 'taekwondocollapsed': 1, 'stall': 6, 'coal': 3, 'extractionreport': 1, 'dogthe': 1, 'givingpeople': 1, 'valentinespresident': 1, 'washingtonnations': 1, 'engineers': 1, 'ponytailsgop': 1, 'committee': 20, 'platform': 7, 'antiscience': 3, 'viewsrussians': 1, 'expect': 10, 'dependsbathroomdisinfectant': 1, 'reinforces': 1, 'obsessivecompulsive': 4, 'disordermeet': 1, 'worldanother': 1, 'generationslouisa': 1, 'lisa': 3, 'gornickvatican': 1, 'banks': 16, 'exchief': 1, 'embezzlement': 2, 'launderingman': 1, 'pair': 6, 'uptraining': 1, '5what': 1, 'timothy': 3, 'shriver': 1, 'funfriends': 1, 'breakdown': 2, 'differentbalanced': 1, 'amid': 25, 'fiscal': 2, 'hypocrisyraccoon': 1, 'rabid': 2, 'fatherreforming': 1, 'debt': 8, 'problemtrump': 1, 'blocking': 5, 'immigrant': 11, 'teencoworkers': 1, 'embrace': 7, 'separationnew': 1, 'custard': 2, 'flandemicalec': 1, 'impersonate': 1, 'impersonator': 2, 'timeiowa': 1, 'candidateshe': 1, 'tuberculosis': 2, 'worriedbreakfast': 1, 'eaten': 10, 'outbush': 1, 'denyside': 1, 'economicsamericans': 1, 'slightly': 10, 'numbness': 1, 'todayvagina': 1, 'medicine': 4, 'itrecent': 1, 'attributed': 1, 'assets': 1, 'longdistance': 4, 'planantifa': 1, 'disrupt': 3, 'dayheres': 3, 'supportersfirstgrade': 1, 'apprehends': 1, 'urinator6yearold': 1, 'locker': 7, 'roomwistful': 1, 'attacks': 25, 'westthe': 1, 'genetic': 4, 'inheritanceteaching': 1, 'vietnamwhy': 1, 'economykhloe': 1, 'braless': 1, 'amazingcops': 1, 'respond': 11, 'unexpectedsiblings': 1, 'powerpoint': 2, 'hash': 1, 'offlimits': 2, 'topics': 1, 'thanksgivingwill': 1, 'victimbadass': 1, 'surgeon': 8, 'fingerless': 1, 'gloves': 2, 'operatinghillary': 1, 'outrageously': 1, 'highgrizzled': 1, 'proofreader': 1, 'wayshow': 1, 'headlockgaunt': 1, 'holloweyed': 1, 'sixth': 6, 'hunger': 11, 'strike': 10, 'proposed': 6, 'budgetformer': 1, 'vicente': 1, 'farmers': 8, 'assistants': 2, 'nannies': 1, 'boardwhat': 1, 'likejake': 1, 'hyland': 3, 'kansas': 12, 'mo': 3, 'chosen': 3, 'designated': 1, 'wiped': 4, 'address9': 1, 'bridesmaids': 1, 'knowproposed': 1, 'congressmen': 2, 'abortionhbos': 1, 'martin': 17, 'luther': 9, 'filmreveals': 1, 'yearsel': 1, 'salvador': 3, 'upholds': 3, 'threedecade': 1, 'term': 9, 'suffered': 1, 'stillbirthpreviously': 1, 'unknown': 6, 'prejudice': 4, 'japanese': 6, 'surfaces': 3, 'battleshipvacationing': 1, 'terse': 2, 'emotionless': 1, 'shipus': 1, 'fundlea': 1, 'michele': 2, 'slays': 4, 'scream': 5, 'queens': 7, 'premieremiddleaged': 1, 'underwearsleeping': 1, 'problemsmale': 1, 'outs': 2, 'anonymityweird': 1, 'announcement': 8, 'disneylucasfilm': 1, 'mergerkiss': 1, 'goodheres': 1, 'destinywatch': 1, 'wrecking': 3, 'destroy': 17, 'lifelocal': 1, 'atlantic': 1, 'oceantommy': 1, 'gamesus': 1, 'bared': 1, 'butts': 1, 'thai': 2, 'releasedtrump': 1, 'storybored': 1, 'barron': 1, 'counts': 6, 'time13': 1, 'ridiculousexcusestostayhome': 1, 'boomerstylebarbra': 1, 'streisand': 3, 'hudson': 5, 'uphalle': 1, 'berry': 4, 'hollywoodwhy': 1, 'sharing': 7, 'harming': 1, 'doneyou': 1, 'itconverse': 1, 'eccentric': 2, 'personalityjust': 1, 'lumbersexual': 1, 'homeobamas': 1, 'awakened': 1, 'consciencefear': 1, 'factor': 1, 'creators': 6, 'heirs': 2, 'ashes': 3, 'collect': 5, 'inheritanceabused': 1, 'crayonpoll': 1, 'narrowsanother': 1, 'noose': 1, 'museums': 2, 'sayjostens': 1, 'rings': 4, 'pregnant': 13, 'highschoolersfelipe': 1, 'bastard': 6, 'kingarea': 1, 'wrongfully': 1, 'employed': 1, 'yearsergonomic': 1, 'lumbar': 2, 'supportthe': 1, 'lowsend': 1, 'clowns': 1, 'zaatari': 1, 'campbald': 1, 'bright': 3, 'timeviolence': 1, 'nigerias': 1, 'oilrich': 2, 'delta': 1, 'regionlawyers': 1, 'marykate': 1, 'ashley': 2, 'olsen': 4, '17hour': 1, 'procedurewatch': 1, 'fmr': 1, 'ambassador': 10, 'zalmay': 1, 'khalilzad': 1, 'discusses': 6, 'policyhow': 1, 'inclusive': 1, 'usualus': 1, 'afghan': 9, 'retaken': 1, 'taliban': 8, 'marines': 4, 'leavecoal': 1, 'baron': 3, 'subsidize': 1, 'operating': 5, 'tablesurgery': 1, 'positionsecretive': 1, 'whcd': 2, 'preparty': 1, 'celebritiesapp': 1, 'mediterraneana': 1, 'rowthemed': 1, 'stokedwar': 1, 'unwinnable': 2, 'generalstressed': 1, 'cope': 3, 'boozereport': 1, 'exported': 1, 'tons': 3, 'crude': 5, 'yearbhp': 1, 'revolutionteen': 1, 'coroner': 1, 'saysvirginia': 1, 'uproar': 3, 'arabic': 1, 'calligraphy': 1, 'lessonsenator': 1, 'alarmed': 3, 'harassedtrump': 1, 'ukyou': 1, 'dcs': 1, 'snowwhat': 1, 'signbird': 1, 'cagelifelong': 1, 'boise': 1, 'resident': 5, 'knudsen': 1, 'nature': 9, 'centerrolling': 1, 'stones': 2, 'humor': 3, 'agebaylor': 1, 'coach': 7, 'ignoring': 8, 'abusesmokers': 1, 'timenorwegian': 1, 'vanilla': 1, 'bunsas': 1, 'issuegroups': 1, 'wide': 4, 'namehow': 1, 'nicer': 1, 'minutesrosie': 1, 'odonnell': 1, 'wifemark': 1, 'skywalker': 1, 'warsgoing': 1, 'flow': 1, 'diana': 4, 'paredes': 1, 'suadephotos': 1, 'instrument': 1, 'photoshoppedobama': 1, 'chased': 2, 'hearingjenna': 1, 'elfman': 3, 'inevitable': 5, 'outfitallfemale': 1, 'skate': 1, 'patriarchy': 3, 'nyccrossfit': 1, 'postbaby': 3, 'amazingboard': 1, 'member': 15, 'milo': 3, 'yiannopoulos': 2, 'invite': 2, 'cpacvets': 1, 'unendorse': 1, 'trumpfarting': 1, 'sparks': 7, 'fightfrom': 1, 'layoffs': 3, 'mediahunters': 1, 'morewest': 1, 'revokes': 1, 'terrain': 2, 'shiftshow': 1, 'cereal': 5, 'creamnew': 1, 'roomba': 1, 'blender': 1, 'smoothie': 3, 'pathscientists': 1, 'pronounce': 1, 'genomeschool': 1, 'mcdonald': 5, 'lived': 9, 'disadvantaged': 1, 'lifestunt': 1, 'biker': 1, 'macaskill': 1, 'scotland': 3, 'obstacle': 2, 'coursefucking': 1, 'oasis': 3, 'ceremonyrock': 1, 'propels': 1, 'senateted': 1, 'oregon': 7, 'downpig': 1, 'needskim': 1, 'jongil': 1, 'lotteryafter': 1, 'katy': 5, 'yoursintersecting': 1, 'interdisciplinary': 1, 'educationhow': 1, 'shapeman': 1, 'avoids': 4, 'texasjesus': 1, 'christ': 6, 'believed': 4, 'infourthgrade': 1, 'polishing': 1, 'grade': 3, 'anymorearea': 1, 'waythe': 2, 'interviewyour': 1, 'symptomim': 1, 'presidentbeyonc': 1, 'sculpted': 1, 'cheese': 13, 'strangely': 1, 'alluringstephen': 1, 'elseyoyo': 1, 'ma': 2, 'practiceobama': 1, 'slips': 3, 'fansmiley': 1, 'dogsenators': 1, 'myspace': 1, 'corporationsreport': 1, 'click': 1, 'linkpolice': 1, 'boatthe': 1, 'blackberry': 1, 'jacquie': 1, 'mcnish': 1, 'andsean': 1, 'silcoffsupporters': 1, 'aggravated': 1, 'selfinterestiggy': 1, 'azalea': 1, 'rolewhy': 1, 'dolezal': 1, 'soongod': 1, 'dumps': 4, 'grandmotherpope': 1, 'instructions': 3, 'spirit': 8, 'townenglish': 1, 'obviously': 10, 'hung': 5, 'overword': 1, 'immunity': 2, 'yearscontroversial': 1, 'faction': 1, 'nailed': 2, 'parallel': 4, 'woodnew': 1, 'wes': 2, 'anderson': 8, 'deadpan': 1, 'delivery': 10, 'meticulous': 1, 'direction': 5, 'issuesharleydavidson': 1, 'designed': 9, 'menundecided': 1, 'decisionkevin': 1, 'latecareer': 1, 'rolesyes': 1, 'bud': 2, 'weisser': 1, 'trespassing': 2, 'budweiser': 1, 'brewerybeforeandafter': 1, 'airbrushing': 2, 'alerts': 2, 'evil': 4, 'waysworkingclass': 1, 'whites': 6, 'counterpartsbuzzfeeds': 1, 'emotionaldebbie': 1, 'robins': 1, 'la': 11, 'bouillerie': 1, 'author': 6, 'television': 14, 'producer': 4, '31st': 1, '58single': 1, 'firstwe': 1, 'looked': 15, 'pet': 12, 'everyonelena': 1, 'dunhams': 1, 'shamefulbarbra': 1, 'electionmosquito': 1, 'confronts': 4, 'partner': 11, 'testing': 7, 'zikahouse': 1, '3month': 3, 'townmicrowave': 1, 'alarm': 6, 'clockdonald': 1, 'univisions': 1, 'jorge': 1, 'ramosthousands': 1, 'deathhow': 2, 'apply': 7, 'lipstick': 1, 'spazallgirls': 1, 'sweetieus': 1, 'limestonebased': 1, 'weekairasia': 1, 'nearmisses': 1, 'vanished': 1, 'planeclimate': 1, 'actionrepublican': 1, 'dismisses': 4, 'independenthow': 1, 'uber': 8, 'younew': 2, 'cases': 12, 'rape': 24, 'incestboston': 1, 'examine': 2, 'iphonestressedout': 1, '8yearold': 3, '12workaholic': 1, 'unimpressive': 1, 'life5': 1, 'foolproof': 1, 'copy': 5, 'weekendnorth': 1, 'capable': 15, 'mileslost': 1, 'jack': 10, 'manuscript': 2, 'doggy': 2, 'foundbill': 1, 'maher': 14, 'spoton': 3, 'monologue32': 1, 'breaches': 2, 'larger': 9, 'sonys': 1, 'yeartwo': 2, 'airportdetroit': 1, 'burned': 4, 'moneythe': 1, 'bogus': 2, 'toostormy': 1, 'cohen': 8, 'deck': 2, 'cardsman': 1, 'suit': 18, 'affecting': 1, 'nonsuited': 1, 'individualsjohn': 1, 'reids': 2, 'nutsobreitbart': 1, 'attentionarea': 1, '1743': 1, '1919james': 1, 'oreillys': 1, 'slavery': 3, 'commentscities': 1, 'uniting': 2, 'monsantoamericas': 1, 'jobseric': 1, 'rosswood': 1, 'audiohalle': 1, 'undying': 1, 'biopicaddressing': 1, 'spiritual': 4, 'bullying': 4, 'fablemccain': 1, 'amounts': 1, 'torturebrands': 1, 'aww5': 1, 'differentiation': 1, 'profitjayz': 1, 'shoutout': 2, 'shareholdazrob': 1, 'reiner': 2, 'shoot': 9, 'nc': 2, 'repealedthe': 1, 'nassar': 4, 'abusedkleenex': 1, 'inadequately': 1, 'coveredare': 1, 'brick': 4, 'checking': 6, 'accounts': 5, 'wallets': 2, 'prepaid': 1, 'debit': 1, 'etcparent': 1, 'typing': 3, 'verbatim': 1, 'postphotographers': 1, '186': 1, 'compete': 7, 'competitionchuck': 1, 'involvedmtvs': 1, 'severed': 3, 'tubcia': 1, 'tremendous': 1, 'consequences': 6, 'iransingles': 1, 'womanseth': 1, 'laueri': 1, 'gracehuckabee': 1, 'decries': 2, 'obamacares': 4, 'crosseyed': 1, 'mulereport': 1, '87': 6, 'orgasm': 2, 'fantasizing': 1, 'gortons': 1, 'fishermanpharmas': 1, 'shortagestwitter': 1, 'gif': 1, 'statementmedia': 1, 'rid': 6, '24yearold': 1, 'employeeevery': 1, 'orphaned': 2, 'guinea': 2, 'home87yearold': 1, 'performing': 8, 'businessalbuquerque': 1, 'chargesstudy': 1, 'congressperson': 1, 'utterly': 1, 'entwined': 1, 'canapparently': 1, 'officegopled': 1, 'dems': 6, 'sitin': 1, 'zikareceding': 1, 'floodwaters': 1, 'extent': 2, 'areas': 3, 'postharvey': 1, 'destructionstop': 1, 'presidentrelationship': 1, 'goingtobathroomwithdooropen': 1, 'stagethe': 2, 'memorable': 2, 'onscreen': 2, 'weddings450pound': 1, 'lecturejudge': 1, 'accusing': 2, 'dealings': 1, 'violating': 4, 'constitutiondad': 1, '350000': 1, 'afford': 11, 'travelkim': 1, 'cattrall': 2, 'curt': 3, 'cynthia': 3, 'runomg': 1, 'duchess': 4, 'vogue': 3, 'uks': 3, 'june': 4, 'starwest': 1, 'fed': 4, 'strikedemocrats': 1, 'hearingsriz': 1, 'ahmeds': 1, 'emmy': 6, 'representation': 3, 'tvhow': 1, 'wicked': 1, 'wanderlust': 1, 'lifenearly': 1, 'injury': 7, 'studymarc': 1, 'mashed': 2, 'potatoesarea': 1, 'rental': 2, 'carsweeping': 1, 'reforms': 5, 'foxconn': 1, 'inhumane': 3, 'homemegachurch': 1, 'pastor': 1, 'abruptly': 3, 'improper': 1, 'publiccomiccon': 1, 'superhero': 3, 'flyovera': 1, 'capsule': 2, 'usman': 1, 'pronounceman': 1, 'pynchon': 1, 'pains': 1, 'visibleheres': 1, 'worldcan': 1, 'spatial': 1, 'writingthe': 1, 'fracturer': 1, 'exceptional': 1, 'smasher': 1, 'indispensable': 1, 'fragmenterufc': 1, 'jon': 6, 'hitandrun': 1, 'involving': 2, 'womanteen': 1, 'anxious': 6, 'cigarette': 6, 'indad': 2, 'lightup': 1, 'shit1984': 1, 'spike': 4, 'conways': 1, 'orwellian': 1, 'interviewstudy': 1, 'swing': 4, 'occurs': 3, 'afterwardsholiday': 1, 'shortwatch': 1, 'bikers': 1, 'naturebertha': 1, 'cceres': 1, 'assassination': 2, 'anotherkentucky': 1, 'dmv': 1, 'chicken': 25, 'testshingles': 1, 'sufferer': 2, 'shingles': 1, 'isenjoy': 1, 'sharknado': 1, 'chilis': 2, 'dcacknowledge': 1, 'onit': 1, 'daddy': 6, 'workcalifornia': 2, 'extend': 4, 'immigrantspederast': 1, 'adultcamp': 1, 'refusal': 3, 'realityunfinished': 1, 'basement': 1, 'iteven': 1, 'jessica': 3, 'permissionpublicist': 1, 'komodo': 1, 'dragon': 2, 'skyfall': 1, 'pregnanta': 1, 'cheer': 6, 'nationalismpossum': 1, 'gazes': 1, 'longingly': 3, 'dogmiddleaged': 1, 'forth': 1, 'homeseveryone': 1, 'attending': 4, 'roombaltimore': 1, 'rawlingsblake': 1, 'reelectionwhat': 1, 'docollege': 1, 'experiences': 8, 'tantalizing': 2, 'taste': 13, 'burrito': 3, 'tablebush': 1, 'invaded': 4, 'iraqreport': 1, 'outofcourt': 1, 'settlementapples': 1, 'safari': 3, 'browser': 2, 'reportcontent': 1, 'awardee': 1, 'nic': 1, 'mccarthyleann': 1, 'rimes': 1, 'pantsawkward': 1, 'mistaken': 7, 'tensionhalloween': 1, 'firefighter': 4, 'uniform20': 1, 'wardrobe5': 1, 'huckabeesee': 1, 'miley': 1, 'hilary': 2, 'duffnations': 1, 'degenerates': 2, 'impatient': 1, 'slippery': 4, 'slope': 5, 'inthis': 1, 'powerfully': 1, 'towns': 5, 'brazilthe': 2, 'europe': 11, 'proudly': 7, 'smallbeijing': 1, 'extinguishes': 1, 'fivealarm': 1, 'cloud': 4, 'smoglike': 1, 'rooting': 1, 'blackcountry': 1, 'jana': 1, 'kramer': 1, 'jolie': 3, 'raeas': 1, 'certified': 2, 'barks': 1, 'electoral': 5, 'colliesons': 1, 'nosebleedsweeping': 1, 'willow': 1, 'bangsman': 1, 'scam': 1, '1945im': 1, 'nail': 4, 'itwhats': 1, 'pamela': 1, 'againcongress': 1, 'falling': 11, 'isiss': 1, 'trap': 7, 'refugeesnba': 1, 'prayforpaulgeorge': 1, 'injurycollege': 1, 'footballs': 3, 'heisman': 1, 'trophy': 3, 'hopefulslawsuit': 1, 'inciting': 1, 'judgethis': 1, 'cfl': 1, 'unsportsmanlike': 1, 'levelthe': 1, 'dadwhat': 1, 'bothers': 1, 'pro': 8, 'dangergroom': 1, 'radiant': 1, 'daymissing': 1, 'vet': 1, 'strangled': 2, 'belize': 1, 'reportsresearchers': 1, 'chuckling': 1, 'groupbank': 1, 'mergertrump': 1, 'intelligence': 13, 'agencies': 1, 'briefinghr': 1, 'workrelatedthe': 1, 'roomfollowing': 1, 'capitol': 9, 'hill8': 1, 'understandkatherine': 1, 'heigl': 1, 'intend': 2, 'difficultfar': 1, 'galaxy': 8, 'starwarschristmascarolsimportant': 1, 'highest': 6, 'idiotline': 1, 'lizards': 1, 'geographic': 3, 'officerose': 1, 'byrne': 3, 'cannavalebusy': 1, 'mel': 2, 'kiper': 1, 'spreadsheet': 1, 'daughterjack': 1, 'lew': 2, 'salestrump': 1, 'waterboarding': 2, 'torment': 1, 'momentcorporation': 1, 'origins': 2, 'economysoftball': 1, 'console': 3, 'jackass': 1, 'captain': 7, 'outtoddler': 1, 'determine': 11, 'peas': 3, 'ear': 4, 'canalscarlett': 1, 'johansson': 1, 'heartwarming': 6, 'studentimpressive': 1, 'nails': 12, 'skateboard': 3, 'trick': 7, 'strollerpanicking': 1, 'lay': 5, 'moderatortrumps': 1, 'saysfrom': 1, 'chewing': 3, 'cherrytomato': 1, 'senses': 2, 'peevestwentysomething': 1, '35an': 1, 'incomplete': 1, 'denzel': 1, 'equalizerheroic': 1, 'prego': 2, 'advertisement': 1, 'replaces': 5, 'refreshed': 3, 'webpages': 1, 'bannerarea': 2, 'onceinalifetime': 1, 'lovepeoplewatcher': 1, 'rare': 17, 'doofusthe': 1, 'defiant': 2, 'dre': 2, 'iovine': 1, 'yeah': 2, 'moneytelevision': 1, 'upcoming': 14, 'reunionoscars': 1, 'cower': 1, 'awe': 2, 'disembodied': 1, 'allknowing': 2, 'proclaims': 4, 'nomineeschris': 1, 'dances': 1, 'roomhuffpollster': 1, 'base': 9, 'sticking': 2, 'himlove': 1, 'result': 4, 'handtracy': 1, 'morgan': 7, 'conditionfda': 1, 'anxietybad': 1, 'appointment': 2, 'shia': 2, 'militiaman': 1, 'iraqi': 5, 'cabinettennessee': 1, 'processwhy': 1, 'disappear': 5, 'matterman': 1, 'anxiously': 3, 'scanning': 4, 'reaction': 9, 'jukebox': 1, 'selectionthis': 1, 'messed': 1, 'pushedworlds': 1, 'upbruce': 1, 'springsteen': 6, 'stance': 4, 'banthe': 2, 'fidel': 4, 'castrorwandan': 1, 'angered': 3, 'aol': 3, 'accesspoll': 1, 'standing': 17, 'boothproduce': 1, 'bursts': 4, 'laughter': 2, 'ferrell': 2, 'applesthis': 1, 'differenthorrified': 1, 'pope': 23, 'humanitys': 2, 'sin': 3, 'godparents': 1, '2yearold': 4, '911new': 1, 'regulation': 4, 'actively': 1, 'habitat': 2, 'fundingseth': 1, 'rips': 12, 'sessions': 11, 'halting': 1, 'partycongresswoman': 1, 'fights': 8, 'violencethe': 2, 'storms': 5, 'batters': 2, 'northwestguinness': 1, 'abridged': 1, 'freaks': 2, 'readers': 6, 'stuffwall': 1, 'liveferguson': 1, 'boycott': 4, 'fridaymilk': 1, 'jug': 2, 'liferare': 1, 'frog': 3, 'cure': 7, 'toah': 1, 'extinctthis': 1, 'alice': 2, 'wonderland': 1, 'rabbit': 3, 'holelonging': 1, 'dreamslearning': 1, 'resilience': 3, 'clintonmichaela': 1, 'watkins': 1, 'femaledriven': 1, 'showsjournalist': 1, 'sources': 10, 'institutionsobama': 1, 'procter': 1, 'gamble': 2, 'va': 6, 'secretarysamsung': 1, '7the': 1, 'planwhole': 1, 'paintingssecrets': 1, 'earthreply': 1, 'inboxeschuck': 1, 'grassley': 2, 'events': 6, 'protesterschilis': 1, 'customer': 16, 'ribs': 1, 'platter': 2, 'complimentary': 3, 'hosedownchristie': 1, 'describes': 3, 'grave': 7, 'towering': 1, 'meaty': 1, 'staring': 6, 'diner': 2, 'corned': 1, 'sandwichwatch': 1, 'weightlifter': 2, 'bronze': 3, 'backflipif': 1, 'progun': 1, 'treats': 10, 'shootingsbest': 1, 'hotels': 6, 'large': 13, 'familieshere': 1, 'livesnew': 1, 'games': 20, 'mockingjay': 1, 'posters': 1, 'battlepair': 1, '26yearolds': 1, 'loans': 3, 'bankin': 1, 'russiagate': 2, 'pencewhat': 1, 'youthlana': 1, 'stevie': 2, 'nicks': 4, 'spell': 3, 'albumprince': 1, 'william': 3, 'manarea': 2, 'exposing': 4, 'belly': 2, 'slap': 2, 'wayremote': 1, 'volleyball': 1, 'endures': 1, 'landing': 2, 'en': 1, 'route': 3, 'tournamentrepublican': 1, 'clues': 1, 'upwoman': 1, 'sow': 2, 'seeds': 3, 'gchatsin': 1, 'landingreport': 1, 'happygop': 1, 'medical': 20, 'repealedhypothetical': 1, 'hypotheticaltexas': 1, 'executionrsum': 1, 'font': 2, 'offends': 4, 'employerthese': 1, 'brave': 2, 'cravingsfacebook': 1, 'bolts': 1, 'newsdemocrats': 1, 'inch': 1, 'closer': 12, 'completionjimmy': 1, 'o': 3, 'yang': 1, 'silicon': 4, 'asians': 3, 'hunks': 1, 'tooworker': 1, 'machinedemocratic': 1, 'resigna': 1, 'beans': 4, 'suckerschicagos': 1, 'promising': 3, 'decadeswoman': 1, 'rearranging': 1, 'rankings': 2, 'giftsarea': 1, 'forks': 1, 'thistheres': 1, 'didexcerpt': 1, 'landscape': 3, 'heartlandhuman': 1, 'snazzy': 1, 'feline': 1, 'impressedmutual': 1, 'processcats': 1, 'whiskers': 1, 'muchbill': 1, 'sham': 2, 'timehigh': 1, 'fuckup': 2, 'charge': 6, 'luggage': 1, 'explosivesnew': 1, 'sitcom': 4, 'envelopekerry': 1, 'downs': 3, 'vodka': 1, 'putins': 3, 'detail': 9, 'outmonica': 1, 'lewinsky': 1, 'anymore': 8, 'movementjudge': 1, 'disclose': 8, 'donation': 7, 'defendant': 3, 'gerrymandering': 1, 'suitarea': 2, 'twentysomething': 2, 'disillusioned': 2, 'disillusionmentmariachi': 1, 'dieda': 1, 'chiefreport': 1, 'peoples': 13, 'ghostcongressman': 1, 'shootingnations': 1, 'ulcers': 1, 'unlimited': 3, 'growth': 5, '2017here': 1, 'months': 31, 'drugseveryone': 1, 'billing': 3, 'timefire': 1, 'fireman': 2, 'firelive': 1, 'coverage': 10, 'midterm': 4, 'inelizabeth': 1, 'loserwearable': 1, 'technology': 13, 'healthcareepa': 1, 'rivers': 6, 'shitstealth': 1, 'houdini': 1, 'roomyou': 1, 'reportsbarack': 1, 'vetoes': 3, 'arabiapegida': 1, 'photonews': 1, '22': 2, '2017i': 1, 'diemom': 1, 'produces': 3, 'decorative': 2, 'thin': 4, 'airis': 1, 'beinghow': 1, 'stay': 20, 'accessoriesbritain': 1, 'impose': 3, 'toughest': 2, 'bansthese': 1, 'influential': 3, 'defy': 1, 'stoner': 2, 'stereotypeleaving': 1, 'elections': 7, 'option4': 1, 'habits': 12, 'haveprice': 1, 'expletives': 1, 'gallonheres': 1, 'likethis': 2, 'britney': 8, 'kinds': 4, 'sultryinsufferable': 1, 'prick': 2, 'distinctly': 1, 'cilantroheidi': 1, 'nastiness': 1, 'backlashwhy': 1, 'bydepartment': 1, 'prepared': 4, 'musicalus': 1, 'citizensyounger': 1, 'siblings': 5, 'healthgiuliana': 1, 'rancic': 1, 'cheating': 3, 'jerry': 8, 'oconnell': 1, 'goodlena': 1, 'serena': 3, 'ronda': 2, 'rouseyconceptual': 1, 'slowmotion': 3, 'footage': 10, 'syrup': 3, 'cascading': 1, 'pancakeshighculture': 1, 'operapaul': 1, 'billionaire': 6, 'rubioeric': 1, 'stonestreet': 1, 'sarah': 8, 'toast': 4, '200th': 1, 'kissthe': 1, 'newlyweds': 2, 'faceneighborhood': 1, 'affordnew': 1, 'works': 14, '2017lone': 1, 'bystander': 2, 'targeted': 5, 'pack': 9, 'takenarizona': 1, 'destroys': 6, 'homes': 8, 'raising': 11, 'seasonfive': 1, 'diagnosed': 5, 'diseasetrump': 1, 'cement': 2, 'guantanamos': 1, 'prisonman': 1, 'backedup': 1, 'shower': 7, 'drain': 4, 'soak5': 1, 'episodes': 3, 'hanukkah': 1, 'tooarea': 2, 'certificaterose': 1, 'moira': 1, 'mactaggert': 1, 'xmen': 2, 'apocalypsegraffiti': 1, 'miami': 5, 'walltowall': 1, 'makeoverequine': 1, 'haven': 2, 'neglected': 4, 'abandoned': 11, 'horseswe': 1, 'uscity': 1, 'concrete': 6, 'stairsebola': 1, 'sierra': 2, 'reminded': 6, 'zonepaul': 1, 'soulcycle': 1, 'instructors': 1, 'speaker': 10, 'role5000': 1, 'jackolanterns': 1, 'stun': 3, 'haunting': 5, 'displaytop': 1, 'bloomformerly': 1, 'pantsa': 1, 'refinancing': 1, 'loansa': 1, 'apartheid': 2, 'onexperts': 1, 'glorious': 5, 'campaignsalcoholic': 1, 'forcourt': 1, 'lawsuitmiley': 1, 'genderbending': 1, 'maya': 1, 'marty': 1, 'perfectbreaking': 1, 'deadactivists': 1, 'sacramento': 1, 'stephon': 2, 'shootinggod': 1, 'creation': 2, 'notimage': 1, 'substance': 1, 'selfmade': 1, 'journeyemerson': 1, 'lake': 6, 'palmer': 2, 'greg': 2, '69divergent': 1, 'views': 6, 'assemblyrobert': 1, 'reich': 1, 'petty': 4, 'vindictive': 1, 'tweetspainful': 1, 'birthdayafter': 1, 'fifthgrader': 3, 'bullied': 4, 'himus': 1, 'yearny': 1, 'columnist': 1, 'explores': 6, 'virtue': 1, 'bookfreak': 1, 'hots': 1, 'populargirl': 1, 'reportconvention': 1, 'joke': 14, 'looksweird': 1, 'dadsincreasingly': 1, 'confirmation': 3, 'comea': 1, 'microwave': 3, 'spectrum': 2, 'disorderit': 1, 'identitybear': 1, 'emerges': 7, 'hibernation': 1, 'killat': 1, 'toronto': 3, 'quieter': 1, 'performances': 1, 'loudest60': 1, 'targeting': 7, 'international': 14, 'risesstudy': 1, 'disorderwhat': 1, 'preexisting': 1, 'condition': 5, 'anywayfollowing': 1, 'ray': 9, 'bradburys': 1, 'kindle': 1, 'demise': 2, 'bookspriests': 1, 'thoughtwhite': 1, 'gaza': 6, 'hamas': 5, 'propagandathere': 1, 'sighs': 1, 'devoted': 3, 'topicthe': 1, 'herehow': 1, 'waste': 13, 'renewable': 3, 'energycapitol': 1, 'dome': 1, 'deflatespixies': 1, 'dayunsurprisingly': 1, 'impressed': 8, 'conferencekim': 1, 'honest': 4, 'kanyesouth': 1, 'wararea': 1, 'sworn': 4, 'randy': 1, 'kotter': 3, 'themebet': 1, 'gal': 1, 'gadot': 2, 'pronounced': 1, 'tflorida': 1, 'aggressive': 2, 'reportingmike': 1, 'tyson': 4, 'drake': 5, 'bling': 2, 'mememotor': 1, 'stripped': 2, 'appearing': 2, 'rod': 1, 'centerfoldborrowed': 1, 'cd': 5, 'integrated': 4, 'collectiontrain': 1, 'elderly': 10, 'tiradethe': 1, 'awkwardness': 1, 'angst': 1, 'aikido': 1, 'anythingadele': 1, 'albumjustin': 1, 'herthe': 1, 'grief': 4, 'embracing': 3, 'lifefreed': 1, 'prisoner': 4, 'realwhy': 1, 'againjonathan': 1, 'franzen': 1, 'weiner': 4, 'againparis': 1, 'jackson': 11, 'expectationsferguson': 1, 'sufficiently': 1, 'inclinedsorry': 1, 'marvel': 6, 'netflixseaworld': 1, 'dynamites': 1, 'orca': 4, 'itself': 16, 'walkwaywoman': 1, 'bossmy': 1, 'gradsfriend': 1, 'hulkman': 1, 'amusement': 2, 'funnel': 1, 'cakeredemption': 1, 'chatting': 1, 'paxton': 2, 'sexton': 1, 'chadwick': 1, 'stokes': 1, 'erik': 1, 'deutschsamantha': 1, 'trumpwhats': 1, 'bullsboss': 1, 'alludes': 1, 'crunch': 5, 'timenewsweek': 1, 'abc': 3, 'expose': 5, 'torture': 12, 'troubled': 4, 'teensrand': 1, 'daylong': 1, 'nsa': 5, 'filibusterpanicked': 1, 'deleting': 3, 'firefox': 1, 'desktopnations': 1, 'uptrump': 3, 'tirade': 2, 'occasionknowshon': 1, 'moreno': 1, 'broncos': 3, 'besides': 3, 'gatoradespooked': 1, 'headquarterslingeriewearing': 1, 'housenra': 1, 'shootings': 4, 'unfortunate': 2, 'commit': 4, 'shootingsmccain': 1, 'jcpenney': 2, 'catalogmodeling': 1, 'jobwhat': 1, 'dreamers': 7, 'nowman': 1, 'chain': 7, 'buglesdemocratic': 1, 'shaping': 3, 'platformcharlottesville': 1, 'amend': 2, 'opencarry': 1, 'lawsgreatest': 1, 'gimmicky': 1, 'deus': 1, 'machina': 1, 'endingisraeli': 1, 'along': 11, 'officialspierce': 1, 'brosnan': 1, 'offended': 4, 'gungod': 1, 'childrengeico': 1, 'discontinuing': 1, 'advertisingjerked': 1, 'yearsfor': 1, 'fulfilling': 2, 'lifelong': 4, 'iowaa': 1, 'pets': 5, 'breakupanderson': 1, 'cooper': 11, 'informs': 9, 'significant': 4, 'day300': 1, 'india': 8, 'restoration': 1, 'gridtaylor': 1, 'ago': 3, 'folkssecrets': 1, 'present': 4, 'purchaserdarren': 1, 'wilson': 8, 'momprofiles': 1, 'courage': 4, 'sometimes': 3, 'thinkemployee': 1, 'daring': 7, '330': 1, 'officeaclu': 1, 'fraud': 7, 'probebreaking': 1, 'mph': 5, 'slick': 2, 'invinciblethis': 1, 'fitness': 6, 'heartnewly': 1, 'journal': 7, 'entries': 1, 'sacagaweas': 1, 'clarki': 1, 'stumble': 1, 'haveinterior': 1, 'aims': 16, 'slice': 3, 'endangered': 8, 'actreid': 1, 'looming': 5, 'shutdowna': 1, 'inspiration': 6, 'katwe': 1, 'youthannual': 1, 'teeth': 7, 'unnoticed': 1, 'rows': 2, 'teethlamelo': 1, 'scores': 6, '92': 5, 'gamesexually': 1, 'transmitted': 1, 'highlights': 10, 'brazils': 2, 'rampant': 2, 'inequalityinternets': 1, 'hooking': 1, 'girllonglost': 1, 'salamander': 1, 'rediscovered': 1, 'guatemalan': 1, 'junglebill': 1, 'stampsnations': 1, 'fourthgraders': 1, 'trail': 4, 'fifthgradersamerican': 1, 'fearful': 2, 'america45minute': 1, 'greatrex': 1, 'ouster': 1, 'laughablewaiting': 1, 'meeting4': 1, 'shaking': 4, 'cardscleveland': 1, 'placeare': 1, 'madefromscratch': 1, 'shops': 2, 'soclinton': 1, 'commissions': 3, 'compose': 2, 'themeexhausted': 1, 'lincoln': 5, 'lawyerthe': 1, 'misleading': 7, 'claimsformer': 1, 'sniper': 9, 'slapped': 1, '3000yard': 1, 'restraining': 1, 'orderangelina': 1, 'babyhasbro': 1, 'concedes': 3, 'rubiks': 1, 'chicken3': 1, 'broadband': 1, 'improving': 4, 'educationcat': 1, 'stevens': 3, 'jihad': 3, 'taylorsubsidiary': 1, 'publication': 1, 'corporations': 7, 'moviethese': 1, 'shelters': 1, 'refugeevery': 1, 'conceivable': 1, 'nook': 1, 'stuffed': 5, 'triparea': 1, 'heath': 1, 'ledger': 1, 'likelesbian': 1, 'exmayor': 2, 'coulters': 1, 'nonsensethe': 1, 'standout': 2, 'carpetelizabeth': 1, 'glasssteagall': 1, 'statements': 6, 'secretarydont': 1, 'khloe': 1, 'sisterwoman': 1, 'jumps': 6, 'suv': 3, 'stabs': 2, 'sayman': 1, 'womanlets': 1, 'resignman': 1, 'wring': 1, 'ounce': 2, 'relaxation': 1, 'vacationlaura': 1, 'penned': 2, 'writerhuffpost': 1, '600': 5, 'shovels': 1, 'lesbians': 3, 'allpueblo': 1, 'indians': 5, 'pace': 5, 'appetite': 2, 'earthenwaregrieving': 1, 'bringing': 12, 'diedmusings': 1, 'museumkate': 1, 'middleton': 3, 'alonehandsoff': 1, 'psychological': 4, 'issues3': 1, 'tripboy': 2, 'deputies': 3, 'dinnerfans': 1, 'erupt': 1, 'elias': 1, 'koteas': 1, 'olinsky': 1, 'pdpets': 1, 'messageddvd': 1, 'unseen': 1, 'moviedaily': 1, 'driverschinese': 1, 'medicinal': 1, 'bile16': 1, 'fashionable': 1, 'sneakers': 1, 'workmental': 1, 'determined': 10, 'stigma': 2, 'define': 4, 'itall': 1, 'faa': 2, 'meant': 7, 'flyzac': 1, 'efrons': 1, 'absmeeting': 1, 'coddling': 1, 'regulationsnfl': 1, 'pregame': 1, 'ceremony': 12, 'retired': 5, '52yearold': 1, 'cornerback': 1, 'playerbarbra': 1, 'dumpreclaiming': 1, 'sacred': 1, 'principlesthe': 1, 'syndromedad': 1, '2150': 1, 'documentarieskate': 1, 'pipes': 2, 'princes': 3, 'umother': 1, 'preschool': 1, 'focuses': 3, 'sondear': 1, 'combating': 3, 'racismtake': 1, 'yellowstone': 1, 'wildest': 2, 'placesthe': 1, 'proposali': 1, 'slurs': 2, 'timewacky': 1, 'forensics': 1, 'autopsyturvywhen': 1, 'entrepreneurship': 1, 'overlapstephen': 1, 'baldwins': 3, 'assistant': 8, 'baldwinnorth': 1, 'normalcy': 1, 'synchronized': 1, 'disco': 1, 'jumprope': 1, 'galaresisting': 1, 'malariareport': 1, 'countless': 1, 'invasive': 3, 'sitesjohn': 1, 'rome': 4, 'blamepathetic': 1, 'merrygoround': 1, 'rideclinton': 1, 'treasurer': 2, 'crushed': 3, 'stack': 1, 'overkendrick': 1, 'deletes': 2, 'rhymezonecom': 1, 'charlottesvillesnoop': 1, 'crip': 1, 'livingnew': 1, 'programsprecocious': 1, 'writegrey': 1, 'parrot': 1, 'aviary': 2, 'idiotsperson': 1, 'interest': 15, 'firebombedhot': 1, 'lingered': 1, 'onbuilding': 1, 'torrential': 1, 'rains': 3, 'mumbaiteen': 1, 'birthright': 1, 'hadnt': 2, 'palestinianstoby': 1, 'keith': 2, 'rhyme': 2, 'ahmadinejad6': 1, 'rooms': 2, 'tile': 1, 'backsplashparamount': 1, 'feature': 12, 'presentationsaturdays': 1, 'funnies': 2, 'editionnervous': 1, 'nsfwcolberts': 1, 'allday': 1, 'prophecies': 1, 'truemelania': 1, 'doesyour': 1, 'youamazon': 1, 'saleintegrating': 1, 'roma': 1, 'withinreport': 1, 'warehouse': 4, 'zone': 2, 'askedgreat': 1, 'reef': 4, 'coral': 4, 'dieoffmichele': 1, 'bachmann': 3, 'shootingbefore': 1, 'yearheres': 1, 'dumb': 10, 'rule12': 1, 'functioning': 2, 'downdoctor': 1, 'contracted': 2, 'prep': 8, 'meanjoe': 1, 'scarborough': 3, 'rob': 5, 'porter': 3, 'allegationsnytimescoms': 1, 'goods': 3, 'moveiman': 1, 'shumpert': 1, 'headphones': 4, 'involvedguy': 1, 'combos': 2, 'threemonth': 1, 'periodwife': 1, 'unfazed': 1, 'womencheney': 2, 'listening': 12, 'phone9': 1, 'brands': 2, 'spokesmodels': 1, '50prince': 1, 'pot': 12, '420': 1, 'prisonjon': 1, 'hamm': 3, 'overenthusiastic': 1, 'everyonewhy': 1, 'smartest': 1, 'refundtravelling': 1, 'feminine': 1, 'mystique': 1, 'feminismthe': 1, 'gilmore': 11, 'griefbreweries': 1, '205000': 1, 'cans': 2, 'victimsactivists': 1, 'warplanes': 1, 'campkim': 1, 'selfie4': 1, 'dresses': 5, 'women9': 1, 'according': 23, 'parentsitunes': 1, 'copyright': 2, 'lawrussian': 1, 'repeatedly': 12, 'informing': 1, 'kremlin': 2, 'campaigns': 6, 'ineptitudethe': 1, 'bod': 1, 'vacationbotanists': 1, 'trees': 3, 'strangle': 1, 'otherno': 1, 'velvet': 1, 'rope': 1, 'abroadman': 1, 'welcomed': 1, 'tvanother': 1, 'trumpian': 2, 'contender': 2, 'links': 10, 'shootingthe': 3, 'cyberspace': 1, 'itrepublicans': 1, 'inequality': 14, 'tipearthquake': 1, 'planswhy': 1, 'grenade': 1, 'launchersno': 1, 'shave': 4, 'crowdfunding': 2, 'hairmeet': 1, '28yearold': 1, 'pave': 2, 'germanytrump': 1, 'golf': 9, 'effects': 12, 'changebruce': 1, 'crashcoach': 1, 'trophyimpulse': 1, 'ep9dnc': 1, 'debbie': 2, 'wasserman': 2, 'schultz': 1, 'payday': 2, 'lenderstrump': 1, 'radicalizing': 1, 'neededpastor': 1, 'ripped': 4, 'towed': 1, 'truckreport': 1, 'concussionsdelta': 1, 'resumes': 4, 'systems': 3, 'outageteen': 1, 'potthink': 1, 'pynkman': 1, 'compliment': 2, 'whose': 13, 'helpman': 1, 'treasured': 1, 'duck': 4, 'rabbitwoman': 1, 'heinous': 1, 'crimes': 8, 'dogvacationing': 1, 'assholethe': 2, 'evangelicalismman': 1, 'orderedsaudi': 1, 'executioner': 1, 'shoulder': 4, 'decapitationscvs': 1, 'cashier': 7, 'cough': 2, 'medicationstrumps': 1, 'hinder': 1, 'healthebola': 1, 'stupidantiimmigrant': 1, 'sanctuaryformer': 1, 'mouseketeer': 1, 'marque': 1, 'tate': 3, 'lynche': 1, '34elderly': 1, 'frequently': 1, 'funeralsthese': 1, 'creepyvoice': 1, 'cheering': 6, 'rebels': 12, 'oh': 3, 'whatthis': 1, 'funky': 1, 'hata': 1, 'bathroomsdepartment': 1, 'interior': 11, 'embezzling': 1, '50000': 7, 'wolvesdon': 1, 'blankenships': 1, 'relief': 11, 'upper': 3, 'branch': 3, 'minersman': 1, 'dives': 1, 'haphazardly': 1, 'wounded': 5, 'ospreyclinton': 1, 'soldbrian': 1, 'todayuse': 1, 'keyboard': 5, 'shortcutsarea': 1, 'recommended': 3, 'caloric': 1, 'intake': 2, 'exceeded': 1, 'amthe': 1, 'journalism': 3, 'mostfox': 1, 'cancel': 7, 'assassinated': 1, 'specialtop': 1, 'changesis': 1, 'fda': 5, 'pillschris': 1, 'hughes': 2, 'republic': 4, 'saledecision': 1, '2000': 7, 'smokefilled': 1, '1997new': 1, 'lifestyle': 1, 'forget': 7, 'itmy': 1, 'disastrous': 4, 'swimsuitlights': 1, 'li': 2, 'wordbehind': 1, 'thiels': 2, 'gawkerrichard': 1, 'griecos': 1, 'goodwillthese': 1, 'shortcuts': 1, 'timelifetime': 1, 'toni': 1, 'braxtons': 1, 'unbreak': 2, 'heartscientists': 1, 'trace': 2, 'campuswide': 1, 'pussy': 3, 'shortage': 4, 'zbt': 1, 'houseguy': 1, 'neck': 7, 'rebootgregg': 1, 'popovich': 1, 'throttle': 1, 'soulless': 1, 'trumpstress': 1, '2chattanooga': 1, 'obtained': 2, 'legally': 3, 'officialschristian': 1, 'lifefamily': 2, 'interrupt': 2, 'skimming': 1, 'orioles': 1, 'radio': 14, 'broadcastecofriendly': 1, 'junkies': 1, 'needle': 2, 'reuse': 1, 'programjames': 1, 'eyeballs': 1, 'springsfingerprints': 1, 'hopefully': 3, 'menstrual': 2, 'bloodhow': 1, 'repealing': 2, 'extrahardno': 1, 'sought': 3, 'assholes': 3, 'murdertrumps': 1, 'revised': 4, 'mired': 1, 'prejudicenew': 1, 'currency': 2, 'expires': 2, 'weeksstruggling': 1, 'footballnah': 1, 'kidding': 1, 'ishow': 1, 'lighten': 2, 'lossmikhail': 1, 'gorbachev': 1, 'nato': 3, 'escalating': 1, 'one10': 1, 'businessiranian': 1, 'scientist': 8, 'annoyed': 7, 'weapons18': 1, 'glassesinnercity': 1, 'earns': 3, 'ged': 1, '11trump': 1, 'murdered': 5, 'ambassadorelderly': 1, 'teeter': 1, 'quiveranother': 1, 'pufferfish': 1, 'friendlessthe': 1, 'several': 13, 'pastry': 4, 'entenmanns': 1, 'bakerysurvey': 1, 'genital': 4, 'stimulation': 1, 'maintains': 4, 'popularitydan': 1, 'fogelberg': 1, 'soothe': 1, 'lite': 1, '108': 1, 'listener5': 1, 'ideaswhole': 1, 'foods': 7, 'vegetable': 1, 'symbolnew': 1, 'confidence': 8, 'otheramazing': 1, 'discovery': 6, 'fishoil': 1, 'consumption': 4, 'lifeblack': 1, 'objectgod': 1, 'raspberries': 1, 'berriliciousbill': 1, 'resting': 1, 'upright': 1, 'debateexclusive': 1, 'ginger': 1, 'minjs': 1, 'premierehero': 1, 'stops': 20, 'statepreparing': 1, 'divorcepeta': 1, 'sat': 9, 'chimpanzees38': 1, 'toback': 2, 'misconductsurge': 1, 'once4': 1, 'caffeine': 1, 'potentialorlando': 1, 'colon': 1, 'tragedyi': 1, 'screamman': 1, 'trekthemed': 1, 'cabin': 2, 'junkpregnant': 1, 'terminate': 1, 'delay': 6, 'treatmentfor': 1, 'countstlcs': 1, 'damaging': 5, 'mormons': 1, 'especially': 5, 'mormon': 3, 'youthjake': 1, 'tappers': 1, 'grim': 2, 'bannon': 15, 'isflaming': 1, '314sabra': 1, 'hummus': 3, 'cedars': 1, 'necessary': 6, 'hummuswhy': 1, 'signfriday': 1, 'speakers': 4, 'problemthings': 1, 'babychristian': 1, 'roomstephen': 1, 'math': 5, 'promised': 9, 'wallsaucespatter': 1, 'investigators': 2, 'reconstruct': 2, 'grisly': 2, 'meatball': 2, 'subscientists': 1, 'charismaarcade': 1, 'bon': 1, 'iver': 1, 'strokes': 2, 'supergroup': 1, 'onlyunpublished': 1, 'twain': 2, 'bp': 1, 'afghanistansuave': 1, '20year': 2, 'leavein': 1, 'conditioner24': 1, 'empowered': 1, 'girlsunemployed': 1, 'writinghere': 1, 'wobbly': 4, 'deal24': 1, 'reactions': 2, 'alcorns': 1, 'deathtrumps': 1, 'armsparents': 1, 'peeved': 1, 'hatchimals': 2, 'cursing': 1, 'stormreport': 1, 'guncampus': 1, 'guides': 4, 'hallegyptian': 1, 'bassem': 1, 'youssef': 1, 'kitus': 1, 'appoints': 6, 'leatherclad': 1, 'postmistress': 1, 'generalthe': 1, 'thrush': 1, 'misconduct': 14, 'claimsmansons': 1, 'utter': 5, 'chaos': 6, 'griefamerican': 1, 'airways': 2, 'inconveniencemore': 1, 'plansex': 1, 'embellished': 1, 'visitpaul': 1, 'giamatti': 3, 'murdereven': 1, 'favor': 9, 'optimistic': 4, 'itcam': 1, 'foreheadscottish': 1, 'togetherwatch': 1, 'drifts': 1, 'match': 8, 'freight': 1, 'train4': 1, 'mediterranean': 4, 'foodman': 1, 'leg': 6, 'whale': 6, 'gonarcissist': 1, 'organsgorsuch': 1, 'paybackaid': 1, 'crisishistorical': 1, 'recipeamazing': 1, 'bidding': 1, 'aretha': 1, 'franklin': 5, 'resolveddeadpool': 1, 'josh': 3, 'brolins': 1, 'xciting': 1, 'mutantsman': 1, 'carabiner': 1, 'rappel': 1, 'morningauthority': 1, 'thissubway': 1, 'spokespersonfisherprice': 1, 'choke': 2, 'versiontrump': 1, 'lawn': 7, 'headquarters': 12, 'manafort': 3, 'custodybarack': 1, 'slumber': 2, 'voterussias': 1, 'medvedev': 1, 'powerlesson': 1, 'wagnerism': 1, 'macon': 1, 'georgia': 7, 'youvladimir': 1, 'besties': 1, 'pal': 2, 'spoofthe': 1, 'obamacarecrazed': 1, 'critically': 5, 'injures': 4, '4study': 1, 'informed': 5, 'stereotype': 2, 'chechensnations': 1, 'prospective': 2, 'applicants': 1, 'princeton': 3, 'rankingsolder': 1, 'heartmegyn': 1, 'attackstudy': 2, 'savertouring': 1, 'raffi': 1, 'sillies': 1, 'outschool': 1, 'committed': 11, 'pressures': 3, 'intro': 1, 'communicationsreda': 1, 'kateb': 1, 'hippocrates9': 1, 'middletondestruction': 1, 'cafe': 2, 'clears': 3, 'hootersfood': 1, 'iteration': 1, 'ingredient': 4, 'combinations': 1, 'completedarea': 1, 'hatnew': 1, 'pepsi': 1, 'specifically': 8, 'demographic': 4, 'namecompany': 1, 'timeconventiongoer': 1, 'tag': 4, 'vanishes': 1, 'guestseveryone': 1, 'outraged': 10, 'priest': 5, 'abouttissue': 1, 'responsibility': 6, 'tissue': 1, 'boxrising': 1, 'kerrys': 3, 'stirring': 3, 'runlocal': 1, 'muppet': 1, 'questioning': 2, 'ringsunset': 1, 'attank': 1, 'operator': 5, 'buddies': 2, 'nowvideo': 1, 'smg': 1, 'virtual': 4, 'dynamic': 1, 'storytellingfrance': 1, 'gradestom': 1, 'clancy': 1, 'expertwoman': 1, 'prejudicesjaws': 1, 'stuff': 9, 'carus': 1, 'loyal': 3, 'followingthe': 1, 'sloveniagreenspan': 1, 'repeating': 3, 'detractors': 1, 'criticisms': 2, 'highpitched': 2, 'girly': 1, 'voicenew': 2, 'brown': 18, 'revelationssunday': 1, 'roundupexclusive': 1, 'outwhy': 1, 'relevantbombing': 1, 'radical': 5, 'ragethese': 1, 'images': 9, 'impeachment': 6, 'agotrump': 1, 'odds': 2, 'communitynation': 1, 'ongoing': 2, 'grieving': 7, 'processeslouis': 1, 'annihilating': 1, 'candidacyreport': 1, 'collective': 3, 'goodwill': 1, 'tragediesafter': 1, 'fouls': 1, 'cardupdate': 1, 'halted': 2, 'narrated': 1, 'pivendonutshaped': 1, 'kitchen': 11, 'junk': 4, 'discernible': 3, 'purpose': 4, 'whatsoeverpuppy': 1, 'babyelderly': 1, 'casually': 10, 'diesarah': 1, 'byrnes': 1, 'happinessto': 1, 'namefoul': 1, 'secondlargest': 1, 'twineman': 1, 'lessfarintothefuturethe': 1, 'beverly': 1, 'hills': 2, 'dangers': 12, 'activismthe': 1, 'alias': 2, 'margaret': 1, 'atwood': 2, 'truecrime': 1, 'mysterymarriage': 1, 'divorcewatch': 1, 'malkovich': 1, 'homagewhy': 1, 'jobnasa': 1, 'voyager': 2, 'streaming': 6, 'browse': 1, 'devicein': 1, 'treatmentjapanese': 1, 'familyfaa': 1, 'installs': 3, '36000foottall': 1, 'traffic': 10, 'lightsdecades': 1, 'gma': 1, 'anniversaryhow': 1, 'axact': 1, 'pakistans': 2, 'mediaun': 1, 'tribunal': 1, 'swayed': 3, 'letters': 8, 'milosevicjoe': 1, 'walsh': 1, 'executed': 5, 'eagles': 3, 'thrillerbold': 1, 'intern': 6, 'officemoving': 1, 'problemsarsenio': 1, 'tapped': 1, 'bet': 6, 'honorsreport': 1, 'annually': 5, 'stepping': 4, 'zoneswomen': 1, 'immune': 1, 'anywhere': 4, 'olympicsdriving': 1, 'pedal': 1, 'pussyvine': 1, 'chris': 18, 'shan': 1, '6x60area': 1, 'busroute': 1, 'change16': 1, 'antiplanned': 1, 'measurewatch': 1, 'reproductive': 5, 'forefront': 1, 'wisconsin': 9, 'acaheregod': 1, '1983': 1, 'speedboat': 1, 'accident': 10, 'heavenreport': 1, '80': 9, 'donorth': 1, 'testfires': 1, 'ballistic': 2, 'defiance': 1, 'pressurewhy': 1, 'muchyou': 1, 'shout': 4, 'restaurantsteven': 1, 'nephews': 2, 'conviction': 5, 'overturnedpsychopath': 1, 'administrationhow': 2, 'relationships': 5, 'nsfwscott': 1, 'whooping': 1, 'cranes': 1, 'begin': 14, 'silently': 3, 'perching': 1, 'himthis': 2, 'gatorade': 1, 'sponsor': 1, 'himalarming': 1, 'indoorsthese': 1, 'superstars': 1, 'supermonsternational': 1, 'filmstrip': 2, 'quietgod': 1, 'wean': 1, 'religionreverend': 1, 'blessed': 2, 'nineinch': 1, 'penisreport': 1, 'brandtrade': 1, 'warnsairbnb': 1, 'campaignbest': 1, 'homewatch': 1, 'hopefuls': 3, 'iraqmorale': 1, 'firedbrutal': 1, 'threestudy': 1, 'lifetravel': 1, 'breakfasts': 2, 'england': 3, 'wadfrench': 1, 'islamcatchphrase': 1, 'guru': 1, 'overheardallamerican': 1, 'ticket': 5, 'panama': 2, 'canal': 2, 'zonewhy': 1, 'bethe': 1, 'warhols': 1, 'silver': 13, 'factorybar': 1, 'mitzvah': 2, 'mana': 1, 'uberize': 1, 'industriesprocrastinating': 1, 'reuses': 1, 'trialisis': 1, 'captive': 3, 'yazidis': 1, 'iraqanna': 1, 'deavere': 1, '44th': 1, 'lecture': 1, 'character10': 1, 'cancera': 1, 'emerging': 3, 'atlantafamily': 1, 'pulling': 12, 'plug': 5, 'unaware': 15, 'helldallas': 1, 'begging': 3, 'shotbus': 1, 'transporting': 1, 'crashes': 10, 'plantnavy': 1, 'halfmarathon': 1, '85pound': 1, 'amputeesman': 1, 'panics': 4, 'reaching': 2, 'prematurely': 1, 'familyheres': 1, 'kris': 3, 'jennerarizona': 1, 'prosecute': 2, 'terroristset': 1, 'access': 12, 'tmz': 1, 'choppers': 1, 'hovering': 3, 'gruesome': 4, 'dresshamburger': 1, 'valentinedaring': 1, 'skullthe': 1, 'timesnew': 1, 'snowden': 2, 'revelation': 1, 'spark': 3, 'turmoil': 3, 'nationsrep': 1, 'scattering': 1, 'untruebrad': 1, 'angelina': 2, 'unbroken': 1, 'premierescifi': 1, 'argue': 3, 'asyetunreleased': 1, 'filmslatest': 1, 'hungry': 4, 'falafelhollywood': 1, 'ditches': 3, 'usual': 3, 'antitrump': 8, 'rallyfederal': 1, 'predawn': 1, 'raidcormac': 1, 'mccarthy': 1, 'flaunts': 1, 'bodyjoy': 1, 'mirror': 5, 'blamelessthe': 1, 'acceptance': 8, 'koreanarea': 1, 'coughs': 1, 'others': 18, 'bathroomwoman': 1, 'geared': 2, 'complain': 5, 'sidelined': 1, 'problemschrissy': 1, 'dinnerstarring': 1, 'blockbusters': 1, 'los': 9, 'angeles': 9, 'famehas': 1, 'spiraled': 1, 'controlrecipe': 1, 'reflections': 2, 'outnumbered': 1, 'malenew': 1, 'healingworld': 1, 'earthsuper': 1, 'moneybreaking': 1, 'bias': 3, 'devin': 2, 'nunesthe': 1, 'ii': 13, 'maiden': 1, 'voyagemcdonalds': 1, 'spearmint': 1, 'afterdinner': 1, 'macburger': 1, 'kings': 4, 'taster': 1, 'deaddear': 1, 'sobnew': 1, 'countrymusic': 1, '1991': 1, 'videomost': 1, 'incompetent': 2, 'partyalcoholthemed': 1, 'successasian': 1, 'woes': 5, '700000': 1, 'starsit': 1, 'unclear': 3, 'settlingeveryone': 1, 'romantic': 3, 'prosecutiontrump': 1, '40monthlong': 1, 'postdo': 1, 'censorlets': 1, 'medicare': 3, 'expand': 6, 'themporn': 1, 'actress': 7, 'ejaculation': 1, 'facesex': 1, 'commission': 4, 'citing': 3, 'shameful': 1, 'resistancebush': 1, 'ark': 1, 'covenant': 1, 'storednew': 1, 'containercory': 1, 'kamala': 2, 'harris': 4, 'nominationis': 1, 'culturebig': 1, 'units': 3, 'restrictionschris': 1, 'farley': 2, 'cardiac': 2, 'arrestterrified': 1, 'refuge': 3, 'unhow': 1, 'breakfastwoman': 1, 'kevin': 8, 'costnerladykiller': 1, 'sentencegop': 1, 'denouncing': 1, 'statementssources': 1, 'lurking': 3, 'manmicrowaveresistant': 1, 'alarms': 1, 'scientistsdad': 1, 'freepsa': 1, 'ok': 10, 'breastfeedstudy': 1, 'yelling': 5, '76': 2, 'timegroom': 1, 'bridereal': 1, 'exhausted': 5, 'parentstwitter': 1, 'bleak': 1, 'futuristic': 2, 'trickortreatin100yearsblindfolded': 1, 'panetta': 1, 'shipped': 2, 'hazing': 3, 'ritualcalifornia': 1, 'uc': 2, 'chancellor': 4, 'resignupset': 1, 'resigh': 1, 'loudereven': 1, 'looms': 4, 'trumpbecause': 1, 'veep': 2, 'jonah': 1, '5justice': 1, 'familydeadly': 1, 'tornadoes': 2, 'rip': 3, 'midwestfood': 1, 'lasagna': 1, 'canurban': 1, 'legends': 2, 'cousins': 3, 'roommatehanukkah': 1, 'rituals': 2, 'lightsman': 1, 'threw': 7, 'upsen': 1, 'burr': 1, 'cloak': 2, 'dagger': 3, 'senatorcia': 1, 'cardssean': 1, 'questionsslain': 1, '37': 3, 'retirementits': 1, 'hayden': 1, 'jesus8': 1, 'chances': 2, 'dreamnetflix': 1, 'switches': 4, 'locationsgop': 1, 'lawfrance': 1, 'electionpoll': 1, 'disputing': 1, 'boundaries': 2, 'neighbors': 11, 'yardbudget': 1, 'splurge': 2, 'weekhigh': 1, 'staging': 1, 'walkoutreal': 1, 'twister': 1, '117un': 1, 'acquiring': 2, 'weaponsis': 1, 'allegiance': 2, 'supremacy': 5, 'greater': 7, 'godarea': 1, 'doctorstudy': 1, 'livelets': 1, 'hardworking': 2, 'families2012': 1, '2024': 2, 'wistfullyapartment': 1, 'tablefeds': 1, 'brutal': 10, 'manfighting': 1, 'ringwhy': 1, 'jeansthe': 1, 'clothing': 5, 'womencharlie': 1, 'nycadolescent': 1, 'exploring': 5, 'stepfathers': 1, 'bodyto': 1, 'survivors': 17, 'alexandria': 1, 'shooters': 8, 'familiaramy': 1, 'cam': 2, 'appearanceviacom': 1, 'extv': 1, 'siteus': 1, 'disciplined': 2, 'nickelscopy': 1, 'editors': 6, 'unhyphenated': 1, 'wordopen': 1, 'productivity': 3, '95former': 1, 'missionary': 1, 'assaultlocal': 1, 'timeheres': 1, 'french': 16, 'banner': 2, 'macronboy': 1, 'allowance': 2, 'inmatespork': 1, 'chop': 2, 'airtight': 1, 'containerthe': 1, 'yearsthis': 1, 'behindtheback': 1, 'bunt': 1, 'first16': 1, 'jj': 2, 'abrams': 6, 'tribeca': 1, 'talkmy': 1, 'victorytv': 1, 'womentuesdays': 1, 'kingpins': 1, 'bustedseinfeld': 1, 'nears': 3, 'yada': 1, 'yadawhat': 1, 'changetrump': 1, 'manbilly': 1, 'eichner': 3, 'boogied': 1, 'ellen': 7, 'detailsdead': 1, 'warthese': 1, 'likeformer': 1, 'mugabe': 1, 'deputy': 7, 'presidentdonald': 1, 'pastors': 2, 'himkentucky': 1, 'punisher': 1, 'logo': 5, 'meansbillionaire': 1, 'donates': 6, 'rats': 3, 'pooramid': 1, 'attain': 1, 'targetsmovie': 1, 'acceptable': 5, 'dadwatch': 1, 'rory': 1, 'teasernavy': 1, 'sailor': 3, 'surprises': 9, 'gamefridays': 1, 'chaosemergency': 1, 'huddlewhile': 1, 'belittles': 1, 'resistancejupiters': 1, 'liberals': 4, 'ammonia': 1, 'footprintarea': 1, 'passworddonald': 1, 'implodes': 1, 'strongest': 5, 'yetlouis': 1, 'showwaitress': 1, 'broadwayarea': 1, 'thorough': 2, 'standbyheres': 1, 'flint': 6, 'elsewherenew': 1, 'grownups': 2, 'cyberbullying': 1, 'parentsobama': 2, 'podestas': 1, 'ufo': 4, 'files': 7, 'releasedbarbershop': 1, 'outlaidoff': 1, 'designs': 3, 'offdemocalypse': 1, 'asswhuppinless': 1, 'couragedirector': 1, 'highschool': 4, 'pocketsolitute': 1, 'creek': 5, 'dynamiclos': 1, 'derails': 3, 'hurtwhats': 1, 'standpointviral': 1, 'soncocktailparty': 1, 'cornered': 1, 'joel': 3, 'steinfashion': 1, 'prabal': 1, 'gurung': 1, 'nepals': 1, 'earthquakevilsack': 1, 'reprimanded': 1, 'blogbloated': 1, 'rotund': 1, 'flints': 1, 'supplyhelen': 1, 'maroulis': 1, 'wrestlingwhen': 1, 'latinos': 9, 'highspokeswoman': 1, 'spokeschildmen': 1, 'mansplained': 1, 'skitmeet': 1, 'matteraziz': 1, 'ansari': 6, 'british': 4, 'larussell': 1, 'accusations': 5, 'notmebudget': 1, 'eternal': 2, 'yearsgirlfriends': 1, 'nightmarish': 1, 'labyrinthian': 1, 'journey': 13, 'darkest': 2, 'depraved': 1, 'desiresepa': 1, 'orangeremembering': 1, '1963this': 1, 'griffin': 5, 'iiis': 1, 'futurehead': 1, 'irs': 1, 'filing': 4, 'returnsdog': 1, 'decade': 4, 'coloradoas': 1, 'lebanon': 1, 'tunisia': 1, 'marryyourrapist': 1, 'nextthis': 1, 'tweet': 12, 'hardballlocal': 1, 'mutilationworld': 1, 'moneykeep': 1, 'beads': 1, 'bought': 6, 'manhattangoodbye': 1, 'freemanhuckabee': 1, 'fundraiser': 3, 'molasses': 1, 'crocka': 1, 'storykfc': 1, 'midas': 1, 'muchanticipated': 1, 'crossover': 1, 'mealthis': 1, 'pup': 2, 'retrieves': 1, 'bags': 6, 'directventure': 1, 'athens': 2, 'greek': 7, 'entrepreneurs': 3, 'newfound': 1, 'hopeall': 1, 'catspaula': 1, 'broadwell': 1, 'couch': 9, 'overgreece': 1, 'slight': 2, 'uptick': 3, 'arrivals': 2, 'augustthe': 1, 'moral': 5, 'couragewhy': 1, 'alis': 1, 'rebellion': 1, 'weekendsgop': 1, 'voucher': 2, 'electionunpaid': 1, 'studentathletes': 1, 'tagperverted': 1, 'parentsviral': 1, 'storysenate': 1, 'breakgop': 1, 'respondersgraphic': 1, 'deaththis': 3, 'recreating': 1, 'legendaryharry': 1, 'styles': 5, 'midpanic': 1, 'restores': 1, 'humanityglaxosmithkline': 1, 'treat': 9, 'sometimesjames': 1, 'nra': 6, 'presidentwhat': 1, 'mathincreasing': 1, 'mapbest': 1, 'abu': 3, 'dhabi': 3, 'narcicysts': 1, 'risewatch': 1, 'homosexual': 3, 'impulsesamericans': 1, 'bravely': 2, 'electing': 2, 'congressmichelle': 1, 'ill': 17, 'backis': 1, 'outcomeswhitey': 1, 'bulger': 2, 'rat': 7, 'bastardnutritionists': 1, '34': 1, 'servings': 3, 'badbrody': 1, 'shade': 4, 'kimye': 3, 'kuwtk': 1, 'teaserhealth': 1, 'foreverseventhday': 1, 'adventists': 1, 'ordination': 1, '2015five': 1, 'victimsreport': 2, 'teensjimmy': 1, 'britains': 3, 'lordsmillions': 1, 'airlifted': 1, 'zoneautonomy': 1, 'youanchor': 1, 'babies': 11, 'manifest': 2, 'destiny': 2, 'politicsdwight': 1, 'fullcourt': 1, 'ownshould': 1, 'footballmichelle': 1, 'keen': 1, 'bangsbernie': 1, 'levi': 1, 'congressprosthetic': 1, 'arm': 5, 'vending': 2, 'machineare': 1, 'shamelessly': 2, 'promotingatm': 1, 'flees': 1, '50000john': 1, 'stamos': 2, 'rehab': 3, 'gratefulpillsbury': 1, 'skittish': 1, 'broomwielding': 1, 'housewifethe': 1, 'wantedtriumph': 1, 'throughguy': 1, 'therepaul': 1, 'earmarksthe': 1, 'gridlocktrump': 1, 'timemarco': 1, 'warms': 2, 'trumparea': 1, 'm4m': 1, 'adliberals': 1, 'sodomy': 1, 'fraudpublic': 1, 'urinator': 1, 'passerby': 1, 'lookbarista': 1, 'thrill': 4, 'restroomhope': 1, 'solo': 7, 'exfifa': 1, 'sepp': 1, 'blatter': 2, 'hertpp': 1, 'follylet': 1, 'speakphyllis': 1, 'schlafly': 1, 'kingmakersman': 1, 'robbed': 1, 'police20': 1, 'cookoutarchaeologists': 1, 'strata': 1, 'merchandiseresearchers': 1, 'airborne': 1, 'antibioticresistant': 1, 'bacteriaspouse': 1, 'anecdote': 5, 'wrong17yearold': 1, 'photographyblaming': 1, 'turnout': 7, 'virginializard': 1, 'bite': 3, 'getssinging': 1, 'startedfbi': 1, 'follows': 4, 'potentially': 3, 'shootingwould': 1, 'hire': 6, 'paygap': 1, 'pipelinethick': 1, 'sweater': 4, 'nipplesalphabits': 1, 'serif': 1, 'fontthe': 1, 'cutthroat': 1, 'appsvietnam': 1, 'revisitedreport': 1, 'dangerously': 3, 'oppressbrian': 1, 'retreats': 3, 'mountainside': 1, 'hut': 3, 'meditate': 2, 'fickle': 1, 'truthmore': 1, 'bancnn': 1, 'inhouse': 1, 'cnncnnsole': 1, 'asias': 1, 'headcory': 1, 'seth': 3, 'safetydrug': 1, 'enabling': 1, 'girlfriendin': 1, 'selfesteemscientists': 1, 'grant': 7, 'melt': 7, 'stuffthousands': 1, 'flooded': 3, 'barn': 2, 'virallesbian': 1, 'actionuncle': 1, 'cardsstudent': 1, 'pricing': 2, 'campaignthis': 1, 'schoolwhite': 1, 'starbucks': 9, 'isolated': 4, 'patternnew': 1, 'initiative': 5, 'hiredcool': 1, 'dentist': 1, 'flossingtig': 1, 'notaro': 2, 'topless': 2, 'conanwhite': 1, 'inappropriate': 2, 'generalted': 1, 'easter': 12, 'weekendlaidback': 1, 'pmles': 1, 'misrables': 1, 'soundpoint': 1, 'restauranta': 2, 'celebritys': 1, 'outlandish': 2, 'design': 10, 'cutler': 2, 'videolet': 1, 'cakethere': 1, 'reasonwill': 1, 'prequel': 2, 'postclimate': 1, 'dystopiaconsole': 1, 'zenith': 1, 'gamespace': 1, 'prolgbt': 1, 'christians': 4, 'sinner': 2, 'anymoretrumphair': 1, 'hashtag': 7, 'deserves20': 1, 'flattering': 1, 'blazers': 1, 'bustsdustin': 1, 'bullsht': 2, 'actorsreport': 1, 'friendsmindful': 1, 'mantras': 1, 'teachersstaples': 1, 'sectionleaked': 1, 'boiling': 3, 'pointstolen': 1, 'ofiesh': 1, 'kaytlin': 1, 'bailey': 1, 'cavewatch': 1, 'mountains': 3, 'yearsbette': 1, 'midler': 4, 'batman': 3, 'v': 11, 'supermanbreathe': 1, 'justicethomas': 1, 'whitbys': 1, 'ascent': 1, 'educatorcindy': 1, 'crawford': 2, 'lookalike': 2, 'eventeddie': 1, 'vedder': 1, 'awayreporters': 1, 'bernardino': 5, 'vulturesdivorced': 1, 'hobbies': 2, 'unsustainable': 3, 'ratereport': 1, 'supported': 3, 'suffocating': 1, 'benefactor': 3, 'pillowbeekeeper': 1, 'hoarderspeak': 1, 'improve5': 1, 'garage': 7, 'sale10': 1, 'chic': 2, 'firstustrained': 1, 'qaeda': 3, 'affiliatefailure': 1, '17yearolds': 1, 'lifeman': 1, 'articulatedwhat': 1, '49carrie': 1, 'fisher': 5, 'leia': 1, 'stupid11': 1, 'excellent': 5, 'brand': 11, 'kirkus': 1, 'collectionshundreds': 1, 'goats': 2, 'invade': 2, 'berkeleymans': 1, 'neuroses': 1, 'genuine': 2, 'wringerwhite': 1, 'infested': 1, 'bedbugs': 1, 'recliner': 2, 'curb5': 1, 'vacations': 1, 'toocheney': 1, 'wows': 3, 'sept': 5, 'glass': 24, 'speaksofficials': 1, 'unveil': 10, 'underused': 1, 'facilityuber': 1, 'rival': 2, 'lyft': 1, 'findsuptight': 1, 'matron': 1, 'handful': 3, 'pillsobama': 1, 'night23andme': 1, 'forensic': 1, 'committed5yearold': 1, 'carousel': 2, 'waving': 3, 'dadnasa': 1, 'delays': 6, 'sheer': 5, 'habitguy': 1, 'moonwalks': 1, '27': 7, 'european': 6, 'notfirstgrader': 1, 'reeks': 1, 'urinenot': 1, 'waysamerican': 1, 'dolls': 2, 'chainsaws': 1, 'hands2017': 1, 'longerwe': 1, 'ittrump': 1, 'miners': 1, 'roulette5': 1, 'massacreman': 1, 'dinersthe': 1, 'griefsingleparent': 1, 'rating': 4, 'kingpinplanned': 1, 'doormitch': 1, 'nominationsillinois': 1, 'deems': 2, 'rahm': 1, 'emanuel': 3, 'sleazy': 1, 'chicagowill': 2, 'alfonso': 1, 'ribeiro': 1, 'reunionjohn': 1, 'avildsen': 1, 'oscarwinning': 1, 'rocky': 8, '81bernie': 1, 'aurora': 2, 'victimrepublicans': 1, 'repealanddelay': 1, 'mitch': 8, 'mcconnells': 1, 'procoal': 1, 'borders': 4, 'immoralnorth': 1, 'activistwoman': 1, 'hardwood': 1, 'homekids': 1, 'pizzaus': 1, 'constitutionimmigrants': 1, 'mixtape': 2, 'videoobama': 1, 'fridaysenators': 1, 'manyheaded': 1, 'denialnbc': 1, 'chuckhope': 1, '30historical': 1, 'shacklesmark': 1, 'wahlberg': 1, 'prays': 2, 'forgive': 5, 'rolearea': 1, 'calf': 2, 'crampeddonald': 1, 'revels': 2, 'recounting': 1, 'towels': 1, 'victimsmicrosofts': 1, 'solitaire': 1, '25everything': 1, 'splinters': 2, 'feetphilandering': 1, 'theorist': 2, 'everythingmens': 1, 'friendshiphuffpost': 1, 'april': 8, '8peeping': 1, 'televisionshould': 1, 'marry': 7, 'guyruth': 1, 'womenusda': 1, 'brunch': 1, 'wealthier': 3, 'districtstexas': 1, 'obs': 1, 'guyour': 1, 'beara': 1, 'proper': 6, 'bermudarightist': 1, 'francisshould': 1, 'workout': 7, 'leggingsmatthew': 1, 'mcconaughey': 4, 'australian': 3, 'yearwatch': 2, 'liveolay': 1, 'premoisturized': 1, 'skindoes': 1, 'obstruction': 3, 'collusionhuffpost': 1, '11jimmy': 1, 'bookolympic': 1, 'thrills': 1, 'chills': 4, 'puget': 1, 'adventureoverworked': 1, 'pajama': 1, 'bottoms': 1, 'pray': 1, 'soonpope': 1, 'pizzastudy': 1, 'drinkers': 4, 'motion': 4, 'deadwhat': 1, 'crazemigrant': 1, 'uninvited': 1, 'welcoming': 2, 'strangerepa': 1, 'p': 7, 'namenot': 1, 'girlsmad': 1, 'lib': 1, 'swearswillow': 1, 'rentedmedical': 1, 'professionals': 2, 'factcheck': 1, 'greys': 1, 'anatomy': 3, 'scenesbusinesses': 1, 'friendly9596': 1, 'answeredofficers': 1, 'gungop': 1, 'lifetimegeopolitical': 1, 'unaffected': 2, 'princessguys': 1, 'chatbreakup': 1, 'taped': 5, 'babysea': 1, 'barbara': 3, 'seaworldillustrators': 1, 'depict': 1, 'everyday': 7, 'refugeesyour': 1, 'bursting': 5, 'bacteriawhere': 1, 'cambridges': 1, 'dressmichael': 1, 'phelps': 5, 'recreates': 2, 'showparents': 1, 'birthdaysman': 1, 'visited': 3, 'minuteshow': 1, 'beautyiran': 1, 'electionbaby': 1, 'boring2024': 1, '258159study': 1, 'descended': 1, 'wolvesmystery': 1, 'czech': 1, 'lebanese': 1, 'countrydeeply': 1, 'halfheartedly': 3, 'disagreespictures': 1, 'smiling': 7, 'murderedmerger': 1, 'advertising': 4, 'skillstransportation': 1, 'repair': 3, 'rickety': 2, 'bridgesjonathan': 1, 'rhys': 1, 'troubling': 7, 'emergeseveral': 1, 'lazy': 4, 'confirmscity': 1, 'poorarea': 1, 'hug': 3, 'youdespite': 1, 'globe': 4, 'advancement': 2, 'challengesdoctors': 1, 'oldperson': 1, 'smellbrain': 1, 'defective': 3, 'puzzles': 1, 'alzheimersnationwide': 1, 'sympathy': 6, 'pours': 6, 'traumatized': 2, 'survivorwant': 1, 'youmy': 1, 'inheritancejim': 1, 'lehrer': 2, 'moderator': 2, 'tonights': 3, 'newshourappeals': 1, 'restricting': 1, 'concealed': 2, 'carrytelevisions': 1, 'nerdiest': 1, 'indian': 8, 'realcolorado': 1, 'trumpwoman': 1, 'survives': 2, '7story': 2, 'plunge': 6, 'bmwwhats': 1, 'obesity6thgrade': 1, 'gamestop': 1, 'dudenasa': 1, 'exhibitfear': 1, 'jokelocal': 1, 'doobie': 2, 'youworlds': 1, 'fattest': 1, 'consumes': 2, 'mozzarella': 1, 'stickreport': 1, 'technique': 2, 'versions': 4, 'productsan': 1, 'andreas': 3, 'fault': 5, 'docatfishing': 1, 'spurred': 1, 'uva': 2, 'gangrape': 2, 'debacletrump': 1, 'appearances': 1, 'speechdonald': 1, 'attempt': 24, 'sickit': 1, 'trolled': 2, 'everyoneivanka': 1, 'goofs': 1, 'televised': 2, 'boastnewspaper': 1, 'wookiees': 1, '40yearold': 3, 'mistakeernie': 1, 'dan': 2, 'aykroyd': 2, 'ghostbusters': 2, 'rebootdonald': 1, 'favors': 1, 'pollgoogle': 1, 'telemarketersonceadventurous': 1, 'salmon': 1, 'birthplace': 3, 'kids7': 1, 'lossschool': 1, 'crimebuilding': 1, 'esteem': 1, 'offguy': 1, 'despises': 1, 'youa': 3, 'jose': 4, 'bautistas': 1, 'bat': 3, 'flipguantanamo': 1, 'courthow': 1, 'january': 7, 'picturesnate': 1, 'unskewing': 1, 'directionafrotextured': 1, 'nappy': 1, 'perminatorspatriotic': 1, 'betrayal': 1, '1960s': 1, 'spiesebola': 1, 'aids': 5, 'plague': 3, 'incwhy': 1, 'koreakeith': 1, 'olbermann': 1, 'presidentelect': 2, 'pygrabber': 1, 'chancearea': 1, 'thingstearyeyed': 1, 'asset': 1, 'collegeusbacked': 1, 'pleas': 1, 'unansweredmlb': 1, 'muchhomeless': 1, 'nanny': 2, 'statemakeareasonablerequest': 1, 'decent': 4, 'seats': 5, 'minnesota': 6, 'timberwolves': 1, 'gamebannons': 1, 'cyst': 1, 'rupturesthe': 1, 'poets': 2, 'spoof': 7, 'saturday': 7, 'gory': 2, 'bloodfesttrump': 1, 'streep': 8, '2015depressed': 1, 'businessman': 6, 'naps': 1, 'daydoes': 1, 'correctness': 1, 'workpersonals': 1, 'hardline': 1, 'antifatties': 1, 'stancescotlands': 1, 'referendumso': 1, 'thimble': 1, 'hasbro': 1, 'axes': 2, 'monopoly': 4, 'tokenpanicking': 1, 'shoves': 1, 'democrat': 6, 'walks': 14, 'pastcdc': 1, 'decrease': 3, 'oral': 4, 'researchers': 6, 'observingrapidly': 1, 'swelling': 1, 'contain': 9, 'traces': 4, 'peanutsquirky': 1, 'deliverdad': 1, 'sparked': 1, 'gunswells': 1, 'fargo': 3, 'barring': 1, 'recipients': 5, 'loansjeopardy': 1, 'weaselsthe': 1, 'macron': 3, 'trumptyler': 1, 'creator': 11, 'tesla': 2, 'elon': 5, 'muskout': 1, 'hourlyapes': 1, 'tits': 1, 'incrediblemoral': 1, 'compass': 1, 'woodssongs': 1, 'supersized': 1, 'chats': 2, 'tffs': 1, 'roland': 2, 'orzabal': 1, 'lang': 1, 'langplusreport': 1, 'oyster': 1, 'crackerwise': 1, 'goodmore': 1, 'baruch': 1, 'deathpuerto': 1, 'cancellation': 2, 'whitefish': 1, 'contractboyfriend': 1, 'harderguy': 1, 'aspect': 1, 'documentarycosbys': 1, 'secures': 2, 'jury': 12, 'biaswildfires': 1, 'airlift': 1, 'safetyspring': 1, 'southern': 5, 'vermontmit': 1, 'otherrace': 1, 'facing': 6, 'itscientists': 1, 'isolate': 1, 'gene': 5, 'simmonsnew': 1, 'isisinspired': 1, 'saynurse': 1, 'geartrump': 1, 'excels': 1, 'politicsman': 1, 'feigngreenspan': 1, 'hikeheres': 1, 'deleted': 5, 'broad': 4, 'beforefederal': 1, 'officialsoverweight': 1, 'introduced': 2, 'partypassengers': 1, 'terrified': 11, 'bound': 3, 'hawaiimarco': 1, 'gain': 2, 'millennial': 8, 'votersjames': 1, 'tourtrumpnetanyahu': 1, 'exercise': 7, 'fawning': 2, 'antipalestinian': 1, 'incitementride': 1, 'mispimpedstripping': 1, 'indirectly': 1, 'ensures': 2, 'successreport': 1, '32': 3, 'deflected': 1, 'satellitespeople': 1, 'starving': 4, 'usbacked': 1, 'forcesqueer': 1, 'bornstein': 2, 'groundbreaking': 2, 'theda': 1, 'hammelblake': 1, 'baseballscott': 1, 'christiansmart': 1, 'shopper': 1, 'labelthe': 1, 'choicefrustrated': 1, 'sycophant': 1, 'hearoh': 1, 'trickled': 1, 'weirdosfirst': 1, 'breakdancing': 1, 'releasedjon': 1, 'dochild': 1, 'blissfully': 4, 'motel': 5, 'pools': 1, 'sordid': 2, 'pastthe': 1, 'creativitystephen': 1, 'hawking': 3, 'venuspaul': 1, 'lakers': 2, 'townbetsy': 1, 'promisesclimate': 1, 'doomsday': 3, 'saysdemocratic': 1, 'muelleroatmeal': 1, 'variety': 2, 'flavor': 7, 'lefttwelve': 1, 'piefucking': 1, 'worksiran': 1, 'chinas': 8, 'footstepsyear': 1, '25yearoldshow': 1, 'homophobic': 3, 'peoplereporter': 1, 'rescuers': 2, 'wateralec': 1, 'confidential': 2, 'secretive': 2, 'conferencethe': 1, 'missing134yearold': 1, 'attributes': 1, 'typographical': 1, 'errordads': 1, 'coolermeaning': 1, 'obvious': 4, 'elsemuslim': 1, 'remove': 16, 'hijab': 2, 'mugshots': 1, 'file': 4, 'lawsuitfind': 1, '2015batboy': 1, 'rememberedbryant': 1, 'gumbel': 1, 'thanked': 2, 'reasonnetanyahu': 1, 'iii': 5, 'wellthe': 1, 'needsa': 1, 'indiana': 6, 'pleasedraft': 1, 'ademployee': 1, 'likens': 3, 'firedventriloquist': 1, 'crosses': 4, 'suggesting': 4, 'actual': 11, 'dummyexhausted': 1, 'todaycongress': 1, 'reassures': 2, 'thisgender': 1, 'ronald': 1, 'unclearromney': 1, 'uncomprehendingly': 1, 'billgroup': 1, 'complaint': 3, 'girlfriendspecial': 1, 'framers': 1, 'constitution': 7, 'amendmentscompany': 1, 'flatout': 1, 'mileage': 1, 'babynordstrom': 1, 'ivanka': 9, 'itlocal': 2, 'neurotic': 3, '32000hourlong': 1, 'procedurewoman': 1, 'cups': 8, 'bedtimehandlers': 1, 'constantly': 9, 'reminding': 5, 'gingrich': 3, 'uninspiring': 1, 'belittling': 2, 'messagewhat': 1, 'selma': 2, 'blairs': 1, 'outburst': 1, 'mixing': 3, 'pills': 5, 'alcoholafter': 1, 'continueshow': 1, 'embraced': 2, 'blahsobama': 1, 'unionwhen': 1, 'pathway': 1, 'up32yearold': 1, 'agewatch': 1, 'promo': 2, '6a': 1, 'painter': 2, 'searches': 4, 'interconnected': 1, 'humanitynation': 1, 'solace': 1, 'presidentjoan': 1, 'ownstop': 1, 'cellulite': 2, 'ask4': 1, 'mistakes': 11, 'againlast': 1, 'monthexpiration': 1, 'fergie': 1, 'yearstufts': 1, 'nutrition': 1, 'berriessaudi': 1, 'flogged': 1, 'courtordered': 2, 'punishment': 3, 'newspaper': 8, 'saysthe': 2, 'basics': 2, 'leavethis': 1, 'hairstyle': 6, 'mustamericans': 1, 'seemsmakeshift': 1, 'viewpoint': 1, 'dominating': 1, 'conversationquiz': 1, 'starry': 1, 'gershwin': 1, 'philadelphiansnew': 1, 'info': 4, 'smartphonesmost': 1, '6calm': 1, 'impending': 2, 'brokeredavengers': 1, 'sequel': 3, 'offfirst': 1, 'nighter': 4, 'choreographer': 1, 'wheeldon': 1, 'gershwins': 1, 'parisdoes': 1, 'forwomen': 1, 'bywomen': 1, 'brandswoman': 1, 'blossom': 2, 'relationshipresidents': 1, 'fireknowing': 1, 'lifeprison': 1, 'beekeeping': 1, 'buzzdinty': 1, 'silence': 13, 'fullpage': 2, 'adtrump': 1, 'indifferent': 1, 'nytis': 1, 'sxsw': 3, '2015hydraulic': 1, 'diamonds': 2, 'sadly': 6, 'foreverthe': 1, 'trainers': 1, 'clients': 2, 'ofpresident': 1, 'controversythis': 1, 'extreme': 8, 'bikinghow': 1, 'intensifying': 1, 'joaquinson': 1, 'edward': 2, 'murrow': 1, 'dirtbag': 1, 'reportersbiden': 1, 'clenches': 1, 'clappinghillarys': 1, '15second': 3, 'strangulationsophie': 1, 'grown': 5, 'upgeorge': 1, 'trump3': 1, 'kneel': 2, 'reversing': 1, 'policymeet': 1, 'clintoncould': 1, 'guccis': 1, 'clueless': 2, 'coopting': 1, 'queercore': 1, 'resistancewedding': 1, 'acorns': 1, 'floating': 6, 'waterreport': 1, 'suspends': 4, 'commentssuperfood': 1, 'cookie': 5, 'dough': 1, 'notfake': 1, 'melania': 4, 'shell': 9, 'retire': 4, 'donalds': 2, 'colberttraveling': 1, 'log': 3, 'twitterbaby': 1, 'yetcharles': 1, 'schulz': 1, 'neverbeforeseen': 2, 'easeldinner': 1, 'reworked': 1, 'specialattention': 1, 'fansrussia': 1, 'monstrous': 1, 'conclusionsruby': 1, 'nonconforming': 2, 'notdonald': 1, 'meetingarea': 2, 'mirrors': 3, 'baconscozy': 1, 'outoftheway': 2, '12th': 3, 'location25': 1, 'proposals': 4, 'worthy': 2, 'champagne': 2, 'toastreport': 1, 'learns': 5, 'download': 1, 'schoolpope': 2, 'pushed': 9, 'malewhy': 1, 'agoblindfolded': 1, 'coaches': 2, 'sidesloft': 1, 'discussed': 3, 'loft': 1, 'abusive': 6, 'nightfive': 1, 'bibhu': 1, 'mohapatrajessica': 1, 'abusehas': 1, 'dday': 1, 'arrived': 1, 'cubayoure': 1, 'happinesssecond': 1, 'canceling': 2, 'spaceytexas': 1, 'hang': 6, 'treeman': 2, 'chinatrumps': 1, 'americabidens': 1, 'feedback': 1, 'dips': 1, 'percentreport': 1, 'xbox': 5, 'roofhow': 1, 'dietretired': 1, 'unlawfully': 1, 'jfk': 2, 'airportthe': 1, 'backcomedian': 1, 'tracey': 1, 'ullman': 1, 'systemchicago': 1, 'subdivisionsdonald': 1, 'dealsbloodied': 1, 'bruised': 1, 'kickboxing': 1, 'tournament': 3, 'bangkok': 1, 'prisonthis': 1, 'pointless': 1, 'netanyahunancy': 1, 'pelosi': 3, 'mixed': 4, 'partiesjoint': 1, 'chiefs': 2, 'chairman': 5, 'junta': 1, 'toalabama': 1, 'native': 8, 'channing': 1, 'tatum': 2, 'followers': 2, 'moore7': 1, 'seller': 1, 'somethingkid': 1, 'competent': 2, 'oncologistfarberware': 1, 'nonstick': 1, 'eggslondons': 1, 'winswhat': 1, 'bodymake': 1, 'gorsuchhilarious': 1, 'lament': 1, 'photospoliticians': 1, 'jowl': 1, 'implantspersonal': 1, 'philosophy': 3, 'jrbest': 1, '114000': 1, 'resultssean': 1, 'ringsprincipal': 1, 'bullyingthis': 1, 'marchhow': 1, 'digitization': 1, 'farmr': 1, 'assadinternet': 1, 'billions': 5, 'onions': 1, 'leakscaf': 1, 'au': 3, 'detroit': 7, 'stylethe': 1, 'selfcontrolfather': 1, 'sacrificed': 1, 'nothingamazon': 1, 'lease': 1, 'manhattanstudy': 1, 'informationterrier': 1, 'squeakqueer': 1, 'singlestretch': 1, 'adoptedmidnight': 1, '1220blue': 1, 'shipthink': 1, 'harmless': 2, 'againreport': 2, 'nsas': 1, 'intercepted': 2, 'targetsfarmer': 1, 'chases': 2, 'monthstinging': 1, 'fences': 1, 'bees': 6, 'swarm': 3, 'padres': 1, 'camp': 10, 'againtelling': 1, 'storyretirees': 1, 'issuesam': 1, 'collegevery': 1, 'constitutional': 3, 'alcoholismthis': 1, 'brexitheres': 1, 'wellsted': 1, 'salivates': 1, 'criminals': 6, 'republicangetting': 1, 'randomly': 6, 'earn': 7, 'livingseedless': 1, 'grips': 2, 'fact': 7, 'kidsdanny': 1, 'cortez': 1, 'manaustralian': 1, 'politician': 7, 'proposes': 4, 'debateleadership': 1, 'imperativereport': 1, 'intriguing': 1, 'disordergirlfriend': 1, 'youbaseball': 1, 'underwaygirlfriends': 1, 'littlestate': 1, 'nullifying': 1, 'regulations30': 1, 'twisted': 5, 'wreckagea': 1, 'wrenching': 1, 'revolutionneonazi': 1, 'himwhite': 1, 'graciously': 1, 'thankyou': 3, 'yemeni': 2, 'headchild': 1, 'entertained': 2, '1000': 4, 'biodegradeattorney': 1, 'privileged20': 1, 'womanno': 1, 'sylvia': 2, 'earle': 1, 'oceansford': 1, 'f150': 2, 'pickup': 3, 'crushing': 5, 'gotolerance': 1, 'silencenew': 1, 'primaryman': 1, 'obiwan': 1, 'kenobi': 1, 'trendingben': 1, 'garnerclass': 1, 'postgrad': 1, 'apartmentnational': 1, 'jaskoviak': 1, 'requests': 5, 'bailoutarea': 1, 'typeashley': 1, 'grahams': 3, 'swimwear': 1, 'unedited': 1, 'paparazzi': 2, 'photosspelling': 1, 'imaginesbrexit': 1, 'prompts': 3, 'eueveryone': 1, 'gagas': 2, 'storyno': 1, 'arboretum': 1, 'towardbullying': 1, 'prevention': 1, 'empathymitt': 1, 'wavingsunday': 1, 'roundupstudy': 1, 'hangin': 1, 'feelformer': 1, 'turkish': 5, 'coup': 2, 'plotters': 1, 'cnncreating': 1, 'chemicallyfree': 1, 'thinkfreelancer': 1, 'scrape': 1, 'livelihood': 1, 'scheduleobama': 1, 'bidens': 6, 'decisions': 3, '20166': 1, 'strictest': 2, 'lawflustered': 1, 'mathematician': 1, 'numberthousands': 1, '85yearold': 1, 'survivorpool': 1, 'noodle': 1, 'herstrangely': 1, 'shybot': 1, 'roams': 1, 'desert': 1, 'humanstimothe': 1, 'chalamet': 1, 'blocked': 3, 'criticizing': 4, 'dylan': 5, 'farrow': 2, 'didnthear': 1, 'phoneperky': 1, 'optimist': 1, 'joy': 11, 'everywhere': 7, 'leavesstar': 1, 'gopro': 1, 'battleburglar': 1, 'pistorius': 7, 'escapetext': 1, 'succinct': 1, 'chronology': 1, 'relatives': 3, 'hospital': 19, 'visitsbikiniclad': 1, 'sonsis': 1, 'ukraine': 5, 'fascistthree': 1, 'storiesaides': 1, 'toenails': 1, 'wash': 4, 'mumbling': 1, 'bedsoreridden': 1, '155th': 1, 'newswhat': 1, 'seasonsavage': 1, 'dhaka': 1, 'shakes': 3, 'expat': 1, 'communitysyrian': 1, 'exit': 7, 'aleppo': 4, 'truce': 2, 'beginsnew': 1, 'techsupport': 1, 'caste': 2, 'arises': 1, 'indiaepa': 1, 'improve': 8, 'sludgescientists': 1, 'develop': 9, 'extrasloppy': 1, 'peachthe': 1, 'sleephorrible': 1, 'themerestaurant': 1, 'waitresses': 1, 'alerted': 1, 'birthdaysupreme': 1, 'spaghetti': 1, 'strap': 1, 'sunrobes': 1, 'springtrump': 1, 'obamacaremale': 1, 'gaze': 1, 'biteswolf': 1, 'infantof': 1, 'campaign6': 1, 'protect43yearold': 1, 'figured': 5, 'wouldve': 3, 'nowhouston': 1, 'contracts': 2, 'flesheating': 2, 'bacterial': 1, 'infection': 2, 'floodwaterswedding': 1, 'videographer': 1, 'ceremonyyou': 1, 'policyarea': 1, 'reduced': 3, 'thiswhat': 1, 'onevoyager': 1, 'badly': 4, 'smashing': 2, 'universelindsey': 1, 'voicemails': 1, 'revealingtry': 1, 'palins': 1, 'wilderwhy': 1, 'cop21': 1, 'educatorwhen': 1, 'sexuality': 1, 'visibility': 2, 'lessnew': 1, 'soreness': 1, 'lifenation': 1, 'muffinanother': 1, 'nu': 1, 'chapter': 5, 'offensive': 5, 'remarks': 3, 'womenworlds': 1, 'luminaries': 1, 'doorcan': 1, 'amazonpolice': 1, 'casegunman': 1, 'spreeobama': 1, 'cease': 2, 'exist': 6, 'himwhy': 2, 'dayclimate': 1, 'healthfox': 1, 'decisionsenator': 1, 'obscene': 1, 'material': 1, 'attentionhillary': 1, 'quietbus': 1, 'seat': 10, 'himhillary': 2, 'pleasantly': 3, 'pocket10': 1, 'escortobamacare': 1, 'uninsured': 4, 'blindingly': 1, 'companieslindsey': 1, 'endtrumps': 1, 'homeland': 6, 'cruelly': 1, 'separating': 1, 'asylumseeking': 1, 'familiesdysfunctional': 1, 'brought': 13, 'liquorcafepresscom': 1, 'sweeping': 3, 'privacy': 5, 'improperly': 1, 'sizes': 3, 'americansphilando': 1, 'castiles': 1, 'honorthis': 1, 'district': 5, 'demolish': 1, 'ceiling': 5, 'allfemale': 3, 'tickettom': 1, 'pursue': 4, 'sectordespite': 1, 'liberalization': 1, 'radicalism': 1, 'abroadtexas': 1, 'autoerotic': 2, 'asphyxiationailing': 1, '750000': 2, 'wordstyson': 1, 'executives': 9, 'chickens': 4, 'physically': 5, 'incapable': 3, 'roomangela': 1, 'bassett': 1, 'direct': 9, 'lifetimes': 3, 'whitney': 2, 'filmempire': 1, 'destructionguitarinstruction': 1, 'manual': 1, 'halen': 1, 'aunt': 6, 'rhody': 1, 'insidegal': 1, 'scholarshipclub': 1, 'iowalocal': 1, 'amuses': 1, 'cafbut': 1, 'longreport': 1, 'neededstudy': 1, 'suicides': 1, 'occur': 2, 'fitted': 1, 'bedhyperrealistic': 1, 'communitiesstudent': 1, 'snaps': 5, 'hydroponicstrump': 1, 'donors': 6, 'weep': 1, 'hoursdemocrats': 1, 'democratsbill': 1, 'blasio': 4, 'movementmelissa': 1, 'joan': 6, 'hart': 5, '90s': 6, 'queensadly': 1, 'certificate': 5, 'loews': 1, 'cinemas': 2, 'manin': 1, 'measly': 1, 'rallydemocrats': 1, 'stronghold25': 1, 'seasonnonindigenous': 1, 'lines8': 1, 'craziest': 1, 'novembersara': 1, 'gilbert': 4, 'subside77': 1, 'dopresidential': 1, 'anesthetizes': 1, 'outbursts': 1, 'debatevatican': 1, 'signalcamera': 1, 'discreetly': 2, 'trails': 1, 'obesity': 3, 'segmentinside': 1, 'explorersuicide': 1, 'competition': 3, '1351': 1, 'debutupdate': 1, 'exboyfriend': 1, 'dornerhuffpost': 1, 'newsbrief': 2, 'october': 3, '20area': 1, 'downgod': 1, 'humanity': 7, 'rockslaura': 1, 'ingraham': 2, 'newsobama': 1, 'oppositiongm': 1, 'driverless': 3, '2021raving': 1, 'maniac': 1, 'hearpuking': 1, 'look4': 1, 'mustknow': 1, 'wealthreport': 1, 'rightpet': 1, 'winterizedpope': 1, 'congregation': 1, 'lowkey': 2, 'yearwhat': 1, 'travelers': 3, 'futureis': 1, 'doomedreport': 1, 'offices': 2, 'imaginewere': 1, 'gymman': 1, 'gratitude': 4, 'jeanscollege': 1, 'madewoman': 1, 'photoshootcatherine': 1, 'zetajones': 1, 'thatclinton': 1, 'zodi': 1, 'nonmonogamyfather': 1, 'tears': 15, 'trumpdemocrats': 1, 'dreamerswhat': 1, 'reverse': 5, 'mortgagefamily': 1, 'rightsscottish': 1, 'winsingle': 1, 'gyms': 1, 'lifekiddie': 1, 'disrepairthe': 1, 'aprilwhitey': 1, 'verdict': 2, 'interrupted': 4, 'shouting': 2, 'commands': 1, 'balconyglimpse': 1, 'shalit': 1, 'waxkim': 1, 'interestingbiting': 1, 'earcincinnati': 1, 'surge': 5, 'citations': 1, 'motorists': 1, 'pedestriansgina': 1, 'rodriguez': 3, 'trailblazing': 1, 'deserveheres': 1, 'addresslocal': 1, 'streetbritney': 1, 'abs': 3, 'motivationnot': 1, 'intoobama': 1, 'supporter': 8, 'improbable': 1, 'explanation': 4, 'absolving': 1, 'scandalsperuvian': 1, 'shockingly': 1, 'knowledgeable': 1, 'historydamning': 1, 'presidenthuman': 1, 'hindus': 2, 'parts': 5, 'worldwith': 1, 'itreb': 1, 'zalmans': 1, 'unique': 4, 'funeraldocumentary': 1, 'indictment': 2, 'filmmaking': 1, 'skillsben': 1, 'hostile': 5, 'interviewcivilization': 1, 'collapsescrush': 1, 'williamssonoma': 1, 'fortuneheres': 2, 'tradelinmanuel': 1, 'disses': 1, 'remix': 1, 'snlteen': 1, 'accurately': 2, 'mapplethorpe': 1, 'gayarkansas': 1, 'reductions': 1, 'sexhouse': 1, 'filibusterpartially': 1, 'faded': 1, 'undermining': 2, 'sayscashstrapped': 1, 'migrant': 3, 'sondog': 1, 'priceless': 2, 'pretends': 3, 'faintresponsible': 1, 'firearms': 1, 'breakdownrex': 1, 'bunk': 1, 'beds': 1, 'wantedmom': 1, 'nowsix': 1, 'tca': 1, 'panelglitch': 1, 'tablespope': 1, 'pardons': 2, 'dodged': 1, 'draft': 4, 'crusadesrich': 1, 'themheart': 1, 'providerwhy': 1, 'neighborsyrian': 1, 'guestsexcited': 1, 'declassify': 1, 'nights': 7, 'mission': 9, 'easthillary': 1, 'metrump': 1, 'whatim': 1, 'smilingphonesex': 1, '0': 1, 'cents': 2, 'minuteare': 1, 'whiny': 1, 'dipshits': 3, 'sneers': 1, 'unprovoked': 1, 'nomineescondoleezza': 1, 'interpretermuseum': 1, 'exhibits': 3, 'picasso': 1, 'citygender': 1, 'crosshairs': 1, 'hatespatial': 1, 'abandon': 3, 'correct': 4, 'tupperware': 1, 'lidalyson': 1, 'womandiscovering': 1, 'petticoat': 1, 'tampas': 1, 'lingerie': 1, 'trovebush': 1, 'alito': 3, 'unanimous': 1, 'confirmationformer': 1, 'monk': 1, 'vets': 2, 'armortom': 1, 'brady': 4, 'commentschristmas': 1, 'ominously': 4, 'twinselderly': 1, 'freezing': 2, 'away8': 1, 'replaced': 5, 'somber': 1, 'arrangementwhat': 1, 'selfrespecting': 1, 'bribegaunt': 1, 'sickly': 3, 'kirby': 1, 'absence': 4, 'diagnosistrump': 1, 'costly': 1, 'tight': 2, 'finishfor': 1, 'moonlight': 1, 'barry': 1, 'jenkins': 3, 'beautifulobama': 1, 'meanest': 1, 'populationnoose': 1, 'africanamerican': 3, 'dcmaryland': 1, 'medalsi': 1, 'selfjudgmentremembering': 1, 'pat': 4, 'conroy': 1, 'quotesthe': 1, 'mediaobsessednew': 1, 'depressed': 5, 'suicideassistancedogsman': 1, 'taser': 1, 'chasesupercuts': 1, 'scalped': 1, 'monthsiblings': 1, 'somedaynew': 1, 'sunscreen': 1, 'layer': 4, 'bloodjamie': 1, 'fetusmesmerizing': 1, 'identifiedporsha': 1, 'gigepa': 1, 'pulsating': 1, 'sludge': 1, 'directorlife': 1, 'electionsfridays': 1, 'trumpcomey': 1, 'sagamass': 1, 'mobs': 1, 'detroits': 2, 'churchesthe': 1, 'therejazzfest': 1, 'performer': 3, 'year5': 2, 'cocoa': 1, 'sustainabletom': 1, 'frightening': 2, 'anklebreaking': 1, 'stuntborder': 1, 'prototype': 1, 'yayoi': 1, 'kusamaas': 1, 'levelsupreme': 1, 'benatar': 1, 'weaponzoo': 1, 'orangutan': 1, 'womannancy': 1, 'suspension': 4, 'tiesjust': 1, 'driving': 18, 'tank': 11, 'stationhow': 1, '65yearold': 1, 'parkinsonspoll': 1, '56': 2, 'eons': 1, 'agoiran': 1, 'enriched': 3, 'uranium': 3, 'dealwhat': 1, 'ceilingyesterdays': 1, 'tomorrows': 2, 'hotspotsvalencia': 1, 'pulse': 2, 'charming': 2, 'baskinrobbins': 1, 'staff5yearold': 1, 'channels': 6, 'solange': 1, 'knowles': 1, 'recreated': 2, 'coverfederal': 1, 'vicechairman': 1, 'notmom': 1, 'projectthe': 1, 'farflung': 1, 'tripporch': 1, 'ceded': 1, 'batseducated': 1, 'bigot': 1, 'terrifyingtrue': 1, 'feminism': 7, 'accountableturns': 1, 'joss': 1, 'whedon': 1, 'comparing': 2, 'ivankaun': 1, 'rocket': 6, 'launchpolitical': 1, 'governmentspielberg': 1, 'comical': 1, 'groin': 1, 'injuries': 6, 'lincolnarea': 1, 'wheelbarrow': 1, 'weekendlive': 1, 'sundance': 1, 'jan': 4, '28the': 1, 'brunches': 1, 'ustiffani': 1, 'thiessens': 1, 'newborn': 6, 'adorabletwo': 1, 'beatboxers': 1, 'jargon': 2, 'doesunhappy': 1, 'daycolleges': 1, 'feds': 5, 'applicationsthe': 1, '5kgod': 1, 'dammit': 1, 'dancingamerican': 1, 'gladiator': 1, 'turbogod': 1, 'angels': 3, 'voicebritish': 1, 'racetrump': 1, 'xi': 3, 'tweetless': 1, 'wintaylor': 1, 'iheartradio': 1, 'speechtearful': 1, 'manufacturers': 2, 'massacre': 6, 'safelyassistant': 1, 'indiscriminationarea': 1, 'friestoy': 1, '2019leonard': 1, '80fareed': 1, 'zakaria': 1, 'presidentjimmy': 1, 'sesame': 2, 'introduce': 6, 'letterum': 1, 'reese': 3, 'witherspoons': 2, 'stunningfacebook': 1, 'verifies': 2, 'article': 12, 'preconceived': 1, 'opinionssources': 1, 'yearworlds': 1, 'marketingbiden': 1, 'winks': 1, 'eggnog': 1, 'partythis': 1, 'mink': 1, 'waterwidely': 1, 'criticized': 8, 'arrogance': 2, 'intentionalpsa': 1, 'funny': 10, 'absurdity': 1, 'valuing': 1, 'opinions': 4, 'equallywhat': 1, 'jediobama': 1, 'kicks': 8, 'hawaiikathy': 1, 'fbomb': 1, 'trumpnewtown': 1, 'pedophilia': 2, 'revealsgod': 1, 'rewinds': 1, 'trampoline': 1, 'againcoworker': 1, 'limit': 5, 'departments': 3, 'ordertrump': 1, 'recognize': 4, 'preserve': 6, 'campsat': 1, 'printing': 1, 'workswhy': 1, 'nics': 1, 'nras': 3, 'socalled': 3, 'carry': 5, 'reciprocity': 1, 'billnations': 1, 'chillingly': 1, 'reappears': 1, 'nowhereglobal': 1, 'markets': 2, 'fearsthis': 1, 'lovejazz': 1, 'drummer': 1, 'terri': 2, 'lyne': 2, 'carrington': 2, 'fond': 3, 'colefor': 1, 'degrading': 1, 'experienceband': 1, 'concertcrops': 1, 'farmlands': 1, 'monsanto': 3, 'armsmom': 1, 'hairstylefamily': 1, 'infighting': 1, 'apparent': 2, 'bookanaheim': 1, 'welter': 2, 'peoplemaking': 1, 'twiceus': 1, 'thoseceo': 1, 'earnings': 2, 'jailgallup': 1, 'rural': 6, 'ahmadinejad': 4, 'obamahow': 1, 'nationals': 2, 'atlanta': 5, 'braves': 2, 'fancdc': 1, 'fullblown': 2, 'blahsevery': 1, 'penexpresslane': 1, 'realhow': 1, 'contamination': 1, 'waterboy': 1, 'mailman': 1, 'mail': 2, 'kindness': 4, 'forwardgay': 1, 'attendedchester': 1, 'benningtons': 1, 'laughing': 3, 'deathsurgical': 1, 'needleswap': 1, 'swedish': 2, 'hivguys': 1, 'edited': 2, 'impossible': 6, 'trailerjudith': 1, 'miller': 9, 'clings': 1, 'stubborn': 1, 'mythshillary': 1, 'singleword': 1, 'resultsted': 1, 'hansom': 1, 'cab': 1, 'whip': 1, 'hourlatest': 1, 'completists': 1, 'onlyobama': 1, 'guantanamo': 2, 'bay': 4, 'closureun': 1, 'resettle': 1, 'fewnumber': 1, 'presentislamic': 1, 'fundamentalists': 2, 'condemn': 4, 'dayits': 1, 'ceoscinema': 1, 'robin': 5, 'williamsdept': 1, 'dhs': 3, 'meni': 1, 'isi': 1, 'carly': 5, 'fiorina': 5, 'inkjet': 1, 'printertaraji': 1, 'henson': 5, 'reacts': 5, 'empiresinead': 1, 'oconnor': 2, 'overdosejudge': 1, 'heather': 2, 'mills': 5, 'eleanor': 1, 'rigbyamy': 1, 'stuns': 4, 'minidress': 1, 'gq': 1, 'partypeople': 1, 'kindly': 1, 'stfusmuggler': 1, 'weedstuffed': 1, 'jailhow': 1, 'needthey': 1, 'spidermans': 1, 'moviesactual': 1, '460congress': 1, 'honcho': 1, 'trumpfbi': 2, 'homey': 1, 'daygood': 1, 'proving': 4, 'rightman': 1, 'pointmake': 1, 'rethink': 3, 'itamazons': 1, 'evolvesfamily': 1, 'candy': 6, 'huddled': 2, 'darkened': 1, 'blitzkate': 1, 'mara': 2, 'superherodesperation': 1, 'againbarneys': 1, '525000': 1, 'profiling60': 1, 'revive': 2, 'dual': 2, 'systemi': 1, 'reader': 6, 'delivering': 1, 'politicized': 1, 'judiciary': 1, 'dreamed': 5, 'aboutexplanation': 1, 'peppered': 2, 'reassurances': 1, 'fundrake': 1, 'clubjapanese': 1, 'mid80s': 1, 'onfor': 1, 'sharpest': 1, 'crayon': 1, 'poemnetwork': 1, 'satellites': 1, 'iraqdiet': 1, 'candys': 1, 'aftertaste': 1, 'experienced': 2, 'laterrewriting': 1, 'nepal': 2, 'marked': 2, 'sparkling': 1, 'englishlanguage': 1, 'debutsaxe': 1, 'douchey': 1, 'empower': 6, 'menthis': 1, 'marvelous': 1, 'messfate': 1, 'cargo': 3, 'joaquin': 4, 'bahamasdrugsniffing': 1, 'develops': 7, 'bitohoneysdonald': 1, 'flails': 1, 'rigged': 2, 'talkwatch': 1, 'carve': 1, 'thronesa': 1, 'chateaucoworkers': 1, 'onehour': 1, 'breakafter': 1, 'irma': 6, 'pens': 5, 'islandshuffpollster': 1, 'hampshirethe': 1, 'bend': 3, 'ushaped': 1, 'skyscraper': 4, 'longest': 3, 'worldotto': 1, 'warmbier': 3, 'neurological': 1, 'injuryillinois': 1, 'quarantine': 2, 'highrisk': 1, 'travelersexiranian': 1, 'mahmoud': 3, 'againben': 1, 'reviewing': 1, 'superman': 2, 'deadscientist': 1, 'nagging': 2, 'accelerator': 1, 'onapples': 1, 'gag': 1, 'division': 2, 'sleekest': 1, 'dateturns': 1, 'vloggersshamefaced': 1, 'stockstill': 1, 'acquaintance': 5, 'zips': 1, 'backpack': 2, 'statesprochoice': 1, 'judgesreligious': 1, 'pamphlet': 1, 'onbush': 1, 'gotherapist': 1, 'daughternation': 1, 'fund': 16, 'rightmichael': 1, 'avenatti': 1, 'releasegynecologists': 1, 'iuds': 1, 'expel': 1, 'backlogged': 1, 'periodsheres': 1, 'womenboycotttwittermarching': 1, 'januaryman': 1, 'antidepressants': 2, 'personalityuhoh': 1, 'deadlineroommate': 1, 'oscarmother': 1, 'correctlyselena': 1, 'bikiniwarning': 1, '1988': 2, 'nintendo': 1, 'intensely': 1, 'nostalgicapple': 1, 'success': 14, 'failurepope': 1, 'butting': 2, 'godpolice': 1, 'kidnapping': 1, 'custodyrelated': 1, 'onesmanager': 1, 'sweetsnow': 1, 'angelresearchers': 1, 'naturally': 2, 'evolved': 2, 'monogamy': 1, 'flings': 1, 'sidehair': 1, 'salons': 1, 'smalltalkesports': 1, 'organizations': 2, 'gamingphiladelphia': 1, 'erects': 1, 'statue9': 1, 'fabulous': 2, 'onesbernie': 1, 'factory': 8, 'votelearn': 1, 'mistake': 5, 'practicefor': 1, 'chimpanzees': 2, 'statementtheres': 1, 'michigans': 2, 'proposalsasha': 1, 'malia': 3, 'superstar': 1, 'argentinaqueen': 1, 'cersei': 1, 'insults': 4, 'bachelor': 10, 'offda': 1, 'treating': 7, 'deficienciesmuseum': 1, 'tshirtrelieved': 1, 'acknowledging': 1, 'speechbeyond': 1, 'classroom': 1, 'innovation': 5, 'upcloseandpersonal': 1, 'sxswnorth': 1, 'cigarettesthe': 1, 'whitecollar': 3, 'testdeath': 1, 'penalty': 3, 'redemption': 2, 'tsarnaev': 5, 'christianitysecret': 1, 'laverne': 2, 'shirley': 1, 'fanmagical': 1, 'gallery': 6, 'artactress': 1, 'cinemax': 1, 'erotic': 2, 'thrillermarijuana': 1, 'copsthis': 1, 'brace': 4, 'seemexico': 1, 'refry': 1, 'beansbusiness': 1, 'realestate': 1, 'moneyuse': 1, 'organic': 5, 'peanut': 3, 'lifeobama': 2, 'safesspider': 1, 'faceno': 1, 'palm': 3, 'deforestationfame': 1, 'sexually': 10, 'transmittedhouthi': 1, 'warit': 1, 'floridaman': 1, 'flamethrower': 2, 'appropriate': 3, 'itqueer': 1, 'techtanzania': 1, 'tackles': 4, 'toolsbig': 1, 'morningwoman': 1, 'stances': 2, 'votingamericans': 1, 'dislike': 1, 'mediahillary': 1, 'doctormore': 1, 'donenestl': 1, 'glass5': 1, 'prodr': 1, 'oz': 4, 'rarely': 2, 'issuespoll': 1, 'type': 9, 'screwdriver': 1, 'headelderly': 1, 'againhorserace': 1, 'announcer': 2, 'dancernew': 1, 'garners': 2, 'deathperson': 1, 'cropped': 1, 'matchcom': 2, 'buzz': 2, 'lightyeardanceclub': 1, 'storyexpressing': 1, 'gaffelittle': 1, 'butterball': 1, 'linethe': 1, 'charitable': 2, 'childrenwhy': 1, 'muslimsthe': 1, 'uptown': 1, 'beggararea': 1, 'boggleemergency': 1, 'wisdomguinea': 1, 'pig': 4, 'returned': 5, 'creditsanders': 1, 'smashed': 1, 'gang': 3, 'pinkerton': 1, 'bustersa': 1, 'ocdtwo': 1, 'intexas': 1, '393rd': 1, 'prisonertruth': 1, 'fictionelderly': 1, 'senility': 1, 'erase': 2, 'lifetime': 10, 'regretful': 1, 'memoriesmonopoly': 1, 'scrabblethemed': 1, 'editionhillary': 1, 'mouthing': 1, 'oathnyse': 1, 'believeyet': 1, 'piles': 1, 'uptwitter': 1, 'imparts': 1, 'wisdom': 3, 'trumpbacktoschooltipswe': 1, 'rule': 10, 'revolvers': 1, 'audiencetrump': 1, 'envoy': 2, 'talkscnbc': 1, 'cameraman': 2, 'blog': 3, 'monitorman': 1, 'stain': 2, 'pantsgroundbreaking': 1, 'protagonist': 2, 'lonernew': 1, 'forcing': 5, 'marrycurly': 1, 'fry': 2, 'inventor': 2, 'curly': 2, 'vealreport': 1, 'illegiblybest': 1, 'sharp': 2, 'sciencestruggling': 1, 'camper': 1, 'presentslost': 1, 'gondolier': 1, 'adriatic': 1, 'seajeb': 1, 'subsidiesbitchy': 1, 'swoop': 1, 'awaythis': 2, 'broke': 11, 'afrolatina': 1, 'journalistsarea': 1, 'isrest': 1, 'biting': 3, 'relationshipindonesian': 1, 'unrest': 1, 'quelled': 1, 'seasonoffice': 1, 'coffeechinese': 1, 'astronomers': 2, 'beijing': 4, '2minute': 2, 'morningsecret': 1, 'prostitution': 1, 'affect': 5, 'madame': 2, 'chartreuse': 1, 'saysharmony': 1, 'tragedy': 6, 'togetherbonobo': 1, 'sexunderground': 1, 'railroad': 2, 'carries': 5, 'brooklyn': 4, 'manhattantelevised': 1, 'sporting': 2, 'obscured': 1, 'graphicsgods': 1, 'returneddemi': 1, 'conflicted': 1, 'deathauthorities': 1, 'shootingmusic': 1, 'montreal': 1, 'festivaltime': 1, 'secondkeg': 1, 'drivescience': 1, 'partspaul': 1, '120': 2, 'campaigntrevor': 1, 'vladimir': 4, 'sinister': 2, 'diplomatsclosed': 1, 'gentrifying': 1, 'chrysalis': 1, 'gastropublindsey': 1, 'monitorlinmanuel': 1, 'millennialswhy': 1, 'justicebernie': 1, 'memorializes': 1, 'votedoll': 1, 'glosses': 1, 'hinged': 1, 'housenorth': 1, 'nukes': 2, 'attentionslovenian': 1, '8thgraders': 1, 'outperformed': 1, 'sciencewoman': 1, 'extrovertdesign': 1, 'startups': 2, 'getgohandshake': 1, 'unusually': 1, 'angle': 1, 'velocityoregon': 1, 'findsarea': 1, 'noticed': 3, 'mastectomypaul': 1, 'lepage': 2, 'guillotine': 1, 'traffickersmarcellus': 1, 'drew': 2, 'shootingdzhokhar': 1, 'courtside': 1, 'pacersheat': 1, 'gamelauren': 1, 'conrad': 2, 'dealing': 3, 'devils': 2, 'assholehong': 1, 'clash': 2, 'headquartersbill': 1, 'obamacarewhen': 1, 'allwalking': 1, 'shocking': 6, 'deathamerican': 1, 'approved': 3, 'judgereport': 1, 'gen': 1, 'irony': 2, 'cynicism': 1, 'obsoleteblack': 1, 'killmongers': 1, 'trendlocal': 1, 'himafter': 1, 'toddler': 6, 'driveellen': 1, 'hetero': 1, 'halloweenreallife': 1, 'pepe': 1, 'pew': 2, 'rapes': 1, 'catcrowd': 1, 'cheers': 4, 'patronizing': 1, 'rhetoricblack': 1, 'safethe': 2, 'weightdepartment': 1, 'episodegop': 1, 'satisfy': 3, 'crazies': 1, 'reidwomen': 1, 'likestephen': 1, 'mocks': 4, 'gunsrocks': 1, 'manchilds': 1, 'bartapas': 1, 'arriving': 3, 'fastjosh': 1, 'earnests': 1, 'wh': 1, 'wellmike': 1, 'butterworth': 1, 'arrivescity': 1, 'councilman': 1, 'avenue': 1, 'centerlets': 1, 'hairtrigger': 1, 'blow': 11, 'planettarantinostyle': 1, 'potentialchrissy': 1, 'postelection': 4, 'thanksgivinggoliath': 1, 'wallet': 1, 'statesnew': 1, 'subvert': 1, 'stereotypesbad': 1, 'filmplane': 1, 'diverted': 2, 'reclines': 1, 'seatnick': 1, 'offermans': 1, 'satirical': 1, 'lunchesyoung': 1, 'jade': 1, 'pettyjohn': 1, 'nickelodeons': 1, 'katee': 1, 'sackhoffsocial': 1, 'startup': 9, 'smug': 2, 'rolenitroglycerin': 1, 'chex': 1, 'gingerly': 1, 'shelvesthese': 1, 'grandpas': 3, 'adorabletaylor': 1, 'vmasalaska': 1, 'steals': 5, 'hubby': 1, 'sayamerican': 1, 'rational': 2, 'measured': 1, 'preclude': 1, 'panicked': 4, 'overreaction': 1, 'futureproud': 1, 'eyebrows': 2, 'timethe': 3, 'hypocrisy': 5, 'underlying': 1, 'movementarea': 1, 'imagines': 2, 'peppermint': 1, 'mocha': 1, 'tongue': 4, 'herselfroomba': 1, 'daughterthe': 1, 'honey': 2, 'boo': 4, 'outpigeons': 1, 'accommodatedsubway': 1, 'unnerved': 2, 'screech': 1, 'sandwiches': 3, 'halfsolar': 1, 'growthall': 1, 'selvesthe': 1, 'fibssenate': 1, 'requestfan': 1, 'grabs': 1, 'backthe': 1, 'cuts3': 1, 'storiessea': 1, 'yanks': 2, 'snatch': 1, 'fishgwen': 1, 'stefani': 4, 'shelton': 1, 'awardsmondays': 1, 'historyhealthcare': 1, 'confusing': 6, 'illnesscancer': 1, '15straight': 1, 'tensile': 1, 'lineswhats': 1, 'eastern': 1, 'trumpismla': 1, 'highspeed': 7, 'lane': 6, 'freewayjenny': 1, 'slate': 2, 'everwhite': 1, 'charlottesville4': 1, 'crazygood': 2, 'kebab': 2, 'recipesgame': 1, 'rewatching': 2, 'breasts': 2, 'likecalifornias': 1, 'drought': 2, 'foreverand': 1, 'revivalthe': 1, 'controlarea': 1, 'marries': 3, 'insurancenew': 1, 'uninhabitableteenage': 1, 'surfer': 2, 'irmas': 3, 'waves': 5, 'barbadosnypd': 1, 'anniversaryreport': 1, 'spidersave': 1, 'multipacksbest': 1, 'socksheroin': 1, 'addicts': 2, 'pressure': 5, 'afghanistanroger': 1, 'president10': 1, 'sephoras': 1, 'vib': 1, 'salezuckerberg': 1, 'immigrationplaying': 1, 'livesarea': 1, 'nauseous': 1, 'scrollingmaid': 1, 'maids': 1, 'householdsapple': 1, 'timeshas': 1, 'curedparking': 1, 'seemingly': 3, 'godbeauty': 1, 'scarfs': 1, 'krispy': 1, 'kreme': 2, 'timearianna': 1, 'payoff': 1, 'reshape': 2, 'industryno': 1, 'visas': 2, 'deniednew': 1, 'everythingsatirical': 1, 'dominated': 2, 'teenager': 3, 'didnew': 1, 'presauced': 1, 'packagelook': 1, 'jean': 5, 'lineall': 1, 'cheapest': 1, 'purchasedbarbara': 1, 'hoursradicals': 1, 'vie': 2, 'iranman': 1, 'fansusan': 1, 'breakout': 1, 'superherogetting': 1, 'twogeneration': 1, 'approachroad': 1, 'overexplains': 1, 'dangersholy': 1, 'curb': 8, 'enthusiasm': 2, 'backcheney': 1, 'thick': 3, 'wavy': 1, 'hairarea': 1, 'cockroach': 2, 'hugerupaul': 1, 'contestants': 2, 'racecrazed': 1, 'loiterer': 1, 'readytibetan': 1, 'philosophy8': 1, 'bruce': 5, 'springsteenphotos': 1, 'colombias': 2, 'floodschris': 1, 'bumped': 1, 'debaterun': 1, 'youin': 1, 'fires': 17, 'studentsim': 1, 'meme': 1, 'abortionworld': 1, 'forecloses': 1, 'farmvulnerable': 1, 'showed': 4, 'winnerwomen': 1, 'onlinewhy': 1, 'nightwomen': 1, 'antimobster': 1, 'weinsteinmichael': 1, 'displeased': 1, 'burnt': 1, 'offeringsanne': 1, 'franco': 4, 'tearing': 4, 'jesse': 1, 'eisenbergman': 1, 'lumbering': 1, 'golem': 1, 'cardsnews': 1, '2017the': 2, 'nixonization': 1, 'trumpuhaul': 1, 'discount': 1, 'city10': 1, 'easytotackle': 1, 'sweetspaul': 1, 'pitching': 1, 'crocodile': 3, 'dundee': 1, 'saturdaymorning': 1, 'cartoonthe': 1, 'smithereens': 1, 'dinizio': 1, '62this': 1, 'muffuletta': 1, 'recipe': 3, 'weekendwearable': 1, 'sensors': 1, 'itten': 1, 'growangela': 1, 'chancellorthe': 1, 'italian': 10, 'commoncharlize': 1, 'theron': 5, 'furious': 8, 'clipgrounded': 1, 'tarmacwhy': 1, 'thingswellmeaning': 1, 'mouse': 6, 'pest': 1, 'setrump': 1, 'sayspaul': 1, 'ryans': 6, 'eliminate': 2, 'mandateheartbroken': 1, 'deathconscious': 1, 'uncoupling': 1, 'wimping': 1, 'lessonshillary': 1, 'fec': 2, 'spitting': 1, 'volunteertragic': 1, 'oscarnight': 1, 'malfunction': 4, 'seven': 8, 'underpublicizedamericans': 1, 'amazons': 4, 'punishing': 2, 'yield': 2, 'sameday': 1, 'shipping': 1, 'productson': 1, '18th': 2, 'bindi': 1, 'irwin': 2, 'lightcar': 1, 'injuring': 2, '4trump': 1, 'expanded': 2, 'bana': 2, 'worklife': 2, 'coolinhibitions': 1, 'seedy': 1, 'roomheres': 1, 'bryants': 1, 'gamea': 1, 'island': 13, 'novels': 2, 'settingonthejob': 1, 'taleswhite': 1, 'sasha': 1, 'played': 11, 'watkinsepic': 1, 'compilation': 1, 'lyfeheres': 1, 'rightsdnc': 1, 'thanking': 3, 'alabamas': 2, 'workingclass': 3, 'voterssearch': 1, 'underway': 1, 'autism': 5, 'lanyard': 1, 'collecting': 5, 'pinsreport': 1, 'louder': 3, 'newstrump': 1, 'skittles': 1, 'settling': 1, 'hearingarea': 1, 'comedynasa': 1, 'flirty': 1, 'skirtthe': 1, 'winters': 1, 'yore': 1, 'wrongnew': 1, 'cocknew': 1, 'nervousenergy': 1, 'callfriend': 1, 'toppinglottery': 1, 'winner': 5, 'childreneveryone': 1, 'factshow': 1, 'yearphotograph': 1, 'absorbed': 1, 'viralstudy': 1, 'mice': 2, 'doorway': 2, 'wallfloor': 1, 'mediocrityhpv': 1, 'rates': 11, 'sayswill': 1, 'palestinechuck': 1, 'cheeses': 2, 'prices': 5, 'dirtiernew': 1, 'desktop': 1, 'folder': 1, 'creative': 12, 'projectsenator': 1, 'constituency': 2, 'homecelebrities': 1, 'messagesdance': 1, 'cage': 1, 'recidivism': 1, 'scenesharon': 1, 'tates': 1, 'tacky': 1, 'manson': 2, 'murderlazy': 1, 'segmented': 1, 'abdomena': 1, 'rauner': 1, 'uberrich': 1, 'candidatetrump': 1, 'visasgodknowswhat': 1, 'cabinthe': 1, 'scientologyapproved': 1, 'differentman': 1, 'unemployedlarry': 1, 'shameless': 3, 'abandonfamily': 1, 'aware': 3, 'surroundingsstudy': 1, 'witnessed': 1, 'windowinside': 1, 'worrells': 1, 'allstar': 3, 'benefithouse': 1, 'cardsstyle': 1, 'virginia100': 1, 'bellissima': 1, 'timechina': 1, 'disputes': 1, 'fentanyl': 2, 'flood': 2, 'stateselderly': 1, 'dennyshow': 1, 'overcome': 7, 'distance': 5, 'relationshipsbill': 1, 'gestapooverburdened': 1, 'thwart': 2, 'treatmentlena': 1, 'raping': 2, '17yearoldapple': 1, 'citizens2015': 1, 'hotelsreport': 1, 'moaning': 1, 'secondsmeet': 1, 'rebirth': 1, 'everybodychina': 1, 'excludes': 1, 'lawcatholic': 1, 'metrosexualityboss': 1, 'yearlac': 1, 'megantic': 1, 'latervladimir': 1, 'putin': 11, 'supermarketthe': 1, 'rogue': 4, 'nerds': 4, 'dreamvengeanceminded': 1, 'glacier': 3, 'biding': 1, 'ageonceloyal': 1, 'enabler': 1, 'betrays': 3, 'therapyreverend': 1, 'sharpton': 1, 'duties': 4, 'appearanceravens': 1, 'steelers': 1, 'playoff': 3, 'grudge': 1, 'matchwhy': 1, 'reasonaerobics': 1, 'nonaerobic': 1, 'purposean': 1, 'contemporaryjon': 1, 'cohosts': 1, 'sportscenter': 1, 'athletesgoldfish': 1, 'teetering': 1, 'edge': 12, 'sanitychrissy': 1, 'bra': 1, 'teddy': 3, 'postmet': 1, 'galai': 1, 'amwomen': 1, 'paula': 1, 'kavolius': 1, 'possibilitiesmilitary': 1, 'backsyrian': 1, 'dyingolympic': 1, 'skater': 5, 'garbageevery': 1, 'shelves': 7, 'copingthe': 1, 'consensusviewers': 1, 'debategreece': 1, 'reopen': 2, 'monday24': 1, 'remainder': 1, '2017pizza': 1, 'huts': 2, 'skinny': 4, 'trueread': 1, 'sally': 1, 'yates': 1, 'banpneumonia': 1, 'offrance': 1, '2040prospective': 1, 'statetrailblazing': 1, 'colleague': 3, 'floorwhat': 1, 'hint': 5, 'neither': 6, 'nor': 4, 'varied': 1, 'muslimsdr': 1, 'scholls': 1, 'lace': 1, 'insolesweakling': 1, 'imaginary': 2, 'bless': 1, 'nationtrump': 1, 'sacrifices': 1, 'presidencycarrie': 1, 'remembered': 3, 'daysupreme': 1, 'lifting': 4, 'classaction': 2, 'workersstunning': 1, 'e3': 1, 'consoles': 1, 'graphics': 2, 'entirelyworking': 1, 'advicea': 1, '2000yearold': 1, 'psalmwacky': 1, 'dj': 5, 'wackier': 1, 'djoh': 1, 'movecomedian': 1, 'janet': 2, 'silverman': 2, '89': 7, 'dck': 1, 'picksmaid': 1, 'selected': 1, 'ability': 13, 'beatinglena': 1, 'heady': 1, 'emmysradio': 1, 'talkshow': 1, 'pointhealth': 1, 'yosemitecitizens': 1, 'needchilds': 1, 'description': 3, 'neardeath': 3, 'dealbask': 1, 'anna': 4, 'kendricks': 1, 'trolls': 15, 'premieregene': 1, 'wilders': 1, 'pryormore': 1, 'vegetables': 1, 'chocolatesaucefilled': 1, 'centers': 3, 'imperativeman': 1, 'ashamed': 2, 'hima': 1, 'miraclewhats': 1, 'killerleaked': 1, 'cables': 1, 'contradict': 2, 'agreementhall': 1, 'oates': 1, 'inducted': 4, 'videolazy': 1, 'daredevil': 2, 'couchespaul': 1, 'embraces': 3, 'rollouthere': 1, 'winners13': 1, 'monthhow': 1, 'feelingsrun': 1, 'solution': 9, 'americana': 2, 'commodore': 1, '1980s': 1, 'heating': 1, 'michigancongressman': 1, 'exposed': 1, 'penis': 9, 'constitute': 4, 'harassmentparkland': 1, 'couldve': 4, 'worseshape': 1, 'monthfamily': 1, 'emotions': 2, 'numberscientists': 1, 'successfully': 6, 'artificial': 4, 'placenta': 1, 'onetime': 1, 'israelpalestine': 1, 'talksvenezuela': 1, 'hunts': 1, 'helicopter': 6, 'attackersdads': 1, 'latelysimple': 1, 'yearqueen': 1, 'alliances': 1, 'arranging': 1, 'greatgrandchildrens': 1, 'marriageswoman': 1, 'hitman': 2, 'grandkids': 3, 'lice': 1, 'sayrihanna': 1, 'circles': 1, 'younigel': 1, 'lonely': 4, 'seabird': 1, 'replica': 2, 'yearsphotographer': 1, 'seriested': 1, 'upsets': 2, 'caucusarea': 1, 'thousandth': 1, 'todayiran': 1, 'sanctions': 4, 'renewalfreshness': 1, 'escaping': 3, 'peasgentle': 1, 'biographers': 1, '28pinemartenaday': 1, 'habitnation': 1, 'curious': 5, 'fullnasa': 1, 'spacetarget': 1, 'wistful': 1, 'eyeskaty': 1, 'awarded': 4, '157': 1, 'entrepreneur': 4, 'interfered': 1, 'convent': 1, 'salereport': 1, 'texting': 3, 'secondsprinceton': 1, 'confront': 4, 'woodrow': 1, 'wilsons': 1, 'legacygovernor': 1, 'lashes': 3, 'scotch': 1, 'poorly': 6, 'rolled': 4, 'cigarsdepartment': 1, 'allocates': 2, 'shortcutno': 1, 'covers': 7, 'blownjohn': 1, 'pit': 5, 'quicksand': 1, 'companionjimmy': 1, 'fingerdomehome': 1, 'manage': 2, 'lowerits': 1, 'signsholy': 1, 'guacamole': 3, 'zealand': 2, 'stealing': 5, 'avocadosnew': 1, 'murderednation': 1, 'fondly': 2, 'regulating': 1, 'violence5': 1, 'strengthen': 1, 'relationshipis': 1, 'quote': 3, 'pornindependent': 1, 'baking': 1, 'documentaryhuffpost': 1, 'newrhea': 1, 'maceris': 1, 'empowerednew': 1, 'dollar': 4, 'menscientists': 1, 'placedespite': 1, 'remoteness': 1, 'antarcticas': 2, 'mattershawaii': 1, 'passwordtexas': 1, 'sunscreencloned': 1, 'cheney': 7, 'charm': 1, 'originalbig': 1, 'sisters': 5, 'entertainments': 1, 'salke': 1, 'sherry': 1, 'lansing': 1, 'awardtribe': 1, 'winterfk': 1, 'muzlim': 1, 'terroist': 1, 'spraypainted': 1, 'carteen': 1, 'shark': 9, 'nbdleadership': 1, 'greatnessnew': 1, 'gallows': 2, 'asphyxiationlast': 1, 'tortoise': 2, 'diesclergy': 1, 'upsilvio': 1, 'berlusconi': 3, 'transferred': 2, 'steamy': 3, 'penitentiarysteve': 1, 'thoughtman': 1, 'feeding': 3, 'hostagesa': 1, 'togetherwhy': 1, 'policyspring': 1, 'rebirthwhat': 1, 'fish': 9, 'organicman': 1, 'interests10': 1, 'everdad': 1, 'honk': 2, 'frontkeynote': 1, 'enlightens': 1, 'theme': 8, 'changingthe': 1, 'binge': 3, 'youtemperature': 1, 'nine': 3, '21st': 7, 'centurydozens': 1, 'shootingkidnapping': 2, 'led': 10, 'crimebill': 1, 'asia': 4, 'tripben': 1, 'stiller': 2, 'peels': 1, 'feetcute': 1, 'shitterminally': 1, 'anymorewhy': 1, 'linera': 1, 'cloakhow': 1, 'moviesdelta': 1, 'captainfarewell': 1, 'nidetch': 1, 'saint': 5, 'watchersdear': 1, 'patricia': 2, 'arquette': 1, 'fought': 3, 'usrosetta': 1, 'pandering': 1, 'candidatesdick': 1, 'masturbates': 2, 'tickle': 1, 'copsworld': 1, 'organization': 8, 'levines': 2, 'antidote': 2, 'mers': 1, 'viruslucrative': 1, 'extraction': 1, 'drilling': 3, 'stationsstephen': 1, 'actionsholly': 1, 'madison': 2, 'releasing': 3, 'memoir': 6, 'playboy': 1, 'mansionben': 1, 'jerrys': 2, 'punny': 1, 'everthis': 1, 'cryptically': 1, 'homage': 3, 'smash': 1, 'classicbird': 1, 'poo': 1, 'creamaziz': 1, 'crimes14': 1, 'recreation': 2, 'foodbetsy': 1, 'weakening': 1, 'reporting': 5, 'protections': 5, 'studentsaccess': 1, 'storycollege': 1, 'scorecard': 1, 'sandbags': 1, 'equity': 3, 'educationdolce': 1, 'gabbana': 1, 'launched': 4, 'hijabs': 1, 'abayas': 1, 'somethings': 1, 'off5': 1, 'themhistory': 1, 'rams': 1, 'sam': 5, '4year': 2, 'multimillion': 1, 'contractgoodwill': 1, 'shaken': 5, 'gigantic': 1, 'pantsthe': 1, 'knowwhy': 1, 'microsofts': 1, 'weaponhow': 1, 'ignored': 5, 'lone': 6, 'childrenone': 1, 'overcame': 1, 'athletedonald': 1, 'sanderswoman': 1, 'jacket': 5, 'indulging': 1, 'pleasure': 3, 'pocketscongressional': 1, 'deniers': 2, '63': 3, 'americansdonald': 1, 'statemore': 1, '5600': 1, 'africa': 7, 'daysheres': 1, 'boydonald': 1, 'expects': 3, 'doesgenetically': 1, 'modified': 2, 'dipping': 3, 'saucee': 1, 'coli': 2, 'outbreak': 9, 'spreads': 4, 'statespack': 1, 'harpies': 1, 'crostini': 1, 'agopentagon': 1, 'shady': 3, 'dealerfriends': 1, 'hostility2018': 1, 'delusional': 2, 'report18': 1, 'kisses': 3, 'kneeslos': 1, 'overpassesabandoned': 1, 'retains': 1, 'eerie': 3, 'vestiges': 1, 'atmospherealaska': 1, 'tomi': 2, 'tami': 1, 'itconservative': 1, 'pundit': 3, 'se': 1, 'cupp': 1, 'ballsmorgan': 1, 'spurlock': 1, 'accusationtime': 1, 'magazines': 1, 'articleman': 1, 'loyalty': 5, 'convenience': 1, 'ceremonyshoe': 1, 'double': 14, 'technologywhy': 1, 'abroad': 3, 'rainbows': 1, 'butterfliesmonsanto': 1, 'hardier': 1, 'strain': 2, 'yields': 3, 'litigationgeorgia': 1, 'std': 1, 'kits': 3, 'againon': 1, 'downitalian': 1, 'dipshit': 1, 'lasagnagenerous': 1, '800': 1, 'nightwoman': 1, 'antihillary': 2, 'pacobama': 1, 'daughtersshitty': 1, 'decreased': 3, 'daylight': 3, 'timeto': 1, 'maskhow': 1, 'mastectomyreport': 1, 'majestically': 1, 'billowing': 1, 'windstudy': 1, 'landings': 1, 'offmiddleaged': 1, '31yearold': 1, 'parentshow': 1, 'billfrantic': 1, 'lookalikeinvestigation': 1, 'management': 3, 'lauers': 1, 'networksanctioned': 1, 'dungeonactor': 1, 'rapaport': 1, 'knee': 1, 'unloads': 2, 'motherfker': 1, 'trumppreemie': 1, 'photographs': 5, 'nicu': 1, 'coasterman': 1, 'breach': 4, 'honoluluabby': 1, 'sunderland': 1, 'concocted': 1, 'historys': 1, 'jobwhatsapp': 1, 'disagreements': 1, 'facebookstephen': 1, 'scariest': 1, 'yetmissing': 1, 'quadruples': 1, 'accidentmegyn': 1, 'accountabletheres': 1, 'kenneth': 1, 'boundariesdespite': 1, 'roadman': 1, 'brandwill': 1, 'scientific': 6, 'breakthroughs': 1, 'climaterelated': 1, 'lawsuitsbilly': 1, 'thornton': 1, 'er': 3, 'accidentnew': 1, 'lobs': 1, 'policepizza': 1, 'hebush': 1, 'blocksintel': 1, 'oversized': 2, 'novelty': 2, 'processorsouth': 1, 'ceremonywho': 1, 'defines': 3, 'murdervice': 1, 'handlers': 2, 'lure': 2, 'cratejudge': 1, 'whatwomans': 1, 'tan': 2, 'sensenew': 1, 'preventative': 2, 'alzheimersroseanne': 1, 'taping': 1, 'classone': 1, 'investor': 2, 'missingman': 1, 'exgirlfriends': 2, 'movingon': 1, 'headwhy': 1, 'yourselfnick': 1, 'offerman': 1, 'rsvps': 1, 'swansonesque': 1, 'waydept': 1, 'mallal': 1, 'roker': 3, 'crestfallen': 1, 'lauer': 3, 'torsothis': 1, 'philanthropistin': 1, 'harveytime': 1, 'bleachingtrump': 1, 'spokesman': 2, 'jason': 6, 'jobbandai': 1, 'gagabored': 1, 'jupiter': 1, 'mouthgay': 1, 'banal': 1, 'sexmathew': 1, 'ward': 1, 'upadvertising': 1, 'impressioncharles': 1, 'durning': 1, 'hocks': 1, 'fourpound': 1, 'chunk': 1, 'phlegmjaguars': 1, 'raiders': 2, 'postseason': 1, 'exhibition': 2, 'londonfighting': 1, 'pests': 1, 'pesticidesnorth': 1, 'gonzaga': 1, 'ncaa': 4, 'titleif': 1, 'flat': 2, 'havent': 7, 'nowvisit': 1, 'parentsbolton': 1, 'drawn': 4, 'syriaalthough': 1, 'incredibleaustralian': 1, 'asylum': 4, 'seekers': 1, 'choose': 7, 'countriesharry': 1, 'chelsea': 1, 'nipplesworlds': 1, 'seaworld': 2, 'orcason': 1, 'fabre': 1, 'race6': 1, 'jordan96foot': 1, 'flamesnew': 1, 'eva': 3, 'brauns': 1, 'husbandabc': 1, 'starsreport': 1, 'peeking': 1, 'fingersjen': 1, 'aniston': 6, 'inject': 1, 'sht': 2, 'facefilm': 1, 'belatedly': 1, 'swordfish': 1, 'zingermy': 1, 'directions6': 1, 'houseconair': 1, 'doublesided': 1, 'curling': 4, 'flawless': 1, 'burnsthousands': 1, 'otto': 2, 'schoolstudy': 1, 'contentment': 1, 'boredom': 2, 'relaxationhow': 1, 'grownup': 3, 'galleryhistorical': 1, 'widespread': 3, 'powder': 1, 'confounds': 1, 'bewiggedhigh': 1, 'healthcarejames': 1, 'locations': 4, 'hampshirephilippine': 1, 'agrees': 11, 'mindanao': 1, 'martial': 6, 'yearlive': 1, 'representativesshaken': 1, 'witnessing': 2, 'accidentmom': 1, 'nowwomen': 1, 'weightcool': 1, 'cybergranny': 1, 'machines': 3, 'livepenis': 1, 'enlargement': 1, 'dogmovie': 1, 'pamplona': 1, 'bulls': 1, 'rundid': 1, 'rubios': 4, 'loyaltyvoters': 1, 'tune': 2, 'vp': 4, 'yearhillary': 2, 'manfelt': 1, 'clarity': 2, 'christs': 2, 'teachingsmommy': 1, 'movingnew': 1, 'archie': 1, 'graphic': 7, 'jugheadmoderator': 1, 'candidatesomeone': 1, 'trumpputin': 1, 'videoreallife': 1, 'michelin': 2, '87the': 1, 'upsides': 1, 'collegepriyanka': 1, 'chopra': 2, 'simpsons': 7, 'apu': 2, 'justifiednew': 1, 'annul': 1, 'marriagehillary': 1, 'activistsbefore': 1, 'satellite': 3, 'marias': 2, 'dominicathe': 1, 'learnashcroft': 1, 'mexicancongress': 1, 'membersmiss': 1, 'mittenaere': 1, 'crown3': 1, 'thanksgivingman': 1, 'everyonemilo': 1, 'ventimiglia': 2, 'crockpot': 1, 'ellenamazon': 1, 'unhingedarea': 1, 'pronunciationnutter': 1, 'butters': 1, 'nutter': 1, 'workeri': 1, 'mindon': 1, 'monica': 3, 'mountaintop': 1, 'envisions': 1, 'lofty': 1, 'cultureso': 1, 'chemowall': 1, 'reintroduces': 2, 'nudes': 3, 'yearlong': 1, 'experimentturns': 1, 'facecommunists': 1, 'ussenate': 1, 'reach': 14, 'imposing': 1, 'russiawestern': 1, 'speechlocal': 1, 'snakes': 1, 'snakebitethe': 1, 'finale': 7, 'afterburnersbig': 1, 'disruption': 3, 'partnering': 1, 'acceleratorsdeparting': 1, 'k': 5, 'positionaides': 1, 'obstacles': 3, 'greatnesswatch': 1, 'lakes': 2, 'eyesfulltime': 1, 'jobwoman': 1, 'lowpeoples': 1, 'hunk': 4, 'gears': 1, 'holidays': 8, 'waynations': 2, 'listtrumpism': 1, 'bullieswatch': 1, 'animal': 10, 'firescam': 1, 'hyperactive': 1, 'overperfect': 1, 'boring': 7, 'crate': 1, 'barrelprosecutor': 1, 'congressmembers': 1, 'drugs': 12, 'racistdodgers': 1, 'coowner': 1, 'johnson': 16, 'bonkers': 1, 'seriesbush': 1, 'lampmans': 1, 'predisposition': 1, 'pushups': 1, 'weekget': 1, 'arts': 4, 'haiku': 1, 'reviewstrump': 1, 'expels': 2, 'russians': 4, 'consulate': 1, 'seattle': 8, 'attackland': 1, 'communitieswoman': 1, 'housesources': 1, 'happening': 4, 'womanreport': 1, '84': 1, 'contestantsprogress': 1, 'laughs': 3, 'tampon': 4, 'jokesjerry': 1, 'sandusky': 2, 'overturn': 6, 'convictionsanta': 1, 'trump10': 1, 'adulthoodas': 1, 'iiteen': 1, 'snapchat': 4, 'generationtoenails': 1, 'regeneratingsacha': 1, 'grimsby': 1, 'underwearimportance': 1, 'cultural': 5, 'frostwhen': 1, 'sugar': 8, 'answerhow': 1, 'paradise': 4, 'handled': 3, 'shutdownthe': 2, 'weekmccain': 1, 'greyhound': 1, 'delayspotential': 1, 'administrationrupert': 1, 'murdoch': 4, 'nonsense': 2, 'concerns': 5, 'foxred': 1, 'lobster': 5, 'defrosted': 1, 'daysdiphtheria': 1, 'outbreakbangladesh': 1, 'bloggers': 2, 'soonobama': 1, 'parrotcontainer': 1, 'recyclables': 1, 'emptied': 1, 'trashnation': 1, 'din': 1, 'dinbuzz': 1, 'aldrin': 2, 'thunderbirds': 1, 'recordnew': 2, 'cryingarea': 2, 'likes': 23, 'foodtrump': 1, 'victimsphantom': 1, 'thread': 1, 'liked': 4, 'again10': 1, 'cardiovascular': 1, 'diabetesreport': 1, '150000': 3, 'angelstill': 1, 'edgy': 1, 'jacaranda': 1, 'musics': 1, 'decadespacex': 1, 'falcon': 2, 'trip7': 1, 'immediatelyjames': 1, 'clapper': 1, 'institutions': 1, 'assaultsweatstaindating': 1, 'unlocks': 2, 'managersgold': 1, 'mail8yearold': 1, 'lunchaustralia': 1, 'levelssammy': 1, 'flub': 1, 'bossquinoa': 1, 'bean': 3, 'layers': 1, 'proteinmichelangelo': 1, 'mutant': 4, 'ninja': 3, 'michelangeloguy': 1, 'shouts': 3, 'thingchipotle': 1, 'predicts': 3, 'irrelevant': 2, 'chainsreport': 1, 'fifthgraders': 1, 'unwind': 2, 'schoolhow': 1, 'gotham': 2, 'typeface': 1, 'era2': 1, 'laundry': 3, 'dignity': 6, 'sockshollywood': 1, 'delicate': 3, 'subjective': 1, 'opinionstudy': 1, 'truckswe': 1, 'gitmo': 2, 'herepolice': 1, 'subdue': 1, 'suspectmount': 1, 'vernon': 1, 'washingtons': 1, 'quixote': 1, 'beckclinton': 1, 'greet': 2, 'leaderspermission': 1, 'deniedwant': 1, 'hedge': 4, 'managernew': 1, 'amendmentis': 1, 'youtubeus': 1, 'kiptriple': 1, 'baghdad': 4, 'kristin': 1, 'trueecstatic': 1, 'rangerman': 1, 'blaming': 4, 'firestudy': 1, 'eatingsmear': 1, 'michigan': 8, 'officeomg': 1, 'frescojudge': 1, 'tweeting': 2, 'tweetsoap': 1, 'pattern': 3, 'forming': 2, 'bonds': 1, 'began': 3, 'preschoolhow': 1, 'kendall': 2, '8000': 1, '200trumplethinskin': 1, 'fablewhat': 1, 'successes': 2, 'failureshispanics': 1, 'fatherinlaws': 1, 'rantdads': 1, 'poster': 10, 'bedtime': 2, 'deservesus': 1, 'dependence': 2, 'toilron': 1, 'beingtrevor': 1, 'grills': 5, 'tracking': 4, 'proposalsudden': 1, 'restart': 2, 'vomits': 3, 'bilious': 1, 'unsaved': 1, 'screenwoman': 1, 'profoundly': 2, 'lyrics': 4, 'intojessica': 1, 'simpson': 6, 'lampoons': 1, 'halloweenare': 1, 'ontalktome': 1, 'lucas': 3, 'braga': 1, 'otaviano': 1, 'canutostreet': 1, 'photography': 1, 'pt': 2, '2this': 1, 'surviving': 4, 'hardshipreport': 1, 'daredevilwatch': 1, 'meloni': 1, 'undergroundbrothers': 1, 'dennis': 2, 'kucinichs': 1, 'machinepope': 1, 'usseating': 1, 'mixup': 4, 'bennett': 1, 'slipknotairman': 1, 'sistersdisillusioned': 1, 'town5': 1, 'blackowned': 1, 'businessesclinton': 1, 'consults': 1, 'behalf': 2, 'homosexualitythursdays': 1, 'momentum': 3, 'grows': 3, 'bump': 4, 'banstar': 1, 'gamer': 1, 'boldly': 2, 'gamersbernie': 1, 'socialismfrance': 1, 'redemptionnation': 1, 'stuffconcert': 1, 'drastically': 2, 'overestimating': 1, 'desire': 5, 'tricknews': 1, '2017trumps': 1, 'speechbush': 1, 'englishwith': 1, 'remainsgame': 1, 'goblin': 2, 'episodewhen': 1, 'partprince': 1, 'humiliates': 1, 'insurgentsthe': 1, 'unsolved': 1, 'tupac': 2, 'notorious': 1, 'seriesjon': 1, 'hendricks': 1, 'vocalese': 1, '96now': 1, '84000report': 1, 'desirable': 2, 'fallsanders': 1, 'ramps': 1, 'hillarytony': 1, 'blair': 3, 'againstruggling': 1, 'lowerclass': 1, 'votenordstrom': 1, 'salemitt': 1, 'pickadam': 1, 'rippon': 2, 'athleteukraine': 1, 'shelling': 1, 'civilianskylie': 1, 'brucedonald': 1, 'presidentlgbt': 1, 'deeplydivided': 1, 'singapore': 2, 'turnoutcapital': 1, 'centurybackup': 1, 'item': 3, 'wellthoughtout': 1, 'lifeleftover': 1, 'bugles': 1, 'fingers': 4, 'signingostrichfarm': 1, 'witnesseskhloe': 1, 'tristan': 1, 'thompsons': 1, 'babyblack': 1, 'depressedbill': 1, 'recipesyria': 1, 'effectcollege': 1, 'inoperable': 1, 'charityprogressive': 1, 'iranian': 5, 'decemberbat': 1, 'americahistory': 1, 'invention': 1, 'popcorn': 2, 'penicillinhalliburton': 1, 'docked': 1, 'hostagenations': 1, 'checksemployees': 1, 'overthe': 1, 'snowflakes': 1, 'sane': 3, 'whothe': 1, 'superficiality': 1, 'appswill': 1, 'gapsfox': 1, 'muellerjury': 1, 'killerare': 1, 'achievment': 1, 'hardhaving': 1, 'helpwhite': 1, 'signsspider': 1, 'bushs': 4, 'brain12': 1, 'dancers': 5, 'actionhammered': 1, 'chiliscottonelle': 1, 'secretstreet': 1, 'harasser': 2, 'haunted': 7, 'intactive': 1, 'isistwitter': 1, 'kessler': 1, 'organizer': 1, 'rallylas': 1, 'reviewjournal': 4, 'ownerbird': 1, 'cagewarren': 1, 'buffett': 1, 'winnerlittle': 1, 'husky': 1, 'howl': 2, 'doggone': 1, 'duetobama': 1, 'dallas': 4, 'tuesdaywhy': 1, 'linmanuel': 2, 'fearless': 3, 'parenthoodget': 1, 'lifebold': 1, 'videos': 4, 'ontrump': 1, 'outlines': 1, 'protestsall': 1, 'goreaclu': 1, 'stopandfrisk': 1, 'widening': 1, 'practicethe': 1, 'unbelievably': 1, 'politicsjames': 1, 'attackssouth': 1, 'expolicemans': 1, 'selection7': 1, 'styling': 2, 'spruce': 1, 'yeardallas': 1, 'retirereason': 1, 'religion': 6, 'horrifyingtop': 1, 'hud': 1, 'analytica': 4, 'biopamela': 1, 'geller': 1, 'islamophobia': 3, 'businessarea': 1, 'sayinghillary': 1, 'classified': 7, 'emailsdice': 1, 'freezerbar': 1, 'bachelordonald': 1, 'effigies': 1, 'burn': 7, 'ritualresearchers': 1, 'generally': 2, 'plowing': 1, 'hydrant': 1, 'aquariumcooking': 1, 'cuff': 3, 'mushrooms': 1, 'meatloaf': 1, 'eitherf': 1, 'graduate': 7, 'diploma': 2, 'lastno': 1, 'reconciliation': 1, 'talkman': 1, 'brandishes': 1, 'yells': 1, 'couplerepublicans': 1, 'arcane': 1, 'handcuff': 1, 'agenciesarea': 1, 'settled': 1, 'formcdonnelldouglas': 1, 'gaydarus': 1, 'ramping': 1, 'assadbush': 1, 'spiritsprivate': 1, 'vehicles': 1, 'ambulances': 1, 'gunshot': 1, 'stabbing': 5, 'victimsthe': 1, 'hampshirecities': 1, 'hollowpoint': 1, 'wave': 5, 'werewolf': 3, 'slayingsgallup': 1, 'callluck': 1, 'horsescheering': 1, 'maidenlove': 1, 'wwii': 1, 'fatherthe': 1, 'grammysmedical': 1, 'examiner': 1, 'amonderez': 1, 'autopsyus': 1, 'yemengreek': 1, 'woken': 1, 'merkels': 1, 'booming': 1, 'loudspeakers': 1, 'countryscientists': 1, 'invented': 1, 'headband': 1, 'otherunambitious': 1, 'canreporter': 1, 'causes': 9, 'oldfashioned': 2, 'chaosbush': 1, 'utterance': 1, '11new': 1, 'biodiversity': 1, 'busses': 1, 'ecosystemspoll': 1, 'problemsthe': 1, 'nunmarauding': 1, 'hordes': 2, 'helpless': 4, 'actlast': 1, 'herea': 1, 'reuther': 1, 'classjudge': 1, 'sheriff': 6, 'arpaioteacher': 1, 'grading': 1, 'papers': 3, 'punchesarea': 1, 'screwing': 3, 'gains': 5, 'memoriesarea': 1, 'ankle': 3, 'weights': 2, 'routineemilys': 1, 'pickall': 1, 'proceeds': 1, 'charityunpopular': 1, 'ridiculed': 1, 'mercilessly': 3, 'loungeteen': 1, 'sucked': 2, 'kattan': 1, 'leftthe': 1, 'essence': 1, 'magicdazed': 1, 'bezos': 6, 'automate': 1, 'himray': 1, 'lahood': 1, 'roadthese': 1, 'gayby': 1, 'bloody': 6, 'funbernie': 1, 'cuomo': 4, 'electionmorley': 1, 'safer': 6, 'stopwatch': 1, 'locking': 1, 'studioeither': 1, 'stationmiddleaged': 1, 'flashy': 2, 'hearsefoods': 1, 'usbusy': 1, 'girlwhat': 1, 'fundingdemi': 1, 'contemplated': 1, '7deray': 1, 'mckesson': 2, 'baton': 3, 'rouge': 2, 'arreststiffany': 1, 'haddish': 1, 'speechprotest': 1, 'hawaii': 8, 'constructionst': 1, 'elsewhere': 1, 'pa': 3, 'grilled': 2, 'howie': 1, 'mandels': 1, 'biographerchers': 1, 'faintly': 1, 'americasouth': 1, 'mount': 6, 'rushmore': 1, 'quarterian': 1, 'mckellen': 2, 'actresses': 1, 'proposition': 2, 'sexthis': 1, 'underwater': 3, 'harnesses': 1, 'energyheres': 1, 'sandlers': 1, 'netflixsorry': 1, 'trumphow': 2, 'alienating': 1, 'daughterteen': 1, '03gop': 1, 'returnyears': 1, 'denial': 1, 'solutionsamericans': 1, '94523': 1, 'counteract': 1, 'moneys': 3, 'politicsflu': 1, 'clinic': 3, '2009': 2, 'vaccine': 4, 'cheapercnn': 1, 'discussion': 2, 'americaluis': 1, 'gutierrez': 1, 'gohmert': 1, 'immigrantsducks': 1, 'breadfranchesca': 1, 'ramseys': 2, 'brilliantfda': 1, 'smoothiefirefighter': 1, 'threateffective': 1, 'budgetpitbull': 1, 'lewd': 1, 'womenmass': 1, 'recipientsclinton': 1, 'reelected': 2, 'marginmom': 1, 'beaten': 5, 'nowon': 1, 'chargeecowarrior': 1, 'vandana': 1, 'shiva': 1, '40000': 5, 'rejoins': 2, 'antigmo': 1, 'crusade': 2, 'victimwatch': 1, 'paddleboarder': 2, 'straightup': 2, 'wrecked': 1, 'dolphingerman': 1, 'auto': 3, 'engineer': 3, 'issued': 3, 'coatnetworks': 1, 'daytime': 1, 'reallife': 5, 'soap': 5, 'testifiespreroe': 1, 'livesreport': 1, 'pottery': 3, 'previous': 6, 'classcond': 1, 'nast': 1, 'peopleman': 1, 'mayan': 3, 'headdress': 1, 'crates': 2, 'apocalypse': 1, 'happensyahoos': 1, 'programmatic': 2, 'videofirstterm': 1, 'roadblocks': 2, 'tablethousands': 1, 'streets': 9, 'nicaraguan': 1, 'presidentdavid': 1, 'spade': 1, 'shotscott': 1, 'profileaides': 1, 'declining': 3, 'omnipotent': 1, 'godditch': 1, 'appsour': 1, '2015sen': 1, 'warselfdefense': 1, 'simulates': 1, 'attacker': 2, 'erectionthe': 1, 'showdown': 3, 'weekreport': 2, 'diners': 2, 'shakers': 1, 'elaborately': 2, 'heistspicassos': 1, 'guernica': 1, 'triples': 1, 'autographed': 2, '1994': 2, 'rangersgop': 1, 'ladies': 6, 'podium': 4, 'fiorinaoutline': 1, 'inhaler': 1, 'comiccon': 1, 'lycra': 1, 'bodysuitliving': 1, 'uncertainty5': 1, 'crisisvice': 1, 'movespainting': 1, 'highteresa': 1, 'giudice': 1, 'dressgirl': 1, 'writes': 7, 'autismking': 1, 'lear': 3, 'tsunamisean': 1, 'dispelling': 1, 'factual': 1, 'showof': 1, 'peoplehow': 1, 'girlspopeyes': 1, 'boiglerizedi': 1, 'negro': 1, 'wordsall': 1, 'brokenfacebook': 1, 'nowunited': 1, 'belligerent': 1, 'meleenew': 1, 'disasterheres': 1, 'christine': 1, 'baranski': 1, 'fightthis': 1, 'tartanfox': 1, 'fulfilled': 1, 'biblical': 3, 'prophecy': 1, 'embassy': 4, 'jerusalemover': 1, 'disorders': 3, 'survey': 3, 'findstearful': 1, 'wandering': 1, 'matejustin': 1, 'biebers': 1, 'carpool': 4, 'hotumass': 1, 'dartmouth': 2, 'applied': 1, 'terrorismthe': 1, 'intelligencetrailblazing': 1, 'thinker': 1, 'co': 1, 'clayton': 1, 'christensens': 1, 'investment': 3, 'firmundercurrent': 1, 'roiling': 1, '50sthemed': 1, 'dinermccain': 1, 'tucks': 2, 'collarcoming': 1, 'againlittle': 1, 'marshmallows': 1, 'gravy': 2, 'presidentmaya': 1, 'angelou': 1, 'stuffarea': 1, 'daydonald': 1, 'americachristian': 1, 'genderaffirming': 1, 'surgeriesmacarthur': 1, 'noseseabed': 1, 'mh370': 2, 'weekautopsy': 1, 'determines': 2, 'losers': 1, 'corpse': 1, 'contained': 2, 'alcoholincredible': 1, 'documentaries': 2, 'elevate': 2, 'heroeswashington': 1, 'reportscockroach': 1, 'cupboard': 3, 'childrendaily': 2, 'youjoel': 1, 'siegel': 1, 'nightbar': 1, 'complexis': 1, 'armour': 1, 'copying': 1, 'nikes': 2, 'playbookthis': 1, 'reportjohn': 1, 'naacp': 4, 'directorstwitter': 1, 'bs': 2, 'indictmentswhite': 1, 'nuke': 3, 'allhuntsman': 1, 'polling': 7, 'votersgop': 1, 'sticks': 4, 'voteresearch': 1, 'isisturkey': 1, 'linkstv': 1, 'newscastarea': 1, 'excitedly': 7, 'rubbing': 2, 'togetherbill': 1, 'nighy': 1, 'carey': 5, 'mulligan': 2, 'renew': 3, 'lovers': 4, 'quarrelsblack': 1, '500million': 1, 'officereport': 1, '38': 4, 'trips': 7, 'burying': 1, 'shallow': 1, 'desertman': 1, 'skydive': 1, 'contradicting': 1, 'stanceblack': 1, 'paydove': 1, 'deodorants': 1, 'alternativefacts': 1, 'administrationcorbett': 1, 'endorsementsrepublican': 1, 'bundlers': 1, '2008mark': 1, 'incessant': 1, 'oversharing': 1, 'something12': 1, '2015dare': 1, 'tokefire': 1, 'deploys': 6, 'unmarked': 1, 'trucksauthor': 1, 'dismayed': 3, 'purchasesun': 1, 'worldwidereport': 1, 'smells': 5, 'foodborne': 1, 'illnesses': 2, 'annuallyturkish': 1, 'penalizes': 1, 'kurdish': 1, 'mounting': 4, 'tensionstelevisa': 1, 'onair': 1, 'hoaxeducator': 1, 'scandalphysicist': 1, 'acceleratelogan': 1, 'themselvesemeril': 1, 'bams': 1, 'groupiethis': 1, 'sewing': 2, 'refugeesreport': 1, 'lifespan': 2, 'decadesinternal': 1, 'sinclair': 1, 'themdeath': 1, 'moneypenny': 1, 'tnt': 1, 'needed': 8, 'monthlong': 1, 'islamophobiais': 1, 'assetthe': 1, 'liescan': 1, 'gear': 5, 'fergusonstruggling': 1, 'qualified': 3, 'jobafter': 1, 'onehalf': 1, 'personalitywatch': 1, 'ghost': 5, 'tractor': 1, 'swing3': 1, 'showeringus': 1, 'metoo': 4, 'storiesarea': 1, 'encyclopedia': 1, 'lifebradley': 1, 'racks': 1, 'staggering': 4, 'nominationscompassionate': 1, 'trout': 3, 'polluted': 3, 'lakeare': 1, 'ladyinwaitingman': 1, 'againdisastrous': 1, 'appeals': 5, 'intelligencetemp': 1, 'agriculturebruce': 1, 'vilanch': 1, 'sodomized': 1, 'homosexualjoy': 1, 'griswold': 1, 'sportscast': 1, 'hereromney': 1, 'apologizing': 3, 'graveweird': 1, 'als': 2, 'obsessionask': 1, 'transition': 8, 'levelretail': 1, 'spoofs': 3, 'ferris': 2, 'bueller': 1, 'bashing': 1, 'tariffstijuana': 1, 'trade17': 1, 'pics': 2, 'cozy': 2, 'boodvd': 1, 'autopilot': 1, 'deathpresident': 1, 'cabinet': 14, 'officesexperts': 1, 'itrip': 1, 'schiller': 1, 'lucy': 1, 'stomp': 1, 'grapescelebrating': 1, 'monthrise': 1, 'include': 8, 'everyonehow': 1, 'worldwhy': 1, 'slacker': 1, 'anthemswhat': 1, 'wealth': 3, 'isntvariety': 1, 'endorsementrick': 1, 'santorum': 6, 'interracial': 2, 'yetis': 1, 'otherimpoverished': 1, 'kenyan': 1, 'picker': 2, 'sensitivitynpr': 1, 'listener': 1, 'kickass': 2, 'tote': 1, 'bagsupermodels': 1, 'outsidelets': 1, 'lefthanded': 1, 'americabackup': 1, 'jar': 1, 'ointmentnoxious': 1, 'minions': 1, 'satan': 2, 'installation': 1, 'julypretty': 1, 'troian': 1, 'bellisario': 1, 'weds': 2, 'suits': 2, 'patrick': 4, 'j': 6, 'adams': 3, 'rustic': 1, 'ceremonywasted': 1, 'industrywhich': 1, 'headlinesstephen': 1, 'cohns': 1, 'sudden': 3, 'housefrustrated': 1, 'onesky': 1, 'jamour': 1, 'lifedepartment': 1, 'commercialsvery': 1, 'axe': 2, 'discoveredall': 1, 'bloom': 2, 'onwoman': 1, 'terroralert': 1, 'workin': 1, 'hypocrisythanks': 1, 'archery': 1, 'howaustralian': 1, 'historical': 4, 'assaultsreport': 1, 'availabilityfashion': 1, 'smashednew': 1, 'primitive': 1, 'hardware': 1, 'storelake': 1, 'girlthe': 1, 'fossil': 5, 'unstoppablenutella': 1, 'lubricanta': 1, 'roadmap': 1, 'risedripping': 1, 'elevatoraffordability': 1, 'attainment': 1, 'graduationfootlong': 1, 'hoagie': 1, 'rulerreport': 1, 'proguncontrol': 1, 'speechtrump': 2, 'problemnontenuretrack': 1, 'electionhuffpost': 1, '26your': 1, 'inspo': 1, 'weekfor': 1, 'endured': 2, 'horrors': 5, 'arpaios': 2, 'doneadnan': 1, 'syed': 2, 'serialscotland': 1, 'relaxed': 2, 'connery': 1, 'reel': 2, 'outofthisworld': 1, 'amazingbaltimore': 1, 'plywood': 1, 'shuttered': 1, 'businesseschris': 1, 'hemsworths': 1, 'elsa': 1, 'pataky': 1, 'birthdaywalker': 1, 'reaganism': 1, 'ageairasia': 1, 'diversann': 1, 'coulter': 4, 'disappointmentparents': 1, 'solventsviolence': 1, 'protrump': 2, 'rallythis': 1, 'hoped': 1, 'likeurban': 1, 'lining': 3, 'electiontrans': 1, 'gavin': 3, 'grimm': 1, 'cox': 3, 'grammysflorida': 1, 'thatfrom': 1, 'cassoulet': 1, 'southwest': 4, 'makingsighing': 1, 'canirs': 1, 'yearfurious': 1, 'yettaylor': 1, 'lorde': 1, 'celebfilled': 1, 'royals6': 1, 'genuinely': 2, 'unforgivablebag': 1, 'potatoes': 2, 'dirtto': 1, 'trafficking': 1, 'kidsnfl': 1, 'insane': 5, 'treadmilljeb': 1, '3state': 1, 'farewell': 4, 'meetinghostage': 1, 'negotiation': 1, 'congressalqaeda': 1, 'chatter': 2, 'deteriorates': 1, 'gossiptypo': 1, 'wolfmanyears': 1, 'gladhanding': 1, 'sabotaged': 1, 'ideatrumps': 1, 'communityexclusive': 1, 'hayes': 2, 'pasteu': 1, 'limbo': 2, 'jo': 1, 'coxhomeless': 1, 'pitbull': 2, 'trembling': 2, 'lovecensus': 1, 'warrant': 1, 'districtanthropologists': 1, 'unearth': 4, 'blist': 1, 'celebritiesfirst': 1, 'incrediblemewesyria': 1, 'hopejohn': 1, 'properly': 6, 'victims14year': 1, 'bandicoot': 1, 'largely': 5, 'unnoticedtwo': 1, 'coasts': 2, 'stupidthe': 1, 'lagisele': 1, 'bundchen': 1, 'dons': 1, 'bodysuitridley': 1, 'trades': 1, 'crowe': 2, 'burton': 2, 'deppandrew': 1, 'cue': 2, 'teaserjailed': 1, 'tickets': 5, 'attempted': 3, 'pointvirginia': 1, 'regulationschef': 1, 'livelosing': 1, 'mindturkey': 1, 'clamps': 1, '301greys': 1, 'caterina': 1, 'scorsone': 1, 'accuses': 7, 'harassmenta': 1, 'toguy': 1, 'frazzledartists': 1, 'citizenschanging': 1, 'divisions': 1, 'heal': 1, 'themliving': 1, 'journeydean': 1, 'hellers': 1, 'debaclenation': 1, 'pang': 1, 'cruzcan': 1, 'enduring': 1, 'ententeresponsibilities': 1, 'dreamman': 1, 'russiarestaurant': 1, 'servers': 1, 'livable': 1, 'wageaustralia': 1, 'smallest': 2, 'continentfootballs': 1, 'eyepostmodern': 1, 'upgraded': 2, 'preapocalypticlast': 1, 'receivesthe': 1, 'sweetestfarewell': 1, 'partyten': 1, 'debtangry': 1, 'soothed': 2, 'prescriptionsempty': 1, 'yogurt': 1, 'tableau': 1, 'containers': 2, 'windowsilldads': 1, 'breakglee': 1, 'flashes': 2, 'truebuilding': 1, 'futureman': 1, 'updatedconceiving': 1, 'familyman': 2, 'chowing': 1, 'cafman': 1, 'wentan': 1, 'proxies': 1, 'powers': 2, 'endanger': 3, 'civilianssystems': 1, 'traineewatch': 1, 'decimate': 2, 'kitsscientists': 1, 'flooralcoholic': 1, 'recoveredthe': 1, 'kidsclassmates': 1, 'awed': 2, 'daypanama': 1, 'misconductwhite': 1, '5th': 2, 'conceptionpassage': 1, 'democratrepublican': 1, '317622318047donald': 1, 'adsports': 1, 'deemphasizedelizabeth': 1, 'toomey': 2, 'swindle': 1, 'townseducation': 1, '2movie': 1, 'marketed': 2, 'genresteen': 1, 'reviewed': 2, 'commercials': 5, 'youmueller': 1, 'awaywyclef': 1, 'fugees': 1, 'reunionabc': 1, 'updatehiroshima': 1, 'guilt': 1, 'japanchicago': 1, 'thunderstorm': 1, 'tent': 2, 'collapsesgetting': 1, 'homophobiadying': 1, 'mastermind': 1, 'levervirgin': 1, 'galactic': 3, 'supersonic': 1, 'airplanereport': 1, 'inconceivable': 1, 'politicsassistant': 1, 'smuggle': 1, 'cakedecorating': 1, 'martha': 3, 'stewartdog': 1, 'waited': 1, 'returnreport': 1, 'workkelsea': 1, 'ballerini': 1, 'musicoxycontin': 1, 'maker': 4, 'campaignlady': 1, 'belts': 3, '50stage': 1, 'lypsinka': 2, 'trilogy': 1, 'billy': 4, 'mozarts': 1, 'flutefans': 1, 'beg': 3, 'aerosmith': 1, 'drugshow': 1, 'insuredarmy': 1, 'conducts': 1, 'exhaustive': 1, 'probetrump': 1, 'cruelest': 1, 'yetsecret': 1, 'disburses': 1, 'healbarack': 1, 'skewers': 2, 'accessthese': 1, 'alonenatalee': 1, 'holloway': 1, 'osama': 3, '5year': 2, 'anniversarydying': 1, 'weeksbon': 1, 'apptit': 1, 'proquiche': 1, 'botsconcerned': 1, 'shootingwoman': 1, 'dots': 1, 'markswhat': 1, 'kendrick': 4, 'owe': 4, 'listenersman': 1, 'shape': 10, 'himwife': 1, 'videotaping': 1, 'elk': 3, 'lifefirst': 1, 'mapmakers': 1, 'opera': 2, 'valuevillereport': 1, 'revolving': 2, 'secretscomplex': 1, 'gutter': 1, 'purposes': 2, 'listman': 2, 'lied': 3, 'profiletrump': 1, 'environmental': 4, 'sustainabilitypolish': 1, 'irish': 5, 'hooligans': 1, 'embraceenormous': 1, 'coastlinebookclub': 1, 'oscarsdogs': 1, 'stations': 4, 'dayrussian': 1, 'reddit': 1, 'commentersecret': 1, 'unofficial': 1, 'operationsaudi': 1, 'explicit': 2, 'permission': 3, 'votelee': 1, 'greenwood': 1, 'iraqman': 1, 'mincedzeev': 1, 'aram': 1, 'videooscars': 1, 'capote': 1, 'categoryvice': 1, 'expansive': 1, 'microstates': 1, 'knowwhat': 1, 'contents': 1, 'purse': 2, 'youellen': 1, 'nugget': 1, 'sabotaging': 2, 'recordresearch': 1, 'dirtynation': 1, 'basset': 1, 'houndsinternet': 1, 'explorer': 2, 'overture': 1, 'browsersupposed': 1, 'buff': 3, 'yetirish': 1, 'nun': 1, 'silky': 1, 'heavenly': 1, 'kickabout': 1, 'copbrief': 1, 'passage': 6, 'manhoodmoviegoer': 1, 'goman': 1, 'psyches': 1, 'flavorman': 2, 'instincthistory': 1, 'repeats': 1, 'itselfenglish': 1, 'soccers': 1, 'outofnowhere': 1, 'machineeveryone': 1, 'bustling': 2, 'elude': 1, 'pursuersdonald': 1, 'triumphs': 1, 'vermonteating': 1, 'depressionthousands': 1, 'footy': 1, 'mcfooty': 1, 'facetrump': 1, 'antisemiticlowly': 1, 'mortal': 3, 'hellkkk': 1, 'blacks': 4, 'hangoverneither': 1, 'isfox': 1, 'apatows': 1, 'virginperson': 1, 'parked': 3, 'upstanding': 2, 'communityblowing': 1, '1400': 1, 'percentjimmy': 1, 'spots': 6, 'oreillyman': 1, 'bosswebmd': 1, 'thisno': 1, 'counting': 2, 'seattleisis': 1, 'psychojanelle': 1, 'monae': 1, 'freeus': 1, 'coolness': 1, 'princewhats': 1, '2015this': 1, 'herocharlie': 1, 'regretselementary': 1, 'studentworthless': 1, 'beeramber': 1, 'reduction': 1, 'wednesdayguinness': 1, 'pounds': 17, 'editorinchiefjohn': 1, 'obamacaremaybe': 1, 'awhile': 1, 'selfthe': 2, 'internetfree': 1, 'vacationold': 1, 'stared': 3, 'atwoman': 1, '75': 3, 'centstaco': 1, 'bells': 3, 'ingredients': 4, 'combined': 1, 'waydepressed': 1, 'klugmannasa': 1, 'astronaut': 2, 'lovell': 1, 'jobbaltimore': 1, 'lifetrumps': 1, 'worries': 3, 'watchdogsgay': 1, 'videoofficemates': 1, 'unwittingly': 1, 'workday': 1, 'grindr10': 1, 'spoilers': 1, '2area': 2, 'telemarketer': 1, 'awash': 1, 'miseryunited': 1, 'petsparents': 1, 'fluctuating': 1, 'asleep': 6, 'upwatching': 1, 'itselfmartin': 1, 'comprehensive': 3, 'planjohnson': 1, 'tarnished': 2, 'gamesports': 1, 'slop': 1, 'healing': 4, 'bostonpastor': 1, 'spiel': 1, 'longfrustrated': 1, 'onopposition': 1, 'annulment': 1, 'erdogan': 1, 'powerstrumps': 1, 'youd': 6, 'childmodernday': 1, 'caligula': 1, 'bagelquestions': 1, 'closettoo': 1, 'eatrebel': 1, 'sneaks': 6, 'tattoothis': 1, 'violate': 2, 'rightsputin': 1, 'fried': 2, 'chickenfrom': 1, 'thanksgivingfran': 1, 'drescher': 2, 'screeches': 1, 'awarenesssupposed': 1, 'feelingsmarco': 1, 'nabs': 3, 'caucussmall': 1, 'procedure': 1, 'managers': 2, 'crowning': 1, 'achievementeric': 1, 'seared': 1, 'foie': 1, 'gras': 1, 'jryou': 1, 'mearea': 1, 'panties': 1, 'bunchadam': 1, 'levine': 2, 'maroon': 1, 'oscarsafghanistan': 1, 'solemnly': 2, 'platoon': 1, 'undiagnosed': 2, 'ptsdstarcraft': 1, 'void': 1, 'novembersenate': 1, '720': 1, 'rightsparents': 1, 'goodother': 1, 'nurse': 2, 'funnynew': 1, 'decoy': 2, 'presidentsnations': 1, 'deans': 1, 'wildwebsites': 1, 'layout': 2, 'deepest': 1, 'betrayalthis': 1, 'shutdownwe': 1, 'trumpcare': 3, 'fornations': 1, 'jobsaudi': 1, 'jobhuffpost': 1, '95': 3, 'starring': 2, 'adventurous': 1, 'girlseast': 1, 'magazineartist': 1, 'jamnew': 1, 'genitalianovelty': 1, 'amgop': 1, 'hostage': 2, 'cutssalmonella': 1, 'sickens': 2, 'multistate': 1, 'recalldays': 1, 'politicsgod': 1, 'presidentburger': 1, 'lowfat': 1, 'cashiertwitter': 1, 'usefulnew': 1, 'wisely': 1, 'educationmom': 1, 'ceramic': 1, 'dealerman': 1, 'mm': 1, 'gods': 7, 'imagethe': 1, 'weekmoms': 1, 'postpartum': 4, 'lossnew': 1, 'aetna': 1, 'procedures': 1, 'outcongressman': 1, 'ondean': 1, 'cain': 2, 'fanpage': 1, '81496things': 1, '12yearold': 4, 'comingdonald': 1, 'ignorance': 3, 'affairs': 3, 'problemcan': 1, 'packers': 1, 'minnesotaobama': 1, 'wnba': 1, 'championspodcast': 1, 'manor': 1, 'robotsno': 1, 'happyleast': 1, 'dogpolice': 1, 'campwalkable': 1, 'smartersafety': 1, 'studentsliterally': 1, 'falsechilds': 1, 'video4': 1, 'highlevel': 3, 'languages': 1, 'frontend': 1, 'developersshitty': 1, 'mona': 3, 'lisameatless': 1, 'seed': 3, 'festivaleverything': 1, 'recordbryan': 1, 'bishop': 3, 'outvets': 1, 'bostons': 1, 'audiobehind': 1, 'curtain': 2, 'patriots': 2, 'dynastywater': 1, 'pistol': 1, 'fired': 14, 'sideways': 2, 'gangsta': 1, 'gripproven': 1, 'craigslistsacred': 1, 'siteswhy': 1, 'statehouse': 1, 'interns': 3, 'vulnerable': 4, 'harassmentmike': 1, 'tortured': 1, 'firstarea': 1, 'backup': 4, 'attractionkentucky': 1, 'alison': 2, 'lundergan': 1, 'grimesmatt': 1, 'attracted': 1, 'ponytailthe': 1, 'atx': 1, 'magicalscientists': 1, 'earthlike': 2, 'economylittle': 1, 'leaguers': 2, 'booted': 1, 'postfullcircle': 1, 'rooted': 1, 'cancerthe': 1, 'alexander': 2, 'pharrell': 3, 'williamsunited': 1, 'cracking': 1, 'spousesclinton': 1, 'bleeds': 2, 'proved': 6, 'standard': 5, 'wordamericans': 1, 'rulingteam': 1, 'sherpas': 1, 'scale': 2, 'everest': 3, 'yearsguy': 1, 'outthe': 2, '25': 6, 'lovesnew': 1, 'tvkendrick': 1, 'untitled': 1, 'unmasteredearth': 1, 'explodesxfiles': 1, 'reboot': 3, 'scullyfourthgrader': 1, 'tooth': 1, 'whitetheres': 1, 'hbcussenate': 1, '3year': 2, 'billyour': 1, 'themfirst': 1, 'chess': 3, 'piecesprogressives': 1, 'infrastructure': 5, 'spendingheartless': 1, 'curators': 3, 'deranged': 2, 'scrawlings': 1, 'display': 6, 'mocknra': 1, 'daygovernor': 1, 'leadsreport': 1, 'bosses': 2, 'yeardespite': 1, 'madaya': 1, 'siegesblog': 1, 'years9': 1, 'augustlocal': 1, 'scope': 1, 'joined': 3, 'armycat': 1, 'cushions': 1, 'vaccinejohn': 1, 'notopening': 1, 'upstaged': 1, 'preshow': 1, 'musicthe': 1, 'norwegian': 3, 'pantsmerkels': 1, 'antiimmigrant': 3, 'afd': 1, 'electionoffice': 1, 'unfriendly': 1, 'remindersthai': 1, 'premier': 3, 'bucket': 5, 'birdflu': 2, 'fearsfirst': 1, 'turtles': 2, 'bebop': 1, 'rocksteadyrookie': 1, 'crimescene': 1, 'tapewhite': 1, 'blinding': 1, 'maleness': 1, 'circlestudy': 1, 'breathteen': 1, 'blinded': 2, 'alive': 10, 'duct': 1, 'challengesen': 1, 'warner': 5, 'powerman': 1, 'policesunday': 1, 'roundupwho': 1, 'congo': 1, 'casesman': 1, 'salutes': 2, 'jake': 4, 'gyllenhaal': 2, 'brokeback': 1, 'mountainonly': 1, 'segways': 1, 'collidejane': 1, 'narrator': 3, 'cultureamerican': 1, 'recap': 7, 'circus': 1, 'spoilersjury': 1, 'itcommas': 1, 'everywherethe': 2, 'strong': 13, 'transportationcnn': 1, 'unafraid': 2, 'factoidsbbc': 1, 'upgrades': 2, 'flap': 1, 'rowsaddam': 1, 'enrages': 1, 'complianceandy': 1, 'dramaadvertiser': 1, 'offset': 2, 'mixedtypography': 1, 'fontchina': 1, 'intensifies': 2, 'koreapabst': 1, 'coasting': 4, '1893': 1, 'win11': 1, 'twins': 5, 'shambles': 1, 'youvictoria': 1, 'pokes': 2, 'pout': 1, 'postget': 1, 'tivo': 1, 'everythingsee': 1, 'moons': 1, 'crater45': 1, 'sayjohn': 1, 'dustin': 5, 'questionsgrieving': 1, 'losses': 1, 'deathcargo': 1, 'ships': 2, 'whalesthis': 1, 'indie': 4, 'filmssolange': 1, 'isaly': 1, 'raisman': 3, 'nassarencouragement': 1, 'motivating': 1, 'indefinitelycbs': 1, 'schieffers': 1, 'thisa': 1, 'rioladies': 1, 'alongdear': 1, 'oxford': 2, 'dictionaries': 2, 'pwnage': 1, 'betrain': 1, 'slices': 1, 'crashbrazil': 1, 'dilma': 2, 'rousseff': 2, 'trialstudy': 1, 'concertarchaeologists': 1, 'hominid': 1, 'themarea': 2, 'songaustria': 1, 'germany': 3, 'offloaded': 1, 'hungaryhumor': 1, 'uniform': 2, 'submissions': 1, 'lowbloodspattered': 1, 'suri': 2, 'drags': 3, 'doorstepput': 1, 'damn': 9, 'swimsuitanother': 1, 'copies': 4, 'constitutionthe': 1, 'highwayschemistry': 1, 'leaderspresident': 1, 'nukes16': 1, 'universally': 2, 'aboutheres': 1, 'truethe': 1, 'derangementdoritos': 1, 'goodoregon': 1, 'bidboyfriend': 1, 'trusted': 3, 'queueeric': 1, 'clapton': 2, 'slower': 1, 'laylaman': 1, 'stuffs': 1, 'obamacareenron': 1, 'fundconservative': 1, 'lossstephen': 1, 'egos': 2, 'cabinetrerun': 1, 'adjusted': 2, 'inflationmotorcyclist': 1, 'salvaged': 1, 'partsreminder': 1, 'disick': 3, 'kourtney': 2, 'kardashians': 4, 'webat': 1, 'gridiron': 1, 'unnew': 1, 'shattereddea': 1, '280': 1, 'bribebe': 1, 'bestselling': 2, 'stewman': 1, 'experiencesjimmy': 1, 'nowtehran': 1, 'normalityend': 1, 'enacting': 1, 'legislationlawrence': 1, 'taylors': 1, 'lynette': 1, 'legendwoman': 1, 'owning': 4, 'crippling': 2, 'debtspacedout': 1, 'flower': 1, 'groovin': 1, 'wavenew': 1, 'partially': 2, 'digested': 1, 'doritos': 5, 'tedious': 1, 'chewingviral': 1, 'dudes': 2, 'themhmo': 1, 'targets': 8, 'campaigntrump': 2, 'trophyastronomers': 1, 'neptune': 1, 'upthe': 3, 'scorpionrepublicans': 1, 'gravitysc': 1, 'statehouseprotesters': 1, 'ignoredsecond': 1, 'fabric': 3, 'eightyearoldrepopulation': 1, 'beginsa': 1, 'realistic': 5, 'momesl': 1, 'textbook': 3, 'concentrates': 1, 'foodpreparation': 1, 'vocabularywhat': 1, 'wantcolbert': 1, 'proctoscopeon': 1, 'remainderthousands': 1, 'missingnative': 1, 'ramp': 1, 'rebrand': 1, 'columbus': 2, 'daytweeters': 1, 'ridicule': 1, 'scrapped': 2, 'visitcherishing': 1, 'hardtrump': 1, 'hostshillary': 1, 'sigh': 3, 'caucusesfourthgrader': 1, 'blank': 1, 'occurrednasa': 1, 'chimpanzee': 1, 'sunrevisiting': 1, 'dealarctic': 1, 'committeecomicbook': 1, 'superrman': 1, 'impervious': 2, 'copyediting10': 1, 'incorrect': 1, 'bigoted': 1, 'plain': 5, 'wrongbernie': 1, 'contested': 2, 'conventionwhy': 1, 'dohillary': 2, 'torches': 1, 'lip': 5, 'trumpnation': 1, 'rather': 12, 'subsequent': 2, 'yearsus': 1, 'cambodia': 1, 'diplomatic': 4, 'didreport': 1, 'saycourt': 1, 'peersfather': 1, 'sonthe': 1, 'schedule': 2, 'sleephandels': 1, 'messiah': 2, 'testamentherman': 1, 'fucklasik': 1, 'monocleairborne': 1, 'luckiest': 1, 'inchesandie': 1, 'macdowell': 1, 'audiences': 2, 'hinders': 1, 'actorsthe': 1, 'fuller': 2, 'teases': 6, 'funthe': 1, 'alarming': 2, 'shortfall': 1, 'womenprepare': 1, 'hypnotized': 1, 'puppies': 3, 'dinnercias': 1, 'brennan': 1, 'follyukraines': 1, 'donbas': 1, 'southrepublican': 1, 'officialsboehner': 1, 'vacuum': 2, 'intactlevis': 1, 'implicated': 2, 'denim': 2, 'cowswho': 1, 'ebolalouvre': 1, 'hurry': 2, 'gogh': 1, 'donor': 3, 'visitthis': 1, 'remixes': 1, 'rap': 7, 'boujee': 1, 'lessonscatapults': 1, 'booksjohn': 1, 'greets': 2, 'ties6day': 1, 'profile': 8, 'picturethis': 1, 'register': 5, 'voteeden': 1, 'baylee': 1, 'stranger': 14, 'sunsetthe': 1, 'marathonsrosie': 1, 'trumpshannon': 1, 'tweed': 1, 'nudity': 1, 'agingwhy': 1, 'industryrupaul': 1, 'reinforcing': 1, 'rebel': 6, 'against5yearold': 1, 'peeing': 2, 'urinals': 1, 'downthe': 3, 'aboutus': 1, 'retakes': 1, 'rankingskobe': 1, 'feuds': 1, 'adsenate': 1, 'woo': 2, 'kool': 1, 'kfc': 3, 'watermelonsthe': 1, 'wimbledon': 1, '2015golden': 1, 'retriever': 2, 'mauls': 1, 'apologistsjessie': 1, 'decker': 2, 'bodiessupreme': 1, '71': 1, 'scalias': 2, 'killerwaymo': 1, 'racethese': 1, 'coloring': 3, 'coolcoast': 1, 'boca': 1, 'raton': 1, 'irmanorth': 1, 'intercontinental': 1, 'missilestudy': 1, 'cows': 2, 'lifescott': 1, 'fumigatedsocial': 1, 'assassins': 3, 'creed': 2, 'levels': 6, 'moviesretired': 1, 'anthemninjalike': 1, 'demonstrate': 3, 'babymuslim': 1, 'texasdallas': 1, 'djener': 1, 'sur': 1, 'lherbe': 1, 'painting': 3, 'monetheroic': 1, 'footlong': 1, 'teriyakififth': 1, 'nostalgia': 1, 'agony': 1, 'disillusionmentscientists': 1, 'syriaremembering': 1, 'schoolsame': 1, 'flightmarco': 1, 'onehow': 1, 'pessimism': 1, 'weightlofty': 1, 'shovel': 3, 'width': 2, 'driveway': 3, 'scaled': 1, 'shoveling': 1, 'carplanet': 1, 'lightsoff': 1, 'outstudy': 1, 'loserswhen': 1, 'detainee': 1, 'bothering': 2, 'neuter': 1, 'loser': 6, 'catdemocrat': 1, 'ballplayers': 1, 'wagehow': 1, 'outsmart': 2, 'populists': 1, 'franceanas': 1, 'rested': 1, 'doubt': 4, 'wordlifeguard': 1, 'drowning': 3, 'godsculptor': 1, 'objectsikea': 1, 'australias': 2, 'wests': 4, 'perfectdirector': 1, 'aspca': 1, 'dogsolive': 1, 'awardsdoctors': 1, 'infallible': 1, 'mostentire': 1, 'dreading': 6, 'fordad': 1, 'enjoyable': 1, 'isstewarthannity': 1, 'weirdfda': 1, 'tabasco': 1, 'boyman': 2, 'cashew': 1, 'buttermom': 1, 'unsolicited': 3, 'bodylet': 1, 'postqueer': 1, 'experiencereport': 1, 'gotten': 5, 'nowtaco': 1, '10000rupert': 1, 'cableinspired': 1, 'filmmen': 1, '2016s': 1, 'commercialsgirl': 1, 'critique': 1, 'voicespiderman': 1, 'distracts': 1, 'octopus': 1, 'hostess': 3, 'piespalestinians': 1, 'shieldsarea': 1, 'achieves': 2, 'dreamnew': 1, 'regulations': 4, 'plants': 2, 'loopholes': 1, '2030news': 1, '2017what': 1, 'moderate': 5, 'carolinadennys': 1, '3000spideregg': 1, 'omeletmaker': 1, 'rethinks': 1, 'storyreport': 2, 'battlebots': 1, 'cpu': 1, 'injuries3yearold': 1, 'pretending': 2, 'accidental': 4, 'pregnancymore': 1, 'cds': 4, 'residencenewborn': 1, 'terrorized': 2, 'shapeless': 1, 'blobsnation': 1, 'nickatnite': 1, 'lineupnations': 1, 'stillundecided': 1, 'seatbelts': 1, 'offjk': 1, 'horcrux': 1, 'onedomestic': 1, 'warnssyria': 1, 'pound': 2, 'rebelheld': 2, 'advances': 4, 'halttoby': 1, 'joining': 3, 'menonly': 1, 'concertbaby': 1, 'oversize': 1, 'multicolor': 2, 'keysreport': 1, 'bounding': 1, 'injurieswoman': 1, 'pointy': 1, 'shoeshow': 1, 'educating': 1, 'businessamazon': 1, 'competitors': 1, 'banning': 4, 'chromecasttales': 1, 'woe': 1, 'associated': 4, 'pressdesperate': 1, 'barnes': 1, 'noble': 2, 'tablets': 2, 'storehazards': 1, 'culturekellyanne': 1, 'pences': 3, 'stuntfcc': 1, 'premium': 1, 'comment': 3, 'fasttrack': 1, 'complaints': 4, '4999': 1, 'monthjessica': 1, 'againnet': 1, 'weekcalifornia': 1, 'farmer': 3, 'cutbacks': 1, 'judgeheroic': 1, 'goldfish': 1, 'flushingtroubled': 1, 'comeys': 5, 'firingjake': 1, 'lyingmom': 1, 'classdeep': 1, 'vomitingfox': 1, 'programmingdrinking': 1, 'pandado': 1, 'succeedclearing': 1, 'aereosecond': 1, 'piazza': 1, 'cavalleria': 1, 'rusticana': 1, 'pagliacci': 1, 'metjohn': 1, 'assuming': 1, 'ignore': 10, 'militarybill': 1, 'bribe': 1, 'umpiresthe': 1, 'fallgop': 1, 'decisive': 1, 'americansconservatives': 1, 'mcconnellbudget': 1, 'agreeing': 1, 'subsidy': 1, 'dealprizes': 1, 'addressthe': 1, 'rinpochelocal': 1, 'leather': 3, 'reasonexecutives': 1, 'bankrupt': 2, 'authority': 3, 'bonuses': 1, 'deniedman': 1, 'snigletslunchbox': 1, 'medicationslifesaving': 1, 'accessible': 4, 'americansan': 1, 'nsyncs': 1, 'bandmates': 1, 'timberlake5': 1, 'weightloss': 1, 'dell': 1, 'refundsean': 1, 'withholding': 3, 'privacya': 1, 'somali': 2, 'storyreturning': 1, 'unceremoniously': 1, 'toss': 3, 'trash19': 1, 'messy': 3, 'imperfect': 1, 'finalethe': 2, 'hunky': 1, 'wellstrung': 1, 'smashreport': 1, 'doughnutssewage': 1, 'portapotties': 1, 'stinky': 1, 'messcabletv': 1, 'overruled': 1, 'networktv': 1, 'judgemigrant': 1, 'greeces': 1, 'intercultural': 1, 'schoolsthat': 1, 'comebacksupportive': 1, 'encourage': 3, 'interests': 5, '15minute': 1, 'drivehillary': 1, 'coursing': 2, 'bodythe': 2, 'weekliving': 1, 'shadow': 6, 'laterbush': 1, 'sr': 1, '80sman': 1, 'schoolelevenyearold': 1, 'shield': 4, 'dodgeball': 1, 'gamecontaining': 1, 'steph': 1, 'impossibleafter': 1, 'methadone': 1, 'sensemischa': 1, 'barton': 1, 'stars23hour': 1, 'failurereport': 1, '156': 1, 'moreobama': 1, 'neighborhoods': 2, 'gogrief': 1, 'passben': 1, 'stable': 4, 'zoolander': 1, 'ridiculously': 4, 'funnybig': 1, 'obamacaresetting': 1, 'campusfat': 1, 'euphemizedsenate': 1, 'equifax': 2, 'insider': 4, 'tradingunderstanding': 1, 'stateus': 1, 'justices': 3, 'plazasony': 1, 'execs': 4, 'prejudicial': 1, 'whyefforts': 1, 'muchbroken': 1, 'hearts': 8, 'eclairsnevada': 1, 'noncitizens': 1, 'statethe': 1, 'tennessees': 1, 'booknation': 1, 'dutifully': 1, 'watches': 4, 'moviehow': 1, 'historyresidents': 1, 'decadesfriends': 1, 'facebook3day': 1, 'practically': 2, 'overbabyshower': 1, 'zinfandelnations': 1, '2012wrecking': 1, 'revitalise': 1, 'paulo': 1, 'razes': 1, 'buildings': 2, 'safetypolitico': 1, 'advancethe': 1, 'nonsensical': 1, 'proposalchris': 1, 'goonies': 1, 'sequelstudy': 1, 'overdoses': 1, 'prevented': 1, 'heroingwen': 1, 'albumit': 1, 'shamdonald': 1, 'shared': 5, 'dadsure': 1, 'apartpurchase': 1, 'jeans': 2, 'ushers': 3, 'ultracool': 1, 'subculture': 5, 'jeanswearing': 1, 'americansonline': 1, 'amass': 1, 'pacethe': 1, 'listeningloser': 1, 'pregnantbye': 1, 'bye': 1, 'byelibrarian': 1, 'amassed': 1, 'humbly': 1, 'collegeiran': 1, 'beneficiary': 1, 'syriathe': 2, 'videohammering': 1, 'americapolls': 1, 'bitterly': 2, 'eu': 7, 'referendum14': 1, 'reps': 2, 'abuser': 2, 'nothingmoms': 1, 'monsantos': 1, 'roundupwells': 1, 'scandalgroup': 1, 'dodge': 2, 'teamjim': 1, 'carreys': 1, 'joker': 1, 'nightmareswomans': 1, 'degeneres': 2, 'showchanning': 1, 'gambit': 2, 'till': 3, 'doa': 1, 'nutcracker': 1, 'photoslouie': 1, 'formwhy': 1, 'robotshistorical': 1, 'facial': 5, 'corsets': 1, 'finallythe': 1, 'resurrection': 1, 'bushreport': 1, 'employers': 5, 'applicantgenerous': 1, 'improv': 1, 'troupe': 1, 'freetoday': 1, 'transvestiteclaire': 1, 'fantasized': 1, 'aboutjeb': 1, 'candidaterumsfeld': 1, 'jerkoff': 1, 'motions': 1, 'powell': 2, 'meetingpuzzled': 1, 'wherebiden': 1, 'snakean': 1, 'inconvenient': 1, 'truthfox': 1, 'holocaustonly': 1, 'guts': 1, 'goodadventures': 1, 'backyardmarilu': 1, 'henner': 1, 'midlevel': 1, 'talentfinancial': 1, 'advises': 3, 'spanstudy': 1, '86': 3, 'stadiums': 2, 'execution12': 1, 'embarrass': 1, 'sonsclinton': 1, 'hillaryflu': 1, 'bodysight': 1, 'horizon': 2, 'arrival': 3, 'statethis': 2, '10pound': 1, 'doughnut6': 1, 'necessities': 1, 'needtimetraveling': 1, 'waytexas': 1, 'classes': 3, 'mexicanamerican': 2, 'studiesstrapping': 1, 'congresswhat': 1, 'honolulus': 1, 'meanbutterfly': 1, 'womanhoodcriminal': 1, 'himtoddler': 1, 'hooked': 2, 'exhilarating': 1, 'highbriefcase': 1, 'pornthese': 1, 'itolive': 1, 'bottle': 6, 'betterstop': 1, 'combatting': 1, 'ocdvin': 1, 'rolenew': 1, 'excites': 2, 'percentage': 2, 'cares': 2, 'anymorenew': 1, 'contributions': 2, 'tucked': 2, 'congressmens': 1, 'pocketfamed': 1, 'homaro': 1, 'cantu': 1, 'deaddrunk': 1, 'streaks': 1, 'priceyeti': 1, 'abdominable': 1, 'videojoe': 1, 'beau': 1, 'obligation': 1, 'dutythese': 1, 'marvels': 2, 'amazing34000': 1, 'petition': 5, 'archbishop': 3, 'popeprocrastinating': 1, 'coronary': 1, 'bypass': 1, 'hospitalnight': 1, 'watchman': 1, 'leno': 2, 'surveillancegeorge': 1, 'museumdonald': 1, 'cookieconservatives': 1, 'nomineebored': 1, 'vetting': 1, 'viable': 1, 'emergescome': 1, 'presidentus': 1, 'incentives': 2, 'turnkathie': 1, 'gifford': 1, 'sincerity': 1, 'implantsarea': 1, 'barber': 2, 'nowravaged': 1, 'floridas': 3, 'adjectivecloseddoor': 1, 'honeyroasted': 1, 'peanutsvideo': 1, 'ecigarette': 1, 'handbagpolice': 1, 'buried': 7, 'alivemiddleaged': 1, 'kittens': 2, 'internetnbc': 1, 'seasonsstrom': 1, 'thurmond': 1, 'cabinetfans': 1, 'greats': 1, 'chuck': 7, 'berrydolphins': 1, 'solving': 2, 'tasksmass': 1, 'diversitystudy': 1, 'undetectable': 1, 'load': 3, 'transmit': 1, 'virusdjimon': 1, 'hounsou': 1, 'worldwatch': 1, 'storyline': 1, 'actuallyhow': 1, 'unregulated': 1, 'madrassa': 1, 'sows': 1, 'strife12': 1, 'mailguy': 1, 'caps': 2, 'snowdenreport': 1, 'invitations': 1, 'unprovokedthe': 1, 'outrageous': 3, '2015the': 1, 'apologizesamericas': 1, 'breuer': 1, 'nowguy': 1, 'canvassed': 1, 'canvassing': 1, 'placethis': 1, 'pledging': 1, 'tuition': 2, 'displaced': 1, 'harveyghost': 1, 'brando': 1, 'cheesecakechristina': 1, 'aguilera': 3, 'recognizable': 1, 'covernew': 1, 'blankenship': 1, 'touts': 8, 'mining': 1, 'disasternorth': 1, 'evillooking': 1, 'brighter': 1, 'featurecalifornias': 1, 'drugsgrown': 1, 'bane': 1, 'givethe': 1, 'coasterstars': 1, 'canceled': 5, 'movieregular': 1, 'enforces': 1, 'ruleis': 1, 'penaltybipartisan': 1, 'flynnsecretary': 1, 'codeus': 1, '10year': 2, 'lowvin': 1, 'collar': 1, 'outcroppingchinese': 1, 'square': 8, 'trillionreport': 1, 'biasedcarson': 1, 'wronglooking': 1, 'ceilingivanka': 1, 'hired': 5, 'herworld': 1, 'deathhistorical': 1, 'historylocal': 1, 'yelpal': 1, 'thirdthese': 1, 'quotes': 5, 'oddly': 2, 'insightfuldonald': 1, 'flush': 2, 'aparthow': 1, 'ostrich': 1, 'videonasas': 1, 'kepler': 1, 'spacecraft': 2, 'recovers': 2, 'unexplained': 3, 'mode10': 1, 'cameron': 2, 'diazarchivists': 1, 'unpublished': 2, 'crichton': 1, 'operates': 1, 'hitchseaworld': 1, 'whales': 4, 'shownations': 1, 'substitute': 3, 'thattrumps': 1, 'outcomesthe': 1, 'prisontrump': 1, 'privately': 3, 'someday': 1, 'forwardall': 1, 'swifts': 5, 'bffs': 1, '1989': 3, 'tourwhy': 1, 'jb': 1, 'smoove': 1, 'oscarsdemocrats': 1, 'oversight': 1, 'interestbush': 1, 'straightfocus': 1, 'debatecost': 1, 'paperlibrary': 1, '3d': 3, 'printer': 1, 'prosthetic': 4, 'girlone': 1, 'humanitarians': 1, 'profound': 3, 'eraava': 1, 'duvernay': 3, 'weaponbecause': 1, 'bootsmom': 1, 'sonnovelty': 1, 'pencil': 1, 'nubfbi': 1, 'undermine': 4, 'unaccountable': 1, 'agenciesreport': 1, 'alleges': 2, 'facilities': 2, 'bordereven': 1, 'geneticguy': 1, '32third': 1, 'knockedover': 1, 'upabusive': 1, 'childarea': 1, 'committeejapan': 1, 'anniversarysystems': 1, 'traineeactor': 1, 'whateverincreasingly': 1, 'dye': 3, 'ivankamlb': 1, 'policymeryl': 1, 'hanks': 3, 'charactersthoughtful': 1, 'borrowing': 1, 'itembattled': 1, 'rove': 4, 'countryhow': 1, 'duped': 3, 'itemssurgeon': 1, 'exercising': 1, 'healthkim': 1, 'sub': 2, 'success4': 1, 'harms': 1, 'securitycongress': 1, 'addresscomey': 1, 'eitherkim': 1, 'westpresident': 1, 'nationsoutherner': 1, 'recognized': 3, 'drivinginacircleman': 1, 'closed': 4, 'closezuckerberg': 1, 'commenting': 1, 'facebooksuicide': 1, '30mark': 1, 'misuse': 1, 'againroth': 1, '401k': 1, 'betterthanks': 1, 'cupmedicare': 1, 'aidslax': 1, 'supervision': 1, 'plagued': 1, 'ap': 2, 'findsthe': 1, 'gridmembers': 1, 'itthis': 2, 'rumor': 1, 'daymagical': 1, 'arthow': 2, 'redefine': 4, 'currysocalled': 1, 'kangaroolabor': 1, 'dept': 2, 'joblessfired': 1, 'archdiocese': 1, 'petitionsdemocrats': 1, '324': 1, 'slogans': 2, 'individuallywake': 1, 'honduran': 1, 'diasporathis': 1, 'scrabble': 2, 'flutterbernadette': 1, 'peters': 7, 'twice': 4, 'dayheartfelt': 1, 'robs': 2, 'cherished': 1, 'grudgeconservatives': 1, 'discredit': 2, 'protestersjohn': 1, 'f': 2, 'honorsafter': 1, 'chemists': 1, 'overseas': 4, 'nano': 1, 'breakthroughchris': 1, 'christies': 3, 'professed': 1, 'prioritiested': 1, 'ladyremember': 1, 'heras': 1, 'pickdoctor': 1, 'definitions': 1, 'hoperandom': 1, 'guitars': 1, 'playan': 1, 'managementkey': 1, 'claimswith': 1, 'ty': 1, 'herndon': 1, 'mindsomarosa': 1, 'yearswoman': 1, 'retrievers': 1, 'visionceo': 1, 'problemsneil': 1, 'robertsguitar': 1, 'greatinvestigative': 1, 'everybodymusical': 1, 'showamerica': 1, 'seeming': 1, 'freedommike': 1, 'emailsriotous': 1, 'gathers': 4, 'deepfrying': 1, 'virginhillary': 1, 'streetcompromising': 1, 'partneringisrael': 1, 'retroactively': 1, 'legalizes': 4, '4000': 1, 'settler': 1, 'homessupporting': 1, 'obligationthe': 1, 'middles': 1, 'brock': 2, 'ciarlelli': 1, 'dealnew': 1, 'wondersplint': 1, 'fractures': 1, 'fullermale': 1, 'bail': 2, 'bondingkylie': 1, 'tyga': 1, 'romance': 2, 'weeklatino': 1, 'gophow': 1, 'tvfree': 1, 'familythis': 1, 'ocd': 2, 'ongroup': 1, 'directs': 1, 'wouldbe': 2, 'memberarea': 1, 'experimented': 2, 'collegestormy': 1, 'arrive': 5, 'dinner6': 1, 'stealingkaley': 1, 'cuoco': 1, 'exhusband': 3, 'herprinceton': 1, 'protestersdeciphering': 1, 'manthis': 1, 'createdsame': 1, 'applausela': 1, 'metros': 1, 'ceo1800eatshit': 1, 'publishes': 3, 'recklessdriving': 1, 'datanew': 2, 'alternativefuel': 1, 'deplete': 1, 'hydrogen': 1, '2070local': 1, 'minneapolis': 1, 'whatwhy': 1, 'untiredisis': 1, 'oncelindsey': 1, 'uphold': 3, 'mccains': 3, 'blindly': 1, 'agenda': 3, 'grumbling': 1, 'minuteskfc': 1, 'owned': 1, 'wingshgar': 1, 'cartoonist': 2, '40th': 2, 'anniversaryus': 1, 'percha': 1, 'goldcher': 1, 'backobama': 1, 'rougetove': 1, 'lo': 1, 'listcast': 1, 'troy': 1, 'hollywoodcroatian': 1, 'turnips2hour': 1, 'nobody': 9, 'usearticle': 1, 'paragraph': 1, '911the': 1, 'unlikely': 3, 'obsessing': 1, 'yearrussian': 1, 'hulking': 1, '200pound': 1, 'gymnast': 2, 'competing': 1, 'korea5': 1, 'wisdomparents': 1, 'sexualitylocal': 1, 'daybehold': 1, 'picnic': 3, 'timeentrepreneur': 1, 'mandy': 1, 'ingber': 1, 'instructorinternet': 1, 'doubleedged': 1, 'stained': 2, 'censorshipbush': 1, 'betterprotecting': 1, 'deathornithologist': 1, 'programmichael': 1, 'retirenew': 1, 'tippingstudy': 1, 'macarthur': 2, 'norfolk': 1, 'virginiawhen': 1, 'misremember': 1, '109': 1, '3can': 1, 'fracking': 4, 'yorkprepare': 1, 'trackharry': 1, 'belafonte': 2, 'supportershomemade': 1, 'yippee': 1, 'ki': 1, 'yayuniversity': 1, 'slaps': 1, 'rinkydink': 1, 'decemberemployee': 1, 'herenchanted': 1, 'necromancer': 1, 'oncedead': 1, 'argumentfriend': 1, 'apartments': 2, 'breakdownyour': 1, 'fafsa': 1, 'answeredwhy': 1, 'acttyra': 1, 'surpriseten': 1, '2015remains': 1, 'foundpeople': 1, 'yanny': 1, 'laurelsuitcase': 1, 'ridebush': 1, 'ducks5': 1, 'formative': 1, 'partymerrick': 1, 'addressat': 1, 'phillys': 1, 'freedom45yearold': 1, 'dayfamily': 1, 'dieman': 1, 'mid90s': 1, 'usableweird': 1, 'parodythe': 1, 'crocs': 2, 'shoesuniversal': 1, 'patents': 1, 'wand': 1, 'attractionhundreds': 1, 'harrelsons': 1, 'smoothlyregistered': 1, 'petco': 1, 'clausla': 1, '30minute': 1, 'highintensity': 1, 'trainingpolice': 1, 'armedvoter': 1, 'palpable': 1, 'intentionally': 1, 'angerstoking': 1, 'rally6': 1, 'friendmathematician': 1, 'equation': 1, 'daycia': 1, 'downtrump': 1, 'schism11': 1, 'huluboy': 1, 'difficulties': 1, 'childposthumously': 1, 'recorded': 3, 'rave': 2, 'reviewsrover': 1, 'backtwo': 1, 'dozen': 3, 'violently': 2, 'proposalim': 1, 'godsla': 1, 'nostalgic': 6, 'timeline': 2, 'easy8': 1, 'buttonedup': 1, 'boringthe': 1, 'mediaeval': 1, 'fortress': 1, 'monemvasia': 1, '6queen': 1, 'wed': 3, 'billthat': 1, 'mariel': 1, 'hemingway': 2, 'snlit': 1, 'kidschaplains': 1, 'counselors': 2, 'bernardinophotos': 1, 'moonsyrian': 1, 'mapping': 1, 'unityyou': 1, 'suspendedboston': 1, 'origin': 5, 'dividelindsey': 1, 'opnation': 1, 'shellie': 1, 'zimmerman': 4, 'perjury': 1, 'leastwomans': 1, 'jumble': 1, 'celtic': 2, 'egyptian': 4, 'symbolsdetective': 1, 'endangers': 1, 'retirementman': 1, 'partying7th': 1, '100th': 1, 'underage': 1, 'episoderyan': 1, 'chugs': 1, 'rhino': 2, 'horn': 2, 'semen': 2, 'middebate': 1, 'boostworlds': 1, 'dumbest': 3, 'shoplifters': 1, 'costcoyou': 1, 'enoughtexas': 1, 'glassestwitter': 1, 'muchneeded': 3, 'addcandytoamovieprince': 1, 'pippa': 1, 'middletons': 1, 'weddingthe': 1, 'debatetaraji': 1, 'mary': 9, 'commerciala': 1, 'tide': 1, 'lifts': 4, 'boats': 1, 'boatless': 1, 'sufferhorse': 1, 'racing': 1, 'championcall': 1, 'bullst': 1, 'trueconsumption': 1, 'buncha': 1, 'reverently': 1, 'paused': 2, 'snipermans': 1, 'streak': 3, '37qtips': 1, 'multispeed': 1, 'swabrepublicans': 1, 'rulestaylor': 1, 'maid': 1, 'weddingford': 1, 'mustang': 1, 'coolchild': 1, 'subjected': 1, 'elaborate': 5, 'hairdoauthor': 1, 'introduction': 1, 'algebra': 1, 'rejected': 3, 'publishers': 2, 'acceptedstudy': 1, 'vegetarian': 5, 'nowgore': 1, 'aquatic': 1, 'mammalnuclear': 1, 'warhead': 1, 'thrilled': 3, 'koreaman': 1, 'travels': 3, 'patterns': 2, 'shirtspolice': 1, 'sf': 1, 'weekendziploc': 1, 'baggieswedding': 1, 'suspenders': 1, 'bow': 3, 'floorguy': 1, 'attractivesee': 1, 'boyexclusive': 1, 'deathkim': 1, 'monsterread': 1, 'shutdownmass': 1, 'rosensteingeorgia': 1, 'swastika': 5, 'flagwoody': 1, 'involvedentire': 1, 'subjectfruit': 1, 'islam': 5, 'soil': 2, 'loomsgrowing': 1, 'transylvaniahere': 1, 'resist': 3, 'marcheslady': 1, 'kidnaps': 1, 'commissioner': 3, 'gordonblack': 1, 'graydisgusted': 1, 'mcnugget': 1, 'mind6': 1, 'houstonhow': 1, 'interactive': 1, 'globecandidates': 1, 'zinc': 1, 'miningrare': 1, 'fishermancomedy': 1, 'cellar': 1, 'comedians': 5, 'workshop': 1, 'apologiesgop': 1, 'speakermatador': 1, 'gored': 1, 'cape': 3, 'bullring': 1, 'video500': 1, 'stereo': 1, 'installed': 1, 'cartourist': 1, 'scams': 1, 'goexnfl': 1, 'laments': 2, 'cte': 1, 'prior': 2, 'careerhow': 2, 'carol': 2, 'channings': 1, 'aroundreport': 1, 'pennsylvaniawhat': 1, 'muslimskids': 1, 'dummies': 2, 'brownback': 1, 'outsources': 1, 'donordefunded': 1, 'fetus': 2, 'goingman': 1, 'outhistorical': 1, 'jest': 1, 'youpolice': 1, 'manfrothing': 1, 'alex': 3, 'imbalance': 1, 'dynamicsorlandos': 1, 'numbertwo': 1, 'disneywatch': 1, '1998': 3, 'torpedo': 1, 'giulianis': 1, 'arguments3': 1, 'lifetwo': 1, 'secretaries': 1, 'spicer7': 1, 'moneyman': 1, 'tinkering': 1, 'datejoe': 1, 'bigotrest': 1, 'zeus': 1, 'tallest': 1, 'poochyou': 1, 'tshirts': 3, 'stripesreport': 1, '2050corporate': 1, 'renders': 1, 'mugs': 2, 'obsoletemiley': 1, 'hospitalizationsamantha': 1, 'nrabeholden': 1, 'costumestrump': 1, 'veteransrate': 1, 'uninformed': 3, 'navy': 8, 'seals': 5, 'skyrockets10': 1, 'parisuk': 1, 'contestant': 2, 'fiercely': 2, 'costars': 1, 'commentsyes': 1, 'ghoulish': 1, 'bony': 1, 'thinlymike': 1, 'campaigngop': 1, 'gaetz': 1, 'lookwilli': 1, 'dorners': 1, 'bodies': 5, 'videocourt': 1, 'cannibal': 1, 'fantasize': 4, 'wantscalifornia': 1, 'discriminatory': 1, 'statesglowing': 1, 'buildinghannity': 1, 'offtherails': 1, 'weirdomichelle': 1, 'proudest': 2, 'achievement': 3, 'daughtersman': 1, 'ageofconsent': 1, 'lawseyes': 1, 'removed': 7, 'attack9': 1, 'nittygritty': 1, 'tipspiers': 1, 'commentsparents': 1, '3rdgrade': 1, 'projectunusual': 1, 'novemberreport': 1, 'thingsman': 1, 'fist': 1, 'desk5': 2, 'nappingarea': 1, 'backyardpalm': 1, 'coconutmonstrously': 1, 'lemmy': 1, 'motorheaddominos': 1, 'napkinstuffed': 1, 'crustarlen': 1, 'specter': 1, 'affiliation': 1, 'minutehow': 1, 'balancemissouri': 1, 'lynching': 1, 'statuemeatless': 1, 'mashuphistory': 1, 'profiting': 3, 'nazi': 5, 'documentariesdads': 1, 'exterior': 1, 'hides': 3, 'resentful': 1, 'centerproud': 1, 'pic': 5, 'folks': 8, 'graduated': 1, 'addictionnorman': 1, 'shonda': 1, 'explore': 5, 'epix': 1, 'seriesman': 1, 'reverseengineer': 1, 'midst': 3, 'meandering': 1, 'absentminded': 1, 'rantfan': 1, 'dodger': 1, 'stadium': 5, 'comfortableteen': 1, 'devil': 3, 'pregnantcaitlyn': 1, 'womanhoodcommon': 1, 'finance': 3, 'sleight': 1, 'handreport': 1, 'smile': 3, 'laughneighbors': 1, 'bmw': 1, 'snowstudy': 1, 'circling': 3, 'markeralmost': 1, 'buttyou': 1, 'beyonce106': 1, 'revolutionthe': 1, 'lessonspamela': 1, 'papua': 1, 'guineathe': 1, 'tyrese': 1, 'reunite': 4, 'taraji': 2, 'empirecop': 1, 'punk': 3, 'pressed': 2, 'partnerthe': 1, 'havingsmoking': 1, 'fragile': 1, 'economywill': 1, 'megabillionaire': 1, 'mayhemthese': 1, 'protesting': 5, 'tantalizingly': 1, 'pointwatch': 1, 'flip': 5, 'downhow': 2, 'erectile': 1, 'dysfunctionstudy': 1, 'lid': 2, 'affairman': 1, 'torn': 2, 'boycotting': 2, 'evansville': 1, 'zooashley': 1, 'judd': 3, 'performancewomen': 1, '100mile': 1, 'underground': 2, 'railroadalqaeda': 1, 'publicrelations': 1, 'consultant': 3, 'himfrank': 1, 'gehry': 3, 'liquid': 1, 'architecturenyc': 1, '19th': 3, 'slaveseerie': 1, 'starwhy': 1, 'outliersthat': 1, 'show5': 1, 'allergy': 1, 'familiessaudi': 1, 'haunt': 2, 'example': 4, 'meritocracyits': 1, 'twilight': 1, 'showstrangers': 1, 'solidaritysam': 1, 'smiths': 4, 'idolthis': 1, 'zip': 2, 'codeeu': 1, 'headscarf': 2, 'ruling': 8, 'backlashthe': 1, 'demonstration': 1, 'seeemerson': 1, 'collins': 2, 'audiodaily': 1, 'joyfulman': 1, 'nonsexual': 2, 'domen': 1, 'separatednarrow': 1, 'widened': 1, 'mondayarea': 1, 'women11': 1, 'seeking': 8, 'themselvesvoice': 1, 'lends': 1, 'legitimacykim': 1, '2014s': 1, 'failspolice': 1, 'confession': 2, 'bombers': 2, 'cellphoneqaddafi': 1, 'personstudy': 1, 'singleparrot': 1, 'householdsman': 1, 'humiliated': 2, 'wifis': 1, 'guestsmissed': 1, 'connections': 2, 'rally31yearold': 1, 'divorceddeath': 1, 'romania': 1, '41': 3, 'arrestedwalmart': 1, 'weirded': 1, 'openingsweatshirt': 1, 'harrowing': 6, 'hoodtop': 1, 'committeeparents': 1, 'yetsixty': 1, 'artwhy': 1, 'consensual': 1, 'pancakesthousands': 1, 'incumbents': 1, 'marched': 1, 'washingtondiscovery': 1, 'neolithic': 1, 'stonehenge': 2, 'attractionman': 1, 'wolves': 3, 'fatherwhy': 1, 'holes': 1, 'windows': 3, 'planesecstasy': 1, 'despair': 1, 'dayexuberant': 1, 'bats': 2, 'arena': 4, 'stagemexicos': 1, 'baja': 1, 'villa': 1, 'palmar': 1, 'loretoemotional': 1, 'lifebaby': 1, 'streetwedding': 1, 'replacing': 2, 'groomkim': 1, 'advicegary': 1, 'assetluther': 1, 'vandross': 1, 'nighta': 1, 'nutritionists': 2, 'restaurantsthe': 1, 'beyhive': 2, 'swarms': 2, 'rachael': 1, 'roythe': 1, 'bearnaked': 1, 'mouthwatering': 1, 'europethis': 1, 'instagrammers': 1, 'dreamcompany': 1, 'creditdad': 1, 'daygov': 1, 'blessings': 2, 'patientswhite': 1, 'accountable': 2, 'crimesam': 1, 'thoughtthe': 1, 'weighing': 4, 'againobama': 1, 'cooking': 6, 'tshirtswomen': 1, 'ciscomasters': 1, 'master': 6, 'writingwoman': 1, 'personahow': 1, 'taillight': 1, 'americaperverted': 1, 'playground': 1, 'childrenfantasy': 1, 'suffers': 7, 'setback': 2, 'embarks': 3, 'weeklong': 2, 'honeymoon': 1, 'accesslie': 1, 'partyreport': 2, 'lauras': 1, 'razorthin': 1, 'familydonald': 1, 'grasp': 4, 'supporttrump': 1, 'untruenew': 1, 'processnation': 1, 'soongop': 1, 'revived': 1, 'rowdy': 3, 'hallfocus': 1, 'homebeing': 1, 'transparent': 2, 'howards': 1, 'filmbask': 1, 'adele': 4, 'midconcertbox': 1, 'retrieved': 1, 'trashcoworker': 1, 'describing': 2, 'commercialwillow': 1, 'rentedrash': 1, 'standare': 1, 'usarea': 2, 'croatiantheater': 1, 'stingy': 1, 'steinreport': 1, 'masking': 1, 'depressionevery': 1, 'visibly': 4, 'filthyamanda': 1, 'peet': 2, 'thronesillinois': 1, 'designationsfoster': 1, 'shipmentfirst': 1, 'gentlemen': 2, 'veronaon': 1, 'screens': 3, 'boldcan': 1, 'mtv': 1, 'ondemand': 1, 'worldmeat': 1, 'easytear': 1, 'perforated': 1, 'beefmaster': 1, 'architect': 2, 'constructs': 2, 'structurally': 1, 'dateyou': 1, 'likepersonals': 1, 'omits': 2, 'goitersunday': 1, 'roundupisraeli': 1, 'jewryreport': 1, 'agochechen': 1, 'infant': 4, 'lulled': 1, 'rumblingperfectly': 1, 'crematednew': 1, 'firstnations': 1, 'voyeurs': 1, 'bushesdoritofactory': 1, 'coolranch': 1, 'clotheskfc': 1, 'vaccinespecial': 1, 'ops': 1, 'undetectedsecurity': 1, 'relax': 1, 'hourstrump': 1, 'indigestion': 1, 'wellprepared': 1, 'mealman': 1, 'ren': 2, 'magrittes': 1, 'estatethe': 1, 'segel': 1, 'rooney': 1, 'afterlifeunpatriotic': 1, 'erection': 1, 'anthemangelina': 1, 'vanity': 2, 'fairs': 1, 'portrayal': 1, 'auditionshistorical': 1, 'onlookers': 1, 'passersbye': 1, 'depart': 2, 'vale': 1, 'boothestudy': 1, 'psychology': 4, 'experimentsword': 1, 'innovate': 1, '650000': 1, 'farkit': 1, 'sexism8': 1, 'pokemon': 3, 'gosight': 1, 'againmarketing': 1, 'heartstringsconclude': 1, 'optionjill': 1, 'zarin': 1, 'ramona': 1, 'singers': 2, 'mile': 4, 'awaysmithsonian': 1, 'nicedixie': 1, 'chicks': 1, 'concert2yearold': 1, 'adorably': 5, 'mangles': 1, 'starspangled': 1, 'pucknew': 1, 'antidrug': 1, 'emojis': 2, 'listented': 1, 'shifting': 2, 'racedespite': 1, 'tort': 1, 'hasty': 1, 'retreatchrysler': 1, 'lebaron': 1, 'ceo8yearold': 1, 'yetapple': 1, 'earbuds': 1, 'unexpectedly': 4, 'bloomedsaturdays': 1, 'jackpot': 3, 'millionzoe': 1, 'saldana': 1, 'tlc': 1, 'scrubs': 2, 'sync': 2, 'battleamerican': 1, 'thaler': 1, 'economics': 1, 'prizescott': 1, 'pvc': 1, 'tube': 1, 'campfire8': 1, 'toddlersarea': 2, 'busboy': 1, 'orderobama': 1, 'sacrifice': 3, '2015you': 1, '2015us': 1, 'partners': 4, 'evade': 2, 'reportnew': 1, 'pilgrims': 1, 'sailed': 1, 'thanksgivingdoctors': 1, 'sciencewhat': 1, 'portraitpure': 1, 'silk': 1, 'assawards': 1, 'blonde': 1, 'buddy': 4, 'eha': 1, 'brooke': 1, 'shields': 2, '50the': 2, 'easiest': 2, 'longevityyour': 1, 'enoughauthorities': 1, 'rapid': 1, 'shootingdaryl': 1, 'dixons': 1, 'revealedvoice': 1, 'clintonswhat': 1, 'axing': 1, 'wheels': 6, 'fundssenate': 1, 'implying': 2, 'blacki': 1, 'muslimsbaseball': 1, 'ortiz': 1, 'antimexican': 1, 'attackswhy': 1, 'naming': 1, 'storiescouple': 1, 'conceive': 1, 'childold': 1, 'classmatesreport': 1, 'consumer': 5, 'blind': 6, 'day5': 1, 'realities': 1, 'grooms': 1, 'knowthe': 1, 'nominating': 1, 'verma': 1, 'indiasmooth': 1, 'forklift': 1, 'operatorrussell': 1, 'nash': 2, 'mathematicianarea': 1, 'overly': 2, 'underwearsalvadoran': 1, '02': 1, 'consciousnesscongo': 1, 'postpones': 2, 'opposition': 6, 'striketarget': 1, 'sponsorship': 1, 'conflicthillary': 1, 'supportno': 1, 'himreflections': 1, 'ivy': 5, 'daywhite': 1, 'launching': 3, 'attackhow': 1, 'urbaneducation': 1, 'worknewsrooms': 1, 'varying': 1, 'videoskeleton': 1, 'nerd': 2, 'dug': 2, 'prehistoric': 3, 'lockercornel': 1, 'counterfeitill': 1, 'obstruct': 1, 'wrapping': 3, 'gloved': 1, 'neckmindfulness': 1, '20s': 2, 'happinesspublicists': 1, 'bentley': 1, 'fevernsa': 1, 'somebody': 2, 'computers': 1, 'uslife': 1, 'unfairpresident': 1, 'dronesjohn': 1, 'ashcroft': 2, 'frolics': 1, 'vault': 3, 'winniethepooh': 1, 'toyscommerce': 1, 'actionpossible': 1, 'investigated': 4, 'italyhubble': 1, 'nasa': 8, 'ganymedeatari': 1, 'adventure': 1, 'gamearea': 1, 'agocrowd': 1, 'theyve': 7, 'hourhuffpollster': 1, 'everwoman': 1, 'sore': 2, 'throat': 4, 'anthraxopera': 1, 'notehungover': 1, 'nightbianchi': 1, 'hournations': 1, 'fraudulent': 1, 'actcrayola': 1, 'presents': 9, 'jarringly': 1, 'shareholders': 1, 'meetinggood': 1, 'nothingdirect': 1, 'marketer': 1, 'mailmajestic': 1, 'americathe': 2, 'performancenation': 1, 'iti': 1, 'yearfrank': 1, 'rejecting': 2, 'momentmacs': 1, 'troll': 2, 'neon': 2, 'nostalgiatrumps': 1, 'enforcer': 1, 'freezes': 1, 'workerfriendly': 1, 'obamamom': 1, 'sixthgrader': 1, 'bottlerachael': 1, 'airhow': 1, 'redefining': 3, 'ownershipsports': 1, 'psychologists': 2, 'tigers': 1, 'slump': 1, 'throughhistory': 1, 'contemptgop': 1, 'joking': 3, 'mammograms': 1, 'themceiling': 1, 'ambition': 1, 'dennysamerican': 1, 'association': 5, 'doctorswoman': 1, 'massage10': 1, 'mains': 1, 'makebig': 1, 'pacs': 2, 'haultheresa': 1, 'mays': 1, 'defeatthis': 1, 'areasgoing': 1, 'favored': 1, 'touristshiphop': 1, 'saltnpepa': 1, 'todaynational': 1, 'antelopeater': 1, 'feedingman': 1, 'earlyhow': 1, 'polished': 1, 'lookarea': 1, '16apartment': 1, 'tenant': 2, 'minnie': 2, 'checks8': 1, 'voidbanks': 1, '75cent': 1, 'surcharge': 1, 'bankdenis': 1, 'leary': 1, 'commercialthe': 1, 'pridewhy': 1, 'runheres': 1, 'confirmed': 4, 'herus': 1, 'emptythreat': 1, 'koreamillie': 1, 'clearknee': 1, 'jeansdancing': 1, 'friendsmad': 1, 'actors': 6, 'filmhousefly': 1, 'slicereport': 1, 'sheldon': 2, 'adelson': 2, 'pledgedelusional': 1, 'nominationlost': 1, 'dimension': 1, 'reportmilla': 1, 'jovovich': 1, 'famehow': 1, 'sidewaystrump': 1, 'praising': 2, 'meangreat': 1, 'evanschildren': 1, 'premieresal': 1, 'eventjared': 1, 'clearancetrump': 1, 'proposing': 2, 'slash': 2, 'fundingtrump': 1, 'delete': 2, 'islamophobic': 2, 'campaignthe': 1, 'ladens': 2, 'bellwetherthat': 1, 'dvdhow': 1, 'construction': 6, 'congresssophie': 1, 'larios': 1, 'sleeparea': 1, '475': 1, 'seconds60yearold': 1, 'hippie': 2, 'pitied': 1, 'punkwedding': 1, 'lightsaber': 2, 'duel': 2, 'dancethe': 1, 'waractress': 1, 'jameela': 1, 'jamil': 1, 'aziz': 4, 'ansari30': 1, 'naughty': 2, 'naturecorrect': 1, 'discarded': 3, 'theoryshots': 1, '2nd': 1, 'siteahead': 1, 'willpence': 1, 'baptizing': 1, 'senatorscoworker': 1, 'obsessively': 1, 'minutesbernie': 1, 'guynypd': 1, 'noquestionsasked': 1, 'dropoffarea': 1, 'boasts': 1, 'selfabsorbencyman': 1, 'doubtstop': 1, 'madnesssharon': 1, 'auctioned': 1, 'conglomeratefreshman': 1, 'masturbation': 1, 'himfamily': 1, '11th': 2, 'yeartraveling': 1, 'dietups': 1, 'doorbell': 1, 'whileresolving': 1, '2015wow': 1, '69': 4, 'bostonmillions': 1, 'votingwhat': 1, 'muchzaire': 1, 'itselfunited': 1, 'dogfights': 1, 'cabina': 1, 'imeldareport': 1, 'corruptionoutofcontrol': 1, 'bystanders': 1, 'showreport': 1, 'prefaced': 1, 'eventsallowance': 1, 'parental': 3, 'dependencejoe': 1, 'berates': 2, 'health100': 1, '2030': 1, 'saysiraqs': 1, 'trim': 2, 'governmentcan': 1, 'tai': 2, 'chi': 3, 'adhdbarbers': 1, 'paunch': 1, 'customerfriday': 1, 'panicthe': 1, 'forgave': 1, 'abuseris': 1, 'mocking': 2, 'instagramrick': 1, 'presidentman': 1, 'annoy': 2, 'partythese': 1, 'solidarity': 3, 'wellendowed': 1, 'womenpastor': 1, 'shithole': 4, 'pencejeopardy': 1, 'categories': 1, 'beginsnissin': 1, 'extralarge': 1, 'drum': 3, 'noodlessenior': 1, 'merits': 1, 'countyclerk': 1, 'candidatesthis': 1, 'likeblack': 1, 'clubs': 1, 'stem': 2, 'centersthe': 1, 'industrys': 4, 'modeling': 1, 'mysterydavid': 1, 'valadao': 1, 'tk': 1, 'racehuckabee': 1, 'raucous': 2, 'carefree': 1, 'frownhomeschooled': 1, 'nooknew': 1, 'meteoritethe': 1, 'gazebo': 1, 'killedfinding': 1, 'depressionsupreme': 1, 'redrawing': 2, 'holdhuffpost': 1, '23christianitys': 1, 'hijacked': 2, 'brandcombat': 1, 'blurs': 1, 'hawaiiflood': 1, 'recessionsen': 1, 'merkley': 1, 'stages': 3, 'allnight': 2, 'nomination17year': 1, 'cicadas': 1, '911hillary': 1, 'exploratory': 2, 'committees3': 1, 'musthave': 1, 'shoes': 3, 'officedivorcing': 1, 'kidsgirl': 1, 'sakethese': 1, 'codeschanging': 1, 'anticipated8': 1, 'positions': 2, 'wellsusan': 1, 'untoward': 1, 'intelligenceabes': 1, 'checkednew': 1, 'monster': 5, '1200': 2, 'volts': 2, 'heartthe': 2, 'drivingmuseum': 1, 'geodejennifer': 1, 'reasonwhat': 1, 'widow': 4, 'spaworld': 1, 'aftershockssavor': 1, 'festivalsback': 1, 'hardwired': 1, '20spains': 1, 'catalan': 2, 'leadersthe': 1, 'problemno': 1, 'dolittle': 1, 'pitcher': 1, 'slamsecret': 1, 'cabinets': 1, 'storepilot': 1, 'somethingcinzano': 1, 'shitholemajestic': 1, 'pine': 3, 'recruited': 1, 'yosemite': 1, 'headhuntersphotos': 1, 'calais': 1, 'flamescapsizing': 1, 'rankingsbill': 1, 'tvarea': 1, 'dumped': 1, '15week': 1, 'anniversarythe': 1, 'wrest': 1, 'portraiture': 1, 'trappingswatch': 1, 'belgium': 1, 'hotbed': 1, 'terrorismclinton': 1, 'depositwinning': 1, 'gamecat': 1, 'lights': 3, 'clapperlist': 1, 'politically': 2, 'codethis': 1, 'simone': 1, 'biles': 2, 'mere': 1, 'allthe': 2, 'weightbodies': 1, 'iddbaylor': 1, 'questionidf': 1, 'recounts': 3, '8monthold': 1, 'childpsychic': 1, 'devastates': 2, 'hiring': 8, 'psychicsfrustrated': 1, 'rely': 2, 'yetburkina': 1, 'faso': 1, 'siege': 2, 'groupparent': 1, 'euthanizing': 1, 'lohan': 2, 'clone': 2, 'completing': 1, 'filmingthe': 1, 'byman': 1, 'repress': 1, 'sadnessastronomers': 1, 'discoverywhy': 1, 'pediatricians': 1, 'rescinded': 1, 'dacapreview': 1, 'expiration': 1, '2game': 1, 'stoneheart': 1, 'theoryno': 1, 'problemfifth': 1, 'discovered10': 1, 'dude': 4, 'romcomus': 1, 'campaigning': 3, 'obamalone': 1, 'superdelegate': 2, 'idiotcongressman': 1, 'necessitysec': 1, 'replay': 1, 'overturns': 3, 'roe': 2, 'wademarket': 1, 'evidently': 3, 'cakecyclist': 1, 'cliffa': 1, 'frequent': 1, 'flighters': 1, 'responsiblyapple': 1, 'patent': 2, 'wisconsinmadisonman': 1, 'bragging': 4, 'waydonald': 1, 'pinning': 1, 'outbrief': 1, 'lucidity': 1, 'attackunderstanding': 1, 'politicsarea': 1, 'mc': 2, 'lyte': 1, 'wikipedia': 3, 'anymoreseveral': 1, 'gunfire': 2, 'waldorf': 1, 'astoria': 1, 'partybill': 1, 'empathy': 1, 'survivalthousands': 1, 'dementia': 1, 'holidayakon': 1, 'ricos': 2, 'rejectedthe': 1, 'intersection': 2, 'kalief': 2, 'browdermarco': 1, 'machinethe': 1, 'warwedding': 1, 'strains': 2, 'pointcommunity': 1, 'vastly': 2, 'improved': 5, 'caringi': 1, '18to36yearold': 1, 'hispanic': 5, 'toms': 1, 'chobaniyour': 1, 'surgerydemocratic': 1, 'womensteven': 1, 'dislikes': 1, 'cannes': 2, 'dutynew': 2, 'vcr': 1, 'communists': 2, 'allegescommence': 1, 'forwardhong': 1, 'asiansmerrick': 1, 'pointing': 3, 'nominationfans': 1, 'riot': 5, 'victoriousaging': 1, 'dieafter': 1, 'turner': 1, 'assaultgroup': 1, 'calledlocal': 1, 'grapple': 1, 'fearmongering': 1, 'policiesa': 1, 'methane': 3, 'regulationheres': 1, 'onproject': 1, 'andrew': 5, 'kaminskijustice': 1, 'scalia': 6, 'flipflopping': 2, 'juvenile': 1, 'justiceman': 1, 'aloud': 1, 'salad24': 1, 'soulbeauty': 1, 'nowmartin': 1, 'thinkingclint': 1, 'dempsey': 1, 'jurgen': 1, 'klinsmann': 1, 'usai': 1, 'woke': 1, 'dis': 1, 'meto': 1, 'seasonobama': 1, 'catchphrasethe': 1, 'twoforone': 1, 'tones': 1, 'timedonors': 1, 'pacbrazilian': 1, 'muralscreationism': 1, 'banned': 3, 'schoolsthursdays': 1, 'thoughtmore': 1, 'couchtesla': 1, 'upgrade': 1, 'roadsterim': 1, 'mercy': 2, 'waystrump': 1, 'crook': 1, 'probehere': 1, 'dealcat': 1, 'nextlevel': 1, 'chillnair': 1, 'incendiary': 1, 'zonethis': 1, 'tripmeet': 1, 'visionary': 1, 'grantwhat': 1, 'haitian': 2, 'faithdonald': 1, 'windfall': 2, 'democratssoldier': 1, 'routetrinidad': 1, 'tobago': 1, 'commemorative': 2, 'leonardo': 3, 'dicaprio': 6, 'postage': 2, 'stamplots': 1, 'scenarious': 1, 'burundi': 1, 'telephonejohn': 1, 'deere': 1, 'lawnmower': 1, 'sidecarsnew': 1, 'plutos': 1, 'vast': 3, 'wastelandwill': 1, 'causechuck': 1, 'todds': 1, 'gopneighbors': 1, 'killermarco': 1, 'primarymilitia': 1, 'probation': 1, 'misdemeanorstrumps': 1, 'chilling': 4, 'effect': 7, 'sciencewendy': 1, 'movementno': 2, 'emma': 7, 'emily': 1, 'reportman': 3, 'rolling': 5, 'swivel': 1, 'formationfucker': 1, 'downloadfuck': 1, 'speechelderly': 1, 'tackled': 2, 'technological': 2, 'lifetimekim': 1, 'gapapplebees': 1, 'steak': 4, 'slatheredhillary': 1, 'jay': 5, 'z': 5, 'voteclinton': 1, 'kidseurostar': 1, 'amsterdamhiphop': 1, 'soundstiny': 1, 'labrador': 1, 'palgop': 1, 'selfdeportation': 1, 'wellangela': 1, 'partial': 3, 'burka': 1, 'niqab': 1, 'germanydownton': 1, 'abbey': 3, 'previews': 2, 'goodbyerapper': 1, 'coolio': 1, 'felony': 2, 'angeleslightning': 1, 'scoresmom': 1, 'shamers': 1, 'babys': 3, 'shootarea': 1, 'digitizeno': 1, 'notices': 4, 'marginal': 1, 'changewatch': 1, 'slam': 2, 'thingvisiting': 1, 'whateverlaughter': 1, 'mask': 2, 'feelingspurple': 1, '91': 1, 'honda': 3, 'lovingly': 2, 'dedicated': 1, 'razawhen': 1, 'patriotism': 1, 'idolatrynevertrump': 1, 'neverhow': 1, 'nebraska': 2, 'greatnessquiet': 1, 'loner': 2, 'storethis': 1, 'storynot': 1, 'upice': 1, 'cube': 3, 'nwa': 1, 'coachellapermission': 1, 'prohibitiongop': 1, 'raceshooting': 1, 'nontargetspeter': 1, 'dinklage': 1, 'promous': 1, 'daysi': 1, 'sightselena': 1, 'whyintergalactic': 1, 'shackles': 1, 'clintontrumps': 1, 'sudan': 4, 'cringeworthytuesdays': 1, '62': 4, 'dayreproductive': 1, 'inequalitypurdue': 1, 'erases': 3, 'obey': 1, 'censorship': 1, 'ruleswoman': 1, 'contort': 1, 'dental': 4, 'technicianhasbro': 1, 'additional': 6, 'marbles': 1, 'hippohunger': 1, 'reliefarea': 1, 'playingbreaking': 1, 'nichols': 1, 'mompoll': 1, 'mitt': 4, 'romneywhat': 1, 'superbug': 1, 'antibiotic': 1, 'resistancenew': 1, 'enables': 1, 'mankind': 2, 'resolve': 1, 'disagreementssix': 1, '90why': 1, 'summer5': 1, 'firstworld': 1, 'anywayinnovative': 1, 'bottomup': 1, 'povertyashleigh': 1, 'banfield': 1, 'claimphysically': 1, 'systemlow': 1, 'unrepresentative': 1, 'californiaour': 1, 'truckers': 2, 'pancake': 2, 'needsplane': 1, 'whinythose': 1, 'massacreminotaurs': 1, 'vampires': 1, 'vampiresparents': 1, 'conditionmarine': 1, 'blockhow': 1, 'embroidered': 1, '100saturdays': 1, 'editionlegionnaires': 1, 'citysenate': 1, 'activist': 5, 'jobsbernie': 1, 'clintonthese': 2, 'vertigoinducing': 1, 'breath': 3, 'awaysecret': 1, 'alan': 5, 'grier': 1, 'fanultra': 1, 'revolutionize': 3, 'poundingmore': 1, 'daysnew': 1, 'namedonald': 2, 'billexpert': 1, 'connecticut': 1, 'guards': 3, 'shootingsstudent': 1, 'crisisdemocrats': 1, 'superdelegates': 1, 'reformsdetectives': 1, 'overlooked': 2, 'anthonys': 2, 'ama': 1, 'redditseek': 1, 'methodshe': 1, 'streamed': 1, 'clubjim': 1, 'carrey': 1, 'psycho': 1, 'portraitnew': 1, 'poultry': 1, 'stripe': 2, 'gum': 4, 'hardly': 2, 'goose': 1, 'minutefcc': 1, 'apriltea': 1, 'flatinstant': 1, 'gratification': 3, 'sped': 1, 'up15': 1, 'sundayscott': 1, '1st': 1, 'armored': 1, 'drycleanerusing': 1, 'voicelessnation': 1, 'though': 3, 'wantedit': 1, 'continually': 1, 'cityif': 1, 'expectationsroy': 1, 'harassing': 2, '1980sto': 1, 'kids8yearold': 1, 'daretom': 1, 'portrays': 1, 'regis': 2, 'philbin': 1, 'showactive': 1, 'hum': 1, 'lifeall': 1, 'otterslocal': 1, 'winnings': 2, '2005entertainment': 1, 'fight3': 1, 'gratitudethis': 1, 'locate': 2, 'policiesfirstgeneration': 1, 'adjusting': 3, 'routinesarea': 1, 'aboutchrissy': 1, 'tuitionmissing': 1, 'rewardarea': 1, 'mystified': 1, 'adjacent': 2, 'krogerheres': 1, 'carls': 2, 'commercialrobin': 1, 'wright': 1, 'cardsparched': 1, 'sip': 2, 'pudding': 4, 'pointstexas': 1, 'generals': 2, 'invents': 3, 'prayersgod': 1, 'andromeda': 1, 'galaxytiny': 1, 'suffocates': 1, 'vuitton': 2, 'baglebron': 1, 'espys': 1, 'hearhundreds': 1, 'misconductleatherjacketed': 1, 'ruleswhat': 1, 'gums': 3, 'freestyle': 2, 'hiphop': 3, 'tabletopsolve': 1, 'waitressbuddy': 1, 'xraykate': 1, 'winslet': 2, 'speechtwo': 1, 'legacies': 1, 'alfred': 1, 'hitchcockbreaking': 1, 'ufos': 2, 'classy': 3, 'porcelain': 1, 'dinnerwareuninsured': 1, 'obamacarepile': 1, 'crap': 4, 'publicistunemployed': 1, 'headacheconjoined': 1, 'twin': 1, 'hogging': 1, 'kidneyman': 1, 'seessee': 1, 'runin': 1, 'hale': 1, 'prepper': 1, 'romcombreaking': 1, 'napkin': 2, 'bare': 5, 'handwhen': 1, 'fairappeals': 1, 'reconsider': 1, 'mcdonnells': 1, 'casewaitress': 1, 'checkjudge': 1, 'wkpour': 1, 'hassan': 1, 'jameelandy': 1, 'samberg': 2, 'casts': 3, 'witch': 4, 'claims71': 1, 'approve': 5, 'ratinggary': 1, 'peaking': 1, 'pollschinese': 1, 'spratlys': 1, 'saysnewspaper': 1, 'scraps': 3, 'references': 2, 'obituarywill': 1, 'cardell': 1, 'behaviormuch': 1, 'adomichael': 1, 'soured': 1, 'journalistsbush': 1, 'mailnutritionists': 1, 'defecatingthe': 1, 'weektravel': 1, 'rights5': 1, 'playgroundkim': 1, 'northreport': 1, 'outsideben': 1, 'nominated': 6, 'damonhighlighting': 1, 'platos': 1, '17nations': 1, 'pivotal': 2, 'wellbeingadam': 1, 'alongside': 1, 'photobirthday': 1, 'religioustaylor': 1, 'darker': 2, 'persona': 2, 'skullfucking': 1, 'maggot': 2, 'boyfriendbarack': 1, 'biographerreport': 1, 'womanclinton': 2, 'memberpublicspeaking': 1, 'gesturingmayor': 1, 'citys': 3, 'waterfront': 1, 'spacesmondays': 1, 'cyberattack': 2, 'youespn': 1, 'analyst': 3, 'footballfreedom': 1, 'caucus': 3, 'hourdavid': 1, 'journeysteve': 1, 'payments': 4, 'lawmoms': 1, 'contradictions': 1, 'motherhoodkangaroo': 1, 'runningkid': 1, 'louisiana': 7, 'sneaky': 1, 'newscast': 2, 'movesembarrassed': 1, 'sony': 2, 'systemselfless': 1, 'escorts': 1, 'ducklings': 1, 'streetthe': 1, 'colonization': 1, 'indiansubcontinentmy': 1, 'bette': 2, 'storykim': 1, '3circus': 1, 'runaway': 4, 'showcardi': 1, 'livesupreme': 1, 'jararchaeologists': 1, 'martysenator': 1, 'wednesdayeveryone': 1, 'kidmatt': 1, 'suspensionamerica': 1, 'livesthe': 2, 'breakthefast': 1, 'kippur': 1, 'buffetreport': 1, 'dread': 1, 'musichow': 1, 'blueprint': 2, 'mothertobaby': 1, 'transmissioncomputer': 1, 'grind': 1, 'wheatflorida': 1, 'promposal': 2, 'signcelebrity': 1, 'smellalike': 1, 'sweats': 1, 'alec': 1, 'baldwinthe': 1, 'prizecancers': 1, 'immunotherapyeven': 1, 'grammysman': 1, 'youmore': 1, '160000': 1, 'evacuated': 2, 'worstever': 1, 'malaysiawilliam': 1, 'mightnew': 1, 'appthese': 1, 'scratchwhat': 1, 'cowboys': 3, 'representfridays': 1, 'youchef': 1, 'jos': 1, 'andrs': 1, 'riconation': 1, 'relate': 3, 'itreport': 1, 'moviemanhattan': 1, 'swept': 2, 'rug': 2, 'allegesobama': 1, 'wageschinas': 1, 'renewed': 2, 'tensionsbacon': 1, 'everarea': 2, 'elsebosss': 1, 'clout': 1, 'evaporates': 1, 'picniclocal': 1, 'clips': 2, 'vitamin': 3, 'enutritious': 1, 'broadcasts': 2, 'scare': 6, 'clearbroad': 1, 'accountability': 1, 'bolster': 2, 'showrock': 1, 'd': 2, 'chordheritage': 1, 'lowers': 2, 'vat': 2, 'strategistshalffabricated': 1, 'rsum': 1, 'unimpressivewoman': 1, 'childrenteenage': 1, 'motoristnick': 1, 'cannon': 7, 'mariah': 3, 'careys': 3, 'waytrump': 2, 'topic': 2, 'housebus': 1, 'scenerydan': 1, 'coats': 1, 'junior': 1, 'unionvillain': 1, 'contends': 1, 'aliketoasting': 1, 'ketogenic': 1, 'koolaidmarco': 1, 'adbruce': 1, 'tardy': 1, 'notepatty': 1, 'extremely': 10, 'distressed': 2, 'ratner': 1, 'allegationshow': 1, 'expertsexploring': 1, 'communitiesrestaurant': 1, 'fallen': 6, 'soldiermissing': 1, 'effectsman': 1, 'willingly': 1, 'rented': 1, 'wrath': 1, 'titans': 3, 'insultedkim': 1, 'jonguns': 2, 'marrybuyer': 1, 'vinci': 2, 'assumed': 3, 'framediscovering': 1, 'gospelpep': 1, 'laced': 1, 'threatsdemolishing': 1, 'propping': 1, 'fuelspankaj': 1, 'mishras': 1, 'indianations': 1, 'optimists': 1, 'nowsingle': 1, 'accompanying': 1, 'presumes': 1, 'beingsteve': 1, 'slated': 3, 'summitinept': 1, 'aboutobama': 1, 'goldstein': 1, 'israelhillary': 1, 'positioned': 1, '2008neighbors': 1, 'sufferingjeff': 1, 'albanys': 1, 'pitchfun': 1, 'bauer': 2, 'videolocal': 1, 'shoesrelieved': 1, 'heraides': 1, 'deafening': 1, 'atonal': 1, 'skull': 5, 'explodedetainees': 1, 'laborwoman': 1, 'blackouts': 2, 'timetop': 1, 'ten': 4, 'ebooks': 1, '21xs': 1, 'os': 1, 'obsession': 6, 'berkeley': 2, 'repconversation': 1, 'behindschool': 1, 'derekroy': 1, 'discrediting': 2, 'sociocultural': 1, 'seriouslyfear': 1, 'ticking': 2, 'biological': 3, 'outdoor': 3, 'ceremonydollar': 1, 'fig': 2, 'nortonslast': 1, 'novelist': 2, 'captivitytrump': 1, 'platinumlevel': 1, 'select': 1, 'journalistsjimmy': 1, 'rebrands': 1, 'disappointimalsohio': 1, 'pregnancyhershey': 1, 'businessthis': 3, 'starred': 1, 'porn4': 1, 'banished': 1, 'unionizerestaurant': 1, 'pizzadelivery': 1, 'dogprejudice': 1, 'discriminatehillary': 1, 'criticizes': 4, 'trumpgreys': 1, 'mcdreamygeorge': 1, 'clooney': 7, 'idiotic': 1, 'mexicowhat': 1, 'expectingmicrowavepopcorn': 1, 'maze': 1, 'arrows': 1, 'instructionsmatt': 1, 'booth': 3, 'campuss': 1, 'day3foottall': 1, 'apartmentbarack': 1, 'street3': 1, 'unusual': 4, '2015hacker': 1, 'prisonstop': 1, 'kidstheater': 1, 'bradley': 1, 'tr': 1, 'knight': 3, 'closetedmichelin': 1, 'tires': 1, 'womenrobert': 1, 'downey': 2, 'jarvisthe': 1, 'presidentfrogs': 1, 'warming': 3, 'croakquaker': 1, 'formulate': 2, 'oldestfashioned': 1, 'oatmealjohn': 1, 'hoses': 1, 'crumbs': 1, 'troop': 2, 'deploymentwoody': 1, 'dispensarytrumps': 1, 'achilles': 1, 'heelformer': 1, '10th': 4, 'mindnumbing': 1, 'boredomsean': 1, 'thoughtbrowns': 1, 'haslam': 1, 'mixes': 2, 'footballclinton': 1, 'hagar': 1, 'speedlimit': 1, 'issuesscalia': 1, 'bundles': 1, 'fur': 2, 'robe': 1, 'blizzardclinton': 1, 'flonasecary': 1, 'elwes': 1, '3the': 1, 'internetman': 1, 'tensionsbehind': 1, 'intricate': 3, 'stingcactus': 1, 'yearsome': 1, 'juxtaposing': 1, 'iconography': 1, 'bodily': 2, 'againtrump': 1, 'startedthe': 1, 'epiphany': 1, 'usreport': 1, 'ladensman': 1, 'unpauses': 1, 'mario': 5, 'choresmoronic': 1, 'mailroom': 1, 'ceoline': 1, 'palin': 9, 'fountainlocal': 1, 'templerandom': 1, 'houseben': 1, 'campaignbetter': 1, 'plussized': 1, 'succulent': 1, 'coverbernie': 1, 'carolinanew': 1, 'facebookhope': 1, 'fades': 1, '1999': 1, 'earthquakeseveral': 1, 'plows': 2, 'mosqueread': 1, 'jrs': 6, 'tavis': 1, 'smileyplease': 1, 'aness': 1, 'samehackers': 1, 'def': 2, 'con': 5, 'vegastoddler': 1, 'shove': 1, 'mouthanderson': 1, 'admire': 1, 'trumpeddie': 1, 'redmayne': 1, 'globespence': 1, 'educate': 3, 'premarital': 1, 'contacthome': 1, 'wheres': 2, 'thatmontana': 1, 'nominate': 4, 'banjo': 1, 'winthis': 2, 'mainsplainer': 1, 'victims35': 1, 'nowron': 1, 'cephas': 1, 'usneedleexchange': 1, 'attracting': 3, 'druggiessarah': 1, 'heaventhis': 1, 'nobrainer': 1, 'quicklymothers': 1, 'arrivesthirsty': 1, 'supply4yearold': 1, 'doll': 4, 'ropesdeadline': 1, 'dryer': 1, 'minutesarab': 1, 'isisbradley': 1, 'unrecognizable': 1, 'rolechris': 1, 'campaigndisturbance': 1, 'arafats': 1, 'curse': 3, 'eastword': 1, 'presumptive': 2, 'hibernation5': 1, 'criticmichelle': 1, 'housefirst': 1, 'draftpornographydesensitized': 1, 'orifice': 1, 'atozzy': 1, 'ozzfestferguson': 1, 'lootersturkey': 1, 'farther': 1, 'villagers': 1, 'killedcensus': 1, 'participants': 3, 'unpatriotic': 1, 'neighborpolice': 1, 'gunhuffpollster': 1, 'estimates': 1, 'cruzmissed': 1, 'cheneyarchaeologists': 1, 'earliest': 2, 'everywhereyoga': 1, 'managementdemocratic': 1, 'vow': 6, 'safetygrin': 1, 'revealed': 7, 'ingredientsbeyoncs': 1, 'postpregnancy': 1, 'menswear': 1, 'collectionchildren': 1, 'fitmary': 1, 'blige': 1, 'showvice': 1, 'raisenation': 1, 'illegitimate': 1, 'daycongress': 1, '34th': 2, 'shortterm': 1, 'patch': 4, 'highwaystsunami': 1, '36': 2, 'americanshemmedin': 1, 'deforestationwhat': 1, '25yearold': 2, 'workercoffee': 1, 'morningfacebook': 1, 'earthpoll': 1, 'storming': 1, 'inhuman': 1, 'monsterbehold': 1, 'charactershow': 1, 'devalue': 1, 'womenclass': 1, 'clown': 4, 'wilmot': 1, 'provisoarea': 1, 'beepingelizabeth': 1, 'ny': 3, 'tapesfamily': 1, 'lax': 1, 'standards': 4, 'auntwerner': 1, 'herzog': 1, 'treadwell': 1, '2003cheney': 1, 'offspring': 1, 'chestcalifornia': 1, 'attacking': 8, 'elementary': 3, 'schoolleaping': 1, 'floridavoter': 1, '17looking': 1, 'itmen': 1, 'behaving': 1, 'badlywoman': 1, 'vulgar': 2, 'messages': 3, 'againdocumentary': 1, 'plymouth': 1, 'interesteddonald': 1, '45th': 3, 'spontaneously': 2, 'subconscious': 2, 'daywhats': 1, 'dystopiagoogle': 1, 'translate': 2, 'moneyjoan': 1, 'peoplebritish': 1, 'empire': 2, '2050man': 1, 'fishman': 1, 'rub': 2, 'alreadyhot': 1, 'toddler6': 1, 'macaucnns': 1, 'bench': 3, 'punditsperhaps': 1, 'oathseric': 1, 'cantors': 1, 'jobrelationship': 1, 'hassledepartment': 1, 'refugethe': 1, 'squadgoals': 1, 'falcons': 2, 'gamei': 1, 'babysitterkesha': 1, 'lifegod': 2, 'rightful': 1, 'bankshy': 1, 'itselfet': 1, 'uninterested': 2, 'childrenactors': 1, 'doctorrenamed': 1, 'wireless': 1, 'amphitheater': 2, 'localswhy': 1, 'roadbenedict': 1, 'cumberbatch': 2, 'trailerwebsite': 1, 'itselfare': 1, 'vital': 2, 'hacking': 4, 'strategythe': 1, 'bachelorettes': 1, 'whaboom': 1, 'worstestablishmentbacked': 1, 'heated': 2, 'texaswhite': 1, 'lighting': 3, 'crossstates': 1, 'alternatives': 2, 'decisionsheres': 1, 'carejustin': 1, 'allnighter': 1, 'tomorrowiraq': 1, 'lawmarqkria': 1, '260': 1, 'wanting': 3, 'lifenations': 1, 'ninetysomethings': 1, 'livesbarber': 1, 'daylunch': 1, 'organssepp': 1, '90day': 1, 'fifapan': 1, 'soak': 2, 'predates': 1, 'roommatesis': 1, 'planflaws': 1, 'evaluate': 1, 'kahnemans': 1, 'slowemotional': 1, 'needthese': 1, 'religions': 1, 'usasean': 1, 'cap': 4, 'presshillary': 1, 'believerspolice': 1, 'sprayedwho': 1, 'debaterelatives': 1, 'togetherglandular': 1, 'helpinghow': 1, 'thaw': 2, 'turkeycassini': 1, 'setup': 1, 'alongbrad': 1, 'movienew': 1, 'workbush': 1, 'feverishly': 1, 'boobytrapping': 1, 'deskfox': 1, 'bret': 1, 'baier': 1, 'mistakejudge': 1, 'resentencing': 1, 'gaybashing': 1, 'casesyrian': 1, 'cryingthis': 1, 'octogenarian': 2, 'minitrainman': 1, 'jobstacy': 1, 'ruizs': 1, 'yourselflarge': 1, 'resounding': 1, 'ovationoregon': 1, 'teamsyrian': 1, 'secure': 2, 'evacuationrelationship': 1, 'unspeakable': 1, 'anxietyreading': 1, 'thermometer': 1, 'sitesmy': 1, 'thankfulhuffpost': 1, 'dayfatherson': 1, 'balancing': 3, 'technologytrusting': 1, 'gracethe': 1, 'odessa': 1, 'playlistus': 1, 'motto': 1, 'ya': 5, 'smileyankees': 1, 'affiliate': 1, 'timing': 1, 'coincidencejohn': 1, 'prejudiced': 1, 'sideelizabeth': 1, 'chat': 2, 'wink': 1, 'nodtodd': 1, 'akin': 1, 'wrongshredyourex': 1, 'dayparents': 1, 'videotape': 1, 'talkdog': 1, 'chastised': 1, 'dogtwitter': 1, 'gopsongsaboutethics8': 1, 'brawl': 2, 'killedmans': 1, 'lifecrocodile': 1, 'bedsense8': 1, 'yetreport': 1, 'seasonal': 3, 'lampthe': 1, 'businessfriend': 1, 'dishonorably': 1, 'discharged': 1, 'navigation': 1, 'exitsitcom': 1, 'intellectual76ers': 1, 'snap': 2, '28game': 1, 'skidsteve': 1, 'allegationshawaiis': 1, 'surf': 1, 'aloha': 1, 'aina': 1, 'barcadr': 1, 'foxmarvel': 1, 'reimagines': 1, 'lefthandedtraffic': 1, 'californiapolice': 1, '96': 3, 'reportwhy': 1, 'breastfeeding': 2, 'gowar': 1, 'republicansarea': 1, 'somethingman': 1, 'effortpolice': 1, 'visithire': 1, 'employmentgeneral': 1, 'charms': 4, 'leprechaun': 1, 'meatrepublicans': 1, 'middleclass': 2, 'householdsqueen': 1, 'monarchys': 1, 'initiativepope': 1, 'washes': 1, 'phillie': 1, 'phanatic4': 1, 'jumpstart': 1, 'freedomemergency': 1, 'dislodge': 1, 'delhiobama': 1, 'adjust': 1, 'chairbouncer': 1, 'clubapple': 1, 'walletjennifer': 1, 'clothesline': 1, 'linemotivational': 1, '264': 1, 'layoffsplease': 1, 'officefat': 1, 'flower92': 1, 'involve': 5, 'pulverizing': 1, 'oreosbill': 1, 'melinda': 2, 'eradicate': 1, '3rdworld': 1, 'mac': 3, '2040white': 1, 'murderernew': 1, 'consult': 2, 'alreadysubscription': 1, 'kidsa': 1, 'causing': 12, 'libidofirst': 1, 'sinoamerican': 1, '2011': 2, 'writtenasian': 1, 'tsunami': 1, 'katrina': 2, 'kashmir': 1, 'disasty': 1, 'awardstruggling': 1, 'shutter': 1, 'magazinemueller': 1, 'subpoena': 1, 'sitdown': 1, 'interviewrecreationalabortion': 1, 'enthusiasts': 1, 'applaud': 1, 'partialbirth': 1, 'banconductor': 1, 'fatigue': 2, 'crashfather': 1, 'timethursdays': 1, 'hurdlesnot': 1, 'baldwinwhat': 1, 'aboutmillions': 1, 'deathsmedical': 1, 'trialcharity': 1, 'exec': 5, 'mt': 2, 'kilimanjaro': 1, 'servicespoll': 1, 'gandalf': 1, 'abyssnew': 1, 'assfrom': 1, 'closehouse': 1, 'hostageinternet': 1, 'pictureswoman': 1, '4minute': 1, 'selfdeprecating': 1, 'preamble': 1, 'mindrobert': 1, 'runway': 4, 'offhelp': 1, 'gassedthe': 1, 'hateneil': 1, 'degrasse': 3, 'slip': 2, 'marstrump': 1, 'sacrificeana': 1, 'navarro': 3, 'impeach': 2, 'thistempurapedic': 1, 'extracrispy': 1, 'deepfried': 1, 'mattressesstand': 1, 'careluxurycraving': 1, 'confidently': 2, 'squandering': 3, 'pre2008': 1, 'levelsless': 1, 'yesamericans': 1, '2to1': 1, 'iraqarea': 1, 'bitethese': 1, 'togethernepal': 1, 'threeno': 1, 'commentsobama': 1, 'debts': 1, 'owed': 1, 'defrauded': 1, 'forprofit': 4, 'studentsvatican': 1, 'communion': 1, 'waferivycovered': 1, 'toowhy': 1, 'watson': 1, 'actingman': 1, 'chippewa': 2, 'eagle': 1, 'fallsdwight': 1, 'africavessel': 1, 'todaythe': 3, 'colorchris': 1, 'tweetcomic': 1, 'logically': 1, 'extremedear': 1, 'perpetuating': 1, 'cultureabortion': 1, 'bufferzone': 1, 'mccullen': 1, 'facade': 1, 'rightsratings': 1, 'npr': 3, 'crewstudy': 1, 'weirdpolice': 1, '11yearolds': 1, 'brandnew': 1, 'onethis': 1, 'urkel': 1, 'happenedhuffpost': 1, '21new': 1, 'vh1': 1, 'enough30': 1, 'about71': 1, 'warmingdont': 1, 'weeksthese': 1, 'pentagon': 4, 'guncheck': 1, 'systemhomeless': 1, 'parents9': 1, 'quitstepson': 1, 'nailing': 2, 'reichamanda': 1, 'hugthat': 1, 'grandmotherthe': 1, 'communitycreditcard': 1, 'metallurgists': 1, 'polonium': 1, 'visa': 2, 'cardtrump': 1, 'stressful': 1, 'daytrumps': 1, 'principlessex': 1, 'sinks': 5, 'klemke': 1, 'bidfamily': 1, 'appreciably': 1, 'rushmoremonarch': 1, 'directorial': 1, 'episodehillarys': 1, 'truthitaly': 1, 'tents': 1, 'amatrice': 1, 'saysdepartment': 1, 'encountersfrolicking': 1, 'deer': 1, 'ticksdonald': 1, 'historically': 3, 'markswoman': 1, 'landingrosemary': 1, 'kowalski': 1, 'imagining': 3, 'beautystupid': 1, 'ranks': 1, 'crapaaa': 1, 'crashchimp': 1, 'direct25yearold': 1, 'immunization': 1, 'insurancehow': 1, 'paragliding': 1, 'perspective': 7, 'lifejj': 1, 'namesim': 1, 'taxed': 1, 'womancommercial': 1, 'placementthe': 1, 'haterobert': 1, 'kirkman': 2, 'theoryduchess': 1, 'coatwashedup': 1, 'to50': 1, 'caseclimate': 1, 'debate4': 1, 'stronger': 2, 'obamaarea': 1, 'juggles': 1, 'jugglingwalmart': 1, 'ar15s': 1, 'militarystyle': 1, 'rifleselementary': 1, 'gymnasiummountaintop': 1, 'maharishi': 1, 'greenspan': 2, 'rupeepalm': 1, 'camerasnew': 1, 'sleepman': 1, 'toddlerlesson': 1, 'chicagoans': 1, 'schoolsman': 1, 'oldhuffpost': 1, 'nazis': 2, 'pavement': 2, 'skulls': 1, 'trumpjurisprudence': 1, 'technicalitynow': 1, 'citiesemboldened': 1, '20week': 2, 'banactress': 1, 'misty': 1, 'upham': 1, 'stuyvesant': 1, 'schoolbiggest': 1, 'leak': 5, 'moneyamericans': 1, '30000': 1, 'repealedkelly': 1, 'vacationattorney': 1, 'deathyep': 1, 'seasonlessons': 1, 'friendsex': 1, 'agebiden': 1, 'trashed': 1, 'residence': 2, 'secondmccain': 1, 'electeda': 1, 'influencer': 2, 'loudest': 2, 'roomnewlywed': 1, 'fatal': 5, 'accidentbryan': 1, 'cranston': 1, 'signalarming': 1, 'fromwhen': 1, 'avoidjewel': 1, 'organizes': 1, 'unicorns': 1, 'benefitkeebler': 1, 'expands': 4, 'residencethemed': 1, 'crackersarthouse': 1, 'missednew': 1, 'conflicttext': 1, 'curt7': 1, 'infections': 1, 'athletes': 5, 'rios': 1, 'contaminated': 3, 'watersdefending': 1, 'trumpall': 1, 'anthemwatch': 1, 'disneylocal': 1, 'insert': 2, 'cointrump': 1, 'systempope': 1, 'russiaperson': 1, 'doling': 1, 'counsel': 1, 'elderastronomers': 1, 'novtakeout': 1, 'shielded': 1, 'weekold': 1, 'newborncereal': 1, 'neglects': 3, 'breakfastmicrosoft': 1, 'carsthinking': 1, 'broadly': 1, 'mothering': 1, 'day8th': 1, 'grader': 4, 'eventwhy': 1, 'discriminationpolice': 1, 'diego': 3, 'suburb': 1, 'sparking': 4, 'protestsculinary': 1, '3star': 1, 'ponycelebrity': 1, 'collector': 1, 'sweeneystunted': 1, '56yearold': 1, 'palahniuk': 1, 'novelsrnc': 1, 'anywayarbys': 1, 'trunk': 4, 'horsey': 1, 'sauceobama': 1, 'seated': 4, 'tense': 3, 'dinnertrevor': 1, 'controlari': 1, 'fleischer': 1, 'toby': 1, 'keithtrumps': 1, 'ties': 11, 'withdraws': 2, 'nominationpaul': 1, 'undecided': 4, 'debatejohn': 1, 'statehood': 1, 'votesthe': 1, 'peshawar': 1, 'pakistan': 4, 'narrativepersonal': 1, 'jrpaul': 1, 'securitystylin': 1, 'crisisman': 2, 'flirting': 2, 'boyfriendthe': 2, 'crushworthy': 1, 'yataylor': 1, 'wind': 4, 'pissing': 2, 'offman': 1, 'lowcut': 2, 'swimsuit': 4, 'sunkissed': 1, 'sardinian': 1, 'covelab': 1, 'partnersasha': 1, 'squirming': 1, 'makeovernaked': 1, 'mingles': 1, 'freely': 2, 'roomcd': 1, 'womanhere': 1, 'winnersdoes': 1, 'lactatebirds': 1, 'nest': 2, '65': 4, 'buttsim': 1, 'exgirlfriend': 1, 'thatman': 1, 'laborhealth': 1, 'crossroads': 3, 'peril47': 1, 'weakwilled': 1, 'peoplewant': 1, 'antebellum': 1, 'southhow': 1, 'authorhouse': 1, 'strikeus': 1, 'explosion': 3, 'syriacops': 1, 'superfriendly': 1, 'womanscientists': 1, 'boosting': 1, 'metabolism': 1, 'sleepamerican': 1, 'zoos': 2, 'escaped': 4, 'cobra': 2, 'mattersgiant': 1, 'bass': 2, 'takenbp': 1, 'spillingchinas': 1, 'freshwater': 1, 'shrinkingsienna': 1, 'humorous': 1, 'useducating': 1, 'gamedear': 1, 'thispaleontology': 1, 'winces': 2, 'whenever': 6, 'fundamentalist': 1, 'handintelligence': 1, 'spy': 3, 'onboyfriend': 1, 'ceremoniously': 1, 'dumpedstudy': 1, 'increased': 3, 'aggression': 1, 'characterscoachellagoers': 1, 'headliner': 1, 'twinsman': 1, '40disc': 1, 'changer': 3, 'cdswhite': 1, 'lahrens': 2, 'pointkenyan': 1, 'sourcenation': 1, 'conferences': 1, 'guyssleepy': 1, 'elephant': 6, 'napping': 2, 'spot5': 1, 'nappingman': 1, 'receivedthe': 1, 'playlistbernie': 1, 'perriello': 1, 'gubernatorial': 4, 'raceis': 1, 'algae': 1, 'youreport': 3, 'bones': 3, 'earhartsome': 1, 'todayunited': 1, 'skips': 2, 'passengerdragging': 1, 'incidentwalmart': 1, 'throatscharlton': 1, 'hestons': 1, 'handstrump': 1, 'work5': 1, 'divorcing': 1, 'friendchrissy': 1, 'usfamily': 1, 'shells': 2, 'cheesegreg': 1, 'behrendt': 1, 'youactivist': 1, 'preaches': 1, 'acceptanceparents': 1, 'reminisce': 1, 'algorithm': 4, 'togetheralqaeda': 1, 'radicalization': 1, 'facetoface': 1, 'onlinemajority': 1, 'reformgary': 1, 'busey': 1, 'drowns': 2, 'poolarea': 1, 'hurtswiping': 1, 'hottie': 1, 'secondbirthplace': 1, 'visitedcelebrities': 1, 'urging': 4, 'australians': 2, 'marriagethe': 1, 'weaker': 2, 'strongerkushner': 1, 'collude': 1, 'rightwhite': 1, 'fencechina': 1, 'tightens': 2, '20th': 3, 'takeoverepa': 1, 'dakotareport': 1, 'anklestoo': 1, 'naildepartment': 1, 'obamaera': 1, 'elkthis': 1, 'ponytail': 2, 'holderheroic': 1, 'journeys': 1, 'ownerthe': 1, 'rev': 1, 'mississippitrump': 1, 'pointmetlife': 1, 'goodyear': 1, 'mergethere': 1, 'outsidethere': 1, 'headon': 1, 'crashmacklemore': 1, 'trumpgoody': 1, 'governess': 1, 'hairbrushes': 1, 'raking': 1, 'scalps': 1, 'insolent': 1, 'girlsamerican': 1, 'teethanonymous': 1, 'cowardly': 1, 'snitchwhat': 1, 'interreligious': 1, 'santegidio': 1, 'meetinggrand': 1, 'theft': 2, 'missions': 2, 'tutoring': 2, 'outreachjj': 1, 'doubted': 1, 'hotnessthis': 1, 'dognothing': 1, 'multinational': 2, 'pepsicoowned': 1, 'mexicanthemed': 1, 'outletsbidens': 1, 'stink': 1, 'altogetherafghan': 1, 'lionel': 1, 'heroeveryone': 1, 'jewishtexas': 1, 'doses': 1, 'pancuronium': 1, 'bromide': 1, '200417': 1, 'sextreport': 1, 'lodged': 1, 'carsuperhero': 1, 'mildmannered': 1, 'issame': 1, 'jumbotron': 1, 'divorcelisten': 1, 'carsons': 3, 'adwhat': 1, 'billdirector': 1, 'alexia': 1, 'kosmider': 1, 'transjourney': 1, 'audiowhy': 1, 'vawa': 1, 'anniversarytrumps': 1, 'latin': 6, 'neighborsnation': 1, 'knowman': 1, 'floridataraji': 1, 'girlthis': 1, 'investing': 2, 'headreport': 1, 'highpowered': 2, 'trillionairemore': 1, 'temps': 1, 'skirt': 2, 'lawsone': 1, 'primarythe': 1, 'billionsenterprising': 1, 'oilwoman': 1, 'madtaxi': 1, 'partyworst': 1, 'pregnantgoddamn': 1, 'ficus': 1, 'instructionslockheed': 1, 'instructed': 2, 'tactical': 1, 'airtosurface': 1, 'weekyoung': 1, 'stray': 2, 'bulletcommercial': 1, 'tomato': 1, 'lookembarrassing': 1, 'imf': 1, 'headquartersstudent': 1, 'accurate': 2, 'sentencesreuters': 1, 'storylyndon': 1, 'editornew': 1, 'pompous': 2, 'cigar': 1, 'aficionadopettingzoo': 1, '4yearolddate': 1, 'upstairs': 3, 'flags7': 1, 'nowdrunk': 1, 'copsthe': 1, 'killingswoman': 1, 'rainforests': 1, 'timeaudience': 1, 'takenapparently': 1, 'witherspoon': 3, 'jcrew': 1, 'doshit': 1, 'ordering': 3, 'crew40yearold': 1, 'spiky': 1, 'hairhow': 1, '3dprinted': 1, 'heartbeyonce': 1, 'affordable': 7, 'lookis': 1, 'sahm': 1, 'mothernurses': 1, 'wishhow': 1, 'stressfestnew': 1, 'ups': 2, 'extendedtracking': 1, 'package': 1, 'dropoffcastro': 1, 'tallersympathy': 1, 'signed': 3, 'assistantcongress': 1, 'splits': 2, 'billcompany': 1, 'yearmr': 1, 'fuji': 1, 'wrestler': 1, '82how': 1, 'bewoman': 1, 'weekendan': 1, 'issuefbi': 1, 'probes': 2, 'phones30th': 1, 'commemoratedhuffpost': 1, '30thwedding': 1, 'eyed': 3, 'timewatch': 1, 'weeknds': 1, 'grey': 2, 'videothe': 3, 'partynasa': 1, 'curiosity': 3, '2gigabyte': 1, 'cardreport': 1, 'handmouth': 1, 'coordinationice': 1, 'devastating': 5, 'tweetdea': 1, 'seizes': 3, 'halfbuilt': 1, 'bogot': 1, 'miamiwant': 1, 'labelswhy': 1, 'biologist': 1, 'gspot': 1, 'mythis': 1, 'jebs': 1, 'convincing': 5, 'tar': 1, 'escapeeconomy': 1, 'overwhelmed': 3, 'schoolersthe': 1, 'nondouchey': 1, 'hoverboardart': 1, 'renoir': 1, 'workdesperate': 1, 'choices': 2, 'ofcolorkylie': 1, 'selfiesalad': 1, 'rendered': 1, 'steps7': 1, 'migrainespulse': 1, 'informant5yearold': 1, 'underfunded': 2, 'in3': 1, 'activated': 1, 'blanketpoll': 1, 'spitea': 1, 'nurses': 3, 'birthsasha': 1, 'digging': 3, 'benghaziwoman': 1, 'makeoverveteran': 1, 'employment': 3, 'priorityromney': 1, '2006why': 1, 'feeds': 2, 'freakish': 1, 'vegetablesrnc': 1, 'attendee': 2, 'tonightchristmas': 1, 'pageant': 1, 'preproductionreturning': 1, 'weekendsecondsemester': 1, 'fling': 1, 'firsttrimester': 1, 'abortionjuror': 1, 'contusions': 1, 'nowgop': 1, 'obamacarejessica': 1, 'slimmer': 2, 'chopping': 3, 'limbspolar': 1, 'cuddly': 1, 'clipits': 1, 'motherhoodwhile': 1, 'languish': 1, 'flourishteen': 1, 'newsweek': 1, 'bombno': 1, 'clam': 2, 'stroke2014': 1, 'festivalbachmann': 1, 'leadmildfires': 1, 'amble': 1, 'californiaflorida': 1, 'deathall': 1, 'concertin': 1, 'memoriam': 2, 'thickes': 1, 'careerhumanizing': 1, 'biopoor': 1, 'needs7': 1, 'inventions': 2, 'mondayprotecting': 1, 'contending': 1, 'pastbill': 1, 'releasedhistorians': 1, 'hoover': 1, 'damarchaeologists': 1, 'pyramids': 2, 'campingsecretary': 1, 'nationsteve': 1, 'didindependent': 1, 'andrea': 1, 'suarez': 1, 'paz': 1, 'impossiblejudge': 1, 'understands': 2, 'fromtrumps': 1, 'usdavid': 1, 'axelrod': 2, 'obamaice': 1, 'cowriting': 2, 'genrecrossing': 1, 'musicalfamily': 1, 'flu': 7, 'early5': 1, 'themuniversity': 1, 'roomglobal': 1, 'lovetwo': 1, 'shipwrecks': 1, '239': 1, 'saysyour': 1, 'cdc': 3, 'warnstour': 1, 'horsethree': 1, 'libyas': 1, 'benghazi': 5, 'medicscoworker': 1, 'theresamantha': 1, 'takedown': 2, 'councilwomenman': 1, 'filth': 1, 'qtip': 1, 'eartrump': 1, 'rioting': 2, 'anywaygreatgrandmother': 1, 'greatmy': 1, 'philly': 2, 'jurorfive': 1, 'channelhair': 1, 'disheveled': 1, 'ritualnew': 1, 'screenan': 1, 'iceberg': 2, 'penguins': 3, 'updategop': 1, 'offense': 2, 'lessdigital': 1, 'programsafter': 1, 'option': 5, 'dcjust': 1, 'lourea': 1, 'excerpt7': 1, 'corral': 1, 'wireswhy': 1, 'appreciated': 1, 'mecheering': 1, 'servicebar': 1, 'options4': 1, 'successarea': 1, 'allergic': 3, 'strenlow': 1, 'dander7': 1, 'pantry': 2, 'staplesarea': 1, '2swe': 1, 'reached': 4, 'cks': 1, 'silencejohns': 1, 'hopkins': 1, 'thumbreport': 1, 'supplying': 1, 'timestoddler': 1, 'sensible': 1, 'midrange': 1, 'tricyclebloody': 1, 'detached': 2, 'julius': 2, 'peppers': 4, 'facemaskbill': 1, 'it10': 1, 'homehow': 1, 'tsourekiman': 1, 'hersh': 2, 'subject': 3, '20page': 1, 'exposchechen': 1, 'strongman': 1, 'catreport': 1, 'ham': 5, 'sandwich75': 1, 'trolley': 1, 'defaulting': 1, 'loansto': 1, 'thisreligious': 1, 'hateful': 6, 'vandalism': 3, 'churchesresolute': 1, 'againcrossing': 1, 'cardiomyopathyman': 1, 'lifenew': 3, '41cent': 1, 'stampthese': 1, 'greeks': 2, 'country7': 1, 'lifestyledickipedia': 1, 'bikram': 1, 'yogaquiet': 1, 'guyan': 1, 'exile': 1, 'illustrated': 1, 'manuscriptsdrunk': 1, 'pilot': 4, 'sobers': 1, 'littlenestle': 1, 'switch': 2, 'cagefree': 1, '2020the': 1, 'amateurism': 1, 'pastoh': 1, 'gumby': 1, 'pokey': 2, 'blizzard': 1, 'lapseovarian': 1, 'publicistearthquakes': 1, 'zealanddead': 1, 'ipod': 2, 'expensivechina': 1, 'railways': 1, 'asiacollege': 1, 'recount': 3, 'faculty': 3, 'committeebirthday': 1, 'meetingpaddlers': 1, 'underwatertoward': 1, 'fairer': 1, 'processequifax': 1, 'efficiently': 1, 'canstress': 1, 'fatbrandon': 1, 'impulse': 1, 'himvideo': 1, 'fellow': 5, 'linehow': 1, 'pumpkin': 5, 'dogcoal': 1, 'stockingsryan': 1, 'antipoverty': 1, 'initiativeparkland': 1, 'unimpressed': 2, 'callelon': 1, 'musk': 7, 'conquer': 1, 'marsmasterchef': 1, 'prawns': 1, 'competearea': 1, 'lazier': 1, 'accomplished': 1, 'yearwhy': 1, 'carmen': 1, 'carrera': 1, 'transdonald': 1, 'machado': 1, 'jobrestaurant': 1, 'rude': 1, 'autismarea': 1, 'decadepope': 1, 'tanktracee': 1, 'ellis': 1, 'playfully': 1, 'gownobama': 1, 'dinnerus': 1, 'deserter': 1, 'bowe': 1, 'sentencing': 2, 'beginsamy': 1, 'ratherdeceptively': 1, 'worktown': 1, 'answereddomhnall': 1, 'gleeson': 1, 'hux': 1, 'brightest': 3, 'actorsmomtobe': 1, 'airplane': 2, 'pilots': 2, 'helpgabby': 1, 'giffords': 1, 'presidenthippocratic': 1, 'hmo': 2, 'boardwhy': 1, 'hollywoodtrump': 1, 'table13': 1, 'mondaysfootball': 1, 'cheerleader': 1, 'colorful': 3, 'surprisesonomas': 1, 'wackiest': 2, 'winerieswhy': 1, 'quinn': 1, 'infuriating': 2, 'ordergod': 1, 'wedges': 1, 'cherub': 1, 'thronehugh': 1, 'hefner': 2, 'beside': 1, 'playboys': 2, 'monroelindsey': 1, 'roof': 4, 'innnot': 1, 'snowing': 3, 'reportsheres': 1, 'guardians': 3, 'galaxypunk': 1, 'punks': 1, 'trumpread': 1, 'voteleslie': 1, 'hack': 4, 'securitymovie': 1, 'hoped9pound': 1, 'canalofficers': 1, 'excessive': 2, 'reporthillary': 1, 'donating': 2, 'sums': 6, 'womanman': 1, 'escalate': 1, 'situationlisten': 1, 'turkeys': 3, 'declared': 4, 'warice': 1, 'cubeobama': 1, 'hw': 5, 'fallcompliment': 1, 'suspiciously': 2, 'vaguevending': 1, 'uptodate': 1, 'schoolnabisco': 1, 'xtreme': 1, 'saltassault': 1, 'saltinestenants': 1, 'landlord': 2, 'micestorybook': 1, 'inflightmagazine': 1, 'marriagetrump': 1, 'buffoonery': 1, 'storenra': 1, 'commonsense': 1, 'deathslatearriving': 1, 'encouraged': 2, 'hoursmike': 1, 'goodkansas': 1, 'shipsbanyans': 1, 'smoothiedisappointing': 1, 'vaults': 1, '37000': 1, 'coversif': 1, 'em': 5, 'screw': 4, 'gutting': 1, 'successors': 1, 'powerbackpacker': 1, 'europeans': 1, 'preconceptions': 1, 'americansauction': 1, 'crab': 3, 'clawhow': 1, 'ficos': 1, 'consumerswarm': 1, 'sweatthe': 1, 'vietnam': 3, 'orangemysterious': 1, 'necrotic': 1, 'kissed': 1, 'cruzas': 1, 'socialism': 1, 'democracytechnology': 1, 'reconnectgot': 1, 'charactervast': 1, 'array': 2, 'lipbalm': 1, 'paralyzes': 2, 'shopperhealth': 1, 'reformkoch': 1, 'registering': 1, 'pacdear': 1, 'superpowers': 2, 'workhillary': 1, 'bounces': 1, 'hampshirehillary': 1, 'nrakroger': 1, 'ceoalabama': 1, 'deny': 7, 'chargeshow': 1, 'theorists': 1, 'worldgrown': 1, 'duping': 1, 'childrennews': 1, 'slain': 3, 'citytrump': 1, 'exorcisewisdom': 1, 'advisor': 2, 'businessthese': 1, 'bookshelf': 1, 'spotonarchitect': 1, 'generic': 5, 'yementhis': 1, 'albinism': 1, 'dispel': 2, 'harmful': 3, 'mythsanthony': 1, 'sext': 1, 'campaignhundreds': 1, 'precipice': 1, 'carenortheast': 1, 'powerfedup': 1, 'doornew': 1, 'molest': 1, 'selffrom': 1, 'culturally': 2, 'barriersarea': 1, 'businessdiscouraged': 1, 'nationsman': 1, 'anywhereuber': 1, 'thissandra': 1, 'swallowed': 1, 'smoked': 2, 'quantity': 2, 'dagirls': 1, 'truthkeeganmichael': 1, 'detroitindiana': 1, 'explicitly': 4, 'do10': 1, '2017donald': 1, 'advisory': 4, 'committeedogs': 1, 'oxytocinmediated': 1, 'bondterrible': 1, 'paperwhite': 1, 'feelingsbernie': 1, 'stresses': 2, 'capitalismscientists': 1, '30yard': 1, 'radius': 2, 'governmentengineered': 1, 'warhistorical': 1, 'owls': 1, 'arseholesracehorse': 1, 'bracesguidebook': 1, 'eyesthe': 1, 'puberty': 1, 'youthwe': 1, 'cravings': 1, 'geneticmark': 1, 'halperin': 2, 'allegationsford': 1, 'sportfutility': 1, 'vehiclethe': 1, 'nevertrump': 1, 'wisconsinhow': 1, 'judith': 1, 'ageismfda': 1, 'architecturally': 1, 'soundmother': 2, 'casanovausher': 1, 'appearanceevenings': 1, 'recapped': 1, 'digitalcamera': 1, 'showhistorical': 1, 'publick': 1, 'spectacleagents': 1, 'nowhopper': 1, 'memenra': 1, 'injusticetop': 1, 'ppm': 1, 'co2': 1, 'numbersswimsuit': 1, 'conceals': 1, 'thigh': 2, 'region11': 1, 'timesoaring': 1, 'ballets': 1, 'noche': 1, 'pasin': 1, 'tango': 1, 'soiredear': 1, '2017even': 1, 'chrissy': 2, 'adelsons': 1, 'crazier': 1, 'thinkjenna': 1, 'fischer': 1, 'pam': 1, 'episodereport': 1, 'kayakgarners': 1, 'actiongrandmother': 1, 'chinacool': 1, 'speechbeijing': 1, 'solidifiesplo': 1, 'krippendorfs': 1, 'tribejeff': 1, 'veteranhere': 1, 'trumpbeyonces': 1, 'corset': 1, 'happenromney': 1, 'nonetheless': 2, 'trumpgummy': 1, 'chicagoarea': 1, 'hospitalexcited': 1, 'poacher': 1, 'stalks': 1, 'preylil': 1, 'ok14yearold': 1, 'logosthe': 1, 'leadstudy': 1, 'frontal': 1, 'lobe': 1, 'occupied': 2, 'sausage': 2, 'linksarea': 1, 'shitreallyloudwhistle': 1, 'whistle': 1, 'loudly40': 1, 'fencechris': 1, 'waffling': 1, 'presidentialthis': 1, 'adulting': 1, 'hardwhere': 1, 'herenabisco': 1, 'discontinues': 5, 'wheat': 3, 'thicksdecision': 1, '10sided': 1, 'dieusedbookstore': 1, 'chairfamily': 1, 'subconsciouskiss': 1, 'vinnie': 1, 'vincent': 3, 'bandtomi': 1, 'theblaze': 1, 'prochoice': 1, 'remarksblocking': 1, 'threatstephen': 1, 'facehow': 1, 'successfritolay': 1, 'contest': 4, 'sunchipshalf': 1, 'catastrophejohn': 1, 'mexicobosnian': 1, 'warflavored': 1, 'gumindiana': 1, 'chasewhy': 1, 'happyproof': 1, 'gapdont': 1, 'again5': 2, 'debatehead': 1, 'integrity': 2, 'oped': 1, 'fraudpainful': 1, 'boil': 3, 'unformed': 1, 'lancekhalid': 1, 'sheikh': 1, 'mohammed': 1, 'confessing': 1, 'torturelebron': 1, 'nbas': 1, 'gamewinner': 1, 'daysthe': 1, 'femaleshobby': 1, 'punishmentsvoters': 1, 'confusion': 1, 'electionhusband': 1, 'separation': 1, 'matereport': 1, 'crooked': 2, 'planting': 1, 'cars2015s': 1, 'yearend': 1, 'incrediblewatch': 1, 'asexualnasa': 1, 'templetonyoung': 1, 'afghans': 2, 'isolation': 1, 'homereport': 1, 'dysmorphia': 2, 'sufferers': 2, 'skewed': 1, 'perception': 1, 'friendsmeryl': 1, 'ruffalo': 3, 'bachelorette': 6, '6millions': 1, 'dresser3': 1, 'hireschnauzers': 1, 'westminster': 1, 'defeathardened': 1, 'snacker': 1, 'rediscover': 2, 'mindblowing': 1, 'nacho': 2, 'highskydivers': 1, 'feetreport': 1, 'summerman': 1, 'respectarea': 1, 'liner': 2, 'albumlarry': 1, 'kramers': 1, 'allgreece': 1, 'edgehelpful': 1, 'visitors': 2, 'skippingrice': 1, 'krispie': 1, 'squarenow': 1, 'kurdi': 1, 'canada3': 1, 'doctorhow': 1, 'connectivity': 1, 'connectionsdetermined': 1, 'diligently': 1, 'topicdems': 1, 'dropping': 5, 'schultzarea': 1, 'recessionwealthiest': 1, 'electionlindsey': 1, 'booed': 3, 'gorsuchmitch': 1, 'courtreport': 1, 'heavens': 1, 'begged': 1, 'forgiveness': 3, 'secondholiday': 1, 'wines': 2, 'nw': 1, 'sommelierneighborhood': 1, 'vanquished': 1, 'trolltrump': 1, 'posttrump': 2, 'conferencethose': 1, '2011the': 1, 'darkness': 2, 'outlast': 1, 'trumpindian': 1, 'familiar': 4, 'dolezals': 1, 'worldus': 1, 'fend': 2, 'flintfull': 1, 'tending': 2, 'edible': 1, 'cherry': 2, 'tomatoentertainment': 1, 'knack': 1, 'concepts': 2, 'readersfergusons': 1, 'answersnpr': 1, 'voicewhite': 1, 'rican': 3, 'dealgovernment': 1, 'agreementpaula': 1, 'poundstone': 1, 'famouscrisis': 1, 'trollsamsung': 1, 'athalf': 1, 'notyetreleased': 1, 'filmbuying': 1, 'hairstylist': 3, '8000area': 1, '360': 1, 'gameactivist': 1, 'wettshirt': 1, 'girlfriendchaotic': 1, 'et': 3, 'complicatednations': 1, 'loyalists': 2, 'nigels': 1, 'bangers': 1, 'mash': 1, 'contestrepublicans': 1, 'ignorant': 1, 'boorjeb': 1, 'bungles': 1, 'homenothing': 1, 'americapolice': 1, 'brazil': 3, 'caseun': 1, 'afghanistanlaura': 1, 'jane': 2, 'communityarea': 1, 'middlelet': 1, 'conflicthuffpollster': 1, 'dump': 2, 'cruzgop': 1, 'weekbarron': 1, 'clarinet': 1, 'performanceaustralia': 1, 'detention': 3, 'investigator': 4, 'homemaker': 2, 'rubbermaid': 1, 'addictionfamily': 1, 'relaxing': 2, 'outdoorsthe': 1, 'dust': 4, 'senselittle': 1, 'pep': 5, 'poolshit': 1, 'fuckchrist': 1, 'singerchelsea': 1, 'aclu': 2, 'upseemingly': 1, 'stuckup': 1, 'sayjazz': 1, 'jennings': 2, 'missionentourage': 1, 'adaptation': 2, 'nuances': 1, 'bookpaul': 1, 'musiceric': 1, 'cantor': 4, 'pressuring': 1, 'positiongod': 1, 'transcendent': 1, 'snowmobilesreport': 1, '45': 5, 'paired': 2, 'pointmore': 1, 'preregistered': 1, 'votefurloughed': 1, 'bison': 1, 'reopenspaying': 1, 'organ': 1, 'enable': 2, 'livesrene': 1, 'zellweger': 2, 'rene': 1, 'typechild': 1, 'popsiclestick': 1, 'crossexsniper': 1, 'statesunited': 1, 'assails': 1, 'neonazisthe': 1, 'doilliterate': 1, 'frustrates': 1, 'ouija': 2, 'playersmissouri': 1, 'mishandled': 1, 'fetal': 2, 'tissueashton': 1, 'kutcher': 1, 'deservethe': 2, 'processors': 1, 'reviewersjimmy': 1, 'globesnorth': 1, 'revamps': 1, 'restarts': 1, 'plantsmartha': 1, 'mastiff': 1, 'ugliest': 2, 'droopy': 1, 'gassy': 1, 'gorgeousjustin': 1, 'prog': 1, 'rockersthe': 1, 'flatirons': 1, 'pointthe': 2, 'belief': 3, 'changereligious': 1, 'practicing': 3, 'platinum': 1, 'rulegovernment': 1, 'valid': 1, 'wellreasoned': 1, 'resignnations': 1, 'moonlit': 1, 'bonfire': 1, 'conjure': 1, 'thompsona': 1, 'livesman': 1, 'interviewgas': 1, 'pump': 2, 'todaythis': 1, 'thrives': 1, 'othershaim': 1, 'andersonadult': 1, 'bookstore': 3, 'enhance': 3, 'cafstudy': 1, '82': 1, 'nathan': 3, 'tractorthousands': 1, 'indonesia': 1, 'againdisgraced': 1, 'crisisdelighting': 1, 'criticismjoy': 1, 'disparaging': 2, 'faithhow': 1, 'abroadan': 1, 'plagiarismdivorced': 1, 'funreport': 1, 'alumni': 3, 'nowseaworld': 1, 'burlesque': 2, 'shownew': 1, 'envelope': 2, 'envelopethe': 1, 'fighters': 1, 'forgotjogger': 1, 'bodybride': 1, 'fiancs': 1, 'asses': 1, 'suncable': 1, 'ace': 1, 'movereport': 2, 'managed': 3, 'penetrate': 1, 'databases': 1, 'arecheap': 1, 'airfare': 1, 'italystrip': 1, 'scar': 1, 'explanationlibrary': 1, 'til': 2, 'hammersmith': 1, 'motrhead': 1, 'registrythe': 1, 'changeclinton': 1, 'uv': 1, 'radiationopen': 1, 'nicethe': 1, 'homepage': 2, 'slidersahmad': 1, 'khan': 2, 'rahami': 1, 'manhattan': 1, 'explosionirans': 1, 'khamenei': 2, 'interference': 2, 'electionfriend': 1, 'friendinspirational': 1, 'yearscoroner': 1, 'throweranne': 1, 'garfieldwomens': 1, 'pillscreating': 1, 'reimagining': 1, 'chewdinner': 1, 'roommatereport': 1, 'thursdayfriend': 1, 'openings': 2, 'mocksworldwide': 1, 'reportterrifying': 1, 'mutation': 2, 'timedepressed': 1, 'locks': 4, 'roomchance': 1, 'rapper': 5, 'pollshero': 1, 'technicality': 1, 'mannra': 1, 'norths': 2, 'colluding': 1, 'powersa': 1, 'humansperus': 1, 'narrow': 2, 'kuczynskiballet': 1, 'hispnico': 1, 'deservegirl': 1, 'storeunfunny': 1, 'togetherit': 1, 'tamponthese': 1, 'bhangra': 1, 'bluesclass': 1, '88': 2, 'thorpe': 1, 'herwho': 1, 'straps': 1, 'smartthat': 1, 'hotanarchy': 1, 'teensfiona': 1, 'sacmom': 1, 'hermione': 2, 'accepta': 1, 'lust': 3, 'controlmean': 1, 'comedythe': 1, 'wsjs': 1, 'polluterscar': 1, 'banged': 2, 'nightgrandma': 1, 'sweaterbutch': 1, 'misconception': 1, 'livessinclair': 1, 'video7': 1, 'septemberfrom': 1, 'employeesfirewood': 1, 'agendathe': 1, 'notdreamers': 1, 'clockpalestinian': 1, 'violenceiraqi': 1, 'swell': 2, 'flee': 4, 'fallujahdont': 1, 'echo': 2, 'anythingbaby': 1, 'mudhistorical': 1, 'housewatch': 1, 'impressions': 2, 'allconservative': 1, 'editorial': 3, 'clintonthis': 1, 'augmented': 1, 'recreate': 2, 'lifedave': 1, 'foundationpierced': 1, 'boringencouraging': 1, 'extremist': 2, '911thanks': 1, 'kickstarter': 1, 'homesdonald': 1, 'columnistpresident': 1, 'unpopular': 3, 'whilechlo': 1, 'sevigny': 1, 'umlautcleanshaven': 1, 'tuxedoed': 1, 'courtroom': 1, 'appearancepearl': 1, 'harbor': 1, 'mannequin': 3, 'seasoned': 1, 'prosnew': 1, 'truenation': 1, 'bulletproof': 1, 'vesttrump': 1, 'disbandstar': 1, 'costner': 2, 'passengersbeyoncs': 1, 'setlist': 1, 'flawlessmnuchin': 1, 'debacle': 1, 'reformstoddler': 1, 'timeoutcar': 1, 'stoplight': 1, 'blasting': 2, 'directionsits': 1, 'mini': 3, 'dawsons': 2, 'reuniontulip': 1, 'popping': 1, 'hotshotgeorgia': 1, 'hurls': 2, '11goodell': 1, 'banners': 1, 'stadiumshow': 1, 'lupita': 2, 'nyongo': 2, 'trevor': 4, 'noahs': 1, 'movieross': 1, 'tourfacebook': 1, 'dayobama': 1, 'splinter': 1, 'groupsamber': 1, '5yearold': 4, 'gay2016': 1, 'yearkanye': 1, 'trump5': 1, 'terrorismrelated': 2, 'crimesnew': 1, 'examining': 2, 'propertiesdianne': 1, 'feinstein': 4, 'eviscerates': 3, 'savage': 4, 'argumentcivil': 1, 'toolcnn': 1, 'renews': 4, 'seasonsrepublican': 1, 'imaginegrateful': 1, 'dayeager': 1, 'lithgow': 1, 'diseasemans': 1, 'aframe': 1, 'realized7': 1, 'effectively': 2, 'motivate': 1, 'othershealth': 1, 'inspector': 2, 'repulsed': 2, 'customerswhat': 1, 'expertsbrown': 1, 'blackhusband': 1, 'experimenting': 2, 'marriagesupreme': 1, 'cockblocks': 1, 'mancalvin': 1, 'handles': 1, 'awarddzhokar': 1, 'campusscreaming': 1, 'schoolgirls': 1, 'bushatecrime': 1, 'stalled': 1, 'prohate': 1, 'lobbycia': 1, 'scheduled': 1, 'hectic': 2, 'weekendnation': 1, 'hatchnew': 1, 'insects': 1, 'skittering': 1, 'goosebumps': 3, 'fanspower': 1, 'prideti': 1, 'offduty': 1, 'womenaide': 1, '911jayzs': 1, '444': 1, 'vulnerablechristian': 1, 'sake': 2, 'godexgirlfriend': 1, 'flashback': 1, 'paralyzed': 2, 'aislewyclef': 1, 'fugeesfebruary': 1, 'accuracy': 1, 'month8': 1, 'marrying': 3, 'againrussia': 1, 'sanctionssun': 1, 'secondstrump': 1, 'vehemently': 2, 'immigrantsa': 1, 'trainwreck': 1, 'refereeing': 1, 'playoffsnew': 1, 'gargamelrescuers': 1, 'packthings': 1, 'togethersuicide': 1, 'upbeatmeghan': 1, 'markles': 2, 'habsburg': 1, 'cousinscustomer': 1, 'receipt10': 1, 'bodas': 1, 'refugeferguson': 1, 'trumpselfdirected': 1, 'turnkey': 1, 'investingbattleship': 1, 'propped': 1, 'departmentfog': 1, 'heightens': 1, 'piano': 4, 'recitalconversations': 1, 'bushnew': 1, 'preys': 1, 'ideasfootball': 1, 'offseason': 1, 'paintmicrosoft': 1, 'flaw3': 1, 'shouldntmens': 1, 'wearhouse': 1, 'clipon': 1, 'trousers': 1, 'pantswoman': 1, 'ediblebeautiful': 1, 'lifetrump': 1, 'confederacy34': 1, '3man': 1, 'antitragedy': 1, 'agendacanoe': 1, 'historysaudiuae': 1, 'tribes': 2, 'qatari': 1, 'emirhow': 1, 'hormonal': 2, 'twotokyo': 1, 'outage': 1, 'warp': 1, 'commutersarea': 1, 'saturn11': 1, 'runningcat': 1, 'internally': 1, 'metharvey': 1, 'despicable': 3, 'bobreuters': 1, 'myanmar': 2, 'crisisminnie': 1, 'optioned': 1, 'harrison': 2, 'fordneil': 1, 'armstrong': 1, 'billionth': 3, 'earthheres': 1, 'bendy': 2, 'classmiddle': 1, 'proceduremark': 1, 'altered': 2, 'electionfacebook': 1, 'fingerprints': 1, 'anewman': 1, 'workweeknewborn': 1, 'bickeringman': 1, 'misogynist': 1, 'lifehuffpollster': 1, 'postelectionno': 1, 'medieval': 1, 'nuns': 1, 'prayer': 4, 'wheelparents': 1, 'celebrated': 1, 'milestoneslouis': 1, 'freezer': 3, 'bagsfrances': 1, 'farright': 5, 'pronazi': 1, 'pastups': 1, 'answering': 2, 'lingeriechrissy': 1, 'criticsembarrassed': 1, 'panicking': 2, 'barnacle': 1, 'datefield': 1, 'biz': 1, 'tradesarea': 1, 'moviesthe': 1, 'insisting': 2, 'heardblack': 1, 'airporttrumps': 1, 'nativist': 1, 'weaken': 1, 'countrylets': 1, 'deeperthere': 1, 'agoman': 1, 'complicated': 4, 'rockthe': 3, 'takeaways': 1, 'negotiatorsthe': 1, 'shifted': 1, 'attitude': 1, 'fearthe': 1, 'bohemian': 1, 'rhapsody': 1, 'hereannoying': 1, 'welladjusted': 1, 'meditating': 1, 'nowfamily': 1, 'epidemicbill': 1, 'mixedracethe': 1, 'bhikkunis': 1, 'monks': 3, 'thailandshake': 1, 'shack': 3, 'boozy': 2, 'waykylie': 1, 'septum': 1, 'shoottrump': 1, 'accusershuffpost': 1, 'aman': 1, 'sethi': 1, 'editorinchiefblissedout': 1, 'hempwearing': 1, 'himolder': 1, 'ravinewatch': 1, 'headquartersdeadeyed': 1, 'nonhumiliating': 1, 'hoursbilly': 1, 'joels': 1, 'diseasesenate': 1, 'jobsserena': 1, 'tennis': 5, 'thiefberkeley': 1, 'lockdown': 2, 'pages': 3, 'benchthe': 1, 'fantastic': 3, 'faroe': 1, 'islandshot': 1, 'showsinside': 1, 'facebooks': 2, 'brainmeet': 1, 'suitors': 3, 'correctly': 3, 'timebush': 1, 'hollerdarius': 1, 'rucker': 1, 'gamecocks': 1, 'centenarians': 1, 'avatartape': 1, 'dispensedwwi': 1, 'liturgy': 1, 'atone': 1, 'warhorrified': 1, 'geologists': 1, 'uncover': 3, 'sprawling': 1, 'gravewoman': 1, 'cycles': 1, 'nonthreatening': 1, 'inflections': 1, 'concernobama': 1, 'immigrationhe': 1, 'mefourthgrade': 1, 'portent': 1, 'gnarled': 1, 'thirdgrade': 1, 'teacherkey': 1, 'divides': 1, 'womenviewer': 1, 'outragedyoung': 1, 'childreniran': 1, 'dealthe': 2, 'democracyhow': 1, 'starheres': 1, 'antivaxxers': 1, 'soundpicking': 1, 'rescheduled': 1, 'thursdayarea': 1, 'itching': 1, 'nativity': 3, 'parkpoll': 1, 'god25': 1, 'effortless': 1, 'wrap': 1, 'summernasa': 1, 'satelliteeli': 1, 'timesfrying': 1, 'misusing': 1, 'oileight': 1, 'redefinition': 1, 'termatheist': 1, 'claymation': 1, 'christa': 1, 'spinoff': 2, 'happeningfind': 1, 'ruddercobwebcovered': 1, 'skeleton': 3, 'gripping': 2, 'termwhale': 1, 'measures': 1, 'elephantsgina': 1, 'tvnations': 1, 'celeb': 3, 'scandalarea': 1, 'boundscreate': 1, 'successseaworlds': 1, 'itfritolay': 1, 'leaned': 1, 'timeanother': 1, 'loteither': 1, 'diddy': 1, 'againclinton': 1, 'woos': 2, 'mercury': 2, 'mustachesubstitute': 1, 'asshole7': 1, 'manufacturinggeorge': 1, 'amal': 3, 'cryingconversion': 1, 'experiencethis': 1, 'stylish': 3, 'suitrichard': 1, 'wolff': 1, 'capitalism': 2, 'explosive': 3, 'societyflorist': 1, 'relationshipman': 1, 'resolves': 2, 'tabs': 3, 'onesman': 2, 'mealsjogger': 1, 'aroundfriends': 1, 'cocreator': 1, 'reboot11': 1, 'selfdiscoveryelena': 1, 'kagan': 1, 'takesscientists': 1, 'theorize': 3, 'firebased': 1, 'lifeturkeys': 1, 'wonderlandinternational': 1, 'atom': 3, 'onekim': 1, 'distracting': 1, 'citizenrygod': 1, 'steelemodern': 1, 'traumacalifornia': 1, 'plenty': 5, 'wasteoscar': 1, 'cricket': 2, 'murdersweet': 1, 'specialists7': 1, 'havana': 1, 'attractions': 1, 'seefour': 1, 'trailblazers': 1, 'empowering': 2, 'colorwatch': 1, 'unionpope': 1, 'catastrophic': 1, 'derailed': 1, 'talkswhy': 1, 'africathe': 1, 'senates': 1, 'stealth': 1, 'seniors': 4, 'carejose': 1, 'fernandez': 1, 'crashmore': 1, 'lean': 2, 'expansionfrom': 1, '12fantastic': 1, 'dumbledore': 1, 'beastsnation': 1, 'suddenus': 1, 'nailbiter': 1, 'canadabarack': 1, 'cynicalbest': 1, 'colored': 3, 'reasonwhy': 1, 'mustard': 2, 'museumjoe': 1, 'anythingwhy': 1, 'brownbackmost': 1, 'emergencynation': 1, 'harper': 2, 'ensuring': 1, 'relevantman': 1, 'allterrain': 2, 'dad11': 1, 'nothingspecial': 1, 'pullout': 1, 'momsadorable': 1, 'cubs': 2, 'fearsome': 1, 'yearblack': 1, 'representativestrumps': 1, 'hhs': 2, 'sweetheart': 1, 'biotech': 1, 'firmoklahoma': 1, 'penitentiary': 1, 'inchamber': 1, 'injectionswest': 1, 'flooding': 8, 'submerges': 1, 'townsparents': 1, 'designate': 1, 'footballhes': 1, 'worldmontreals': 1, 'osm': 1, 'concludes': 4, 'couchetard': 1, 'vire': 1, 'classique': 1, 'attendance10': 1, 'endofschool': 1, 'madnesstomato': 1, 'genetically': 1, 'expensivepaul': 1, 'lapierre': 3, 'postingtheres': 1, 'wax': 1, 'likenesscharleston': 1, 'americafiona': 1, 'makeovereccentric': 1, 'colonel': 1, 'blazerthankful': 1, 'discourse': 2, 'reckoningtrump': 1, 'abusedad': 1, 'websitedepartment': 1, 'derelict': 1, 'demolitionweather': 1, 'breitbart': 3, 'storywhat': 1, 'nexplanon': 1, 'implant': 1, 'armthese': 1, 'recite': 2, 'drakestudy': 1, 'measure': 3, 'scalianess': 1, 'pickspanama': 1, 'authorities': 7, 'immunitypoint': 1, 'friendsenior': 1, 'yearcollege': 1, 'beforevatican': 1, 'bansomeone': 1, 'cheeseburgerstuffed': 1, 'glazed': 1, 'donutman': 1, '130000': 1, 'budgethomosexual': 1, 'jerseyspring': 1, 'musthaves': 2, 'curvy': 3, 'girlsoscar': 1, 'meyer': 1, 'wiener': 1, 'scootergrandma': 1, 'knitting': 1, 'laddertrash': 1, 'windowmelanias': 1, 'recuperateswolf': 1, 'resultsmichele': 1, 'soondouble': 1, 'amputee': 1, 'anythingnew': 1, 'contraception': 2, 'peers': 3, 'selecting': 1, 'methodqueen': 1, 'bun': 1, 'rollsparody': 1, 'crotchhitting': 1, 'greenlitreport': 1, 'thatwealthy': 1, 'lastditch': 3, 'credibilityempty': 1, 'wildwhat': 1, 'deeper': 3, 'levelprevent': 1, 'planningreport': 2, 'standcraig': 1, 'sager': 2, 'agame': 1, 'facetime': 1, 'patientmodel': 1, 'shottour': 1, 'oneonone': 1, 'mckinleys': 1, 'assassinin': 1, 'mbta': 1, 'issueobama': 1, 'invading': 1, 'nowthe': 2, 'leftovers': 4, 'deserveenraged': 1, 'tvwendy': 1, 'whelans': 1, 'nycb': 1, 'rain': 7, 'pas': 1, 'deuxsandwich': 1, 'flattened': 1, 'backpackcarrie': 1, 'fishers': 1, '1992': 1, 'humorwhats': 1, 'decisionlebanons': 1, 'expm': 1, 'saudisanimals': 1, 'selvesis': 1, 'religionfcc': 1, 'downman': 1, '7day': 1, 'vacation18': 1, 'sandwichesjohn': 1, 'lennons': 1, 'trumpashes': 1, 'deceased': 5, 'rubbed': 1, 'hallowed': 1, 'traditionsyrian': 1, 'meningitis': 1, 'townwhen': 1, 'powernew': 1, 'excedrin': 1, 'spotacademy': 1, 'studios': 4, 'hollywoodmeghan': 1, 'jeweler': 1, 'kindnostalgic': 1, 'releasewhy': 1, 'womenhow': 1, 'helpwarm': 1, 'syrupy': 1, 'veins': 2, 'mattressreport': 1, 'happenpolice': 1, 'dashcam': 1, 'cheesenations': 1, 'winovomit': 1, 'productionalan': 1, 'keyes': 1, 'campaigningthe': 1, 'photoschild': 1, 'baffled': 3, 'stationary': 1, 'nonviolent': 4, 'imagesshaun': 1, 'exchangedroughtravaged': 1, 'alternatesideofstreet': 1, 'firefightinglast': 1, 'ragtag': 1, 'assembly': 2, 'friendsnew': 1, 'chilly': 2, 'galaxyus': 1, 'attackchinese': 1, 'japan': 4, 'meetinggentrification': 1, 'mockumentary': 1, 'prowling': 1, 'pounce': 1, 'boardingfiat': 1, 'chrysler': 1, '105': 1, 'recallsus': 1, 'bathroomi': 1, 'purged': 1, 'guiltyheres': 1, 'upfbi': 1, 'manhunt': 2, 'managerlogo': 1, 'masturbating': 6, 'spiceshooting': 1, 'lawsthe': 1, 'ali': 5, '74progressive': 1, 'warrens': 2, 'bookbar': 1, '4beer': 1, 'musicwho': 1, 'glee': 1, 'starslast': 1, 'cleanerlatest': 1, 'shortkissinger': 1, 'instructs': 2, 'finer': 2, 'clandestine': 2, 'bombingjoan': 1, 'successfularchitect': 1, 'corbusier': 1, 'mallgreenspan': 1, 'unannounced': 1, 'smallclub': 1, 'showsus': 1, 'spendingserena': 1, 'perfected': 1, 'gap911': 1, 'borrowed': 2, 'timeobama': 1, 'guantnamo': 1, 'presidentmexico': 1, 'guzman': 1, 'evades': 3, 'captureaustralian': 1, 'g20': 4, 'viralunpopular': 1, 'committing': 1, 'racially': 2, 'motivated': 2, 'supportexhausted': 1, 'lawsuitsgod': 1, 'dinosaursas': 1, 'housechances': 1, 'todaya': 1, 'harbors': 1, 'filmsixth': 1, 'pony': 2, 'trilogycan': 1, 'friar': 1, '21stcentury': 2, 'tradean': 1, 'futuresfcc': 1, 'occasion': 2, 'vaginal': 2, 'penetration': 1, 'sitcomactivists': 1, 'poorunsold': 1, 'africaliving': 1, 'ducharmebadpicturemonday': 1, 'nowkitten': 1, 'daysong': 1, 'frank': 6, 'storyteamwork': 1, 'karenproof': 1, 'piss': 3, 'palindog': 1, 'chained': 1, 'freeif': 1, 'finland': 2, 'emoji': 3, 'youpalestinians': 1, 'justicerussia': 1, 'actj': 1, 'crews': 3, 'jenna': 2, 'lyons': 1, 'workdeparting': 1, 'shoos': 1, 'housebreaking': 1, 'mancolin': 1, 'kaepernicks': 1, 'himtrump': 2, 'conceding': 1, 'electionprogressive': 1, '78': 3, 'earningtrump': 1, 'ceaseanddesist': 1, 'bannonlapd': 1, 'silencedaylight': 1, 'surpluslab': 1, 'cancerpro': 1, 'wrestlers': 1, 'biggernew': 1, 'nudetaliban': 1, 'afghanistanqueen': 1, 'victorias': 1, 'secretlifting': 1, 'leafradio': 1, 'hewitt': 1, 'inclined': 1, 'outeveryone': 1, 'sears': 2, 'childyou': 1, 'showerparents': 1, 'sonmatt': 1, 'bomer': 3, 'zachary': 1, 'quinto': 2, 'broadwaylawsuit': 1, 'bracelets': 1, 'themselvesthe': 2, 'carrybernie': 1, 'bowery': 1, 'midnightvladimir': 1, 'russiaarea': 1, '93yearold': 2, 'deathafterlife': 1, 'experiencenew': 1, 'readinganthony': 1, 'kennedys': 1, 'disclosure': 1, 'salve': 1, 'workingexcited': 1, 'rites': 1, 'casedump': 1, 'trumpers': 1, 'wina': 1, 'baked': 2, 'goods5': 1, 'divorcenancy': 1, 'onreport': 1, '4way': 1, 'frisbee': 2, 'circlenations': 1, 'knopfler': 1, 'meetandgreetwoman': 1, 'expressed': 1, 'interpreted': 1, 'angerzachary': 1, 'steamedtemple': 1, 'allegationslocal': 1, 'runofthemill': 1, 'imaginationhannibal': 1, 'buress': 2, 'routinemissing': 1, 'scout': 8, 'publicity': 2, 'badgeshirtless': 1, 'tonga': 1, 'pita': 2, 'taufatofua': 1, 'againant': 1, 'colony': 1, 'halt': 3, 'workerliberal': 1, 'planman': 1, 'beesnewly': 1, 'gawker': 2, 'presshis': 1, 'mohamed': 1, 'kidcarhartt': 1, 'rugged': 1, 'thongcat': 1, 'bolted': 1, 'roomted': 1, 'danson': 1, 'interviewamericas': 1, 'scary': 3, 'politicaleach': 1, 'mastercard': 1, 'evokes': 2, 'memoryandrew': 1, 'ricks': 1, 'deadtrump': 1, 'ponders': 1, 'negativeafghanistan': 1, 'journalistssome': 1, 'abusing': 4, 'peoplecopies': 1, 'litter': 1, 'siteman': 1, 'personalized': 2, 'newgirlfriend': 1, 'cdair': 1, 'controller': 1, 'goingwho': 1, 'presidentstruggling': 1, 'feefran': 1, 'cinched': 1, 'notchalec': 1, 'thoughtmongol': 1, 'sack': 2, 'usan': 1, 'seasondivorce': 1, 'doorjohn': 1, 'stossel': 1, 'newsnation': 1, 'satisfied': 3, 'selena': 3, 'sexualized': 1, 'playthingwhy': 1, 'borrowers': 1, 'otherschristmas': 1, 'diagnosisesa': 1, 'lander': 3, 'landingamazing': 1, 'loch': 1, 'nesscity': 1, 'maoist': 1, 'maoistmom': 1, 'christmastaylor': 1, 'culturerunning': 1, 'mainly': 2, 'programmingsenate': 1, 'homos': 1, 'billsinging': 1, 'bluesyou': 1, 'notus': 1, '2018the': 1, 'momentwatch': 1, 'abby': 2, 'wambach': 1, 'gamewednesdays': 1, 'chinathis': 1, 'beyonce': 3, 'readinghousefly': 1, 'virginity': 1, 'rotting': 1, 'beefvan': 1, 'salesuniversal': 1, 'dominationits': 1, 'einsteingod': 1, 'pouch': 2, 'crystals': 1, 'powers4': 1, 'mondaycash': 1, 'consumersunder': 1, 'tilted': 1, 'favorus': 1, 'menfriends': 1, 'encountered': 1, 'yeardirections': 1, 'eds': 1, 'houseoutgoing': 1, 'tommy': 2, 'briefcase': 1, 'vaccinediscarded': 1, 'tragicomic': 1, 'tableaujoe': 1, 'counterparts': 2, 'tweetscountry': 1, 'guyhundreds': 1, 'hbcu': 3, 'votelarva': 1, 'leavesbrussels': 1, 'attacksstarbucks': 1, 'cupa': 1, 'signguy': 1, 'electionphoto': 1, 'devotion': 1, 'likeukraine': 1, 'shaky': 1, 'holdingaxl': 1, 'range': 4, 'chartnew': 1, 'ronco': 1, 'exposer': 1, 'overnightthursdays': 1, 'shotsthe': 1, 'postsecret': 1, 'lurks': 1, 'man21yearold': 1, 'hissy': 1, 'fitchris': 1, 'stapleton': 1, 'songhillary': 1, 'speechhomebrewing': 1, 'longoverdue': 1, 'conclusionkristen': 1, 'dax': 1, 'shepard': 1, 'globesthis': 1, 'supermarkets': 1, 'sellowner': 1, 'pharoah': 2, 'holidaysisrael': 1, 'talksreport': 1, 'insidemuslims': 1, 'unitynation': 1, 'schedules': 2, 'recurring': 1, 'monthly': 5, 'streamline': 1, 'processnewtown': 1, 'realitythe': 1, 'antonin': 4, 'consistently': 1, 'rightwatch': 1, 'ageistdear': 1, 'mcneil': 1, 'atrans': 1, 'womansingle': 1, 'engulfed': 2, '50page': 1, 'printout5': 1, 'deferredayesha': 1, 'networkone': 1, 'snubbed': 1, 'debateimmunedeficient': 1, 'realtor': 1, 'bubblefast': 1, 'worldfrenzied': 1, 'piecesgisele': 1, 'bndchen': 1, 'makeupfree': 1, 'italia': 1, 'coverarea': 1, 'inadequatemerv': 1, 'supply': 1, 'jiffy': 2, 'charityyou': 1, 'itliving': 1, 'morality': 1, 'greenbuild': 1, '2014trumps': 1, 'trojan': 2, 'cutthe': 1, '1960sstudy': 1, 'secondsthe': 1, 'stigmas': 1, 'facecardboard': 1, 'snowflake': 1, 'maskingtaped': 1, 'doorthe': 1, 'successnew': 1, 'mcadoo': 1, 'reesedonald': 1, 'congratulate': 3, 'blunder': 1, 'memeincome': 1, 'electionspanicked': 1, 'apgar': 1, 'testwhat': 1, 'marriagechick': 1, 'corea': 1, 'communistsjustice': 1, 'reconfirmation': 1, 'hearingall': 1, 'beenstartup': 1, 'benefitsbodybuilders': 1, 'skinthe': 1, 'bureau': 6, 'promoting': 4, 'growthstripper': 1, 'adequate': 1, 'joba': 2, 'timesthe': 1, 'kidshighway': 1, 'hacked': 3, 'trumpfirsttime': 1, 'nick': 5, 'barborakkim': 1, 'licenses': 2, 'firednew': 1, 'homeowner': 2, 'fascinated': 3, 'moldinglate': 1, 'airs': 3, 'uns': 1, 'mandonald': 1, 'silencing': 1, 'exploited': 1, 'workerscolorful': 1, 'multicultural': 1, 'mural': 2, 'talentthese': 1, '2017how': 1, 'jobsformer': 1, 'tvjake': 1, 'breakgray': 1, 'sighted': 1, 'yearsgrowing': 1, 'scoutings': 1, 'closetits': 1, 'visitlove': 1, 'increasing': 4, 'marginswhos': 1, 'lollapaloozaalan': 1, 'colmes': 2, 'nephewtrump': 1, 'harold': 3, 'reportbombs': 1, 'churches': 3, 'cruces': 1, 'mexicoalcoholics': 1, 'soberthe': 1, '61': 1, 'pledgefda': 1, 'dosage': 1, 'acetaminophen': 2, 'shitwhat': 1, 'oversenate': 1, 'measures2012': 1, 'baghdadandrew': 1, 'mccabe': 1, 'pension': 3, 'requirementstwo': 1, 'slayingdeer': 1, 'fanreport': 1, 'wastes': 5, '77': 1, 'winwoods': 1, 'thingsfridays': 1, 'roast': 5, 'fastslovakias': 1, 'fico': 1, 'journalistgay': 1, 'certificatesbob': 1, 'hawk': 1, 'lotthe': 2, 'thembin': 1, '60th': 2, 'birthdaynations': 1, 'shortened': 1, '6day': 1, 'yearjohnny': 1, 'depp': 2, 'whimsically': 1, 'teetertottering': 1, 'plank': 1, 'swaying': 1, 'beamwhy': 1, 'brandon': 3, 'yearbodybuilder': 1, 'armi': 1, 'summertime': 1, 'momstudy': 1, 'crave': 3, 'weaktheyre': 1, 'smallholocaust': 1, 'believers': 1, 'skeptics': 1, 'alikenader': 1, 'kerryque': 1, 'vivan': 1, 'amos': 1, 'casaemployees': 1, 'moneynation': 1, 'savoring': 1, 'latefebruary': 1, 'earlymarch': 1, 'daystalking': 1, 'wx': 1, 'geeks': 1, 'sunday': 3, 'showmy': 1, '55': 3, 'monthspolice': 1, 'subduing': 1, 'jurythe': 1, 'vanishinghillary': 1, 'brandgirls': 1, 'slender': 1, 'adultsrevolutionary': 1, 'hammersnetflix': 1, 'problemsnew': 1, 'strugglesht': 1, 'pooping': 1, 'officegeorge': 1, 'monthwoman': 1, 'rack': 2, 'eyesarea': 1, 'reasonplussize': 1, 'tutus': 1, 'sequins': 1, 'outhometown': 1, 'enoughchevron': 1, 'hybridpowered': 1, 'platformsgirls': 1, 'jenni': 1, 'konner': 1, 'workswatch': 1, 'carneys': 1, 'reportersterrible': 1, 'awardsrubio': 1, 'trumpswsj': 1, 'reneged': 1, 'issueis': 1, 'liceserena': 1, 'adthe': 2, 'knowim': 1, 'apathy': 1, 'tooalgebra': 1, 'notebook': 2, 'brunt': 1, 'lyricsobamas': 1, 'imaginedsomali': 1, 'pirates': 3, 'sailors': 2, 'captivityincreasing': 1, 'surrogates': 1, 'babyreport': 1, 'underpaid': 1, 'laborers': 2, 'fifa': 1, 'defensechipper': 1, 'shotwhen': 1, 'sienna': 1, 'wives': 4, 'homeqa': 1, 'writerdirector': 1, 'caird': 1, 'halfwaywhat': 1, 'tablewhite': 1, 'plundered': 1, 'turksthis': 1, 'seeis': 1, 'bandaid': 2, 'pcossome': 1, 'beforewoman': 1, 'pee': 4, 'sample': 3, '7eleven': 1, 'microwavethe': 1, 'riversmexican': 1, '19inch': 1, 'lifeteslas': 1, 'robotsnake': 1, 'nightmarespatiently': 1, 'ofmajority': 1, 'basescalias': 1, 'exposedmob': 1, 'bumrush': 1, 'botanical': 1, 'gardenwhy': 1, 'fatextension': 1, 'cord': 4, 'spotlight': 4, 'rallya': 1, 'mumbaipolice': 1, 'oneswhy': 1, 'coursewhere': 1, 'mastectomy': 1, 'brasseth': 1, 'dubs': 1, 'woods': 5, 'golfingreport': 1, 'dessert': 2, 'doesmichelle': 1, 'ownthe': 1, 'lenderspennsylvanias': 1, 'menwould': 1, 'max': 2, 'rockatansky': 1, 'nux': 1, 'fury': 2, 'roadhalliburton': 1, 'cheneylivid': 1, 'cassidy': 1, 'nightfrances': 1, 'electionsterrorist': 1, 'pursejohnny': 1, 'rockets': 2, 'wormhole': 1, '1950srodent': 1, 'bannons': 5, 'meetingfda': 1, 'usedroommate': 1, 'skulking': 1, 'victorian': 1, 'childimproving': 1, 'workplaceworld': 1, 'explosions': 1, 'deathdefensive': 1, 'adthese': 1, 'portraits': 4, 'gallerymans': 1, 'undoing': 1, 'handiwork': 1, 'selfcheckout': 1, 'machinebosss': 1, 'cautious': 2, 'latelygorgeous': 1, '79syria': 1, 'unravellingwoman': 1, 'wantsheroin': 1, 'poppy': 1, 'farmeronly': 1, 'cliff': 1, 'clavinthe': 1, 'pollutionthis': 1, 'thriving': 2, 'southcentral': 1, 'firedsecond': 1, 'fictitious': 1, 'truetig': 1, 'sickened': 2, 'indianai90': 1, 'crosscountry': 1, 'manatlanta': 1, 'standoff': 4, 'himselfcloseted': 1, 'rendezvous': 1, 'repealedhellmanns': 1, 'unbefitting': 1, 'mayonnaise': 1, 'magnatedisillusioned': 1, 'hacker': 2, 'electionthe': 1, 'idlibadele': 1, 'worships': 1, 'altar': 2, 'usdont': 1, 'picky': 1, 'eatersmithsonian': 1, 'institution': 3, 'politely': 2, 'sofa': 1, 'chargetrumps': 1, 'promisedman': 1, 'timewomen': 1, 'abysmally': 1, 'underrepresented': 2, 'leadershipglutenfree': 1, 'quinoa': 1, 'mushroomsthese': 1, 'walkout': 3, 'whytoo': 1, 'bearseia': 1, 'carving': 1, 'factoriesjeff': 1, 'opposes': 1, 'bipartisan': 2, 'billenraged': 1, 'stageplay': 1, 'unconvincingly': 1, 'restrained': 2, 'actorsstudy': 1, 'safetyrepublican': 1, 'montana': 4, 'creationist': 2, 'stancesuperstitious': 1, 'beliefs': 4, 'pollsreport': 1, 'everjayz': 1, 'namespopsicle': 1, 'flavorwhat': 1, 'loudcnn': 1, 'schoolhouse': 2, 'nightmaredeath': 1, 'schoolchildren': 2, 'sensegroup': 1, 'sharksfrom': 1, 'tacos': 1, 'pad': 3, 'recipeshow': 1, 'loansfighting': 1, 'acid': 3, 'victimsclinton': 1, 'weigh': 4, 'says5': 1, 'faulty': 1, 'dysfunction': 1, 'carehow': 1, 'nikki': 2, 'homebuilding': 1, 'sprinklersterrified': 1, 'bananas': 3, 'daysexercising': 1, 'developingfunloving': 1, 'laidback': 1, 'nerdy': 1, 'servicediamond': 1, 'jubilee': 1, 'fuckfrom': 1, 'goofy': 2, 'ridge': 2, 'illinoisparkland': 1, 'awaywith': 1, 'responsibilitydeflategate': 1, 'businessantidrug': 1, 'marijuanayearbook': 1, 'shotirans': 1, 'overlookedconservatives': 1, 'catholics': 2, 'houseus': 1, 'woooolocal': 1, 'visitor': 3, 'websitetrump': 2, 'dairy': 3, 'theirsmark': 1, 'blistering': 3, 'wordsamericans': 1, 'showsfinding': 1, '911half': 1, 'unfairlyeveryone': 1, 'singular': 1, 'notgarden': 1, 'movienation': 1, 'vitaminwater': 1, 'itfirstgrader': 1, 'drilldemocrats': 1, 'victoryhey': 1, '20somethings': 1, 'breathefriend': 1, 'avoidedstate': 1, 'interferenceinvestigation': 1, 'nightstand': 1, 'morningairline': 1, 'reclineturning': 1, 'medicarewhen': 1, 'ecuador': 1, 'courtkim': 1, 'shootingslife': 1, 'infinitely': 3, 'flexiblewhen': 1, 'worthwhile': 2, 'feesembarrassed': 1, 'fleece': 1, 'jackets': 2, 'scattergories': 1, 'gameonion': 1, 'hivdont': 1, 'charter': 4, 'yetchartering': 1, 'usdallas': 1, 'hospitalmodel': 1, 'railroading': 1, 'mistressongoing': 1, 'chicanery': 1, 'memorialolivia': 1, 'munn': 1, 'aaron': 2, 'rodgers': 2, 'nerdsjohn': 1, 'craig': 2, 'smugbastard': 1, 'summitnew': 1, 'plugin': 1, 'prescriptions': 3, 'upprog': 1, 'noir': 1, 'levin': 1, 'cactus': 1, 'carmine': 1, 'appice': 1, 'mccarty': 1, 'shimabukurochampagne': 1, 'secondplace': 1, 'beveragekimye': 1, 'marriedkindergarteners': 1, 'passionate': 1, 'incomprehensiblereport': 1, 'stuf': 1, 'oreos': 2, 'stufhow': 1, 'aluminum': 2, 'foil': 1, 'elbow': 2, 'greasesweating': 1, 'ownspain': 1, 'hourdarrell': 1, 'retiring': 1, 'congresskristen': 1, 'wiig': 2, 'goddess': 2, 'sketch7': 1, 'upgradeae': 1, 'biography': 2, 'elleninspired': 1, 'grabvillanova': 1, 'piccolo': 1, 'coaster': 4, 'madnesshalf': 1, 'racecreators': 1, 'rodriguezs': 2, 'transphobiastudy': 1, 'uttering': 1, 'divorcemister': 1, 'rogers': 2, 'gerrymandered': 2, 'serve': 4, 'fridays': 4, 'makebelieve': 1, 'agendajj': 1, 'setliterally': 1, 'chafee': 1, 'pollsecretary': 1, 'creekkeystone': 1, 'xl': 2, 'projectisraels': 1, 'disregard': 1, 'aligning': 1, 'nicelyrediscovering': 1, 'destroyeda': 1, 'pencesecond': 1, 'staten': 2, 'walkmitch': 1, 'judgesman': 1, 'attends': 2, 'nursery': 3, 'reunionwhy': 1, 'pakistanreport': 1, 'granite': 2, 'engraved': 1, 'victimsuniversities': 1, 'microaggressionswere': 1, 'morereal': 1, 'sired': 1, '3time': 2, 'remax': 1, 'championfeinstein': 1, 'isistamra': 1, 'housewives': 1, 'countyno': 1, 'imagefilipino': 1, 'effigyaftershock': 1, 'victimsbetsy': 1, 'richly': 1, 'deserved': 3, 'gottired': 1, 'telekinetic': 1, 'schoolshotels': 1, 'treetop': 1, 'staysreport': 1, 'relying': 1, 'grandparents': 6, 'kidswhere': 1, 'surrounds': 1, 'accountmute': 1, 'awakes': 1, 'vocalize': 1, 'unscripted': 1, 'sentimentdivorced': 1, 'weightwatch': 1, 'countryrecords': 1, 'staged': 2, 'suicideboehner': 1, 'export': 1, 'bandonald': 1, 'averting': 1, 'shutdowncnns': 1, 'ana': 1, 'wolfs': 3, 'snowflakeshow': 1, 'videosunday': 1, 'thatll': 3, 'buttseveral': 1, 'unauthorized': 1, 'headquartersstarting': 1, '2014chicago': 1, 'stationsjian': 1, 'ghomeshi': 1, 'rightfully': 1, 'draggedwhy': 1, 'wardrobepaul': 1, 'gauguin': 1, 'fondation': 1, 'beyeler': 1, 'videomelania': 1, 'procedurethe': 1, 'vaticans': 1, 'stampspast': 1, 'armageddon': 1, 'zippori': 1, 'israels': 1, 'archaeological': 2, '1reform': 1, 'poised': 2, 'resolutiondeadly': 1, 'stampede': 3, 'bangladesh': 2, '23monster': 1, 'truckjwoww': 1, 'knowingly': 2, 'drank': 2, 'pregnantformer': 1, 'vikings': 4, 'allegationsrnc': 1, 'hastert': 3, 'programrefrigerator': 1, 'appliancesalvation': 1, 'areabillionaire': 1, 'allens': 2, 'wrecks': 1, 'cayman': 1, 'reefpope': 1, 'godman': 1, 'smacked': 1, 'daytoughguy': 1, 'parentscollege': 1, 'usearea': 1, '201': 1, 'guyfather': 1, '18hour': 1, 'saypeople': 1, 'pines': 1, 'lookalikeastronomers': 1, 'galaxythese': 1, 'failpreholiday': 1, 'preps': 2, '4gop': 1, 'laterthe': 1, 'korra': 1, 'happytom': 1, 'bradys': 1, 'gala': 6, 'sidelineshuffpollster': 1, 'kansasbaconscented': 1, 'undies': 1, 'panty': 1, 'curedpeer': 1, 'alfox': 1, 'trackfilmschool': 1, 'videostore': 1, 'jobarea': 1, 'vicariously': 1, 'bullyarea': 1, 'spaceynato': 1, 'rocketstrump': 1, 'lawyerelderly': 1, 'suppository103': 1, 'assaulting': 5, 'cnnchurch': 1, 'region': 3, 'dagestan': 1, 'reportneighborhood': 1, 'busybody': 1, 'gunshotsa': 1, 'marshawn': 1, 'facebookla': 1, 'phils': 1, 'pellas': 1, 'mlisande': 1, 'glowing': 2, 'success5': 1, 'mika': 1, 'thinkan': 1, 'communitydwayne': 1, 'melts': 3, 'processwhere': 1, 'answered': 2, 'agoconstruction': 1, 'incidence': 1, 'steel': 4, 'ibeamsme': 1, '35th': 2, 'yearlocal': 1, 'cheesecakeare': 1, 'bejennifer': 1, 'anistons': 2, 'lbdfriday': 1, 'playthe': 1, '1964massive': 1, 'togos': 1, '50year': 1, 'dynastyfather': 1, '10yearold': 3, 'systemblogger': 1, 'kbeauty': 1, 'ching': 1, 'chongs': 1, 'clotheswoman': 1, 'shampoo': 1, 'poetry': 4, 'daybush': 1, 'treaty': 1, 'kyotopot': 1, 'consequence': 3, 'commonamy': 1, 'throwbackhow': 1, 'betterhuffpollster': 1, 'unlikelyusmexico': 1, 'presidentpence': 1, 'onstage': 2, 'helllistening': 1, 'nas': 1, 'complicit': 1, 'misogynoirteen': 1, 'highteen': 1, 'fred': 2, 'durstim': 1, 'iphone6': 1, 'learnedwarren': 1, 'buffetts': 1, 'colorcensus': 1, 'wallal': 1, 'franken': 5, 'actionshot': 1, 'chartsnew': 1, 'violencerubenesque': 1, 'picassoesque': 1, 'facei': 1, 'wallmagical': 1, 'marseille80': 1, 'handmade': 1, 'jewelry5': 1, 'bounty': 2, 'recessionnations': 1, 'huggers': 1, 'herelone': 1, 'restaurantthe': 1, 'sweaters': 2, 'againclay': 1, 'aiken': 1, 'gained': 2, 'bojangles': 1, 'campaignthere': 1, 'thatnew': 1, 'nugent': 1, 'cologne': 1, 'finduntil': 1, 'bloc': 1, 'conservatives': 8, 'controls': 2, 'governmentman': 1, 'precedent': 1, 'hello': 3, 'morningdisembodied': 1, 'elevator': 2, 'josejimmy': 1, 'dehydration': 1, 'canadabill': 1, 'somewhereguests': 1, 'shellshocked': 1, 'weddinghugh': 1, 'livein': 1, 'peacockfeeling': 1, 'minoritymom': 1, 'stripesrudy': 1, 'presspolice': 1, 'addition': 2, 'misconductthe': 1, 'betelgeuse': 1, 'supernovathis': 1, 'aftermathharry': 1, 'nominees3': 1, 'nutrients': 2, 'brainpowerside': 1, 'awesomedisturbing': 1, 'confessions': 1, 'hoarderobama': 1, 'nationsthese': 1, 'identitywoman': 1, 'handing': 3, 'roadeu': 1, 'sakharov': 1, 'yazidi': 1, 'isislouis': 1, 'molesting': 3, 'snlgo': 1, 'danwatch': 1, 'schieffer': 1, 'offpope': 1, 'painquantum': 1, 'hypothesize': 1, 'headed': 5, 'directions': 2, 'simultaneouslyjon': 1, 'neonazisafter': 1, 'seal': 2, 'applicantclinton': 1, 'rideovereager': 1, 'simpleton': 1, 'mostmargarita': 1, 'behold': 1, 'tequiladisney': 1, 'visitorbush': 1, 'preraphaelite': 1, 'contessahorrible': 1, 'ideassnowstorm': 1, 'murders5': 1, 'weightlady': 1, 'grammysvacationing': 1, 'feedmagical': 1, 'clearheaded': 1, 'reactionist': 1, 'moronssuborbital': 1, 'ballisticpropulsion': 1, 'scientistpele': 1, 'hospitalized': 1, 'surgeryruhroh': 1, 'chaseronald': 1, 'presidentmila': 1, 'boyband': 1, 'showgoogle': 1, 'opioid': 9, 'epidemicmiley': 1, 'clarksons': 1, 'yetuniverse': 1, 'suzanne': 1, 'pleshettefeds': 1, 'wrongdoing': 1, 'gms': 1, 'defectred': 1, 'customersschool': 1, 'underpants': 1, 'halfmastarea': 1, 'centertrevor': 1, 'kardashiansshowers': 1, 'cleansingfocusedlyft': 1, 'deceptive': 2, 'staygoogle': 1, 'reportenchanted': 1, 'molests': 1, 'selfcouple': 1, 'affordthe': 1, 'liveaction': 2, 'commercialstrump': 1, 'programsreport': 1, 'carjackers': 1, 'backseatdoes': 1, 'easthow': 1, 'ever12': 1, 'zack': 1, 'sitswatch': 1, 'escalator': 1, 'flyers': 1, 'flyingdonald': 1, 'insult': 2, 'kardashiansnow': 1, 'darn': 1, 'scaryhospital': 1, 'bearaustralian': 1, 'dwindling': 1, 'reservesjob': 1, 'placement': 1, 'alma': 2, 'mater': 1, 'companyclinton': 1, 'aide': 6, 'collapsed': 2, 'carolinamentors': 1, 'existare': 1, 'emotionsspringing': 1, 'maycharitable': 1, 'catchupwhy': 1, 'philosophical': 1, 'runningformer': 1, 'foulest': 1, 'worldhistorians': 1, 'themselvesworking': 1, 'thingus': 1, 'improves': 2, 'transnational': 1, 'stripkerry': 1, 'healthwoman': 1, 'antited': 1, 'cruzpanicked': 1, 'gladwell': 1, 'foretells': 1, 'popularityprom': 1, 'freshly': 2, 'pickupwomen': 1, 'mollie': 1, 'spilman': 1, 'criteoblack': 1, 'inclusion7': 1, 'presidencyman': 1, 'knowgallant': 1, 'fringe': 1, 'reviewersthis': 1, 'mailerrevolutionary': 1, 'immersion': 1, 'lowering': 2, 'gaysthe': 1, 'canned': 2, 'badthe': 1, 'socializing': 1, 'celebsi': 1, '240000th': 1, 'campaignbergdahl': 1, 'desertionwealthy': 1, 'described': 4, 'downtoearth': 1, 'whom': 1, 'lyingfaith': 1, 'strongraindrenched': 1, 'familygirlfriend': 1, 'commercialbill': 1, 'scoggins': 1, 'charityman': 1, 'himvenus': 1, 'planetsmother': 1, 'publicize': 1, 'abductedivanka': 1, 'nordstrom': 2, 'electiontoothbrush': 1, 'oblivion': 1, 'hypnotic': 2, 'videobush': 1, 'attentiontodeficit': 1, 'disordertoddler': 1, 'themhorrible': 1, 'influencescharlize': 1, 'megyn': 2, 'ailesnew': 1, 'unsafegranta': 1, 'derided': 1, 'philistinesthe': 1, 'fridaydiscrimination': 1, 'printondemand': 1, 'toomaryland': 1, 'gonekansas': 1, 'cityarea': 1, 'tipaaron': 1, 'systemic': 1, 'jails': 1, 'prisonsglobal': 1, 'crises': 1, 'fuelscan': 1, 'fullyloaded': 1, 'inyourface': 1, 'diabetic': 1, 'reactionstudy': 1, 'flushing': 1, 'toilets': 2, 'annuallyyou': 1, 'cheezburger': 1, 'toppings': 1, 'withpolice': 1, 'smokerthomas': 1, 'edison': 1, 'ideasthe': 1, 'suspect4': 1, 'nashvilleexclusive': 1, 'elsereport': 1, 'nightrich': 1, 'thrillseeker': 1, 'buscollege': 1, 'chikfila': 1, 'body3': 1, 'trickortreatersstudy': 1, 'mirrortop': 1, 'explosionsjackie': 1, 'chans': 1, 'ancestors': 2, 'blooper': 1, 'reeljakob': 1, 'songwriterobama': 1, 'tennessee': 2, 'housestate': 1, '900': 1, 'nameshedge': 1, 'gambling': 2, 'stateshow': 1, 'jonas': 4, 'baiting': 1, 'offpanicking': 1, 'lady10': 1, 'errordonald': 1, 'billszeke': 1, 'assaultwoman': 1, 'assaulted': 2, 'corroborate': 1, 'storykris': 1, 'onlinephotographer': 1, 'photosmitch': 1, 'billletter': 1, 'recommendation': 1, 'reused': 1, 'internthe': 1, 'handleamerican': 1, 'daydemocrats': 1, 'gorsuchs': 1, 'backing': 3, 'donorsmarketers': 1, 'wheresomething': 1, 'anchormans': 1, 'eyespentagon': 1, '44': 1, 'lickingdonald': 1, 'nixonnina': 1, 'garcia': 3, 'problempope': 1, 'jesuit': 1, 'paraguay': 1, 'indigenous': 2, 'peoplesformer': 1, 'plouffe': 1, 'psychopathhistorians': 1, 'goodfellas': 1, 'fragments': 1, 'worknew': 1, 'viacom': 1, 'workbrave': 1, 'beachgoers': 1, 'sharkadorable': 1, 'hardhitting': 2, 'happythe': 1, 'szep': 1, 'circusbill': 1, 'hypocritical': 1, 'evangelicalsbrave': 1, 'onlinearea': 1, 'raccoonsadorable': 1, 'bei': 2, 'debutmasondixon': 1, 'renamed': 4, 'ihopwaffle': 1, 'linelimbaugh': 1, 'remnant': 1, 'administrationred': 1, 'jamaican': 1, 'quitshotshot': 1, 'helmet': 5, 'statuscorpses': 1, 'ranger': 4, 'blvd': 1, 'othersvariety': 1, 'hip': 4, 'neighborhoodcaitlyn': 1, 'myselfthis': 1, 'graders': 2, 'lifeeverything': 1, 'skinhow': 1, 'daytv': 1, 'olivia': 2, 'metzger': 1, 'parting': 1, 'agencypoll': 1, 'cantmalala': 1, 'laureatesnew': 1, 'campaignsport': 1, 'aretebaseballnext': 1, 'aerosmithsome': 1, 'womenneighbors': 1, 'killerwhy': 1, 'themselvesreport': 1, 'kimora': 2, 'pregnantmit': 1, 'better6': 1, 'rentboys': 1, 'rentboy': 1, 'hustler': 2, 'economylocal': 1, 'tcby': 1, 'changestrump': 1, 'optioncaught': 1, 'birdnaps': 1, 'peacockreview': 1, 'teethmarks': 1, 'eileen': 2, 'battersbythis': 1, 'catfight': 1, 'pauses': 4, 'heche': 1, 'ohs': 1, 'rivalry': 1, 'revealaretha': 1, 'fudgenaval': 1, 'chakra': 1, 'snyderfederal': 1, 'someones': 2, 'offenseswealthy': 1, 'donations': 2, 'choicesnational': 1, 'submissionspersonal': 1, 'timeman': 1, 'bricks': 1, 'alivelos': 1, 'lapd': 2, 'masturbators': 1, 'democracysnl': 1, 'soulful': 3, 'endofepisode': 1, 'musicpostmodern': 1, 'foundart': 1, 'objectstudy': 1, 'figuring': 4, 'startsreport': 1, 'grossass': 1, 'gourd': 1, 'bumpy': 1, 'shitflorida': 1, 'smokes': 1, 'saytrumps': 1, 'reportryan': 1, 'deadpool': 2, 'wordsurvival': 1, 'youneighbors': 1, 'actuallywhole': 1, 'jacking': 1, 'fluwashington': 1, 'quitoptimist': 1, 'shitthe': 2, 'backpacker': 1, 'instagrams': 1, 'timegreat': 1, 'debtthe': 1, 'farbp': 1, 'billionget': 1, 'gigi': 2, 'gorgeousthe': 1, 'vettingryan': 1, 'gosling': 3, 'fullbody': 1, 'camouflagenew': 1, 'incinerates': 1, 'allergiesnumber': 1, 'fourthat': 1, 'chinesetop': 1, 'cuteann': 1, 'rescheduling': 1, 'berkeleydem': 1, 'probethere': 1, 'either': 6, 'syriasweating': 1, 'pharmaceutical': 1, 'anytime': 1, 'wantstrumps': 1, 'fumingman': 1, 'jk': 1, 'rowlings': 2, 'severus': 1, 'anywaychoreographer': 1, 'dendy': 1, 'labyrinthheres': 1, 'seychelles': 1, 'meetinglocal': 1, 'lovemom': 1, 'bang': 3, 'theorythe': 1, 'whereas': 1, 'recalibrateselizabeth': 1, 'honky': 1, 'tonkin': 1, 'cliptaking': 1, 'fraught': 1, 'negativesfeds': 1, 'hydropower': 1, 'capacity': 3, 'doubled': 4, 'ussole': 1, 'flavormore': 1, 'adults': 3, 'hpvthese': 1, 'homedanny': 1, 'tanner': 1, 'guydonald': 1, 'cryptic': 2, 'deletioncreepy': 1, 'insight': 1, 'economyoffice': 1, 'teambuilding': 2, 'exercisejoan': 1, 'cautionary': 1, 'talesenior': 1, 'diminished': 1, 'bawdylimerick': 1, 'recallarchitects': 1, '2026': 1, 'lunchpope': 2, 'benedict': 5, 'reasonthe': 1, 'mythsa': 1, 'beginners': 1, 'spite': 1, 'griefsomeone': 1, 'nonsigning': 1, 'fantasticheres': 1, 'obamacareartificial': 1, 'microsoft': 3, 'jeanphilippe': 1, 'courtois': 1, 'saysseeing': 1, 'othergay': 1, 'ostracizedparents': 1, 'dynamitesamesex': 1, 'complicationstrump': 1, 'erratic': 1, 'impulseslife': 1, 'bombingstransmasculine': 1, 'mediaofficials': 1, 'filters': 3, 'handlechrissy': 1, 'thanksgrandmas': 1, 'againopen': 1, 'casket': 2, 'vibe': 1, 'funeralrubio': 1, 'christiekerry': 1, 'kerryprimary': 1, 'sexterry': 1, 'mcauliffe': 1, 'commentman': 1, 'dogrotesque': 1, 'misshapen': 1, 'raisins': 1, 'mixfertility': 1, 'chickworlds': 1, 'yoyo': 1, 'youjoe': 1, 'revives': 2, 'birther': 2, 'conspiracytrump': 1, 'briefings': 2, 'spicerdaily': 1, 'oceanthe': 1, 'mesmerizing': 2, 'schlegel': 1, 'hyatt': 1, 'viennathis': 1, 'sciencewhite': 1, 'leadernews': 1, 'herd': 1, 'stampedesee': 1, 'uniformlucky': 1, 'comamel': 1, 'gibson': 2, 'payback': 1, 'creditwhy': 1, 'pub': 1, 'marys': 1, 'university2018': 1, 'cancelled': 3, 'inclement': 1, 'weatherchristina': 1, 'ricci': 1, 'pregnantsaudi': 1, 'shiite': 1, 'cleric': 2, 'protestsfbi': 1, 'heavily': 2, 'redacted': 1, 'coworkerslearning': 1, 'ulcerative': 1, 'colitisman': 1, 'formanufacturer': 1, 'manufactures': 1, 'wifekatie': 1, 'couric': 3, 'flirts': 1, 'cardinal': 2, 'airexclusive': 1, 'unleash': 3, 'showrecently': 1, 'mugged': 1, 'suddentesting': 1, 'enhanced': 1, 'classroomtracy': 1, 'forgives': 2, 'himobama': 1, 'violencewill': 1, 'inspection1': 1, 'ut': 1, 'campusthe': 1, 'obamacaretrans': 1, 'trumpland': 1, 'victories': 1, 'hopehouse': 1, 'midtermsman': 1, 'obnoxiouswednesdays': 1, 'italymaking': 1, 'exploringamericans': 1, 'decisionacting': 1, 'whitedonald': 1, 'pollstersterrorism': 1, 'terrorismbeyoncs': 1, 'coachella': 1, 'performancegive': 1, 'purposelast': 1, 'ivorybilled': 1, 'woodpecker': 1, 'weeksso': 1, 'heck': 1, 'probioticmike': 1, 'johanns': 1, 'nowwhy': 1, 'stonewalling': 1, 'stonewallpiggies': 1, 'clouds': 1, 'angelshallmark': 1, '1squareinch': 1, 'anythingno': 1, 'harrys': 1, 'weddinginsufferable': 1, 'utters': 1, 'craft': 4, 'movementnbc': 1, 'allegationsjames': 1, 'scenebar': 1, 'toonations': 1, 'beekeepers': 1, 'wrathnuestra': 1, 'palabra': 1, 'saythis': 1, 'winsactivist': 1, '6000mile': 1, 'videopaul': 1, 'fullyinflating': 1, 'threatas': 1, 'firearmsroyal': 1, 'bornnicaraguan': 1, 'blankettrump': 1, 'knowsdrunk': 1, 'him3': 1, 'busythe': 1, 'cias': 1, '60year': 1, 'corrupted': 1, 'writersnbc': 1, 'csioffbeat': 1, 'squirrel': 3, 'garnering': 1, 'cult': 2, 'followingstephen': 1, 'kimmelhayley': 1, 'rocker': 3, 'chad': 1, 'together5': 1, 'vacationgoogle': 1, 'lawsuitsadapting': 1, 'complexitygovernment': 1, 'cakephilippine': 1, 'mud': 2, 'landslide20': 1, 'purposeobama': 1, 'kenya': 2, 'victorycollege': 1, 'lanyardwearing': 1, 'kindbillboard': 1, 'sting': 2, 'foxwoods': 1, 'casinochina': 1, 'vulnerabilities': 2, 'systemssupreme': 1, 'issue6yearold': 1, 'shits': 3, 'turkeyradish': 1, 'herebee': 1, 'nutsfemale': 1, 'placeloud': 1, 'squawking': 1, 'crow': 4, 'indefinitelystartling': 1, 'electionhorse': 1, 'accident3': 1, 'trumpdan': 1, 'harmon': 1, 'delaysbeautiful': 1, 'timelapse': 2, 'transformspoton': 1, 'ownersjoe': 1, 'concentration': 2, 'closednorth': 1, 'harvests': 1, 'strengtha': 1, 'upsetmiddle': 1, 'glovers': 1, 'historiandepartment': 1, 'locates': 1, 'goat10': 1, 'shopsorority': 1, 'washlovelorn': 1, 'aches': 1, 'locationplanet': 1, 'subtitles': 1, 'ussias': 1, 'grammatical': 1, 'errortrivial': 1, 'pursuit': 3, 'skillsthousands': 1, 'unemployment': 3, 'strikerapper': 1, 'trackshredding': 1, 'postconstitutional': 1, 'americawhite': 1, 'premiums': 2, 'torrent': 2, 'parotid': 1, 'glandsreince': 1, 'carefulchiquita': 1, 'easygrip': 1, 'bananaputin': 1, 'exdefense': 1, 'usscientists': 1, 'immortality': 1, 'ian': 1, '73most': 1, 'credibleus': 1, 'soapy': 1, 'secretarygeneralderek': 1, 'jeter': 3, 'obamacollege': 1, 'ideal': 2, 'subsidizing': 1, 'studentsghost': 1, 'carl': 2, 'sagan': 1, 'superstition15': 1, 'momsnew': 1, 'bestows': 1, 'tablesmom': 1, 'vegetablesanders': 1, 'troubleslocks': 1, 'silo': 1, 'hairseattles': 1, 'planetal': 1, 'inhofe': 1, 'senateliu': 1, 'xiaobo': 1, 'nowmicrosoft': 1, 'requestsmother': 1, 'stationed': 1, 'basepoet': 1, 'poemhistorical': 1, 'preenactors': 1, 'runkitchenaid': 1, 'pinsevenyearold': 1, 'manhistoric': 1, 'statesman': 1, 'jesuscool': 1, 'glitch': 2, 'bombingnearly': 1, 'myanmarus': 1, 'gushing': 1, 'tweetshigh': 1, 'custodian': 1, 'guidancedrew': 1, 'brees': 1, 'overnightdui': 1, 'memizzou': 1, 'melissa': 2, 'clicknew': 1, 'recognition': 3, 'software': 3, 'factors': 3, 'fullanne': 1, 'pregnancybecome': 1, 'cyborg': 1, 'somethingnew': 1, 'chainstale': 1, 'rulepolice': 1, 'ingrained': 1, 'prejudices': 1, 'namea': 1, 'practiced': 1, 'dodging': 1, 'debateshow': 1, 'organizationnew': 1, 'wantarchangels': 1, 'ultrarealistic': 1, 'breathesheryl': 1, 'unsuccessful': 2, 'beginsbedtime': 1, 'againdeath': 1, 'injectionscarlett': 1, 'johanssons': 1, 'imaginedwhat': 2, 'soldierman': 1, 'awoke': 1, 'coma': 1, 'withinthe': 1, 'later12': 1, 'westwanda': 1, 'sykes': 1, 'disschild': 1, 'sternly': 1, 'todayel': 1, 'leaderwhy': 1, 'dribble': 1, 'kansasretired': 1, 'ecclesiastic': 1, 'researchis': 1, 'documentattorney': 1, 'weinsteins': 2, 'reportarea': 1, 'buck': 2, 'homeamplifydcom': 1, 'peets': 1, 'milkasthmatic': 1, 'asthmaticsaclu': 1, 'critichero': 1, 'honoredart': 1, 'fakedad': 1, 'scarier': 1, 'tempercameron': 1, 'diaz': 3, 'thereloveless': 1, 'fourbedroom': 1, 'optionrehabilitated': 1, 'otter': 1, 'chainpoll': 1, 'inappropriatedad': 1, 'possiblelong': 1, 'roseburg': 1, 'one97': 1, 'neons': 1, 'colorsthis': 1, 'intolerance': 1, 'doingbreaking': 1, 'fogaffable': 1, 'antisemite': 1, 'mediamike': 1, 'conservation': 3, 'lifetrevor': 2, 'somethingava': 1, 'godsadopted': 1, 'doe': 1, 'amended': 1, 'certificatesbill': 1, 'presidentjerry': 1, 'gefloigel': 1, 'surgeryhouse': 1, 'whitesempire': 1, 'trai': 1, 'byers': 1, 'squashes': 1, 'showpizza': 1, 'shiftwell': 1, 'gresham': 2, 'supergroupshe': 1, 'himadvice': 1, 'application': 5, 'writingpope': 1, 'nametrump': 1, 'matters8': 1, 'therapyloyal': 1, 'homecherokee': 1, 'fraction': 1, 'actresss': 1, 'bloodlinewolf': 1, 'realtime': 1, 'beardwhat': 1, 'hurricanethe': 1, 'getaway': 4, 'foraging': 2, 'futurewise': 1, 'oracle': 1, 'barbecue': 2, 'raindropnext': 1, 'watcher': 1, 'saysacademy': 1, 'viewership': 1, 'yearbill': 1, 'impish': 1, 'brit': 1, 'easyarmed': 1, 'guarding': 1, 'recruiting': 1, 'chattanooga': 1, 'shootingelection': 1, 'movewoman': 1, 'earlier': 5, 'thatstudy': 1, 'lasik': 1, 'surgeries': 1, 'incinerating': 1, 'sidejohn': 1, 'sympathizers': 1, 'housea': 1, 'hugman': 1, 'himhelpful': 1, 'recently': 2, 'beforerussian': 1, 'sistersan': 1, 'droughtsikh': 1, 'homesupreme': 1, 'albumman': 1, 'folding': 1, 'oncegrindr': 1, 'onlineemployees': 1, 'multitasking': 1, 'workrand': 1, 'pauls': 2, 'outarea': 1, 'movers': 1, 'friendspentagon': 1, 'hormone': 1, 'servicemembers': 1, 'doubling': 1, 'sexus': 1, 'prisonerspakistani': 1, 'iranwith': 1, 'walkover': 1, '1237trump': 1, 'pardonspharma': 1, 'bro': 1, 'shkreli': 4, 'bushtrump': 1, 'halfstaff': 2, 'bloodsdemocratic': 1, 'partys': 3, 'forwardwill': 1, 'bowlboardroom': 1, 'hokey': 1, 'doronnie': 1, 'wood': 2, 'diagnosisjennifer': 1, 'frankly': 2, 'bradeven': 1, 'paymonster': 1, 'cyclone': 1, 'devastation': 1, 'vanuatuairline': 1, 'youraid': 1, 'bug': 2, 'itgone': 1, 'roleobama': 1, 'appointing': 1, 'effortla': 1, 'rodney': 1, 'laterminnesota': 1, 'protestolympic': 1, 'horribly': 1, 'disfigured': 2, 'urinepeprally': 1, 'skit': 1, 'crossdressing': 1, 'principalcongress': 1, 'seating': 1, 'insurancederegulationbill': 1, 'stampedecongress': 1, 'increasehannibal': 1, 'disorderly': 1, 'intoxication': 1, 'miamination': 1, 'windy': 1, 'daydoctor': 1, 'halfhearted': 1, 'suggestions': 2, 'drugstexas': 1, 'overwhelmingly': 3, 'rightsheres': 1, 'instantly': 1, 'confidencemessage': 1, 'womanmike': 1, 'myers': 1, 'worlda': 1, 'chinaivanka': 1, 'battlewhy': 1, 'rampage': 1, 'isishighschool': 1, 'fightmom': 1, 'cincinnati': 2, 'incidentarea': 1, 'everythingthe': 1, 'extinctionseasonal': 1, 'numb': 1, 'familylocal': 1, 'twittereuropean': 1, 'dodianne': 1, 'bencheddoctor': 1, 'scribbles': 1, '30year': 3, 'painkiller': 1, 'addictiontheresa': 1, 'translation': 1, 'addresshr': 1, 'scrawling': 1, 'czardesperate': 1, 'chives': 2, 'mornin': 1, 'campaignstoned': 1, 'backwardsrunning': 1, 'landrieu': 1, 'clintonkfc': 1, 'stockpiling': 2, '576piece': 1, 'bucketromney': 1, 'maniacactual': 1, 'manone': 1, 'evaluating': 1, 'advisorhow': 1, 'mooncreate': 1, 'nestif': 1, 'hb': 1, 'weakest': 1, 'nation20yearold': 1, 'syndrome': 2, 'youngest': 1, 'townadopting': 1, 'proceed': 1, 'reverentlyim': 1, 'coupleswhat': 1, 'tinderrepublicans': 1, 'wellsunday': 1, 'roundupbush': 1, 'inzoologists': 1, 'ape': 1, 'neurology': 1, 'bananaobsessed': 1, 'humansteenage': 1, 'gunfight': 1, 'isisjay': 1, 'cologneswedes': 1, 'stumped': 2, 'foxto': 1, 'marchheavenly': 1, 'overheating': 1, 'planetseaworld': 1, '5pound': 1, 'burgereating': 1, 'challengetea': 1, 'coloreds': 1, 'fallbeyonce': 1, 'twinsemotionally': 1, 'continuously': 1, 'manipulating': 1, 'stayingchristian': 1, 'prop': 2, 'wowing': 1, 'coastpreet': 1, 'bharara': 1, 'sentencefacebook': 1, 'itdramatic': 1, 'wildfireshouse': 1, 'privatesouth': 1, 'balanced': 1, 'northobama': 1, 'automatic': 2, 'residency': 1, 'cuban': 5, 'migrantspeople': 1, 'frederick': 2, 'douglassdavid': 1, 'blaine': 2, 'starves': 1, 'daysneighbor': 1, '20pound': 1, 'fedexedskip': 1, 'instead5': 1, 'nap': 4, 'anywherethe': 1, 'summoning': 1, 'soulmateamerican': 1, 'paralympic': 1, 'triathlonclassified': 1, 'americadont': 1, 'vietnams': 1, 'leaderknow': 1, 'fark': 1, 'timewoman': 1, 'formsrepublicans': 1, 'repealedmen': 1, 'apartmentcaitlyn': 1, 'mattersdonald': 1, 'mockery': 1, 'normsthe': 1, 'boosts': 3, 'moodflorence': 1, 'jobun': 1, 'dealsean': 1, 'penn': 3, 'tactics': 2, 'defamation': 2, 'fightuncertainty': 1, 'findsbarbaric': 1, 'gouges': 1, 'binder': 2, 'punchblack': 1, 'spotlightmelania': 1, 'shroud': 1, 'shrieking': 3, 'crowsa': 1, 'megadrought': 1, 'itdozens': 1, 'defianceconcert': 1, 'verifying': 1, 'coatcheck': 1, 'pocketted': 1, 'establishes': 1, 'suggestion': 2, 'hotlineman': 1, 'decadeslong': 1, 'alcoholismswans': 1, 'arch': 2, 'necks': 1, 'anymorethe': 1, 'corgi': 1, 'cultureloving': 1, 'heartan': 1, 'optical': 2, 'illusion': 2, 'srvgsvatn': 1, 'trippyparents': 1, 'camp19': 1, 'doggie': 1, 'savethedates': 1, 'ruff': 1, 'dayday': 1, 'mosel': 1, 'riesling': 1, 'divinebush': 1, 'nonmasturbating': 1, 'generaltrump': 1, 'festering': 1, 'shitman': 1, 'bingewatching': 1, 'neighborjeff': 1, 'roasting': 1, 'bieberusing': 1, 'roofs': 1, 'dollarsarea': 1, 'oneshandsfree': 1, 'siri': 1, 'sayseight': 1, 'mustreads': 1, 'fashionistasariana': 1, 'licks': 1, 'proclaiming': 1, 'americawild': 1, 'waterbead': 1, 'stunt': 4, 'endangerment': 1, 'chargebrowns': 1, 'rehabunidentified': 1, 'leaning': 5, 'wall7': 1, 'misconceptions': 1, 'hebrew': 1, 'bibleaclu': 1, 'tomiddleaged': 1, 'snacks': 4, 'mockingly': 1, 'slurfriend': 1, 'floorloud': 1, 'misinterpreted': 1, 'sarcasm': 1, 'miscalculationarea': 1, 'somethingberlin': 1, 'shootout': 2, 'saysparenting': 1, 'actthis': 1, 'heartbreakingly': 1, 'truenations': 1, 'hearty': 2, 'grad': 3, 'anyoneman': 2, 'outwhite': 1, 'kiosk': 2, 'cornfieldmom': 1, 'marveling': 1, 'homejared': 1, 'clearance': 3, 'downgradeellen': 1, 'recruits': 2, 'obamadepartmentstore': 1, 'chinawarepaul': 1, 'conservativesexcitement': 1, 'consecutive': 9, 'dayhillary': 2, 'protestersthe': 1, 'paradoxbody': 1, 'religionthwarting': 1, 'emptyteachers': 1, 'schoolsread': 1, 'officepeyton': 1, 'manning': 6, 'seniorspassport': 1, 'descent': 1, 'closednun': 1, 'chainsaw': 1, 'debris': 2, 'prothe': 1, 'disconnectcam': 1, 'newton': 1, 'panthers': 1, 'nfc': 1, 'championship': 2, 'seasonwhy': 1, 'industrynations': 1, 'bullshitlook': 1, 'tarotarea': 1, 'indeed': 2, 'belong': 2, 'applebeesgop': 1, 'americafema': 1, 'gobag': 1, 'themthe': 1, '40please': 1, 'onman': 2, 'timenurse': 1, 'draining': 3, 'bile': 3, 'liverjindal': 1, 'westboro': 1, 'funerals': 1, 'arrestmiamidade': 1, 'styrofoam': 2, 'beachesthe': 1, 'whycontent': 1, 'hotter': 2, 'socialnetflix': 1, 'wingsa': 1, 'truthdysfunctional': 1, 'statistically': 2, 'averageteen': 1, 'socially': 1, 'waysugh': 1, 'bartenders': 2, 'tienew': 1, 'stoked': 2, 'noticing': 3, 'duotwomonth': 1, 'freelance': 1, 'gig': 2, 'careers': 1, 'aredead': 1, 'cartman': 1, 'pictureclinton': 1, 'unpledged': 1, 'carget': 1, 'deadwellesley': 1, 'allgirl': 1, 'brochure20': 1, 'unspoken': 1, 'lesbianschilds': 1, 'anarchy': 1, 'restaurantactress': 1, 'criticssales': 1, 'chamomile': 1, 'masks': 4, 'sharplyi': 1, 'hyundai': 2, 'treesdavid': 1, 'cruztwo': 1, 'dacamented': 1, 'homeun': 1, 'convoy': 1, 'bombed': 1, 'aleppoheres': 1, 'truthers': 1, 'wrongmans': 1, 'accompanied': 1, 'wizardskittish': 1, 'juniorsdepartment': 1, 'againsun': 1, 'myung': 1, 'reportattention': 1, 'solicitor': 1, 'burwellconservationist': 1, 'deathdonald': 1, 'sadnew': 1, 'dominos': 2, 'pizzas': 1, 'digestive': 3, 'systemnipsey': 1, 'volume': 1, 'coupletssat': 1, 'biased': 1, 'nonhungoverdonald': 1, 'nicest': 1, 'conan': 2, 'spoofanonymous': 1, 'woodward': 1, 'relevant': 1, 'yearstrump': 1, 'addressdrink': 1, 'tomatoes13': 1, 'healthleftover': 1, 'stirs': 5, 'emotionthe': 1, 'getaways': 1, 'dayferocious': 1, 'pupcondom': 1, 'spermicidewalmart': 1, 'greeter': 1, 'deaths': 8, 'doorcongress': 1, 'syriabernie': 1, 'movementthankyou': 1, 'passiveaggressiveanchor': 1, 'adlibs': 1, '97': 1, 'accuracyman': 1, 'dayas': 1, 'yahoo': 2, 'roils': 1, 'mediavape': 1, 'franciscos': 1, 'reductionthat': 1, 'soontobeaudited': 1, 'manonline': 1, 'offensiveness': 1, 'authors': 2, 'caseglasses': 1, 'accessorytrump': 1, 'forge': 1, 'excusing': 1, 'testifyingtrump': 1, 'settlements': 2, 'processnow': 1, 'placewhere': 1, 'offensefbi': 1, 'raids': 2, 'fridgehow': 1, 'googlers': 1, 'fairsecondgrader': 1, 'purple': 4, 'lastcackling': 1, 'returnsthe': 1, 'kisscbs': 1, 'newsscott': 1, 'numbersnation': 1, 'marktwitters': 1, 'antitroll': 1, 'timedwoman': 1, 'miscarriage': 1, 'hermean': 1, 'buffoon': 2, 'pollclown': 1, 'photohow': 1, 'campslets': 1, 'deemphasize': 1, 'motivations': 1, 'advocacy': 3, 'volunteeringramadan': 1, 'reflection': 3, 'burmas': 2, 'campsexecutive': 1, 'electricians': 1, 'lunchgrandsons': 1, 'jigsaw': 1, 'puzzle': 2, 'patheticits': 1, 'reviewer': 1, 'chef5': 1, 'escapes': 1, 'moneyin': 1, 'fireimpossible': 1, 'frazzled': 1, 'workwatchdog': 1, 'implementation': 1, 'banvideo': 1, 'peopledallas': 1, 'prwhite': 1, 'punched': 3, 'wallalmost': 1, 'heritage': 4, 'findsguy': 1, 'racetrack': 1, 'todayhere': 1, 'neonaziswhite': 1, 'seahawks': 1, 'puntelizabeth': 1, 'acaamazing': 1, 'photoeulogy': 1, 'prochristian': 1, 'propagandaxabraxian': 1, 'planetwhat': 1, 'boots': 1, 'fadsits': 1, 'questionchinese': 1, 'viewersallowing': 1, 'succeedwith': 1, 'mull': 1, 'cases30': 1, 'horseschris': 1, 'hatewatch': 1, 'unreal5': 1, 'tipsmike': 1, 'pompeos': 1, 'disqualify': 2, 'himinsane': 1, 'godrl': 1, 'stine': 1, 'titles': 2, 'publisher': 3, 'rejectedthis': 1, 'nj': 2, 'housed': 1, 'veterans120000': 1, 'adoptions': 1, 'nokill': 1, 'shelterpregnant': 1, 'propeciahandling': 1, 'incidentrepublicans': 1, 'humiliate': 1, 'teeup': 1, 'crisisjudge': 1, 'battlesjohn': 1, 'urschel': 1, 'schoolrestaurant': 1, 'signus': 1, 'inspectors': 1, 'iraq5': 1, 'yearsalan': 1, 'unreported': 1, 'colmesgrieving': 1, 'strollerreport': 1, 'timesletters': 1, 'mosques': 3, 'genocidealito': 1, 'circuit': 2, 'courtcongress': 1, 'contractorsmom': 1, 'softlypolice': 1, 'elf': 1, 'stolenho': 1, 'ho': 1, 'christmasnypd': 1, 'weekarea': 2, 'raving': 3, 'phoenix': 2, 'airportlittle': 1, 'flabbergasted': 1, 'touchscreenmasochistic': 1, 'craving': 1, 'pissjews': 1, 'chickenhalfasleep': 1, 'socksheres': 1, 'antisemitismnew': 1, 'seaside': 1, 'cottage': 1, 'greatly': 1, 'productivityharry': 1, 'legeno': 1, 'hiking': 1, 'valley4yearolds': 1, 'optimism': 1, 'familyben': 1, 'anarchynetflixs': 1, 'jonbent': 1, 'case15': 1, 'ireporters': 1, 'afghanistandisney': 1, 'unified': 1, 'universethe': 1, 'abcs': 4, 'twisthow': 1, 'modify': 1, 'happinessgop': 1, 'lindsey': 1, 'partyhow': 2, 'reddpeak': 1, 'directorswhat': 1, 'dadsstraight': 1, 'weekendthis': 1, 'you22yearold': 1, 'financeshouse': 1, 'brainstorming': 2, 'francisbears': 1, 'doorim': 1, 'directiveno': 1, 'engagedgovernment': 1, 'halliburton': 1, 'favorseven': 1, 'solitary': 5, 'confinementa': 1, 'refs': 1, 'allthirdparty': 1, 'forms': 5, 'monthsdrag': 1, 'performers': 2, 'facebookhillary': 1, 'symbolism': 1, 'specificsfrustrated': 1, 'controlthis': 1, 'crisisnetanyahu': 1, 'foolsdeath': 1, 'icy': 4, 'kennedywhy': 1, 'warsgetting': 1, 'intimidatinghow': 1, 'losswoman': 1, 'commitmentnyc': 1, 'bulbs': 1, 'libertys': 1, 'eyescalifornia': 1, 'teacherscalls': 1, 'hotlines': 1, 'antitransgender': 1, 'actionreport': 1, 'linenaacp': 1, 'upnchristian': 1, 'bends': 1, 'lovenew': 1, 'snack': 4, 'burrows': 1, 'heartcornered': 1, 'defies': 2, 'videoformer': 1, 'abolish': 1, 'mutilation': 1, 'somaliadc': 1, 'squatters': 1, 'departmentemerald': 1, 'contaminationshrinking': 1, 'legalizationan': 1, 'eyeopening': 1, 'playgrounds': 1, 'worldpolice': 1, 'highprofile': 1, 'charging': 4, 'crimesclinton': 1, 'protein': 1, 'pastecampus': 1, 'quickfaa': 1, 'safe5thgrade': 1, 'apeshit': 1, 'giverhanding': 1, 'pickiranian': 1, 'roomhow': 1, 'navigate': 1, 'studentsec': 1, 'confidencedelayed': 1, 'connecting': 2, 'flightfbi': 1, 'declassifies': 1, 'edgar': 1, 'hoovers': 1, 'munster': 1, 'familypoll': 1, 'kirk': 1, 'shrivelscoworkers': 1, 'herculean': 1, 'commutethere': 1, 'conferencelooking': 1, 'psychotic': 2, 'thresholds': 1, 'chicagoinclement': 1, 'liar': 3, 'workscientists': 1, '6000yearold': 1, 'stainreallife': 1, 'hitchcock': 1, 'versioncardinals': 1, 'goingaway': 2, 'dive': 4, 'baridea': 1, 'traction': 1, 'congressformer': 1, 'roomreport': 1, 'harshest': 1, 'critickevin': 1, 'oscarsjoe': 1, 'institutional': 2, 'billwhy': 2, 'americanations': 1, 'grandfathers': 1, 'shipment': 3, 'treasurymike': 1, 'drapes': 1, 'shawl': 1, 'immodest': 1, 'statuemayim': 1, 'bialik': 1, 'opedapple': 1, 'fleeting': 1, 'excitementfrench': 1, 'ministry': 1, 'libyaus': 1, 'invest': 2, 'communitybased': 1, 'businessesfinding': 1, 'wingscritics': 1, 'gores': 1, 'realisticarea': 1, 'brokenbiden': 1, 'renothis': 1, '14yearold': 1, 'cellist': 1, 'classical': 1, 'musicdozens': 1, 'beaches13': 1, '20snew': 1, 'chinabloated': 1, 'podiumfrom': 1, 'courtrecovery': 1, 'expressions': 1, 'blew': 2, 'mind50': 1, 'workplaces': 2, 'diversity5': 1, 'daughterssuspect': 1, 'ambushstyle': 1, 'officersheartbreaking': 1, 'smilejanet': 1, 'wissam': 1, 'manaevery': 1, 'tragedywrestling': 1, 'seeinggay': 1, 'therapists': 1, 'suicidekasparov': 1, 'derezzedhollywood': 1, 'lewisarea': 1, 'bluray': 1, 'breakers': 1, 'himselfroommate': 1, 'sexgeneral': 1, 'yearsis': 1, 'moderation': 1, 'crapbruno': 1, 'funk': 1, 'bowlmiss': 1, 'clothed': 2, 'dayguantanamo': 1, 'wingcomey': 1, 'mechanism': 1, 'feelingsbreaking': 1, 'israelites': 1, 'sinai': 1, 'pharaoh': 1, 'forecastalldad': 1, 'disappointmentputin': 1, 'summitisis': 1, 'videoexciting': 1, 'pawns': 1, '26yearold': 1, 'gameus': 1, 'courtmorbidly': 1, 'contestmedia': 1, 'standardshoodie': 1, 'hop': 1, 'impart': 1, 'buddhist': 1, 'wisdomwoman': 1, '5k': 1, 'unrewardinggoogles': 1, 'controversythere': 1, 'captioned': 1, 'meetinga': 1, 'speechtravel': 1, 'regales': 2, 'roadharvey': 1, 'farunusually': 1, 'levelheaded': 1, 'charismatic': 1, 'lichen': 1, 'obamawhatsapp': 1, 'subscription': 2, 'feesfda': 1, 'boxgame': 1, 'unkempt': 1, 'castafter': 1, 'highs': 1, 'lows': 1, 'girlsinternational': 1, 'lodging': 1, 'venice': 2, 'italyohio': 1, 'injuredman': 1, '643457this': 1, 'womenkylie': 1, 'canadaclinton': 1, 'stump': 1, 'hackerscandlelight': 1, 'vigilante': 1, 'commemorating': 2, 'handsstill': 1, 'citysaving': 1, 'rhinothe': 1, 'dissentersjudge': 1, 'ricoobama': 1, 'detestable': 1, 'partstom': 1, 'bosley': 1, 'napsreport': 1, 'morbid': 1, '79': 3, 'purchasesthats': 1, 'karenina': 1, 'christmasits': 1, 'announcea': 1, 'burial': 2, 'wifea': 1, 'spectrumdonald': 1, 'didhow': 1, 'spousebitcoin': 1, 'fiat': 1, 'currencyfidelity': 1, 'matches': 2, 'ira': 1, 'contributionsben': 1, 'higgins': 2, 'bushnell': 1, 'buzzwords': 1, 'againharlem': 1, 'ajit': 2, 'paibrazils': 1, 'officegay': 1, 'lookin': 1, 'goodmosquito': 1, 'tickborne': 1, 'tripled': 1, 'changeweinstein': 1, 'industryarne': 1, 'librarydaily': 1, 'creativityteen': 1, 'catcallers': 1, 'attackphysics': 1, 'ta': 1, 'uslaura': 1, 'nicholas': 1, 'cruzmike': 1, 'flagrantly': 1, 'displaying': 1, 'organswhy': 1, 'womanvladimir': 1, 'isunsuspecting': 1, 'kidnappers': 1, 'vanrestaurants': 1, 'nachoscoworker': 1, 'arounddelta': 1, 'passedmarriage': 1, 'workhouse': 1, 'lawsuitabraham': 1, 'lincolns': 1, 'dna': 1, 'counterbruno': 1, 'coveted': 3, 'grammyssenatorial': 1, 'racearea': 1, 'reuben': 2, 'sandwichestrump': 1, 'volatility': 1, 'threatsdog': 1, '400th': 1, 'dayblowing': 1, 'smoke': 2, 'warmingrustic': 1, 'feastshalf': 1, 'clinics': 2, 'yearspaintball': 1, 'memorialpenn': 1, 'fined': 1, 'casealy': 1, '175': 1, 'enoughcar': 1, 'shittiest': 1, 'car300': 1, 'crashjetblue': 1, 'saletuesdays': 1, 'overhaulstaylor': 1, 'garfield': 1, 'davisjudge': 1, 'persky': 1, 'cleared': 3, 'stanford': 1, 'casetarget': 1, 'membersscenes': 1, 'huddle': 1, 'menchris': 1, 'deputizes': 1, 'absencedrug': 1, 'paraphernalia': 1, 'catmichael': 1, 'dukakis': 2, 'electionis': 1, 'separated': 2, 'videoheres': 1, 'activistteach': 1, 'applicationsman': 1, 'permissioninstead': 1, 'lawfleshlighthouse': 1, 'weary': 1, 'texturewill': 1, 'onnew': 3, 'aspershirt': 1, 'relieves': 1, 'torso': 1, 'painnations': 1, 'dadmedicaid': 1, 'expansion': 3, 'disabilitiessavion': 1, 'glover': 3, 'ticketheres': 1, 'everythinglucky': 1, 'marshmallow': 1, 'headborrowers': 1, 'skyhigh': 1, 'subprime': 1, 'bubble': 4, 'carstrumps': 1, 'confirmationwall': 1, 'stippleportrait': 1, 'artistshot': 1, 'chixx': 1, 'creditcard': 1, 'billi': 1, 'stormy': 2, 'sonny': 1, 'perdue': 1, 'guythe': 2, 'intervention': 3, 'altrightgay': 1, 'divorcecandy': 1, 'screenwriter': 4, 'righttv': 1, 'depicting': 2, 'murderben': 1, 'bradythis': 1, 'dividedonald': 1, 'deport': 1, 'aidezoo': 1, 'roam': 1, 'tbilisisyrian': 1, 'longawaited': 2, 'familygerber': 1, '60000': 4, 'jars': 1, 'poisonkatie': 1, 'ballgownkashmir': 1, 'violencebearded': 1, 'nicethis': 1, 'converted': 1, 'indoor': 3, 'gardencharlize': 1, 'clipsleaterkinney': 1, 'bowies': 1, '2017man': 1, 'companyresponsible': 1, 'ownershipleah': 1, 'remini': 2, 'scientologycolleges': 1, 'careerlink': 1, 'connects': 1, 'alumswhy': 1, 'yetstudy': 1, 'cooperate': 1, 'exhibited': 1, 'pizzacat': 1, 'unfazedstaples': 1, 'weekdare': 1, 'whereverlets': 1, 'gleefully': 2, 'slamming': 2, 'instagramstates': 1, 'enacted': 1, '2016little': 1, 'mittstranded': 1, 'rescuereport': 1, 'dzhokhar': 1, 'ownerfailure': 1, 'element': 3, 'successrussia': 1, 'largescale': 1, 'exercises': 1, 'disputed': 2, 'territoriesjeb': 1, 'pipewielding': 1, 'thugs': 1, 'delegates': 2, 'weekrestaurants': 1, '20147': 1, 'olympicsdismembered': 1, 'nate': 1, 'dumpster': 2, 'headquarterssxsw': 1, 'ibtihaj': 1, 'hijabprocess': 1, 'presentness': 1, 'lundcustody': 1, 'childpaul': 1, 'krugmans': 1, 'timesproof': 1, 'friendsthree': 1, 'clintonlets': 1, 'allkenneth': 1, 'starr': 1, 'lbj': 1, 'exhumed': 1, 'improprietythis': 1, 'moneyrep': 1, 'costa': 1, 'california8': 1, 'travelingthis': 1, 'buttdialed': 1, 'lorne': 1, 'michaelsman': 1, 'nowuber': 1, 'travis': 3, 'kalanick': 2, 'councilthis': 1, 'lolworthypopular': 1, 'spinal': 1, 'attached': 2, 'lungfirefighter': 1, 'evergop': 1, 'nominatescasual': 1, 'nuclearwastedisposal': 1, 'technicianstwas': 1, 'contestjoe': 1, 'nolongercovert': 1, 'wifetom': 1, 'daley': 1, 'togetherhow': 1, 'loveda': 1, 'nurture': 1, 'originatefor': 1, 'fingerbook': 1, 'dataclysmhow': 1, 'pennsylvaniabased': 1, 'gooddolores': 1, 'huerta': 1, 'newcomerstoo': 1, 'napgay': 1, 'hostilitymarco': 1, 'tussle': 2, 'votescollege': 1, 'lively': 3, 'ideamorton': 1, 'individually': 1, 'wrapped': 3, 'grainsfrom': 1, 'easierhuffpost': 1, 'teams': 5, 'hiresowner': 1, 'businesspublic': 1, 'dealssevenfoottall': 1, 'animatronic': 1, 'rodent': 2, 'terrifies': 1, 'agocathedral': 1, 'texters': 1, 'bumping': 1, 'itprofessor': 1, 'slammed': 3, 'officeremail': 1, 'organize': 4, 'officewide': 1, 'outing': 1, 'unbearably': 1, 'sadwere': 1, 'restaurantuniversity': 1, 'alleviate': 1, 'concussionsteen': 1, 'spoke': 1, 'violencestudy': 1, 'qualms': 1, 'caging': 1, 'humanswhat': 1, 'mizzoudonald': 1, 'briefingtaylor': 1, 'shattered': 1, 'trusttown': 1, 'uglification': 1, 'bagsg7': 1, 'deposit': 2, 'g8': 2, 'tshirtstotal': 1, 'weirdo': 3, 'cemeteryfanatically': 1, 'pegg': 1, 'momentmack': 1, 'wilds': 3, 'firedheavily': 1, 'processed': 1, 'claimspornographic': 1, 'himgallup': 1, 'forecast': 3, 'primarywhats': 1, '270': 1, 'officestephen': 1, 'conspiracies': 1, 'nsfw': 1, 'diagrammarine': 1, 'shortens': 1, 'slogan': 1, 'fewthe': 1, 'gabriel': 2, 'garca': 1, 'mrquez': 1, 'reportscartoon': 1, 'biggestbush': 1, 'surges': 2, 'pommel': 1, 'horsereceiving': 1, 'thankshillary': 1, 'principalplussize': 1, 'yearsecretary': 1, 'penny': 1, 'circulationhow': 1, 'untested': 2, 'citythe': 1, 'homeparents': 1, 'nowtrump': 1, 'fortuneher': 1, 'morekennedy': 1, 'dishonor': 1, 'gottfriedtexas': 1, 'environmentalists': 1, 'brisket': 1, 'bbq': 2, 'sayphotographer': 1, 'saldiva': 1, 'fearslesbian': 1, 'hen': 2, 'housenation': 1, 'bullshit': 2, 'healingrussia': 1, 'batshitzaniabernie': 1, 'wageavoiding': 1, 'accomplishment': 1, 'settles': 1, 'concise': 1, 'askswatch': 1, 'illusionreport': 1, 'instructorssomeone': 1, 'chauvinist': 1, 'womenparents': 1, 'listswhy': 1, 'generouslychina': 1, 'slaughters': 1, 'outbreak5': 1, 'behaviors': 4, 'commona': 1, 'yearus': 1, 'middlemen': 1, 'outjames': 1, 'presold': 1, 'saysblearyeyed': 1, 'stuporous': 1, 'houseguest': 1, 'greatarea': 1, 'throatclearer': 1, 'moviegeo': 1, 'whistleblower': 3, 'violations': 3, 'adelanto': 1, 'centerconservative': 1, 'annoyingly': 1, 'racistreport': 1, 'diabeteslocal': 1, 'gynecologist': 2, 'truthun': 1, 'malfunctioning': 1, 'pyrotechnicsoregons': 1, 'vacationsdad': 1, 'obamacarenew': 1, 'secretarywill': 1, 'reconciliation23': 1, 'sleephow': 2, 'detroitlin': 1, 'manuel': 1, 'platt': 1, 'mashupscott': 1, 'bakula': 1, 'reportsperverted': 1, 'creep': 2, 'wearingscience': 1, 'recalldj': 1, 'khaled': 1, 'arianna': 3, 'dinnerthe': 1, 'dubai': 1, 'diaries': 2, 'lifechanging': 2, 'eventmexico': 1, 'risky': 2, 'vaquita': 1, 'porpoisebreaking': 1, 'nowyoure': 1, 'winbernie': 1, 'electedcellmate': 1, 'suge': 1, 'knights': 1, 'beefslocal': 1, 'talk49ers': 1, 'ot': 1, 'chargersthird': 1, 'tenuous': 1, 'trickamericans': 1, 'findstrump': 1, 'tweak': 1, 'ruleslocal': 1, 'newswomans': 1, 'coanchornations': 1, 'anthemrangers': 1, 'fielder': 1, 'chewedup': 1, 'dugoutmillions': 1, 'lips': 4, 'povertythe': 1, 'agearea': 2, 'spoon': 3, 'rinsed': 1, 'monthshistorical': 1, 'toil': 1, 'hometrump': 1, 'fraudben': 1, 'tormented': 1, 'thoughtswithin': 1, 'crisisthe': 1, 'whitewashing': 1, 'browndrug': 1, 'percentcontroversial': 1, 'peasant': 1, 'jrthe': 1, 'firstlevel': 1, 'leadershipamc': 1, 'retrospective': 3, 'godermatologists': 1, 'demonic': 1, 'bulging': 1, 'skinlas': 1, 'sands': 1, 'sec': 2, 'macaupope': 1, 'womenfamily': 1, 'peyton': 4, 'wanders': 1, 'pocketkevin': 1, 'humors': 2, 'wtf': 1, 'weekchinese': 1, 'lunchguy': 1, 'hourstommy': 1, 'wiseau': 1, 'expectdozens': 1, 'ethiopia': 1, 'protestanother': 1, 'arrested2yearold': 1, 'youdecades': 1, 'mantlewas': 1, 'electionarea': 1, 'thinga': 1, 'passover': 1, 'easteris': 1, 'badconfusing': 1, 'clarifiedtoddler': 1, 'imitates': 4, 'dancer': 2, 'duetyou': 1, 'belair': 2, 'introgogetter': 1, 'eliminates': 3, 'processsavings': 1, 'womancouple': 1, 'verbalizing': 1, 'visionim': 1, 'everywherejohn': 1, 'edwards2016com': 1, 'casehouse': 1, 'obamacarethe': 1, 'grossest': 2, 'hangover': 3, 'remedies': 1, 'ranked22': 1, 'capabilities': 1, 'themman': 1, 'lawsredford': 1, 'redigitize': 1, 'battlehere': 1, 'winners8': 1, 'perrypepsi': 1, 'pepsiawareness': 1, '00000000001': 1, 'percent5': 1, 'weightchild': 1, 'issuesfreudian': 1, 'kneecalifornias': 1, 'overcrowdingpope': 1, 'vestments': 2, 'hotelhistorical': 1, 'inaccuracy': 1, 'showthe': 1, 'rooftop': 2, 'uswhere': 1, 'vetslibyans': 1, 'qaddafi': 1, 'leavingtrump': 1, 'racelearning': 1, 'argentinanasa': 1, 'warninggeneral': 1, 'bedford': 1, 'forrest': 2, 'ku': 1, 'klux': 1, 'klanbuttons': 1, 'kneesnew': 1, 'audubon': 1, 'plumagethe': 1, 'iiera': 1, 'clubresourceful': 1, 'cobble': 1, 'annoyancesmy': 1, 'colorlets': 1, 'mizzoulatino': 1, 'reporta': 1, 'trumpelton': 1, 'tourprison': 1, '52': 2, 'deadnew': 1, 'singlepayer': 1, 'supportlea': 1, 'candid': 3, '60sylville': 1, 'modelhere': 1, 'states20': 1, 'beginnersdog': 1, 'graduatedamerican': 1, 'ridley': 1, 'campusessurvivalist': 1, 'troopermore': 1, 'trumphorde': 1, 'monsters': 3, 'exits': 2, 'tanning': 1, 'salonquantico': 1, 'ode': 1, 'daysan': 1, 'manall': 1, 'wacky': 3, 'memorabilia': 1, 'buywhy': 1, 'themlooking': 1, 'americaold': 1, 'faithful': 2, 'brutally': 5, 'teensstudy': 1, 'entitled': 2, 'opinionreport': 1, 'irans': 3, 'lettertoddler': 1, 'unsettled': 1, 'possessed': 1, 'facehuffpost': 1, '20moms': 1, 'thembetween': 1, 'duterte': 4, 'aseanghost': 1, 'alvah': 1, 'roebuck': 1, 'declineyour': 1, 'paved': 2, 'grasswe': 1, 'lettervans': 1, 'rocking': 2, 'discourages': 1, 'knockernot': 1, 'sicily': 1, 'naplestwitter': 1, 'revoking': 1, 'verified': 1, 'profiles': 1, 'rulesguest': 1, 'deflate': 1, 'night7': 1, 'intergenerational': 1, 'friendshipnew': 1, 'reversal': 1, 'prescriptionyou': 1, 'seasondesktop': 1, 'assistantlena': 1, 'dings': 1, 'allenlittle': 1, 'wiser': 1, 'yousen': 1, 'lugar': 1, 'hamstringnapkinless': 1, 'greasecovered': 1, 'creationwhite': 1, 'showundertakers': 1, 'embalmings': 1, 'sloppymichele': 1, '2012rain': 1, '1986': 1, 'returnsamy': 1, 'boyfriendrepublicans': 1, 'retain': 4, 'householdsamsung': 1, 'exchanges': 1, '7how': 1, 'dinnerplan': 1, 'credits': 1, 'failsfolk': 1, 'visitorthe': 1, 'imitating': 2, 'satire': 1, 'versahouse': 1, 'chaplain': 2, 'weakass': 1, 'wound': 2, 'everyoneblack': 1, 'bishops': 1, 'predominantly': 1, 'churchrussian': 1, 'altitude': 1, 'saysdiabetic': 1, 'evertrevor': 1, 'childpermanent': 1, 'racethis': 1, 'treatcanon': 1, '5d': 1, 'iv': 1, 'dslr': 2, 'preview': 3, 'videojpso': 1, 'orleansa': 1, 'colombiaon': 1, 'peaches': 1, 'monroee': 1, 'linevideo': 1, 'heartbreak': 1, 'alzheimerswhy': 1, 'teamlogan': 1, 'revivalmedicine': 1, 'gapjoy': 1, 'unlikelycontent': 1, 'evolve': 2, 'elsemicrobrewer': 1, 'beerwednesdays': 1, 'unioncancer': 1, 'votewe': 1, 'invisible': 4, 'dalit': 1, 'womenasshole': 1, 'partnerships': 1, 'otherhonest': 1, 'confessionsafter': 1, 'gator': 1, 'familymonaco': 1, 'penthousing': 1, 'projectstoys': 1, 'pavlovian': 1, 'childnobody': 1, 'governmenttj': 1, 'maxx': 1, 'shoplifti': 1, 'violencelone': 1, 'looters': 1, 'fergusona': 1, 'concurs': 1, 'centerlady': 1, 'illusionjohn': 1, 'wgn': 1, 'cancellation5': 1, 'successaccidentally': 1, 'lifeart': 1, 'capitalizing': 3, 'nameman': 1, 'drifting': 3, 'redirects': 1, 'himashley': 1, 'debatewhat': 1, 'rockwell': 2, 'usbiden': 1, 'installing': 2, 'aboveground': 1, 'poolsrepublican': 1, 'recanting': 1, 'officelawyers': 1, 'collusiongina': 1, 'haspel': 3, 'rubs': 1, 'hearingumpqua': 1, 'gunfree': 1, 'zonekhloe': 1, 'contentthe': 1, 'outbreakchained': 1, 'yearns': 1, 'bankmy': 1, 'soapsobama': 1, 'afghanistanwoman': 1, 'selfesteem': 2, 'selfesteemebola': 1, 'expectedhow': 2, 'choicecultural': 1, 'gems': 1, 'ofdo': 1, 'onevin': 1, '6philanderer': 1, 'backcharlize': 1, 'doesvocalist': 1, 'differencesits': 1, 'trumptransit': 1, 'outofservice': 1, 'buses': 1, '2006decorative': 1, 'positively': 1, 'aching': 1, 'plump22yearold': 1, 'discriminationboundless': 1, 'buddha': 3, 'doodlechipotle': 1, 'caresstudy': 1, 'skimp': 1, 'diseaseann': 1, 'landers': 1, 'latemom': 1, 'calmly': 1, 'emptying': 1, 'dishwasher': 1, 'agostudy': 2, 'unfit': 4, 'consumptionthe': 1, 'imagery': 2, 'stitched': 1, 'galalisten': 1, 'otherman': 1, 'somedaystudy': 1, 'clinical': 1, 'dunked': 1, 'onpoll': 1, 'overhear': 1, 'ballotnyt': 1, 'column': 3, 'asserts': 2, 'stinkstories': 1, 'odoms': 1, 'nbasources': 1, 'cmon': 1, 'alreadythe': 1, 'apes': 2, 'blockbustersfriend': 1, 'sequencesales': 1, 'surged': 1, 'monthwhat': 1, 'cottons': 1, 'claimheartbreaking': 1, 'polar': 5, 'islandmiracle': 1, 'overpass': 1, 'urinehow': 1, 'humanitiespizza': 1, 'cheesestuffed': 1, 'boyreport': 2, 'threefourths': 1, 'dosevere': 1, 'sizethe': 1, 'trumpnew': 2, 'manly': 2, 'nightthe': 1, 'ofthe': 1, 'bridesmaidsemployee': 1, 'timetraditional': 1, 'basically': 3, 'concludesscifi': 1, 'models5': 1, 'debtpodiatrist': 1, 'jerkfollowing': 1, 'yauch': 1, 'frees': 1, 'tibetus': 1, 'kilauea': 1, 'volcano': 3, 'civilianssudanese': 1, 'forgetstudy': 1, 'allconsuming': 1, 'selfpity': 1, 'expartneramazon': 1, 'alexa': 2, 'eavesdropped': 1, 'portland': 5, 'familyguy': 1, 'atv': 1, 'wooded': 1, 'trailjohn': 1, 'againman': 1, 'dogyou': 1, 'unsee': 1, 'otterpaddle': 1, 'angles': 1, 'raceamerican': 1, 'universities': 3, 'sinoforeign': 1, 'venturesjudge': 1, 'aerobics': 1, 'trial13': 1, 'dayintact': 1, 'benetton': 1, 'miraculously': 2, 'rubble': 2, 'laternra': 1, 'schoolersno': 1, 'planhillary': 1, 'yetheres': 1, 'nightmaxine': 1, 'linehan': 1, 'ireland': 2, 'interpretation': 3, 'songsautopsy': 1, 'todd': 2, 'ewen': 1, 'cteleaving': 1, 'tedx': 1, 'talkreport': 1, 'billionbassnectar': 1, 'unlimitedtheres': 1, 'cuteness': 1, 'dograting': 1, 'feedheineken': 1, 'specialrelease': 1, 'beerjimmy': 1, 'againryan': 1, 'haircutiraqi': 1, 'danube': 1, 'rivermall': 1, 'cinnabontographya': 1, 'lesley': 1, 'kagenclass': 1, 'wayprince': 1, 'weddingjames': 1, 'allegations5': 1, 'slidein': 1, 'samequeen': 1, 'djshumiliated': 1, 'embroidery': 1, 'pocketsworldweary': 1, 'rents': 1, 'risingreport': 1, 'stagnant': 1, 'infrastructuredrunk': 1, 'songwhats': 1, 'terrence': 1, 'malick': 1, 'freida': 1, 'pintodonald': 1, 'bodyguards': 1, 'herreport': 2, 'caucasians': 1, 'countryfactory': 1, 'existence': 3, 'refrigeratorisraeli': 1, 'netanyahus': 1, 'statefor': 1, 'predictions': 1, 'themallbusiness': 1, 'beelines': 1, 'pinhead': 1, 'maskkfc': 1, 'boneless': 1, 'ceohistorical': 1, 'weekley': 1, 'resultscalifornia': 1, 'newsom': 1, 'presidentcnn': 1, 'contributor': 1, 'gulp': 1, 'farleys': 1, 'deathword': 1, 'swallowwhat': 1, 'swimmer': 3, 'shock': 4, 'youbowling': 1, 'alumnus': 1, 'outhillary': 1, 'turnouthillary': 1, 'offgop': 1, 'reformwhat': 1, 'fearevan': 1, 'westworld': 2, 'costarslas': 1, 'newspaperhamilton': 1, 'brouhahaa': 1, 'marnisuarezs': 1, 'uruguaylame': 1, 'atms': 1, 'sweepers': 1, 'machinesgorsuch': 1, 'rbg': 1, 'coupleunited': 1, 'monthhippocratic': 1, 'shieldit': 1, 'humane': 1, 'slaughterhousepromotional': 1, 'virusobama': 1, 'offshore': 5, 'drillingtoyota': 1, '1993': 1, 'camry': 1, 'nowdwayne': 1, 'nurturing': 1, 'videoteen': 1, 'barging': 1, 'folded': 2, 'clothesarea': 1, 'gurneyed': 1, 'productmovie': 1, 'coachvoter': 1, 'nostalgically': 1, 'candidatescdc': 1, 'longplanned': 1, 'presidentnewly': 1, 'swornin': 1, 'eventually': 1, 'executedwoman': 1, 'treadmill': 1, 'cranks': 2, 'incline': 1, 'degreesfifty': 1, 'marina': 1, 'literary': 2, 'sensationrefugees': 1, 'countrygame': 1, 'thwarts': 1, 'failuregun': 1, 'stocks': 1, 'soar': 4, 'control3': 1, 'perspectivemale': 1, 'cloaked': 2, 'mysteryyoga': 1, 'womeneven': 1, 'rightsstudy': 1, 'rotisseries': 1, 'advancingthe': 1, 'wack': 1, 'projectfridays': 1, 'monthsmike': 1, 'huckabees': 2, 'somethingwhy': 1, 'aumf3': 1, 'coping': 2, 'holidaysnations': 1, 'outfoxed': 1, 'sheriffs': 1, 'dirt6yearold': 1, 'blockboise': 1, 'likedianne': 1, 'disintegrates': 2, 'chambersad': 1, 'bunchmia': 1, 'fathered': 2, 'sinatra': 1, 'puzo': 1, 'mcgovern': 1, 'altman': 2, 'anthony': 3, 'perkins': 1, 'milton': 2, 'berle': 2, 'redford': 2, 'caine': 1, 'aiello': 1, 'dernpingpong': 1, 'girlfrienddog': 1, 'hurrydaily': 1, 'orderunreasonable': 1, 'happinessno': 1, 'responding': 1, 'reportexclusive': 1, 'publiciststhe': 1, 'jongunwow': 1, 'erykah': 1, 'badu': 1, 'tipsarsons': 1, 'linked4yearold': 1, 'italyendangered': 1, 'programcyclist': 1, 'signaling': 1, 'turnsnew': 1, 'loveproduce': 1, 'footballheavy': 1, 'southenchilada': 1, 'premonition': 2, 'passthe': 1, 'weekapparently': 1, 'peanutsmelania': 1, 'escort7': 1, 'cornmans': 1, 'glaze': 1, 'serflike': 1, 'subjugationmurphy': 1, 'airliterary': 1, 'narratives': 1, 'derived': 1, 'conflictjohn': 1, 'twicegop': 1, 'estimated': 1, 'debtamount': 1, 'collected': 1, 'percentgrannys': 1, 'songolder': 1, 'elements': 1, 'replacementthe': 1, 'murderer': 2, 'outtammy': 1, 'nowworld': 1, 'suffused': 1, 'antinazi': 1, 'undertonesdick': 1, 'dyke': 1, 'dennys': 2, 'impromptu': 1, 'chitty': 2, 'bangcardi': 1, 'anythingreport': 2, '1996': 1, 'reintegration11': 1, 'yearcop': 1, 'tickled': 1, 'policereport': 1, 'handmaids': 3, 'amoklaptop': 1, 'tenjoe': 1, 'zee': 1, 'glamorous': 2, 'resolutionjoe': 1, 'considerationportrait': 1, 'wallwhy': 1, 'bullyinghistorical': 1, 'publiksunday': 1, 'administrations': 3, 'liesstudents': 1, 'superintendent': 1, 'daythis': 1, 'jaws': 1, 'huckabeeragnar': 1, '3clear': 1, 'onscientists': 1, 'epidemicdogs': 1, 'dinnernew': 1, 'hammers': 2, 'impulsive': 1, 'buttondonald': 1, 'courtlaidoff': 1, 'zoologist': 1, 'tranquilizing': 1, 'rampagesilvio': 1, 'moneyasimo': 1, 'tricked': 1, 'stairsd23s': 1, 'sanderson': 1, 'clubfatherinlaw': 1, 'onesentence': 1, 'problemssuri': 1, '11heres': 1, 'shootingmedical': 1, 'inconveniencing': 1, 'familymom': 1, 'unheardof': 1, 'homeriaa': 1, 'songswant': 1, 'flirt': 1, 'morei': 1, 'struggled': 1, 'sonamerican': 1, 'seatmates': 1, 'interestsfbi': 1, 'upi': 1, 'speeches': 2, 'likecant': 1, 'escapeman': 1, 'functional': 2, 'monoxide': 1, 'detector': 4, 'somewhereheres': 1, 'landlinesthe': 1, 'killedkenny': 1, 'chesney': 1, 'kenny': 3, 'chesneyamerican': 1, 'spoilerssebastian': 1, 'gorka': 1, 'supremacists': 3, 'terrorismtide': 1, 'sour': 1, 'detergent': 1, 'podssupreme': 1, 'mentioning': 3, 'nike': 1, 'ruling5yearold': 1, 'girlfriendsdoctor': 1, 'coffeefriday': 1, 'raining': 1, 'shoessyrian': 1, 'pentagonnation': 1, 'travoltaactivists': 1, 'washingtonlast': 1, 'childan': 1, 'heirloomsimone': 1, 'costas': 1, 'famousbehold': 1, 'darcyover': 1, 'mourners': 1, 'hospital9': 1, 'printable': 1, 'procrastinatorsshredding': 1, 'pastguillermo': 1, 'toro': 2, 'festivalpriest': 1, 'celibacy': 1, 'furriesmilitary': 1, 'executionmother': 1, 'pinching': 1, 'flightbill': 1, 'radioheads': 1, 'athese': 1, 'carts': 1, 'stylemad': 1, 'mixedwhy': 1, 'sickdick': 1, 'cheneys': 1, 'exploredowner': 1, 'creepiest': 1, 'barnew': 1, 'discipline': 2, 'teeniowan': 1, 'sobbing': 2, 'hallbush': 1, 'defender': 1, 'impeached': 2, 'bergdahlrestaurant': 1, 'unwanted': 2, 'cuisinedrunk': 1, 'humansmcconnell': 1, 'revs': 1, 'butdene': 1, 'benton': 1, 'aka': 2, 'ruby': 1, 'broadwaynew': 1, 'modeled': 1, 'heyday': 1, 'fm': 2, 'radiolocal': 1, 'foremost': 1, 'usohio': 1, 'aquamanviagra': 1, 'comedianswe': 1, 'hatesicario': 1, 'denis': 1, 'villeneuve': 1, 'senseless': 1, 'filminterim': 1, 'engages': 1, 'activity': 4, 'whatsoeverparanoid': 1, 'burglar': 1, 'himbudapest': 1, 'sorosfounded': 1, 'universitythis': 1, 'jobmonopoly': 1, 'editionmemories': 1, 'sand': 3, 'gush': 1, 'katif': 1, 'disengagementnew': 1, 'okcupid': 1, 'backtaiwans': 1, 'proindependence': 1, 'electionwinning': 1, 'wasstudy': 1, '1is': 1, 'twostate': 2, 'israelwhen': 1, 'clicks': 1, 'placeiraqi': 1, 'reshingling': 1, 'roofguantnamo': 1, 'crowdarnold': 1, 'schwarzenegger': 1, 'apprentice': 1, 'catchphrasebusy': 1, 'recital7': 1, 'agedog': 1, 'homeeverything': 1, 'citylawyer': 1, 'trialdj': 1, 'vuthe': 1, 'crisiselizabeth': 1, '0trump': 1, 'seething': 1, 'crowdobama': 1, 'detaineesrichard': 1, 'dawkins': 1, 'betraying': 1, 'movementtim': 1, 'buzzwordchild': 1, 'puppyheres': 1, 'studentsjennifer': 1, 'feminismbroadway': 1, 'hamiltonmidwesterners': 1, 'descend': 3, 'filescreepy': 1, 'asylums': 1, 'nightnude': 1, 'wireactivists': 1, 'abductionsecluded': 1, 'americatrump': 3, 'surrogate': 3, 'minutemueller': 1, 'overjames': 1, 'omarosaalpha': 1, 'tridelta': 1, 'femalesenate': 1, 'mockingbird6': 1, 'summer15': 1, 'instantlystoners': 1, 'stoned': 1, 'thatone': 1, 'brook': 1, 'jostled': 1, 'citydozens': 1, 'shrinedoes': 1, 'mattersouth': 1, 'dakotas': 1, 'crops': 1, 'weekendobama': 1, 'emissionsnaomi': 1, 'watts': 2, 'liev': 1, 'schreiber': 1, 'emmys': 4, 'carpeteyeopening': 1, 'flips': 1, 'violencenations': 1, 'straw': 2, 'animalsthese': 1, 'schoolsfaa': 1, 'asiana': 1, 'crashchance': 1, 'livestreams': 1, 'mosquitoborne': 1, 'illnesschristian': 1, 'neutered': 1, 'statuette': 1, 'oscarwhy': 1, 'zestturkeys': 1, 'predicament': 1, 'dangersmarcus': 1, 'mariota': 1, 'adrahm': 1, 'listeningblack': 1, 'jobvespa': 1, 'corporation': 3, 'enchants': 1, 'manchilddhs': 1, 'profiling': 1, 'freeforall': 1, '11inside': 1, 'cbs': 2, 'colbertnews': 1, 'movegorgeous': 1, 'mooncollege': 1, 'all34yearold': 1, 'piecestudents': 1, 'curriculum': 2, 'sparing': 1, 'freespirited': 1, 'anticshusband': 1, 'pier': 1, 'importstransgender': 1, 'franciscodutch': 1, 'antidefamation': 1, 'closesmall': 1, 'patrols': 1, 'carsindia': 1, 'mohandas': 1, 'gandhi': 2, 'nucleartesting': 1, 'facilityprayers': 1, 'uncaring': 1, 'universehuffpost': 1, 'somehowhawaii': 1, 'hazard': 1, 'erupting': 2, 'lazecouples': 1, 'fifawhy': 1, 'janitorpoll': 1, 'briefcases': 1, 'cutechuck': 1, 'yoda': 1, 'goodtrump': 1, 'heartchris': 1, 'campaigndesperate': 1, 'masturbate': 1, 'restroomssecret': 1, 'sexting': 2, 'stingman': 1, 'creepkushner': 1, 'issuesgazebo': 1, 'underutilizedcourt': 1, 'harley': 1, 'mamahow': 1, 'biv': 1, 'devoe': 1, 'poison': 2, 'classicman': 1, 'commits': 2, '7season': 1, 'seriesgop': 1, 'carepaula': 1, 'abduls': 1, 'ithelicopter': 1, 'mating': 2, 'beginsfour': 1, 'technologynetwork': 1, 'insights': 1, 'worldclass': 1, 'gurumta': 1, 'walkpresidential': 1, 'fairmuslim': 1, 'berated': 1, 'daydog': 1, 'reformcreative': 1, 'alcoholic': 2, 'lotlook': 1, 'queengreeces': 1, 'craggy': 1, 'ogres': 1, 'gaulles': 1, 'iiracial': 1, 'mds': 1, 'friendliest': 1, 'firedreport': 1, 'celebrityarea': 1, 'sumptuous': 2, '20second': 1, 'massagefiorina': 1, 'announcementabandoned': 1, 'maternal': 1, 'instincts': 2, 'kittensexplosive': 1, 'illusionsguy': 1, 'itholy': 1, 'highwaysharing': 1, 'teamnocancercranky': 1, 'quieted': 1, 'brightly': 1, 'gamelets': 1, 'failshould': 1, 'dining': 2, 'janice': 2, 'emailmike': 1, 'viewspittsburgh': 1, '31': 3, 'stanley': 1, 'cupruth': 1, 'k2the': 1, 'nighthero': 1, 'granddont': 1, 'retiree': 1, 'costsour': 1, 'childbarack': 1, 'emancipation': 1, 'proclamationbarack': 1, 'ithow': 1, 'imani': 1, 'boyettes': 1, 'depressiontrumps': 1, 'saytoddler': 1, 'chokes': 3, 'taiwanesemade': 1, 'toynew': 1, 'compassionnew': 1, 'marijuanaadorable': 1, 'hitch': 1, 'lakeweirdlooking': 1, 'manages': 2, 'picturebrooklyn': 1, 'showcase': 2, 'anniversarydont': 1, 'seasonseditors': 1, '2013': 2, 'sonata': 1, 'carethics': 1, 'colts': 1, 'stuntthree': 1, 'towerbiden': 1, 'vixen': 1, 'ceilingmichelle': 1, 'happinessmandatory': 1, 'impossiblecharleston': 1, 'shootingsisis': 1, 'enormousiran': 1, 'renegotiate': 1, 'deallets': 1, 'redemptionred': 1, 'decimating': 1, 'populations': 1, 'cheddar': 3, 'baylorna': 1, 'meditations': 1, 'blackness': 1, 'americawhy': 2, 'bellynot': 1, 'julian': 1, 'assange': 3, 'nowdonald': 1, 'foe': 1, 'lawsthis': 1, 'accomplish': 2, 'movieair': 1, 'wick': 1, 'pissscented': 1, 'diffuserhillary': 1, 'monthfight': 1, 'infriend': 1, 'graduationsuspect': 1, 'nams': 2, 'saypoll': 1, '68': 2, 'oswald': 1, 'assholestates': 1, 'rancher': 1, 'bundy': 2, 'nevada': 2, 'governordetermined': 1, 'wrestle': 1, 'tigerarea': 1, 'basilicagallant': 1, 'welfareduke': 1, 'arthur': 1, 'albaghdadiopen': 1, 'hilariousscuba': 1, 'diver': 2, 'terrorenglish': 1, 'saysextensive': 1, 'riverprogressive': 1, 'sexlawmakers': 1, 'perilsalma': 1, 'hayek': 1, 'americaadvocates': 1, 'generate': 1, 'schoolsthe': 1, 'holidaysdog': 1, 'homethis': 1, 'moose': 1, 'takeover': 2, 'machinesbath': 1, 'mankindguy': 1, 'itjames': 1, 'foley': 2, 'photojournalist': 1, 'isisbidens': 1, 'wellmatt': 1, 'imaginationjane': 1, 'anaconda': 1, 'reallyseth': 1, 'notsosubtle': 1, 'trumpmelting': 1, 'sheets': 1, 'wobbles': 1, 'axis': 1, 'nasanuclearbomb': 1, 'pentagonman': 1, 'womanirans': 1, 'lifted': 2, 'nextstudy': 1, 'conducting': 3, 'driving13': 1, 'pathwhy': 1, 'importantsunday': 1, 'roudupputting': 1, '2015police': 1, 'dctrump': 1, 'daniel': 6, 'ragsdale': 1, 'homanthis': 1, 'tampons': 1, 'pads': 1, 'evacuees': 1, 'louisianaspawn': 1, 'eyeskanye': 1, 'liposuction': 1, 'addictionchuck': 1, 'comparison': 1, 'putinteen': 1, 'dependencefireworks': 1, 'floridagop': 1, 'obstructionist': 1, 'strategywhy': 1, 'fiorinas': 1, 'follyyour': 1, 'answered2012': 1, 'prius': 1, 'rudimentary': 1, 'systemthis': 1, 'congressmans': 1, 'illustrates': 2, 'obstructionism': 1, 'obamaman': 1, 'beautyyale': 1, 'shooterfda': 1, 'viruswomen': 1, 'rightsapartment': 1, 'broker': 2, 'renthillary': 1, 'movewhy': 1, 'pinterest': 2, 'addictive': 2, 'benefita': 1, 'pavel': 1, 'durovnew': 1, 'sundanceboy': 1, 'friendsthis': 1, 'deed': 1, 'outpaces': 1, 'office17': 1, 'swim': 2, 'essentialsdonut': 1, 'ampringles': 1, 'fallingmiley': 1, 'bangerz': 1, 'tonightfor': 1, 'epperson': 1, 'unmaskedcarly': 1, 'faceoff': 1, 'trumpcommunity': 1, 'sprouts': 1, 'condom': 1, 'wrapper': 1, 'springregulator': 1, 'informationdog': 1, 'tail': 1, 'thispetulant': 1, 'chuckaustin': 1, 'vandalized': 4, 'staywill': 1, 'changedina': 1, 'manzo': 1, 'feudfirefighters': 1, 'mowing': 1, 'lawngood': 1, 'stopstephen': 1, 'murky': 1, 'introconstable': 1, 'eviction': 1, 'girlhabitat': 1, '92yearold': 1, 'laborer': 1, 'sitea': 1, 'ava': 2, 'duvernays': 1, 'wrinkle': 2, 'timeprivileged': 1, 'artiste': 1, 'ohsoprecious': 1, 'moleskine': 1, 'notebookthat': 1, 'closestammys': 1, 'seriouslythis': 1, 'renaissancestyle': 1, 'sculptures': 1, 'twistwho': 1, 'coursesnew': 1, 'stateben': 1, 'undercut': 1, 'directionsiranian': 1, 'rouhani': 1, 'jabs': 1, 'hardliners': 1, 'protestsdemocrats': 1, 'dreamersaustria': 1, 'marriageapartment': 1, 'sweetens': 1, 'tenants': 2, 'furnituregay': 1, 'dunks': 1, 'rimwhat': 1, 'realitywant': 1, 'belonging': 1, 'immigrantscoworker': 1, 'breathes': 2, 'loudly': 3, 'arrhythmically': 1, 'deskwaitingroom': 1, 'hiltonbush': 1, 'nationthe': 2, 'vertical': 1, 'meekermop': 1, 'permanent': 1, 'roomjakrapong': 1, 'kongmalai': 1, 'mentor': 5, 'errorworlds': 1, 'xray': 2, 'upgradenutritionists': 1, 'restaurantparents': 1, 'unhappierfans': 1, 'laureates': 1, 'stockholmhuffpost': 1, '5cia': 1, 'disappearsnorth': 1, 'governors': 3, 'makingsports': 1, 'banquet': 1, 'fightmegachurchgoer': 1, 'devout': 1, 'jumbotronstop': 1, 'movehow': 1, 'amazonwidower': 1, 'gardening': 1, 'ringleonardo': 1, 'screamed': 2, 'cried': 1, 'revenant': 1, 'oscarjeb': 1, 'cialis': 1, 'lota': 1, 'tally': 1, 'scoring': 1, 'drakemeek': 1, 'mill': 1, 'beefluke': 1, 'coffeeurineproof': 1, 'perpsreport': 1, 'downton': 2, 'booklegislators': 1, 'nonissuesformer': 1, 'congresswoman': 3, 'reflects': 5, '90sbreaking': 1, 'plummetingno': 1, 'waitskyrocketingno': 1, 'plummetingjimmy': 1, 'interviewmy': 1, 'huffingtonread': 1, 'plop': 1, 'vday': 1, 'vasekathy': 1, 'familydetroit': 1, 'begs': 3, 'manufacturereport': 1, 'suzy': 1, 'qs': 1, 'lotsburundi': 1, 'neighbor': 5, 'downpeople': 1, 'healthcaregov': 1, 'panickingheroic': 1, 'cancergrandparents': 1, 'bribing': 1, 'nameslongtime': 1, 'libslavesinfo': 1, 'wideawakepatriotcomtrump': 1, 'distract': 1, 'griftagainst': 1, 'divestment': 1, 'differencejohn': 1, 'earnest': 2, 'higherin': 1, 'glimmer': 2, 'crisisill': 1, 'bastards': 3, 'tearyeyed': 3, 'locket': 1, 'containing': 1, 'comeya': 1, 'reclaims': 1, 'officehillary': 1, 'herwould': 1, 'sciencechildren': 1, 'doubletree': 1, 'resort6': 1, 'cadillac': 1, 'taxthe': 1, 'revolttrumpcare': 1, 'legislating': 1, 'crueltyzell': 1, 'offenseyup': 1, 'chicagovenus': 1, 'internettrump': 1, 'yearbilly': 1, 'kidsbabys': 1, 'registered': 1, 'trademarksmom': 1, 'dreamlike': 1, 'kidsi': 1, 'cantjewsponsored': 1, 'trackdangelo': 1, 'russells': 1, 'uh': 1, 'accuratepaintings': 1, 'protestors': 1, 'backthis': 1, 'typical': 2, 'hemophiliacman': 1, 'generator': 1, 'outageprinceton': 1, 'inn': 2, 'bouncersnew': 1, '92grain': 1, 'depletes': 1, 'resourcesthe': 1, 'danish': 1, 'storyman': 2, 'nodding': 1, 'circlethis': 1, 'irregularities': 1, 'fraudnyc': 1, 'puddles': 1, 'fluid': 2, 'ebolabraindead': 1, 'braindead': 1, 'tooamerica': 1, 'revulsion': 1, 'appointmentsicet': 1, 'coco': 1, 'pregnancybiden': 1, 'pisslone': 1, 'sixpack': 2, 'partyingchinese': 1, 'machinescollege': 1, 'cheerleader9': 1, 'artprosecutor': 1, 'walter': 2, 'communityenergy': 1, 'strongkylo': 1, 'sortado': 1, 'parties': 1, 'warntrumps': 1, 'tariffs': 1, 'exempt': 1, 'canada': 3, 'mexiconew': 1, 'mummified': 2, 'closetcan': 1, 'fatan': 1, 'allglowedup': 1, 'wizards': 1, 'waverly': 1, 'weddingretire': 1, 'googlemelinda': 1, 'womenmedia': 1, 'monthspet': 1, 'backstate': 1, 'trooper': 1, 'say3': 1, 'hatecrime': 1, 'killings': 3, 'areaeverything': 1, 'womencongressional': 1, 'reachedconference': 1, 'awesomecreative': 1, 'deskwatch': 1, 'liveemma': 1, 'globeslets': 1, 'raceowls': 1, 'assholesthis': 1, 'obamathemed': 1, 'lifethe': 1, 'askarea': 1, 'envelopeheartbreaking': 1, 'illustrations': 2, 'menwhy': 2, 'scampi': 1, 'week7': 1, 'themed': 1, 'moorebrew': 1, 'loveobamas': 1, 'recordbreaking': 2, 'bankrupting': 2, 'acluart': 1, 'object': 4, 'purchased': 4, 'depothow': 1, 'cultureoverworked': 1, 'vacationfamily': 1, 'exgay': 1, 'therapyhow': 1, 'drinkingresponsible': 1, 'portion': 2, 'withhollywood': 1, 'maintenance': 3, 'plotholesthe': 1, 'huffpollster': 1, 'depends': 1, 'pollpregnancyrelated': 1, 'healthmore': 1, 'conditionsdisgruntled': 1, 'paceestablishment': 1, 'rubiothe': 1, 'pap': 1, 'hpv': 1, 'combinationdefinition': 1, 'fudgetastic': 1, 'stretchedarea': 1, 'reinvent': 1, 'lifefinding': 1, 'authorpublisher': 1, 'adria': 1, 'ciminoheres': 1, 'aloneoklahoma': 1, 'striking': 1, 'carearth': 1, 'invasioncustom': 1, 'fireplace': 1, 'guncouple': 1, 'cages': 1, 'thinggwyneth': 1, 'paltrow': 3, 'creeps': 1, 'goopwhy': 1, '50ophthalmologist': 1, 'surgerydepressed': 1, 'pmpizza': 1, 'pepperonino': 1, 'happensman': 1, 'clocked': 1, '137': 1, 'rpg': 2, 'cutscenesuber': 1, 'datacongratulations': 1, 'kushnerman': 1, 'tentatively': 1, 'badmouthing': 1, 'timeshanghai': 1, 'chinesesecondgrade': 1, 'historianthe': 1, 'stepbystep': 1, 'travelinghow': 1, 'moodnation': 1, 'lessonsturkey': 1, 'netherlandsafter': 1, '46': 1, 'yoko': 2, 'ono': 2, 'credited': 1, 'imagineterrorism': 1, 'storylines': 1, 'dropped4': 1, 'unintentionally': 1, 'parentsamerican': 1, 'salute': 2, 'germany12yearold': 1, 'seductive': 1, 'dressyear': 1, 'worldview': 1, 'yearmcdonalds': 1, 'bereavement': 3, 'pricescarry': 1, 'llc': 1, 'charitynew': 1, 'debunked': 1, 'legalcelebrity': 1, 'fansniper': 1, 'cookerhuffpollster': 1, 'stricter': 1, 'shootingobama': 1, 'compiles': 1, 'shortlist': 1, 'transsexual': 1, 'scaliaisis': 1, 'jihadson': 1, 'cultivate': 1, 'moviesrepublican': 1, 'wifetract': 1, 'influenceshow': 1, 'seriouswho': 1, 'emmys2014': 1, 'centurydetective': 1, 'twoway': 2, 'directlythe': 1, 'accesslocal': 1, 'cleverly': 1, 'alters': 1, 'logowhats': 1, '2016materialistic': 1, 'moneyhumane': 1, 'nippy': 2, 'dachshund': 1, 'sweatshops': 1, 'sweatshopsarea': 1, 'cvsred': 1, 'allyoucaneat': 1, 'krill': 1, 'festcollege': 1, 'convicts': 1, 'nowovercoming': 1, 'selfdoubt': 1, 'tormentormars': 1, 'healthmy': 1, 'storyguy': 1, 'strokesman': 1, 'clinging': 3, 'muted': 1, 'preserverthere': 1, 'allmale': 2, 'panels': 1, 'ubs': 1, 'saysgaza': 1, 'ruinsaccused': 1, 'kayak': 1, 'paddle': 2, 'fiance': 1, 'prosecutorfriend': 1, 'sooneramericans': 1, 'sodadrake': 1, 'doris': 1, 'burkeanother': 1, 'bigscreen': 1, 'bombexplosion': 1, 'signify': 1, 'savingsglass': 1, 'cubicle': 1, 'livingthe': 1, 'freedommother': 1, 'outgrown': 1, 'scooterhow': 1, 'selfesteemperson': 1, 'embalm': 1, 'therepeople': 1, 'minds': 1, 'molten': 2, 'copperstaffer': 1, 'puddle': 2, 'slime': 1, 'cocooned': 1, 'oozethe': 1, 'ofafter': 1, 'kristen': 2, 'chenoweth': 1, 'gloria': 1, 'estefan': 1, 'sir': 1, 'webberfish': 1, 'nowbananas': 1, 'sweep': 2, 'primates': 1, 'awardssentient': 1, 'windowweekend': 1, 'myopia': 1, 'koreaburglar': 1, 'portraitlarry': 1, 'nassars': 1, 'examgetting': 1, 'marriagemiddle': 1, 'palestine': 1, 'weatherrodeo': 1, 'insidethe': 1, 'queerbaitingfetus': 1, 'uterine': 1, 'wallchild': 1, 'immediate': 2, 'waterextravagant': 1, 'blinds': 1, 'venicemakers': 1, 'box66': 1, 'pollstudy': 1, 'onethird': 1, 'obsolete': 1, 'summerold': 1, 'refrigerator': 1, 'anymorerudy': 1, 'nypdserena': 1, 'drivethruthe': 1, 'weekearth': 1, 'orbit': 3, 'fasterteen': 1, 'vaguely': 2, 'pantyhose': 1, 'kmartdinosaur': 1, 'birthjake': 1, 'murdereron': 1, 'girlquestions': 1, 'linger': 1, 'issuesfox': 1, 'outside9': 1, 'assault5': 1, 'kansasoklahoma': 1, 'gameoncehomeless': 1, 'waysubject': 1, 'delicately': 1, 'broachedarea': 1, 'showering': 1, 'nightkevins': 1, 'usreince': 1, 'smiles': 3, 'flipping': 2, '2016john': 1, 'crashnew': 1, 'supervised': 1, 'yorkmore': 1, 'beforedrunken': 1, 'careens': 1, 'internetis': 1, 'fatherhood': 1, 'cheyenne': 1, 'futureronda': 1, 'investigatepoll': 1, 'cataclysmic': 1, 'wiping': 2, 'tidal': 2, 'wavedoublejointed': 1, 'earlypresident': 1, 'approach': 10, 'actfuneral': 1, 'headstone': 1, 'withstand': 3, 'batstampede': 1, '34visit': 1, 'undiscovered': 1, 'showpicture': 1, 'closely': 1, 'resembling': 1, 'deletednonvoter': 1, 'waythese': 1, 'farmfamilys': 1, 'euphemism': 1, 'genitals': 2, 'weirddad': 1, 'dolled': 2, 'lowesmindful': 1, 'reactvisa': 1, 'dolelure': 1, 'disgruntled': 1, 'arbys': 3, 'employeechristie': 1, 'nominationjill': 1, 'schoolspete': 1, 'cardinals': 2, 'consistent': 2, 'policiesleft': 1, 'behindhow': 1, 'winterdecember': 1, 'estimatearea': 1, 'workgop': 1, 'statisticians': 1, 'scenarios': 1, 'gaptotally': 1, 'strolling': 1, 'reasonwhats': 1, 'abramsonresigning': 1, 'thwartedblagojevich': 1, 'birthdayhow': 1, 'conflictmoving': 1, 'grievingreport': 1, '2016black': 1, 'homophobiabirchbox': 1, 'founders': 3, 'receivedthousands': 1, 'lowcarbon': 1, 'dietkylie': 1, 'amateureveryone': 1, 'cancermeet': 1, 'adoptionturns': 1, 'alldarren': 1, 'prosecutorial': 1, 'manipulation': 1, 'systemthe': 1, 'funded': 3, 'steele': 1, 'dossier': 1, 'whatapplebees': 1, 'appetizers': 3, 'specialsheldon': 1, 'hackgetting': 1, 'bushedthe': 1, 'stylekhloe': 1, 'extremethe': 1, '2015i': 1, 'tirelessly': 1, 'addressus': 1, 'mubaraks': 1, 'severance': 1, 'packageobama': 1, 'gazing': 1, 'futuredanny': 1, 'mastersons': 1, 'raped': 2, 'witha': 1, 'cornell': 1, 'frat': 2, 'surprisedthe': 1, 'iowabrutalized': 1, 'toothbrush': 3, 'dieapple': 1, '9aspen': 1, '2015souter': 1, 'roberts': 5, 'birdstgi': 1, 'boozeonmeatwithcheese': 1, 'thingjon': 1, 'stewarts': 1, 'whopping': 2, '22m': 1, 'charityfirefighters': 1, 'elevatorbollywood': 1, 'showstopping': 1, 'numbersreport': 1, 'growthdesperate': 1, 'premieres': 1, 'nova': 1, 'boobs': 1, 'abouncinsnake': 1, 'reptile': 1, 'slithers': 1, 'midflight': 1, 'frightrubio': 1, '6000': 2, 'isisdebunking': 1, 'emotionsscout': 1, 'quicker': 1, 'checkout': 3, 'eastdems': 1, 'refugeesteacher': 1, 'closereport': 1, 'unconsciousbiden': 1, 'huddling': 1, 'scorpions': 1, 'ticketslatina': 1, 'filmed': 3, 'chant': 2, 'backlashgrown': 1, 'karate': 2, 'studiokid': 1, 'solemn': 2, 'cannonballapple': 1, 'obsoletereport': 1, 'lord7': 1, 'datadriven': 2, 'narratives42000': 1, 'homefamily': 1, 'diedpsychiatrists': 1, 'prescribe': 1, 'ketamine': 1, 'antidepressantlocal': 1, 'biblechrissy': 1, 'lickingmalcolmjamal': 1, 'roman': 2, 'polanski': 1, 'controversieswe': 1, 'canyon': 1, 'earthdont': 1, 'officialstrump': 1, 'mummiesjames': 1, 'doublequick': 1, 'exhausting': 2, 'watchthe': 1, 'roadreport': 1, 'debtboy': 1, 'worshipping': 1, '3i': 1, 'prostitute': 1, 'travelerpolka': 1, 'loeffelmacher': 1, 'latelythe': 1, 'hbos': 2, 'palmargaret': 1, 'thatchers': 1, 'scattered': 1, 'marketnorth': 1, 'blowbacka': 1, 'syndromewildebeest': 1, 'stampedethis': 1, 'nighttime': 1, 'wrinkles': 1, 'sayssnuggle': 1, 'marketers': 2, '1834': 1, 'competitorscientists': 1, 'harness': 1, 'intense': 5, 'debatesnew': 1, 'macrowave': 1, 'defrost': 1, 'stufflebron': 1, 'educationgross': 1, 'dayjojo': 1, 'tringle': 1, 'songsmitch': 1, 'inflates': 1, 'dominance': 1, 'maleshulk': 1, 'recipientclinton': 1, 'tooartists': 1, 'decayscientists': 1, 'cloudyour': 1, 'cerebral': 2, 'palsy': 2, 'whatobama': 1, 'busreally': 1, 'hammerheadhow': 1, 'eyeliner': 1, 'kochfunded': 1, 'grouphow': 1, 'homefifth': 1, 'showeredheres': 1, 'diagnoses': 1, 'backmartin': 1, 'lowest': 3, 'castegirl': 1, 'insecure': 1, 'drunkobama': 1, 'victimsno': 1, 'artdna': 1, 'swab': 1, 'vanessa': 1, 'marcottes': 1, 'killinghow': 1, 'kittensthis': 1, 'workersthe': 1, 'fastestgrowing': 2, 'abouttrump': 1, 'fraudlawmaker': 1, 'whitesperot': 1, 'expedition': 1, 'saysactually': 1, 'respects': 1, 'democracyacademy': 1, 'runnersup': 1, 'outlining': 1, 'improvejames': 1, 'manchester': 2, 'knowdiabetes': 1, 'leveling': 1, 'offryan': 1, 'robberygated': 1, 'positionmark': 1, 'fright': 1, 'livestrumps': 1, 'planthe': 1, 'thinkers': 1, 'risea': 1, 'againguard': 1, 'suspiciouslooking': 1, 'mirrorlocal': 1, 'papertowel': 1, 'holenondenominational': 1, 'faithsproposed': 1, 'cry5': 1, 'christiecnbc': 1, 'televisionmonoclewearing': 1, 'barons': 1, 'clenched': 1, 'platformgay': 1, 'rationalitygeorge': 1, 'responsibilitiesyoung': 1, 'tvrachel': 1, 'jokesimply': 1, 'unworkable': 1, 'provision': 3, 'billbeyonce': 1, 'daywatch': 1, 'abbi': 1, 'gagafood': 1, 'fieri': 1, 'rubmysterious': 1, 'disappearsresponsible': 1, 'thoughtful': 1, 'charlie': 4, 'sheen': 2, 'situationthe': 1, 'nationalismthe': 1, 'fraudirans': 1, 'minorities': 2, 'assert': 1, 'takingsmall': 1, 'pornchallenging': 1, 'dawkinswoman': 1, 'plunges': 4, 'cellarembed': 1, 'routines': 1, 'principle': 1, 'principles': 3, 'effectivenesshotel': 1, 'entirety': 1, 'catalogue': 1, 'checkin': 1, 'processon': 1, 'bestdressed': 1, 'showobamacare': 1, 'wonbp': 1, 'multifloor': 1, '1000pump': 1, 'flagship': 2, 'squarerussia': 1, 'jetchris': 1, 'dreaming': 4, '72inch': 2, 'subsales': 1, 'disappointingconversational': 1, 'lamprey': 1, 'partyman': 1, 'regurgitating': 1, 'monologue': 2, 'henry': 2, 'vwhats': 1, '3335krispy': 1, 'sioux': 1, 'relativesa': 1, '1224': 1, 'recordus': 1, 'bases': 1, 'raidtwitter': 1, 'taunt': 2, 'teamman': 1, 'hr': 3, 'eyeing': 1, 'paperwork': 2, 'broughtvanquished': 1, 'gobletchris': 1, 'penns': 1, 'gokoko': 1, 'gorilla': 2, 'offbashar': 1, 'peopleis': 1, 'bookseu': 1, 'relocation': 1, 'goalsbeverly': 1, 'whipple': 1, 'unsung': 2, 'rightsman': 1, 'exwife': 1, 'boardmelania': 1, 'straightens': 1, 'platformjames': 1, 'mermaid': 1, 'spoofreport': 1, 'yolk': 1, 'dripping': 1, 'sandwichchuck': 1, 'goodekid': 1, 'psychicremington': 1, 'shotgun5': 1, 'positivewhy': 1, 'eatinggop': 1, 'ossoff': 1, 'lossoffmitch': 1, 'yearmaybelline': 1, 'injectable': 1, 'internal': 1, 'organsvoters': 1, 'walkerdaddy': 1, 'spotifyregal': 1, 'yearsyou': 1, 'assaultedmasturbatory': 1, 'prose': 3, 'climaxthings': 1, 'themnancy': 1, 'graveyard': 1, 'sucking': 1, 'marrow': 1, 'caylee': 1, 'boneswhy': 1, 'cleaneddon': 1, 'allmilitants': 1, 'hoursserver': 1, 'dessertman': 1, 'disables': 1, 'gridair': 1, 'seatingnations': 1, 'comingexhausted': 1, 'togetherdonald': 1, 'align': 2, 'adviserreport': 1, 'takeshow': 1, 'onehurricane': 1, 'ophelia': 1, 'sheds': 1, 'concernsanta': 1, 'electricrazor': 1, 'crashvegas': 1, 'atm': 2, 'backstudy': 1, 'agobunch': 1, 'hick': 1, 'nobodies': 1, 'toxicwaste': 2, 'exposuresecurity': 1, 'meteor': 1, 'streaking': 1, 'skynew': 1, 'punish': 1, 'workerskourtney': 1, 'reignmalala': 1, 'yousafzai': 1, 'oxfordtexas': 1, 'provider': 1, 'harveycalifornia': 1, 'foundtexas': 1, '3drug': 1, 'molotov': 1, 'cocktailwonder': 1, 'shatters': 1, 'everstudy': 1, 'domesticating': 1, 'upsingapores': 1, 'hijabwearing': 1, 'womanrussian': 1, 'agochristian': 1, 'devotees': 1, 'crucifixionfashion': 1, 'modelsstudy': 1, 'hermits': 1, 'heartsoregon': 1, 'accountsilicon': 1, 'industrythe': 1, 'weektop': 1, 'watchdog': 3, 'officepoll': 1, 'willem': 1, 'dafoehow': 1, 'sitetwitter': 1, 'dishwhen': 1, 'reportertransgender': 1, 'statesuber': 1, 'arbitration': 1, 'harassmentsecret': 1, 'embarrassing': 4, 'congressmanthe': 1, 'vitriol': 1, 'unamericanlocal': 1, 'dancedate': 1, 'rapist': 3, 'tossing': 2, 'mortarboard': 1, 'yourumors': 1, 'extramarital': 1, 'weaponsscientists': 1, 'sleepinducing': 1, 'agribusiness': 1, 'pillaatish': 1, 'taseer': 1, 'sanskrit': 1, 'novelspace': 1, 'endeavour': 1, 'menited': 1, 'sttes': 1, 'toughens': 1, 'umlautsobama': 1, 'wary': 3, 'public26yearold': 1, 'himcollegeaged': 1, 'straws': 1, 'nowclinton': 1, 'outart': 1, 'hustlerrolling': 1, 'talents': 1, 'inserting': 1, 'ovenjay': 1, 'za': 1, 'shopfda': 1, 'tumorsorry': 1, 'scully': 1, 'postseasonkim': 1, '42': 2, 'insta': 1, 'raciest': 1, 'yetpublic': 1, 'showsnelson': 1, 'mandela': 2, 'idiotsteacher': 1, 'popularbritain': 1, 'expresident': 1, 'maldives': 1, 'saysexecutive': 1, 'legislative': 1, 'branches': 1, 'mergeman': 1, 'seenover': 1, '165': 1, 'agreementgun': 1, 'insaneapocalypto': 1, 'comedydepressed': 1, '8second': 1, 'highlightsman': 1, 'problem13': 1, 'liberiaclimatologists': 1, 'spinning': 2, 'outjenna': 1, 'federally': 1, 'wetlands': 1, 'drillingcharles': 1, 'systemchris': 1, 'justiceteens': 1, 'eulogy': 2, 'nickelback': 1, 'lyricsgene': 1, 'wannabe': 1, 'jobgeorgia': 1, 'packagecomic': 1, 'adamant': 1, 'torch': 2, 'flamesan': 1, 'leadershipmccain': 1, 'forbearance': 1, 'moralitygeorge': 1, 'lotterytraveler': 1, 'mexicansmy': 1, 'meplease': 1, 'menold': 1, 'takesverizon': 1, 'comcast': 2, 'assetsman': 1, 'cramped': 1, 'onebedroom': 1, 'friendsarea': 1, 'seniorsalqaedas': 1, '114': 1, 'runthe': 2, 'visionpregnant': 1, 'topsnew': 1, 'foundationted': 1, 'skyrockets': 1, 'sealed': 1, 'maskrepublican': 1, 'falsely': 1, 'partsclinton': 1, 'aliveslowthinking': 1, 'cons': 1, 'riverangry': 1, 'lumberjack': 1, 'breakfastblack': 1, 'racistmy': 1, 'hillaryheres': 1, 'offjames': 1, 'teamaspiring': 1, 'elitist': 2, 'yorksunday': 1, 'roundupsyria': 1, 'benchmark': 1, 'progresscreative': 1, 'asterisk': 1, 'fuckjapanese': 1, 'restrooms': 1, 'phonesjoy': 1, 'pumpedup': 1, 'managerus': 1, 'russiamore': 1, 'listis': 1, 'putinpaul': 1, 'atmosphere': 1, 'plausible': 1, 'deniabilityblack': 1, 'obamaandkids': 1, 'hashtagcrimean': 1, 'timehistorical': 1, 'logtop': 1, 'gametrumps': 1, 'planpalestinian': 1, 'trending': 1, 'twitterhotels': 1, 'hotelsdaily': 1, 'earthstarfucker': 1, 'jobthe': 1, 'multidisciplinary': 1, 'treatmentdonald': 1, 'roughed': 1, 'ralliessingle': 1, 'stickers': 1, 'stationstart': 1, 'chia': 1, 'stylepublicist': 1, 'crafted': 1, 'manic': 1, 'selfabsorbed': 2, 'lunaticmysterious': 1, 'planted': 2, 'sayshow': 2, 'legitwhy': 1, 'jelly': 1, 'shriveled': 1, 'skinwhen': 1, 'colorblind': 1, 'marriagereport': 1, 'dossierdem': 1, 'equalitywhos': 1, 'introducing': 4, 'powellzamboni': 1, 'scandalsupporters': 1, 'gillibrand': 1, 'smearvacationing': 1, 'weekendquaker': 1, 'flavorblasted': 1, 'cakescongress': 1, 'liberties': 2, 'fightrashida': 1, 'rewind': 1, 'videohere': 1, 'breathtaking': 1, 'displaysspiders': 1, 'worldfilm': 1, 'grishamcrane': 1, '360the': 1, 'paradiseoat': 1, 'barleypolice': 1, 'mistaking': 1, 'gunwomens': 1, 'gratuitousgroundbreaking': 1, 'postponed9': 1, 'weekendthe': 1, 'grads': 2, '2015in': 1, 'lovemugger': 1, 'mp3': 1, 'playerkfc': 1, 'weirder': 1, 'norm': 1, 'macdonald': 1, 'col': 1, 'sandersstripper': 1, 'herhedwig': 1, 'musicallookalike': 1, 'disquietinguzo': 1, 'adubas': 1, 'artrevealing': 1, 'attire': 1, 'ogreoverworked': 1, 'unplug': 1, 'ricolondon': 1, 'lifedirector': 1, 'marriagefaith': 1, 'healer': 2, 'miraclepersonal': 1, 'excusesobamas': 1, 'continuity': 1, 'contradictionsarea': 1, 'timetwoyear': 1, 'cellphone': 1, 'knowcalifornia': 1, 'officialshillary': 1, 'wagehog': 1, 'farmland': 1, 'stylekickstarter': 1, 'schoolsnew': 2, 'pregnanciesis': 1, 'geniuses': 1, 'gapsrick': 1, 'flagkenya': 1, 'somalian': 1, 'raiddip': 1, 'goodcaught': 1, 'cliffnurse': 1, 'exorcism': 1, 'inmatemake': 1, 'racismus': 1, 'counterfeit': 3, 'dollarkylie': 1, 'bikinivideo': 1, 'supergirl': 1, 'misogynisticcrucifix': 1, 'testament': 1, 'wealthmost': 1, 'medicaidgeorge': 1, 'housebernie': 1, 'streetsouth': 1, 'heightened': 1, 'celebrationcarlos': 1, 'rosario': 1, 'educates': 1, 'americans10': 1, 'ucsf': 1, 'benioff': 1, 'oaklandgummi': 1, 'tract': 1, 'unharmedj': 1, 'magically': 2, 'personrockin': 1, 'drinkingpixar': 1, 'animator': 1, 'storieshow': 1, 'uscuba': 2, 'policyarm': 1, 'representative': 1, 'sxswhow': 1, 'smalltown': 1, 'financesfull': 1, 'unsliced': 1, 'refreshingtrumpcare': 1, 'scored': 2, 'senatewatch': 1, 'playtrump': 1, 'shames': 1, 'lawsuitamy': 1, 'lawrences': 1, 'herhydraulic': 1, 'crushes': 3, 'holidaysmen': 1, 'dedicationwoman': 1, 'ingrown': 1, 'robber': 3, 'pillaging': 1, 'deadthe': 1, 'dayprotecting': 1, 'newsrooms': 1, 'dialoguefan': 1, 'everfor': 1, 'reckoning': 1, 'enoughopposition': 1, 'lawn7yearold': 1, 'shrek': 2, 'commercialarea': 1, 'crisispandering': 1, 'harmony': 1, 'againabandoning': 1, 'shortsighteddonald': 1, 'vogueaffair': 1, 'livinganxious': 1, 'gina': 2, 'mirrortoddlers': 1, 'artisthow': 1, 'schedulebeyonc': 1, 'coldplay': 1, 'halftime': 3, 'showwither': 1, 'democratsboehner': 1, 'partiesmpaa': 1, 'testamentjenna': 1, 'jameson': 1, 'dropthecover': 1, 'motherhoodgeorge': 1, 'yearseasy': 1, 'vol': 1, 'vent': 1, 'brie': 1, 'jaminauguration': 1, 'happennew': 1, 'visor': 1, 'poorwhat': 1, 'recycling': 1, 'sortsperm': 1, 'storeddorm': 1, 'decorated': 2, 'bottles': 2, 'adderallwarcraft': 1, 'blizzconmnuchin': 1, 'unconstitutional': 2, 'lineitem': 1, 'vetojohn': 1, 'jettisons': 1, 'henchmen': 2, 'airlockhuffpost': 1, 'thatsnl': 1, 'kingdomdejected': 1, 'sheltertrump': 1, 'stormydr': 1, 'fightdaily': 1, 'sustenancenew': 1, 'longstates': 1, 'scramble': 1, 'chipyup': 1, 'ee': 1, 'creamdiego': 1, 'luna': 2, 'sexdemocrats': 1, 'votingryan': 1, 'zinke': 2, 'wildfiredisney': 1, 'pooh': 1, 'cornerargentinian': 1, 'trumpgod': 1, 'monkeysnew': 1, 'caleb': 1, 'selfassured': 1, 'seventhgrader': 1, 'schoolsouth': 1, 'philippines': 2, 'optionssnack': 1, 'unthinkable': 3, 'itselfthat': 1, 'skirmish': 1, 'tweetgore': 1, 'voteturn': 1, 'cubesson': 1, 'spec': 1, 'atticreinvest': 1, 'economiesstandoff': 1, 'fullscale': 3, 'blurbcollege': 1, 'pairing': 3, 'otheron': 1, 'doping': 3, 'probeits': 1, 'supervisor': 2, 'eruptsi': 1, 'communitywhoops': 1, 'lgbtfriendly': 2, 'tooduke': 1, 'themes': 1, 'nuditycolorado': 1, 'phasechristophe': 1, 'michalak': 1, 'superherosex': 1, 'thirdconor': 1, 'walton': 1, 'thingslaverne': 1, 'oitnb': 1, 'issuessuspect': 1, 'deathmobile': 1, 'moneychelsea': 1, 'iceits': 1, 'ivankanew': 1, '1500': 2, 'blastthis': 1, 'hoursnew': 1, 'humansfbi': 1, 'jackettroubling': 1, 'dayromantic': 1, 'girlfriend6': 1, 'climatedenying': 1, 'thanksgivingfrank': 1, 'grandkidsafghanistan': 1, 'morally': 1, 'corrupting': 1, 'warisraeli': 1, 'examvatican': 1, 'signaldomestic': 1, 'grounds': 1, 'mississippiarea': 1, 'sexthe': 1, 'weektoddler': 1, 'yanking': 1, 'wincing': 1, 'sourcesarticle': 1, 'fries': 2, 'todayhealing': 1, 'allegories': 1, 'wizard': 1, 'curtainshape': 1, 'monthblade': 1, '2049': 1, 'originalindecency': 1, 'warshow': 1, 'outremarkable': 1, 'childrenadult': 1, 'hydraulic': 1, 'robotic': 2, 'fisting': 1, 'armthe': 1, 'seasonenjoyment': 1, 'bands': 2, 'americanom': 1, 'funneled': 1, 'statetrump': 1, 'medonald': 1, 'heartbroken': 1, 'peopleto': 1, 'tableorangutans': 1, 'oilshirtless': 1, 'chair49yearold': 1, 'premiddleschool': 1, 'levelssweden': 1, 'notify': 1, 'disturbingly': 3, 'onionsgod': 1, 'damns': 1, 'requestedtwitter': 1, 'bury': 1, 'himman': 1, 'aimlessly': 1, 'yearsarmchair': 1, 'quarterback': 2, 'blitzedimpressive': 1, 'mellencamp': 1, 'japanhillary': 1, 'regain': 1, 'michiganstate': 1, 'champs': 1, 'triumphal': 1, 'archthe': 1, 'canoeing': 1, 'congressmangeorge': 1, 'rousing': 1, 'packpier': 1, 'imports': 1, 'vase': 1, 'invaderprotesters': 1, 'demonstrations': 1, 'acquittal': 1, 'copdream': 1, 'houseplantproduction': 1, 'iceman': 1, 'cometh': 1, 'commercial6': 1, 'house3': 1, 'bales': 1, 'mosesdonald': 1, 'homecomingpatrick': 1, 'dempseys': 1, 'divorcesundried': 1, 'sparrow': 1, 'hosehow': 1, 'spoiled': 2, 'girlsclimate': 1, 'type2': 1, 'diabeteslife': 1, 'encounteras': 1, 'pounceoffice': 1, 'sharpie': 1, 'redesignbathroom': 1, 'shitlifelong': 1, 'newport': 1, 'smoker': 1, 'pleasurenews': 1, '2017time': 1, 'defund': 3, 'industryall': 1, '2miley': 1, 'instagrammed': 1, 'christmas20': 1, 'attacksfacts': 1, 'diagnosis': 3, '91ted': 1, 'amnesty': 2, 'iranrunning': 1, 'buttocks': 1, 'undulate': 1, 'hypnotically': 1, 'sexualitychallenging': 1, 'replay911': 1, 'girders': 1, 'zeromale': 1, 'campingthis': 1, '594foothigh': 1, 'worldasian': 1, 'friendssudafed': 1, 'drill': 1, 'congestion': 1, 'reliefyes': 1, 'technologyhungover': 1, 'inaugural': 3, 'hors': 1, 'doeuvres': 1, 'sashas': 1, 'boxbeyonc': 1, 'scholarships': 1, 'studentsafter': 1, 'astana': 1, '85teens': 1, 'livespuppy': 1, 'fordisgruntled': 1, 'bolton': 4, 'selfthis': 1, 'xxs': 1, 'funnystate': 1, 'authorityfishermen': 1, 'carolinasinfants': 1, 'piling': 3, 'orphanages': 1, 'addressmigrant': 1, 'deadfridge': 1, 'magnet': 2, 'limits7': 1, 'wallpaperbra': 1, 'completefifth': 1, 'momwatch': 1, 'seatsthe': 1, '2014the': 1, 'novelistfamily': 1, 'episode8': 1, 'recommit': 1, 'fading': 2, 'resolutionsfracking': 1, 'employer': 2, 'graduatesgloria': 1, 'steinem': 1, 'rightslocal': 1, '15by20foot': 1, 'roomsmore': 1, 'hebdo': 1, 'symbolic': 3, 'subjectray': 1, 'leppard': 1, 'albumthe': 1, 'busiest': 1, 'travelwhy': 1, 'reformers': 2, 'overlook': 1, 'newarks': 1, 'familiesvideos': 1, 'cedrick': 1, 'chatman': 1, 'releasedreminders': 1, 'threateninggalpagos': 1, 'survival': 2, 'municipalitytony': 1, 'anymoredavid': 1, 'roomhuffpost': 2, 'giddily': 1, 'loserman': 1, 'posters25pound': 1, 'wedged': 1, 'refrigeratorf': 1, '8trey': 1, 'gowdy': 2, 'themselves12': 1, 'seriouslysociologists': 1, 'emergence': 1, 'beforenewsweek': 1, 'nextrumsfeld': 1, 'toner': 1, 'printerhillary': 1, 'crowdstudy': 1, 'not3': 1, 'feminismstate': 1, 'windmill': 1, 'tourdog': 1, 'familytoeing': 1, 'not7': 1, 'lessgwen': 1, 'rossdale': 1, 'painfulheroin': 1, 'pennsylvaniaentire': 1, 'napoleon': 1, 'dynamite': 1, 'pieced': 1, 'quotesbarbara': 1, 'treatmenthappy': 1, 'vaginamotorcyclists': 1, '2wide': 1, 'doingweird': 1, 'longjcrew': 1, 'casualwear': 1, 'mannequinsofficials': 1, 'plastics': 1, 'oceanscollege': 1, 'buildtheater': 1, 'nph': 1, 'hedwig': 1, 'radcliffe': 1, 'impressive': 2, 'cripple': 1, 'immensity': 1, 'isntwhat': 1, 'changerslawsuit': 1, 'saysi': 1, 'girleither': 1, 'ming': 1, 'yuan': 1, 'dynasty': 1, 'mainland': 1, 'chinabackpage': 1, 'pimping': 1, 'lawextending': 1, 'featurehistorians': 1, 'countriesonly': 1, 'jumper': 1, 'specieslawyers': 1, 'scamsfood': 1, 'tailgate': 1, 'everswearing': 1, '1supreme': 1, 'belgiums': 1, 'rulingsi': 1, 'profumi': 1, 'di': 2, 'firenze': 1, 'scents': 1, 'storytotal': 1, 'plantwatch': 1, 'podiumthe': 1, 'aboutthe': 1, 'aboutpakistani': 1, 'friendshipdeclassified': 1, 'arabiagame': 1, 'fingered': 1, 'discussing': 1, 'wintercensus': 1, 'vibesbreathing': 1, 'problemssecondgrader': 1, 'expelled': 1, 'farmeric': 1, 'loads': 1, 'commuteevery': 1, 'lamars': 1, 'billboards': 2, '100man': 1, 'workdaytrevor': 1, 'leadersanother': 1, 'captainsamerica': 1, 'fiery': 2, 'speechgoogles': 1, 'lenseskendrick': 1, 'journalismcuba': 1, 'unionjellyfish': 1, 'honeymoontony': 1, 'commentsjeremy': 1, 'lin': 1, 'lakerscardinal': 1, 'canonized': 1, 'prosecutionkareem': 1, 'abduljabbar': 1, 'commentson': 1, 'gunns': 1, 'providerman': 1, 'questionstrumps': 1, 'switzerland': 1, 'riskfox': 1, 'americawatermelon': 1, 'capital': 6, 'unchallengedthe': 1, 'payfda': 1, 'induce': 1, 'vomiting': 1, 'nowchild': 1, 'slapping': 1, 'bearshocking': 1, 'kemp': 1, 'backfiresgratitude': 1, 'smartwatchrepealing': 1, 'spur': 1, 'pai': 1, 'blaring': 1, 'nationnearly': 1, 'enrolled': 2, 'unthis': 1, 'cyclonesean': 1, 'irreplaceable6': 1, 'juneyour': 1, 'bowlevery': 1, 'drunksubway': 1, 'noxious': 2, 'predatorsdrakes': 1, 'phoneinhibitions': 1, 'beerdave': 1, 'dadgated': 1, 'savagesi': 1, 'breakdownfreaky': 1, 'trigger': 3, 'changefamily': 1, 'giftfda': 1, 'marlinbiden': 1, 'vomit': 3, 'fundamentally': 1, 'americanobama': 1, 'shutarea': 1, 'foolishly': 2, 'entrusted': 1, 'codetrump': 1, 'wouldnations': 1, 'nowjon': 1, 'todayukraines': 1, 'yatseniuk': 1, 'resignswhat': 1, 'staffnuclear': 1, 'alijosh': 1, 'hutcherson': 1, 'moviesman': 1, 'gorgedavid': 1, 'roth': 1, 'jumpmodern': 1, 'prophesied': 1, 'guidethe': 2, 'mcbeal': 3, 'thatthe': 1, 'martyr': 1, 'romero10': 1, 'corners': 1, 'weddingflower': 1, 'fullyseniors': 1, 'decked': 1, 'studentsin': 1, 'journalistsquaker': 1, 'oats': 2, 'canister': 2, 'relabeled': 1, 'playhow': 1, 'changeit': 1, 'selfdoubtwoman': 1, 'zodiac': 1, 'killergrandma': 1, 'defiantly': 1, 'scone': 1, 'graveman': 1, 'resuscitated4yearolds': 1, 'barbie': 1, 'collisionssenators': 1, 'flexible': 1, 'controlstennis': 1, 'gabriella': 1, 'wimbledonscott': 1, 'christianoddsmakers': 1, 'oakland': 1, 'alienation': 1, 'doctorsa': 1, 'fbia': 1, 'patriotic': 2, 'neighborpride': 1, 'linekris': 1, 'soninlaw': 1, 'intentionsskiers': 1, 'suckershow': 1, 'travelingsoda': 1, 'rulesfyi': 1, 'elephantsbiggest': 1, 'collegepregame': 1, 'eveninggop': 1, 'candidateshy': 1, 'excitigroup': 1, 'clique': 1, 'exgoldman': 1, 'housefacebook': 1, 'salesrandy': 1, 'erika': 2, 'marriagebiden': 1, 'gt': 1, 'cola': 1, 'trickortreaterhow': 1, 'conservatismnetanyahu': 1, 'dealpwc': 1, 'mishapgirl': 1, 'flaws': 1, 'seriouslyoregon': 1, 'fateenrique': 1, 'iglesias': 1, 'kournikova': 1, 'twinstrevor': 1, 'selflessly': 1, 'embodies': 2, 'traitstroubling': 1, 'dreamily': 1, 'sliding': 1, 'kissing': 4, 'porch': 2, 'plummets': 2, '78airport': 1, 'screening': 2, '70000': 2, 'yearinside': 1, 'deliberationssyrians': 1, 'worthless': 1, 'goodnightcarbonmonoxide': 1, 'snooze': 1, 'recalledtwitch': 1, 'streamer': 1, 'camerapet': 1, 'costumes': 5, '2015that': 1, 'flaming': 1, 'faker': 1, 'thoughttrumps': 1, 'choicefrom': 1, 'adopted': 1, 'parentingincreased': 1, 'politicswho': 1, 'ouchless': 1, 'adhesive': 1, 'funding10yearold': 1, 'guinness': 3, 'millionairedemocrats': 1, 'rightselderly': 1, 'outlived': 1, 'muchhilary': 1, 'duffs': 2, 'petdavid': 1, 'geist': 1, 'flamethe': 1, 'seafood': 2, 'frytarget': 1, 'syriaimages': 1, 'saturns': 1, 'colorgenetics': 1, 'emphatically': 1, 'truthtense': 1, 'unplayed': 1, 'acoustic': 1, 'guitar': 3, 'wallhow': 1, 'heroinaddicted': 1, 'soberin': 1, 'pollutionpaul': 1, 'halffinished': 1, 'officesalmon': 1, 'grizzly': 2, 'mouthmark': 1, 'advicewhat': 1, 'childrenjust': 1, 'stayinbed': 1, 'declaressushi': 1, 'yourselvessurprise': 1, 'rig': 1, 'electionchildrens': 1, 'dependent': 2, 'teri': 1, 'hatchers': 1, 'parliamentwhy': 1, 'schizophrenic': 1, 'bipolarsimon': 1, 'helberg': 1, 'guild': 1, 'signcory': 1, 'brakes': 2, 'talkbodies': 1, 'beachflower': 1, 'itonly': 1, '2k16': 1, 'trailergun': 1, 'cityto': 1, 'meatloving': 1, 'feminists': 1, 'arrivedwhy': 1, 'hurtfulreport': 1, 'nowmike': 1, 'wantonly': 1, 'darkhow': 1, 'childmercedes': 1, 'ruehl': 1, 'mercedes': 1, 'ruehltraveler': 1, 'listingdonald': 1, 'devilsome': 1, 'suchdrunk': 1, 'honoredcuriosity': 1, 'arrivesa': 1, 'frustrationassad': 1, 'strikeneil': 1, 'armstrongs': 1, 'hobby': 4, 'craptrump': 1, 'attended': 2, 'informantdistracted': 1, 'plumbing': 1, 'problem2': 1, 'grillspreading': 1, 'gospel': 4, 'diplomacylack': 1, 'skews': 1, 'summitfuckbuddy': 1, 'fuckfiancshy': 1, 'personalitymeasles': 1, 'vaccinations6': 1, 'junefbi': 1, 'anthrax': 1, 'stockpileoverweight': 1, 'overweightimmigrant': 1, 'deportednew': 1, 'afterschool': 1, 'minutesconnection': 1, 'friendsabsolutely': 1, 'itexpendables': 1, 'steroids': 1, 'meatcinephiles': 1, 'philipamericas': 1, 'commitment': 1, 'problemrevolutionary': 1, 'toobush': 1, 'warkasich': 1, 'charisma': 1, 'incite': 1, 'frenzynasa': 1, 'voyager1': 1, 'carried': 3, 'systemtrump': 1, 'populism': 2, 'chancerosa': 1, 'depotteen': 1, 'oscarworthy': 1, 'danceastronaut': 1, 'didntcartoon': 1, 'translated': 1, 'seamlessly': 2, 'noodlethe': 1, 'racketman': 1, 'pathetically': 1, 'comb': 1, 'nightfriends': 1, 'bout': 1, 'subside': 1, 'himape': 1, 'threeandahalfminute': 1, 'interruption': 1, 'channelsurfingman': 1, 'observes': 1, 'impregnable': 1, 'perchmcdonalds': 1, 'playplaceantivaxxers': 1, 'uncertaintyfreins': 1, 'injuriesman': 1, 'nomans': 1, 'areasauthorities': 1, 'evacuate': 2, 'bracketschris': 1, 'groupsmudslide': 1, 'partwedding': 1, 'cutstate': 1, 'antisemitism': 2, 'staffhusband': 1, 'stove': 1, 'correctlyresearchers': 1, 'personmythbusters': 1, 'zeuseveryone': 1, 'bararea': 1, 'liebill': 1, 'tweetschiropractor': 1, 'vertebrae': 1, 'sessionvisiting': 1, 'expendables': 1, 'dvdchrist': 1, 'thingsjoe': 1, 'tamp': 1, 'raidsas': 1, 'winwhat': 1, 'firstthe': 1, 'partnershipkarl': 1, 'presidentson': 1, 'conned': 1, 'weekraucous': 1, 'chaffetz': 2, 'trumpyin': 1, 'inroads': 1, 'yangremembering': 1, 'hornernew': 1, 'byproduct': 1, 'fatlottery': 1, 'winnerarea': 1, 'ticket9': 1, 'differentlythe': 1, 'midtown': 1, 'ithursdays': 1, 'banwhy': 2, 'doesntbill': 1, 'prekaustralias': 1, 'scandalsentomologists': 1, 'retract': 1, 'determining': 2, 'clump': 1, 'hairgeorge': 1, 'della': 1, 'issuesrobbin': 1, 'atlantareports': 1, 'manmontana': 1, '60day': 1, 'incest': 1, 'sentenceman': 1, 'everywhere20': 1, 'washingtonamericans': 1, 'doesreport': 1, 'reeklocal': 1, 'craftmatic': 1, 'adjustable': 1, 'accidentstudy': 1, 'washer': 1, 'deathpraise': 1, 'treesus': 1, 'photowomen': 1, 'novemberjessica': 1, 'prekmedia': 1, 'zuckerbergwatch': 1, 'freakshow': 1, 'lifethreatening': 1, 'stuntgeologists': 1, 'intact': 2, 'rockhusky': 1, 'bubbles': 2, 'awwwwidower': 1, 'terriblydoes': 1, 'goodmoms': 1, 'frustrationsguidance': 1, 'selfmutilating': 1, 'deadlinesbus': 1, 'rider': 3, 'awayhigh': 1, 'romeo': 1, 'juliet': 1, 'playman': 1, 'anymorefrom': 1, 'maturityuber': 1, 'schoolcastoff': 1, 'parklatest': 1, 'everyone20': 1, 'americarun': 1, 'presidentdarfur': 1, 'ia': 1, 'shapefeeling': 1, 'changereport': 1, 'amorphous': 1, 'indefinable': 1, '2006god': 1, 'extinctweird': 1, 'dadowner': 1, 'fixes': 1, 'flicker': 1, 'rightnasa': 1, 'planetinequality': 1, 'questionthree': 1, 'electrocutionman': 1, 'loudlywomen': 1, 'ithelp': 1, 'hscc2015spoof': 1, 'wayjulian': 1, 'tattletalecaptain': 1, 'debtdavid': 1, 'publics': 1, 'endurance': 1, 'limitoxiclean': 1, 'stainremoving': 1, 'scissorspatrick': 1, 'distraught': 1, 'hatamericas': 1, 'violentlythese': 1, 'dayuniversal': 1, 'investmentnaacp': 1, 'lineupsrest': 1, 'outbackup': 1, 'spatula': 1, 'happensnew': 1, 'ecofriendly': 3, 'alexandra': 1, 'zissu': 1, 'videoclassmates': 1, 'adoption': 1, 'lovesnar': 1, 'expectwomen': 1, 'networknations': 1, 'stalls': 2, 'bathrooms': 1, 'corresponding': 1, 'sexmore': 1, 'futurebrooke': 1, 'sleep10': 1, 'shook': 1, 'regressive': 2, 'worldauthor': 1, 'holtzclaw': 1, 'lopsided': 1, 'accounteric': 1, 'ossifiescover': 1, 'tailored': 2, 'earlyshould': 1, 'mobile': 1, 'phonemlks': 1, 'agendasevangelical': 1, 'vigil': 1, 'houseno': 1, 'ballet': 1, 'smellthis': 1, '1915comics': 1, '85000th': 1, 'storyrock': 1, 'electronicamonopoly': 1, 'wheelbarrowgingrich': 1, 'electionbotanists': 1, 'botany': 1, 'afterwork': 1, 'drinksstudy': 1, 'outperforming': 1, 'schoolkendall': 1, 'operawhy': 1, 'redfrontier': 1, 'fatalitykate': 1, 'maras': 1, 'angelic': 1, 'itwhy': 1, 'contractor': 1, 'jihadistskeystone': 1, 'hoursfederal': 1, 'foodcrime': 1, 'equivalency': 1, 'ratings45': 1, 'rupturenew': 1, 'ibiza': 1, 'tel': 1, 'avivonly': 1, 'remainwoman': 1, 'waltz': 1, 'officewhat': 1, 'dealman': 1, 'romanticized': 1, 'beansarchaeologists': 1, 'trumpet': 1, 'agesecond': 1, 'consentrush': 1, 'limbaughs': 1, 'endconscience': 1, 'advisorblood': 1, 'drains': 1, 'houseyour': 1, 'retold': 2, 'paintingswhy': 1, 'cutback': 1, 'neededfarmworker': 1, 'cone': 1, 'dayaunt': 1, 'crimenewly': 1, 'deployed': 1, 'kidmall': 1, 'hysterically': 1, 'toddlerclintons': 1, 'sawed': 1, 'offpanthers': 1, 'victimseveryone': 1, 'rockcollege': 1, 'orientationman': 1, 'swells': 1, 'locatorthe': 1, 'vergara': 2, 'opportunitygop': 1, 'outsider': 2, 'onehumans': 1, 'momentus': 2, 'dietary': 1, 'guidelines': 1, 'planetfilmmakers': 1, 'canbys': 1, 'overlong': 1, 'paceddoes': 1, 'portraying': 1, 'disabilitypranksters': 1, 'rename': 1, 'mexicos': 1, 'mapssignature': 1, 'dominates': 1, 'card5': 1, 'wurst': 1, 'barbecuecops': 1, 'poopgangsta': 1, 'shootingwhy': 1, 'all3dprinted': 1, 'naturallyfacebook': 1, 'governmenteconomic': 1, 'distrust': 1, 'rushchris': 1, 'kidsbath': 1, 'lotion': 1, 'tastingsdont': 1, 'educationaltomi': 1, 'lowskilled': 1, 'onin': 2, 'outsize': 1, 'ruleowen': 1, 'factorymaxwell': 1, 'sing': 8, 'ithr': 1, '2651': 1, 'billnight': 1, 'offbalance': 1, 'someonemost': 1, 'point20th': 1, 'greenlights': 1, 'predatoroutofstyle': 1, 'issuesabortion': 1, 'tobaccoindustry': 1, 'executivesthreat': 1, 'downgraded': 1, 'insect': 1, 'ladybugbehind': 1, 'sleepthis': 1, 'peopleindividuals': 1, 'networksweden': 1, 'wakening': 1, 'ecointegrityelderly': 1, 'eaglesdonald': 1, 'misspeaks': 1, 'countrycameron': 1, 'businessbombing': 1, 'syrias': 3, 'ghouta': 1, 'bodieswhippoorwill': 1, '3note': 1, 'lifety': 1, 'cobb': 1, 'enchanted': 1, 'forest': 3, 'toadstoolmoving': 1, 'unicef': 1, 'refugeesdonald': 1, 'sacking': 1, 'republicanscaitlyn': 1, 'inaugurationin': 1, 'hopehouseguest': 1, 'input': 2, '2news': 1, 'uncles': 1, 'filterreport': 1, 'careswhy': 1, 'sudans': 1, 'implosion': 1, 'countryamazing': 1, 'dazzle': 1, 'highflying': 1, 'trapeze': 1, 'routinearea': 1, 'lifetimegreece': 1, 'reassure': 1, 'tensions': 3, 'risewatermelon': 1, 'slushiebush': 1, 'yutu': 1, 'lunar': 1, 'roveracknowledging': 1, 'historyheroes': 1, 'aurorawhat': 1, 'event6000yearold': 1, 'nationfemale': 1, 'quinoneskennel': 1, 'isshes': 1, 'nowstruggling': 1, 'rickles': 1, 'audiencenorth': 1, 'birthdaypat': 1, 'itband': 1, 'ihopchineseamerican': 1, 'espionageterrified': 1, 'tonto': 1, 'makeupbiden': 1, 'implores': 1, 'debatesee': 1, 'tearsreport': 1, 'deathgorgeous': 1, 'magicnations': 1, 'themeless': 1, 'closeshuffpost': 1, '15watch': 1, 'jermaine': 1, 'dupri': 1, 'showinstagram': 1, 'colorediting': 1, 'featureshow': 1, 'colorcomplete': 1, 'nikolai': 1, 'rimskykorsakov': 1, 'inventive': 1, 'orchestratorsteve': 1, 'letterthis': 1, 'shutting': 2, 'waysuicide': 2, 'partyshareware': 1, 'paidexcited': 1, 'juror': 1, 'herreese': 1, 'premierethe': 1, 'pies': 1, 'pinterestmonday': 1, 'unbreakable': 1, 'bulldogsthe': 1, 'councils': 1, 'observationsfourth': 1, '11yearoldswatch': 1, 'brandy': 1, 'sings': 4, 'noticegod': 1, 'shadows': 3, 'sunwoman': 1, 'hungrysenate': 1, '918video': 1, 'childnew': 1, 'inequalitycecily': 1, 'changekid': 1, 'daywhat': 1, 'depressionhes': 1, 'stockbroker': 1, 'excitingtrump': 1, 'hillaryhuma': 1, 'fantasythursdays': 1, 'senatorsreflections': 1, 'uneasy': 3, 'achievements': 1, 'trumping': 1, 'nonsensemoving': 1, 'forwardmeanwhile': 1, 'worldwhat': 1, 'liarfunniest': 1, 'week3weekold': 1, 'jackolantern': 1, 'slightest': 1, 'mushvictorias': 1, '3inch': 1, 'satin': 1, 'bodycircus': 1, 'showamazon': 1, 'xfiles': 1, 'beginsrepublican': 1, 'commercialsmoana': 1, 'weekendkenya': 1, 'dealteachers': 1, 'lounge': 2, 'breakdowns': 1, 'todaysometimes': 1, 'roach': 1, 'infestationkim': 1, 'prankster': 1, 'weekhow': 1, 'goodfsus': 1, 'dalvin': 1, 'battery': 1, 'teamsurprise': 1, 'rncs': 1, 'womencolbert': 1, 'speechless': 1, 'melaniabankrupt': 1, 'dotcom': 1, 'gratershaunted': 1, 'extraspooky': 1, 'interstatea': 1, 'mooc': 1, 'namevatican': 1, 'daughterskourtney': 1, 'disicks': 1, 'staywomen': 1, 'nawal': 1, 'motawi': 2, 'teleworksred': 1, 'dropoff': 1, 'bins': 2, 'convenienceamtrak': 1, 'vermont': 1, 'hospitalcity': 1, 'campaignkalamazoo': 1, 'switched': 3, 'rampagereport': 1, 'timeno': 1, 'happenspope': 1, 'goddeath': 1, 'mudslides': 1, 'foundnavy': 1, 'port': 1, 'starboardwill': 1, 'thunk': 1, 'mooshbarooartists': 1, 'influences': 1, 'ebony': 1, 'magazinesdelicate': 1, 'worldbiden': 1, 'beltwitty': 1, 'bebob': 1, 'trench': 2, 'centuryvoice': 1, 'yelled': 3, 'attrump': 1, 'mulling': 1, 'sayswltz': 1, 'hartfords': 1, 'continuous': 2, 'hitsbigot': 1, 'subhumandonald': 1, 'oustingdog': 1, 'yardpolitician': 1, 'beginsprogress': 1, 'sayscops': 1, 'implicating': 1, 'doghow': 1, 'megahit': 1, 'highlighting': 1, 'futility': 1, 'warjulian': 1, 'pentagondeparting': 1, 'sendoffhanes': 1, 'themterminally': 1, 'serpent': 1, 'renounces': 1, 'eviltransgender': 1, 'themgrizzled': 1, 'weathers': 1, 'showerroadmap': 1, '6garage': 1, 'demo': 2, 'philharmonic': 1, 'showformer': 1, 'ocallaghan': 1, 'gaypaul': 1, 'yorkpamela': 1, 'candidly': 1, 'foundationtrump': 1, 'ovationbirdman': 1, 'astonishing': 2, 'filmfall': 1, 'seestephen': 1, 'speechwriting': 1, 'centerthing': 1, 'happenschipotle': 1, 'heavy': 3, 'sandwichkellyanne': 1, 'unblemished': 1, 'supple': 1, 'tasty': 1, 'skinalarming': 1, 'yearthese': 1, 'crumbletexas': 1, 'lawsuitwhy': 1, 'iceland': 1, 'summerwhere': 1, 'aveagewild': 1, 'rescues': 3, 'boars': 1, 'lakebreakthrough': 1, 'infantsled': 1, 'bulb': 1, 'outlive': 1, 'friendsadjunct': 1, 'todayare': 1, 'tellon': 1, 'centennial': 1, 'scholars': 2, 'wwi': 2, 'todaychurch': 1, 'dearly': 1, 'signohio': 1, 'blaster': 2, 'diplomasamerican': 1, 'faithlisten': 1, 'triphow': 2, 'wilmer': 2, 'valderrama': 2, 'dusk': 1, 'nightsdistrust': 1, 'actionsfirst': 1, 'yearssecondperson': 1, 'enthralling': 1, 'yougop': 1, 'moderates': 1, 'messmara': 1, 'tomskeegans': 1, 'upmore': 1, 'obamacareemeritus': 1, 'weekends': 1, 'canonizationswhat': 1, 'carssarah': 1, 'infasttrack': 1, 'democracyarchaeologists': 1, 'seatsnate': 1, 'projects': 2, 'microscopic': 1, 'electorate': 1, 'petri': 1, 'dishjoe': 1, 'paterno': 2, 'superiors': 1, 'tomorrownation': 1, 'hardthe': 1, 'scandalgop': 1, 'khans': 1, 'commonstephen': 1, 'tapetheyre': 1, 'comingits': 1, 'worserepublicans': 1, 'mick': 1, 'mulvaneys': 1, 'carolinaall': 1, 'daykanye': 1, 'partnership': 1, 'adidas': 1, 'hugejohn': 1, 'carpenter': 1, 'supremacypeople': 1, 'shootingdwts': 1, 'rumbateacher': 1, 'simulate': 1, 'sizesmiling': 1, 'nowdataentry': 1, 'reapplies': 1, 'carmex': 1, '17minute': 1, 'intervalsthe': 1, 'trait': 1, 'needwhite': 1, 'sufficiency': 1, 'racesinternal': 1, 'tophow': 1, 'fooled': 1, 'retweeting': 1, 'benito': 1, 'mussolinimayo': 1, 'endometrial': 1, 'cancermexican': 1, 'lords': 2, 'cartelsemilys': 1, 'congressarea': 1, 'minuteswhy': 1, 'officelindsey': 1, 'racehillary': 1, 'flintept': 1, 'onlyjanet': 1, 'sonoverhydrated': 1, 'blocksgrowing': 1, 'fatearther': 1, 'quintillion': 1, 'overweightsan': 1, 'officemovie': 1, 'bigname': 1, 'gafferjil': 1, 'festivalobama': 1, 'gassed': 1, 'lawnbatman': 1, 'promotion': 2, 'filmgoers': 1, 'withsenate': 1, 'jobsgroceries': 1, 'strategically': 1, 'conveyor': 1, 'spotsominous': 1, 'descending': 2, 'webpage': 1, 'portends': 1, 'autoplaying': 1, 'comereport': 1, 'sincere': 1, 'jokes6': 1, 'homesmeet': 1, 'hero11': 1, 'sweatshirtthe': 1, 'hereclinton': 1, 'onpowerful': 1, 'electionsobama': 1, 'rambling': 2, 'vilsack': 2, 'meetingman': 1, 'miscast': 1, 'fathertaylor': 1, 'roverbanking': 1, 'chickfila': 1, 'bankchadwick': 1, 'boseman': 1, 'speechfrom': 1, 'festivals': 2, 'campuses': 1, 'communitieshard': 1, 'dada': 1, 'notsweatshop': 1, 'irregular': 1, 'nemo': 1, 'sweatshirtthis': 1, 'adoptednorth': 1, 'lawbuoyant': 1, 'displacedtherapy': 1, 'counseling': 1, 'sessionsmegan': 1, 'daydreaming': 1, 'megan': 1, 'nakedfox': 1, 'askreport': 1, 'excusesprince': 1, 'horsegay': 1, 'rounded': 1, 'chechnya': 1, 'disappears': 1, 'sidewalk': 2, 'dayschris': 1, 'rollout': 1, 'terriblepistachio': 1, 'biscotti': 1, 'kirschsoaked': 1, 'dried': 1, 'cherrieswhen': 1, 'upwhy': 1, 'liean': 1, 'tragedycoworker': 1, 'aroundobesitystudy': 1, 'sweet13': 1, 'nranations': 1, 'positionsole': 1, 'showerrecapping': 1, 'advantagewhen': 1, 'sheep': 1, 'mindgrandma': 1, 'grandmajeff': 1, 'weedpoz': 1, 'andor': 1, 'hivaidsdancing': 1, 'thievesobama': 1, 'economyonly': 1, 'statistic': 1, 'announcedexpect': 1, 'nationmanager': 1, 'gogod': 1, 'collaborating': 1, 'partnerliferaft': 1, 'companion': 1, 'juicy': 4, 'steakwhat': 1, 'sisterhoodconsumer': 1, 'defanged': 1, 'trumpdisheartened': 1, 'flightgwen': 1, 'bares': 1, 'amasfar': 1, 'parliamentaverage': 1, 'droppingthese': 1, 'waycurses': 1, 'fistshaking': 1, 'ringleader': 1, 'trainsweating': 1, 'agreefifthgrader': 1, 'notebooka': 1, 'administrationreport': 1, 'quitting': 1, 'scenariosporn': 1, 'directstudy': 1, 'anthemupandcoming': 1, 'twocassette': 1, 'dealsingle': 1, 'eveliebermans': 1, 'overlords': 1, 'displeased9': 1, 'caredenzel': 1, 'colorism': 1, 'problemclassic': 1, 'audiencesnostalgic': 1, 'badgeobama': 1, 'stumping': 1, 'preside': 1, 'statesempty': 1, 'crosshairsbird': 1, 'attenborough': 1, 'courtship': 1, 'dancehow': 1, 'body7yearold': 1, 'cornenvironmentalists': 1, 'loggingbattle': 1, 'wits': 1, 'unwieldy': 1, 'endgamecrude': 1, 'hewn': 1, 'facingbumgarner': 1, 'awaystudy': 1, 'newt': 3, 'gingrichmorgan': 1, 'freemans': 1, 'perfectman': 1, 'fanny': 1, 'satchelstruggling': 1, 'courtsea': 1, 'flipflopsculpture': 1, 'stereotypical': 1, 'pizzerias': 1, 'excellenceobama': 1, 'courtkennedy': 1, 'lobbying': 1, 'fundingunremarkable': 1, 'resembles': 1, 'burt': 1, 'wardthe': 1, 'voiceindianamerican': 1, 'adorableidaho': 1, 'legislature': 1, 'knowlearning': 1, 'homophobiathe': 1, 'peacewhat': 1, 'caredad': 1, 'daynapaquake': 1, 'napastrongrowdy': 1, 'raunchy': 1, 'jetsetting': 1, 'barbados': 1, 'thinkif': 1, 'douglassreport': 1, 'erections': 2, 'timepublic': 1, 'convict': 1, 'rehabilitation': 1, 'beltmassachusetts': 1, 'womancelebrate': 1, 'povertywhy': 1, 'much11': 1, 'ultraproductive': 1, 'differentlyfifty': 1, 'directorlucky': 1, 'wheeled': 1, 'airporttrump': 1, 'baconted': 1, 'deserving': 1, 'candidatewhy': 1, 'elect': 1, 'decadesis': 1, 'burkinitoyota': 1, 'futurethemed': 1, 'campaignlatino': 1, 'immigrationfemale': 1, 'junot': 1, 'virulent': 1, 'misogynynew': 1, 'themejohn': 1, 'campaignhow': 2, 'nonvoters': 1, 'electionwhich': 1, 'roadtrip': 1, 'youbeyonc': 1, 'towhy': 1, 'servicesolivia': 1, 'wildes': 1, 'powerundecided': 1, '3rd': 1, 'conventionpocket': 1, 'electronicbibleverse': 1, 'database': 2, 'covetedthe': 1, 'rulesits': 1, 'transitionrelated': 1, 'careman': 1, 'derives': 1, 'chaindiorama': 1, 'daymeet': 1, 'axel': 1, 'whaleriding': 1, 'stuntmanthe': 1, 'successeverything': 1, 'againprominent': 1, 'fisheries': 1, 'fundingcopycat': 1, 'mimic': 2, 'liquor': 2, '1822macklemore': 1, 'lobbynation': 1, 'menndez': 1, 'murderers': 2, 'starsrunaways': 1, 'neglect': 1, 'schoolhighway': 1, '75mile': 1, 'detourfraternity': 1, 'sensitivity': 1, 'hazingmad': 1, 'om': 1, 'isnational': 2, 'actionslauren': 1, 'redhead': 1, 'nowtransgender': 1, 'yeswatch': 1, 'camerafey': 1, 'distinction': 1, 'homosexualsreport': 1, 'lastinside': 1, 'serialthere': 1, 'outbreakwoman': 1, 'confiding': 1, 'sisterfacebook': 1, 'russialinked': 1, 'congressgina': 1, 'salsa': 1, 'ricoarmy': 1, 'loners': 1, 'militarywhy': 1, 'azores': 1, 'destination': 2, 'travelersgop': 1, 'huntsman': 2, 'againtarget': 1, 'reportvagina': 1, 'oclock': 1, 'shadow6': 1, 'animalsdr': 1, 'buscemi': 1, 'puppymandy': 1, 'shiny': 1, 'awardshow': 1, 'wong': 2, 'foo': 1, 'phenomenonprince': 1, 'weddingfirst': 1, 'atatnew': 1, 'frontman': 3, 'styxa': 1, 'practicalben': 1, 'argo': 1, 'bostonmystery': 1, 'vile': 1, 'graffiti': 2, 'loveno': 1, 'wifewhy': 1, 'primaries60': 1, 'presscrush': 1, 'lasts': 4, 'ridekelly': 1, 'ripa': 1, 'absencewhy': 1, 'happinessemails': 1, 'enbridges': 1, 'clonejohn': 1, 'goodmans': 1, 'spews': 2, 'snlocaseys': 1, 'theatereric': 1, 'bolling': 1, 'coverrick': 1, 'iowaroof': 1, 'capn': 1, 'consumptionruthless': 1, 'mostbill': 1, 'votekris': 1, 'kristofferson': 1, 'lordalligator': 1, 'weekrichie': 1, 'gayer': 2, 'charactermasked': 1, 'unfriending': 1, 'facebookellen': 1, 'simulatorlongtime': 1, 'heckler': 2, 'hecklingus': 1, 'swimmers': 2, 'policenfl': 1, 'tweetsaurora': 1, 'reportchaka': 1, 'collaborates': 1, 'sinatras': 1, 'youflashanimated': 1, 'capturedarea': 1, 'eatsinclair': 1, 'broadcasting': 1, 'videomans': 2, 'followup': 1, 'questionsreal': 1, 'terribly': 1, 'brightwe': 1, 'ustrump': 1, 'watergate': 3, 'nixonian': 1, 'levelswatch': 1, 'juliette': 1, 'emancipatedpatients': 1, 'blindnessobama': 1, 'knowledgewhy': 1, 'azealia': 1, 'faggotjorge': 1, 'ramos': 1, 'incidentfrances': 1, 'namebin': 1, 'theories': 2, 'problemreport': 1, 'desensitized': 1, 'dramanation': 1, 'preoccupation': 1, 'manner': 1, 'processedcheney': 1, 'pointernofilter': 1, 'hairs': 1, 'weekmelissa': 2, 'etheridge': 1, 'linda': 1, 'wallemiraq': 1, 'engulf': 1, 'warnew': 1, 'therapyemails': 1, 'facilitated': 1, 'lng': 1, 'terminals': 1, 'permitsegypt': 1, 'homosexualityalcoholic': 1, 'posthumous': 1, 'sainthood': 1, 'familynorth': 1, 'denuclearization': 1, 'usgreenspan': 1, 'remakecelebrating': 1, 'postpeterson': 1, 'sentencemy': 1, 'mancrush': 1, 'garnerkarate': 1, 'chopadventures': 1, 'instructional': 1, 'coaching': 1, 'teach1': 1, 'failurejudge': 1, 'backcongress': 1, 'backblack': 1, 'rink': 1, 'dooming': 1, 'sharkswhy': 1, 'winthe': 1, 'label': 2, 'activistsempowering': 1, 'worthrecycling': 1, 'economystunning': 1, 'strangest': 1, 'landscapesdavid': 1, 'knowguinness': 1, 'bookglade': 1, 'freshener': 1, 'maskstop': 1, 'referring': 1, 'babysittersgoing': 1, 'safetyariel': 1, 'slayed': 1, 'momentthe': 1, 'prisongovernment': 1, 'illiteracy': 1, 'copied': 1, 'encyclopediabarber': 1, 'feign': 1, 'saystake': 1, 'grexit': 1, 'failurelosingpowerballnumbers': 1, '17th': 1, 'hourseattle': 1, 'debatejayz': 1, 'plungesa': 1, 'historykanye': 1, 'yorkuk': 1, 'auditors': 1, 'iithe': 1, 'tryingmaking': 1, 'morasswriter': 1, 'theall': 1, 'audioshowoff': 1, 'pallbearer': 1, 'himselftrump': 1, 'possibledonald': 1, 'anywaywhy': 1, 'costcomichelle': 1, 'reassigned': 1, 'presidentantislavery': 1, 'genocidal': 1, 'slaver': 1, '20local': 1, 'grammysll': 1, 'rollerballman': 1, 'upfrom': 1, 'atop': 1, 'friendsprotests': 1, 'anniversarytowels': 1, 'claimbrad': 1, 'scampers': 1, 'detecting': 1, 'pagesus': 1, 'uruguayits': 1, 'charlestoninanimate': 1, 'despisedheaven': 1, 'guardrail': 1, 'edgenato': 1, 'aegean': 1, 'smuggling': 1, 'refugeesfamily': 1, 'likesarea': 1, 'fanhotelrestaurant': 1, 'guestjessica': 1, 'hsn': 1, 'scratching': 1, 'headsimpoverished': 1, 'reasonsupreme': 1, 'georgianorth': 1, 'sayspartygoers': 1, 'drunkenly': 1, '4h': 1, 'pledgeveterans': 1, 'corpsstudy': 1, 'harshly': 1, 'further': 2, 'peerstexas': 1, 'trumpstyle': 1, 'crackdowntucker': 1, 'carlson': 1, 'croutonsto': 1, 'moneywatch': 1, 'eyespanicked': 1, '911jeff': 1, 'bridges': 4, 'globesevangelical': 1, 'religioushow': 1, 'acquaintanceslastminute': 1, 'iwho': 1, 'fps': 1, 'frame': 1, 'min': 1, '1850s': 1, 'editionwhite': 1, 'wiretapping': 2, 'wiretappingfictional': 1, 'reallets': 1, 'equitynew': 1, 'dinosaurs': 1, 'cretaceous': 1, 'hospicehow': 1, 'electionshaun': 1, 'minimizing': 1, 'harassmentskeleton': 1, 'abductionnational': 1, 'surfing': 2, 'shitwoman': 1, 'drawnon': 1, 'mouthgreat': 1, 'civilization': 1, 'denarea': 1, 'reasona': 1, 'sgt': 2, 'pepperstyle': 1, '2016african': 1, 'presidentfriend': 1, '8minute': 1, 'delusionalholy': 1, 'anus': 1, 'nsfwthis': 1, 'rulingdotard': 1, 'rocketman': 1, '2017pope': 1, 'pursues': 1, 'rooftopsoffice': 1, '32minute': 1, 'nonstop': 1, 'streakmourners': 1, 'scalialeast': 1, 'avid': 2, 'tasked': 2, 'fetching': 1, 'roundbuilding': 1, 'futureanother': 1, 'parenthoodromney': 1, 'presidencysam': 1, 'understandarea': 1, 'unread5': 1, 'musttry': 1, 'macaroni': 1, 'cheesethe': 1, 'agonate': 1, 'berkus': 1, 'jeremiah': 1, 'brent': 1, 'marriedharper': 1, 'caretaker': 1, 'countrywith': 1, 'strings': 1, 'attachedhow': 1, 'salmonellaeric': 1, 'sandersivana': 1, 'boyshrimp': 1, 'atthis': 1, 'irannew': 1, 'unread': 2, 'bronx': 2, 'islandmichelle': 1, 'reasonsexual': 1, 'fluidity': 1, 'kacy': 1, 'womanfather': 1, 'reportbetsy': 1, 'kidscuba': 1, 'womenmichael': 1, 'plasticthe': 1, 'helpconscious': 1, 'decisionthe': 1, 'changeman': 1, 'halfassing': 1, 'thingsweekend': 1, 'civilizations': 1, 'withinafter': 1, 'iconmost': 1, 'bloggersnew': 1, 'chargebrazils': 1, 'bumbum': 1, 'nsfwnorth': 1, 'theaterliberals': 1, 'inexperience': 1, 'appointeesstephen': 1, 'horseunbelted': 1, 'frontthe': 1, 'churchill': 1, 'domanchester': 1, 'religionarea': 1, 'whatcnns': 1, 'corey': 1, 'lewandowski': 1, 'reignites': 1, 'longdebunked': 1, 'theoryiran': 1, 'crackdowntop': 1, 'programlung': 1, 'scandriver': 1, 'error': 3, 'crashmasturbating': 1, 'flay': 1, 'southwestern': 1, 'bufferingwho': 1, 'sugary': 1, 'obesityblack': 1, 'clintonquentin': 1, 'tarantino': 1, 'uma': 1, 'thurman': 2, 'lifespineless': 1, 'caves': 1, 'children7': 1, '50twitterverse': 1, 'verse': 1, 'tweetnetflix': 1, 'criers': 1, 'streamingdonald': 1, 'conventionnorma': 1, 'operathe': 1, 'founded': 3, 'mafia': 1, 'protectionlatest': 1, 'postitafter': 1, 'peddling': 1, 'arabiatrumpismo': 1, 'fascismtrumps': 1, 'challengemarilyn': 1, 'monroes': 1, 'unbelievable': 2, 'sumthese': 1, 'deathsam': 1, 'housechaps': 1, 'unnecessarystandard': 1, 'deviation': 1, 'perverted': 1, 'statistician15000': 1, 'somewhere30yearold': 1, 'moneylocal': 1, 'dullard': 1, 'opts': 3, 'vocational': 1, 'schoolman': 1, 'finelove': 1, 'hurts': 1, 'mature': 2, 'surmise': 1, 'rejection': 2, 'heartachea': 1, 'childless': 1, 'outcharity': 1, 'shkrelis': 1, 'donationheres': 1, 'dateposter': 1, 'vandal': 1, 'phallus': 1, 'periodflu': 1, 'microchipsfederal': 1, 'documentsspeed': 1, 'neapolitanroller': 1, 'artistic': 3, 'sullied': 1, 'tieintime': 1, 'traveler': 1, '2008': 2, 'ecigarettedonald': 1, 'awaylester': 1, 'holt': 2, 'chosemillions': 1, 'brookings': 1, 'tanker': 2, 'aground': 1, 'crimea': 2, 'coastobama': 1, 'allegation': 1, 'wiretapped': 1, 'brags': 1, 'didntirish': 1, 'blurtry': 1, 'kpop': 1, 'bts': 1, 'seriesour': 1, 'wearinggreyhound': 1, 'tiredmiddle': 1, 'learningtrump': 1, 'drillinghard': 1, 'donecouple': 1, 'thisman': 1, 'himselfknowing': 1, 'hillarywedding': 1, 'buttercup': 1, 'requestcindy': 1, 'gallop': 1, 'impacting': 2, 'worldpope': 1, 'reverses': 2, 'oreosgay': 1, 'ostracizedheadon': 1, 'collision': 1, 'fordsolidarity': 1, 'torched': 1, 'churchesglaad': 1, 'gaybritney': 1, 'coolest': 2, 'momindustrious': 1, 'otters': 1, 'spillswhere': 1, 'iconhow': 1, 'martyrtom': 1, 'stuntbob': 1, 'heavenavoid': 1, 'seano': 1, 'meala': 1, 'paddys': 1, 'existsits': 1, 'marijuanaautism': 1, 'coliseumhillary': 1, 'chekhovian': 1, 'monologueoutside': 1, 'postprimary': 1, 'carolinathis': 1, 'signifierman': 1, 'sighting': 1, 'careergod': 1, '87th': 1, 'holiest': 1, 'earthphotos': 1, 'yearssuspected': 1, 'investigationwhat': 1, 'personman': 1, 'classilana': 1, 'glazer': 1, 'harassers': 1, 'lick': 1, 'ballsnational': 1, 'dusted': 1, 'off10': 1, 'illegalseventhgraders': 1, 'hysloppolice': 1, 'militia': 1, 'standoffnew': 1, 'swiss': 2, 'risksniall': 1, 'horan': 1, 'ginuwines': 1, 'treatdemocrats': 1, 'insist': 1, 'nomineeel': 1, 'hippo': 2, 'prosecutors': 2, 'sayan': 1, 'examines': 1, 'internetbetsy': 1, 'trumpist': 1, 'movementobama': 1, 'billin': 1, 'daysflorida': 1, 'timeswomen': 1, 'tougher': 1, 'whystruggling': 1, 'limits': 4, 'walgreen': 1, 'swissarea': 1, 'livingamber': 1, 'tamblyns': 1, 'poems': 1, 'illuminate': 2, 'actressesnews': 1, '28': 3, 'forjohn': 1, 'irankevin': 1, 'pierrelouis': 1, 'experienceserial': 1, 'appreciation': 2, 'albert': 1, 'fishare': 1, 'hackathons': 1, 'businessis': 1, 'negotiate': 2, 'denuclearizationsingle': 1, 'daughterentirety': 1, 'plummerscambridge': 1, 'awkwardaunt': 1, 'flo': 1, 'videoexperts': 1, 'awayaverage': 1, 'daypants': 1, 'convey': 2, 'cantindependent': 1, 'fuckers': 2, 'themim': 1, 'lawst': 1, 'protesters8yearold': 1, 'firefight': 2, 'afghanistanwomen': 1, 'clothescops': 1, 'unlock': 1, 'fingerdan': 1, 'aykroydsthis': 1, 'watsonparents': 1, 'parentingstudy': 1, 'gamecity': 1, 'victimswhy': 1, 'discriminated': 2, 'againstbrad': 1, 'bodygop': 1, 'fueled': 1, 'crisisdisgusting': 1, 'interacting': 1, 'publicthe': 1, 'transformation': 3, 'manmake': 1, 'reform3': 1, 'mouthbilly': 1, 'showpope': 1, 'franciss': 1, 'depicted': 1, 'bookstudy': 1, 'boywhat': 1, 'trumpwho': 1, 'yazidibetty': 1, 'friedan': 1, 'secondclass': 1, 'stampwatch': 1, 'mirjana': 1, 'lucicbaroni': 1, 'motivational': 2, 'speechthese': 1, 'carpetsame': 1, 'thang': 1, 'strangnationwide': 1, 'marches': 1, 'returnssupreme': 1, 'opinionshillary': 1, 'debatefood': 1, 'souvenir': 1, 'homejack': 1, '20american': 1, 'philanthropydemocrats': 1, 'ceding': 1, 'electionrick': 1, 'candidatedespite': 1, 'shortblackbacked': 1, 'jackals': 1, 'measureup': 1, 'cholera': 1, 'whorip': 1, 'hellerman': 1, 'weavers': 1, 'influentialreport': 1, 'wonfamily': 1, 'snowstormwill': 1, 'immigrationdrexel': 1, 'satiricalwhite': 1, 'nationalists': 2, 'yearstruthful': 1, 'forgetful': 1, 'editionmy': 1, 'artistreport': 1, 'monththis': 1, 'roald': 1, 'dahl': 1, 'truepolice': 1, 'skeletonretailers': 1, 'holidayseverything': 1, 'promotionrobert': 1, 'productivitytortured': 1, 'ugandan': 1, 'uganda': 1, 'oilnew': 1, 'atmospherewhats': 1, 'pedicure': 1, 'tubsnunes': 1, 'contextless': 1, 'selectivelyedited': 1, 'truthcanadian': 1, 'probing': 1, 'terrorismpregnant': 1, 'glows': 3, 'rageyale': 1, 'humanists': 2, 'obeliskjupiter': 1, 'planetthings': 1, 'rubiowhat': 1, 'digitalhealth': 1, 'hivaidsman': 1, 'breathe': 2, 'sayheaven': 1, 'breaknew': 2, 'practicepaul': 1, '10minute': 1, 'pained': 1, 'expressionstupid': 1, '16yearold': 1, 'adderall': 2, 'healthmannequins': 1, 'loveaweinspiring': 1, 'servicewomen': 1, 'runwaychristina': 1, 'premierehurricane': 1, 'problemthe': 1, 'listed': 1, 'endangeredcalifornias': 1, 'opportunitydress': 1, 'crackedliberals': 1, 'ourselves': 3, 'mirrorjoe': 1, 'billslena': 1, 'psychiatric': 1, 'medicationkasich': 1, 'runeveryone': 1, 'schoolyard': 1, 'allegeaisle': 1, 'stagelocal': 1, 'todaywatch': 1, 'coulier': 1, 'housequaaludes': 1, 'quaaludetaking': 1, 'journalistbaby': 1, 'christmassyrian': 1, 'antirefugee': 1, 'rhetoric': 3, 'wordsmy': 1, 'psych': 2, 'wardacademy': 1, 'troublingarea': 1, 'womanreceipt': 1, 'brazenly': 1, 'permissiona': 1, 'booklisten': 1, 'broadcastanother': 1, 'destroyedtheres': 1, 'verdicthelping': 1, 'speciescutting': 1, 'politwoops': 1, 'interestsupreme': 1, 'casewhats': 1, 'beansreince': 1, 'hackednew': 1, 'existeddefense': 1, 'guywhile': 1, 'wheelsarea': 1, 'grandkidsdisabled': 1, 'saysbrunch': 1, 'livened': 1, 'trios': 1, 'breakupcar': 1, 'urgent': 3, 'midstreet': 1, 'goingthey': 1, 'miamis': 1, '2014heres': 1, 'weekjulianne': 1, 'custom': 1, 'chanelpen': 1, 'foeyoutuber': 1, 'sealant': 1, 'moronhow': 1, 'holidaysoutback': 1, 'theoryadvocates': 1, 'facilityan': 1, 'solutionpentagon': 1, 'surround': 1, 'decoysthe': 1, 'bittersweet': 2, 'divorcefrom': 1, 'bulldogs': 1, 'mathis': 1, 'mancini': 1, 'anson': 1, 'rmhromney': 1, 'raidthe': 1, '3how': 1, 'jobbringing': 1, 'westconsumers': 1, 'tabletop': 1, 'gamesobama': 1, 'whispered': 1, 'untelevised': 1, 'controlguy': 1, 'parlor': 2, 'slicearea': 1, 'roofer': 1, 'badmouths': 1, 'collegedeath': 1, 'dielack': 1, 'russiagap': 1, 'closures': 1, 'secondyou': 1, 'obnoxiousbernie': 1, 'recanvassbush': 1, 'soonweird': 1, 'stancesobama': 1, 'quarterly': 1, 'bonusemployees': 1, 'ramble': 1, 'deskgame': 1, 'tyrion': 1, 'betray': 1, 'daenerysthe': 1, 'daymariah': 1, 'tvpolar': 1, 'speciesresident': 1, 'gentrificationman': 1, 'mucus': 1, 'seasonradiator': 1, 'clank': 1, '332': 1, 'amtrevor': 1, 'adpepsico': 1, '300': 6, 'lemonlime': 1, 'campaignus': 2, 'okinawa': 1, '1972trumps': 1, 'nasas': 1, 'fundinggeithner': 1, 'domea': 1, 'multiethnic': 2, 'eastersean': 1, 'humiliationsbest': 1, 'offtheradar': 1, 'spotswho': 1, 'nobody4': 1, 'marketrush': 1, 'limbaugh': 1, 'rantmichelle': 1, 'specialswhat': 1, 'octoberhero': 1, 'heropartygoer': 1, 'aroundwhy': 1, 'showabc': 1, 'corrects': 2, 'stocksbestlaid': 1, 'cheeserelatedblack': 1, 'micchina': 1, 'lunchamerican': 1, 'torturous': 1, 'teasers': 1, '6infomercial': 1, 'skeptical': 1, 'product11': 1, 'doodles': 1, 'heartbreaksamantha': 1, 'billand': 1, 'aredad': 1, 'advicejanuary': 1, 'violet': 1, 'greywatch': 1, 'stationofficials': 1, 'eclipsedirty': 1, 'slush': 1, 'winterukraine': 1, 'insignia': 1, 'symbolismindonesias': 1, 'futurestudy': 1, 'asleepinternet': 1, 'weekblagojevich': 1, 'startedwhy': 1, 'policing': 2, 'problematicdepositions': 1, 'exaggerate': 1, 'insultnew': 1, 'croft': 1, 'crofts': 1, 'pennycan': 1, 'groove': 1, 'backcamila': 1, 'cabellos': 1, 'powerarea': 1, 'allbody': 1, 'scansthe': 1, 'bundle': 1, 'listway': 1, 'bronchitis': 1, 'researchthe': 1, 'streetmarkpaul': 1, 'gosselaar': 1, 'authored': 1, 'imdb': 1, 'triviaprotesters': 1, 'buildingthis': 1, 'impressivean': 1, 'astronauts': 1, 'effectiveastronaut': 1, 'nasafewer': 1, 'documentariesthat': 1, 'kniferelated': 1, 'incidentgwyneth': 1, 'testifies': 1, 'stalking': 1, 'yearsfather': 1, 'awakeninglandmarks': 1, 'daylittle': 1, 'whistlenewly': 1, 'hospitalizationsing': 1, 'worldthese': 1, 'loneliness': 1, 'insomniawhen': 1, 'platesone': 1, 'businesswhat': 1, 'dressed5': 1, 'nondisclosure': 1, 'agreementscientific': 1, 'painfulsylvester': 1, 'stallones': 1, 'bonafide': 1, 'modelmom': 1, 'returnedclever': 1, 'bedhogging': 1, 'strugglesveterans': 1, 'outlook': 1, 'outdoorswow': 1, 'groans': 1, 'norways': 1, 'kontiki': 1, 'nominationmillions': 1, 'mentalistfall': 1, 'internship': 2, 'internshipgop': 1, 'unseat': 1, 'samethe': 1, 'sobering': 1, 'burrellour': 1, 'withinnavy': 1, 'satans': 1, 'mediabeanie': 1, '226': 1, 'eyesambitious': 1, 'saucerfetish': 1, 'drownaquarium': 1, 'environmentman': 1, 'questionsunder': 1, 'hatea': 1, 'pastoral': 1, 'frustrating': 1, 'millennialsthirdperson': 1, 'omniscient': 1, 'endingus': 1, 'electionking': 1, 'offobese': 1, 'impaled': 2, 'wickerchair': 1, 'disasterdisturbingly': 1, 'emanates': 2, 'costumetrump': 1, 'yearsprocrastinating': 1, 'rosaries': 1, 'behindcool': 1, 'ashtray': 1, 'foundtrump': 1, 'flamescheck': 1, 'decathlontrump': 1, 'shutdowni': 1, 'pranked': 1, 'muhammed': 1, 'alichloe': 1, 'chillax': 1, 'gnar': 1, 'gnarthe': 1, 'neutralitycompany': 1, 'complainfor': 1, 'clientticketed': 1, 'motorist': 1, 'neededdonald': 1, 'millionschild': 1, 'muchhundreds': 1, 'excop': 3, 'acquitted': 3, '2011why': 1, 'emailsbryan': 1, 'dismiss': 1, 'lawsuitman': 1, 'cokes': 1, 'gettingman': 1, '14month': 1, 'salesflight': 1, 'arereport': 1, 'fiber': 1, 'optics': 1, 'thingencouraging': 1, 'discouraged': 1, 'readervain': 1, 'galpagos': 1, '90miley': 1, 'breastsgod': 1, 'phasing': 1, 'trinityman': 1, 'changeim': 1, 'epidemicsreport': 1, 'impersonatorfamily': 1, 'sonroyal': 1, 'ultrasound': 2, 'imageshirtless': 1, 'facepennsylvania': 1, 'done2': 1, 'themfreddie': 1, 'greenshameless': 1, 'nurseentire': 1, 'shaving': 1, 'soldierstrump': 1, 'isheres': 1, 'repeathow': 1, 'saltroasted': 1, 'salsacockroach': 1, 'superfood': 1, 'sciencearea': 1, 'ethic': 1, 'indepth': 2, 'lawoxford': 1, 'nowsmiling': 1, 'willie': 2, 'womennevertheless': 1, 'persisted': 1, 'higha': 1, 'evercryptozoologist': 1, 'againpolice': 1, 'officerinvolved': 1, 'smoothlysenate': 1, 'appropriations': 2, 'hopesblissful': 1, 'commemorated': 1, '910': 1, 'anniversaryrecent': 1, 'upvicious': 1, 'maneating': 1, 'carnivores': 2, 'arcticzoomed': 1, 'alohahuffpost': 1, 'wardpetsmart': 1, 'enclosures': 1, 'day11': 1, 'tv30': 1, 'tinder': 2, 'datemillionaire': 1, 'uppermiddle': 1, 'classfilm': 1, 'brownstone': 1, 'jobgop': 1, 'conventionchrissy': 1, 'metz': 1, 'stepfather': 1, 'teenheavy': 1, 'universitys': 2, 'canteen': 1, 'minutesjames': 1, 'minutesjustin': 1, 'startmonkfish': 1, 'monkfish': 1, 'rageseth': 1, 'rogen': 2, 'dmsnew': 1, 'problemsegypt': 1, 'parliamentary': 1, 'electionsfork': 1, 'manufacturer': 1, 'tine': 1, 'mouthfulsme': 1, 'diedoverpopulation': 1, 'marketsadeles': 1, 'bodyguard': 1, 'meltdownlovebird': 1, 'windshield': 2, 'wipers': 3, 'raindonald': 1, 'fourlegged': 1, 'stoolobama': 1, 'shootingmy': 1, 'herjustice': 1, 'progressbold': 1, 'condomscientists': 1, 'posit': 1, 'weekendrare': 1, 'saleauthors': 1, 'rallying': 1, 'langston': 1, 'harlem': 1, 'everyonereport': 1, 'blackthe': 1, 'amaranth': 1, 'vegetableonly': 1, 'plunging': 1, 'weddingread': 1, 'hearings': 3, 'picksfool': 1, 'oncearea': 1, 'partynational': 1, 'personwatch': 1, 'hadid': 1, 'shoebernie': 1, 'historywill': 1, 'shelfpope': 1, 'wallswant': 1, 'likegay': 1, 'thingjohn': 1, 'familymost': 1, 'victorytownsperson': 1, 'barrelberkeley': 1, 'altright': 1, 'againstudents': 1, 'humongous': 2, 'whiteboardgrandchild': 1, 'projectbill': 1, 'cliven': 1, 'withcaged': 1, 'saddam': 2, 'balllargemouth': 1, 'largemouth': 1, 'sassairlines': 1, 'carryon': 2, 'rulesambitious': 1, 'companyobamacare': 1, 'enrollees': 1, 'await': 1, 'coveragereallife': 1, 'flume': 1, 'familyivanka': 1, 'womennotes': 1, 'midwestern': 1, 'waitsyellowstone': 1, 'programdnc': 1, 'keynote': 3, 'latinohow': 1, 'hairvisa': 1, 'salarywalton': 1, 'schoolshow': 1, 'locationpool': 1, 'borrowthe': 1, 'resistdeath': 1, 'mourning': 2, 'holidaynato': 1, 'slideredstate': 1, 'leon': 1, 'erick': 1, 'erickson': 1, 'exitwhy': 1, 'dreamingpuerto': 1, 'themwoman': 1, 'emailspresident': 1, 'selfcenteredshelling': 1, 'caribbeans': 1, 'allure': 1, 'vessel': 1, 'crossed': 1, 'watersrupert': 1, 'addressing': 3, 'exoskeletongop': 1, 'governmentfunded': 1, 'violencefda': 1, 'foodname': 1, 'clearerwomen': 1, 'minson': 1, 'systemsmissed': 1, 'unlovablelive': 1, 'algeriafrustrated': 1, 'nearest': 1, 'clinicword': 1, 'miniwheats': 1, 'impossiblemagna': 1, 'carta': 1, 'taleis': 1, 'coffeebernie': 1, 'fdaben': 1, 'winscurb': 1, 'trumpeighthgrader': 1, 'yearstates': 1, 'hightech': 2, 'jobstoday': 1, 'halloweenthe': 1, 'lightsthe': 1, 'manziel': 1, 'investigationharry': 1, 'datesto': 1, 'swamp': 2, 'dilute': 1, 'floodroad': 1, 'cardick': 1, 'wipe': 1, 'democratspeople': 1, 'inherently': 1, 'muttersthis': 1, 'critter': 1, 'dayjesse': 1, 'awardsgreyhound': 1, 'drivewaynike': 1, 'refugees5': 1, 'fallrick': 1, 'statesheres': 1, 'medusa': 1, 'nebula': 1, 'prettier': 1, 'namesakeobama': 1, 'blanks': 1, 'ineffectually': 1, 'nowlowbudget': 1, 'panders': 1, 'featuremario': 1, 'legacyred': 1, 'mechanical': 1, 'jumbo': 1, 'ridean': 1, 'disciplineits': 1, 'oxfordplan': 1, 'startmom': 1, 'mealastronomers': 1, 'yetgingrich': 1, 'jew': 1, 'trail12': 1, 'useful': 3, 'needsguys': 1, 'nonguysreggie': 1, 'foxs': 1, 'atheletes': 1, 'talkpoll': 1, 'celestial': 1, 'utopia': 1, 'pleasuremachines': 1, 'evolvewillie': 1, 'aidgeneral': 1, 'kmart': 1, 'lotpodcaster': 1, 'episodehow': 1, 'earlylife': 1, 'cratemeasuring': 1, 'ringmike': 1, 'childrengeorge': 1, 'peoplepolitical': 1, 'blogger': 3, 'weeksthis': 1, 'formedheres': 1, 'cosbys': 2, 'teamwhat': 1, 'oprah': 1, 'truejustin': 1, 'goto': 1, 'amazingwhy': 1, 'womencampaign': 1, 'debatesaustralian': 1, 'omelet': 1, 'cookeramerican': 1, 'idol': 1, 'fametrump': 1, 'impressivehow': 1, 'agingfor': 1, 'miranda10': 1, 'readernew': 1, 'core': 3, 'habitable': 1, '2060roy': 1, 'religionprogressive': 1, 'challenger': 1, 'birmingham': 1, 'frontline': 1, 'policiesman': 1, 'sayspirit': 1, 'streetbiden': 1, 'copresents': 1, 'starlet': 1, 'shyla': 1, 'stylez': 1, 'avn': 1, 'showdid': 1, 'rabbiunhappy': 1, 'excerpt': 1, 'shitfaced': 1, 'musings': 1, 'drunkfuck': 1, 'scentedthis': 1, 'cleft': 1, 'palate': 1, 'lifemoderation': 1, 'modernity': 1, 'moroccan': 1, 'islam97': 1, 'camaros': 2, 'mustachesbehold': 1, 'tatted': 1, 'malikstudent': 1, 'papersthis': 1, 'wizardsana': 1, 'marginalized': 2, 'chancethe': 2, 'gameali': 1, 'kriegers': 1, 'setbacks': 1, 'stridefamily': 1, 'neighborhoodhillary': 1, 'cruises': 2, 'primaryblearyeyed': 1, 'cosmopolitan': 1, 'manyamaha': 1, 'skis': 1, 'alto': 1, 'saxophones': 1, 'snowmobiles': 1, 'generators': 1, 'scooters': 1, 'cartsstudy': 1, 'altering': 1, 'aspirationscommunity': 1, 'eyesoreif': 1, 'whynation': 1, 'teamsretired': 1, 'philippine': 2, 'rodrigo': 1, 'suspectsassad': 1, 'timefake': 1, 'tara': 2, 'conans': 1, 'callsroom': 1, 'scanned': 1, 'ebaynew': 1, 'togetherguatemalan': 1, 'alertsouth': 1, 'trailerrussian': 1, 'medallist': 1, 'violation': 1, 'reportemmanuelle': 1, 'seigner': 1, 'intervieweverything': 1, 'happinesshow': 1, 'dial': 1, 'downtwo': 1, 'groupbizarre': 1, 'assemblage': 1, 'pocketsget': 1, 'edwardshere': 1, 'agencyinside': 1, 'allay': 1, 'reviewnbc': 1, 'svu': 1, 'electiontwitterverse': 1, 'xs': 1, 'jokesman': 1, 'cartonlocal': 1, 'onjohn': 1, 'survivorsthe': 1, 'lifefrom': 1, 'turret': 1, 'gunner': 1, 'guerilla': 1, 'fighterdaily': 1, 'mantrajunk': 1, 'letterbox': 1, 'deliveredman': 1, 'confinement': 1, 'enoughmaximum': 1, 'strollers': 1, '8firefighter': 1, 'newbornprima': 1, 'donna': 2, 'halffull': 1, 'theaternursinghome': 1, 'captivitystuffed': 1, 'shelffbi': 1, 'picturedisc': 1, 'herb': 2, 'alperts': 1, 'whipped': 1, 'delightsmanaging': 1, 'madness': 2, 'eastfbi': 1, 'counterterrorists': 1, 'downplaying': 1, 'bridgeisrael': 1, 'homeroomwatch': 1, 'nicki': 1, 'minajs': 1, 'snlcongress': 1, 'resurrect': 1, 'earmarks': 1, 'awayparis': 1, 'salah': 1, 'abdeslam': 1, 'belgiumthis': 1, 'criminalize': 2, 'drowsy': 1, 'state911': 1, 'answerswill': 1, 'dozensman': 1, 'venture': 2, 'confirmedthe': 1, 'mergerreplacement': 1, 'socialite': 1, 'cunt': 1, 'castfrugal': 1, 'porn2': 1, 'albany': 1, 'incidentdewey': 1, 'decimal': 1, 'categorize': 1, 'belushi': 1, 'bookman': 1, 'fightingwatch': 1, 'videoa': 2, 'davenport': 1, 'sbarro': 1, 'asksstephen': 1, 'unheeded': 1, 'humanitymaking': 1, 'probiotics': 1, 'prebioticshow': 1, 'morphed': 1, 'feudnsa': 1, 'prism': 1, 'invasivebitcoin': 1, 'moneysonoma': 1, 'misinformation': 1, 'wildfiresnew': 1, 'spiritually': 1, 'themrussians': 1, 'frances': 2, 'frontrunner5': 1, 'bowls': 1, 'cerealnew': 1, 'studiesheroic': 1, 'pickles': 1, 'insideretired': 1, '50s': 2, 'familywatch': 1, 'weasel': 2, 'teeny': 2, 'behindsleeping': 1, 'sensory': 1, 'deprivation': 1, 'tankugly': 1, 'outdonald': 1, 'frontrunnerhow': 1, 'happyexpectant': 1, 'thins': 1, 'childmariah': 1, 'holidayswell': 1, 'shortly': 1, 'conductor': 1, 'tracksman': 1, 'pinned': 3, 'dayshealth': 1, 'candlesjames': 1, 'tragedyvietnamese': 1, 'pageantgovernor': 1, 'iowasdazed': 1, 'evilmyanmar': 1, 'homefederal': 1, 'lawsephora': 1, 'eyedivorce': 1, 'reconciliation5yearold': 1, 'overworked': 2, 'upscientific': 1, 'bodyparamount': 1, 'allheidi': 1, 'klum': 1, 'fallonman': 1, 'huffs': 1, 'watchesamy': 1, 'carpetiranian': 1, 'hurriedly': 1, 'inspectionstupid': 1, 'manbetter': 1, 'generationwomans': 1, 'mixedattractiveness': 1, 'relationshipits': 1, 'techjohn': 1, 'confronting': 1, 'unavoidablebashar': 1, 'bikesharing': 1, 'programreport': 1, 'laidoff': 1, 'economywhy': 1, 'furguy': 1, 'finethe': 1, 'conflictwhen': 1, 'vulnerabletrumps': 1, 'saysbank': 1, 'tellers': 1, 'robberyjayzs': 1, 'drugsgod': 1, 'humansoutkast': 1, 'acceptedlaw': 1, 'foran': 1, 'womanwalking': 1, 'huffposts': 1, 'tennesseebusinessman': 1, 'familychild': 1, 'existfirst': 1, 'paynes': 1, 'constellationsunivision': 1, 'trumpimf': 1, 'lagarde': 1, 'tycoon': 1, 'payout': 1, 'trialconnecticut': 1, 'victormitch': 1, 'criticismjames': 1, 'tweetpoor': 1, 'kwanzaa': 1, 'disappoint': 1, 'retailersentire': 1, 'blogosphere': 1, 'postnation': 1, 'ugh': 2, 'niceconservatives': 1, 'lash': 1, 'ryancare': 1, 'growscop': 1, 'exonerated': 1, 'defenseless': 1, 'tellingi': 1, 'talearmy': 1, 'masterminds': 1, 'careercolorado': 1, 'developmental': 1, 'disabilitiescuban': 1, 'adrift': 1, 'urine': 2, 'aliveits': 1, 'datingnortheast': 1, 'mvps': 2, 'gamesroyal': 1, 'crawlingdemocrats': 1, 'condemnation': 1, 'frankenkidnapped': 1, 'captors': 1, 'basementtight': 1, 'updatebitch': 1, 'gettin': 1, 'waywest': 1, 'statewide': 2, 'striketake': 1, 'minutecould': 1, 'cannabis': 2, 'seizuresoxford': 1, 'camerafriends': 1, 'genuinesuper': 1, 'consultants': 1, 'oilman': 1, 'rideproducer': 1, 'anywayweird': 1, 'thanksgivinghuffpollster': 1, 'concedethis': 1, 'asleepgrandma': 1, 'rollups': 1, 'recesses': 1, 'cupboardprivate': 1, 'prisons': 2, 'problemmother': 1, 'ammunition': 1, 'schoolprego': 1, 'marinara': 1, 'momentthis': 1, 'miracleolivia': 1, 'wilde': 1, 'riders': 1, 'womenwoman': 1, 'monopolizing': 1, 'conversationa': 1, 'harveyreport': 1, 'poolthis': 1, 'hypocrisies': 1, 'countingthe': 1, 'thousand': 2, 'cutsthe': 1, 'bunny': 1, 'starwelcome': 1, 'myanmars': 2, 'obama13': 1, 'badass': 2, 'youmeg': 1, 'gavins': 1, 'perspectivegm': 1, 'tarp': 1, 'adventures': 1, 'nowlatinos': 1, 'divide': 3, 'carecompletely': 1, 'sober': 3, 'partynew': 1, 'issuesgod': 1, 'gunsister': 1, 'stopphilip': 1, 'virusyou': 1, 'methodstwas': 1, 'romper': 1, 'anklesdonald': 1, 'terrorswomen': 1, 'diseasepaula': 1, 'jarrels': 1, 'compassionchrissy': 1, 'insinuated': 1, 'diggersurprise': 1, 'trumpwhy': 2, 'populationenzyme': 1, 'biochemical': 1, 'reactionin': 1, 'proclintonthomas': 1, 'pogge': 1, 'yale': 1, 'sayssteve': 1, 'pheromone': 1, 'secretionhouse': 1, '26vegetarian': 1, 'vsarea': 1, 'upflying': 1, 'timevideo': 1, 'chokingblack': 1, 'shoplifting': 1, 'yorkhillary': 1, 'streetdont': 1, 'passively': 1, 'residentsmigrant': 1, 'unmarried': 1, 'qatarmumford': 1, 'mandolins': 1, 'christmastucker': 1, 'carlsons': 1, 'ducameek': 1, 'notchsarah': 1, 'crony': 1, 'capitalismfrom': 1, 'dramas': 1, 'erahanson': 1, 'nambla': 1, 'awardsnato': 1, 'crashhalloweiner': 1, 'frankfest': 1, 'relic': 1, 'gonehow': 1, 'susan': 3, 'anspach': 1, 'completed': 1, 'circlejanice': 1, 'threenaked': 1, 'pursued': 1, 'paircholera': 1, 'africainvisibility': 1, 'realityhow': 1, 'behaviormccain': 1, 'letterthese': 1, 'wallis': 1, 'conversationduke': 1, 'burgundy': 1, 'seekhlo': 1, 'instagramvoters': 1, 'midterms': 1, 'trackarea': 1, 'websitewinter': 1, 'usthe': 1, 'paramedic': 1, 'arcticgame': 1, 'maisie': 1, 'claps': 1, 'headlinetrump': 1, 'hussein': 2, 'goodted': 1, 'billguy': 1, 'streetreallife': 1, 'placementairbnb': 1, 'handwritten': 1, 'cocktail': 1, 'fridgeheat': 1, 'answersrwanda': 1, 'migrantsfreak': 1, 'upromney': 1, 'presentationthis': 1, 'pacificdonald': 1, 'elaine': 2, 'chao': 2, 'transportationargument': 1, 'saydefense': 1, 'typo': 1, 'ira3': 1, 'believeplanet': 1, 'serengetinew': 1, 'unplanned': 1, 'pregnancies': 1, 'lyricsclimate': 1, 'mentioned': 1, 'planulysses': 1, 'trimmeron': 1, 'jerk4': 1, 'uncommon': 1, 'complicationsthe': 1, 'itteen': 2, 'crafting': 1, 'marketable': 1, 'grammyromney': 1, 'quinceaera': 1, 'voteheroic': 1, 'skunk': 1, 'ducklingsactual': 1, 'soonjimmy': 1, 'bithistory': 1, 'doomed': 2, 'pantswomen': 1, 'jayzs': 1, 'duvernaylarry': 1, 'wilmore': 1, 'showheartbroken': 1, 'memorableits': 1, 'countyaides': 1, 'complexcontroversial': 1, 'egypts': 1, 'intergalactic': 1, 'spaceportdivest': 1, 'downwhat': 1, 'secretarytiq': 1, 'milan': 2, 'liberationno': 1, 'berning': 1, 'hillaryinnercity': 1, 'stab': 1, 'himcourt': 1, 'adnan': 1, 'trialnave': 1, 'playmemphis': 1, 'placecan': 1, 'itheres': 1, 'colossal': 3, 'schoolsapple': 1, 'oncewhy': 1, 'allhow': 1, 'rainadorable': 1, 'nowa': 1, 'toobenghazi': 1, 'breachyoung': 1, 'togetherthe': 1, 'crossfit': 1, 'payreport': 1, 'deterioratingll': 1, 'ban6': 1, 'diseasemicrosoft': 1, 'snooping': 1, 'emailequal': 1, 'salary': 2, 'historieschipotle': 1, 'queso': 1, 'aroundneil': 1, 'shotpanicked': 1, 'isthis': 1, 'raisedclinton': 1, 'againapril': 1, 'crueltytrump': 1, 'bh': 1, 'residentsbarack': 1, 'clintonlocal': 1, 'complainmotivational': 1, 'educationan': 1, 'christdate': 1, 'linelights': 1, 'xxxxii': 1, 'namenoaa': 1, '2015irs': 1, 'refunds': 1, 'yearstudy': 1, '25foottall': 1, 'mediathis': 1, 'starone': 1, 'olderjill': 1, 'ladyamerica': 1, 'casualties': 1, 'epidemicsatan': 1, 'tripwhatsapp': 1, 'fullyencrypted': 1, 'servicedonald': 1, 'solowatch': 1, 'shep': 1, 'videoanderson': 1, 'privateeverything': 1, 'italianfellow': 1, 'thingreport': 1, 'this29yearold': 1, 'willwant': 1, 'firstsam': 1, 'nunberg': 2, 'meltdownheres': 1, 'plantbased': 1, 'diet9': 1, 'readsthis': 1, 'onlyrural': 1, 'dakotan': 1, 'unharmednew': 1, '40gigabite': 1, 'ihop': 1, 'pancakespope': 1, 'needy': 2, 'refugeeshair': 1, 'dyed': 1, 'colornew': 1, 'careerjohn': 1, 'billionkatie': 1, 'issuesshipwreck': 1, 'endure': 1, 'shipstunning': 1, 'beautysalmonella': 1, 'cuddling': 1, 'chickensonly': 1, '22450': 1, 'substantial': 1, 'campaignscientific': 1, 'compoundsjonathan': 1, 'lipnicki': 1, 'spaceballs': 1, 'prequelthese': 1, 'snowiest': 1, 'resorts': 1, 'yearobesity': 1, 'individualized': 1, 'therapybuttery': 1, 'goodness': 2, 'productshirtless': 1, 'lifeguard': 1, 'investigates': 1, 'paranormal': 1, 'phenomenawhy': 1, 'modernizationman': 1, 'adrenaline': 1, 'inthe': 2, 'ohioroomba': 1, 'gerbillas': 1, 'balks': 1, 'covering': 3, 'ownerreport': 1, 'barobamas': 1, 'coincides': 1, 'protestwoman': 1, 'hippothemed': 1, 'giftscyber': 1, 'fraudsters': 1, 'reap': 1, 'wiretransfer': 1, 'scamscharter': 1, 'cardamy': 1, 'goldie': 1, 'hawnforensic': 1, 'struggleeven': 1, 'deallatinx': 1, 'talentdisney': 1, 'mascot': 1, 'himself34yearold': 1, 'careerjoe': 1, 'nation21st': 1, 'yearfcc': 1, 'consolidationpennsylvania': 1, 'justicesman': 1, 'threatsin': 1, 'malalas': 1, 'educationtexas': 1, 'reclaim': 2, 'arizonasuspect': 1, 'nicecartoon': 1, 'foreboding': 1, 'entreescorrugatedcardboard': 1, '535': 1, 'corrugatedcardboardrelated': 1, 'issueshow': 1, 'hbcus': 1, 'studentswhy': 1, 'migration': 1, 'concernmore': 1, 'violated': 2, 'policytrump': 1, 'closefridays': 1, 'gocongress': 1, 'packageman': 1, 'timethese': 1, 'gifs': 1, 'labeouf': 1, 'tuesdayunited': 1, 'nissan': 1, 'planteddie': 1, 'millionthe': 1, 'knowauthorities': 1, 'waterstons': 1, 'housemillennials': 1, 'themwith': 1, 'lawstrumps': 1, 'issuepope': 1, 'moloch': 1, 'basilicadeloitte': 1, 'accountant': 1, 'audit': 1, 'calculationsheres': 1, 'pushkansas': 1, 'spelling': 2, 'cannsas': 1, 'saysjohn': 1, 'politicsandrew': 1, 'wk': 1, 'staunch': 1, 'partyadvocacy': 1, 'positioncincinnati': 1, 'premature': 2, 'stepsprotecting': 1, 'presidentchicagos': 1, 'shedd': 1, 'aquarium': 1, 'ghastly': 2, 'mistakehundreds': 1, 'doorstops': 1, 'doorblocker': 1, 'outrep': 1, 'trey': 1, 'presidenthuffpost': 1, 'appellate': 1, 'rulingthe': 1, 'gaziantepreport': 1, 'forsupreme': 1, 'rightshuffpost': 1, '23supreme': 1, 'placerobin': 1, 'foundationtwitter': 1, 'hillarybashing': 1, 'sarandon': 1, 'womenunite': 1, 'globesthe': 1, 'fromqatar': 1, 'gambles': 1, 'labour': 2, 'criticstennis': 1, 'tripman': 1, 'scoutsnew': 1, 'staplings': 1, 'percentmotorcycle': 1, 'explodes': 1, 'istanbulchild': 1, 'yarnviral': 1, 'pocketsmissing': 1, 'comet': 1, 'philae': 1, 'search7': 1, 'bomerthe': 1, 'reality10': 1, 'maple': 1, 'syrupthe': 1, 'itmove': 1, 'debuted': 1, 'livestream': 1, 'dangerousparoled': 1, 'backdozens': 1, 'gravestones': 1, 'toppled': 1, 'cemeteryobama': 1, 'interventionsbeauty': 1, 'thisnew': 1, 'passengersford': 1, 'spooked': 1, 'lanternshadow': 1, 'attracts': 2, 'protesterseric': 1, 'bucking': 1, 'rodeobill': 1, 'disoriented': 1, 'verdictspeaking': 1, 'riggednew': 1, 'secondsbeautiful': 1, 'cadbury': 1, 'creme': 1, 'milkshakessean': 1, 'spicers': 2, 'halo': 1, 'multiplayer': 1, 'lobbyoffice': 1, 'exiles': 1, 'menstruating': 1, 'managerwhy': 1, 'attach': 1, 'foundationhow': 1, 'tweetthese': 1, 'goesformer': 1, 'voided': 1, 'sexjohn': 1, 'dire': 3, 'prosettlement': 1, 'officewhy': 1, 'profilethe': 1, 'minisequel': 1, 'rickman': 2, 'thompsonman': 1, 'forbusinessphone': 1, 'watereddown': 1, 'momsudden': 1, 'rupture': 1, 'gossip': 1, 'pipelinebrad': 1, 'hairromney': 1, 'verbatimso': 1, 'nyes': 1, 'songbritish': 1, 'billionghost': 1, 'wreckage': 1, 'sunk': 1, 'jamaicatrump': 1, 'being9': 1, 'stopone': 1, 'luckyarea': 1, 'foodtea': 1, 'constituent': 1, 'everywherehollande': 1, 'valls': 1, '493': 1, 'reformnations': 1, 'economists': 1, 'evacuating': 1, 'familiesthe': 1, 'weekman': 1, 'crueltybeyonce': 1, 'chanel': 1, 'surfbort': 1, 'cr': 1, 'bookpope': 1, 'poprock': 1, 'novemberamericans': 1, 'reasonstudy': 1, 'arrestdetective': 1, 'circumstances': 2, 'deceasedbruce': 1, 'votingantonio': 1, 'darren': 1, 'remorseless': 1, 'killingjoe': 1, 'chairmexican': 1, 'seriesmark': 1, 'stereotypesnhl': 1, 'ozzie': 1, 'guillen': 1, 'pay5': 1, 'philando': 2, 'castilethese': 1, 'christmasinspired': 1, 'brightunclear': 1, 'shirtless': 3, 'blackandwhite': 1, 'attractivecandidate': 1, 'rapecomics': 1, 'sketchesbizarre': 1, 'dune': 1, 'morse': 1, 'codemom': 1, 'saketed': 1, 'candidatesson': 1, 'cared': 1, 'adventuressamuel': 1, 'sucks': 1, 'pilsnerdefiant': 1, 'oversteve': 1, 'forgottenmike': 1, 'watergateera': 1, 'nicknamein': 1, 'slowlywere': 1, 'anymorehuffpost': 1, '13why': 1, 'nomineewaiting': 1, 'housekatie': 1, 'vaginamusic': 1, 'worsepriebus': 1, 'withhackers': 1, 'antidoping': 1, 'soldiersmelania': 1, 'idly': 1, 'headsup': 1, 'yorkcnn': 1, 'blitzers': 1, 'conductfinancing': 1, 'flames': 3, 'nif': 1, 'fracas': 1, 'mainstreamvisit': 1, 'splurged': 1, 'firedtrump': 1, 'intimidate': 1, 'athletesthis': 1, 'thatatlanta': 1, 'pouring': 1, 'couplegerrymandering': 1, 'mishap': 1, 'whatsoevermysterious': 1, 'hole8th': 1, 'caked': 1, 'confidentrestaurant': 1, 'corroboration': 1, 'dietwhats': 1, 'upside145': 1, 'pledged': 2, 'statesdonald': 1, 'reinstate': 1, 'incidentman': 1, 'gurney': 1, 'ambulanceblack': 1, 'billexperimental': 1, 'theoretically': 1, 'goodreport': 1, 'pushier': 1, 'parentpersonal': 1, 'dreamtampons': 1, 'transphobia': 2, 'taboochicago': 1, 'wagenews': 1, '2017virgin': 1, 'nightlight': 1, 'accusingly': 1, 'masturbateswith': 1, 'mia': 1, 'postracialnews': 1, '2017this': 2, 'flatearth': 1, 'movementdont': 1, 'dwell': 3, 'magazinegood': 1, '19481999': 1, 'carlos': 2, 'santana': 3, 'timeelderly': 1, 'heartless': 1, 'lifetimeobama': 1, 'groundglobal': 1, 'betteri': 1, 'tomike': 1, 'indianapolis': 1, 'taxpayers': 4, '88000turkeys': 1, 'decrees': 1, 'democracyus': 1, 'diplomats': 1, 'drafted': 1, 'dissent': 2, 'objecting': 1, 'banman': 1, 'dumptoday': 1, 'journaliststest': 1, 'buttonchili': 1, 'beardthe': 1, 'sometimesgross': 1, 'intimacy': 3, 'friendshipsman': 1, 'kidsjobless': 1, 'first90': 1, 'argumentin': 1, 'testgulf': 1, 'opecfrom': 1, 'builders': 1, 'neededjamie': 1, 'latinothe': 1, 'votetrump': 1, 'universewinning': 1, 'hindsightchurchills': 1, 'iceand': 1, 'messagenra': 1, 'baskets': 1, 'victimsnational': 1, 'illusions': 1, 'realitiesdonald': 1, 'oftenseymour': 1, 'uncovers': 3, 'aboutincredible': 1, 'waterspout': 1, 'lakepillow': 1, 'comrade': 1, 'groundreport': 1, 'ducts': 1, 'youarea': 1, 'pelvic': 1, 'spanwedding': 1, 'ceremonyjcpenney': 1, '45second': 1, 'saleman': 1, 'supplied': 1, 'chargestire': 1, 'razzledazzlefathers': 1, 'hivboss': 1, 'blend': 1, 'constructive': 1, 'attacksromes': 1, 'revelers': 1, 'marchthe': 1, 'seenpatti': 1, 'lupone': 1, 'madonna': 1, 'bagpeople': 1, 'hummingbirds': 1, 'incubators': 1, 'comefacebook': 1, 'parentsstormy': 1, 'doppelgngerbus': 1, 'curried': 1, 'odorlessgod': 1, 'unconscious': 1, 'directtv': 1, 'satelliterudy': 1, 'othermegan': 1, 'greenhow': 1, 'broth': 3, 'televisionthis': 1, '911terrifying': 1, 'internetlook': 1, 'prideiraqi': 1, 'vivid': 1, 'mosuls': 1, 'falllaidoff': 1, 'cremeinjecting': 1, 'elsewherelisten': 1, 'reys': 1, 'honeymoonscientists': 1, 'ancestorstudy': 1, 'relationshipcashstrapped': 1, 'furloughs': 1, 'nonessential': 1, 'childrenraffle': 1, 'disgustscientists': 1, 'thoughtguy': 1, 'shortimmigration': 1, 'fourthgrader': 1, 'school45': 1, 'themnew': 2, 'ptas': 1, 'inherent': 2, 'inwill': 1, 'downus': 1, 'anachronism': 1, 'trumangirl': 1, 'traded': 1, 'pizzawhen': 1, 'excitementman': 1, 'recordmy': 1, 'anxietydemocratic': 1, 'resignrepublicans': 1, 'inaccuracies': 1, 'documentarycat': 1, 'youchris': 1, 'chops': 1, 'eventbush': 1, 'businessrepublicans': 1, 'congressthese': 1, 'violenceif': 1, 'minorityan': 1, 'silencepolice': 1, 'valleyketchup': 1, 'manpaul': 1, 'rescinding': 1, 'resignationmartin': 1, 'albumus': 1, 'keqiangdating': 1, 'controldemoralized': 1, 'succumbs': 2, 'epidemicthe': 2, 'empowerment': 1, 'generationthe': 1, 'setschrissy': 1, 'overallsmacaulay': 1, 'culkin': 1, 'himnews': 1, '2017james': 1, 'karaokecondo': 1, 'bylaw': 1, 'coverup': 2, 'debmalaysia': 1, 'escalatesdemocrats': 1, 'studentsof': 1, 'stylist': 2, 'ginathe': 1, 'fallacy': 1, 'statecoerced': 1, 'officiants': 1, 'primer': 3, 'chaplaincytheres': 1, 'intensity': 1, 'billalone': 1, 'hurdles': 1, 'fieldcoworker': 1, 'spoilersswiss': 1, 'avalanche': 1, 'neutraldonald': 1, 'outdoors': 1, 'itfastlearning': 1, 'issteve': 1, 'saliva': 1, 'nestterrified': 1, 'case8': 1, 'yearhow': 2, 'skillingsbolle': 1, 'bunsnfl': 1, 'referee': 1, 'hochuli': 1, 'newtons': 1, 'ageism': 1, 'allegationsdonald': 1, 'googlewoundup': 1, 'pajamasreport': 1, 'vulgaria': 1, 'possess': 2, 'flyingcar': 1, 'technologywhat': 1, 'irelandfather': 1, 'firstborn': 1, 'ninemontholdelmo': 1, 'puppeteerdixie': 1, 'droughtravaged': 1, 'africano': 1, 'commercial4': 1, 'claimsprincess': 1, 'knightnew': 1, 'buttonwhy': 1, 'phrasenew': 1, 'sportswriter': 1, 'tweetbartender': 1, 'unfinished': 2, 'drinkweakened': 1, 'spares': 1, 'areasmothers': 1, 'trashdonald': 1, 'systemreport': 1, 'anywayclinton': 1, 'holeinthewall': 1, 'aboutlunatic': 1, 'truenew': 1, 'demography': 1, 'demographer': 1, 'demographicsuccess': 1, 'kidromney': 1, 'pundits': 2, 'policiesgame': 1, 'tormunds': 1, 'brienne': 1, 'screenmustsee': 1, 'enforced': 1, 'lawrookie': 1, 'thickloyal': 1, 'face5': 1, 'getawaysthis': 1, 'colorchanging': 1, 'eatstudy': 1, 'fastest': 1, 'reflexgood': 1, 'unload': 1, 'arenadonald': 1, 'deficitthe': 1, 'liveabc': 1, 'episode4yearold': 1, 'epitome': 2, 'cutewoman': 1, 'mediabill': 1, 'superficial': 1, 'na': 1, 'controversymost': 1, 'countrieswaking': 1, 'beingtuesdays': 1, 'fightmexico': 1, 'officephilly': 1, 'cheesesteak': 1, 'disgustingsingle': 1, 'tablebarehanded': 1, 'miracles': 1, '3cheney': 1, 'oxygen12': 1, 'cancerjonathan': 1, 'adlers': 1, 'missionthe': 1, 'loyalsouth': 1, 'banbb': 1, 'intelligenceitalys': 1, 'etna': 1, 'magnificentcritics': 1, 'reasonsif': 1, 'iud': 1, 'doctorlegal': 1, 'reinforced': 2, 'leprosygop': 1, 'monthbubble': 1, 'skinbernie': 1, 'cautiously': 2, 'upsetright': 1, 'identifiednations': 1, '2199': 1, 'somethingwhat': 1, 'camecreator': 1, 'queerness': 1, 'theaterthis': 1, 'plight': 1, 'lowwage': 1, 'workersbob': 1, 'saget': 1, 'actsproud': 1, 'extinction': 2, 'humanswhy': 1, 'boarding': 2, 'memberstaxpayer': 1, 'millionanne': 1, 'geddes': 1, 'shooterwhen': 1, 'opinionwhite': 1, 'muellerrev': 1, 'metuesdays': 1, 'investigationphilip': 1, '2018man': 1, 'stationcoroner': 1, 'asianclif': 1, 'savory': 1, 'clif': 1, 'loafa': 1, 'terrorismadjusting': 1, 'sliders': 1, 'recording': 2, 'aoki': 1, 'nervousburglary': 1, 'dinner12': 1, 'peoplewhite': 1, 'secrets12': 1, 'summerchaffetz': 1, 'governmentshillary': 1, 'winvogue': 1, 'tattoosdecaying': 1, 'revitalized': 2, 'scenestudents': 1, 'materdozens': 1, 'blackrubberclad': 1, 'masochists': 1, 'jobworldweary': 1, 'stallnation': 1, 'todaynew': 1, 'morale': 2, 'fitsecret': 1, 'hinky': 1, 'skylight': 1, 'fountainmom': 1, '3ring': 1, 'schoolthe': 1, 'choiceworld': 1, 'premierenew': 1, 'yellows': 1, 'timeface': 1, 'debateaustin': 1, 'publicaerobics': 1, 'selfromneys': 1, 'life60': 1, 'rememberheimlich': 1, 'maneuver': 1, 'royaltiespregnant': 1, 'copsmark': 1, 'bearretrocrazed': 1, 'reelect': 1, 'carterfucker': 1, 'cleveland3': 1, 'teamsales': 1, 'issuejanuary': 1, 'renewalmechanic': 1, 'sayreport': 1, 'midwifes': 1, 'promisehousing': 1, 'parentsif': 1, 'yourselffor': 1, 'councilvalentines': 1, 'relationshipnew': 1, 'crunchy': 2, 'gooey': 1, 'thoughtmom': 1, 'ridecurt': 1, 'schilling': 2, 'slot': 2, 'rncare': 1, 'happywoman': 1, 'macethe': 1, 'trumpsciencefiction': 1, 'posits': 2, 'sketchedjohn': 1, 'travolta': 1, 'poophow': 1, 'crimehotcake': 1, 'briskmindfulness': 1, 'understanding': 1, 'stressyou': 1, 'trademarkeddon': 1, 'journaliststudy': 1, 'everythings': 1, 'worksjeremy': 1, 'corbyn': 1, 'interestburger': 1, 'fryerharvey': 1, 'spawns': 1, 'devastate': 1, 'houston10': 1, 'yearobama': 1, 'airspace': 1, 'freerange': 1, 'preservenations': 1, 'goslinghypothetical': 1, 'billboardstouffers': 1, 'steaksthere': 1, 'povertyryan': 1, 'claimsdog': 1, 'climb': 2, 'friendbilly': 1, 'netflixtrump': 1, 'reportalex': 1, 'roadiowa': 1, 'tourism': 2, 'des': 2, 'moines': 1, 'perate': 1, 'campaign60': 1, 'segregated': 1, 'schoolspanicked': 1, 'awe13': 1, 'seriesformer': 1, 'leftchris': 1, 'cheeky': 3, 'instagramnina': 1, 'heartwrenching': 1, 'setdye': 1, 'foils': 1, 'thiefhillary': 1, 'practiceproactive': 1, 'nailbiting': 1, 'habitsaddam': 1, 'checknewt': 1, 'nepotism': 1, 'administrationunregistered': 1, 'notifies': 1, 'wayshark': 1, 'tvdonald': 1, 'protestgun': 1, 'resultsmcdonalds': 1, 'tripledecker': 1, 'burgersantorum': 1, 'headlinesarea': 1, 'prettiest': 1, 'boxhelp': 1, 'timereese': 1, 'disappointment': 1, 'diversityman': 1, 'whatsoeveronlookers': 1, 'insuranceferguson': 1, 'lootingstunned': 1, 'minutesinconsiderate': 1, 'tunahelp': 1, 'buildabear': 1, 'ewok': 1, 'terrifyingnew': 1, 'musiciansbeing': 1, 'hb2tpp': 1, 'answerbeyond': 1, 'bounds': 1, 'conservationcast': 1, 'stroketrump': 1, 'monitors': 2, 'greenhouse': 1, 'gasesno': 1, 'fart': 1, 'jokefirst': 1, 'yetfacebook': 1, 'videoswhite': 1, 'lids': 1, 'firmly': 2, 'againfox': 1, 'attacksman': 1, 'failureseighthgrader': 1, 'twelfthgrade': 1, 'levelwhats': 1, 'goneaides': 1, 'rotate': 1, 'audienceearth': 1, 'twostep': 1, 'differencedonald': 1, 'mending': 1, 'partywhitney': 1, 'leadcat': 1, 'styleewww': 1, 'bugnado': 1, 'radarthe': 1, 'westernization': 1, 'emojiarea': 1, 'okaywhy': 1, 'musicians3': 1, 'leaderarea': 1, 'hookup': 1, 'linkedinthe': 1, 'fixerstephen': 1, 'spank': 1, 'presidentclimate': 1, 'drift': 1, 'awayexasperated': 1, 'traveled': 1, 'bloodsouth': 1, 'sentencean': 1, 'wordrisk': 1, 'champ': 2, 'flunks': 1, 'testid': 1, 'crispy': 1, 'truthful': 1, 'weeksto': 1, 'judgingnew': 1, 'parentarea': 1, 'prowhite': 1, 'antinobodyyoung': 1, 'developer': 1, 'scholly': 1, 'cto': 1, 'pirolloman': 1, 'nonexistent': 1, 'daughterfeds': 1, 'vaccinesdonald': 1, 'birtherism': 1, 'itcancer': 1, 'lobbies': 1, 'fundingthe': 2, 'rookies': 1, 'seasonhillary': 1, 'obamadonald': 1, 'candidatecongress': 1, 'repairs': 1, 'carolyn': 1, 'maloney': 1, 'recorderpope': 1, 'demon': 1, 'papal': 1, 'floorconfounded': 1, 'pollsters': 2, 'mercurial': 1, 'beguiling': 2, 'votemaine': 1, 'peoplealien': 1, 'recordsick': 1, 'saysmourners': 1, 'vigils': 1, 'victimsnew': 1, 'brokenin': 1, 'cameltoe911': 1, 'unmaking': 1, 'centuryrare': 1, 'taiwanwhats': 1, 'fatshaming': 1, 'biasthe': 1, 'datingas': 1, 'hurl': 1, 'attacksenate': 1, 'unanimously': 2, 'sanctionstrump': 1, 'membersjeff': 1, 'monthdennys': 1, 'researcher': 1, 'breakfastthe': 2, 'psychics': 1, 'astrologymanly': 1, 'viewingbillcosbycom': 1, 'internetmelos': 1, 'triangle': 1, 'offensecrisis': 1, 'galacticus': 1, 'finalsrevised': 1, 'patriot': 2, 'acta': 1, 'coldfda': 1, 'harderthese': 1, 'anxietyrelapse': 1, 'lifeewan': 1, 'mcgregor': 1, 'haterscoach': 1, 'audiencewant': 1, 'airbnb': 1, 'rescuenew': 1, 'submit': 1, 'jobhow': 2, 'onepolice': 1, 'hikeron': 1, 'tophappy': 1, 'liningshorrifying': 1, 'notpeanut': 1, 'salmonella': 1, 'outbreakstephen': 1, 'nightjustice': 1, 'jobpalestinian': 1, 'mediator': 1, 'processfetid': 1, 'shitcovered': 1, 'systemdonald': 1, 'wadehow': 1, 'classroomtanned': 1, 'exquisitely': 1, 'coiffed': 1, 'offerthe': 1, 'fastestshrinking': 1, 'americareport': 2, 'candidatesudden': 1, 'africanamericans': 1, 'saysresearchers': 1, 'transformations': 1, 'starsnew': 1, 'specialedition': 1, 'directorscut': 1, 'dvdbratz': 1, 'releasedblack': 1, 'panafrican': 1, 'importantthe': 1, 'helproommates': 1, 'duration': 1, 'relationshipstreets': 1, 'toothbrushesmichigan': 1, 'unhappy': 1, 'snyderoccidental': 1, 'investigationcall': 1, 'undertaker': 1, 'hereelections': 1, 'ukrainelight': 1, 'timestanford': 1, 'growsisis': 1, 'saysconsidering': 1, 'selfemployment': 1, 'yourselfnorman': 1, 'reedus': 2, '6poll': 1, 'drummerteacher': 1, 'glassesbotanist': 1, 'barsad': 1, 'accountcosmopolitan': 1, 'fattening': 1, 'winterstudy': 1, 'preventable': 1, 'personachieving': 1, 'presentation': 2, 'zenautomated': 1, 'teller': 1, 'tellergreat': 1, 'cooperation': 1, 'policymakingchris': 1, 'overseer': 1, 'resignsbreaking': 1, 'americaexpert': 1, 'bluffs': 1, 'iowatransitional': 1, 'stylechris': 1, 'portraitart': 1, 'museummankind': 1, 'worldmitch': 1, 'createdmesquite': 1, 'visine': 1, 'texasits': 1, 'merriamwebster': 1, 'dictionary20': 1, 'painjellyfish': 1, 'nyadcat': 1, 'anal': 1, 'facehillary': 1, 'issuesantonin': 1, '835': 1, 'milliontrump': 1, 'countrytwitter': 1, 'instagrambird': 1, 'arthritis': 1, 'ignoredforeign': 1, 'fashionably': 1, 'wherever': 1, 'fromdear': 1, 'savingchinas': 1, 'potemkin': 1, 'villagesthe': 1, 'preventionrural': 1, 'archbishops': 1, 'droves': 1, 'vaticandole': 1, 'sheettrevor': 1, 'advisergeorgia': 1, 'mathcaptain': 1, 'destroyer': 2, 'bathroomwet': 1, 'cadad': 1, 'dixie': 1, 'chicksamos': 1, 'womanwhy': 1, 'epidemicgolden': 1, 'brass': 1, 'urnmark': 1, 'predictably': 1, 'memes': 1, 'galorethe': 1, 'techfree': 1, 'frontier4': 1, 'indonesiapsychic': 1, 'sniff': 1, 'skunkman': 1, 'cluster': 1, 'frames': 1, 'itcollege': 1, 'giftnba': 1, 'odombighair': 1, 'jesusyou': 1, 'childrennorth': 1, 'shortrange': 1, 'coastgrandmother': 1, 'yeardeath': 1, '91young': 1, 'counties': 1, 'directionalmost': 1, 'corrupt': 1, 'thingnew': 1, 'remorse': 1, 'landslide': 1, 'ravine': 1, 'itwrong': 1, 'gravestonemartha': 1, 'raddatz': 1, 'mvp': 1, 'debateofficials': 1, 'talksdo': 1, 'administratorson': 1, 'generational': 1, 'wagesplease': 1, 'whimpers': 1, 'republicanauthored': 1, 'fisa': 1, 'releasednew': 1, 'unhealthily': 1, 'repressing': 1, 'emotioncreature': 1, 'craps': 1, 'foodnations': 1, 'trig': 1, 'functionsnot': 1, 'brokencaptor': 1, 'humormute': 1, 'jawthousands': 1, 'changelifelong': 1, 'adversityclinton': 1, 'iowan': 2, 'sundownman': 1, 'knifewielding': 1, 'onesits': 1, 'mildly': 1, 'infectious': 2, 'treatableyet': 1, 'discriminationarea': 1, 'overtime': 2, 'anywayhigh': 1, 'retarded': 1, 'faggot': 1, 'himselfnew': 1, 'battlefront': 1, 'slicker': 1, 'greasedup': 1, '3poperipheral': 1, 'molestedits': 1, 'talented': 1, 'therelove': 1, 'humblerestoring': 1, 'decency': 1, 'destructive': 3, 'politicsdubai': 1, 'dubaiisraeli': 1, 'dermer': 1, 'predictionleaving': 1, 'protocolsunclear': 1, 'timealy': 1, 'gymnastics': 2, 'leotards': 1, 'abuse4': 1, 'lifebiggest': 1, 'pumpkintrump': 1, 'guam': 1, 'tourismcretinous': 1, 'reprobate': 1, 'holidayssingapore': 1, 'casualtiesyoure': 1, 'summeruninsured': 1, 'housethe': 1, 'reclusive': 1, 'bankrolling': 1, 'cruzlet': 1, 'badthat': 1, 'promisefakeawish': 1, 'favre': 1, 'lookalikeeverything': 1, 'filipino': 2, 'breakfastshow': 1, 'queensanta': 1, 'obliges': 1, 'snoozing': 1, 'camerajohn': 1, 'nratv': 1, 'killerpope': 1, 'gatheringheat': 1, 'contrarian5': 1, 'anywherein': 1, 'plantrepublicans': 1, 'downreport': 2, 'nownobody': 1, 'cornerat': 1, '2018brooklyns': 1, 'colorfan': 1, 'doubtful': 1, 'burger10': 1, 'crazybeautiful': 1, 'weekendmartin': 1, 'clintonquincy': 1, 'murderoscar': 1, 'wrist': 1, 'reportswhere': 1, 'mayarea': 1, 'shirtshacker': 1, 'typos': 1, 'tuitionkids': 1, 'pregnancyparty': 1, 'yeta': 1, 'questionwe': 1, 'robots': 1, 'jobsqueen': 1, 'approaching': 2, 'ceremonymike': 1, 'mulan': 1, 'combatstack': 1, 'oneyear': 1, 'anniversarynew': 1, 'influencekirsten': 1, 'dunst': 1, 'starpaul': 1, 'datingarea': 1, 'sucksshackled': 1, 'chechen': 1, 'scarred': 1, 'mentor6': 1, 'overlookedthe': 1, 'islams': 1, 'conflictthe': 1, 'housebritain': 1, '125': 1, 'cameronmy': 1, 'grammysamericans': 1, 'nobannowalldetroit': 1, 'halfton': 1, 'horsepower': 1, 'motown': 2, 'singertestdriven': 1, 'evaluations': 1, 'outseth': 1, 'countrymedia': 1, 'overemphasize': 1, 'widereaching': 1, 'conclusionsmilosevic': 1, 'albanians': 1, 'underwearmeet': 1, 'unitedman': 1, 'anothers': 1, 'baraaron': 1, 'expatriots': 1, 'murderpodiatrist': 1, 'jerkkentucky': 1, 'coupleshuffpollster': 1, 'clintonyou': 1, 'clint': 1, 'eastwoodhusband': 1, 'sleeprookie': 1, 'rotten': 1, 'orangegop': 1, 'lineup': 1, 'listeners10': 1, 'stressfree': 2, 'suppers': 1, 'familydnc': 1, 'aiming': 1, 'reconnect': 1, 'hamiltoninspired': 1, 'serieswhite': 1, 'sexnaderite': 1, 'damcasual': 1, 'unitedcomthe': 1, 'prophet': 1, 'jordans': 1, 'mistsaisle': 1, 'countrygrandma': 1, 'lately5': 1, 'presentationsenator': 1, 'debatesecret': 1, 'armor': 1, 'yesterdayveterans': 1, 'chicagos': 1, 'traumawreckage': 1, 'foundhow': 1, 'lovetaylor': 1, 'alternate': 3, 'bodybuilding': 1, 'harrisonpraise': 1, 'lebowski': 1, 'works6': 1, 'cowboyspop': 1, 'consolidateeverything': 1, 'teenjon': 1, 'couragemeeting': 1, 'irl': 1, 'timebahai': 1, 'passingnation': 1, 'articulate': 1, 'themfox': 1, 'sponsors': 1, 'wallthe': 2, 'hopebrotherly': 1, 'advicewatch': 1, 'stuffing': 1, 'tigertoiletpaper': 1, 'triangular': 1, 'foldtrump': 1, 'calms': 1, 'nerves': 1, 'existsgoogle': 1, 'combats': 1, 'holocaustdenying': 1, 'updatewhat': 1, 'msnbc': 1, 'rightwing': 1, 'smear': 1, 'campaignren': 1, 'stimpy': 1, 'girlsaliens': 1, 'alpha': 1, 'centaurimore': 1, '1300': 1, 'agpekingese': 1, 'westminsterlessons': 1, 'teeball': 1, 'dugoutnevada': 1, 'drugsdid': 1, 'marriageteen': 1, 'brink': 1, 'motherhood': 1, 'condompoland': 1, 'democracyhypochondriac': 1, 'cancername': 1, 'misspelledfirst': 1, 'feisty': 1, 'payne': 1, 'schnookbiker': 1, 'repeals': 1, 'lawconcert': 1, 'stepped': 1, 'mic': 3, 'chorushow': 1, 'toursthis': 1, 'voicecarrie': 1, 'telecastselfconscious': 1, 'overheard': 1, 'gianthow': 1, 'legoland': 1, 'mustsee': 2, 'timequeer': 1, 'aussie': 1, 'pictoriala': 1, 'medicaidparalympian': 1, 'disabledpaul': 1, 'speakerstudy': 1, 'disingenuous': 1, 'network15': 1, 'darling': 1, 'wardrobealiens': 1, 'dipsman': 1, 'forreport': 1, 'workpatty': 1, 'sequelbreaking': 1, 'godfbi': 1, 'particular': 1, 'artundocumented': 1, 'receiptsmoments': 1, 'romneys': 1, 'concession': 2, 'hilariousas': 1, 'gillespies': 1, 'warritalin': 1, 'gummis': 1, 'unveiledblind': 1, 'kingmitt': 1, 'countryhonoring': 1, 'animalsminnesota': 1, 'enslaving': 1, 'nannyolivia': 1, 'shondaland': 1, 'clipthis': 1, 'homoji': 1, 'shorthand': 1, 'messageswomen': 1, 'tooba': 1, 'marwat': 1, 'signarama3': 1, 'heremondays': 1, 'deadvelociraptor': 1, 'jurassic': 1, 'diesemma': 1, 'jumpsuithouse': 1, 'gut': 2, 'onslaught': 1, 'criticismslaughtering': 1, 'unscientific': 1, 'unethical': 1, 'arest': 1, 'dorian': 1, 'leadegypt': 1, 'eastif': 1, 'thisthe': 2, 'starks': 1, 'thronesfcc': 1, 'showswoman': 1, 'tournamentwhy': 1, 'invitationwhat': 1, 'toothbrushespoll': 1, 'slobbery': 1, 'hound': 2, 'popewas': 1, 'alive50': 1, 'jewelry': 4, 'carscompany': 1, 'manpower': 1, 'layoffssanders': 1, 'herwoman': 1, 'specialneeds': 1, 'partyif': 1, 'honestpaul': 1, 'specificity': 1, 'planseaworld': 1, 'tanks': 2, 'halfwaythis': 1, 'fatty': 1, 'foodsparanoid': 1, 'himpublic': 1, 'existential': 2, 'emergencywhy': 1, 'prosecuting': 1, 'defendersboxing': 1, 'traumahuffpost': 1, '9guy': 1, 'washing': 2, 'surgeryancient': 1, 'melanesian': 1, 'thundered': 1, 'exhibitno': 1, 'protectionsto': 1, 'copsconfusing': 1, 'roadside': 1, 'bicycle': 1, 'rotary': 1, 'kindhow': 1, 'homeschool': 1, 'shackled': 1, 'childrenreport': 1, 'oiljailed': 1, 'poornew': 1, 'bulleted': 1, '30report': 1, 'whatsoevera': 1, 'shortcomings': 2, 'hearken': 1, '1960sbears': 1, 'cagesstop': 1, 'votersnascar': 1, 'flamesinvasive': 1, 'stateproof': 1, 'certainly': 3, 'reptilewill': 1, 'upstormy': 1, 'pornhub': 1, 'coopersupreme': 1, 'snowstorm': 1, 'iceairlines': 1, 'airpornhub': 1, 'accountsthe': 1, 'merry': 1, 'christmasthis': 1, 'zimmermanfruit': 1, 'loomscbs': 1, 'jokeshe': 1, 'jobobama': 1, 'crisissherwinwilliams': 1, 'triumphantly': 1, 'paintpaul': 1, 'antisemitic': 1, 'campaignswhy': 1, 'occasionwayne': 1, 'coyne': 1, 'onfrustrated': 1, 'dysfunctional': 1, 'weddingseating': 1, 'chart': 2, 'frontlifeguard': 1, 'summerexclusive': 1, 'stanleynewspaper': 1, 'eradoctors': 1, 'wakeon': 1, 'entourage': 1, 'kidshuffpollster': 1, 'partisanshiphomeless': 1, 'nyctrumps': 1, 'againfew': 1, 'straightened': 1, 'afghanistanit': 1, 'americareallife': 1, 'grinch': 1, 'hanukkahp': 1, 'pssy': 1, 'alphabet': 2, 'dreamsdiary': 1, 'atticentire': 1, 'blindsnras': 1, 'ominous': 2, 'appeallatinos': 1, 'debate100': 1, 'connect': 3, 'intimately': 1, 'partneradam': 1, 'sandler': 1, 'intelligent': 2, 'nuanced': 1, 'performanceteachers': 1, 'education5': 1, 'roomcheetah': 1, 'printwearing': 1, 'minime': 1, 'floorthe': 1, 'primal': 2, 'passions': 1, 'americaasparagus': 1, 'springdemocrats': 1, 'nudge': 1, 'outwatching': 1, 'psychenew': 1, 'studentspanasonic': 1, 'portable': 1, '500disc': 1, 'ipodjk': 1, 'potterhelpful': 1, 'herselfsurprise': 1, 'bidder': 1, 'embattled': 1, 'womensarah': 1, 'freea': 1, 'bradythe': 1, 'onlinesome': 1, 'thatthis': 1, 'kitty': 1, 'shamereport': 1, 'crane': 2, 'occupation': 2, 'usbaseball': 1, 'famesens': 1, 'selfinflicted': 1, 'fightpolice': 1, 'tracing': 1, 'iamthegoldenstatekillercom': 1, 'websiteit': 1, 'violencetrump': 1, 'zones': 2, 'issueboehner': 1, 'clintonfrocked': 1, 'shine': 1, 'prestateoftheunion': 1, 'ritualsgeorgia': 1, 'gregory': 1, 'lawler': 1, 'defenseflorida': 1, 'scottthe': 1, 'erdogans': 1, 'tyrannical': 2, 'governancefamily': 1, 'tvafter': 1, 'downstairs': 1, 'jumpin': 1, 'flashup': 1, 'bart': 1, 'mob': 1, 'holdup19': 1, 'love98': 1, 'ownersuscuba': 1, 'foul': 2, 'gamemagicmarkered': 1, 'thiefrun': 1, 'heidi': 1, 'stagedear': 1, '201453inch': 1, 'nothingnew': 1, 'inventing': 1, 'theatersamesex': 1, 'dayschool': 2, 'psychologist': 2, 'injurytrumps': 1, 'techniques': 1, 'mediathese': 1, 'familypallbearers': 1, 'nielsens': 1, 'coffin': 1, 'oneamonth': 1, 'hazard5': 1, 'weddingfrom': 1, 'syringe': 1, 'banhillary': 1, 'snyder': 2, 'recalledlady': 1, 'antebellums': 1, 'kelley': 1, 'dadnelly': 1, 'reiterates': 1, 'sexliking': 1, 'stancepope': 1, 'basilicathe': 1, 'meldonium': 1, 'performanceobama': 1, 'bathroomsmicrosft': 1, 'bids': 1, 'fileshe': 1, 'thereeverything': 1, 'numbersstudy': 1, 'hpvgap': 1, 'lightweight': 1, 'linen': 1, 'summerbiden': 1, 'shag': 1, 'machineatlantaarea': 1, 'ceremonially': 1, 'olympicsmad': 1, 'lit': 1, 'finishing': 2, 'bloomsday': 1, 'devicehostage': 1, 'lording': 1, 'elseapple': 1, 'submits': 1, 'requestlucid': 1, 'horizons': 1, 'researchferguson': 1, 'shotskeletons': 1, 'coins': 1, 'pompeii': 1, 'excavationtrump': 1, 'concede': 1, 'jonesgeorge': 1, 'zimmermans': 1, 'trayvon': 1, 'impressionthis': 1, 'interfaith': 1, 'apartcan': 1, 'learnamerican': 1, 'leaveincredible': 1, 'worldindian': 1, 'dollarcruz': 1, 'cronyist': 1, 'cartel': 1, 'governortokyo': 1, '100story': 1, 'toadstool': 1, 'skylinehow': 1, 'blowout': 1, 'longertouring': 1, 'catsuitsrussell': 1, 'westbrook': 1, 'offthebackofthedefender': 1, 'buzzerbeaterthis': 1, 'nottrump': 1, 'rebuffs': 1, 'declare': 2, 'emergencyuneasy': 1, 'dtente': 1, 'beeman': 1, 'blownexecutive': 1, 'reschedules': 1, 'octobereminem': 1, 'trackpizza': 1, 'topped': 1, 'pizzaspulitzer': 1, 'tooincredible': 1, 'relationshipsthe': 1, 'unquenchable': 1, 'thirst': 1, 'paine': 1, 'neocons': 1, 'againstudy': 1, 'judged': 2, 'timeconspiracy': 1, 'institutionalized': 1, 'americawanda': 1, 'hallburtons': 1, 'selftalkusa': 1, 'editorialmalawi': 1, 'violenceexplained': 1, 'tweeted': 3, 'hortonstyle': 1, 'adbehind': 1, 'crucifixionjohn': 1, 'pardoning': 1, 'arpaioelmore': 1, 'leonard': 1, 'succinctly': 1, 'expired': 1, 'gloomy': 1, 'oldwhy': 1, 'liberty6': 1, 'yetmoderator': 1, 'earlyedtech': 1, 'goingembarrassed': 1, 'albums': 2, 'intolets': 1, 'daymartin': 1, 'anniversarytimes': 1, 'racesapple': 1, 'wozniak': 1, 'scandalheres': 1, 'debaterepublicans': 1, 'watchdoglawyers': 1, 'interviewing': 1, 'jurorstv': 1, 'relates': 1, 'realitymondays': 1, 'recession': 4, 'whywhat': 1, 'violencerepublicans': 1, 'bill5': 1, 'personalitykim': 1, 'daviss': 1, 'time2014': 1, 'islanderschinese': 1, 'worstcompletely': 1, 'officedoug': 1, 'speechcry': 1, 'hourfemale': 1, 'womenhere': 1, 'instagramsgod': 1, 'hauntedthis': 1, 'rendition': 1, 'adeles': 1, 'createdlocal': 1, 'overconcerts': 1, 'luckdakota': 1, 'degeneresstudy': 1, 'busier': 1, 'evercrowd': 1, 'collegetwo': 1, 'yorkgeorge': 1, 'clarence': 2, 'blackfacetheory': 1, 'schoolboard': 1, 'disprovenfranz': 1, 'ferdinand': 1, 'gavrilo': 1, 'princip': 1, 'bassistgop': 1, 'pushganging': 1, 'violencegardner': 1, 'hallssellout': 1, 'sellout': 1, 'bandthe': 1, 'winsbrian': 1, 'letterman': 3, 'appearanceone': 1, 'overthink': 1, 'itmagnetic': 1, 'migration50': 1, 'brusselscable': 1, 'baniphone': 1, 'gadget': 1, 'instrumentphotos': 1, 'engulfing': 1, 'ankara': 1, 'bombsleazy': 1, 'watchalan': 1, 'itspace': 1, 'racecoworker': 1, 'orderchrissy': 1, 'poolside': 1, 'thighnew': 1, 'lawncare': 1, 'greenarea': 1, 'ladys': 1, 'gentleman': 2, 'employ': 1, 'lubemanaging': 1, 'restless': 2, 'employeeamericans': 1, 'president19': 1, 'betterjimmy': 1, 'thursdaymillion': 1, '1000000': 1, 'robotsnew': 1, 'foodthe': 1, 'potty': 1, 'trainingjonathan': 1, 'kozols': 1, 'mustreadobama': 1, 'resigning': 1, 'heyers': 1, 'motherclinton': 1, 'whammys8': 1, 'onestephen': 1, 'sentencecheney': 1, 'conventionarea': 1, 'morningnra': 1, 'rifle': 1, 'shootingto': 1, 'privileged': 1, 'kidshamed': 1, 'inmatesclarissa': 1, 'clarissa': 1, 'bookcongress': 1, 'legislationpromotional': 1, 'everywhererussian': 1, 'roulette': 1, 'trillions': 1, 'derivativesheartless': 1, 'anythingheres': 1, 'vacationsnew': 1, 'asshole8': 1, 'sexsprite': 1, 'colaflavored': 1, 'spritefear': 1, 'thones': 1, 'showwith': 1, 'greetings': 1, 'europea': 1, 'happenjournalist': 1, 'shootinghow': 1, 'sensitive': 2, 'boardwoman': 1, 'diessenator': 1, 'baucus': 1, 'bodygore': 1, 'hostagessamantha': 1, 'booktom': 1, 'supervillainloophole': 1, 'archaeologist': 1, 'hookkfc': 1, 'antibiotics': 2, 'poultryboss': 1, 'fridaynavy': 1, 'admiral': 2, 'importantsecretarian': 1, 'receptionists7': 1, 'bisexual': 1, 'pansexual': 1, 'monthmississippi': 1, 'averagereport': 1, 'holidays6': 1, 'someoneconnect': 1, 'fourplaying': 1, 'sis': 1, 'sneakywhy': 1, 'houseafter': 1, 'legalize': 4, 'unionscancer': 1, 'itemtwo': 1, 'slayingwant': 1, '102': 1, 'howdad': 1, 'cheesecake': 2, 'vibrating': 1, 'pagerpope': 1, 'hosteating': 1, 'conteststudy': 1, 'intestinal': 1, 'worldinside': 1, 'wrestling': 3, 'bruising': 1, 'workplace12': 1, 'truths': 1, 'kidsits': 1, 'performancewhy': 1, 'dosevan': 1, 'toughguy': 1, 'trumpzilla': 1, 'snowflakenew': 1, 'cameooffbeat': 1, 'intowhy': 1, 'forgiven': 1, 'fatherus': 1, 'sayjared': 1, 'gunpoint': 2, 'identitiesreport': 1, 'oneconan': 1, 'advertise': 1, 'hearingwelterweight': 1, 'sustaining': 1, 'boutred': 1, 'bloodbath': 1, 'semifinals': 1, 'resultslook': 1, 'thors': 1, 'huesyellowstone': 1, '6month': 2, 'acadia': 1, 'parkcrowd': 1, 'dangles': 1, 'balconybank': 1, 'programchina': 1, 'winnersarah': 1, 'uptransmuting': 1, 'blessing': 1, 'uscverdugo': 1, 'hospitalbuddhist': 1, 'lobsters': 2, 'destined': 1, 'freenations': 1, 'zipdementia': 1, 'idyllic': 1, '1950sfugoo': 1, 'review7': 1, 'miserablefather': 1, 'rocksarah': 1, 'gellar': 1, 'buffy': 1, 'rebootchristina': 1, 'lettuceobama': 1, 'urgings': 1, 'callings': 1, 'relocate': 1, 'roadblockmilo': 1, 'strips': 1, 'ellena': 1, 'constituents': 3, 'insteadcheck': 1, 'outweighs': 1, 'carin': 1, 'lowpaid': 1, 'workerssetting': 1, 'maximum': 1, 'effectbetsy': 1, 'argues': 2, 'shootersshould': 1, 'credit8yearold': 1, 'puking': 1, 'foreverentire': 1, 'phish': 2, 'posterthese': 1, 'tapping': 1, 'kegis': 1, 'athleteshow': 1, 'religionmade': 1, 'certaintytrumps': 1, 'investigationdad': 1, 'salesmans': 1, 'hdtv': 1, 'compatibilityarea': 1, '16th': 1, 'depressioncruz': 1, 'universityman': 1, 'sleepthai': 1, 'peppersdept': 1, 'signthis': 1, 'marriageseed': 1, 'middleschool': 1, 'classbiden': 1, 'nameformer': 1, 'huntchicago': 1, 'violenceleaping': 1, 'kangaroo': 1, 'unsuspecting': 1, 'cyclist': 3, 'australiachris': 1, 'hinting': 1, 'runariana': 1, 'singlehandedly': 1, 'snlobama': 1, 'turretmounted': 1, 'tanksdad': 1, 'guyomg': 1, 'horns': 1, 'clintonon': 1, 'luigi': 1, 'fineounemployed': 1, 'confidencedesperate': 1, 'melanie': 2, 'hutsell': 1, 'dvdvalerie': 1, 'deadlier': 1, 'breastonline': 1, 'lifegunman': 1, 'congressmanoh': 1, 'billdad': 1, 'cruella': 1, 'smallminded': 1, 'bigotskendrick': 1, 'damnhollywood': 1, 'depiction': 1, 'horriblethe': 1, 'amazonarea': 1, 'reincarnationtai': 1, 'dislocates': 1, 'kneeus': 1, 'ebolastricken': 1, 'africaicymi': 1, 'blizzardsfriends': 1, 'revel': 1, 'prescriptionsthe': 1, 'victorymom': 1, 'fruitvale': 1, 'stationwhy': 1, 'twiceamericans': 1, 'polarized': 1, 'showsnew': 1, 'funhow': 1, 'debtmom': 1, 'coyote': 1, 'situationnew': 1, 'watertrump': 1, 'lifejill': 1, 'soloway': 1, 'sexismtranslator': 1, 'spectral': 1, 'testimonyzoning': 1, 'shithow': 1, 'kesha': 1, 'awardsvin': 1, 'sequelbill': 1, 'polarizedmom': 1, 'gunning': 1, 'befriend': 1, 'babysitter': 1, '3minute': 1, 'interactionsconservative': 1, 'problemroommate': 1, 'amazingsenator': 1, 'barsmccains': 1, 'emphasizes': 1, 'grease': 1, 'sleeverollingupj': 1, 'balvin': 1, 'nicky': 1, 'jam': 2, 'reggaetn': 1, 'misogynistcanvas': 1, 'doorknobescape': 1, 'fallujasuspected': 1, 'smugglers': 1, 'tragedynew': 1, 'subtember': 1, '11lives': 1, 'meaningmadonnas': 1, '2016wiz': 1, 'khalifa': 1, 'fetty': 1, 'wap': 1, 'omi': 1, 'internets': 1, 'moststreamed': 1, 'summermark': 1, 'skywalkerfather': 1, 'expresses': 2, 'inflationperfect': 1, 'mallcover': 1, 'wordforword': 1, 'mobydicktrump': 1, 'hillaryextra': 1, 'edgebodyslam': 1, 'gianforte': 1, 'scorching': 1, 'memesi': 1, 'hypocrisydesperate': 1, 'homeopathic': 1, 'methodsa': 1, 'pregnancyamericans': 1, 'thermometerlaptop': 1, 'usb': 1, 'ports': 1, 'stuffed12': 1, 'carrie': 2, 'fisherelectoral': 1, 'preventirvin': 1, 'yalom': 1, 'conversationjury': 1, 'prettyderay': 1, 'effectcooking': 1, 'recommendationwhen': 1, 'shakyreport': 1, 'stopnight': 1, 'judys': 1, 'missbreaking': 1, 'whartonwharton': 1, 'sakejeff': 1, 'knocks': 2, 'birthersstreet': 1, 'pariscrucial': 1, 'staysarea': 1, 'muster': 1, '140': 1, 'racewelcome': 1, 'herointown': 1, 'jerseys': 1, '4thlargest': 1, 'citygreyhound': 1, 'inbus': 1, 'magazinereport': 1, 'year50': 1, 'ragefather': 1, 'killedpepperidge': 1, 'milanosthis': 1, 'snoozedick': 1, 'vitale': 1, 'bracketological': 1, 'examinationanteater': 1, 'ants': 1, 'awhilejamie': 1, 'dimon': 1, 'relentless': 1, 'companytylenol': 1, 'gel': 2, 'unbalanced': 2, 'humorswondrous': 1, 'fishes': 2, '4176711': 1, 'swallowwhats': 1, 'carnage': 1, 'kabulmaralago': 1, 'caddy': 2, 'coursebelgrades': 1, 'serbiathe': 1, 'everyonedoctors': 1, 'overprescribe': 1, 'narcoticskuwait': 1, 'dcthe': 1, 'mathison': 1, 'oscarnominated': 1, '65report': 1, 'teachersa': 1, 'collaborations': 1, 'bouldermiranda': 1, 'lambert': 2, 'fanradio': 1, 'bullshitsometimes': 1, 'namechange': 1, 'personallyfurious': 1, 'dianne': 1, 'hergreat': 1, 'scuba': 1, 'divers': 1, 'deathiran': 1, 'pits': 1, 'alliesdiary': 1, 'toalton': 1, 'forgettingwoman': 1, 'comforting': 1, 'stickspiers': 1, 'wellsquirrel': 1, 'paws': 1, 'wheelsobama': 1, 'publicive': 1, 'tookittens': 1, 'firejapan': 1, 'launchcoworker': 1, 'stormed': 1, 'badassburmese': 1, 'python': 2, 'toddlerparkland': 1, 'communitiessuppression': 1, 'votedoes': 1, 'summons': 3, 'warrants': 1, 'crimeso': 1, 'codes': 1, 'meannew': 1, 'respectedthe': 1, 'violencewatch': 1, 'gregg': 1, 'rolie': 1, 'neal': 1, 'schon': 1, 'albumcustodian': 1, 'custodymartin': 1, 'wageangela': 1, 'changefacebook': 1, 'whatsapp': 1, 'encryption': 1, 'datafridays': 1, 'obamacarebrazils': 1, 'rouseffs': 1, 'impeachmentarianna': 1, 'webcam': 1, 'implanted': 1, 'foreheadisla': 1, 'thingssnl': 1, 'grabbies': 1, 'awardsstudy': 1, 'chubby': 1, 'cherubsuburban': 1, 'neardef': 1, 'experiencebernie': 1, 'jumpshot': 1, 'winmental': 1, 'bipolar': 2, 'skinallstate': 1, 'racketnational': 1, 'pandas': 1, 'divorcechris': 1, 'tucker': 1, 'projectsfearful': 1, 'awayrubio': 1, 'secrecy': 1, 'eventa': 1, 'filmtrumps': 1, 'ponzi': 1, 'schemehouston': 1, 'surveying': 1, 'unchecked': 1, 'industrializationhow': 1, 'datecheetos': 1, 'arguing': 3, 'chester': 1, 'cheetahs': 1, 'voicebarb': 1, 'downclaims': 1, 'lubitzs': 1, 'stigmatize': 1, 'illnessesyemen': 1, 'leaderreport': 1, 'reprehensible': 1, 'weekdespite': 1, 'deception': 1, 'statedrugrelated': 1, 'dutertes': 1, 'electionconfused': 1, 'stoplightssenate': 1, 'administrationmeg': 1, 'steady': 1, 'minuteselon': 1, 'existspalestinian': 1, 'israelgaza': 1, 'protestsgumption': 1, 'rewarded': 1, 'workmass': 1, 'costeffective4': 1, 'mindset': 1, 'helpdisneys': 1, 'gosarah': 1, 'scandalgoing': 1, 'foodloving': 1, 'ordealamazon': 1, '1click': 1, 'bankrupts': 1, 'parkinsons': 1, 'sufferermcdonalds': 1, 'mccrazy': 1, 'burgergod': 1, 'gerbilsdental': 1, 'hygienist': 1, 'flossingman': 1, 'policetrump': 1, 'intention': 2, 'updatethis': 1, 'tricking': 1, 'moneycompany': 1, 'letterthe': 1, 'burnsfederal': 1, 'deported': 1, 'timewhy': 1, 'millionsmarchsfhow': 1, 'wreak': 1, 'stateseveryone': 1, 'recommendationtrue': 1, 'endstockphoto': 1, 'armsreport': 1, 'firethe': 1, 'timesdaily': 1, 'sacredsnack': 1, 'resided': 1, 'deliciousliberal': 1, 'conditionmilitary': 1, 'warnina': 1, 'solomons': 1, 'bookits': 1, 'poop': 2, 'bagstrump': 1, 'downplays': 3, 'rolepete': 1, 'buttigieg': 1, 'futuretenants': 1, 'anythingare': 1, 'extroverts': 1, 'happierhealth': 1, 'overeating': 1, 'healthierlatest': 1, 'emailsspouse': 1, 'painmontana': 1, 'isisstonerturneddoctor': 1, 'potdick': 1, 'againhannity': 1, 'payment': 3, 'casesmy': 1, 'copinternational': 1, 'policyparamount': 1, 'filmargentina': 1, 'mishaporrin': 1, 'bondagefetish': 1, 'dungeonovercrowded': 1, 'construct': 2, 'stadiumnasa': 1, '1969': 1, 'nativesmike': 1, 'frontsole': 1, 'sleptslowly': 1, 'rotating': 2, 'metaphor': 1, 'marriagerare': 1, 'rossini': 1, 'caramoorno': 1, 'happenscivilian': 1, 'casualty': 1, 'flattered': 2, 'leaderissa': 1, 'sterlings': 2, 'childrenwhats': 1, 'turkeycomedians': 1, 'abortionscientists': 1, 'wheneverreport': 1, 'mindreading': 1, 'level4': 1, 'crashboth': 1, 'civiliansspy': 1, 'assimilate': 1, 'civilian': 2, 'lifejogger': 1, 'greatiran': 1, 'topsecret': 1, 'negotiationsattention': 1, 'sushi': 1, 'yoursreport': 1, 'smut': 1, 'consumed': 1, 'filthiest': 1, '1beauty': 1, 'bobbi': 2, 'sunblock': 1, 'botoxnew': 1, 'leatherbound': 1, 'creativitythis': 1, 'illnesssome': 1, 'grangerkristen': 1, 'haders': 1, 'monologuebartender': 1, 'plotting': 2, 'boehneryou': 1, 'hutturkeys': 1, 'hitlers': 1, 'governmentfrancis': 1, 'coppola': 1, 'godfather': 1, 'lasagnastop': 1, 'alreadythese': 1, 'wrongwatch': 1, 'cordenwomans': 1, 'activate': 1, 'serverstudy': 1, 'acute': 1, 'stairsgoogle': 1, 'pagedriven': 1, 'carroyal': 1, 'friendschildhood': 1, 'depressiondonald': 1, 'yearnasa': 1, 'sunbeyonc': 1, 'truebathroom': 1, 'inoffended': 1, 'huffy': 1, 'unnoticedstaffers': 1, 'christmaspope': 1, 'errors': 1, 'handling': 1, 'chile': 1, 'allegationsnew': 1, 'cleaner': 1, 'livingoklahoma': 1, '1914': 1, 'abraham': 1, 'orville': 1, 'redenbacherfavorite': 1, 'insidemeet': 1, 'reshaping': 2, 'fastball': 1, 'timeif': 1, 'superbly': 1, 'willbacking': 1, 'vilify': 2, 'studentssearch': 1, 'continuesnew': 1, 'emptiedweird': 1, 'dousing': 1, 'watersurge': 1, 'amazondaniel': 1, 'tosh': 1, 'chuckles': 1, 'rapewell': 1, 'bull4': 1, 'kalethe': 1, 'independentthe': 1, 'lonelinessreport': 1, 'hoffmans': 1, 'deathtraining': 1, 'educationtrumps': 1, 'whistleukraine': 1, 'mollifying': 1, 'russiarepublican': 1, 'adieu': 1, '2012when': 1, 'vaccinate': 1, 'pricejennifer': 1, 'theroux': 1, 'parisi': 1, 'childreport': 1, 'articleobama': 1, 'kenyajay': 1, 'browders': 1, 'emailnorth': 1, 'againpodcast': 1, 'robotsbassist': 1, 'everyonereese': 1, 'starstudded': 1, 'fomogore': 1, '2004': 1, 'cabinhatchet': 1, 'hikers': 1, 'sayjeff': 1, 'spits': 1, 'interrogator': 1, 'trumpexperts': 1, 'pitchperfect': 1, 'cameo': 1, '2017nations': 1, 'bicyclists': 1, 'helmets': 1, 'month95': 1, 'nigerian': 2, 'firenetflix': 1, 'carters': 1, 'peanutsamy': 1, 'racewhy': 1, 'troubleperson': 1, 'deceasedunemployed': 1, 'obamasee': 1, 'minutesflock': 1, 'suicidal': 1, 'geese': 1, 'jetlinerweary': 1, 'haitians': 1, 'shrug': 1, 'ragnark': 1, 'portauprincetae': 1, 'kwon': 1, 'mismatched': 1, '8yearoldsthe': 1, 'wiz': 1, 'excellence': 1, 'tvlynda': 1, 'heffernans': 1, 'selfcompassionwhite': 1, 'departing': 1, 'staffersbeyonc': 1, 'channeled': 1, 'kims': 1, 'copecar': 1, 'timeguy': 1, 'doas': 1, 'inheres': 1, 'jamesrepublican': 1, 'youwhen': 1, 'microphone': 1, 'realarea': 1, 'feverjonah': 1, 'neuroscience': 1, 'falsify': 1, 'quoteschristmas': 1, 'mallhair': 1, 'weave': 1, 'shaved': 1, 'offjapanese': 1, 'homeman': 1, 'helplessly': 2, 'hearddinner': 1, 'includedwhy': 1, 'itsources': 1, 'drudge': 1, 'yearsemma': 1, 'hollywoodjeeves': 1, 'wartsnation': 1, 'parkgym': 1, 'killedgiving': 1, 'seasonbest': 1, 'acclaimed': 1, 'lens': 1, 'flareshowerin': 1, 'bridal': 1, 'usain': 1, 'bolt': 2, 'olympicsamazons': 1, 'netflixbuzzfeed': 1, 'greatpaul': 1, 'briefingskasich': 1, 'voterspaul': 1, 'mccartney': 1, 'beatles': 3, 'onopuerto': 1, 'manmade': 1, 'disasterarea': 1, 'quoteworlds': 1, '194': 1, '2016lateworking': 1, 'vainblinds': 1, 'haegue': 1, 'yangs': 1, 'leeum': 1, 'seoulfeeding': 1, 'deathsee': 1, 'yet5': 1, 'marathoncontroversial': 1, 'anthemrolling': 1, 'unraveledrnc': 1, 'pedophiliagops': 1, 'deepens': 2, 'shiftmexico': 1, 'bondinspired': 1, 'paradeshrimp': 1, 'dieting': 1, 'resultsjared': 1, 'received': 2, 'reportsatheists': 1, 'religionman': 1, 'mailbox': 1, 'birthunemployed': 1, 'photoshops': 1, 'photothese': 1, 'sneakier': 1, 'arewamu': 1, 'chaplevthe': 1, 'lowcalorie': 1, 'mixers': 1, 'nowgiddy': 1, 'byhow': 1, 'heaviest': 1, 'century5': 1, 'powerfulrotting': 1, 'decaying': 1, 'walldoctor': 1, 'exoskeletonreport': 1, 'shapeobama': 1, 'themrepublican': 1, 'contradicts': 1, 'stanceman': 1, 'squat': 2, 'nsfwbangladesh': 1, 'businessreport': 1, 'againstperson': 1, 'zimmermantrump': 1, 'membersdeadlocked': 1, 'twiceholy': 1, 'knockoff': 1, 'batarang': 1, 'copsstrength': 1, 'powerkerry': 1, 'marie': 1, 'claireryan': 1, 'livelys': 1, 'tweetteenage': 1, 'contestpope': 1, 'thinkingfbi': 1, 'torrenting': 1, 'picket': 1, 'fencesnew': 1, 'courtrepublicans': 1, 'senate4': 1, 'reportformer': 1, 'counterterrorism': 1, 'firingblack': 1, 'plantationrupaul': 1, 'emperor': 1, 'clothestoddler': 1, 'horsesweating': 1, 'roofafter': 1, 'leo': 1, 'gugu': 1, 'personalityhuffpost': 1, '22man': 1, 'aheadstudy': 1, 'kinky': 1, 'perverts': 1, 'aremeet': 1, 'plutonew': 1, 'trumplearning': 1, 'failurethe': 1, 'balding': 2, 'awakensa': 1, 'bordersmost': 1, 'trumpletter': 1, 'thankfully': 1, 'ballscopying': 1, 'incidenthayley': 1, 'kiyoko': 1, 'forits': 1, 'backray': 1, 'lamontagne': 1, 'vinylvideo': 1, 'whiteness': 1, 'electionislamophobe': 1, 'indiscriminately': 1, 'muslimsverizon': 1, 'chargeyouatwhim': 1, 'planselfhelped': 1, 'selflong': 1, 'toenail': 1, 'overnever': 1, 'chibokbernie': 1, 'evergreen': 1, 'cbo': 1, 'scorethis': 1, 'thrive': 1, 'fielddefiant': 1, 'garlands': 1, 'aloft': 1, 'buildingleadership': 1, 'righthungover': 1, 'guillermo': 1, 'afterpartytrapped': 1, 'chronically': 1, 'personcelebration': 1, 'destructionseaworld': 1, 'orcas': 1, 'tanksreport': 1, 'adredwood': 1, '300year': 1, 'leftncis': 1, 'editiontrump': 1, 'sideshow': 1, 'acts': 4, 'terrorget': 1, 'ideastrump': 1, 'electionsjust': 1, 'eyelashes': 1, 'mitesfrom': 1, 'educationabdul': 1, 'malik': 2, 'abdul': 1, 'kareem': 1, 'conspiring': 1, 'attackthe': 1, 'generationtrump': 1, 'dem': 1, 'pennsylvaniagirlfriend': 1, 'clingy': 1, 'pregnantreport': 1, 'fuse': 1, 'isman': 1, 'laptophow': 1, 'highfat': 1, 'brainwill': 1, 'presidentjim': 1, 'poetfor': 1, 'freelancers': 1, 'riskexercise': 1, 'findisrael': 1, 'ample': 1, 'siteswatch': 1, 'selfdescribed': 1, 'petcare': 1, 'elevatorfbi': 1, 'scamcia': 1, 'nuclearattack': 1, 'insurancecollege': 1, 'monthdemocratic': 1, 'hearingrep': 1, 'legitimate': 3, 'frauddoctor': 1, 'babynew': 1, 'systemwatch': 1, 'sceneobama': 1, 'refugeeswhats': 1, '2016rouhani': 1, 'postnuclear': 1, 'electionhundreds': 1, 'seasonfbi': 1, 'helpwhats': 1, 'namethe': 1, 'workbench': 1, 'chargersentire': 1, 'divorceman': 1, 'hangnail': 1, 'unravels': 1, 'bodykids': 1, 'innate': 1, 'humansfiona': 1, 'depressedclinton': 1, 'shitlabor': 1, 'backjeb': 1, 'lobbyists': 2, 'themwhat': 1, 'electionpregnant': 1, 'fetusesteen': 1, 'pullups': 1, 'lifetimeend': 1, 'industrydateline': 1, 'eventsobamas': 1, 'fivedecade': 1, 'failuremom': 1, 'topping': 1, 'decadedonald': 1, 'dubious': 1, 'jackie': 1, 'evanchogay': 1, 'proudgay': 1, 'prayershampton': 1, 'concierge': 1, 'hostess5': 1, 'zoneconsumer': 1, 'excellentearth': 1, 'measuring': 1, 'cheeseburgershackers': 1, 'researcherslisterine': 1, 'gelelderly': 1, 'rite': 3, 'toothpaste': 1, 'prolong': 1, 'contactget': 1, 'portugal': 1, 'minutereport': 1, 'turmoilsupreme': 1, 'worldly': 1, 'vicesthese': 1, 'differently7': 1, 'chemistry': 1, 'conversationserial': 1, 'marshmonster': 1, 'escapesreport': 1, 'glance': 1, 'personthe': 1, 'stupidest': 1, 'peoplefriends': 1, 'showmoviegoer': 1, 'sneak': 3, 'earning': 2, 'hourhuffpost': 1, 'distribute': 1, 'classiest': 1, 'mresforprofit': 1, 'sayswatch': 2, 'alig': 1, 'prisontiny': 1, 'unlabeled': 1, 'hilariousjapans': 1, 'forecastworld': 1, 'walls': 2, 'brazilians5': 1, 'misspharmaceutical': 1, 'createcontrarian': 1, 'upends': 1, 'consensus': 1, 'microfiber': 1, 'towelsoscars': 1, 'shrugging': 1, 'uproariously': 1, 'kimmels': 1, 'monologuenevada': 1, 'downpour': 1, 'damagealton': 1, 'reformhow': 1, 'trumphurricane': 1, 'floridanow': 1, 'kitchennepotism': 1, 'synergywhy': 1, 'ails': 1, 'themlights': 1, 'xx': 1, 'gratefuleven': 1, 'sittingchelsea': 1, 'showinspire': 1, 'worldsix': 1, 'architectural': 1, 'communitiessamsung': 1, 'unveiled': 1, 'thinnest': 1, 'everambassador': 1, 'phrasebook': 1, 'strengthened': 1, 'pakistaneddie': 1, 'clotheslee': 1, 'majors': 1, 'existdustin': 1, 'behaviorbayer': 1, 'billiontrump': 1, 'outclassically': 1, 'trained': 1, 'cuethe': 1, '2015man': 1, 'bebuddy': 1, 'deathillinois': 1, 'familiesknobbyfaced': 1, 'foursjustice': 1, 'sotomayor': 2, 'bending': 1, 'copswkzntv': 1, 'broadcast': 2, 'daymike': 1, 'wrongtwitter': 1, 'paintstyle': 1, 'tweetwere': 1, 'keyoregon': 1, 'prooftony': 1, 'randall': 1, 'secedes': 1, 'randaliajohn': 1, 'toogoodyear': 1, 'tiresno': 1, 'orgasmwith': 1, 'grimes': 1, 'bestival': 1, 'michiganborn': 1, 'techno': 1, 'wonderlandnascar': 1, 'creeping': 1, 'usaweism': 1, 'transcendencea': 1, 'daughterdetails': 1, 'rangeus': 1, 'ping': 1, 'bitethis': 1, 'emailan': 1, 'coachs': 1, 'prospect': 1, 'gaytomb': 1, 'lieutenant': 1, 'billwhat': 1, 'systemsbush': 1, 'doorare': 1, 'republicansmom': 1, 'insistent': 1, 'othermiss': 1, 'cara': 1, 'mund': 1, 'crowned': 1, 'americabride': 1, 'itd': 1, 'objected': 1, 'weddingfour': 1, 'travelthe': 1, 'needthe': 1, 'fuelsthe': 1, 'secondguessing': 1, 'bodyman': 1, 'flightthis': 1, 'tastypassersby': 1, 'kidssponsor': 1, 'marshall': 1, 'protestits': 1, 'eisenhower': 1, 'reagans': 1, 'anymoreinverted': 1, 'supercuts': 1, 'hairstyles': 2, 'preserved': 1, 'generationswhy': 1, 'scarily': 1, 'similar': 1, 'gamesemotional': 1, 'ruddernation': 1, 'retelling': 2, 'controversysunday': 1, 'easierespn': 1, 'eaves': 1, 'onis': 1, 'oldfashionedkendrick': 1, 'sza': 1, 'videotampa': 1, 'monumenthow': 1, 'relief5': 1, 'workaholic': 1, 'roommoving': 1, 'kindnesssign': 1, 'signage': 1, 'restrictionsyes': 1, 'spotsnations': 1, 'spokesbeastwhos': 1, 'anywayssomeone': 1, 'dinosaurel': 1, 'chapo': 1, 'chargescongolese': 1, 'warthe': 1, 'realmichelle': 1, 'controversycanadas': 1, 'inuit': 1, 'languagesupdating': 1, 'cubas': 1, 'leadersheres': 1, 'washbush': 1, 'dreamstrump': 1, 'ruling7': 1, 'spainmarvel': 1, 'daredevilsanta': 1, 'fe': 2, 'kokopellied': 1, 'outkerry': 1, 'lobsterboy': 1, 'programsice': 1, 'stoppingarea': 1, 'monument': 2, 'bodythursdays': 1, 'marriagearea': 1, 'toiletpornstore': 1, 'gummed': 1, 'again13': 1, 'youtrue': 1, 'airlinestrack': 1, 'reinvested': 1, 'blackjack': 1, 'futuresnew': 1, 'smellim': 1, 'officeyou': 1, 'arelazy': 1, 'icethemed': 1, 'partycult': 1, 'actuallystormy': 1, 'smirks': 1, 'trumpa': 1, 'celinda': 1, 'lakesixyearold': 1, 'ballerina': 1, 'veterinarian': 1, 'horsebackriding': 1, 'princessted': 1, 'cruzjohn': 1, 'starthaving': 1, 'venue': 1, 'lunch14': 1, 'attentiontrump': 1, 'contextbill': 1, 'womenim': 1, 'dealheres': 1, 'blooms': 1, 'picsfamily': 1, 'lifethree': 1, 'aereo': 1, 'answersexperts': 1, 'incidences': 1, 'soaking': 1, 'wetcouldve': 1, 'grumbles': 1, 'workerskanye': 1, 'kubrickguy': 1, 'pistachios': 1, 'housetrump': 1, 'housebest': 2, 'itunhrc': 1, 'decay': 1, 'treatmentlove': 1, 'dyllan': 1, 'videolegal': 1, 'blockhospital': 1, 'comprehension': 1, 'firstgrade': 1, 'leveljustin': 1, 'intensive': 1, 'unit': 2, 'booedobama': 1, 'expelling': 1, 'continued': 4, 'risencowpoke': 1, 'lassoes': 1, 'perched': 1, 'caraudiobook': 1, 'cajun': 1, 'accentrookie': 1, 'trucker': 1, 'cb': 1, 'motherwalking': 1, 'it12yearold': 1, 'curled': 1, 'glamourhouse': 1, 'trialadeles': 1, 'yearcharmin': 1, 'disposable': 1, 'paperthat': 1, 'mouththerapist': 1, 'faadeburger': 1, 'openmy': 1, 'itlights': 1, 'xxxx': 1, 'learningpool': 1, 'cues': 1, 'unused': 1, 'fightcourt': 1, 'assaultreport': 1, 'dicksnegative': 1, 'assuring': 2, 'diversityshould': 1, 'debtfree': 1, 'retireopenminded': 1, 'metalarea': 1, 'everyonelynn': 1, 'whitfield': 1, 'otherreport': 1, 'milly': 1, 'outword': 1, 'educationantiabortion': 1, 'songtrump': 1, 'adsrichard': 1, 'engel': 1, 'addressstudy': 1, 'bollywood': 2, 'zanjeervisiting': 1, 'apartmentwhy': 1, 'educationadvisors': 1, 'campaign7': 1, 'doamericans': 1, '37325': 1, 'yearnation': 1, 'handcrafted': 1, 'productboardwalk': 1, 'chumpsbone': 1, 'trend5': 1, 'legislatureromney': 1, 'reboots': 1, '72nd': 1, 'weekinstitutionalized': 1, 'lessonsstate': 1, 'reconsidered': 1, 'wren': 1, 'attackreddi': 1, 'wip': 1, 'directedthe': 1, 'weekno': 1, 'failuredonald': 1, 'rift': 1, '14day': 1, 'cleansewatch': 1, 'conventionsthe': 1, 'photosflorida': 1, 'headbutts': 1, 'outthieves': 1, 'docentsepa': 1, 'emissions': 1, 'apocalyptic': 1, 'hellscape': 1, 'becomeis': 1, 'futurekid': 1, 'pencils': 2, 'knuckles': 1, 'uppokemon': 1, 'boutiqueunclear': 1, 'fountain': 1, 'ineducation': 1, 'evidencepoll': 1, 'collapses10': 1, 'colorized': 1, 'lighttormund': 1, 'hearttgi': 1, 'saucereport': 1, 'swimmingis': 1, 'sick8': 1, 'herbs': 1, 'disease10': 1, 'csectioncongress': 1, 'legislated': 1, 'pricey': 1, 'scarves': 1, 'emailwe': 1, 'adoptee': 1, 'suicidebiden': 1, 'weedpoll': 1, 'courseon': 1, 'gettingmissing': 1, 'betterpaying': 1, 'jobman': 1, 'infrequently': 1, 'carerumsfeld': 1, 'donenew': 1, 'snail': 1, 'carread': 1, 'irmathis': 1, 'secretaliens': 1, 'alienssilvio': 1, 'prostituteshares': 1, 'hazmatsuit': 1, 'newsauthor': 1, 'ghostwriters': 1, 'behalftrump': 1, 'sleepmom': 1, 'suitstarbucks': 1, 'slapdont': 1, 'yourestaurant': 1, 'yeardepraved': 1, '100000aday': 1, 'habitsixthgraders': 1, 'scrutinypolice': 1, 'spoilersdick': 1, 'pridemy': 1, 'pilgrimagemarital': 1, 'changebrock': 1, 'turners': 1, 'mugshot': 1, 'textbookroots': 1, 'weekendwe': 1, 'episcopalians': 1, 'coupleselecting': 1, 'considernemesis': 1, 'coversouth': 1, 'halfbrother': 1, 'leadertrump': 1, 'garbageholding': 1, 'accountableguy': 1, 'storebill': 1, 'watterson': 1, 'shreds': 1, 'calvin': 1, 'hobbes': 1, 'spite9': 1, 'attendants': 1, 'insanehillary': 1, 'subminimum': 1, 'disabilitiespostmaster': 1, 'zipcode': 1, 'riskmild': 1, 'hasslevatican': 1, 'canonizes': 1, 'diegop': 1, 'spaceimprove': 1, 'healthwatch': 1, 'sophie': 1, 'marchsmalltown': 1, 'peoplejustice': 1, 'baker': 1, 'caseoscars': 1, 'sadkevin': 1, 'spacey': 2, 'oscarsrichard': 1, 'estrogen': 1, 'tentits': 1, 'ithillary': 1, 'heartoutfit': 1, 'officershark': 1, 'oahus': 1, 'shorejohn': 1, 'masseuse': 1, 'putinandrea': 1, 'tantaros': 1, 'testpushy': 1, 'hermit': 1, 'ininside': 1, 'colorssix': 1, 'turkeytv': 1, 'edgy3': 1, 'nowmonkey': 1, 'videojustice': 1, 'bodyjames': 1, 'accusationsthe': 1, 'declaration': 1, 'killercouple': 1, 'cakesthese': 1, 'radically': 1, 'daycan': 1, 'twitter12yearold': 1, 'rubiojenny': 1, 'beavan': 1, 'clap': 1, 'oscarsinside': 1, 'fashions': 1, 'yourself30millionyearold': 1, 'itold': 1, 'ol': 1, 'bastardobama': 1, 'lyricspaul': 1, 'workingcuban': 1, '21gun': 1, 'squadchina': 1, 'namesgarroting': 1, 'banrising': 1, 'sailboatwatch': 1, 'professes': 1, 'tina': 3, 'fey': 3, 'sexysenators': 1, 'domenici': 1, 'oftenheres': 1, 'debatesso': 1, 'twicederek': 1, 'movebudweiser': 1, 'adsnowman': 1, 'sucksprime': 1, 'rib': 1, 'mostmat': 1, 'kearney': 1, 'toursuicide': 1, 'caseschwarzenegger': 1, 'horseman': 2, 'apocalypsehowie': 1, 'spotsinsurance': 1, 'transplant44': 1, 'packages': 1, 'detonated': 1, 'treedave': 1, 'reprises': 2, 'chappelles': 1, 'snlreport': 1, 'vacationdonald': 1, 'enquirer': 1, 'loomstrumps': 1, 'misguidedparade': 1, 'interchangeable': 1, 'starlets': 1, 'delights': 2, 'populacekermit': 1, 'shaggys': 1, '2000s': 1, 'goldcoddled': 1, 'potted': 1, 'outsidenew': 1, 'magdalene': 1, '12soccer': 1, 'supporterbuilt': 1, 'spiritmissouri': 1, 'redefines': 1, 'lobbyistonlawmaker': 1, 'giftatt': 1, 'windowless': 1, 'towerthe': 1, 'salaam': 1, 'youcollateral': 1, 'sorroworlando': 1, 'interim': 1, 'memorialbidens': 1, 'visit21': 1, 'parentsnow': 1, 'whiteonwhiteobama': 1, 'mandela7': 1, 'fitdespondent': 1, 'briefinguma': 1, 'ethan': 1, 'hawke': 1, 'sire': 1, 'homo': 1, 'celbritansmy': 1, 'outmccain': 1, 'brandwatch': 1, 'timbaland': 1, 'memoirpower': 1, 'dekkers': 1, 'postballethillary': 1, 'bulliesno': 1, '2000taliban': 1, 'afghanistanhollywood': 1, 'stewartkourtney': 1, 'otherstandards': 1, 'fridgeoverfunded': 1, 'bandrelationship': 1, 'woodcraftstwitter': 1, 'sunglassesapproved': 1, 'catcallsthe': 1, 'witching': 1, 'revisitedbernie': 1, 'statesrestaurateur': 1, 'chang': 1, 'culturefocused': 1, 'company7': 1, 'daya': 1, 'protestersinfant': 1, 'relationshipa': 1, 'trumpyes': 1, 'sweaterwayne': 1, 'harpooning': 1, 'spree': 1, 'pointtrump': 1, 'himpsychiatrists': 1, 'congresscommunity': 1, 'massesin': 1, 'depth': 1, 'veil': 1, 'thereexxonmobil': 1, 'yeardow': 1, '400study': 1, 'salarieszero': 1, 'thirty': 1, 'homeowners': 1, 'dealwhy': 1, 'attractionad': 1, 'proverb': 1, 'meanshawaii': 1, 'medically': 1, 'suicideany': 1, 'sourceswhat': 1, 'nonfarm': 1, 'payroll': 1, 'estateheres': 1, 'boywatch': 1, 'badusps': 1, 'princeinspired': 1, 'workersbejewel': 1, 'twinkle': 1, 'tushrelaxing': 1, 'workkerry': 1, 'behaviorisraeli': 1, 'hezbollah': 1, 'syriastudy': 1, 'bodycongress': 1, 'antisocial': 1, 'insecurity': 1, 'actkid': 1, 'mickey': 2, 'hugcompanys': 1, 'basethe': 1, 'havingworld': 1, '1939': 1, 'polandotherwise': 1, 'savvy': 1, 'mascara': 1, 'makers': 1, 'againdollar': 1, 'quarterhow': 1, 'capitalize': 1, 'milestone': 3, 'rulingembattled': 1, 'rescuethe': 1, 'branding': 1, 'startgov': 1, 'markell': 1, 'nominationgun': 1, 'actstock': 1, '1928': 1, 'investorsdelta': 1, 'againsanders': 1, 'michiganstyle': 1, 'ohiosylvester': 1, 'stallone': 1, 'himselfolympic': 1, 'ariana': 1, 'kukors': 1, 'sportsall': 1, 'terrifiedemployees': 1, '3hourlong': 1, 'seduction': 1, 'trainingat': 1, 'chanceshouse': 1, 'lunchesthis': 1, 'visualize': 1, 'heartbeat': 1, 'citycalifornia': 1, 'lawexwall': 1, 'banker': 2, 'tipsdonut': 1, 'awfully': 1, 'ambitiousthe': 1, 'cancerkate': 1, 'mckinnons': 1, 'snlwhen': 2, 'itvoter': 1, 'disinterest': 1, 'expensivesome': 1, 'speechworkers': 1, 'unionizeava': 1, 'guysexperts': 1, 'isislupe': 1, 'falsesprints': 1, 'monthtearyeyed': 1, 'daynation': 1, 'romantically': 1, 'losersgarrison': 1, 'keillor': 1, 'deflates': 1, 'sightrevor': 1, 'showjury': 1, 'exemployee': 1, 'roscoes': 1, 'waffles': 2, '16m': 1, 'streetjacobs': 1, 'ulbrichtballet': 1, '2014harrison': 1, 'nowpretty': 1, 'stuffa': 1, 'playful': 2, 'adviceman': 1, 'theorysenate': 1, 'knowledgestressed': 1, 'earsgirlfriend': 1, 'alikefrench': 1, 'foodshow': 1, 'love20yearold': 1, 'fianc': 1, 'armdesperate': 1, 'wheel': 1, 'swear': 2, 'wordselizabeth': 1, 'ohiosurging': 1, 'rechargeable': 1, 'batteries': 2, 'americaus': 1, 'chinais': 1, 'isecretary': 1, 'raspberry': 1, 'margaritastarr': 1, 'sittin': 1, 'songacousticguitarwielding': 1, 'americasecurity': 1, 'auditoriumspanish': 1, 'curbing': 1, 'nihilistic': 1, 'forwarddawn': 1, 'officetrans': 1, 'questionsjustice': 1, 'countrymexican': 1, 'winssuccess': 1, 'relationshipschina': 1, 'jinping': 1, 'indefinitelyobamacare': 1, 'be98': 1, 'manicdepressiveconservative': 1, 'ryanhospitals': 1, 'zoneaclu': 1, 'speechformer': 1, 'munsters': 1, 'actingwomenonly': 1, 'considerationsless': 1, 'countriescooking': 1, 'bluefish': 1, 'saor': 1, 'venetian': 1, 'favoritefed': 1, 'wells': 1, 'scandals6': 1, 'amber': 1, 'riley': 1, 'iconvideogame': 1, 'groundnew': 1, 'dapl': 1, 'downa': 1, 'nukeshungover': 1, 'panelmarvels': 1, 'shehulk': 1, 'purposeman': 1, 'fad': 2, 'flavorful': 1, 'goodlast': 1, 'tortoisebreaking': 1, 'nothingtresspasser': 1, 'saybilly': 1, 'slickers': 1, 'westworldnew': 1, 'pfizer': 1, 'patentsits': 1, 'welltimetraveling': 1, 'commodities': 1, 'trader': 1, 'hog': 1, 'futureaziz': 1, 'snls': 1, 'hostobama': 1, 'hardship': 2, 'unionbob': 1, 'barr': 1, 'twoparty': 1, 'waaah': 1, 'waaahsomeone': 1, 'inserted': 1, 'soundsjohn': 1, 'gobi': 1, 'desertjerky': 1, 'prankcall': 1, 'fandear': 1, 'dokate': 1, 'coronation': 1, 'anniversaryobamas': 1, 'bookkelly': 1, 'suppressed': 1, 'outmonique': 1, 'goodarea': 1, 'flourishing': 1, 'correspondence': 1, 'mailerdaemonwhat': 1, 'mindif': 1, 'legos': 1, 'timearea': 1, '7hour': 1, 'interruptionofficers': 1, 'peacenepal': 1, 'tamer': 1, 'paperworklocal': 1, 'birthdaythousands': 1, 'flocking': 1, 'actiongun': 1, 'day43': 1, 'dogsrick': 1, 'electrifies': 1, 'racistsbrad': 1, 'testify': 3, 'bicep': 1, 'regimenflynn': 1, 'himselfkosovo': 1, 'hqman': 1, 'himarea': 1, 'whims': 1, 'marketdennys': 1, 'meatyoga': 1, 'stressproblems': 1, 'infowhat': 1, 'cancerhistorical': 1, 'hygenic': 1, 'apportionment': 1, 'paperkanye': 1, 'ridden': 1, 'plantation': 1, 'gotgod': 1, 'zoroastrianismtrumps': 1, 'monumental': 2, 'betrayaltwitter': 1, 'homepod': 1, 'paperstripper': 1, 'throughhousekeeper': 1, 'sassykesha': 1, 'statementembarrassed': 1, 'whiffing': 1, 'lunging': 1, 'toadin': 1, 'breyer': 4, 'confinementnew': 1, 'rumsfeld': 2, 'geopolitical': 1, 'invasionsiblings': 1, 'momroommates': 1, 'sodaa': 1, 'weighty': 1, 'resolutiondont': 1, 'conceiveqtip': 1, 'multipronged': 1, 'swabjohn': 1, 'demoted': 1, 'widowmiley': 1, 'kisspregnancy': 1, 'prettypostal': 1, 'christmasdukes': 1, 'hazzard': 1, 'kitsch': 1, 'valuearbys': 1, 'herethese': 1, 'summerwatch': 1, 'beautifully': 2, 'portray': 1, 'relationshipit': 1, 'skeletonmolly': 1, 'hatchet': 1, 'districtdonald': 1, 'dargers': 1, 'imaginationbush': 1, 'timebritish': 1, 'saytaylor': 1, 'tattootgi': 1, 'americanawatch': 1, 'mayoremperor': 1, 'smeltfather': 1, 'moviesgeorge': 1, 'weaponswhy': 1, 'incheskim': 1, 'baptism': 1, 'jerusalemdoubleentendre': 1, 'scrutinybar': 1, 'materialismstate': 1, 'hungover': 1, 'attorneyqatar': 1, 'covert': 1, 'helpprotesters': 1, 'raidsabu': 1, 'ghraib': 1, 'worldthe': 1, 'problematic': 2, 'punctuation': 1, 'explainedsnapchats': 1, 'snapcash': 1, 'peertopeer': 1, 'safeman': 1, 'mcchicken': 1, 'antibioticsgawker': 1, 'retool': 1, 'sitemalaysian': 1, 'shipments': 1, 'videopolling': 1, 'therewere': 1, 'lipinski': 1, 'weirs': 1, 'moveslarry': 1, 'kudlow': 1, 'connecticuta': 1, 'photographwedding': 1, 'herearea': 1, 'economywatch': 2, 'chill': 1, 'robberyleslie': 1, 'commentating': 1, 'skating': 1, 'dreamtop': 1, 'learnedthe': 1, 'chemicalsglade': 1, 'meadow': 1, 'extinguisherwoman': 1, 'deriving': 1, 'lifestyleus': 1, 'housethis': 1, 'drunkentire': 1, 'eyestyra': 1, 'drakes': 1, 'debateterri': 1, 'schiavo': 1, 'embarrassmentdemocrats': 1, 'informantfirst': 1, 'aspnonpriest': 1, 'molestationresearchers': 1, 'smartnessfashion': 1, 'upbeat': 1, 'attitudeit': 1, 'himearlymorning': 1, 'jogger': 1, 'sleepingtoys': 1, 'operations': 1, 'workerspart': 1, 'mindnuns': 1, 'popepolitical': 1, 'moneyholocaust': 1, '88creepy': 1, '18to24yearoldsemojisinthewild': 1, 'instagrammom': 1, 'dishesstephen': 1, 'nazisdonald': 1, 'oscarsthomas': 1, 'piketty': 1, 'francaisbeware': 1, 'bumblercoworker': 1, 'yearireland': 1, 'churchwebsite': 1, 'tripped': 1, 'disappearsteacher': 1, 'cashstrapped': 1, 'centuriesold': 1, 'firearmslava': 1, 'eruption': 1, 'kite': 1, 'photosgovernment': 1, 'commissionnew': 1, 'floppy': 1, 'disksuninformed': 1, 'comprehends': 1, 'swiftparkland': 1, 'presidentformer': 1, 'lyingsmiley': 1, 'scrubbing': 1, 'devour': 2, 'childthe': 1, 'weekwhile': 1, 'waged': 1, 'streetsfriday': 1, 'prelude': 1, 'seasonsnoop': 1, 'needamerica': 1, 'catastropheun': 1, 'riveramerica': 1, 'ferrera': 2, 'tearful': 1, 'griefnation': 1, 'preciousness': 1, 'shootingantivaxxers': 1, 'worriedvigilante': 1, 'handschris': 1, 'watercolor': 1, 'candidacy': 1, 'peaked': 1, 'soonhow': 1, 'floodsturtle': 1, 'bocce': 1, 'balled': 1, 'aroundcoworkers': 1, 'kidsthis': 1, 'donehodors': 1, 'deathwoman': 1, 'rifles': 2, 'sympathize': 1, 'pettinessvolkswagen': 1, 'billiontv': 1, 'nielsenstrump': 1, 'talksmariah': 1, 'sayhere': 1, '2014kids': 1, 'depressionbusinessman': 1, 'schoolgirlmom': 1, 'stacked': 2, 'roster4': 1, 'drooling': 1, 'selftime': 1, 'subscribers': 1, 'closeup': 1, 'cruzs': 1, 'facetexas': 1, 'changejimmy': 1, 'dispute': 1, 'heirsretarded': 1, 'party4': 2, 'farmtoschool': 1, 'policiesstevie': 1, 'mooni': 1, 'techwendys': 1, 'disapproves': 1, 'marriagestephen': 1, 'gavels': 1, 'mouthbanners': 1, 'dominion': 1, 'sitearea': 1, 'excuses': 1, 'friendmit': 1, 'smithereenkevin': 1, 'homosexualityamerican': 1, '112th': 1, 'electionwomen': 1, 'mediaghostwriter': 1, 'reisers': 1, 'lifewhite': 1, 'allegationsscottish': 1, 'busker': 1, 'gudmunsen': 1, 'scotsman': 1, 'canscientists': 1, 'tiniest': 1, 'hedgehog': 2, 'everunstoppable': 1, 'tonerus': 1, 'deficit': 2, 'cbonhl': 1, 'victimschamillionaire': 1, 'immigrantdad': 1, 'eyelinermuhammad': 1, 'islamophobiafed': 1, 'owndoctors': 1, 'weekgay': 1, 'nominationconfused': 1, 'aljazeera': 1, 'leaderchagas': 1, 'yellow': 1, 'cardpopular': 1, 'singles': 2, 'scroll': 1, 'throughentirety': 1, 'faking': 1, 'happeningcolombias': 1, '52year': 1, 'passeschristianity': 1, 'unanswered': 1, 'prayera': 1, 'toocameron': 1, 'esposito': 1, 'yuletide': 1, 'photodennis': 1, 'neckkirk': 1, 'creflo': 1, 'campaignunited': 1, 'planeimagine': 1, 'messaging': 1, 'preached': 1, 'responsibilityjennifer': 1, 'rearended': 1, 'driverliterally': 1, 'aboutcosme': 1, 'acceptedgraffiti': 1, 'masterwork': 1, 'leaffalse': 1, 'panicchurchgoer': 1, 'weektuesdays': 1, 'africameet': 1, 'crossservice': 1, 'wedconfessions': 1, 'menorth': 1, 'reportarabamerican': 1, 'mediagrandmas': 1, 'racismarea': 1, 'watcherdogs': 1, 'solidespn': 1, 'leesamantha': 1, 'itsan': 1, 'userschristian': 1, 'growingif': 1, 'pastorparentthird': 1, 'stepdad': 1, 'goateecarrie': 1, 'underwood': 1, 'stitchesleonardo': 1, 'oscaroscar': 1, 'sentencingjesse': 1, 'shootpoll': 1, '8ounce': 1, 'withanthropologists': 1, 'democracy60yearold': 1, 'grotesquely': 1, 'hashtagsaltless': 1, 'bulbheated': 1, 'rackdefunct': 1, 'internetmothers': 1, 'elegant': 2, 'wow': 1, 'momfriend': 1, 'tablereport': 1, 'insomniacs': 1, 'worldsstrongestman': 1, 'competitionsarea': 1, 'visitingwhiteonwhite': 1, 'receivable': 1, 'supervisorselena': 1, 'grimmie': 1, 'concertdonald': 1, 'cycletruth': 1, 'fictiontheres': 1, 'missingnew': 1, 'exxons': 1, 'plans11': 1, 'revisit': 1, 'harassmentnew': 1, 'pfizers': 1, 'ofnew': 1, 'sucksjessica': 1, 'limbsarea': 1, 'eyesore': 1, 'technicianeverything': 1, 'premieredirector': 1, 'schwartz': 1, 'hunchbackbouncer': 1, 'insyrian': 1, 'rebelsheres': 1, 'trademillionaire': 1, 'royce': 1, 'ornamentfrustrated': 1, 'gre': 1, 'booksthese': 1, 'caucuseswalmart': 1, 'epidemic10': 1, 'romcoms': 1, 'avoidlocal': 1, 'brainwashed': 1, 'idiots': 1, 'week55': 1, 'worldin': 1, 'devastated': 1, 'tenacity': 1, 'neighborsis': 1, 'complicatedwedding': 1, 'caterer': 1, 'loveyou': 1, 'melynx': 1, 'heart10': 1, 'cksean': 1, 'mccarthys': 1, 'sketcheswhy': 1, 'chairmanmike': 1, 'startled': 1, 'beware': 2, 'scrawled': 1, 'mirrornation': 1, 'escapethe': 1, 'evertom': 1, 'clancys': 1, 'hardlas': 1, 'tear': 3, 'ricklesanyone': 1, 'sprayed': 1, 'champagneoil': 1, 'eaglea': 1, 'otherfriends': 1, 'funnystep': 1, 'technicolor': 1, 'brazilian': 1, 'motelskfc': 1, 'paleontologists': 1, '24piece': 1, 'legkasich': 1, 'multiterm': 1, 'governorhow': 1, 'local12': 1, 'absurd': 1, 'dating13': 1, 'incurable': 1, 'pubertydeconstructing': 1, 'damores': 1, 'memomother': 1, 'gunned': 1, 'restauranttime': 1, 'natothe': 1, 'meanmortgage': 1, 'industrysean': 1, 'medias': 1, 'statementsphoto': 1, 'masked': 1, 'releasedstudy': 1, 'possessive': 1, 'lovebachelorette': 1, 'firemenconjoined': 1, 'accidentjealous': 1, 'chargerlouis': 1, 'fallonkinky': 1, 'legsformer': 1, 'commish': 1, 'copps': 1, 'seenthe': 1, 'egocentric': 1, 'artworld': 1, '1987': 1, 'likehistorical': 1, 'twenty': 1, 'topmost': 1, 'presentamerica': 1, 'courtindian': 1, 'manvendra': 1, 'singh': 1, 'gohil': 1, 'groundsdefiant': 1, 'icbma': 1, 'selfless': 1, 'community13': 1, 'situations': 1, 'throughthe': 1, 'pta': 1, 'hellmight': 1, 'rbgs': 1, 'roostwatch': 1, 'openerderek': 1, 'flawlessly': 1, 'hustled': 1, 'golfsexual': 1, 'survivorslondon': 1, 'ceremonies': 1, 'stadiumoklahoma': 1, '37day': 1, 'glossipgop': 1, 'aunties': 1, 'tooleslie': 1, 'merritt': 1, 'freeway': 1, 'guywatch': 1, 'skateboarder': 1, 'downhill': 1, 'mphwalk': 1, 'vmas': 1, 'carpetpieces': 1, 'duckbeyonc': 1, 'surgical': 1, 'remakearea': 1, 'lint': 1, 'cardigan': 1, 'prized': 1, 'stallionkristen': 1, 'bleached': 1, 'dosenate': 1, 'carscc': 1, 'sabathia': 1, 'rehabarea': 1, 'nowcubs': 2, 'arrieta': 1, '8th': 3, 'nohitter': 1, 'gamethese': 1, 'pinterestdoctors': 1, 'centuries': 1, 'cryosleepembracing': 1, 'givingbo': 1, 'bark': 1, 'americanervous': 1, 'urinereport': 1, 'preschooljudge': 1, 'bakers': 1, 'firstamendment': 1, 'rightsstill': 1, 'martenswhy': 1, 'christiansreport': 1, 'survivingmiracle': 1, 'toddlerwhy': 1, 'conditioner': 2, 'showersteve': 1, 'inflamed': 1, 'pulsing': 1, 'meetingchapter': 1, 'manuscriptblack': 1, 'weekunesco': 1, 'designates': 1, 'sitessimpsons': 1, 'offendedcosta': 1, 'ricas': 1, 'feat': 1, 'planetdallas': 1, 'reformfridays': 1, 'weinsteinstate': 1, 'statewild': 1, 'leopard': 2, 'peoplepolice': 1, 'wronged': 2, 'empathybeyonce': 1, 'heartsrussian': 1, 'submarine': 1, 'shipyardspotify': 1, 'bumble': 1, 'tastescript': 1, 'saysman': 2, 'variants': 1, 'multiply': 1, 'breakneck': 1, 'speedit': 1, 'dumbed': 1, 'downtaylor': 1, 'giftshistory': 1, 'millard': 1, 'fillmorepanicked': 1, 'stressing': 2, 'memike': 1, 'rrated': 1, 'movieshuffpost': 1, '26stephen': 1, 'warriorsalex': 1, 'trebek': 1, 'deftly': 1, 'prolongs': 1, 'agonizing': 1, 'talkusain': 1, 'goldacademy': 1, 'illinoistrump': 1, 'muellerwho': 1, 'bondgeorge': 1, 'amazingmiss': 1, 'peacedonald': 1, 'returnstsa': 1, 'ineffective': 1, 'programnot': 1, 'appsrussian': 1, 'encounters': 1, 'officialarea': 1, 'bookstoreprotecting': 1, 'southeast': 1, 'chicagoostensibly': 1, 'heterosexual': 1, 'assesthe': 1, 'shall': 2, 'spareddoj': 1, 'monitoring': 1, 'castilethe': 1, 'justicefans': 1, 'carchase': 1, 'scenesdonald': 1, 'translator': 1, 'fx': 1, 'nword': 1, 'atlantaindira': 1, 'memorythe': 1, 'scoop': 1, 'moviesarea': 1, 'exodus': 1, 'theaterspharmaceutical': 1, 'promotespope': 1, 'uncooperative': 1, 'catholicwhy': 1, 'heartburnking': 1, 'ralph': 1, 'retro': 1, 'referencehopes': 1, 'soviet': 1, 'shortwhats': 1, 'shelfie': 1, 'styleman': 1, 'kidshuffpost': 1, 'helppressure': 1, 'goptrumps': 1, 'court88': 1, '2018happy': 1, 'americahigh': 1, 'dayborder': 1, 'stopman': 1, 'living5': 1, 'suckwoman': 1, 'pornonew': 1, 'android': 1, 'forearm': 1, 'circuitry': 1, 'withinunnamed': 1, 'variantviewing': 1, 'contributing': 1, 'economytheres': 1, 'figuresteens': 1, 'commercialsolympic': 1, 'skier': 1, 'forbidding': 1, 'lifegordon': 1, 'ramsays': 1, 'tipswinneshiek': 1, 'rockdonald': 1, 'conveniently': 1, 'nukesisraeli': 1, 'strifehiv': 1, 'experimentdept': 1, 'markers': 1, 'raccoonsukrainian': 1, 'lyudska': 1, 'podoba': 1, 'sexualities': 1, 'horsescara': 1, 'delevingne': 1, 'paparazzo': 1, 'parisblack': 1, 'ouncesfamily': 1, 'sondccc': 1, 'bidcnn': 1, 'jennerreport': 1, 'sciencebased': 1, 'documentsis': 1, 'strategyaustralian': 1, 'shocks': 1, 'driversyou': 1, 'facebookcoworker': 1, 'chipper': 1, 'fuckertbs': 1, 'networks': 1, 'nielsen': 1, 'ratingsalqaeda': 1, 'hallmarks': 1, 'militarynevada': 1, 'metheres': 1, 'hairmars': 1, 'staggers': 1, 'broketoo': 1, 'talkative': 1, 'cashierinternational': 1, 'operators': 1, 'you32': 1, 'throwback': 2, 'runislamic': 1, 'womensong': 1, 'jd': 1, 'mcpherson': 1, 'bossytonight': 1, 'yetsex': 1, 'dildoshaped': 1, 'boxschwarzenegger': 1, 'costumelook': 1, 'feetas': 1, 'becometania': 1, 'bruguera': 1, 'rightsthis': 1, 'drivingyou': 1, 'cassini': 1, 'petrified': 1, 'administratoruneducated': 1, 'schoolarea': 1, 'guitarair': 1, 'births': 1, 'globedrunk': 1, 'policehalf': 1, 'threatenedchristmas': 1, 'recipesunwatched': 1, 'unblinking': 1, 'eyerobert': 1, 'pattinson': 1, 'fka': 1, 'twigs': 1, 'chateau': 1, 'marmontamerican': 1, 'assholesee': 1, 'crya': 1, 'builtraid': 1, 'nachosupremacist': 1, 'compound': 1, 'guacamolemaking': 1, 'materialsgender': 1, 'everyonescould': 1, 'kellen': 1, 'experimentobamas': 1, 'realfbi': 1, 'encrypted': 1, 'administrationthursdays': 1, 'faceoffthe': 1, 'herethis': 1, 'yeardegrees': 1, 'debtfire': 1, 'supplants': 1, 'choicejosh': 1, 'transparencya': 1, 'dressedtrevor': 1, 'hannitymy': 1, 'ymcaloral': 1, 'homeuncaged': 1, 'futures': 2, 'nowrapper': 1, 'collegeanthony': 1, 'congresspope': 1, 'wraps': 1, 'banado': 1, 'norte': 1, 'slumnative': 1, 'dismal': 1, 'educationbernie': 1, 'timenew': 1, 'scumbag': 1, 'scumbagstranger': 1, 'pugs': 1, 'internetby': 1, 'electionsoops': 1, 'bathroomjohn': 1, 'dowd': 1, 'probeamericans': 1, 'economythese': 1, 'createdcongress': 1, '19this': 1, 'refreshing': 1, 'changevet': 1, 'solidersjerry': 1, 'overtimelenos': 1, 'voicemail': 1, 'laughtermarilinda': 1, 'hampshirethis': 1, 'propose': 1, 'deskthe': 1, 'scaramucci': 1, 'timegirl': 1, 'now8monthold': 1, 'poohs': 1, 'daywhy': 1, 'tidalnew': 1, 'booths': 2, 'ballotquaker': 1, 'assemblyline': 1, 'oops': 1, 'berries': 1, 'incidentdear': 1, 'antimarcoses': 1, 'promarcoses': 1, 'lawnasa': 1, 'happenslamar': 1, 'miraculous': 1, 'improvementbruno': 1, 'crime5': 1, 'costsreport': 1, 'lardfaceexplosions': 1, 'aleppothese': 1, 'rentcomsuperstitious': 1, 'moonmommy': 1, 'sleepovermisbuttoned': 1, 'lunaticnikki': 1, 'scheduling': 2, 'hardreport': 1, 'americansnational': 1, 'quietyou': 1, '6th': 1, 'coverofficials': 1, 'condense': 1, 'wordgop': 1, 'showingmercy': 1, 'patientthe': 1, 'tastertrump': 1, 'kathleen': 1, 'hartnett': 1, 'backyardcolton': 1, 'dixon': 2, 'hannah': 2, 'moviesupport': 1, 'stolencelebrity': 1, 'smokingnational': 1, 'percentbid': 1, 'gawkercom': 1, 'shortare': 1, 'pencesenate': 1, 'trumpproud': 1, 'accountepa': 1, 'dumping': 2, 'drainthese': 1, 'forwardjesus': 1, 'walkon': 1, 'appearanceslightly': 1, 'powertroop': 1, 'gradually': 1, 'withdrawsparrots': 1, 'rightroyal': 1, 'eyesjohn': 1, 'disavowing': 1, 'nazisamandla': 1, 'stenberg': 1, 'dazed': 1, 'magazinedemocratic': 1, 'moneywe': 1, 'reportsschool': 1, 'grass': 2, 'frontsenate': 1, 'againhigher': 1, 'practicesicelandic': 1, 'question14': 1, 'pantsmadeleine': 1, 'albright': 2, 'hellmit': 1, 'testertrump': 1, 'complying': 1, 'threatlife': 1, 'selfstoragearea': 1, 'populationflu': 1, 'staffindiana': 1, 'saysjust': 1, 'muppets': 1, 'nwas': 1, 'yourselfoxford': 1, 'dictionary': 1, 'editionthe': 2, 'saleschool': 1, 'principal': 3, 'comesbaltimore': 1, 'pigeons': 1, 'shitting': 2, 'statues': 2, 'goneand': 1, 'shaq': 1, 'controlletter': 1, 'girlamazing': 1, 'garbage5': 1, 'cheapprotesters': 1, 'ejected': 1, 'pocket': 1, 'constitutionstrevor': 1, 'boringmans': 1, 'hunting': 2, 'tipsfree': 1, 'crotch': 1, 'followmuseums': 1, 'othersjeff': 1, 'tables': 1, 'costcutting': 1, 'slavesis': 1, 'richnatural': 1, 'manpelosi': 1, 'extenders': 1, 'wireredbox': 1, 'touchscreen': 1, 'pornographic': 1, 'featuresisis': 1, 'groupactivists': 1, 'spines': 1, 'confirmationsnato': 1, 'victorytop': 1, 'doubter': 1, 'himmitt': 1, 'partyauthorities': 1, 'kidrichard': 1, 'bransons': 1, 'globalwarming': 1, 'tripswatch': 1, 'demandsman': 1, 'grimly': 1, 'wroughtdonald': 1, 'pollhot': 1, 'awardsironickitschappreciation': 1, 'novelnew': 1, 'comingman': 1, 'reissue': 1, 'macs': 1, 'rumours': 1, 'editionwinners': 1, 'scripps': 1, 'beespy': 1, 'worldfamousfourth': 1, 'sanitizernorth': 1, 'aidwhat': 1, 'profilediversity': 1, 'dumbasses': 1, 'journalismperfect': 1, 'trunkgop': 1, 'upnew': 1, 'anxietythis': 1, 'peak': 2, 'thema': 1, 'lifeline': 1, 'disappearing': 1, 'codwoman': 1, 'changedstudent': 1, 'cssmoke': 1, 'remote5': 1, 'downed': 1, 'hawaiimore': 1, 'authorarkansas': 1, 'certificatesnation': 1, 'reeling': 2, 'megasuccess': 1, 'poppers': 1, 'penguinssenate': 1, 'spendingnyc': 1, 'leverages': 1, 'marketinghilton': 1, 'besta': 1, 'dayit': 1, 'gathering': 2, 'iowa6': 1, 'huntfat': 1, 'strongmom': 1, 'nightgown': 1, 'modeyoga': 1, 'nowgeorge': 1, 'appearancefacereading': 1, 'advantage': 1, 'vomitarkansas': 1, 'enforce': 2, 'primarieshow': 1, 'childstudy': 1, 'russiaiowa': 1, 'lateguardians': 1, 'masterfully': 1, 'impersonatorpresidential': 1, 'accessexercise': 1, 'consideredread': 1, 'officialswill': 1, 'p51': 1, 'dealwatch': 1, 'sorted': 1, 'hogwarts': 1, 'irlallknowing': 1, 'bead': 1, 'storesmark': 1, 'facebookkurrencykookcom': 1, 'reviewthe': 1, 'nightthis': 1, 'studentsheres': 1, 'milliondaddy': 1, 'floor85': 1, 'slaw': 1, 'uneatenlot': 1, 'apartmentone': 1, 'youel': 1, 'chapecoense': 1, 'y': 1, 'piloto': 1, 'bolivianodrunk': 1, 'incites': 1, 'allah': 1, 'policecalifornia': 1, 'campaignrob': 1, 'portman': 1, 'lawsuits': 1, 'immigrationman': 1, 'chairwho': 1, 'youits': 1, 'mapbailiff': 1, 'judge12': 1, 'historymaking': 1, 'worlddont': 1, 'stuffperson': 1, 'roles': 1, 'womenromney': 1, 'turnaround': 1, 'mitty': 1, 'citythis': 1, '1927': 1, 'bookstwitter': 1, 'covertrump': 1, 'ivankas': 1, 'lightheartedtwitter': 1, 'nfldespite': 1, 'armie': 1, 'housekeeping': 1, 'flop': 2, 'officeretirees': 1, 'lowcost': 1, 'countrydeputy': 1, 'representswhat': 1, 'photosmichelle': 1, 'shutters': 1, 'runimmigration': 1, 'whattrumps': 1, 'comingchristian': 1, 'slater': 1, 'dropmom': 1, 'relatable': 2, 'photoswoman': 1, 'transitions': 1, 'pregnantserial': 1, 'unsuspectingthomas': 1, 'followingswiss': 1, 'writhing': 1, 'tentacles': 1, 'devouring': 1, 'francisdonald': 1, 'saysjoe': 1, 'ponytailkim': 1, 'updateclimate': 1, 'parisasexually': 1, 'reproduced': 1, 'sponge': 1, 'herselfdionne': 1, 'warwick': 1, 'kristina': 1, 'brownannoying': 1, 'skydivingbiden': 1, 'forges': 1, 'dokken': 1, 'montharizona': 1, 'saysmom': 1, 'planschristian': 1, 'machineceleb': 1, 'revolutiontime': 1, 'natoconstrictive': 1, 'severs': 1, 'mcadams': 1, 'waistheartshaped': 1, 'jacuzzi': 1, 'clogged': 1, 'again142': 1, 'noticeablereport': 1, 'isisdog': 1, 'pastreport': 1, 'abductions': 1, 'knowsmans': 1, 'handwhy': 1, '20swhite': 1, 'tallhhs': 1, 'repeala': 1, 'secondbysecond': 1, 'commentsastronomers': 1, 'caution': 2, 'damned': 1, 'eclipsead': 1, 'influencedbyadcampaigns': 1, 'demographicbaseball': 1, 'instadium': 1, 'suite': 1, 'momsunited': 1, 'santas': 1, 'poleteacher': 1, 'studentsarea': 1, 'thinkingi': 1, 'maria': 1, 'strangersprison': 1, 'americaboss': 1, 'seanman': 1, 'cunningly': 1, 'houseman': 1, 'recommendations': 1, 'timenight': 1, 'uninterrupted': 1, 'offisraeli': 1, 'friendson': 1, 'doorstep': 1, 'gatlinburg': 1, 'firesmedicine': 1, 'sickerbingewatching': 1, 'depressedhuffpollster': 1, 'tuesdaytrial': 1, 'adjourned': 1, 'dataman': 1, 'islocal': 1, 'outletbike': 1, 'helmetinspired': 1, 'beatingolder': 1, 'omran': 1, 'daqneesh': 1, 'sustained': 1, 'airstrikehillary': 1, 'campaignshy': 1, 'samplemans': 1, 'contribution': 2, 'periodically': 1, 'helpdecision': 1, 'emission': 1, 'itpersonnel': 1, 'enjoyed': 2, 'youman': 2, 'gamed': 1, 'raphaelnras': 1, 'dana': 1, 'loesch': 1, 'shootingswhy': 1, 'bonda': 1, 'overdue': 1, 'trumptrump': 1, 'die85yearold': 1, 'rooma': 2, 'menaced': 1, 'doorstepthe': 1, 'tinderbox': 1, 'africapacific': 1, 'quarantined': 1, 'shipdating': 1, 'technologyman': 1, 'cheetos': 1, 'convictednorman': 1, 'deadhouse': 1, 'reauthorizes': 1, 'lawnew': 1, 'deservenations': 1, 'nummierman': 1, 'thanksgivingchloe': 1, 'independencepolice': 1, 'assgrit': 1, 'strategynj': 1, 'wayarea': 1, 'herwatch': 1, 'reportsunday': 1, 'roundupdescription': 1, 'hotdog': 1, 'picnicsingle': 1, 'minimumwage': 1, 'momenthussein': 1, 'udays': 1, 'nutsack': 1, 'shockerfyre': 1, 'customersbiologists': 1, 'anonymously': 1, 'sacramentocow': 1, 'milking': 3, 'op': 1, 'curdled': 1, 'liquidcomey': 1, 'bolsters': 1, 'trumpchocolate': 1, 'barrelbaseball': 1, 'slugger': 1, 'womenpaul': 1, 'backfiresthis': 1, 'againarea': 1, 'yousteven': 1, 'cannesthe': 1, 'silencea': 1, 'steaksgrandmother': 1, '10step': 1, 'denwednesdays': 1, 'matterswhat': 1, 'regarding': 1, 'treatments': 1, 'ebolapreserving': 1, 'phoenician': 1, 'tyre': 1, 'eastpresumed': 1, 'deadthese': 1, 'sparkly': 1, 'geode': 1, 'worldcoworkers': 1, 'exerciseare': 1, 'selfish': 2, 'selfresponsiblecomedian': 1, 'thanksgiving86yearold': 1, 'databasestrangulation': 1, 'magazinejohn': 1, 'codepink': 1, 'silencearea': 1, 'tantricmaking': 1, 'debatekam': 1, 'gymracist': 1, 'merely': 1, 'misspokecops': 1, 'detaining': 1, 'vegetationnation': 1, 'soonwho': 1, 'obamacareidina': 1, 'menzel': 1, 'anthemreport': 1, 'cafsnew': 1, 'stakepathetic': 1, 'whimhollywood': 1, 'noncelebrity': 1, 'handsmy': 1, 'mei': 1, 'hardhow': 1, 'playlist': 1, 'corny': 2, 'djsgod': 1, 'usif': 1, 'sully': 1, 'gradea': 1, 'remarkschilling': 1, 'minoritylabor': 1, 'enjoyharveys': 1, 'rainfall': 1, 'worsethe': 1, 'newwoman': 1, 'participation': 1, 'patient3': 1, 'proven': 1, 'incomeecoconscious': 1, 'synergyall': 1, 'males': 1, 'dudleypolitico': 1, '2016an': 1, 'antisemitismspotify': 1, 'representing': 1, 'young3': 1, 'relieve': 1, 'chiboyfriend': 1, 'detailxavier': 1, 'dolan': 1, 'farmlittle': 1, 'dojos': 1, 'trophies': 1, 'cowering': 1, 'fearconcerned': 1, 'arsenic': 1, 'elementsstate': 1, 'libyawhy': 1, 'carolinawhat': 1, 'aroundjohn': 1, 'emotion': 1, 'exhibitwhat': 1, 'trumpchristopher': 1, 'plummer': 1, 'somewhereviolent': 1, 'terrific': 1, 'onceroommates': 1, 'speak10': 1, 'partymerkel': 1, 'repulsive': 1, 'charlottesvilleromney': 1, 'horsesplant': 1, 'yourhode': 1, 'tomorrowwork': 1, '34elaborate': 1, 'facilitates': 1, 'omission': 1, 'boyfriendwoman': 1, 'himcongressional': 1, 'ad25yearold': 1, 'whatsappgeorge': 1, 'hairst': 1, 'quotaand': 1, 'isreport': 1, 'soonsaddle': 1, '2how': 1, 'attemptwhen': 1, 'it8': 1, 'knowromney': 1, 'electionandy': 1, 'emmythe': 1, 'lunchnation': 1, 'extrump': 1, 'reabsorblawsuit': 1, 'chihuly': 1, 'plagiarizing': 1, 'workremembering': 1, 'dinaphiladelphia': 1, 'overboard': 1, 'bell13': 1, 'companyit': 1, 'handicapped': 1, 'sidewalkgiving': 1, 'kickline': 1, 'sickin': 1, 'louisi': 1, 'grapevine': 1, 'motowns': 1, 'prospects': 1, 'upmiami': 1, 'jobtrump': 1, '4731st': 1, 'droptrumps': 1, 'unpresidentialwhos': 1, 'guytrump': 1, 'programurban': 1, 'namesformer': 1, 'blackwater': 1, 'hatenorth': 1, 'scientisttv': 1, 'ichabod': 1, 'tnts': 1, 'headless': 1, 'showtimes': 1, 'riderwoman': 1, 'warrantstarr': 1, 'baylor': 1, 'scandalsomebody': 1, 'trophyenvironmental': 1, 'showeringsean': 1, 'bandeadly': 1, 'peoplewelcome': 1, 'contextdriven': 1, 'marketingthis': 1, 'nordstromfourthgraders': 1, 'oversimplifies': 1, 'reportslowes': 1, '2way': 1, 'ladder': 1, 'downoverstockcom': 1, 'programmingpublic': 1, 'fastdepleting': 1, 'nonrenewable': 1, 'resource': 1, 'skyrocketsa': 1, 'priyanka': 1, 'americagroom': 1, 'partymean': 1, 'spongebob': 1, 'squarepants': 1, 'nominationsus': 1, 'bahamasun': 1, 'blacklists': 1, 'syriapit': 1, 'grassveteran': 1, 'monthsmore': 1, 'trumppug': 1, 'wrongtheater': 1, 'caesar': 1, 'controversybernie': 1, 'rouseycaptivating': 1, 'personnelmadonna': 1, 'clintonnprs': 1, 'format': 1, 'softspoken': 1, 'guysalgerian': 1, 'causescamera': 1, 'griffith3': 1, 'healthgraphic': 1, 'outcrydisgusting': 1, 'gyro': 1, 'midnight15': 1, 'lol': 1, 'hardjemele': 1, 'nabjs': 1, 'breakhappy': 1, 'childfree': 2, 'announcing': 1, 'yearthe': 1, 'carclinton': 1, 'unnamed': 1, 'joseph': 3, 'ruckeyserclinton': 1, 'hitchhikes': 1, 'jazzfestmichael': 1, 'everywherehillary': 1, 'carolinadads': 1, 'reindeer': 1, 'fearsan': 1, 'wishnobody': 1, 'punch': 2, 'partynfl': 1, 'jerseyim': 1, 'pursesbeing': 1, 'fearbiden': 1, 'felonies': 1, 'applicationbaywatch': 1, 'flops': 1, 'officepot': 1, 'potent': 1, 'psychosis12': 1, 'kongarea': 1, 'ridiculousjayz': 1, 'beyoncted': 1, 'duplicitybruce': 1, 'eligible': 1, 'parole': 1, 'murdersbush': 1, 'depress': 1, 'markets12': 1, 'monthsartist': 1, 'sketchbook': 1, 'uncomfortablehow': 1, 'monthstrumps': 1, 'trumpskywriter': 1, 'trailed': 1, 'skyeditorone': 1, 'dinners': 1, 'messthese': 1, 'recharge': 1, 'drivecompelling': 1, 'cubawhen': 1, 'dysphoria': 1, 'compounds': 1, 'recoveryinaugural': 1, 'rulesthe': 2, 'weektheir': 1, 'intent': 1, 'politicsiraq': 1, 'beheading': 2, 'rerunsstage': 1, 'broadways': 1, 'gerard': 1, 'alessandriniarea': 1, 'himauto': 1, 'install': 1, 'suvsright': 1, 'mcgorry': 1, 'actorwill': 1, 'rosenstein': 1, 'matterrussian': 1, 'alina': 1, 'zagitova': 1, 'teammatebaron': 1, 'cleancourageous': 1, 'herdont': 1, 'hermadonna': 1, 'fbombs': 1, 'marchnothing': 1, 'flyswattin': 1, 'verge': 2, 'selfrealization': 1, 'godone': 1, 'homophobicflag': 1, 'halfstaffgeniuses': 1, 'mustwatch': 1, 'formation': 1, 'laws13': 1, 'derail': 1, 'resolutionsevil': 1, 'kungfu': 1, 'legions': 1, 'timedo': 1, 'eathanes': 1, 'noway': 1, 'pantiesreflecting': 1, 'daywomen': 1, 'philanthropistteen': 1, 'maci': 1, 'bookout': 1, 'girlbush': 1, 'timmy': 1, 'blanchard': 1, 'actgop': 1, 'delegate': 1, 'supporterswomen': 1, 'represented': 1, 'blacklistkiller': 1, 'uterus': 1, 'contextthe': 1, 'againkarzai': 1, 'selfstudy': 1, 'yourepublicans': 1, 'electionpuerto': 1, 'puig': 1, 'gold10': 1, 'diane': 1, 'keatons': 1, 'bookearths': 1, 'completion': 1, 'reflect': 3, 'habitslofts': 1, 'anticipated': 1, 'herebelt': 1, 'childhillary': 1, 'policies12': 1, 'phillips': 1, 'hustle': 1, 'jasmine': 1, 'buyers': 2, 'philomena': 1, 'screenwriting': 2, 'class14word': 1, 'stretched': 1, 'pagestank': 1, 'window15000': 1, 'endeavor': 1, 'culminate': 1, 'run3': 1, 'poundsman': 1, 'intercept': 1, 'thingswho': 1, 'questionbreitbart': 1, 'postlondon': 1, 'attacklos': 1, '138': 1, 'usiraqi': 1, 'retake': 1, 'nimrud': 1, 'isisdesperate': 1, 'okays': 2, 'babysittertelescope': 1, 'showdownromney': 1, 'caterers8': 1, 'mancherokee': 1, 'winnersound': 1, 'technicians': 1, 'sutherland': 1, 'stairwell': 1, 'optimal': 1, 'voiceover': 1, 'toneto': 1, 'gazafredericks': 1, 'anchorage': 1, 'crotchless': 1, 'underwearsenior': 1, 'armys': 1, 'itselfnaked': 1, 'leadershipsenate': 1, 'gratuity': 1, 'provisions': 2, 'moreryan': 1, 'fishfox': 1, 'newsreport': 1, 'punctured': 1, 'yetdreamworks': 1, 'skg': 1, 'jh': 1, 'sixyear': 1, 'dealhow': 1, 'giftthe': 1, 'brablake': 1, 'noticingjail': 1, 'deathstudy': 1, 'unprepared': 1, 'stabilitymy': 1, 'pleasures': 1, 'iibush': 1, 'soilarea': 1, 'jewishbritish': 1, 'cilla': 1, '72linebacker': 1, 'genocidedude': 1, 'knit': 1, 'libationsmadeleine': 1, 'congratulates': 2, 'jen': 2, 'coachjk': 1, 'sportfrench': 1, 'frenchgiants': 1, 'betrayed': 1, 'eaglesfat': 1, 'shaming': 1, 'heartlobbying': 1, 'lowscambridge': 1, 'hitlerthe': 1, 'drugwhole': 1, 'maytag': 1, 'riskmichael': 1, 'anthropomorphic': 1, 'giraffe': 1, 'lawyerwhat': 1, 'aarp': 1, 'candidateswidower': 1, 'wearalzheimers': 1, 'todayarmy': 1, '120year': 1, 'airclinton': 1, 'airlifts': 1, 'headquartersa': 1, 'relationsjared': 1, 'placecorey': 1, 'somewherepaul': 1, 'lynde': 1, 'impersonation': 1, 'friendsmother': 1, 'seasonhoward': 1, 'scandalbernie': 1, 'caucusnew': 1, 'finest': 1, 'richestsmithsonian': 1, 'mcdermott': 1, 'practicecould': 1, 'ladiesbrexit': 1, 'legacymedia': 1, 'temperaturescyclone': 1, 'thousandsbully': 1, 'castreport': 1, 'comfortably': 1, 'streetsworking': 1, 'alloutside': 1, 'summermom': 1, 'daughtersgrammy': 1, 'victor': 1, 'musical2016': 1, 'politicslawrence': 1, 't1': 1, 'tourblender': 1, 'companytrevor': 1, 'presidentegyptian': 1, 'grip': 1, 'testegyptians': 1, 'toppling': 1, 'inwoman': 1, 'requesting': 1, 'epiduraltrump': 1, 'koreaacclaimed': 1, 'falsemarine': 1, 'iraqichild': 1, 'useless': 1, 'familyflorida': 1, 'senatorgratitude': 1, 'inescapable': 1, 'spiralobama': 1, 'drawnout': 1, 'ordeal': 1, 'tostill': 1, 'offthing': 1, 'years6': 1, 'negate': 1, 'otherrand': 1, 'hawksame': 1, 'polled': 1, 'constitutionality': 2, 'reformmore': 1, 'illnational': 1, 'stored': 1, 'threering': 1, 'binderman': 1, 'overthrow': 1, 'governmentla': 1, '88million': 1, 'campusesstatue': 1, 'statuelisa': 1, 'kittenswhy': 1, 'comedyyour': 1, 'dramasauthorities': 1, 'itjoe': 1, 'stalinirans': 1, 'spiderwoman': 1, 'realschool': 1, 'biasspilled': 1, 'dishing': 1, 'daphnesenator': 1, 'nothingdont': 1, 'matteramerican': 1, 'oscarbrie': 1, 'larson': 1, 'workoutsaerial': 1, 'reeducation': 1, 'campsmarvel': 1, 'variant': 1, 'coversmosul': 1, 'saysstudy': 1, 'sidebysidethe': 1, 'planetsteven': 1, 'blackhawk': 1, 'filmmike': 1, 'defects': 1, 'programhugging': 1, '76000': 1, 'percentthe': 1, 'gi': 1, 'secretarydemocrats': 1, 'wee': 1, 'lowsave': 1, 'timberlaketom': 1, 'warsoldier': 1, 'someplace': 1, 'tropical': 1, 'nextjoe': 1, 'classdonald': 1, 'falwell': 1, 'inwhat': 1, 'liketechview': 1, 'linus': 1, 'torvalds': 1, 'linuxrobin': 1, 'threeday': 1, 'freeassociation': 1, 'bingeselfconscious': 1, 'flasher': 1, 'coat8bit': 1, 'icons': 2, 'yesprovences': 1, 'pont': 1, 'du': 1, 'gard': 1, 'greatness': 1, 'romefunniest': 1, 'weekpolitical': 1, 'inequalitynothing': 1, 'cosplaythe': 1, 'aboutnew': 1, 'guycongress': 1, 'shutdownscience': 1, 'factsarrested': 1, 'guiltythe': 1, 'onenightonly': 1, 'fetetollbooth': 1, 'barfree': 1, 'lunches': 1, 'starvingnyt': 1, 'readoffice': 1, 'sentientparty': 1, 'drying': 1, 'happierruptured': 1, 'backpackheroes': 1, 'medicineread': 1, 'debatetelevision': 1, 'stagekeith': 1, 'ellison': 1, 'ahmed18yearold': 1, 'soldierarea': 1, 'tuscan': 1, 'name16': 1, 'scourge': 1, 'marriage5': 1, 'estranged': 1, 'childmike': 1, 'baseballarea': 1, 'atvvendingmachine': 1, 'deploykate': 1, 'hudsons': 2, 'onhow': 1, 'humiliationegyptian': 1, 'thistrump': 1, 'statesgenius': 1, 'datespregnant': 1, 'rowland': 1, 'formfitting': 1, 'gowntom': 1, 'seasteve': 1, 'fkup': 1, 'commercialpiece': 1, 'kritzer': 1, 'teal': 1, 'wicksnew': 1, 'decompositionthe': 1, 'monthhuffpost': 1, '8kids': 1, 'dietsone': 1, 'deas': 1, 'traffickers': 1, 'peacebanking': 1, 'carefriends': 1, 'toothat': 1, 'showan': 1, 'intolerant': 1, 'chickenanne': 1, 'response8': 1, 'bayus': 1, 'internetfew': 1, 'filmformer': 1, 'providence': 1, 'cianci': 1, '74this': 1, '23monthold': 1, 'recited': 1, 'doingpalestinian': 1, 'solutionthing': 1, 'girlfriendtalented': 1, 'kiddos': 1, 'talentatt': 1, 'directv': 1, 'acquisition': 1, 'sundayhouse': 1, 'communitiespostmaster': 1, 'workingwe': 1, 'titlewatch': 1, 'goslings': 1, 'moveslibyan': 1, 'fulltime': 1, 'jobsa': 1, 'leaps': 1, 'impalafat': 1, 'girlobama': 1, 'action5': 1, 'perfectforpacking': 1, 'saladshippie': 1, 'isit': 1, 'collegeprescription': 1, 'incyber': 1, 'retailers': 2, 'terabytes': 1, 'informationkod': 1, 'showmustgoon': 1, 'talentstudy': 1, 'ineffectiveblackish': 1, 'hollywooddaddy': 1, 'mommy': 2, 'timetim': 1, 'suggestive': 2, 'commentsthe': 1, 'exclusivitysouth': 1, 'hungrypelosi': 1, 'authoritarian': 1, 'impulses': 1, 'themlone': 1, 'festivalholiday': 1, 'styletrump': 1, 'harward': 1, 'jobnurse': 1, 'diemarriage': 1, 'mondaydamon': 1, 'albarn': 1, 'denmark': 1, '5hour': 1, 'setobama': 1, 'cavs': 1, 'houseinnocent': 1, 'prisonmidlife': 1, 'alzheimerscan': 1, 'minutesbehold': 1, 'majesty': 1, 'turkeya': 1, 'massively': 1, 'disengaged': 1, 'workforce': 1, 'decksole': 1, 'thisgobbler': 1, 'seerussias': 1, 'offofficemax': 1, 'bigwhat': 1, 'messageswhy': 1, 'returnarea': 1, 'coronachance': 1, 'staywokeandvote': 1, 'selfdefensecarly': 1, 'redid': 1, 'rebootstates': 1, 'lawjames': 1, 'dyson': 1, 'dryerleno': 1, 'oj': 1, 'jokereligious': 1, 'sidney': 1, 'adlertai': 1, 'realistically': 1, 'weekhoratio': 1, 'sanz': 1, 'emmysstudy': 1, 'findjj': 1, 'oscarssowhitearea': 1, 'strategyparents': 1, 'accomplishmentconan': 1, 'coped': 1, 'downjennifer': 1, 'brad': 2, 'pittgood': 1, 'karma': 1, 'turban': 1, 'boyfive': 1, 'stupendous': 1, 'buglers': 1, 'syriagirlfriend': 1, 'underneath': 1, 'monthsreport': 1, 'midsize': 1, 'classman': 1, 'urinating': 1, 'trumpdisgusted': 1, 'tsa': 1, 'patdownsman': 1, 'strapped': 1, 'backinperson': 1, 'rightlocal': 1, 'obituaryclipping': 1, 'hobbysan': 1, 'carolinas': 1, 'lawblack': 1, 'profanity': 1, 'theological': 1, 'reflectionhillary': 1, 'organized': 1, 'labors': 1, 'endorsementfont': 1, 'smallstudents': 1, 'themwrapped': 1, 'labeled': 1, 'spare': 2, 'bedroomtina': 1, 'societyhot': 1, 'highwayssudden': 1, 'eitherteslas': 1, 'scrambledeath': 1, 'homicidehistorical': 1, 'tricornered': 1, 'shoessweet': 1, 'briar': 1, 'homogenization': 1, 'educationthe': 1, 'warwhy': 1, 'thiel': 2, 'courtlocal': 1, 'oddball': 1, 'expends': 1, 'thingskid': 1, 'hugged': 1, 'plungethe': 1, 'endnew': 1, 'rendering': 1, 'breasts10pack': 1, 'bracing': 2, 'alone50': 1, 'fridayftc': 1, 'devry': 1, 'defraudedyou': 1, 'masochistssyrias': 1, 'bloodshed': 1, 'photoshootin': 1, 'machiavellian': 1, 'masterstoke': 1, 'invitationtearful': 1, 'stool': 1, 'tochelsea': 1, 'womanthese': 1, 'maroonedmeryl': 1, 'oscarsstephen': 1, 'crazythis': 1, 'identitybigot': 1, 'therecow': 1, 'eclipseabc': 1, 'reannounces': 1, 'morenate': 1, 'futuremeghan': 1, 'weddinggop': 1, 'startchita': 1, 'rivera': 2, 'carnegie': 1, 'hallpapal': 1, 'infallibility': 1, 'invoked': 1, 'wordhamas': 1, 'uprising': 1, 'moveheres': 1, 'worldthis': 1, 'grrrl': 1, '90spristine': 1, 'inspectorssports': 1, 'missingtwin': 1, 'peaks': 1, 'cliffhangerbox': 1, 'bodymateblack': 1, 'freshmen': 1, 'lynchingsmillions': 1, 'directing': 1, 'warsfridays': 1, 'responsesen': 1, 'billwhere': 1, 'gowhat': 1, 'greenwoods': 1, 'joshua': 1, 'beals': 1, 'chicagoblood': 1, 'representatives': 1, 'speakeralice': 1, 'waters': 1, 'digesting': 1, 'eatmissing': 1, 'headlinesshould': 1, 'weaponshiring': 1, 'businessdemocrats': 1, 'probelessons': 1, 'kodakhere': 1, 'debatecall': 1, 'violenceselfdescribed': 1, 'dragonriders': 1, 'pern': 1, 'timeinnocent': 1, 'unrepentantbill': 1, 'billchildhood': 1, 'emailsdonald': 1, 'playshoddy': 1, 'chinesemade': 1, 'collapsestexas': 1, 'chickenonchicken': 1, 'violence9000': 1, 'usconnecticut': 1, 'zonethousands': 1, 'speechthis': 1, 'women5': 1, 'justicegoogle': 1, 'doodle': 1, 'planetary': 1, 'waymans': 1, 'knifepoint': 1, 'videotrumps': 1, 'lanebrazil': 1, 'austerityin': 1, 'unionizekim': 1, 'cattralls': 1, 'homemet': 1, 'janitors': 1, 'hurrying': 1, 'crucified': 1, 'lobbytrumps': 1, 'qaedathis': 1, 'itexwife': 1, 'abuseclimate': 1, 'moredog': 1, 'dunhamtreasury': 1, 'billhilary': 1, 'taseredbill': 1, 'retrial': 1, 'cooled': 1, 'downkim': 1, 'trafficwhat': 1, 'professionalmarvel': 1, 'thor': 1, 'soonpectoral': 1, 'fundamentalistskevin': 1, 'movieextra': 1, 'screenprima': 1, 'intactprince': 1, '29yearold': 1, 'idahoan': 1, 'jennie': 1, 'doestweeters': 1, 'warmonger': 1, 'boltonbernie': 1, 'billionaireher': 1, 'breakhelen': 1, 'mirren': 1, 'tattoofootball': 1, 'tuesdaydaytimetalkshow': 1, '1000pound': 1, 'makeovermiddleaged': 1, 'angelsmillions': 1, 'survivorbanana': 1, 'experiencea': 1, 'empathysean': 1, 'teenvaliant': 1, 'factcheckers': 1, 'corruptionour': 1, 'businessmen': 1, 'moneyarizona': 1, 'englishtrevor': 1, 'probeisrael': 1, 'protestcharlotte': 1, 'edgepastor': 1, 'agnostic': 1, 'weddingpaula': 1, 'songguys': 1, 'beerwhat': 1, 'seasonroyal': 1, 'mealmuchcriticized': 1, 'softball': 1, 'tacticschinese': 1, 'dealnra': 1, 'stupidif': 1, 'myselffederal': 1, 'coupletrump': 1, 'claimssouthwest': 1, 'windowboehner': 1, 'heap': 2, 'breaksbrewers': 1, 'basestrumps': 1, 'conflicting': 1, 'truthpeople': 1, 'redeem': 1, 'itselfdrone': 1, 'destroyed': 3, 'happened20something': 1, 'royals': 3, 'isissteve': 1, 'vai': 1, 'kidsyou': 1, 'stuckhush': 1, 'madoff': 1, 'cellmate': 1, 'throatthis': 1, 'adores': 1, 'pratt': 1, 'totes': 1, 'himdubious': 1, 'inclusions': 1, 'credibility': 1, 'collectiondraft': 1, 'adharvey': 1, 'performative': 1, 'wokeness20': 1, 'planters': 1, 'houseplants': 1, 'heightsblack': 1, 'whackerexprosecutor': 1, 'romancehuman': 1, 'skeletons': 1, 'nycs': 1, 'parkivanka': 1, 'abusersmozambique': 1, 'devises': 2, 'marriageparty': 1, 'intendedhuffpost': 1, '13annoying': 1, 'prowhalinghow': 1, 'hollywooduneventful': 1, 'manal': 1, 'kozlewski': 1, 'kozlewskidennis': 1, 'humiliation': 1, 'enoughelle': 1, 'airbrushes': 1, 'naomi': 1, 'altogetherobama': 1, 'actrosamund': 1, 'pike': 1, 'ravishing': 1, 'redsingle': 1, 'replaceable': 1, 'raiseeric': 1, 'tripmysterious': 1, 'blabs': 1, 'againentire': 1, 'pitches': 1, 'yosemitedraftkings': 1, 'fanduel': 1, 'sportsgerman': 1, 'luftwaffle': 1, 'superioritycoworkers': 1, 'toffeethese': 1, 'hopestreet': 1, 'bucketsflatbread': 1, 'fatherun': 1, 'distributing': 1, 'malnourished': 1, 'customersfat': 1, 'playheres': 1, 'immigrationlady': 1, 'topthe': 1, 'showcases': 1, 'glorifying': 1, 'violencegreat': 1, 'imaginationits': 1, 'biggie': 1, 'conflictthis': 1, 'adopting': 1, 'rulesdoes': 1, 'creativeradio': 1, 'cohostreporter': 1, 'storyrepublicans': 1, 'disposing': 1, 'diseaseridden': 1, 'corpses12': 1, 'thinkingfor': 1, 'novelists': 1, 'monetarywhoa': 1, 'couchpence': 1, 'aborted': 1, 'trimestergentrification': 1, 'upslight': 1, 'inconsistency': 1, 'bibledefiant': 1, 'milosevic': 1, 'sloppy': 1, 'trialsuperstitious': 1, 'thursdaylinmanuel': 1, 'juan': 1, 'mayor13': 1, 'essays': 1, '2016fda': 1, 'pending': 3, 'offmassive': 1, 'alliesrape': 1, 'billutah': 1, 'lawoval': 1, 'trumpdysfunctional': 1, 'otherthis': 1, 'outel': 1, 'evo': 1, 'es': 1, 'pueblois': 1, 'debtor': 1, 'nationbody': 1, 'kayaker': 1, 'missingdancing': 1, 'overheatscurvy': 1, 'punniest': 1, 'waylittle': 1, 'piggy': 1, 'rihannas': 1, 'dayprescription': 1, 'flygop': 1, 'resigndetective': 1, 'endlessly': 1, 'killerone': 1, 'harrington': 1, '86leaked': 1, 'awakensstudy': 1, 'rewarding': 1, 'childrenthis': 1, 'womenpoll': 1, 'boobytrapped': 1, 'bladesjerry': 1, 'troglodyte': 1, 'cavesearth': 1, 'dayreport': 1, 'watering': 1, 'zinnias': 1, 'belowrubio': 1, 'aheadnetflix': 1, 'choiceformer': 1, 'perrys': 1, 'foldedsxsw': 1, 'associatethe': 1, 'arabs': 1, 'citizenshipcalifornia': 1, 'planwoman': 1, 'onceburger': 1, 'angers': 1, 'belgian': 1, 'royalheres': 1, 'videoim': 1, 'mishandling': 2, 'electionquest': 1, 'coastsonion': 1, 'onionman77insane': 1, 'fxs': 1, 'americansarea': 1, 'tender': 1, 'lastelder': 1, 'crisisclinton': 1, 'boon': 1, 'businesssleeping': 1, 'childlike': 1, 'vulnerablehank': 1, 'preparednessthis': 1, 'dogwhen': 1, 'autocorrect': 1, 'collidesleepover': 1, 'roomfiona': 1, 'concertwhy': 1, 'jillian': 1, 'michaels': 1, 'reclaiming': 1, 'fag': 1, 'dyketheater': 1, 'audiencea': 1, 'fighter': 2, 'mayweathermcgregor': 1, 'fightdisneys': 1, 'fridayisnt': 1, 'fatherarea': 1, 'possibleirans': 1, 'youthpope': 1, 'citynasa': 1, 'diving': 1, 'waterdeath': 1, 'shoppers': 1, 'alshabaab': 1, 'fracturing': 1, 'jihadismtime': 1, 'cocktailnationcraze': 1, 'storywoman': 1, 'joggingscientists': 1, 'emailsan': 1, 'labeloklahoma': 1, 'biathlon': 1, 'eventcalle': 1, 'kissgerman': 1, 'predictablytrumps': 1, 'boiled': 1, 'tweetrise': 1, 'trendwhy': 1, 'metaiwan': 1, 'birthdaystaff': 1, 'offtrumps': 1, 'abusebanksy': 1, 'trademark': 1, 'rats10': 1, 'vacationmysterious': 1, 'fireball': 1, 'saykansas': 1, 'serieslethal': 1, 'prisontop': 1, '2015whiteness': 1, 'collegeiconic': 1, 'sexualityman': 1, 'newshow': 2, 'workplacewith': 1, 'magicis': 1, 'hoarding': 1, 'lawful': 1, 'surveillanceman': 1, 'sweatshirt': 1, 'midriffaltruism': 1, 'mockedbackground': 1, 'fridayterrified': 1, 'hunterslast': 1, 'tamagotchi': 1, 'captivityseth': 1, 'photochita': 1, 'dancelocal': 1, 'welder': 1, 'welders': 1, 'block10': 1, 'mallinformal': 1, 'candidatesyou': 1, 'cookieshorrible': 1, 'boogie': 1, 'neckbusy': 1, 'earlybernice': 1, 'humanitymiley': 1, 'gopadvice': 1, 'himwe': 1, 'dotom': 1, 'gump': 1, 'accenter': 1, 'todaykellyanne': 1, 'kellysupreme': 1, 'tradingreeva': 1, 'steenkamp': 1, 'failurewere': 1, 'likewarranty': 1, 'outlasts': 1, 'companyariana': 1, 'demi': 1, 'festeven': 1, 'birdsspinal': 1, '11study': 1, 'analyze': 1, 'americahow': 1, 'palinpreviewing': 1, 'octavia': 1, 'janelle': 1, 'monelibrary': 1, 'forever2018': 1, 'startexxonmobil': 1, 'yearyellow': 1, 'liters': 1, 'donationsmargaret': 1, 'aliensjames': 1, '297': 1, 'trumpcatching': 1, 'depressingly': 1, 'manageablemore': 1, 'frontrunner': 1, 'hamiltonarea': 1, 'moneybob': 1, 'iger': 1, 'rupert': 1, 'foxrob': 1, 'falsifying': 2, 'resumeroommate': 1, 'dayssafetyconscious': 1, 'doorwhen': 1, 'interviewers': 1, 'mutters': 1, 'microphoneeric': 1, 'greencolleges': 1, 'establishing': 1, 'cubascott': 1, 'baio': 1, 'relentlessly': 1, 'clintonbe': 1, 'selfserving': 1, 'millenial': 1, 'activismreport': 1, 'strawberries': 1, 'hypejoin': 1, 'debatespelling': 1, 'doodlewhat': 1, 'eraman': 1, 'wifeariana': 1, 'deets': 1, 'showserena': 1, 'dyingunpopped': 1, 'kernels': 1, 'billionslooking': 1, 'stopyou': 1, 'playersconan': 1, 'audiobook': 1, 'soundsnasim': 1, 'nasrs': 1, 'zaeefeh': 1, 'wretchedness': 1, 'shadi': 1, 'gagprojects': 1, 'adelaideberlin': 1, 'dubailululemon': 1, 'intactman': 1, 'guysbritain': 1, '1580s7': 1, 'bonus': 1, 'exemption': 1, 'marijuanawhy': 1, 'closurethis': 1, 'overinflated': 1, 'bradyailey': 1, 'itanother': 1, 'dubsmash': 1, 'youif': 1, 'seapopular': 1, 'ran': 2, 'whatnewly': 1, 'recordings': 1, 'groupexplosive': 1, 'casesold': 1, 'paso': 1, 'kitwhat': 1, 'tour15': 1, 'destinations': 2, 'summerdoomsday': 1, '3cheese': 1, 'jalapeo': 1, 'meltgoose': 1, 'flapping': 1, 'wingstearful': 1, 'blacklight': 1, 'barbarian': 1, 'faux': 1, 'paswednesdays': 1, 'goparea': 1, 'meatpolice': 1, 'personallysenior': 1, 'restocks': 1, 'rum': 1, 'creammayonnaise': 1, 'sandwichhome': 1, 'securitywatch': 1, 'rhetoricsoybean': 1, 'canceled11': 1, 'doughnut': 1, 'seenkentuckys': 1, 'bromance': 1, 'loverudy': 1, 'grinning': 1, 'ceremonymother': 1, 'soaps': 1, 'shitelizabeth': 1, 'primarydwarf': 1, 'demeaningcrunch': 1, 'munch': 1, 'crunchiness': 1, 'munchabilitywhite': 1, 'privacyman': 1, 'noddingget': 1, 'pokmon': 1, 'smartphoneaarp': 1, 'comfier': 1, 'dennyschristian': 1, 'pharmacist': 1, 'prescriptionvideo': 1, 'pepperspraying': 1, 'breathecarly': 1, 'whoever': 1, 'aregod': 1, 'tattooed': 1, 'backdiy': 1, 'bagpolitical': 1, 'headcalifornia': 1, 'homelessnessstephen': 1, 'ai': 2, 'lakeis': 1, 'gendernonconforming': 1, 'carolinatourists': 1, 'permutations': 1, 'exhaustedthis': 1, 'recyclepresident': 1, 'presidencythe': 1, 'bigotrylets': 1, 'norman': 1, 'paintingis': 1, 'femininestudy': 1, 'jeanstrump': 1, 'tortureteacher': 1, 'rearrestedwhy': 1, 'couldnt8': 1, 'kidsguardians': 1, 'officedad': 1, 'deliverbang': 1, 'crosshairsmom': 1, 'luncha': 1, 'teachhow': 1, 'departmentcall': 1, 'foundobama': 1, 'approvalelect': 1, 'currys': 1, '3pointersweve': 1, 'excused': 1, 'anymorelocal': 1, 'homestunned': 1, 'crashrihanna': 1, 'teebartender': 1, 'statustoni': 1, 'braxton': 1, 'yearcocky': 1, 'operate': 2, 'backfiresexecutive': 1, 'decisionssunday': 1, 'roundupshocking': 1, 'piracywhy': 1, 'runners': 1, 'themselvesoverweight': 1, 'encased': 1, 'geo': 1, 'metromaintaining': 1, 'actionterrifying': 1, 'uniformed': 1, 'barthis': 1, 'twittersubpoenaed': 1, 'jerseyas': 1, 'internetthe': 1, 'intrinsic': 1, 'educationwhy': 1, 'specialstudy': 1, 'buffetoveruse': 1, 'enzymebased': 1, 'cleaners': 1, 'resistant': 1, 'superstainswhooshsnapsbiz': 1, 'informationgunman': 1, 'mallfirefighters': 1, 'wildfires': 3, 'smoky': 1, 'washingtondavid': 1, 'depose': 1, 'home10': 1, 'adjunct': 1, 'teachertime': 1, 'mp': 1, 'taro': 1, 'konogiant': 1, 'murals': 1, 'tides': 1, 'forevermeet': 1, 'darth': 1, 'vader': 1, 'memorabiliautah': 1, 'physician': 1, 'wontusa': 1, 'originalliterally': 1, 'discountsheres': 1, 'privilegeromney': 1, 'sleeves': 1, 'headplanned': 1, 'videotop': 1, 'seasonall': 1, 'hermionesick': 1, 'euthanized': 1, 'timeteam': 1, 'gasp': 1, 'explainedgina': 1, 'mixupa': 1, 'galen': 1, 'homebags': 1, 'problemstown': 1, 'streetoffice': 1, 'chummy': 1, 'otherheres': 1, 'italyarea': 1, 'stuffhow': 1, 'smartphones': 1, 'dermatologiststeen': 1, 'yearbookrick': 1, 'steves': 1, 'cleaned': 1, 'gypsiesif': 1, 'gail': 1, 'saltz': 1, 'videojon': 1, 'russiaeric': 1, 'mustnt': 1, 'inquire': 1, 'tellnra': 1, 'loaded': 3, 'daythese': 1, 'turds': 1, 'rr': 1, 'fansdietary': 1, 'supplements': 1, 'yearpoll': 1, 'confirming': 1, 'gorsuchkofi': 1, 'annan': 1, '4000pound': 1, 'wreath': 1, 'gravethe': 1, 'problemsfirefighters': 1, 'hamsters': 2, 'oxygen': 2, 'maskstupacs': 1, '35000these': 1, 'timelapses': 1, 'jaw': 2, 'dropmedicare': 1, 'sayvolkswagens': 1, 'effortsus': 1, 'chen': 1, 'redeems': 1, 'recordsetting': 1, 'skatehow': 1, 'dreamssenator': 1, 'lostbizarre': 1, 'natives': 1, 'inhabited': 1, 'europeansdog': 1, 'costumed': 1, 'sportsteam': 1, 'preferenceheres': 1, 'bonkersjohnsville': 1, 'il': 1, '11717megan': 1, 'carpetsasha': 1, 'debatepolice': 1, 'fancyclothes': 1, 'copcan': 1, 'assemblymansurfer': 1, 'discontent': 1, 'officehow': 1, 'misled': 1, 'goexxon': 1, 'lifesurgeon': 1, 'froot': 1, 'nicknames': 1, 'signsthe': 1, 'tweeters': 1, 'gleetrump': 1, 'fairnumber': 1, 'congregations': 1, 'electiontexas': 1, 'move10': 1, 'olderromney': 1, 'requiring': 2, 'mates': 1, 'moneybushs': 1, 'eyelid': 1, 'wallsudanese': 1, 'whatsoeverimpersonal': 1, 'weightsstate': 1, 'himconservation': 1, 'wastefultourist': 1, 'himpeoples': 1, 'racialized': 1, 'imagegym': 1, 'pantywaisttrash': 1, 'readingantidefamation': 1, 'antibullying': 1, 'programslove': 1, 'beyondnations': 1, 'backa': 1, 'truelife': 1, 'devotionstd': 1, 'vr': 1, 'immersive': 1, 'krugmanhigh': 1, 'systemaddressing': 1, 'distractiontony': 1, 'moredonald': 1, 'warmelania': 1, 'grimacekuwait': 1, 'troopjames': 1, 'fenimore': 1, 'famed': 1, '224syria': 1, 'eastleonardo': 1, 'posse': 1, 'specieswhy': 1, 'airbnbwayne': 1, 'cpac': 1, 'speechto': 1, 'iraqchinese': 1, 'monkeys': 1, 'applicationsnbc': 1, 'schedulessome': 1, 'ordinanceafghan': 1, 'ashraf': 1, 'ghani': 1, 'grouphard': 1, 'choicestruck': 1, 'enormous': 1, 'roadthe': 2, 'iphonefeatures4politiciansmit': 1, 'thinktank': 1, 'ideaslester': 1, 'dinnercan': 1, 'syriansobama': 1, 'actdeputies': 1, 'suspectnations': 1, 'uscouple': 1, 'divorceexercise': 1, 'studywarden': 1, 'ought': 1, 'schizophreniccolorado': 1, 'moonmcdonalds': 1, '6piece': 1, 'ncnoltes7yearold': 1, 'deedee': 1, 'isrick': 1, 'absent': 1, 'shootersron': 1, 'markersdepartment': 1, 'padded': 1, 'bumper': 1, 'intoxicated': 1, 'driversmysterious': 1, 'coupon': 2, 'establishments': 1, 'mailboxwoman': 1, 'acupuncturewhy': 1, 'chose': 1, 'onethe': 1, 'murderer100yearold': 1, '100meter': 1, 'recordtide': 1, 'rumsfelds': 1, 'barracks': 1, 'talkjeb': 1, 'fatherman': 1, 'captivating': 1, 'lifeincreased': 1, '1861hillary': 1, 'tvthese': 1, 'adorablescalia': 1, 'recuses': 1, '80shigh': 1, 'memorizationphoto': 1, 'oneclimate': 1, 'haunts': 1, 'croptan': 1, 'timeskilled': 1, 'sothebys': 1, 'auctioneer': 2, 'auction': 1, 'millionbush': 1, 'monthgrandmother': 1, 'grandbabyisrael': 1, 'hagel': 1, 'defensemeet': 1, 'charth': 1, 'vaderthe': 1, 'heartsport': 1, 'arete': 1, '2016fox': 1, 'wellcelebrating': 1, 'addictionus': 1, 'nonoilrich': 1, 'criticism3': 1, 'mudslidenew': 1, 'petticoats': 1, 'fieldantihomosexuality': 1, 'sermon': 1, 'wellinformedyou': 1, 'sandwichesmore': 1, 'providing': 3, 'materials': 1, 'recyclablethe': 1, 'hopecoworkers': 1, 'dinnershas': 1, 'crisisdid': 1, 'spawn': 1, 'sayaides': 1, 'airplanemoms': 1, 'rotates': 1, 'demonically': 1, 'antique': 1, 'wicker': 1, 'furnitureold': 1, 'attractivefriend': 1, 'professionalpete': 1, 'townshend': 1, 'explainblack': 1, 'numbersobama': 1, 'groupamericans': 1, 'endgame': 1, 'constituentswhat': 1, 'perspectivenew': 1, 'toothbrushesnavy': 1, 'frogmen': 1, 'headlawn': 1, 'mower': 1, 'worklonesome': 1, 'seahow': 1, 'stepstaco': 1, 'bagscensus': 1, '9000': 1, 'bureaujimmy': 1, 'fallons': 1, 'myroommateisweird': 1, 'sum': 1, 'roomie': 1, 'nightmaresnew': 1, 'complainingdemi': 1, 'break5': 1, 'againwhy': 1, 'presidencywaxmuseum': 1, 'devito': 1, 'statuesrachel': 1, 'reportingvirginia': 1, 'catch10': 1, 'differentlynba': 1, 'insurancecan': 1, 'workingcaitlyn': 1, 'transitioninternet': 1, 'cory': 1, 'monteiths': 1, 'worthchris': 1, 'pratts': 1, 'trolling': 1, 'himget': 1, 'attentionnew': 1, 'superfast': 1, 'transport': 1, 'powered': 1, 'screamsdeath': 1, 'attemptsthe': 1, 'backnobody': 1, 'adseverything': 1, 'kelleythis': 1, 'generationprotesters': 1, 'policersum': 1, 'filevirginia': 1, 'dutyjudge': 1, 'reopened': 1, 'case1998': 1, 'foodservice': 1, 'jobnew': 1, 'triplex': 1, 'dinosaur': 2, 'nevadathis': 1, 'xfl': 1, 'televisions': 1, 'everparole': 1, 'psychologychurch': 1, 'tryoutlakers': 1, 'midgamefcc': 1, 'profanity7yearold': 1, 'himparent': 1, 'wideeyed': 1, '7yearoldthe': 1, 'countrywhat': 1, 'himlimbo': 1, 'slager': 1, 'backsurviving': 1, 'storyindias': 1, 'handilyhow': 1, 'mode': 1, 'issuesice': 1, 'joplin': 1, 'hellreport': 1, 'checkold': 1, 'man6': 1, 'homejury': 1, 'poisonstupid': 1, 'workseaworld': 1, 'ridethis': 1, 'abroadfederal': 1, 'zonehbo': 1, 'weekend10': 1, 'phrases': 2, 'todaywonder': 1, 'allreport': 1, '2002entire': 1, 'settingsreinventing': 1, 'winwinwin': 1, 'relationshipsconscious': 1, 'helmfire': 1, 'creatives10': 1, 'relationshipswatch': 1, 'endsarea': 1, 'driveman': 1, 'wordnewlydiscovered': 1, 'smallpox': 1, 'vials': 1, 'thoughtnorth': 1, 'dream5millioncar': 1, 'pileup': 1, 'dallasfort': 1, 'worthhow': 1, 'feedbush': 1, 'blindfaithbased': 1, 'initiativesthree': 1, 'cache': 1, 'holland': 1, 'tunnelthat': 1, 'showeven': 1, 'leaveaaron': 1, 'brotherhow': 1, 'oneal': 1, 'drillingin': 1, 'interestsalaska': 1, 'securityus': 1, 'byproducts': 1, 'imaginedwoman': 1, 'manhair': 1, 'nagel': 1, 'printsouth': 1, 'rubiochemicals': 1, 'flare': 1, 'bagyour': 1, 'schoolferguson': 1, 'copsamerica': 1, 'negotiationswords': 1, 'hearinguppermiddleclass': 1, 'about25yearold': 1, 'rentfree': 1, 'arrangement': 1, 'bottomguns': 1, 'georgiawhite': 1, 'sessionsweird': 1, 'dayfirst': 1, 'starhenry': 1, 'rollins': 1, 'laboriously': 1, 'rockformer': 1, 'companypipeline': 1, 'coldios': 1, 'iphonesdisney': 1, 'castgabrielle': 1, 'floodgates': 1, 'women10': 1, 'billpostretirement': 1, 'yearson': 1, 'afghanistandubai': 1, 'marsa': 1, 'resortfacebook': 1, 'teamour': 1, 'environmenthotshot': 1, 'consumermccain': 1, 'silences': 1, 'cartwheelobama': 1, 'affordableadvice': 1, 'brokenheartedsecretary': 1, 'inappropriately': 1, 'politicsyoulookdisgusting': 1, 'perfection': 1, 'realis': 1, 'osha': 1, 'administrationblack': 1, 'livesgroup': 1, 'fifthgrade': 1, 'woodsthe': 1, 'marchadam': 1, 'ishmo': 1, 'campaignrussia': 1, 'lastclosing': 1, 'streetscoalition': 1, 'buzzed': 1, 'rankingsnick': 1, 'tilsen': 1, 'reservationthe': 1, 'rulingwoman': 1, 'scheme': 1, 'workdaypoll': 1, 'nspw2017new': 1, 'browniesfanfavorite': 1, 'dvdconsumers': 1, '50ryan': 1, 'coogler': 1, 'wakanda': 1, 'spinoffjustin': 1, 'charlottesvilles': 1, 'strengthwhats': 1, 'uberreport': 1, 'somethingdemocrats': 1, 'surrender': 1, 'completejunior': 1, 'househillary': 1, '2008romney': 1, '612th': 1, 'monthgold': 1, 'bracelet': 1, 'pharmacyformer': 1, 'damore': 1, 'altright11': 1, 'kitnation': 1, 'abuzz': 1, 'cspan': 1, 'moviemccain': 1, 'burmashavestyle': 1, 'campaignlocal': 1, 'handicappedcocaine': 1, 'knowsaircraft': 1, 'highellen': 1, 'possibleglutenfree': 1, 'mania': 1, 'confusionwhat': 1, 'hulu': 1, 'beingwalking': 1, 'youtubepissed': 1, 'slangrykey': 1, 'filibustersarizona': 1, 'corrections': 1, 'backlashfrustrated': 1, 'refrain': 1, 'chants': 1, 'speakingcouple': 1, 'ceremonynations': 1, 'bitchscott': 1, 'walkerbacked': 1, 'defeated': 2, 'racenetflix': 1, 'fawlty': 1, 'towerslong': 1, 'armfinding': 1, 'wiselymike': 1, 'deportationsbill': 1, 'mefacebook': 1, 'itlawmakers': 1, 'controversyreport': 1, 'ounces': 1, 'nowmays': 1, 'dutymooney': 1, 'caseysecretary': 1, 'banterdonald': 1, 'jokeshome': 1, 'homosexuality': 1, 'availableobama': 1, 'snatching': 1, 'washingtonreport': 1, 'consumedamelia': 1, 'boynton': 1, 'robinson': 1, '104no': 1, 'smokerhot': 1, 'effing': 1, 'beacon': 1, 'stormjane': 1, 'seymours': 1, '50dancing': 1, 'stormtroopers': 1, 'cowells': 1, 'realitynabisco': 1, 'roadsreport': 1, 'introverted': 1, 'us7': 1, 'argumentsan': 1, 'attorneys': 1, 'themdeaf': 1, 'homelesscleansed': 1, 'crisp': 1, 'crippled': 2, 'dapperly': 1, 'disabledcate': 1, 'blanchett': 1, 'glamour': 1, 'globesi': 1, 'contemporary': 1, 'journalismmonster': 1, 'groverreport': 1, 'statewritein': 1, 'winauthorities': 1, 'plates': 1, 'roomfuneral': 1, 'mistyeyed': 1, 'corpsehow': 1, 'relied': 1, 'mena': 1, 'beforeandafter': 1, 'photosolympics': 1, '6daylong': 1, 'performanceuk': 1, 'electronics': 1, 'africaconfessions': 1, 'hoarderdonald': 1, 'telephonewashington': 1, 'mentexas': 1, 'indictedwall': 1, 'secondmany': 1, 'manhouse': 1, 'hearingcongressional': 1, '1971': 1, 'remarkmusic': 1, 'fests': 1, 'concertsvideo': 1, 'maturity': 1, 'upamericans': 1, 'halfpipecdc': 1, 'diethe': 1, 'injustice': 1, 'minimumshonoring': 1, 'recipient': 1, 'raoul': 1, 'wallenberg': 1, 'differencesecretary': 1, 'prelims': 1, 'roosevelt': 1, 'cynics': 1, 'realistsover': 1, 'chime': 1, 'reputationthe': 1, 'momsarea': 1, 'settlethe': 1, 'tibetan': 1, 'meditationbaseballs': 1, 'sillier': 1, 'thoughthillary': 1, 'endorsementyour': 1, 'choiceits': 1, 'makeoverspoons': 1, 'pint': 1, 'jarring': 1, 'sittingharvards': 1, 'decisionsbush': 1, 'memoirsstyrofoam': 1, 'omaha': 1, 'oceanarea': 1, 'wetdonald': 1, 'inaugurationcelebrity': 1, 'fanfiercest': 1, 'kobaninorth': 1, 'icbm': 1, 'nextyou': 1, 'smarter': 1, 'howreport': 1, 'alongreality': 1, 'inmy': 1, 'frenchievideo': 1, 'meetup': 1, 'barrelsarea': 1, 'participates': 1, 'cashless': 1, 'myselfmedia': 1, 'mixedrace': 1, 'womanoklahoma': 1, 'revolt': 1, 'spreadsarea': 1, 'scarelook': 1, 'likekim': 1, 'yet9': 1, 'selfassuring': 1, 'affirmations': 1, 'boostwar': 1, 'goliathmusic': 1, 'shelterbill': 1, 'latte': 1, 'salutesessions': 1, 'disqualified': 1, 'dominicans': 1, 'himcowboys': 1, 'occupiers': 1, 'thatsyrian': 1, 'conditionsnew': 1, 'studentsnext': 1, 'billafrodisney': 1, 'scrappedfreshman': 1, '870': 1, 'fanstrump': 1, 'slaughter': 1, 'protestersnoel': 1, 'comries': 1, 'selfaffirmationsmotherhood': 1, 'sportspring': 1, 'businesstrophy': 1, 'ageariana': 1, 'davidsonkhrushchevs': 1, 'granddaughter': 1, 'stalin3yearold': 1, 'fajita': 1, 'platterchild': 1, 'eveningsirelands': 1, 'fusion': 1, 'energirobed': 1, 'infiltrates': 2, 'torchlit': 1, 'ahca': 2, 'capitolgoing': 1, 'congo16': 1, 'dinnertime': 1, 'hotterthe': 1, 'gamehilarious': 1, 'halloweendo': 1, 'blissmissing': 1, '2400mile': 1, 'triptea': 1, 'districtlaverne': 1, 'cisgender': 1, 'womenmillions': 1, 'rehearsed': 1, 'speechesafter': 1, 'officenew': 1, 'gutsy': 1, 'gameplan': 1, 'toglobal': 1, 'hijabista': 1, 'burqa': 1, 'magazinegordon': 1, 'ramsay': 1, 'minutes43yearold': 1, 'fooling': 1, 'anyoneolympic': 1, 'cosplays': 1, 'jaime': 1, 'lannister': 1, 'routinedavid': 1, 'anymorehuffpollster': 1, '2016sotomayor': 1, 'ballistics': 1, 'court6': 1, 'treatable': 1, 'dementiamegachurch': 1, 'ultrachurchnothing': 1, 'todaydetroitarea': 1, 'airdesperate': 1, 'abducted': 1, 'unnation': 1, 'oberstmost': 1, 'surveycan': 1, 'fryq': 1, 'manualjoin': 1, 'birthfor': 1, 'publicitytrump': 1, 'abusers': 1, 'winnersthats': 1, 'blackamerican': 1, 'perspectivelocal': 1, 'flieractivism': 1, 'gangthe': 1, 'listcouple': 1, 'anniversarychild': 1, 'goodtwo': 1, 'communitiesread': 1, 'debatemanager': 1, 'slits': 1, 'pagemitch': 1, 'replacementloser': 1, 'bettermotivational': 1, 'minutesaunt': 1, 'toessexual': 1, 'humansclooney': 1, 'scouting': 1, 'darfurbased': 1, 'comedycontrol': 1, 'wrested': 1, 'man17': 1, 'fantastically': 1, 'stemtwo': 1, 'nonbinary': 1, 'campuswednesdays': 1, 'reformolder': 1, '8yearoldrace': 1, 'paralyzing': 1, 'childrendozens': 1, 'multiplex': 1, 'employeearea': 1, 'pastnicole': 1, 'richies': 1, 'pregnancyatonal': 1, 'composers': 1, 'atony': 1, 'awardsmeredith': 1, 'vieiras': 1, 'silencepretty': 1, '501': 1, 'yorkpaul': 1, 'rudd': 1, 'showered': 1, 'beerchelsea': 1, 'americaplease': 1, 'williamscourt': 1, '1025': 1, 'aoldisney': 1, 'confettitrump': 1, 'mooch': 1, 'editionmuseum': 1, 'episodebaby': 1, 'travelercalifornia': 1, 'practices10': 1, 'careerchild': 1, 'parentsusbacked': 1, 'raqqahow': 1, 'homelessgreed': 1, 'sarawaks': 1, 'rainforestairbnb': 1, 'librariesheres': 1, 'dicaprios': 1, 'islandinterpol': 1, 'lisabiden': 1, 'japanhomosexuality': 1, 'addictionthe': 1, 'marketsrob': 1, 'blac': 2, 'chyna12': 1, 'marcel': 1, 'duchamp': 1, 'artistjohn': 1, 'mcenroe': 1, 'williamsnew': 1, 'bisongop': 1, 'abolishing': 1, 'irssnl': 1, 'wayjohn': 1, 'cakewalk': 1, 'iraqpanicked': 1, 'emergencyinaccuracy': 1, 'minutedonald': 1, 'saturdays': 1, 'debatecalifornia': 1, 'enoughwoman': 1, 'soulmate': 1, 'socioeconomic': 1, 'bracketthe': 1, 'underrated': 1, 'irsit': 1, 'thermosensing': 1, 'eyesman': 1, 'goatstexas': 1, 'platformuniversity': 1, 'renames': 1, 'vito': 1, 'corleone': 1, 'benefactordakota': 1, 'lapses': 1, 'violenceactivists': 1, 'cabinettrue': 1, 'runfederal': 1, 'schedulenation': 1, 'descends': 2, 'dear': 1, 'deathfor': 1, 'futureearth': 1, 'longprofessor': 1, 'islamthe': 1, 'parenttuesdays': 1, 'countryphilippine': 1, 'isislinked': 1, 'boyshow': 1, 'pluto': 1, 'heartfirst': 1, 'bornmondays': 1, 'trumpimmigration': 1, 'deathsman': 1, 'countrythis': 1, 'surfaced': 1, 'satisfying15': 1, 'notebooks': 1, 'writetvs': 1, 'mork': 1, 'filmheres': 1, '10piece': 1, 'wardrobe': 2, 'checklistfrom': 1, 'momthe': 1, 'enterprise': 1, 'workscybersecurity': 1, 'curryfactory': 1, 'hoursthe': 1, 'hoaxno': 1, 'happensterry': 1, 'gropedcounselors': 1, 'homesick': 1, 'campershow': 1, 'day8': 1, 'dos': 2, 'donts': 1, 'religionthemed': 1, 'costumesgrown': 1, '37th': 1, 'figurineman': 1, 'coldmost': 1, 'nomineeukraine': 1, 'crisistheir': 1, 'daystudy': 1, 'apex': 1, 'predatorkamikaze': 1, 'pearl': 3, 'harborglobal': 1, 'superpowerted': 1, 'favorability': 1, 'speechbarack': 1, 'giftdid': 1, 'ernest': 1, 'sisterinlawstudy': 1, 'restoring': 1, 'environmentclimate': 1, 'glaciers': 2, 'parkminnesota': 1, 'smithhousehold': 1, 'onemagnanimous': 1, 'occupy': 1, 'protesterstrump': 1, 'protestsouth': 1, 'geunhyenas': 1, 'ghostbustersinspired': 1, 'sartorial': 1, 'heroesfreethinking': 1, 'boxstudy': 1, 'uneducated': 1, 'outbreeding': 1, 'intelligentsia': 1, '2to1michael': 1, 'votingtimmy': 1, 'trumpthese': 1, 'packpolice': 1, 'impersonatorphotojournalist': 1, 'habitat22': 1, 'pajamas': 1, 'becausenations': 1, 'restaurantsformer': 1, 'girlongirl': 1, 'videodepleted': 1, 'brueggers': 1, 'bagels': 1, 'rowcounting': 1, 'trumpcompany': 1, 'familynational': 1, 'preventionperfect': 1, 'womanclimate': 1, 'talkmans': 1, 'pops': 1, 'formedman': 1, 'hasman': 1, 'firstseth': 1, 'spielbergs': 1, 'faceto': 1, 'deathmisery': 1, 'miseryneighborhood': 1, 'levelnick': 1, 'rumorsarea': 1, 'candidatenew': 1, '1980': 1, '1989this': 1, 'grandmahow': 1, 'prioritizing': 1, 'winging': 1, 'boxesobama': 1, 'pollsthis': 1, 'whalescheney': 1, 'clotheslines': 1, 'aideted': 1, 'adhuffpost': 1, 'growsmillions': 1, 'ballnation': 1, 'people29yearold': 1, 'yearsman': 1, 'pulp': 1, 'girlfriendstudy': 1, 'moraledestinys': 1, 'faceman': 1, 'teamskin': 1, 'failstart': 1, 'detoxneed': 1, 'issuethe': 1, 'daydistributor': 1, 'reportstatus': 1, 'pizzaincredibly': 1, 'swims': 1, 'hawaiis': 1, 'lava': 1, 'stickjordan': 1, 'calmhappy': 1, 'obamagrandma': 1, 'staffkate': 1, 'momswhy': 1, 'communitiestrendy': 1, 'communal': 1, 'napkindonald': 1, 'steakfueled': 1, 'madnesstokyo': 1, 'squeezes': 1, 'residentsreport': 1, 'harassmenturban': 1, 'electionreport': 1, 'stopping': 1, 'nowmoms': 1, 'parentteen': 1, 'firejeb': 1, 'reductionmiracle': 1, 'septupletswife': 1, 'husbandnursinghome': 1, '525': 1, 'houramy': 1, 'krouse': 1, 'rosenthals': 1, 'project10': 1, 'authentic': 1, 'onlineresearchers': 1, 'scrap': 1, 'killertaco': 1, 'raciststhe': 1, 'doduncle': 1, 'giftarea': 1, 'on7': 1, 'kidsholiday': 1, 'aficionado': 1, 'snowmannation': 1, '29yearoldshorrifying': 1, 'mutants': 1, 'abovegroundben': 1, 'timebernie': 1, 'humble': 1, 'homemade': 1, 'lecternmom': 1, 'gift5': 1, 'bookworm': 1, 'takeradical': 1, 'preacher': 1, 'anjem': 1, 'choudary': 1, 'isisnick': 1, 'culpo': 1, 'spliteducating': 1, 'collegiate': 1, 'madness6': 1, 'aboutits': 1, 'sickflesheating': 1, 'footletter': 1, 'accusationslibrary': 1, 'insignificantlost': 1, 'immediatelyscandals': 1, 'morton': 1, 'premierealternative': 1, 'stragglersa': 1, 'nomineesindonesian': 1, 'sews': 1, 'childrennew': 1, 'globehow': 1, 'sexiestyoko': 1, 'flulike': 1, 'symptomssecret': 1, 'deng': 1, 'xiaopinggop': 1, 'lackluster': 1, 'stache': 1, 'setbackyou': 1, 'subsidiary': 1, 'kraft': 1, 'foodsstate': 1, 'preceded': 1, 'yearpeople': 1, 'humanly': 1, 'possiblecharlize': 1, 'rail': 1, 'fridaybarbecue': 1, 'panini': 1, 'howardrelated': 1, 'causes100': 1, 'forwild': 1, 'seriesmom': 1, 'panseared': 1, 'scallops': 1, 'lightat': 1, 'cop20': 1, 'damageganymede': 1, 'totalled': 1, 'threemoon': 1, 'pileupsenates': 1, 'familiesexcited': 1, 'firefighters': 1, 'tricyclegatedcommunity': 1, 'dodunkin': 1, 'almond': 1, 'timereport': 1, 'billiards': 1, 'game32yearold': 1, 'lactoseelder': 1, 'changespecifics': 1, 'boringhigh': 1, 'dinnerdemocrats': 1, 'rulestropical': 1, 'caribbeannew': 1, 'toastables': 1, 'microwavable': 1, 'pretoasted': 1, 'breadprogressive': 1, 'windsarah': 1, 'outrageheres': 1, 'officewhos': 1, 'surpriseradio': 1, 'todayelderly': 1, 'tonightmy': 1, 'parkinsonsnet': 1, 'leftist': 1, 'causehow': 1, 'diamondscolbert': 1, 'punishmentmotion': 1, 'predatorthe': 1, 'unravelling': 1, 'historydirector': 1, 'moviekelly': 1, 'rowlands': 1, 'momstyphoon': 1, 'meranti': 1, 'mayhemlost': 1, 'tooting': 1, 'horncops': 1, 'educators': 1, 'arming': 1, 'ideadont': 1, 'egyptians': 1, 'townthe': 1, 'healthamber': 1, 'snapsdeep': 1, 'sadness': 1, 'overlookedfrench': 1, 'carthe': 1, 'organizationcolleges': 1, 'sanctuaries': 1, 'undocumentedthousands': 1, 'dismembered': 1, 'catacombs': 1, 'motor': 1, 'plantreallife': 1, 'nancy': 1, 'hpvjudge': 1, 'probethese': 1, 'ionceoverheard': 1, 'geniusjake': 1, 'nastier': 1, 'putincreating': 1, 'presentlea': 1, 'romances': 1, 'videoafter': 1, 'abbaskensington': 1, 'louisadrift': 1, 'nationsromney': 1, 'ravemom': 1, 'nowbatumi': 1, 'activities': 1, 'notthis': 1, 'realtimeout': 1, 'burnoutfasting': 1, 'pillars': 1, 'exceptions': 1, 'rulebill': 1, 'denying': 1, 'electionrap': 1, 'reportthis': 1, 'gopbush': 1, 'stockloser': 1, 'smilemtv': 1, 'videoidiotic': 1, 'sidewalkgame': 1, 'triumphpigeon': 1, 'nonchalant': 1, 'sidewalksenate': 1, 'wrays': 1, 'directoractual': 1, 'proctor': 1, 'partymiracle': 1, 'daysdonald': 1, 'picka': 1, 'budgetjust': 1, 'luckthe': 1, 'introversion': 1, 'comicstiny': 1, 'tugs': 1, 'pant': 1, 'legeverything': 1, 'bikejay': 1, 'kissernatalie': 1, 'bedbernie': 1, 'lifethis': 1, 'diedbill': 1, 'admission': 1, 'saymitch': 1, 'yearslong': 1, 'lauerarea': 1, 'eccentricitiesusa': 1, 'crossword': 1, 'intelligencein': 1, 'consent': 1, 'broadway10': 1, 'literatureclemson': 1, 'lsu': 1, 'rankingsi': 1, 'firedmass': 1, 'dieoff': 1, 'deepwater': 1, 'spillhow': 1, 'lifehow': 1, 'spreereport': 1, 'instances': 1, 'wingsin': 1, 'aidheres': 1, 'unionkylie': 1, 'pinup': 1, 'shootnewsweeks': 1, 'firedarchaeologists': 1, '17thcentury': 1, 'congressmanal': 1, 'aprilbobby': 1, 'jindals': 1, 'benefited': 1, 'administrationford': 1, 'welding': 1, 'workworthless': 1, 'crimesmagazine': 1, 'cover10': 1, 'premieresimple': 1, 'giftsfor': 1, 'daze': 1, 'themmatchbox': 1, 'australiawhat': 1, 'memoryplot': 1, 'length': 1, 'therapistman': 1, 'memorialwe': 1, 'peoplewhat': 1, 'wonparents': 1, 'wallpopekilling': 1, 'victimstudy': 1, 'bareknuckle': 1, 'boxingromney': 1, 'stanceweary': 1, 'divisive': 1, 'offthe': 1, 'fictional': 1, 'televisioncharlottesville': 1, 'goddammagazine': 1, 'ckmentally': 1, 'needednetanyahu': 1, 'utmost': 1, 'moneyhotel': 1, 'minibarnations': 1, 'brownkindergarten': 1, 'coverageback': 1, 'stresshow': 1, 'handicapblake': 1, 'marchvillagers': 1, 'overnightdavid': 1, 'ameringer': 1, 'mcenery': 1, 'yohemiranda': 1, 'sheltona': 1, 'christlatinos': 1, 'novemberimpoverished': 1, 'monte': 1, 'carlo': 1, 'racecarpatricia': 1, 'elliott': 1, 'tonywinning': 1, '77the': 1, 'weekstrange': 1, 'solvedtwo': 1, 'cholesterolits': 1, 'goalsmore': 1, 'schemesmike': 1, 'robes': 1, 'addressdisabilities': 1, 'lifechanger': 1, 'forwardobama': 1, 'innuendothe': 1, 'nextgovernment': 1, 'cam5minute': 1, 'realstyle': 1, 'substancegoogle': 1, 'cryseaworld': 1, 'chum': 1, 'increasereport': 1, 'worldflorida': 1, 'dope': 1, 'haul': 1, 'seized': 1, 'packets': 1, 'imagecia': 1, 'inert': 1, 'exercisejoe': 1, 'officethe': 1, 'agerelated': 1, 'cognitive': 1, 'declinedead': 1, '220': 2, 'liability': 1, 'settlementheres': 1, 'nixon': 2, 'generationbush': 1, 'oncethere': 1, 'thinkassads': 1, 'revolutionin': 1, 'territory': 2, 'halluninsured': 1, 'christmasuniversities': 1, 'lifeaging': 1, 'succubus': 1, '40000climbers': 1, 'avalanchestick': 1, 'recordtop': 1, 'diesmagazine': 1, 'mindy': 1, 'kaling': 1, 'womanhundreds': 1, 'usus': 1, 'dumpcoachs': 1, 'ballers': 1, 'dudescouple': 1, 'nods': 1, 'mechanicdelisting': 1, 'bearor': 1, 'notsenators': 1, 'hunttented': 1, 'richerwhat': 1, 'videobernie': 1, 'narrowing': 1, 'statereport': 1, 'businessesman': 1, 'goneanother': 1, 'healthychelsea': 1, 'youcompanies': 1, 'cottonthis': 1, '1913': 1, 'suffragist': 1, 'lifekarl': 1, 'creamed': 1, 'clinton5': 1, 'lifenervous': 1, 'bingeeats': 1, 'removalfind': 1, 'neighborhoodssick': 1, 'enthroned': 1, 'tissuesfemale': 1, 'recommendationboy': 1, 'bossget': 1, 'citiesthousands': 1, 'reliefgreek': 1, 'bailout': 1, 'saysclintons': 1, 'albanian': 1, 'crisisperson': 1, 'outtwo': 1, 'wield': 1, 'lightsabers': 1, 'sidereport': 1, 'highstakes': 1, 'tournamentcomposer': 1, 'licht': 1, 'dexterdnc': 1, 'particularobamacare': 1, 'upstatensndcdwf': 1, 'someonesarah': 1, 'unsarah': 1, 'cpacchinese': 1, 'wargenie': 1, 'constructionist': 1, 'wishflashback': 1, 'uniformslady': 1, 'kinney': 1, 'hotindoor': 1, 'autumnchaplain': 1, 'transgenderism': 1, 'obeying': 1, 'satanerika': 1, 'christensen': 1, 'maness': 1, 'marriedgleeful': 1, 'snlman': 1, 'browsera': 1, 'castentire': 1, 'consists': 1, 'handcarpope': 1, 'snowapple': 1, 'wearable': 1, 'techback': 1, 'dock': 1, 'rowboat': 1, 'passengersa': 1, 'daypeoples': 1, 'volumesnephew': 1, 'yearmelania': 1, 'bio': 1, 'studiesfda': 1, 'shampoolast': 1, 'tuesdays': 1, 'boostnews': 1, '2017meet': 1, 'soberwildlife': 1, 'gulls': 1, 'againwednesdays': 1, 'cusp': 1, 'giveaway3': 1, 'yearfirstgeneration': 1, 'father16': 1, 'stuffhuffpost': 1, 'bribes': 1, 'winyears': 1, 'japans': 1, 'piecesugandas': 1, 'electionparents': 1, 'treatingyoulikeanadult': 1, 'halfhour': 1, 'visitzayn': 1, 'fansthe': 1, 'needsthe': 1, 'itamericans': 1, 'farts': 1, 'everrachel': 1, 'leyco': 1, 'filmdonald': 1, 'outpope': 1, 'messageone': 1, 'harm7': 1, 'sickman': 1, 'schoolrod': 1, 'desertpanicked': 1, 'halfnaked': 1, 'eugenics3': 1, 'ivfvictorias': 1, 'secrethow': 1, 'ditched': 2, 'pizzajust': 1, 'illegalize': 1, 'begprinces': 1, 'tyka': 1, 'amasthe': 1, 'juicero': 1, 'juicer': 1, 'foreveraarp': 1, 'sweepstakes': 1, 'scams11': 1, 'wabi': 1, 'sabi': 1, 'decor': 1, 'imperfectionhow': 1, 'bodyass': 1, 'fame5': 1, 'lists': 1, 'yearbrittle': 1, 'jewess': 1, 'wearingjennifer': 1, 'dreamgirl3': 1, 'empathythe': 1, 'risking': 1, 'balls': 1, 'continuesmidwestern': 1, 'objectsthousands': 1, 'brazilians': 1, 'impeachmentlisten': 1, 'discoveries': 1, '2014glitter': 1, 'birkenstocks': 1, 'buyfirst': 1, 'routineclimatologists': 1, 'captivitylaffy': 1, 'taffy': 1, 'disdains': 1, 'bazookagraduation': 1, 'lucrative': 1, 'careerthe': 1, 'romneythe': 1, 'beginreview': 1, 'potential10': 1, 'equalitykate': 1, 'slickest': 1, 'carpettaylor': 1, 'updatetrumps': 1, 'goon': 1, 'squadsirs': 1, 'tiphail': 1, 'sacked': 1, 'christmasmadrids': 1, '72yearold': 1, 'politics10': 1, 'coworkerthreeyearold': 1, 'awaymasterpiece': 1, 'cakeshop': 1, 'mistrial': 1, 'tampers': 1, 'evidencehappy': 1, 'studentsthe': 1, 'weekus': 1, 'louisgirl': 1, 'chemo': 1, 'herselfscotland': 1, 'wales': 1, 'abortionsbush': 1, 'suitcompany': 1, 'firstperson': 1, 'chatroulettehousekeeper': 1, 'sassybest': 1, 'aditya': 1, 'vikram': 1, 'senguptas': 1, 'lovethe': 1, 'vacationgovernment': 1, 'bails': 1, 'dow': 1, 'pointsstudy': 1, 'donaldson': 1, '2027judge': 1, 'enforcing': 1, 'guidance': 2, 'nationwidefred': 1, 'politicianthe': 1, 'himavoid': 1, 'mistakesat': 1, 'ukraineweird': 1, 'linesit': 1, 'rallypowerful': 1, 'hers': 1, 'legislationwhat': 1, 'degreehbos': 1, 'liveone': 1, 'linkedins': 1, 'ceoit': 1, 'corporationted': 1, 'applauds': 1, 'weddingin': 1, 'madagascarcitizens': 1, 'billthe': 1, 'americaantimdma': 1, 'othersbush': 1, 'abbas': 1, 'speaksstop': 1, 'scaring': 1, 'dadsboston': 1, 'fellowship': 1, 'investigationseclipse': 1, 'blackoutfacebook': 1, 'harvested': 1, 'analyticaprogressive': 1, 'carernc': 1, 'shotthe': 1, 'relationshipstoy': 1, 'wheelsgirl': 1, 'clintonannual': 1, '6sentence': 1, 'smoothlyopening': 1, 'twomoresongs': 1, 'warningnew': 1, 'eager': 1, 'slack': 1, 'coworkersjoe': 1, '46former': 1, 'partnerbarksdale': 1, 'prisonhow': 1, 'schoolcongress': 1, 'herepoland': 1, 'decomposepolly': 1, 'zehnderswaders': 1, 'unwinding': 1, 'daycouple': 1, 'expensesglutenfree': 1, 'sandbible': 1, 'aptitude': 1, 'testhero': 1, 'herograndma': 1, 'unmoved': 1, 'grandchildrendrakes': 1, 'smoothwoman': 1, 'weaning': 1, 'facebookamount': 1, 'handtohand': 1, '2065surviving': 1, 'miner': 1, 'workjohn': 1, 'obeyresearchers': 1, '1994arresting': 1, 'poorest': 1, 'citylas': 1, 'steer': 1, 'newsroom': 1, 'month5': 1, 'gainsuper': 1, 'christpitbulls': 1, 'tasteless': 1, 'cool;': 1, 'lamethe': 1, 'nonsensegoingoutofbusiness': 1, 'whatsoeverweak': 1, 'ineffectual': 1, 'filetaylor': 1, 'cruise10': 1, 'smallspace': 1, 'lifesaversgoing': 1, 'tempurpedic': 1, 'workfully': 1, 'gentrified': 1, 'shopsaol': 1, 'messenger': 1, 'yearsrussian': 1, 'saberrattling': 1, 'counterproductive': 1, 'themblack': 1, 'headoutback': 1, 'walkabout': 1, 'wildernessroy': 1, 'photoshop': 1, 'evilfear': 1, 'notshe': 1, 'docile': 1, 'crowdthe': 1, 'secretsjim': 1, 'heathcliff': 1, 'cartoonsmakeup': 1, 'donts2': 1, 'endorsements': 1, 'moore5yearold': 1, 'viewingthis': 1, 'donutshaped': 1, 'homer': 2, 'truewhos': 1, 'slants': 1, 'whopan': 1, 'officewant': 1, 'expectationswhy': 1, 'ceo87': 1, 'reportmiracle': 1, 'surviveswhole': 1, 'squash': 1, 'seedsnorth': 1, 'unlike': 1, 'flystudy': 1, 'primarily': 1, 'sleep6': 1, 'accomplishlovestruck': 1, 'lifecongress': 1, 'uncool': 1, 'legislationif': 1, 'coneshuge': 1, 'tempting': 1, 'rationalmoving': 1, 'changeapathy': 1, 'outpacing': 1, 'mindcampus': 1, 'firstwalmart': 1, 'whichever': 1, 'shitkushner': 1, 'reportkey': 1, 'sanderssecretary': 1, 'claude': 1, 'brinegarmachiavellian': 1, 'groundskeeper': 1, 'staffteens': 1, 'youmtv': 1, 'youthisis': 1, 'teaming': 1, 'nsfwadventures': 1, 'museuma': 1, 'miraclecorn': 1, 'stomachlingerie': 1, 'boutique': 1, 'thattronc': 1, 'levinsohn': 1, 'behaviorbar': 1, 'upaol': 1, 'timewarner': 1, 'largestever': 1, 'expenditure': 1, 'moneyworld': 1, 'presidentparents': 1, 'ceremonycollege': 1, 'lawsuitstressed': 1, 'breathingthe': 1, 'sleepers': 1, 'knowjeff': 1, 'departmentsniper': 1, 'moustache': 1, 'crosshairs5': 1, 'engagementsupreme': 1, 'mythcongress': 1, 'warmaking': 1, 'authoritysupreme': 1, 'officeted': 1, 'trumpaid': 1, 'bickeringbetsy': 1, 'pitbullthe': 1, 'populist': 1, 'davosmissouri': 1, 'characterinsect': 1, 'limitless': 1, 'pupilinflatable': 1, 'offwalnuts': 1, 'saladap': 1, 'returnguy': 1, 'sipping': 1, 'snowboard': 1, 'somethingpoll': 1, 'lifegop': 1, 'heretrip': 1, 'despised': 1, 'coworkerrussian': 1, 'boris': 1, 'yeltsinsecurity': 1, 'houserepublicans': 1, 'africans': 1, 'criticizedman': 1, '180000': 1, 'deductible2016': 1, 'podcastsoutside': 1, 'problemstrump': 1, 'itselfchief': 1, 'eulogizes': 1, 'seasonswant': 1, 'bello': 1, 'clowncarli': 1, 'worldbette': 1, 'rupturesgap': 1, 'missingguy': 1, 'finewoman': 1, 'deathlook': 1, 'syriasanta': 1, 'shootinga': 1, 'kurdistan11': 1, 'shivers': 1, 'spinehow': 1, 'nylon': 1, 'disappeargingrich': 1, 'peoplewhovote': 1, 'votejohn': 1, 'bellopponents': 1, 'paid5': 1, 'youtrump': 1, 'browserin': 1, 'save5yearold': 1, 'againboris': 1, 'ministerpalestinian': 1, 'trooperheaven': 1, 'spikes': 1, 'cherubs': 1, 'gateaudience': 1, 'pans': 1, 'spattering': 1, 'wallexgirlfriends': 1, 'electricbill': 1, 'uncashed': 1, 'walletwhy': 1, 'offno': 1, 'happensfast': 1, 'momto': 1, 'unseenarea': 1, 'donationterry': 1, 'womenflynns': 1, 'disorientedthis': 1, 'anorexia': 1, 'anyway3': 1, 'jealousyjosh': 1, 'ritter': 1, 'musicsuper': 1, 'resultsthe': 1, 'wowpeter': 1, 'suggestsboy': 1, 'regardless': 1, 'woodssparring': 1, 'referendumstheres': 1, 'bellhop': 1, 'dutywhite': 1, 'banundercover': 1, 'threealarm': 1, 'blazejimmy': 1, 'melanoma': 1, 'brainsamantha': 1, 'wordslinmanuel': 1, 'mirandas': 1, 'campdonald': 1, 'defeating': 1, 'election510': 1, 'grapesoda': 1, 'induced': 1, 'frenzynation': 1, 'celebration': 2, 'extremismsilence': 1, 'weakens': 1, 'brutalityoutofcontrol': 1, 'careening': 1, 'immigrationthe': 1, 'gymnastchinese': 1, 'tanklast': 1, 'raceprinces': 1, 'flame': 1, 'sheila': 1, 'forevernorth': 1, 'lawmueller': 1, 'foundationmans': 1, 'dealpregnant': 1, 'marriedfusion': 1, 'worldheartbreaking': 1, 'peoplerebel': 1, 'typethis': 1, 'cauliflower': 1, 'crusted': 1, 'spotlighta': 1, 'litany': 1, 'thanksgivingstudy': 1, 'perceive': 1, 'them6': 1, 'grandpagame': 1, 'fandoctors': 1, 'appendix': 1, 'soulcongress': 1, 'avert': 1, 'diedfact': 1, 'legendfec': 1, 'candidatesdonald': 1, 'existwaiter': 1, 'appsreport': 1, 'shoeshuffpollster': 1, 'partiesshared': 1, 'that7': 1, 'nowstarship': 1, 'screenjesus': 1, 'wept': 1, 'cairo55': 1, 'goodwe': 1, 'graduates5': 1, 'outweighed': 1, 'stateschool': 1, 'funnywhat': 1, 'virusus': 1, '2043microsoft': 1, 'teleportationarea': 1, 'redbox': 1, 'planolder': 1, 'whya': 1, 'castroeconomic': 1, 'stimulus': 1, 'warmthrihannas': 1, 'happensean': 1, 'tunnelwhy': 1, 'seatheres': 1, 'hollywoodworthy': 1, 'hearmans': 1, 'muscle': 1, 'turfdiabetic': 1, 'goutridden': 1, 'koreabarbara': 1, 'couplemovie': 1, 'there4': 1, 'amtrak': 1, 'michiganphysics': 1, 'classjewish': 1, 'resurrects': 1, 'brooklyns': 1, 'tensionwoman': 1, 'nowjohnny': 1, 'galecki': 1, 'sorrowtrevor': 1, 'trumpcares': 1, 'unbelievablewoman': 1, 'handwritingdocumentary': 1, 'copycat': 1, 'documentarieseconomists': 1, 'straits': 1, 'tourkatrina': 1, 'responserc': 1, 'girlsinternet': 1, 'mexicostudy': 1, 'persons': 1, 'questioningpfizer': 1, 'fdas': 1, 'approvalman': 1, 'girlfriendsanta': 1, 'specialwants': 1, 'childrenjohn': 1, 'bandog': 1, 'shitchristies': 1, 'biteslatest': 1, 'theatersgoing': 1, 'limb': 1, 'senateairplanes': 1, 'good16yearold': 1, 'yearleah': 1, 'rediscovers': 1, 'scientology': 1, 'lifedan': 1, 'rightsremembering': 1, 'martamy': 1, 'poehler': 1, 'ike': 1, 'barinholtz': 1, 'discriminatingrotating': 1, 'vortex': 2, 'investigationrobbie': 1, 'knievel': 1, 'awarenessi': 1, 'mediums': 1, 'frauds': 1, 'knownthree': 1, 'videodeath': 1, 'gingrichfrankie': 1, 'visitmumford': 1, 'vest': 1, 'grammytechnophile': 1, 'listagedefying': 1, '50report': 1, 'comfynavratri': 1, 'hindu': 1, 'goddessvacation': 1, '40hour': 1, 'weekprevious': 1, 'bothered': 1, 'mildewnew': 1, 'homeshow': 1, 'nomineereport': 1, 'worseentourage': 1, 'caresbored': 1, 'browses': 1, 'confiscated': 1, 'itemslady': 1, 'jlaw': 1, 'legislationtv': 1, 'overratedmaking': 1, 'dent': 1, 'downcheerleader': 1, 'dwalmart': 1, 'diapersbillionaire': 1, 'trumpdevos': 1, 'agendawhat': 1, 'fever': 1, 'autismstop': 1, 'usehey': 1, 'rabbi': 1, 'romcombob': 1, 'votehousehold': 1, 'seasonfall': 1, 'trendsgop': 1, 'ins': 1, 'representativefrustrated': 1, 'trafficsenate': 1, 'warnstuff': 1, 'floorgus': 1, 'sant': 1, 'shotforshot': 1, 'remakenew': 1, 'popularity': 1, 'onlinecongress': 1, 'feingolds': 1, 'todayusher': 1, 'onfinancial': 1, '$5heroin': 1, 'surging': 1, 'bottlestrumpfriendly': 1, 'grabbed': 1, 'aiderepublicans': 1, 'househumiliated': 1, 'longdead': 1, 'castrodonald': 1, 'rapesource': 1, 'jealousy': 1, 'successfulfailed': 1, 'hyperbole': 1, 'deadon': 1, 'statisticobama': 1, 'spock': 1, 'earshow': 1, 'othersmachetewielding': 1, 'videonewlywed': 1, 'debtmeatless': 1, 'asbell': 1, 'juicedbreaking': 1, 'breakingfighting': 1, 'campaignteach': 1, 'acquired': 1, 'investors': 1, 'teacherpowerplant': 1, 'lunchboxscary': 1, 'planethow': 1, 'illustrationsmoon': 1, 'hatchesfirst': 1, 'shalhoub': 1, 'braindeadnew': 1, 'converts': 1, 'meatparents': 1, 'insaneman': 1, 'millennium': 1, 'tunneldepressed': 1, 'buttercovered': 1, 'spotcongaline': 1, 'participant': 1, 'beckons': 1, 'ominouslypublic': 1, 'diplomacy': 1, 'pacificcharlottesville': 1, 'tacit': 1, 'figuredoes': 1, 'romans': 1, 'koreababy': 1, 'beagle': 1, 'yearsiran': 1, 'strategyromney': 1, 'tailors': 1, 'daynew': 1, 'syndicate': 1, 'factored': 1, 'expensesken': 1, 'romanos': 1, 'narrate': 1, 'documentaryjohn': 1, 'justicethis': 1, 'nicknamestephen': 1, 'awardstea': 1, 'partyfisherprice': 1, 'utero': 1, 'gymjohn': 1, 'soonthis': 1, 'fumingwhy': 1, 'mao': 1, 'chinareport': 1, 'childbirtharea': 1, 'sad5': 1, 'beforetop': 1, 'loretta': 1, 'lynchthis': 1, 'scarynathan': 1, 'defeats': 1, 'racethe': 1, 'turpangreen': 1, 'streetshardline': 1, 'hawks': 1, 'trumpheres': 1, 'onehanded': 1, 'catchthe': 1, 'sometimesjulianne': 1, 'hough': 1, 'endometriosisjohn': 1, 'bel': 1, 'vitter': 1, 'subtlehow': 1, 'virginias': 1, 'strikemembers': 1, 'mensuburban': 1, 'invests': 1, 'grillwarm': 1, 'canvasser': 1, 'againnation': 1, 'sterilized': 1, 'fridayedge': 1, 'metaphoranderson': 1, 'dumpsterinside': 1, 'jewryon': 1, 'advicestudy': 1, 'lifehomeland': 1, 'banthis': 1, 'tppchinese': 1, 'observe': 1, 'tiananmen': 1, 'massacresmalltown': 1, 'raisingmeet': 1, 'megadonor': 1, 'movementat': 1, 'saysmichael': 1, 'employeescandal': 1, 'congeniality': 1, '2003news': 1, 'articleskerry': 1, 'committeeassisted': 1, 'sherbetgm': 1, 'volkswagens': 1, 'dieselgate': 1, 'leftviewer': 1, 'reefsron': 1, 'racegiddy': 1, 'thom': 1, 'yorke': 1, 'soonerjohn': 1, 'warsobesity': 1, 'helpalpha': 1, 'trickortreater': 1, 'established': 1, 'houserod': 1, 'auntmichelle': 1, 'fathera': 1, 'video15': 1, 'adventuresthursday': 1, 'conflictfor': 1, 'thingsgop': 1, 'coordination': 1, 'convictionthe': 1, 'fightprince': 1, 'hinted': 1, 'namemilky': 1, 'shimmers': 1, 'videoteenage': 1, 'tablehere': 1, 'billmindfulness': 1, 'chaospat': 1, 'robertson': 1, 'vaccination': 1, 'mandates250pound': 1, 'lifewhy': 1, 'pinka': 1, 'granddaughtersemmanuel': 1, 'amused': 1, 'islamophobiadads': 1, 'toddlersone': 1, 'shamanismsuccessories': 1, 'shopliftedoysters': 1, 'datewhy': 1, 'israelwant': 1, 'saydo': 1, 'ratiochina': 1, 'ebolahit': 1, 'liberiajim': 1, 'parsons': 1, 'divine': 1, 'insteadsuspect': 1, 'rockford': 1, 'referencemothers': 1, 'thatbyblos': 1, 'brims': 1, 'ofbarack': 1, 'engagementspirits': 1, 'nightwhy': 1, 'raiding': 1, 'savingsgucci': 1, 'mane': 1, 'sentencedwhats': 1, 'studentsfrom': 1, 'familieshunger': 1, 'jena': 1, 'malone': 1, 'instagramalcohol': 1, 'machinery': 1, 'bearableparents': 1, 'fromnew': 1, 'fishmister': 1, 'smearsean': 1, 'accuserthe': 1, 'expectationlas': 1, 'gofundme': 1, 'victimsglobalwarming': 1, 'delightful': 1, 'midfebruary': 1, 'afternoonpolice': 1, 'thingmeth': 1, 'fouryear': 1, 'increaseexhausted': 1, 'clintonis': 1, 'resisting': 1, 'enoughxavier': 1, 'dolans': 1, 'exuberant': 1, 'cannesjustin': 1, 'timberlakes': 1, 'boneshaking': 1, 'remixrep': 1, 'charlestonreport': 1, 'countrywatch': 1, 'bates': 1, 'previewthe': 1, 'snows': 1, 'resurrectionmy': 1, 'stewartescalating': 1, 'programsbroncos': 1, 'quinncold': 1, 'justicejimmy': 1, 'keanu': 1, 'reeves': 1, 'kidsentitled': 1, 'deadbeat': 1, 'dependencysecretary': 1, 'upben': 1, 'distractionthe': 1, 'squeezed': 1, 'worldkids': 1, 'buscreepy': 1, 'anythinggeorge': 1, 'casamigos': 1, 'ibizawho': 1, 'bert': 1, 'bernsnaacp': 1, 'backyardsman': 1, 'complimentis': 1, 'bullied13': 1, 'themthis': 1, 'debtblack': 1, 'tipsseriously': 1, 'socks': 1, 'gosexualized': 1, 'flapper': 1, 'someoneguidance': 1, 'prefaces': 1, 'flawsclimate': 1, 'againelena': 1, 'ferrante': 1, 'magazinewhy': 1, 'derby': 1, 'womenbill': 1, 'liveclinton': 1, 'intrump': 1, 'censusamericans': 1, 'blackface': 1, 'costumesken': 1, 'subways': 1, 'againhigh': 1, 'alivepolitical': 1, 'déja': 1, 'vu': 1, 'chihuahua': 1, 'dane': 1, 'endearing': 1, 'timebjrk': 1, 'moma': 1, 'videorudy': 1, 'towers': 1, '911inexperienced': 1, 'streaker': 1, 'real7': 1, 'slumpwhat': 1, 'womenastronomers': 1, 'nitrogenobamacare': 1, 'dilemmaunpopular': 1, 'thingright': 1, 'saycheer': 1, 'democratstourist': 1, 'cdscopycat': 1, 'adapting': 1, 'adaptationsjohn': 1, 'alassadbreaking': 1, 'jobstoried': 1, 'ip': 1, 'addressthomas': 1, 'reenacts': 1, 'wifedear': 1, 'areearth': 1, 'planetthe': 1, 'dea': 1, 'silentbingedrinking': 1, 'promiscuous': 1, 'medicinetrump': 1, 'pornsave': 1, 'helms': 1, 'overreachsonia': 1, 'pursuing': 1, 'courtworse': 1, 'crisissmall': 1, 'onceostracized': 1, 'artistold': 1, 'gypsy': 1, 'consequencearea': 1, 'nominated7yearold': 1, 'putty': 1, 'posterity5': 1, 'opponentshollywood': 1, 'mayer': 2, 'beckoning': 1, 'laphistorians': 1, 'druid': 1, 'makeout': 1, 'spotjihadist': 1, 'boysclinton': 1, 'feces': 1, 'detractorsnewsroom': 1, '2016leonardo': 1, 'morphs': 1, 'hairy': 1, 'oscarwhat': 1, 'snowy': 1, 'coldchrissy': 1, 'teigens': 1, 'roomwill': 1, 'fortes': 1, 'shavegeorge': 1, 'icu': 1, 'improvedfrances': 1, 'twospeed': 1, 'tighter': 1, 'borderssingleengine': 1, 'cessna': 1, 'bushfyi': 1, 'hardcore': 1, 'dayitaly': 1, 'finalsobama': 1, 'it18': 1, 'playedout': 1, 'dormroom': 1, 'posters2': 1, 'centerpawnshop': 1, 'backsupreme': 1, '193000000115000000': 1, 'decisionhistorical': 1, 'buttonsdid': 1, 'debatepfizer': 1, 'onkanye': 1, 'vmasal': 1, 'womenrecordbreaking': 1, 'rainstorms': 1, 'pummel': 1, 'carolinas10': 1, 'race22yearold': 1, 'websiteedward': 1, 'ciajudge': 1, 'geekier': 1, 'sonsif': 1, 'thisjustin': 1, 'smoresharpers': 1, 'index': 2, 'harpers': 1, '98a': 1, 'partybaby': 1, 'tasting': 1, 'timedog': 1, 'starvingthis': 1, 'anywaynobel': 1, 'colombiathe': 1, 'prayi': 1, 'bureaucratic': 1, 'messtrump': 1, 'wallarea': 1, 'girlfriendall': 1, 'timewasting': 1, 'popemobile': 1, '84why': 1, 'potentialwhat': 1, 'motivates': 1, 'whistleblowerbrian': 1, 'willams': 1, 'defenseon': 1, 'friendthese': 1, 'freedomups': 1, 'messagemexicans': 1, 'nationmit': 1, 'other9': 1, 'grandparenting': 1, 'gamegoogle': 1, 'itdamning': 1, 'jolson': 1, 'blackfaceyou': 1, 'gamejustice': 1, 'deliberating': 1, 'ittrain': 1, 'dozensdonors': 1, 'ratenatures': 1, 'dayson': 1, 'emmett': 1, 'spectacle': 1, 'misappropriationdonating': 1, 'vengeance': 1, 'infidelhuman': 1, 'reportreport': 1, '893000': 1, 'over1930s': 1, 'outsmarted': 1, 'bedthursdays': 1, 'prepping': 1, 'testcool': 1, 'retirementfrat': 1, 'hotness': 1, 'laid4': 1, 'earsjames': 1, 'artrebel': 1, 'songsnostalgic': 1, 'polio': 1, 'vaccineeric': 1, 'peopleamazon': 1, 'websiteman': 1, 'hammerinduced': 1, 'almightyelizabeth': 1, 'yearsson': 1, 'somethingmom': 1, 'onlinee': 1, 'saladeighty': 1, '2s': 1, 'deadreport': 1, 'asleepits': 1, 'bombs': 1, 'fallingtrumpcare': 1, 'nexthere': 1, 'russiansteacher': 1, 'seniority': 1, 'professionthe': 1, 'contention21': 1, '$20nations': 1, 'hansel': 1, 'gretel': 1, 'perhaps': 1, 'timegreenpeace': 1, 'forestdog': 1, 'breeders': 1, 'mastiffeaglenewly': 1, 'fossils': 1, 'bonyheres': 1, 'everheres': 1, 'kci': 1, 'nowpaul': 1, 'grudgingly': 1, 'milesbiologists': 1, 'foxes': 1, 'sneakiest': 1, 'kingdommeownisota': 1, 'ears': 1, 'festivaltonights': 1, 'blownoff': 1, 'screenwhy': 1, 'stoudemire': 1, 'weekshower': 1, 'coated': 1, 'dazzling': 1, 'filmssubscribing': 1, 'successemployee': 1, 'largeralumni': 1, 'improvementsheadline': 1, 'okaycelebrating': 1, 'christmasworlds': 1, 'agehousing': 1, 'stroll': 1, 'neighborhoodthe': 1, 'hb2': 1, 'repealedtoaster': 1, 'stride': 1, 'recentlycrossshaped': 1, 'unconstitutionalnetwork': 1, 'brownface': 1, 'muslimbig': 1, 'nationspeople': 1, 'lipstickshouse': 1, 'elks': 1, 'memberswhite': 1, 'managementhulus': 1, 'fiennes': 1, 'bananashow': 1, 'bodiesjack': 1, 'antonoff': 1, 'depressionastronaut': 1, 'peake': 1, 'marathon': 1, 'notifications': 1, 'facebookthalia': 1, 'cassuto': 1, 'brainwashing': 1, 'denier': 1, 'inhofedrivetime': 1, 'commute': 1, 'jampacked': 1, 'entertainmentstudy': 1, 'routes': 1, 'countrylone': 1, 'partyinghome': 1, 'bombkourtney': 1, 'crewpolice': 1, 'investigationthe': 1, 'repealyeah': 1, 'soto': 1, 'politicsbakery': 1, 'cakes': 1, 'altogether': 1, 'progay': 1, 'rulingmillennium': 1, '2001': 1, 'note5': 1, 'photossiri': 1, 'truckrolling': 1, 'touramerican': 1, 'torturing': 1, 'outsourcedjordan': 1, 'klepper': 1, "gun'": 1, 'mythdoll': 1, 'overstays': 1, 'dollhouse': 1, 'welcomewall': 1, '"insiders"': 1, 'dol': 1, 'rulethe': 1, 'brandname': 1, 'medicationsprocessing': 1, "ferguson's": 1, 'behow': 1, "'secret'": 1, 'discounted': 1, 'airfarestunning': 1, 'proposedwalletless': 1, 'handcuffed': 1, 'bedpostlesser': 1, "pen's": 1, 'viabilitysamsung': 1, 'slashes': 1, 'smartphoneatlantic': 1, 'beachfront': 1, 'mansion': 1, 'soonheinz': 1, 'quickrecovery': 1, 'ketchupwatch': 1, 'sillyretired': 1, '103year-old': 1, 'in-lawstyrofoam': 1, 'clamshell': 1, 'exquisite': 1, "sandwich'tis": 1, "'jingle": 1, "butts'": 1, 'videonypd': 1, 'witnesses': 1, "didn't": 2, 'deadman': 1, 'unfamiliar': 1, 'knobs': 1, 'stallionregarding': 1, 'sontagever': 1, 'placecruz': 1, "'serial": 1, "philanderer'": 1, "'pathological": 1, "liar'": 1, 'attackspring': 1, 'hacksbags': 1, 'anti-flood': 1, 'technologyphiladelphiatheatreco': 1, 'aligned5': 1, 'pointz': 1, 'luxury': 2, 'condos': 1, 'destroyedmy': 1, 'infamydonald': 1, 'kidsartist': 1, 'merges': 1, 'genders': 1, 'nsfwwant': 1, 'behaved': 1, "they're": 1, 'lovedbroke': 1, 'playstation': 1, 'christmasa': 1, 'housecorporate': 1, 'teamno': 1, 'limo': 1, 'wantedphysician': 1, 'ratingphilip': 1, 'cylindricalyou': 1, 'venison': 1, 'unclereport': 1, 'anonymity': 1, 'tragedywatch': 1, "'let": 1, "go'": 1, 'italycamel': 1, 'dollarpresident': 1, 'monthformer': 1, 'prisonjeff': 1, 'complaintscorporation': 1, 'soonai': 1, '1celine': 1, 'awardstouching': 1, "it's": 2, 'grandkidswhen': 1, 'dietingeveryone': 1, 'womancricket': 1, 'locatednation': 1, 'yoke': 1, 'womennew': 1, 'us.': 1, 'fronts': 1, 'moneymaking': 1, 'operationsjoe': 1, "paterno's": 1, 'abuseelection': 1, 'orgy': 1, 'ini': 1, 'teacher.': 1, "you've": 1, 'gotan': 1, 'heir': 1, 'isabel': 1, 'allende': 1, 'bussievidence': 1, 'professorsunhappy': 1, 'childrendonald': 1, "he's": 1, "'troubled'": 1, 'shootingquiz': 1, 'abroadmariah': 1, 'interviewnew': 1, 'hawkgreyhound': 1, '$15': 2, 'aislefema': 1, "emergencyit's": 1, 'ownershipcriminal': 1, 'prosecuted': 1, 'fullest': 1, 'budgetman': 1, "week's": 1, "doesn't": 4, 'badhealth': 1, "company's": 1, "customers'": 1, 'carewomen': 1, 'lootroyal': 1, 'nownew': 1, 'panoramic': 1, 'streepstudy': 1, 'unnecessary': 1, 'purchasesinside': 1, "angeles'": 1, "farmers'": 1, 'marketnasa': 1, 'earthtroop': 1, "'tried": 1, "friend'": 1, 'badgebeijing': 1, 'brazil,': 1, '7-1': 1, 'soccer,': 1, 'societygears': 1, '2006': 1, 'locust': 1, 'horde': 1, 'justifiedyoung': 1, "billionaire's": 1, "nation's": 1, 'egoidris': 1, 'elba': 1, 'bondbisexual': 1, "'it'": 1, 'wordjuan': 1, 'deathcongress': 1, 'prosecutorbudweiser': 1, 'lager': 1, 'tavernthis': 1, 'co-workers': 1, 'commonpolice': 1, 'demonstrating': 1, 'gunreport': 1, "coworkers'": 1, 'disdain': 1, '70how': 1, "candidate's": 1, "haddad's": 1, 'unheard-of': 1, 'diesguest': 1, 'low-traffic': 1, 'areain': 1, 'india,': 1, 'sterilization': 1, 'vulnerablewhat': 1, 'blindnesschemical': 1, 'u.s.': 5, 'inchicago': 1, "jenner's": 1, 'announcementbreaking': 1, "'breaking'": 1, 'meaningreport': 1, 'antismoking': 1, 'cigarettesdrug': 1, 'greatcongress': 1, 'intervene': 1, 'weddinggeorge': 1, 'rereleasekey': 1, "trump's": 9, 'arabiatony': 1, 'tiger,': 1, 'toucan': 1, "kellogg's": 1, 'mascots': 1, "'speak": 1, "out'": 1, 'bullyingchinese': 1, 'cyber-attacks': 1, 'posturerotating': 1, 'investigationwife': 1, "she'll": 1, 'backcontroversial': 1, 'e-cigarettes': 1, 'formaldehyde': 1, 'exposuresamantha': 1, "'trump-positive'": 1, 'piecethe': 1, "'tomb": 1, "raider'": 1, 'joliethe': 1, 'ebayheadphones-wearing': 1, 'pedestrian': 1, 'statusamericans': 1, "attackssomeone's": 1, 'antacid': 1, 'gumtiger': 1, 'zooasghar': 1, 'farhadi': 1, "don't": 2, 'streetdrone': 1, "housewarpaint's": 1, 'wayman': 1, "band's": 1, "'vivid'": 1, 'inevitably': 1, 'careercompletely': 1, 'uninhibited': 1, 'choosing': 1, 'workis': 1, 'prisoners?10': 1, 'workwhy': 1, '--': 2, "republicans'mother": 1, 'essentially': 1, "raped,'": 1, 'mourdock': 1, 'holelondoners': 1, 'polite': 1, 'bombingssemester': 1, 'siegeencouraging': 1, 'indicates': 1, 'mathlong': 1, "silver's": 2, 'deep-fried': 1, 'poseidon': 1, 'mealfrom': 1, 'bicycles': 1, 'spaceships': 1, '(video)teachers': 1, "'trump": 1, "effect'": 1, 'warping': 1, "kids'": 1, 'mindsobjection,': 1, 'honornation': 1, 'envy': 1, '15-year-old': 1, 'jots': 1, 'margin': 1, "'to": 1, "mockingbird'report": 1, 'subsean': 1, 'credentials': 1, 'outletsjoe': 1, "arpaio's": 1, 'motoristdemocrats': 1, 'banthere': 1, 'cpac,': 1, 'huh?depressed': 1, 'wasminimum-wage': 1, 'increases:': 1, 'redistributionreport:': 1, "child's": 1, 'failuremichael': 1, 'talks,': 1, 'saywoman': 1, "she's": 1, 'concertguerrilla': 1, "jumper's": 1, 'plummet': 1, 'dropwoman': 1, 'talk-show': 1, 'hostbrendan': 1, 'pre-movie': 1, 'trivia': 1, 'questionburger': 1, 'deep-steamed': 1, 'fries5': 1, "i've": 1, 'boyromney': 1, 'speech:': 1, "'man,": 1, "field'watch": 1, 'gladness': 1, 'againlocal': 1, 'masturbatingjohn': 1, "teigen's": 1, 'themi': 1, 'misplaced': 1, 'assignment': 1, 'peru,': 1, 'dayancient': 1, 'gravestone': 1, 'battering': 1, 'ram': 1, 'lightning': 1, 'heistarea': 1, 'movingu.s.': 1, 'infringement': 1, "shakiratrump's": 1, 'problemhow': 1, 'prisons$80': 1, 'spinach': 1, 'filmkaty': 1, "diamondhere's": 1, 'debaterahm': 1, 'aboutflat': 1, 'sodagraduates,': 1, 'timedreamcatcher': 1, 'rearview': 1, 'driverbusy': 1, 'best-dressed': 1, 'slideshow': 1, 'tab': 1, 'savored': 1, 'leisurepope': 1, 'countries9': 1, "you're": 2, 'crazythe': 1, 'developmentarchaeologists': 1, "friend's": 1, 'artinternal': 1, 'weakness': 1, 'coworkersinsufferable': 1, '8-year-old': 1, "won't": 1, "'romney'7": 1, 'starsobama': 1, "'epidemic": 1, "violence'in": 1, "kenya's": 1, 'forests,': 1, 'violentthere': 1, "'alt-right.'": 1, 'supremacy.expert': 1, 'conversation:': 1, "'the": 1, "claim'how": 1, 'owncameron': 1, 'monaghan': 1, 'adaptationmerriam-webster': 1, "'doctor": 1, "who'": 1, 'fansindependent-film': 1, 'troopsyou': 1, "mcdonald's": 1, 'breakfastworld': 1, 'hunger:': 1, 'frito-lay': 1, 'zestitos': 1, 'problem?danny': 1, 'burtonhug': 1, 'factorycuriosity': 1, 'synagoguethis': 1, "terrifying'house": 1, "cards'": 1, 'onozone': 1, 'repletion': 1, 'finishedjeff': 1, "dudechrist's": 1, "canvas'without": 1, "anything,'": 1, 'blankly': 1, "'sex": 1, "city'": 1, "'a": 1, "show'khloe": 1, 'disickuniverse': 1, 'starlight': 1, 'vigila': 1, 'appealmike': 1, 'anti-abortion': 1, 'crowd:': 1, 'mold': 1, 'scaliabush': 1, 'suitthe': 1, 'dominican': 1, 'republicstadium': 1, 'stage,': 1, "'hey": 1, "jude'what": 1, 'cravingsradical': 1, "charges'affluenza'": 1, 'tonya': 1, 'curfew': 1, 'eased': 1, 'jobblm': 1, 'inequalitythe': 1, 'usingphilippine': 1, 'killingsmiddle': 1, 'cushion': 1, 'one-time': 1, 'thingwhite': 1, 'man,': 1, "'i": 1, "happen'seth": 1, 'holiday-themed': 1, 'slanga': 1, "snowden's": 1, '(video)brad': 1, '1,000': 1, 'peaceshake': 1, 'off:': 1, 'reviewaids': 1, "'onion'": 1, 'reporterexplainthe90sin4words': 1, 'resurfaces': 1, "'90s": 1, "referenceshere's": 1, "'fear": 1, "dead'report:": 1, '.00003': 1, "matter'scooter'": 1, 'libby': 1, "he'd": 1, 'coveragewe': 1, "they've": 2, 'polleddeep': 1, 'slowly,': 1, 'topher': 1, "grace's": 1, 'careeraudio': 1, 'degascrullers': 1, 'explaineddirector': 1, 'himselfelizabeth': 1, 'wades': 1, "party's": 1, "candidates'": 1, 'viewstrump': 1, "testify'out": 1, "sight':": 1, '360-degree': 1, 'ignoresfriends': 1, 'eventsean': 1, 'deep-state': 1, 'rentbrie': 1, "larson's": 1, "'trainwreck'": 1, 'schumerindianapolis': 1, 'columnbiden': 1, "workers'": 1, 'comp': 1, 'claimreal': 1, "isn't": 1, 'criteria': 1, 'listwhy': 1, 'peoplefussy': 1, 'eater': 1, '38syrian': 1, 'crashes;': 1, 'rebelstrans': 1, 'texans': 1, "measure'snl'": 1, 'tweetsjohnson': 1, '&': 1, 'leave-in': 1, 'q-tipshope': 1, 'directormike': 1, 'marriageshaky': 1, 'ukrainian': 1, 'holdsthe': 1, "today's": 1, 'office:': 1, 'boomersarea': 1, 'macbook': 1, 'qualify': 1, 'geniustearful': 1, 'robotrixhow': 1, 'jailhandwriting': 1, 'cursive3,500-year-old': 1, 'doorstopstudy:': 1, '73': 1, 'closets': 1, "wife's": 1, 'crouched': 1, 'insidemyth': 1, 'openthe': 1, "women's": 1, 'healthwhat': 1, 'couchstudy': 1, 'motherswatch': 1, 'mockument': 1, 'standupmissouri': 1, 'gov.': 1, 'greitens': 1, 'america?queer': 1, '2015bernie': 1, 'overdrive': 1, 'nhbill': 1, 'clinton:': 1, "'fact-free'i'm": 1, 'addicted': 1, 'facebook!7': 1, 'divorce,': 1, 'hollywoodman': 1, "'star": 1, 'wars:': 1, "awakens'": 1, 'goodgrab': 1, 'hand:': 1, 'nonverbal': 1, 'behaviormaya': 1, 'angelou,': 1, 'poet,': 1, 'author,': 1, 'activist,': 1, 'andholy': 1, 'cowtony': 1, 'awardnominated': 1, 'actress,': 1, 'professor,': 1, 'editor,': 1, 'streetcar': 2, 'conductor—really?': 1, 'conductor?': 1, 'wow—calypso': 1, 'singer,': 1, 'performer,': 1, 'journalist,': 1, '86fisherman': 1, 'rescuing': 1, 'netshow': 1, 'mostjournalist': 1, 'articleburmese': 1, 'neckquiznos': 1, '6-foot-long': 1, "manphish's": 1, 'ageskentucky': 1, 'marriagesthe': 1, "that's": 1, 'calmviolence': 1, 'jerusalemstudio': 1, "over'these": 1, "crazy':": 1, 'brewers': 1, 'changea': 1, 'brooklyn,': 1, 'york,': 1, '$55,000': 1, 'salaryhuffpost': 1, 'rise:': 2, '25what': 1, 'meanstractor-pull': 1, 'aboutrepublicans': 1, 'ituniverse': 1, 'tripping': 1, 'mushroomsnew': 1, 'einstein': 1, 'devised': 1, 'relativity': 1, "wasn't": 2, 'invent': 1, 'wordthe': 1, 'broadcasthero': 1, 'hungerjaded': 1, 'seismologist': 1, '7.0': 1, 'richter': 1, 'scalea': 1, 'arizonafabled': 1, "everyone's": 1, 'friesthe': 1, 'insaneu.s.': 1, 'reserves': 1, 'depletedhuffpost': 1, 'last-minute': 1, "mother's": 1, 'timeactually,': 1, 'quadriplegiccoming': 1, 'soon:': 1, "'fraggle": 1, "rock'": 1, 'moviemoms': 1, 'away,': 1, "too!'strange": 1, "looking'": 1, 'familyadvertising': 1, 'inner-child': 1, 'demographicvoter': 1, 'workermichigan': 1, 'cases,': 1, 'findscbs': 1, "debategrandfather's": 1, "who's": 1, 'longgop': 1, "'climate": 1, "shock'": 1, 'trailsenator': 1, 'overprivileged': 1, 'americanssecretary': 1, 'passesdonald': 1, 'colbertben': 1, "damon's": 1, 'filmsu.s.': 1, 'netanyahu': 1, 'promiseswall': 1, '500,000': 1, '2018vanilla': 1, 'ice,': 1, 'co-sign': 1, "lease'oh": 1, 'god,': 1, "night?'": 1, 'groggy': 1, 'waking': 1, 'wifebus-stop': 1, 'citizenhillary': 1, "massachusetts'": 1, 'primaryfinancially': 1, 'riotmy': 1, 'hero:': 1, 'manmom': 1, 'itatheists': 1, 'hindus,': 1, 'vegans,': 1, 'satanists': 1, 'monumentpresident': 1, 'trump,': 1, 'acathis': 1, "allergieswhat's": 1, 'gay?': 1, '(video)trump': 1, 'scapegoats': 1, 'everythingdesperate': 1, "'time'": 1, "'man": 1, "june'james": 1, "'doesn't": 1, "deserve'": 1, 'badgehbo': 1, "'brief": 1, "nudity'actors": 1, 'dated': 1, 'costarsnelson': 1, '94th': 1, 'parolethese': 1, 'changeneighbor': 1, 'brief,': 1, 'apartmentjustice': 1, 'session': 1, 'adjudicatory': 1, 'tendonoscar': 1, 'inedibles': 1, 'successthewrap': 1, 'mediapalestinians': 1, 'invasionsmoker': 1, 'smokersudan': 1, 'darfur': 1, 'attacks,': 1, 'saysracists': 1, 'jurycritics': 1, "kentucky's": 1, "'religious": 1, "freedom'": 1, 'studentsaliens': 1, "shooting'rupaul's": 1, "3'": 1, 'recap:': 1, 'warhol': 1, 'crowns': 1, 'queenreports:': 1, 'complaintsaudubon': 1, 'seen,': 1, 'like,': 1, "birdsdunkin'": 1, 'donutsbaskin': 1, 'robbinspizza': 1, 'huttaco': 1, 'belllong': 1, 'opensfender': 1, "guitarsgawker's": 1, 'loathingdo': 1, "i'm": 1, 'over,': 1, 'aggressive,': 1, 'today,': 1, 'sir?why': 1, 'post-debate': 1, "terrible'riverdale'": 1, 'lili': 1, 'reinhart': 1, 'insensitive': 1, 'tweetastronomers': 1, 'tiny,': 1, 'systemcreative': 1, "hopenation's": 1, 'tracy': 1, 'chapman': 1, "'can't": 1, "wait'": 1, 'lilith': 1, 'fairarea': 1, "woman's": 1, 'impactarea': 1, 'sweaty': 1, 'outlinetranscanada': 1, 'applicationmonths': 1, 'painstaking': 1, 'critiquing': 1, 'womanwhite': 1, 'enviros': 2, 'pact,': 1, 'otherwisewilliam': 1, 'safire': 1, 'whoppers': 1, 'juniortracy': 1, 'k.': 1, "america's": 2, 'laureatedavid': 1, 'petraeus:': 1, 'wrong,': 1, "'fair": 1, "game'": 1, 'criticismfuming': 1, 'putinanother': 1, 'southerners': 1, 'deadfireflies': 1, 'salvage': 1, "man's": 1, 'dayfour': 1, 'wagearianna': 1, 'lifeplastic': 1, 'leathery': 1, 'wifewildfire': 1, "controltuesday's": 1, 'email:': 1, 'worsegod': 1, 'universecarlos': 1, '45-minute': 1, 'solointernational': 1, 'gearfriday': 1, 'sale!homeless': 1, 'dogjimmy': 1, "'stealing": 1, "ideas'": 1, 'daughterfive-year-old': 1, "backyardgrandparents'": 1, 'serenade': 1, 'upjames': 1, '(udpate)scientists': 1, 'mollusks': 1, 'humansmisophonia:': 1, 'furyreport:': 1, "'you've": 1, "mail'": 1, 'snowed': 1, 'todaycorporate': 1, 'sexism,': 1, 'chartthe': 1, 'irelandman': 1, 'bushealth-food-store': 1, 'lungpaul': 1, "ryan's": 1, 'wonk': 1, 'shtick': 1, 'oldthirtysomething': 1, 'lossdeadly': 1, 'capitalbirthday': 1, 'giftsbreathalyzer': 1, 'partyjudge': 1, 'chargefinding': 1, 'universeartists': 1, 'loveclinton': 1, 'follow-up': 1, 'failures': 1, 'firstselene': 1, 'chin:': 1, 'skillshouse': 1, 'record-breaking': 1, 'paceclinton': 1, 'mastered': 1, 'art,': 1, 'saycan': 1, 'usurp': 1, 'responsibilities?pope': 1, 'cutoff': 1}
# Tính độ lệch chuẩn tần suất xuất hiện của các từ
std = np.std(list(dic.values()))
mean = np.mean(list(dic.values()))

val = list(dic.values())
key = list(dic.keys())
count = 0
filtered_words = []
# Lọc ra những từ có ảnh hưởng đến dataset
for i in range(len(val)):
  if key[i] not in stop and val[i] >= abs(mean-std) and val[i] <= abs(mean+std):
    filtered_words.append(key[i])
    count +=1
print(len(filtered_words))

54


In [70]:
# hàm Chuyển 1 câu sang vectors từ (bag words)
def Sentences2Vector(s,w):
  v = []
  for i in range(len(w)):
    v.append(s.count(w[i]))
  return v
# chuyển training data sang dạng vector
vectors = []
for i in range(len(data)):
  vectors.append(Sentences2Vector(data[i,0],filtered_words))
vectors = np.array(vectors)


v_sarcasm = []
v_none_sarcasm = []
#Phân loại data ra sarcasm và none sarcasm
for i in range(len(vectors)):
  label = data[i,-1]
  t = vectors[i,:] 
  if label == 1:
    v_sarcasm.append(t)
  if label == 0:
    v_none_sarcasm.append(t)

# chuyển sang numpy
v_sarcasm = np.array(v_sarcasm) 
v_none_sarcasm = np.array(v_none_sarcasm)


# Sử dụng Model Multinomial Naive Bayes để dự đoán


In [71]:
# Hàm Smooth
def Smooth(v):
  if 0 in v:
    v = v + 1
  v = v/sum(v)
  return v

In [72]:
# Tính tổng các vector của 2 labels
sum_v_sarcasm = np.sum(v_sarcasm,axis=0)
sum_v_none_sarcasm = np.sum(v_none_sarcasm,axis=0)
# Smooth
smooth_v_sarcasm = Smooth(sum_v_sarcasm)
smooth_v_none_sarcasm = Smooth(sum_v_none_sarcasm)


In [73]:
# Chuyển Testing data sang dạng vector
v_test = []
for i in range(len(data2)):
  v_test.append(Sentences2Vector(data2[i,0],filtered_words))
v_test = np.array(v_test)

In [76]:
S = len(v_sarcasm)
N = len(v_none_sarcasm)
total = S + N
result = []
count = 0
error = 0
for i in v_test:
  PS = S/total 
  PN = N/total
  RS = PS * np.prod(pow(smooth_v_sarcasm,i)) # Xác suất tin sarcasm
  RN = PN * np.prod(pow(smooth_v_none_sarcasm,i)) # Xác suất tin none sarcasm
  if RS >= RN:
    result.append(1)
  else:
    result.append(0)
  # Kết quả dự đoán khác kết quả thực tế tăng error lên 1
  if result[count] != data[count,1]:
    error += 1
  count += 1

print(error/len(data2))





0.4508868683857505
